In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Lecture 1\n",
    "\n",
    "In this notebook, we first illustrate the basic machine learning workflow in Python (using the well-known [Iris dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set)), and then show an implementation of the decision tree learning algorithm for classification.\n",
    "\n",
    "To run these examples, you need to have a Python installation that includes [scikit-learn](https://scikit-learn.org/stable/) (for machine learning), [matplotlib](https://matplotlib.org/) (for plotting), and [pandas](https://pandas.pydata.org/) (for loading the dataset). If you use the Anaconda distribution, you'll have these libraries out of the box.\n",
    "\n",
    "You'll also need to install [graphviz](https://pypi.org/project/graphviz/) to make the tree drawing work. If you use Anaconda, you need to install [graphviz](https://anaconda.org/anaconda/graphviz) and [python-graphviz](https://anaconda.org/conda-forge/python-graphviz)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "%config InlineBackend.figure_format = 'svg' \n",
    "plt.style.use('seaborn')\n",
    "%matplotlib inline  "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Loading the iris data\n",
    "\n",
    "You can download the dataset as a CSV file [here](http://www.cse.chalmers.se/~richajo/dit866/lectures/l1/iris.csv). We first use [pandas](https://pandas.pydata.org/) to load the CSV file that stores the Iris data. Please note that you need to change the path to where you stored the csv file.\n",
    "\n",
    "We shuffle the dataset and split it into an input part `X` and an output part `Y`. In this case, we want to predict the type of iris: [*setosa*](https://en.wikipedia.org/wiki/Iris_setosa), [*versicolor*](https://en.wikipedia.org/wiki/Iris_versicolor), or [*virginica*](https://en.wikipedia.org/wiki/Iris_virginica)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "metadata": {},
   "outputs": [],
   "source": [
    "import pandas as pd\n",
    "\n",
    "data = pd.read_csv('iris.csv')\n",
    "data_shuffled = data.sample(frac=1.0, random_state=0)\n",
    "X = data_shuffled.drop('species', axis=1)\n",
    "Y = data_shuffled['species']"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's take a peek at the data. As you can see, there are four numerical column that we use as input, and then the discrete output column representing the species, which we'll use as the output."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>sepal_length</th>\n",
       "      <th>sepal_width</th>\n",
       "      <th>petal_length</th>\n",
       "      <th>petal_width</th>\n",
       "      <th>species</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>114</th>\n",
       "      <td>5.8</td>\n",
       "      <td>2.8</td>\n",
       "      <td>5.1</td>\n",
       "      <td>2.4</td>\n",
       "      <td>virginica</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>62</th>\n",
       "      <td>6.0</td>\n",
       "      <td>2.2</td>\n",
       "      <td>4.0</td>\n",
       "      <td>1.0</td>\n",
       "      <td>versicolor</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>33</th>\n",
       "      <td>5.5</td>\n",
       "      <td>4.2</td>\n",
       "      <td>1.4</td>\n",
       "      <td>0.2</td>\n",
       "      <td>setosa</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>107</th>\n",
       "      <td>7.3</td>\n",
       "      <td>2.9</td>\n",
       "      <td>6.3</td>\n",
       "      <td>1.8</td>\n",
       "      <td>virginica</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>5.0</td>\n",
       "      <td>3.4</td>\n",
       "      <td>1.5</td>\n",
       "      <td>0.2</td>\n",
       "      <td>setosa</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "     sepal_length  sepal_width  petal_length  petal_width     species\n",
       "114           5.8          2.8           5.1          2.4   virginica\n",
       "62            6.0          2.2           4.0          1.0  versicolor\n",
       "33            5.5          4.2           1.4          0.2      setosa\n",
       "107           7.3          2.9           6.3          1.8   virginica\n",
       "7             5.0          3.4           1.5          0.2      setosa"
      ]
     },
     "execution_count": 3,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data_shuffled.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "The example above uses a pandas [`DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) to store the data. If you want to convert into a raw NumPy matrix, it can be done easily by calling `to_numpy`."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[5.8, 2.8, 5.1, 2.4],\n",
       "       [6. , 2.2, 4. , 1. ],\n",
       "       [5.5, 4.2, 1.4, 0.2],\n",
       "       [7.3, 2.9, 6.3, 1.8],\n",
       "       [5. , 3.4, 1.5, 0.2]])"
      ]
     },
     "execution_count": 4,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "X.to_numpy()[:5]"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Basic workflow\n",
    "\n",
    "Let's have some basic machine learning examples.\n",
    "\n",
    "We start by splitting the data into a training and test part. 40% of the data will be used for testing and the rest for training. We use the utility function `train_test_split` from the scikit-learn library.\n",
    "\n",
    "The train/test split is done randomly. The parameter `random_state` is used to set the random seed to a constant value, so that our results are reproducible."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.model_selection import train_test_split\n",
    "\n",
    "Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.4, random_state=0)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We import a scikit-learn class representing [decision tree classifiers](https://en.wikipedia.org/wiki/Decision_tree_learning).\n",
    "\n",
    "We create a decision tree classifier and train it on the training set (`fit`)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.tree import DecisionTreeClassifier\n",
    "\n",
    "clf = DecisionTreeClassifier()\n",
    "clf.fit(Xtrain, Ytrain);"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Let's see what the classifier says about one particular instance. We take a look at the first instance in the test set:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>sepal_length</th>\n",
       "      <th>sepal_width</th>\n",
       "      <th>petal_length</th>\n",
       "      <th>petal_width</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>118</th>\n",
       "      <td>7.7</td>\n",
       "      <td>2.6</td>\n",
       "      <td>6.9</td>\n",
       "      <td>2.3</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "     sepal_length  sepal_width  petal_length  petal_width\n",
       "118           7.7          2.6           6.9          2.3"
      ]
     },
     "execution_count": 7,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "Xtest.head(1)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We comput the classifier's prediction for this instance, and in this case the classifier thinks that this is a [*virginica*](https://en.wikipedia.org/wiki/Iris_virginica)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['virginica'], dtype=object)"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "one_instance = Xtest.head(1)\n",
    "\n",
    "clf.predict(one_instance)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "By looking at the output part of the test set, we can verify that the classifier was correct in this case."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "118    virginica\n",
       "Name: species, dtype: object"
      ]
     },
     "execution_count": 9,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "Ytest.head(1)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We can compute predictions for all instances in the test set. Note that the `predict` method for scikit-learn predictors normally expects a *collection* of instances, not a single instance."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['virginica', 'virginica', 'virginica', 'setosa', 'versicolor',\n",
       "       'setosa', 'virginica', 'virginica', 'virginica', 'versicolor',\n",
       "       'setosa', 'virginica', 'setosa', 'versicolor', 'versicolor',\n",
       "       'virginica', 'setosa', 'virginica', 'versicolor', 'versicolor',\n",
       "       'virginica', 'versicolor', 'virginica', 'versicolor', 'virginica',\n",
       "       'setosa', 'setosa', 'setosa', 'setosa', 'setosa', 'virginica',\n",
       "       'virginica', 'versicolor', 'versicolor', 'versicolor', 'virginica',\n",
       "       'setosa', 'virginica', 'setosa', 'virginica', 'versicolor',\n",
       "       'setosa', 'virginica', 'setosa', 'virginica', 'versicolor',\n",
       "       'virginica', 'setosa', 'virginica', 'versicolor', 'setosa',\n",
       "       'versicolor', 'versicolor', 'virginica', 'setosa', 'virginica',\n",
       "       'setosa', 'virginica', 'versicolor', 'versicolor'], dtype=object)"
      ]
     },
     "execution_count": 10,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "all_predictions = clf.predict(Xtest)\n",
    "\n",
    "all_predictions"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "How good are our predictions? We compute the *accuracy* of our classifier for this test set. The accuracy is defined as the proportion of right answers. (See [here](https://en.wikipedia.org/wiki/Evaluation_of_binary_classifiers) for some other ways to evaluate classifiers.)\n",
    "\n",
    "The iris dataset is quite easy, and the accuracy is quite high for our classifier."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9166666666666666"
      ]
     },
     "execution_count": 11,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.metrics import accuracy_score\n",
    "\n",
    "accuracy_score(Ytest, all_predictions)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Alternatively, we can call the method `score` for our classifier. This method will predict on the test set and then evaluate the predictions using the accuracy."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.9166666666666666"
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "clf.score(Xtest, Ytest)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Normally, we'll carry out several evaluations while we are selecting the best model. This will be done using a separate *validation set*, like a second test set, or using [*cross-validation*](https://en.wikipedia.org/wiki/Cross-validation_(statistics)).\n",
    "\n",
    "In scikit-learn, there are a couple of ways that we can do cross-validation. The simplest way is to call `cross_val_score`, which for each cross-validation fold will call the method `score` mentioned above."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([1.        , 0.94444444, 0.94444444, 1.        , 1.        ])"
      ]
     },
     "execution_count": 13,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.model_selection import cross_val_score\n",
    "\n",
    "cross_val_score(clf, Xtrain, Ytrain, cv=5)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Alternatively, you can call `cross_validate`, where you can specify what kind of metric to use. The output of this function is also a bit more detailed."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{'fit_time': array([0.00350046, 0.0029099 , 0.0029614 , 0.00295329, 0.00317836]),\n",
       " 'score_time': array([0.00181007, 0.0015552 , 0.00157094, 0.00171113, 0.00149918]),\n",
       " 'test_score': array([1.        , 0.94444444, 0.94444444, 1.        , 1.        ])}"
      ]
     },
     "execution_count": 14,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.model_selection import cross_validate\n",
    "\n",
    "cross_validate(clf, Xtrain, Ytrain, cv=5, scoring='accuracy')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Finally, let's evaluate some other classifiers in addition to the decision trees. We first consider linear [support vector classifiers](https://en.wikipedia.org/wiki/Support-vector_machine)."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{'fit_time': array([0.01575923, 0.01592994, 0.01044774, 0.01160312, 0.01101136]),\n",
       " 'score_time': array([0.02344155, 0.00100851, 0.00102115, 0.00110912, 0.00090623]),\n",
       " 'test_score': array([1.        , 0.94444444, 0.94444444, 0.94444444, 1.        ])}"
      ]
     },
     "execution_count": 15,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.svm import LinearSVC\n",
    "\n",
    "clf2 = LinearSVC(max_iter=10000)\n",
    "cross_validate(clf2, Xtrain, Ytrain, cv=5, scoring='accuracy')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "It is useful to compare classifiers to a *baseline*, such as a trivial majority-class classifier."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "{'fit_time': array([0.00194883, 0.00218606, 0.00099993, 0.00099182, 0.00108719]),\n",
       " 'score_time': array([0.00083756, 0.00039721, 0.0003593 , 0.00038004, 0.00040412]),\n",
       " 'test_score': array([0.33333333, 0.33333333, 0.38888889, 0.38888889, 0.33333333])}"
      ]
     },
     "execution_count": 16,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.dummy import DummyClassifier\n",
    "\n",
    "majority_baseline = DummyClassifier(strategy='most_frequent')\n",
    "cross_validate(majority_baseline, Xtrain, Ytrain, cv=5, scoring='accuracy')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Illustrating the iris example\n",
    "\n",
    "We'll provide some illustrations of how the various types of classifiers work by drawing the *decision boundaries* for a two-dimensional dataset.\n",
    "\n",
    "Since the iris dataset has four dimensions, we select two columns (petal length and width) so that we can plot the data in a two-dimensional scatterplot."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/svg+xml": [
       "<?xml version=\"1.0\" encoding=\"utf-8\" standalone=\"no\"?>\n",
       "<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n",
       "  \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n",
       "<!-- Created with matplotlib (https://matplotlib.org/) -->\n",
       "<svg height=\"302.345312pt\" version=\"1.1\" viewBox=\"0 0 314.3 302.345312\" width=\"314.3pt\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n",
       " <defs>\n",
       "  <style type=\"text/css\">\n",
       "*{stroke-linecap:butt;stroke-linejoin:round;}\n",
       "  </style>\n",
       " </defs>\n",
       " <g id=\"figure_1\">\n",
       "  <g id=\"patch_1\">\n",
       "   <path d=\"M 0 302.345312 \n",
       "L 314.3 302.345312 \n",
       "L 314.3 0 \n",
       "L 0 0 \n",
       "z\n",
       "\" style=\"fill:none;\"/>\n",
       "  </g>\n",
       "  <g id=\"axes_1\">\n",
       "   <g id=\"patch_2\">\n",
       "    <path d=\"M 28.1 279 \n",
       "L 307.1 279 \n",
       "L 307.1 7.2 \n",
       "L 28.1 7.2 \n",
       "z\n",
       "\" style=\"fill:#eaeaf2;\"/>\n",
       "   </g>\n",
       "   <g id=\"matplotlib.axis_1\">\n",
       "    <g id=\"xtick_1\">\n",
       "     <g id=\"line2d_1\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 41.318826 279 \n",
       "L 41.318826 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_2\"/>\n",
       "     <g id=\"text_1\">\n",
       "      <!-- 1 -->\n",
       "      <defs>\n",
       "       <path d=\"M 37.25 0 \n",
       "L 28.46875 0 \n",
       "L 28.46875 56 \n",
       "Q 25.296875 52.984375 20.140625 49.953125 \n",
       "Q 14.984375 46.921875 10.890625 45.40625 \n",
       "L 10.890625 53.90625 \n",
       "Q 18.265625 57.375 23.78125 62.296875 \n",
       "Q 29.296875 67.234375 31.59375 71.875 \n",
       "L 37.25 71.875 \n",
       "z\n",
       "\" id=\"ArialMT-49\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(38.538357 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_2\">\n",
       "     <g id=\"line2d_3\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 84.126003 279 \n",
       "L 84.126003 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_4\"/>\n",
       "     <g id=\"text_2\">\n",
       "      <!-- 2 -->\n",
       "      <defs>\n",
       "       <path d=\"M 50.34375 8.453125 \n",
       "L 50.34375 0 \n",
       "L 3.03125 0 \n",
       "Q 2.9375 3.171875 4.046875 6.109375 \n",
       "Q 5.859375 10.9375 9.828125 15.625 \n",
       "Q 13.8125 20.3125 21.34375 26.46875 \n",
       "Q 33.015625 36.03125 37.109375 41.625 \n",
       "Q 41.21875 47.21875 41.21875 52.203125 \n",
       "Q 41.21875 57.421875 37.46875 61 \n",
       "Q 33.734375 64.59375 27.734375 64.59375 \n",
       "Q 21.390625 64.59375 17.578125 60.78125 \n",
       "Q 13.765625 56.984375 13.71875 50.25 \n",
       "L 4.6875 51.171875 \n",
       "Q 5.609375 61.28125 11.65625 66.578125 \n",
       "Q 17.71875 71.875 27.9375 71.875 \n",
       "Q 38.234375 71.875 44.234375 66.15625 \n",
       "Q 50.25 60.453125 50.25 52 \n",
       "Q 50.25 47.703125 48.484375 43.546875 \n",
       "Q 46.734375 39.40625 42.65625 34.8125 \n",
       "Q 38.578125 30.21875 29.109375 22.21875 \n",
       "Q 21.1875 15.578125 18.9375 13.203125 \n",
       "Q 16.703125 10.84375 15.234375 8.453125 \n",
       "z\n",
       "\" id=\"ArialMT-50\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(81.345535 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_3\">\n",
       "     <g id=\"line2d_5\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 126.933181 279 \n",
       "L 126.933181 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_6\"/>\n",
       "     <g id=\"text_3\">\n",
       "      <!-- 3 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.203125 18.890625 \n",
       "L 12.984375 20.0625 \n",
       "Q 14.5 12.59375 18.140625 9.296875 \n",
       "Q 21.78125 6 27 6 \n",
       "Q 33.203125 6 37.46875 10.296875 \n",
       "Q 41.75 14.59375 41.75 20.953125 \n",
       "Q 41.75 27 37.796875 30.921875 \n",
       "Q 33.84375 34.859375 27.734375 34.859375 \n",
       "Q 25.25 34.859375 21.53125 33.890625 \n",
       "L 22.515625 41.609375 \n",
       "Q 23.390625 41.5 23.921875 41.5 \n",
       "Q 29.546875 41.5 34.03125 44.421875 \n",
       "Q 38.53125 47.359375 38.53125 53.46875 \n",
       "Q 38.53125 58.296875 35.25 61.46875 \n",
       "Q 31.984375 64.65625 26.8125 64.65625 \n",
       "Q 21.6875 64.65625 18.265625 61.421875 \n",
       "Q 14.84375 58.203125 13.875 51.765625 \n",
       "L 5.078125 53.328125 \n",
       "Q 6.6875 62.15625 12.390625 67.015625 \n",
       "Q 18.109375 71.875 26.609375 71.875 \n",
       "Q 32.46875 71.875 37.390625 69.359375 \n",
       "Q 42.328125 66.84375 44.9375 62.5 \n",
       "Q 47.5625 58.15625 47.5625 53.265625 \n",
       "Q 47.5625 48.640625 45.0625 44.828125 \n",
       "Q 42.578125 41.015625 37.703125 38.765625 \n",
       "Q 44.046875 37.3125 47.5625 32.6875 \n",
       "Q 51.078125 28.078125 51.078125 21.140625 \n",
       "Q 51.078125 11.765625 44.234375 5.25 \n",
       "Q 37.40625 -1.265625 26.953125 -1.265625 \n",
       "Q 17.53125 -1.265625 11.296875 4.34375 \n",
       "Q 5.078125 9.96875 4.203125 18.890625 \n",
       "z\n",
       "\" id=\"ArialMT-51\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(124.152712 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-51\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_4\">\n",
       "     <g id=\"line2d_7\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 169.740359 279 \n",
       "L 169.740359 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_8\"/>\n",
       "     <g id=\"text_4\">\n",
       "      <!-- 4 -->\n",
       "      <defs>\n",
       "       <path d=\"M 32.328125 0 \n",
       "L 32.328125 17.140625 \n",
       "L 1.265625 17.140625 \n",
       "L 1.265625 25.203125 \n",
       "L 33.9375 71.578125 \n",
       "L 41.109375 71.578125 \n",
       "L 41.109375 25.203125 \n",
       "L 50.78125 25.203125 \n",
       "L 50.78125 17.140625 \n",
       "L 41.109375 17.140625 \n",
       "L 41.109375 0 \n",
       "z\n",
       "M 32.328125 25.203125 \n",
       "L 32.328125 57.46875 \n",
       "L 9.90625 25.203125 \n",
       "z\n",
       "\" id=\"ArialMT-52\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(166.95989 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-52\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_5\">\n",
       "     <g id=\"line2d_9\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 212.547537 279 \n",
       "L 212.547537 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_10\"/>\n",
       "     <g id=\"text_5\">\n",
       "      <!-- 5 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.15625 18.75 \n",
       "L 13.375 19.53125 \n",
       "Q 14.40625 12.796875 18.140625 9.390625 \n",
       "Q 21.875 6 27.15625 6 \n",
       "Q 33.5 6 37.890625 10.78125 \n",
       "Q 42.28125 15.578125 42.28125 23.484375 \n",
       "Q 42.28125 31 38.0625 35.34375 \n",
       "Q 33.84375 39.703125 27 39.703125 \n",
       "Q 22.75 39.703125 19.328125 37.765625 \n",
       "Q 15.921875 35.84375 13.96875 32.765625 \n",
       "L 5.71875 33.84375 \n",
       "L 12.640625 70.609375 \n",
       "L 48.25 70.609375 \n",
       "L 48.25 62.203125 \n",
       "L 19.671875 62.203125 \n",
       "L 15.828125 42.96875 \n",
       "Q 22.265625 47.46875 29.34375 47.46875 \n",
       "Q 38.71875 47.46875 45.15625 40.96875 \n",
       "Q 51.609375 34.46875 51.609375 24.265625 \n",
       "Q 51.609375 14.546875 45.953125 7.46875 \n",
       "Q 39.0625 -1.21875 27.15625 -1.21875 \n",
       "Q 17.390625 -1.21875 11.203125 4.25 \n",
       "Q 5.03125 9.71875 4.15625 18.75 \n",
       "z\n",
       "\" id=\"ArialMT-53\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(209.767068 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_6\">\n",
       "     <g id=\"line2d_11\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 255.354714 279 \n",
       "L 255.354714 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_12\"/>\n",
       "     <g id=\"text_6\">\n",
       "      <!-- 6 -->\n",
       "      <defs>\n",
       "       <path d=\"M 49.75 54.046875 \n",
       "L 41.015625 53.375 \n",
       "Q 39.84375 58.546875 37.703125 60.890625 \n",
       "Q 34.125 64.65625 28.90625 64.65625 \n",
       "Q 24.703125 64.65625 21.53125 62.3125 \n",
       "Q 17.390625 59.28125 14.984375 53.46875 \n",
       "Q 12.59375 47.65625 12.5 36.921875 \n",
       "Q 15.671875 41.75 20.265625 44.09375 \n",
       "Q 24.859375 46.4375 29.890625 46.4375 \n",
       "Q 38.671875 46.4375 44.84375 39.96875 \n",
       "Q 51.03125 33.5 51.03125 23.25 \n",
       "Q 51.03125 16.5 48.125 10.71875 \n",
       "Q 45.21875 4.9375 40.140625 1.859375 \n",
       "Q 35.0625 -1.21875 28.609375 -1.21875 \n",
       "Q 17.625 -1.21875 10.6875 6.859375 \n",
       "Q 3.765625 14.9375 3.765625 33.5 \n",
       "Q 3.765625 54.25 11.421875 63.671875 \n",
       "Q 18.109375 71.875 29.4375 71.875 \n",
       "Q 37.890625 71.875 43.28125 67.140625 \n",
       "Q 48.6875 62.40625 49.75 54.046875 \n",
       "z\n",
       "M 13.875 23.1875 \n",
       "Q 13.875 18.65625 15.796875 14.5 \n",
       "Q 17.71875 10.359375 21.1875 8.171875 \n",
       "Q 24.65625 6 28.46875 6 \n",
       "Q 34.03125 6 38.03125 10.484375 \n",
       "Q 42.046875 14.984375 42.046875 22.703125 \n",
       "Q 42.046875 30.125 38.078125 34.390625 \n",
       "Q 34.125 38.671875 28.125 38.671875 \n",
       "Q 22.171875 38.671875 18.015625 34.390625 \n",
       "Q 13.875 30.125 13.875 23.1875 \n",
       "z\n",
       "\" id=\"ArialMT-54\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(252.574246 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-54\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_7\">\n",
       "     <g id=\"line2d_13\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 298.161892 279 \n",
       "L 298.161892 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_14\"/>\n",
       "     <g id=\"text_7\">\n",
       "      <!-- 7 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.734375 62.203125 \n",
       "L 4.734375 70.65625 \n",
       "L 51.078125 70.65625 \n",
       "L 51.078125 63.8125 \n",
       "Q 44.234375 56.546875 37.515625 44.484375 \n",
       "Q 30.8125 32.421875 27.15625 19.671875 \n",
       "Q 24.515625 10.6875 23.78125 0 \n",
       "L 14.75 0 \n",
       "Q 14.890625 8.453125 18.0625 20.40625 \n",
       "Q 21.234375 32.375 27.171875 43.484375 \n",
       "Q 33.109375 54.59375 39.796875 62.203125 \n",
       "z\n",
       "\" id=\"ArialMT-55\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(295.381423 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-55\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"matplotlib.axis_2\">\n",
       "    <g id=\"ytick_1\">\n",
       "     <g id=\"line2d_15\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 28.1 275.51992 \n",
       "L 307.1 275.51992 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_16\"/>\n",
       "     <g id=\"text_8\">\n",
       "      <!-- 0.0 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.15625 35.296875 \n",
       "Q 4.15625 48 6.765625 55.734375 \n",
       "Q 9.375 63.484375 14.515625 67.671875 \n",
       "Q 19.671875 71.875 27.484375 71.875 \n",
       "Q 33.25 71.875 37.59375 69.546875 \n",
       "Q 41.9375 67.234375 44.765625 62.859375 \n",
       "Q 47.609375 58.5 49.21875 52.21875 \n",
       "Q 50.828125 45.953125 50.828125 35.296875 \n",
       "Q 50.828125 22.703125 48.234375 14.96875 \n",
       "Q 45.65625 7.234375 40.5 3 \n",
       "Q 35.359375 -1.21875 27.484375 -1.21875 \n",
       "Q 17.140625 -1.21875 11.234375 6.203125 \n",
       "Q 4.15625 15.140625 4.15625 35.296875 \n",
       "z\n",
       "M 13.1875 35.296875 \n",
       "Q 13.1875 17.671875 17.3125 11.828125 \n",
       "Q 21.4375 6 27.484375 6 \n",
       "Q 33.546875 6 37.671875 11.859375 \n",
       "Q 41.796875 17.71875 41.796875 35.296875 \n",
       "Q 41.796875 52.984375 37.671875 58.78125 \n",
       "Q 33.546875 64.59375 27.390625 64.59375 \n",
       "Q 21.34375 64.59375 17.71875 59.46875 \n",
       "Q 13.1875 52.9375 13.1875 35.296875 \n",
       "z\n",
       "\" id=\"ArialMT-48\"/>\n",
       "       <path d=\"M 9.078125 0 \n",
       "L 9.078125 10.015625 \n",
       "L 19.09375 10.015625 \n",
       "L 19.09375 0 \n",
       "z\n",
       "\" id=\"ArialMT-46\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(7.2 279.098826)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-48\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_2\">\n",
       "     <g id=\"line2d_17\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 28.1 224.589182 \n",
       "L 307.1 224.589182 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_18\"/>\n",
       "     <g id=\"text_9\">\n",
       "      <!-- 0.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(7.2 228.168088)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-48\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_3\">\n",
       "     <g id=\"line2d_19\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 28.1 173.658443 \n",
       "L 307.1 173.658443 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_20\"/>\n",
       "     <g id=\"text_10\">\n",
       "      <!-- 1.0 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(7.2 177.237349)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_4\">\n",
       "     <g id=\"line2d_21\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 28.1 122.727705 \n",
       "L 307.1 122.727705 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_22\"/>\n",
       "     <g id=\"text_11\">\n",
       "      <!-- 1.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(7.2 126.306611)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_5\">\n",
       "     <g id=\"line2d_23\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 28.1 71.796966 \n",
       "L 307.1 71.796966 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_24\"/>\n",
       "     <g id=\"text_12\">\n",
       "      <!-- 2.0 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(7.2 75.375872)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_6\">\n",
       "     <g id=\"line2d_25\">\n",
       "      <path clip-path=\"url(#pf92a7cc5c8)\" d=\"M 28.1 20.866227 \n",
       "L 307.1 20.866227 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_26\"/>\n",
       "     <g id=\"text_13\">\n",
       "      <!-- 2.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(7.2 24.445134)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_1\">\n",
       "    <defs>\n",
       "     <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"C0_0_fa2ee4cac7\"/>\n",
       "    </defs>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"216.828254\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"31.052375\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"169.740359\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"173.658443\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"268.196868\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"255.354714\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"20.866227\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"54.160979\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"244.961477\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"199.705383\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"203.986101\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"132.913852\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"169.740359\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"238.231843\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"132.913852\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"191.143948\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"199.705383\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"153.286148\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"195.424666\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"199.705383\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"132.913852\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"265.333773\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"191.143948\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"186.86323\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"153.286148\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"244.961477\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"54.160979\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"234.775329\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"208.266819\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"71.796966\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"191.143948\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"79.845286\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"203.986101\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"41.318826\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"79.845286\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"234.775329\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"182.582512\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"139.775334\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"173.658443\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"67.003132\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"234.775329\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"233.951125\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"191.143948\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"208.266819\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"238.231843\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"51.424671\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"165.459641\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"132.913852\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"71.28385\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"244.961477\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"216.828254\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"112.541557\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"178.301794\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"169.740359\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"153.286148\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"233.951125\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"61.610818\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"54.160979\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"216.828254\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"41.238523\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"67.003132\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"214.403034\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"148.33677\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"173.658443\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"233.951125\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"242.512561\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"20.866227\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"212.547537\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"246.793279\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"165.459641\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"163.472295\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"259.635432\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"41.238523\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"199.705383\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"112.541557\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"161.178923\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"163.472295\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"208.266819\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"216.828254\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"191.143948\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"102.355409\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"212.547537\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"81.983114\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"199.705383\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"132.913852\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"221.108972\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"71.796966\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"191.143948\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"112.541557\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"67.003132\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"216.828254\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"81.983114\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"178.301794\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"152.617488\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"169.740359\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"195.424666\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"132.913852\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"255.354714\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"45.599544\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"265.333773\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"225.38969\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"81.983114\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"178.301794\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"153.286148\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"71.28385\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"234.775329\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"208.266819\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"216.828254\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"126.933181\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"163.472295\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"244.961477\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"191.143948\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"259.635432\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"20.866227\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"178.301794\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"265.333773\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"251.073997\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"61.610818\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"242.512561\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"41.238523\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"246.793279\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"51.424671\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"238.231843\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"61.610818\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"67.003132\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"67.003132\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"216.828254\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"71.796966\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"242.512561\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"61.610818\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"54.160979\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"244.961477\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"229.670408\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"41.238523\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"212.547537\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"71.796966\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"229.670408\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"61.610818\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"54.160979\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"246.793279\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"112.541557\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"244.961477\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"54.160979\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"71.28385\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"234.775329\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"169.740359\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"251.073997\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"41.238523\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"281.039021\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"61.610818\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"265.333773\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"191.143948\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"186.86323\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"132.913852\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"49.880261\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"71.28385\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"224.589182\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"182.582512\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"234.775329\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"293.881174\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"41.238523\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"139.775334\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"173.658443\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"272.477585\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"71.796966\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"186.86323\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"132.913852\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"265.333773\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"203.986101\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"49.880261\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"285.319739\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"71.796966\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"244.961477\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"67.003132\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"259.635432\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"81.983114\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"238.231843\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"31.052375\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"174.021077\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"165.459641\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"153.286148\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"148.33677\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"173.658443\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"225.38969\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"41.238523\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"221.108972\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"41.238523\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"208.266819\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"122.727705\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"212.547537\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"102.355409\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"67.003132\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"156.898206\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"173.658443\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"238.231843\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"31.052375\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"216.828254\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"81.983114\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"195.424666\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"174.021077\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"203.986101\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"186.86323\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"54.160979\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"234.775329\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"62.722415\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"265.333773\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"238.231843\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"92.169261\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#8c564b;stroke:#8c564b;stroke-width:0.3;\" x=\"174.021077\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"173.658443\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#17becf;stroke:#17becf;stroke-width:0.3;\" x=\"285.319739\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"51.424671\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pf92a7cc5c8)\">\n",
       "     <use style=\"fill:#1f77b4;stroke:#1f77b4;stroke-width:0.3;\" x=\"58.441697\" xlink:href=\"#C0_0_fa2ee4cac7\" y=\"255.147625\"/>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"patch_3\">\n",
       "    <path d=\"M 28.1 279 \n",
       "L 28.1 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_4\">\n",
       "    <path d=\"M 307.1 279 \n",
       "L 307.1 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_5\">\n",
       "    <path d=\"M 28.1 279 \n",
       "L 307.1 279 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_6\">\n",
       "    <path d=\"M 28.1 7.2 \n",
       "L 307.1 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "  </g>\n",
       " </g>\n",
       " <defs>\n",
       "  <clipPath id=\"pf92a7cc5c8\">\n",
       "   <rect height=\"271.8\" width=\"279\" x=\"28.1\" y=\"7.2\"/>\n",
       "  </clipPath>\n",
       " </defs>\n",
       "</svg>\n"
      ],
      "text/plain": [
       "<Figure size 360x360 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "X2 = X[['petal_length', 'petal_width']]\n",
    "Y_encoded = Y.replace({'setosa':0, 'versicolor':1, 'virginica':2})\n",
    "\n",
    "plt.figure(figsize=(5,5))\n",
    "plt.scatter(X2.petal_length, X2.petal_width, c=Y_encoded, cmap='tab10');"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "We define a utility function that makes a scatterplot of a dataset and also draws the decision boundary. Here, we're using a bit of NumPy and matplotlib tricks. Don't worry about the details."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_boundary(clf, X, Y, cmap='tab10', names=None):\n",
    "\n",
    "    if isinstance(X, pd.DataFrame):\n",
    "        if not names:\n",
    "            names = list(X.columns)\n",
    "        X = X.to_numpy()\n",
    "\n",
    "    x_min, x_max = X[:,0].min(), X[:,0].max()\n",
    "    y_min, y_max = X[:,1].min(), X[:,1].max()\n",
    "\n",
    "    x_off = (x_max-x_min)/25\n",
    "    y_off = (y_max-y_min)/25\n",
    "    x_min -= x_off\n",
    "    x_max += x_off\n",
    "    y_min -= y_off\n",
    "    y_max += y_off\n",
    "    \n",
    "    xs = np.linspace(x_min, x_max, 250)\n",
    "    ys = np.linspace(y_min, y_max, 250)\n",
    "    \n",
    "    xx, yy = np.meshgrid(xs, ys)\n",
    "    \n",
    "    lenc = {c:i for i, c in enumerate(clf.classes_)}\n",
    "    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])\n",
    "    Z = np.array([lenc[z] for z in Z])\n",
    "    Z = Z.reshape(xx.shape)\n",
    "    Yenc = [lenc[y] for y in Y]\n",
    "    plt.figure(figsize=(5,5))\n",
    "    plt.contourf(xx, yy, Z, cmap=cmap, alpha=0.15)\n",
    "    plt.contour(xx, yy, Z, colors='k', linewidths=0.2)\n",
    "        \n",
    "    sc = plt.scatter(X[:,0], X[:,1], c=Yenc, cmap=cmap, alpha=0.5, edgecolors='k', linewidths=0.5);\n",
    "    \n",
    "    plt.legend(handles=sc.legend_elements()[0], labels=list(clf.classes_))\n",
    "    \n",
    "    if names:\n",
    "        plt.xlabel(names[0])\n",
    "        plt.ylabel(names[1])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now we can plot some decision boundaries. Here, we see the how it looks for a linear classifier:"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/svg+xml": [
       "<?xml version=\"1.0\" encoding=\"utf-8\" standalone=\"no\"?>\n",
       "<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n",
       "  \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n",
       "<!-- Created with matplotlib (https://matplotlib.org/) -->\n",
       "<svg height=\"316.534063pt\" version=\"1.1\" viewBox=\"0 0 328.359844 316.534063\" width=\"328.359844pt\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n",
       " <defs>\n",
       "  <style type=\"text/css\">\n",
       "*{stroke-linecap:butt;stroke-linejoin:round;}\n",
       "  </style>\n",
       " </defs>\n",
       " <g id=\"figure_1\">\n",
       "  <g id=\"patch_1\">\n",
       "   <path d=\"M 0 316.534063 \n",
       "L 328.359844 316.534063 \n",
       "L 328.359844 0 \n",
       "L 0 0 \n",
       "z\n",
       "\" style=\"fill:none;\"/>\n",
       "  </g>\n",
       "  <g id=\"axes_1\">\n",
       "   <g id=\"patch_2\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 321.159844 279 \n",
       "L 321.159844 7.2 \n",
       "L 42.159844 7.2 \n",
       "z\n",
       "\" style=\"fill:#eaeaf2;\"/>\n",
       "   </g>\n",
       "   <g id=\"matplotlib.axis_1\">\n",
       "    <g id=\"xtick_1\">\n",
       "     <g id=\"line2d_1\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 52.493177 279 \n",
       "L 52.493177 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_2\"/>\n",
       "     <g id=\"text_1\">\n",
       "      <!-- 1 -->\n",
       "      <defs>\n",
       "       <path d=\"M 37.25 0 \n",
       "L 28.46875 0 \n",
       "L 28.46875 56 \n",
       "Q 25.296875 52.984375 20.140625 49.953125 \n",
       "Q 14.984375 46.921875 10.890625 45.40625 \n",
       "L 10.890625 53.90625 \n",
       "Q 18.265625 57.375 23.78125 62.296875 \n",
       "Q 29.296875 67.234375 31.59375 71.875 \n",
       "L 37.25 71.875 \n",
       "z\n",
       "\" id=\"ArialMT-49\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(49.712708 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_2\">\n",
       "     <g id=\"line2d_3\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 96.278488 279 \n",
       "L 96.278488 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_4\"/>\n",
       "     <g id=\"text_2\">\n",
       "      <!-- 2 -->\n",
       "      <defs>\n",
       "       <path d=\"M 50.34375 8.453125 \n",
       "L 50.34375 0 \n",
       "L 3.03125 0 \n",
       "Q 2.9375 3.171875 4.046875 6.109375 \n",
       "Q 5.859375 10.9375 9.828125 15.625 \n",
       "Q 13.8125 20.3125 21.34375 26.46875 \n",
       "Q 33.015625 36.03125 37.109375 41.625 \n",
       "Q 41.21875 47.21875 41.21875 52.203125 \n",
       "Q 41.21875 57.421875 37.46875 61 \n",
       "Q 33.734375 64.59375 27.734375 64.59375 \n",
       "Q 21.390625 64.59375 17.578125 60.78125 \n",
       "Q 13.765625 56.984375 13.71875 50.25 \n",
       "L 4.6875 51.171875 \n",
       "Q 5.609375 61.28125 11.65625 66.578125 \n",
       "Q 17.71875 71.875 27.9375 71.875 \n",
       "Q 38.234375 71.875 44.234375 66.15625 \n",
       "Q 50.25 60.453125 50.25 52 \n",
       "Q 50.25 47.703125 48.484375 43.546875 \n",
       "Q 46.734375 39.40625 42.65625 34.8125 \n",
       "Q 38.578125 30.21875 29.109375 22.21875 \n",
       "Q 21.1875 15.578125 18.9375 13.203125 \n",
       "Q 16.703125 10.84375 15.234375 8.453125 \n",
       "z\n",
       "\" id=\"ArialMT-50\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(93.498019 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_3\">\n",
       "     <g id=\"line2d_5\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 140.063799 279 \n",
       "L 140.063799 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_6\"/>\n",
       "     <g id=\"text_3\">\n",
       "      <!-- 3 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.203125 18.890625 \n",
       "L 12.984375 20.0625 \n",
       "Q 14.5 12.59375 18.140625 9.296875 \n",
       "Q 21.78125 6 27 6 \n",
       "Q 33.203125 6 37.46875 10.296875 \n",
       "Q 41.75 14.59375 41.75 20.953125 \n",
       "Q 41.75 27 37.796875 30.921875 \n",
       "Q 33.84375 34.859375 27.734375 34.859375 \n",
       "Q 25.25 34.859375 21.53125 33.890625 \n",
       "L 22.515625 41.609375 \n",
       "Q 23.390625 41.5 23.921875 41.5 \n",
       "Q 29.546875 41.5 34.03125 44.421875 \n",
       "Q 38.53125 47.359375 38.53125 53.46875 \n",
       "Q 38.53125 58.296875 35.25 61.46875 \n",
       "Q 31.984375 64.65625 26.8125 64.65625 \n",
       "Q 21.6875 64.65625 18.265625 61.421875 \n",
       "Q 14.84375 58.203125 13.875 51.765625 \n",
       "L 5.078125 53.328125 \n",
       "Q 6.6875 62.15625 12.390625 67.015625 \n",
       "Q 18.109375 71.875 26.609375 71.875 \n",
       "Q 32.46875 71.875 37.390625 69.359375 \n",
       "Q 42.328125 66.84375 44.9375 62.5 \n",
       "Q 47.5625 58.15625 47.5625 53.265625 \n",
       "Q 47.5625 48.640625 45.0625 44.828125 \n",
       "Q 42.578125 41.015625 37.703125 38.765625 \n",
       "Q 44.046875 37.3125 47.5625 32.6875 \n",
       "Q 51.078125 28.078125 51.078125 21.140625 \n",
       "Q 51.078125 11.765625 44.234375 5.25 \n",
       "Q 37.40625 -1.265625 26.953125 -1.265625 \n",
       "Q 17.53125 -1.265625 11.296875 4.34375 \n",
       "Q 5.078125 9.96875 4.203125 18.890625 \n",
       "z\n",
       "\" id=\"ArialMT-51\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(137.28333 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-51\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_4\">\n",
       "     <g id=\"line2d_7\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 183.849109 279 \n",
       "L 183.849109 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_8\"/>\n",
       "     <g id=\"text_4\">\n",
       "      <!-- 4 -->\n",
       "      <defs>\n",
       "       <path d=\"M 32.328125 0 \n",
       "L 32.328125 17.140625 \n",
       "L 1.265625 17.140625 \n",
       "L 1.265625 25.203125 \n",
       "L 33.9375 71.578125 \n",
       "L 41.109375 71.578125 \n",
       "L 41.109375 25.203125 \n",
       "L 50.78125 25.203125 \n",
       "L 50.78125 17.140625 \n",
       "L 41.109375 17.140625 \n",
       "L 41.109375 0 \n",
       "z\n",
       "M 32.328125 25.203125 \n",
       "L 32.328125 57.46875 \n",
       "L 9.90625 25.203125 \n",
       "z\n",
       "\" id=\"ArialMT-52\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(181.068641 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-52\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_5\">\n",
       "     <g id=\"line2d_9\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 227.63442 279 \n",
       "L 227.63442 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_10\"/>\n",
       "     <g id=\"text_5\">\n",
       "      <!-- 5 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.15625 18.75 \n",
       "L 13.375 19.53125 \n",
       "Q 14.40625 12.796875 18.140625 9.390625 \n",
       "Q 21.875 6 27.15625 6 \n",
       "Q 33.5 6 37.890625 10.78125 \n",
       "Q 42.28125 15.578125 42.28125 23.484375 \n",
       "Q 42.28125 31 38.0625 35.34375 \n",
       "Q 33.84375 39.703125 27 39.703125 \n",
       "Q 22.75 39.703125 19.328125 37.765625 \n",
       "Q 15.921875 35.84375 13.96875 32.765625 \n",
       "L 5.71875 33.84375 \n",
       "L 12.640625 70.609375 \n",
       "L 48.25 70.609375 \n",
       "L 48.25 62.203125 \n",
       "L 19.671875 62.203125 \n",
       "L 15.828125 42.96875 \n",
       "Q 22.265625 47.46875 29.34375 47.46875 \n",
       "Q 38.71875 47.46875 45.15625 40.96875 \n",
       "Q 51.609375 34.46875 51.609375 24.265625 \n",
       "Q 51.609375 14.546875 45.953125 7.46875 \n",
       "Q 39.0625 -1.21875 27.15625 -1.21875 \n",
       "Q 17.390625 -1.21875 11.203125 4.25 \n",
       "Q 5.03125 9.71875 4.15625 18.75 \n",
       "z\n",
       "\" id=\"ArialMT-53\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(224.853951 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_6\">\n",
       "     <g id=\"line2d_11\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 271.419731 279 \n",
       "L 271.419731 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_12\"/>\n",
       "     <g id=\"text_6\">\n",
       "      <!-- 6 -->\n",
       "      <defs>\n",
       "       <path d=\"M 49.75 54.046875 \n",
       "L 41.015625 53.375 \n",
       "Q 39.84375 58.546875 37.703125 60.890625 \n",
       "Q 34.125 64.65625 28.90625 64.65625 \n",
       "Q 24.703125 64.65625 21.53125 62.3125 \n",
       "Q 17.390625 59.28125 14.984375 53.46875 \n",
       "Q 12.59375 47.65625 12.5 36.921875 \n",
       "Q 15.671875 41.75 20.265625 44.09375 \n",
       "Q 24.859375 46.4375 29.890625 46.4375 \n",
       "Q 38.671875 46.4375 44.84375 39.96875 \n",
       "Q 51.03125 33.5 51.03125 23.25 \n",
       "Q 51.03125 16.5 48.125 10.71875 \n",
       "Q 45.21875 4.9375 40.140625 1.859375 \n",
       "Q 35.0625 -1.21875 28.609375 -1.21875 \n",
       "Q 17.625 -1.21875 10.6875 6.859375 \n",
       "Q 3.765625 14.9375 3.765625 33.5 \n",
       "Q 3.765625 54.25 11.421875 63.671875 \n",
       "Q 18.109375 71.875 29.4375 71.875 \n",
       "Q 37.890625 71.875 43.28125 67.140625 \n",
       "Q 48.6875 62.40625 49.75 54.046875 \n",
       "z\n",
       "M 13.875 23.1875 \n",
       "Q 13.875 18.65625 15.796875 14.5 \n",
       "Q 17.71875 10.359375 21.1875 8.171875 \n",
       "Q 24.65625 6 28.46875 6 \n",
       "Q 34.03125 6 38.03125 10.484375 \n",
       "Q 42.046875 14.984375 42.046875 22.703125 \n",
       "Q 42.046875 30.125 38.078125 34.390625 \n",
       "Q 34.125 38.671875 28.125 38.671875 \n",
       "Q 22.171875 38.671875 18.015625 34.390625 \n",
       "Q 13.875 30.125 13.875 23.1875 \n",
       "z\n",
       "\" id=\"ArialMT-54\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(268.639262 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-54\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_7\">\n",
       "     <g id=\"line2d_13\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 315.205041 279 \n",
       "L 315.205041 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_14\"/>\n",
       "     <g id=\"text_7\">\n",
       "      <!-- 7 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.734375 62.203125 \n",
       "L 4.734375 70.65625 \n",
       "L 51.078125 70.65625 \n",
       "L 51.078125 63.8125 \n",
       "Q 44.234375 56.546875 37.515625 44.484375 \n",
       "Q 30.8125 32.421875 27.15625 19.671875 \n",
       "Q 24.515625 10.6875 23.78125 0 \n",
       "L 14.75 0 \n",
       "Q 14.890625 8.453125 18.0625 20.40625 \n",
       "Q 21.234375 32.375 27.171875 43.484375 \n",
       "Q 33.109375 54.59375 39.796875 62.203125 \n",
       "z\n",
       "\" id=\"ArialMT-55\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(312.424573 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-55\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_8\">\n",
       "     <!-- petal_length -->\n",
       "     <defs>\n",
       "      <path d=\"M 6.59375 -19.875 \n",
       "L 6.59375 51.859375 \n",
       "L 14.59375 51.859375 \n",
       "L 14.59375 45.125 \n",
       "Q 17.4375 49.078125 21 51.046875 \n",
       "Q 24.5625 53.03125 29.640625 53.03125 \n",
       "Q 36.28125 53.03125 41.359375 49.609375 \n",
       "Q 46.4375 46.1875 49.015625 39.953125 \n",
       "Q 51.609375 33.734375 51.609375 26.3125 \n",
       "Q 51.609375 18.359375 48.75 11.984375 \n",
       "Q 45.90625 5.609375 40.453125 2.21875 \n",
       "Q 35.015625 -1.171875 29 -1.171875 \n",
       "Q 24.609375 -1.171875 21.109375 0.6875 \n",
       "Q 17.625 2.546875 15.375 5.375 \n",
       "L 15.375 -19.875 \n",
       "z\n",
       "M 14.546875 25.640625 \n",
       "Q 14.546875 15.625 18.59375 10.84375 \n",
       "Q 22.65625 6.0625 28.421875 6.0625 \n",
       "Q 34.28125 6.0625 38.453125 11.015625 \n",
       "Q 42.625 15.96875 42.625 26.375 \n",
       "Q 42.625 36.28125 38.546875 41.203125 \n",
       "Q 34.46875 46.140625 28.8125 46.140625 \n",
       "Q 23.1875 46.140625 18.859375 40.890625 \n",
       "Q 14.546875 35.640625 14.546875 25.640625 \n",
       "z\n",
       "\" id=\"ArialMT-112\"/>\n",
       "      <path d=\"M 42.09375 16.703125 \n",
       "L 51.171875 15.578125 \n",
       "Q 49.03125 7.625 43.21875 3.21875 \n",
       "Q 37.40625 -1.171875 28.375 -1.171875 \n",
       "Q 17 -1.171875 10.328125 5.828125 \n",
       "Q 3.65625 12.84375 3.65625 25.484375 \n",
       "Q 3.65625 38.578125 10.390625 45.796875 \n",
       "Q 17.140625 53.03125 27.875 53.03125 \n",
       "Q 38.28125 53.03125 44.875 45.953125 \n",
       "Q 51.46875 38.875 51.46875 26.03125 \n",
       "Q 51.46875 25.25 51.421875 23.6875 \n",
       "L 12.75 23.6875 \n",
       "Q 13.234375 15.140625 17.578125 10.59375 \n",
       "Q 21.921875 6.0625 28.421875 6.0625 \n",
       "Q 33.25 6.0625 36.671875 8.59375 \n",
       "Q 40.09375 11.140625 42.09375 16.703125 \n",
       "z\n",
       "M 13.234375 30.90625 \n",
       "L 42.1875 30.90625 \n",
       "Q 41.609375 37.453125 38.875 40.71875 \n",
       "Q 34.671875 45.796875 27.984375 45.796875 \n",
       "Q 21.921875 45.796875 17.796875 41.75 \n",
       "Q 13.671875 37.703125 13.234375 30.90625 \n",
       "z\n",
       "\" id=\"ArialMT-101\"/>\n",
       "      <path d=\"M 25.78125 7.859375 \n",
       "L 27.046875 0.09375 \n",
       "Q 23.34375 -0.6875 20.40625 -0.6875 \n",
       "Q 15.625 -0.6875 12.984375 0.828125 \n",
       "Q 10.359375 2.34375 9.28125 4.8125 \n",
       "Q 8.203125 7.28125 8.203125 15.1875 \n",
       "L 8.203125 45.015625 \n",
       "L 1.765625 45.015625 \n",
       "L 1.765625 51.859375 \n",
       "L 8.203125 51.859375 \n",
       "L 8.203125 64.703125 \n",
       "L 16.9375 69.96875 \n",
       "L 16.9375 51.859375 \n",
       "L 25.78125 51.859375 \n",
       "L 25.78125 45.015625 \n",
       "L 16.9375 45.015625 \n",
       "L 16.9375 14.703125 \n",
       "Q 16.9375 10.9375 17.40625 9.859375 \n",
       "Q 17.875 8.796875 18.921875 8.15625 \n",
       "Q 19.96875 7.515625 21.921875 7.515625 \n",
       "Q 23.390625 7.515625 25.78125 7.859375 \n",
       "z\n",
       "\" id=\"ArialMT-116\"/>\n",
       "      <path d=\"M 40.4375 6.390625 \n",
       "Q 35.546875 2.25 31.03125 0.53125 \n",
       "Q 26.515625 -1.171875 21.34375 -1.171875 \n",
       "Q 12.796875 -1.171875 8.203125 3 \n",
       "Q 3.609375 7.171875 3.609375 13.671875 \n",
       "Q 3.609375 17.484375 5.34375 20.625 \n",
       "Q 7.078125 23.78125 9.890625 25.6875 \n",
       "Q 12.703125 27.59375 16.21875 28.5625 \n",
       "Q 18.796875 29.25 24.03125 29.890625 \n",
       "Q 34.671875 31.15625 39.703125 32.90625 \n",
       "Q 39.75 34.71875 39.75 35.203125 \n",
       "Q 39.75 40.578125 37.25 42.78125 \n",
       "Q 33.890625 45.75 27.25 45.75 \n",
       "Q 21.046875 45.75 18.09375 43.578125 \n",
       "Q 15.140625 41.40625 13.71875 35.890625 \n",
       "L 5.125 37.0625 \n",
       "Q 6.296875 42.578125 8.984375 45.96875 \n",
       "Q 11.671875 49.359375 16.75 51.1875 \n",
       "Q 21.828125 53.03125 28.515625 53.03125 \n",
       "Q 35.15625 53.03125 39.296875 51.46875 \n",
       "Q 43.453125 49.90625 45.40625 47.53125 \n",
       "Q 47.359375 45.171875 48.140625 41.546875 \n",
       "Q 48.578125 39.3125 48.578125 33.453125 \n",
       "L 48.578125 21.734375 \n",
       "Q 48.578125 9.46875 49.140625 6.21875 \n",
       "Q 49.703125 2.984375 51.375 0 \n",
       "L 42.1875 0 \n",
       "Q 40.828125 2.734375 40.4375 6.390625 \n",
       "z\n",
       "M 39.703125 26.03125 \n",
       "Q 34.90625 24.078125 25.34375 22.703125 \n",
       "Q 19.921875 21.921875 17.671875 20.9375 \n",
       "Q 15.4375 19.96875 14.203125 18.09375 \n",
       "Q 12.984375 16.21875 12.984375 13.921875 \n",
       "Q 12.984375 10.40625 15.640625 8.0625 \n",
       "Q 18.3125 5.71875 23.4375 5.71875 \n",
       "Q 28.515625 5.71875 32.46875 7.9375 \n",
       "Q 36.421875 10.15625 38.28125 14.015625 \n",
       "Q 39.703125 17 39.703125 22.796875 \n",
       "z\n",
       "\" id=\"ArialMT-97\"/>\n",
       "      <path d=\"M 6.390625 0 \n",
       "L 6.390625 71.578125 \n",
       "L 15.1875 71.578125 \n",
       "L 15.1875 0 \n",
       "z\n",
       "\" id=\"ArialMT-108\"/>\n",
       "      <path d=\"M -1.515625 -19.875 \n",
       "L -1.515625 -13.53125 \n",
       "L 56.734375 -13.53125 \n",
       "L 56.734375 -19.875 \n",
       "z\n",
       "\" id=\"ArialMT-95\"/>\n",
       "      <path d=\"M 6.59375 0 \n",
       "L 6.59375 51.859375 \n",
       "L 14.5 51.859375 \n",
       "L 14.5 44.484375 \n",
       "Q 20.21875 53.03125 31 53.03125 \n",
       "Q 35.6875 53.03125 39.625 51.34375 \n",
       "Q 43.5625 49.65625 45.515625 46.921875 \n",
       "Q 47.46875 44.1875 48.25 40.4375 \n",
       "Q 48.734375 37.984375 48.734375 31.890625 \n",
       "L 48.734375 0 \n",
       "L 39.9375 0 \n",
       "L 39.9375 31.546875 \n",
       "Q 39.9375 36.921875 38.90625 39.578125 \n",
       "Q 37.890625 42.234375 35.28125 43.8125 \n",
       "Q 32.671875 45.40625 29.15625 45.40625 \n",
       "Q 23.53125 45.40625 19.453125 41.84375 \n",
       "Q 15.375 38.28125 15.375 28.328125 \n",
       "L 15.375 0 \n",
       "z\n",
       "\" id=\"ArialMT-110\"/>\n",
       "      <path d=\"M 4.984375 -4.296875 \n",
       "L 13.53125 -5.5625 \n",
       "Q 14.0625 -9.515625 16.5 -11.328125 \n",
       "Q 19.78125 -13.765625 25.4375 -13.765625 \n",
       "Q 31.546875 -13.765625 34.859375 -11.328125 \n",
       "Q 38.1875 -8.890625 39.359375 -4.5 \n",
       "Q 40.046875 -1.8125 39.984375 6.78125 \n",
       "Q 34.234375 0 25.640625 0 \n",
       "Q 14.9375 0 9.078125 7.71875 \n",
       "Q 3.21875 15.4375 3.21875 26.21875 \n",
       "Q 3.21875 33.640625 5.90625 39.90625 \n",
       "Q 8.59375 46.1875 13.6875 49.609375 \n",
       "Q 18.796875 53.03125 25.6875 53.03125 \n",
       "Q 34.859375 53.03125 40.828125 45.609375 \n",
       "L 40.828125 51.859375 \n",
       "L 48.921875 51.859375 \n",
       "L 48.921875 7.03125 \n",
       "Q 48.921875 -5.078125 46.453125 -10.125 \n",
       "Q 44 -15.1875 38.640625 -18.109375 \n",
       "Q 33.296875 -21.046875 25.484375 -21.046875 \n",
       "Q 16.21875 -21.046875 10.5 -16.875 \n",
       "Q 4.78125 -12.703125 4.984375 -4.296875 \n",
       "z\n",
       "M 12.25 26.859375 \n",
       "Q 12.25 16.65625 16.296875 11.96875 \n",
       "Q 20.359375 7.28125 26.46875 7.28125 \n",
       "Q 32.515625 7.28125 36.609375 11.9375 \n",
       "Q 40.71875 16.609375 40.71875 26.5625 \n",
       "Q 40.71875 36.078125 36.5 40.90625 \n",
       "Q 32.28125 45.75 26.3125 45.75 \n",
       "Q 20.453125 45.75 16.34375 40.984375 \n",
       "Q 12.25 36.234375 12.25 26.859375 \n",
       "z\n",
       "\" id=\"ArialMT-103\"/>\n",
       "      <path d=\"M 6.59375 0 \n",
       "L 6.59375 71.578125 \n",
       "L 15.375 71.578125 \n",
       "L 15.375 45.90625 \n",
       "Q 21.53125 53.03125 30.90625 53.03125 \n",
       "Q 36.671875 53.03125 40.921875 50.75 \n",
       "Q 45.171875 48.484375 47 44.484375 \n",
       "Q 48.828125 40.484375 48.828125 32.859375 \n",
       "L 48.828125 0 \n",
       "L 40.046875 0 \n",
       "L 40.046875 32.859375 \n",
       "Q 40.046875 39.453125 37.1875 42.453125 \n",
       "Q 34.328125 45.453125 29.109375 45.453125 \n",
       "Q 25.203125 45.453125 21.75 43.421875 \n",
       "Q 18.3125 41.40625 16.84375 37.9375 \n",
       "Q 15.375 34.46875 15.375 28.375 \n",
       "L 15.375 0 \n",
       "z\n",
       "\" id=\"ArialMT-104\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(151.691719 307.018906)scale(0.11 -0.11)\">\n",
       "      <use xlink:href=\"#ArialMT-112\"/>\n",
       "      <use x=\"55.615234\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"111.230469\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"139.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "      <use x=\"194.628906\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"216.845703\" xlink:href=\"#ArialMT-95\"/>\n",
       "      <use x=\"272.460938\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"294.677734\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"350.292969\" xlink:href=\"#ArialMT-110\"/>\n",
       "      <use x=\"405.908203\" xlink:href=\"#ArialMT-103\"/>\n",
       "      <use x=\"461.523438\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"489.306641\" xlink:href=\"#ArialMT-104\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"matplotlib.axis_2\">\n",
       "    <g id=\"ytick_1\">\n",
       "     <g id=\"line2d_15\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 226.988889 \n",
       "L 321.159844 226.988889 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_16\"/>\n",
       "     <g id=\"text_9\">\n",
       "      <!-- 0.5 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.15625 35.296875 \n",
       "Q 4.15625 48 6.765625 55.734375 \n",
       "Q 9.375 63.484375 14.515625 67.671875 \n",
       "Q 19.671875 71.875 27.484375 71.875 \n",
       "Q 33.25 71.875 37.59375 69.546875 \n",
       "Q 41.9375 67.234375 44.765625 62.859375 \n",
       "Q 47.609375 58.5 49.21875 52.21875 \n",
       "Q 50.828125 45.953125 50.828125 35.296875 \n",
       "Q 50.828125 22.703125 48.234375 14.96875 \n",
       "Q 45.65625 7.234375 40.5 3 \n",
       "Q 35.359375 -1.21875 27.484375 -1.21875 \n",
       "Q 17.140625 -1.21875 11.234375 6.203125 \n",
       "Q 4.15625 15.140625 4.15625 35.296875 \n",
       "z\n",
       "M 13.1875 35.296875 \n",
       "Q 13.1875 17.671875 17.3125 11.828125 \n",
       "Q 21.4375 6 27.484375 6 \n",
       "Q 33.546875 6 37.671875 11.859375 \n",
       "Q 41.796875 17.71875 41.796875 35.296875 \n",
       "Q 41.796875 52.984375 37.671875 58.78125 \n",
       "Q 33.546875 64.59375 27.390625 64.59375 \n",
       "Q 21.34375 64.59375 17.71875 59.46875 \n",
       "Q 13.1875 52.9375 13.1875 35.296875 \n",
       "z\n",
       "\" id=\"ArialMT-48\"/>\n",
       "       <path d=\"M 9.078125 0 \n",
       "L 9.078125 10.015625 \n",
       "L 19.09375 10.015625 \n",
       "L 19.09375 0 \n",
       "z\n",
       "\" id=\"ArialMT-46\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 230.567795)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-48\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_2\">\n",
       "     <g id=\"line2d_17\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 174.558333 \n",
       "L 321.159844 174.558333 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_18\"/>\n",
       "     <g id=\"text_10\">\n",
       "      <!-- 1.0 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 178.13724)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_3\">\n",
       "     <g id=\"line2d_19\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 122.127778 \n",
       "L 321.159844 122.127778 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_20\"/>\n",
       "     <g id=\"text_11\">\n",
       "      <!-- 1.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 125.706684)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_4\">\n",
       "     <g id=\"line2d_21\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 69.697222 \n",
       "L 321.159844 69.697222 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_22\"/>\n",
       "     <g id=\"text_12\">\n",
       "      <!-- 2.0 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 73.276128)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_5\">\n",
       "     <g id=\"line2d_23\">\n",
       "      <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 17.266667 \n",
       "L 321.159844 17.266667 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_24\"/>\n",
       "     <g id=\"text_13\">\n",
       "      <!-- 2.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 20.845573)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_14\">\n",
       "     <!-- petal_width -->\n",
       "     <defs>\n",
       "      <path d=\"M 16.15625 0 \n",
       "L 0.296875 51.859375 \n",
       "L 9.375 51.859375 \n",
       "L 17.625 21.921875 \n",
       "L 20.703125 10.796875 \n",
       "Q 20.90625 11.625 23.390625 21.484375 \n",
       "L 31.640625 51.859375 \n",
       "L 40.671875 51.859375 \n",
       "L 48.4375 21.78125 \n",
       "L 51.03125 11.859375 \n",
       "L 54 21.875 \n",
       "L 62.890625 51.859375 \n",
       "L 71.4375 51.859375 \n",
       "L 55.21875 0 \n",
       "L 46.09375 0 \n",
       "L 37.84375 31.0625 \n",
       "L 35.84375 39.890625 \n",
       "L 25.34375 0 \n",
       "z\n",
       "\" id=\"ArialMT-119\"/>\n",
       "      <path d=\"M 6.640625 61.46875 \n",
       "L 6.640625 71.578125 \n",
       "L 15.4375 71.578125 \n",
       "L 15.4375 61.46875 \n",
       "z\n",
       "M 6.640625 0 \n",
       "L 6.640625 51.859375 \n",
       "L 15.4375 51.859375 \n",
       "L 15.4375 0 \n",
       "z\n",
       "\" id=\"ArialMT-105\"/>\n",
       "      <path d=\"M 40.234375 0 \n",
       "L 40.234375 6.546875 \n",
       "Q 35.296875 -1.171875 25.734375 -1.171875 \n",
       "Q 19.53125 -1.171875 14.328125 2.25 \n",
       "Q 9.125 5.671875 6.265625 11.796875 \n",
       "Q 3.421875 17.921875 3.421875 25.875 \n",
       "Q 3.421875 33.640625 6 39.96875 \n",
       "Q 8.59375 46.296875 13.765625 49.65625 \n",
       "Q 18.953125 53.03125 25.34375 53.03125 \n",
       "Q 30.03125 53.03125 33.6875 51.046875 \n",
       "Q 37.359375 49.078125 39.65625 45.90625 \n",
       "L 39.65625 71.578125 \n",
       "L 48.390625 71.578125 \n",
       "L 48.390625 0 \n",
       "z\n",
       "M 12.453125 25.875 \n",
       "Q 12.453125 15.921875 16.640625 10.984375 \n",
       "Q 20.84375 6.0625 26.5625 6.0625 \n",
       "Q 32.328125 6.0625 36.34375 10.765625 \n",
       "Q 40.375 15.484375 40.375 25.140625 \n",
       "Q 40.375 35.796875 36.265625 40.765625 \n",
       "Q 32.171875 45.75 26.171875 45.75 \n",
       "Q 20.3125 45.75 16.375 40.96875 \n",
       "Q 12.453125 36.1875 12.453125 25.875 \n",
       "z\n",
       "\" id=\"ArialMT-100\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(15.073594 170.923125)rotate(-90)scale(0.11 -0.11)\">\n",
       "      <use xlink:href=\"#ArialMT-112\"/>\n",
       "      <use x=\"55.615234\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"111.230469\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"139.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "      <use x=\"194.628906\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"216.845703\" xlink:href=\"#ArialMT-95\"/>\n",
       "      <use x=\"272.460938\" xlink:href=\"#ArialMT-119\"/>\n",
       "      <use x=\"344.677734\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"366.894531\" xlink:href=\"#ArialMT-100\"/>\n",
       "      <use x=\"422.509766\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"450.292969\" xlink:href=\"#ArialMT-104\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_1\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 43.280326 279 \n",
       "L 44.400808 279 \n",
       "L 45.52129 279 \n",
       "L 46.641771 279 \n",
       "L 47.762253 279 \n",
       "L 48.882735 279 \n",
       "L 50.003217 279 \n",
       "L 51.123699 279 \n",
       "L 52.244181 279 \n",
       "L 53.364663 279 \n",
       "L 54.485145 279 \n",
       "L 55.605627 279 \n",
       "L 56.726109 279 \n",
       "L 57.846591 279 \n",
       "L 58.967073 279 \n",
       "L 60.087555 279 \n",
       "L 61.208037 279 \n",
       "L 62.328518 279 \n",
       "L 63.449 279 \n",
       "L 64.569482 279 \n",
       "L 65.689964 279 \n",
       "L 66.810446 279 \n",
       "L 67.930928 279 \n",
       "L 69.05141 279 \n",
       "L 70.171892 279 \n",
       "L 71.292374 279 \n",
       "L 72.412856 279 \n",
       "L 73.533338 279 \n",
       "L 74.65382 279 \n",
       "L 75.774302 279 \n",
       "L 76.894784 279 \n",
       "L 78.015265 279 \n",
       "L 79.135747 279 \n",
       "L 80.256229 279 \n",
       "L 81.376711 279 \n",
       "L 82.497193 279 \n",
       "L 83.617675 279 \n",
       "L 84.738157 279 \n",
       "L 85.858639 279 \n",
       "L 86.979121 279 \n",
       "L 88.099603 279 \n",
       "L 89.220085 279 \n",
       "L 90.340567 279 \n",
       "L 91.461049 279 \n",
       "L 92.58153 279 \n",
       "L 93.702012 279 \n",
       "L 94.822494 279 \n",
       "L 95.942976 279 \n",
       "L 97.063458 279 \n",
       "L 98.18394 279 \n",
       "L 99.304422 279 \n",
       "L 100.424904 279 \n",
       "L 101.545386 279 \n",
       "L 102.665868 279 \n",
       "L 103.78635 279 \n",
       "L 104.906832 279 \n",
       "L 106.027314 279 \n",
       "L 107.147796 279 \n",
       "L 108.268277 279 \n",
       "L 109.388759 279 \n",
       "L 110.509241 279 \n",
       "L 111.629723 279 \n",
       "L 112.750205 279 \n",
       "L 113.870687 279 \n",
       "L 114.991169 279 \n",
       "L 116.111651 279 \n",
       "L 117.232133 279 \n",
       "L 118.352615 279 \n",
       "L 119.473097 279 \n",
       "L 120.593579 279 \n",
       "L 121.714061 279 \n",
       "L 122.834543 279 \n",
       "L 123.955024 279 \n",
       "L 125.075506 279 \n",
       "L 126.195988 279 \n",
       "L 127.31647 279 \n",
       "L 127.596591 279 \n",
       "L 128.436952 278.181325 \n",
       "L 128.717073 277.908434 \n",
       "L 128.717073 276.816867 \n",
       "L 128.717073 275.725301 \n",
       "L 128.717073 274.633735 \n",
       "L 128.717073 273.542169 \n",
       "L 128.717073 272.450602 \n",
       "L 128.717073 271.359036 \n",
       "L 128.717073 270.26747 \n",
       "L 128.717073 269.175904 \n",
       "L 129.557434 268.357229 \n",
       "L 129.837555 268.084337 \n",
       "L 129.837555 266.992771 \n",
       "L 129.837555 265.901205 \n",
       "L 129.837555 264.809639 \n",
       "L 129.837555 263.718072 \n",
       "L 129.837555 262.626506 \n",
       "L 129.837555 261.53494 \n",
       "L 129.837555 260.443373 \n",
       "L 130.677916 259.624699 \n",
       "L 130.958037 259.351807 \n",
       "L 130.958037 258.260241 \n",
       "L 130.958037 257.168675 \n",
       "L 130.958037 256.077108 \n",
       "L 130.958037 254.985542 \n",
       "L 130.958037 253.893976 \n",
       "L 130.958037 252.80241 \n",
       "L 130.958037 251.710843 \n",
       "L 130.958037 250.619277 \n",
       "L 131.798398 249.800602 \n",
       "L 132.078518 249.527711 \n",
       "L 132.078518 248.436145 \n",
       "L 132.078518 247.344578 \n",
       "L 132.078518 246.253012 \n",
       "L 132.078518 245.161446 \n",
       "L 132.078518 244.06988 \n",
       "L 132.078518 242.978313 \n",
       "L 132.078518 241.886747 \n",
       "L 132.078518 240.795181 \n",
       "L 132.91888 239.976506 \n",
       "L 133.199 239.703614 \n",
       "L 133.199 238.612048 \n",
       "L 133.199 237.520482 \n",
       "L 133.199 236.428916 \n",
       "L 133.199 235.337349 \n",
       "L 133.199 234.245783 \n",
       "L 133.199 233.154217 \n",
       "L 133.199 232.062651 \n",
       "L 133.199 230.971084 \n",
       "L 134.039362 230.15241 \n",
       "L 134.319482 229.879518 \n",
       "L 134.319482 228.787952 \n",
       "L 134.319482 227.696386 \n",
       "L 134.319482 226.604819 \n",
       "L 134.319482 225.513253 \n",
       "L 134.319482 224.421687 \n",
       "L 134.319482 223.33012 \n",
       "L 134.319482 222.238554 \n",
       "L 134.319482 221.146988 \n",
       "L 135.159844 220.328313 \n",
       "L 135.439964 220.055422 \n",
       "L 135.439964 218.963855 \n",
       "L 135.439964 217.872289 \n",
       "L 135.439964 216.780723 \n",
       "L 135.439964 215.689157 \n",
       "L 135.439964 214.59759 \n",
       "L 135.439964 213.506024 \n",
       "L 135.439964 212.414458 \n",
       "L 135.439964 211.322892 \n",
       "L 136.280326 210.504217 \n",
       "L 136.560446 210.231325 \n",
       "L 136.560446 209.139759 \n",
       "L 136.560446 208.048193 \n",
       "L 136.560446 206.956627 \n",
       "L 136.560446 205.86506 \n",
       "L 136.560446 204.773494 \n",
       "L 136.560446 203.681928 \n",
       "L 136.560446 202.590361 \n",
       "L 136.560446 201.498795 \n",
       "L 137.400808 200.68012 \n",
       "L 137.680928 200.407229 \n",
       "L 137.680928 199.315663 \n",
       "L 137.680928 198.224096 \n",
       "L 137.680928 197.13253 \n",
       "L 137.680928 196.040964 \n",
       "L 137.680928 194.949398 \n",
       "L 137.680928 193.857831 \n",
       "L 137.680928 192.766265 \n",
       "L 138.52129 191.94759 \n",
       "L 138.80141 191.674699 \n",
       "L 138.80141 190.583133 \n",
       "L 138.80141 189.491566 \n",
       "L 138.80141 188.4 \n",
       "L 138.80141 187.308434 \n",
       "L 138.80141 186.216867 \n",
       "L 138.80141 185.125301 \n",
       "L 138.80141 184.033735 \n",
       "L 138.80141 182.942169 \n",
       "L 139.641771 182.123494 \n",
       "L 139.921892 181.850602 \n",
       "L 139.921892 180.759036 \n",
       "L 139.921892 179.66747 \n",
       "L 139.921892 178.575904 \n",
       "L 139.921892 177.484337 \n",
       "L 139.921892 176.392771 \n",
       "L 139.921892 175.301205 \n",
       "L 139.921892 174.209639 \n",
       "L 139.921892 173.118072 \n",
       "L 140.762253 172.299398 \n",
       "L 141.042374 172.026506 \n",
       "L 141.042374 170.93494 \n",
       "L 141.042374 169.843373 \n",
       "L 141.042374 168.751807 \n",
       "L 141.042374 167.660241 \n",
       "L 141.042374 166.568675 \n",
       "L 141.042374 165.477108 \n",
       "L 141.042374 164.385542 \n",
       "L 141.042374 163.293976 \n",
       "L 141.882735 162.475301 \n",
       "L 142.162856 162.20241 \n",
       "L 142.162856 161.110843 \n",
       "L 142.162856 160.019277 \n",
       "L 142.162856 158.927711 \n",
       "L 142.162856 157.836145 \n",
       "L 142.162856 156.744578 \n",
       "L 142.162856 155.653012 \n",
       "L 142.162856 154.561446 \n",
       "L 142.162856 153.46988 \n",
       "L 143.003217 152.651205 \n",
       "L 143.283338 152.378313 \n",
       "L 143.283338 151.286747 \n",
       "L 143.283338 150.195181 \n",
       "L 143.283338 149.103614 \n",
       "L 143.283338 148.012048 \n",
       "L 143.283338 146.920482 \n",
       "L 143.283338 145.828916 \n",
       "L 143.283338 144.737349 \n",
       "L 143.283338 143.645783 \n",
       "L 144.123699 142.827108 \n",
       "L 144.40382 142.554217 \n",
       "L 144.40382 141.462651 \n",
       "L 144.40382 140.371084 \n",
       "L 144.40382 139.279518 \n",
       "L 144.40382 138.187952 \n",
       "L 144.40382 137.096386 \n",
       "L 144.40382 136.004819 \n",
       "L 144.40382 134.913253 \n",
       "L 144.40382 133.821687 \n",
       "L 144.263759 132.73012 \n",
       "L 144.123699 132.593675 \n",
       "L 143.143277 131.638554 \n",
       "L 143.003217 131.502108 \n",
       "L 142.022796 130.546988 \n",
       "L 141.882735 130.410542 \n",
       "L 140.902314 129.455422 \n",
       "L 140.762253 129.318976 \n",
       "L 139.781832 128.363855 \n",
       "L 139.641771 128.22741 \n",
       "L 138.52129 128.22741 \n",
       "L 137.540868 127.272289 \n",
       "L 137.400808 127.135843 \n",
       "L 136.420386 126.180723 \n",
       "L 136.280326 126.044277 \n",
       "L 135.299904 125.089157 \n",
       "L 135.159844 124.952711 \n",
       "L 134.179422 123.99759 \n",
       "L 134.039362 123.861145 \n",
       "L 133.05894 122.906024 \n",
       "L 132.91888 122.769578 \n",
       "L 131.938458 121.814458 \n",
       "L 131.798398 121.678012 \n",
       "L 130.817976 120.722892 \n",
       "L 130.677916 120.586446 \n",
       "L 129.697494 119.631325 \n",
       "L 129.557434 119.49488 \n",
       "L 128.436952 119.49488 \n",
       "L 127.45653 118.539759 \n",
       "L 127.31647 118.403313 \n",
       "L 126.336049 117.448193 \n",
       "L 126.195988 117.311747 \n",
       "L 125.215567 116.356627 \n",
       "L 125.075506 116.220181 \n",
       "L 124.095085 115.26506 \n",
       "L 123.955024 115.128614 \n",
       "L 122.974603 114.173494 \n",
       "L 122.834543 114.037048 \n",
       "L 121.854121 113.081928 \n",
       "L 121.714061 112.945482 \n",
       "L 120.733639 111.990361 \n",
       "L 120.593579 111.853916 \n",
       "L 119.473097 111.853916 \n",
       "L 118.492675 110.898795 \n",
       "L 118.352615 110.762349 \n",
       "L 117.372193 109.807229 \n",
       "L 117.232133 109.670783 \n",
       "L 116.251711 108.715663 \n",
       "L 116.111651 108.579217 \n",
       "L 115.131229 107.624096 \n",
       "L 114.991169 107.487651 \n",
       "L 114.010747 106.53253 \n",
       "L 113.870687 106.396084 \n",
       "L 112.890265 105.440964 \n",
       "L 112.750205 105.304518 \n",
       "L 111.769784 104.349398 \n",
       "L 111.629723 104.212952 \n",
       "L 110.509241 104.212952 \n",
       "L 109.52882 103.257831 \n",
       "L 109.388759 103.121386 \n",
       "L 108.408338 102.166265 \n",
       "L 108.268277 102.029819 \n",
       "L 107.287856 101.074699 \n",
       "L 107.147796 100.938253 \n",
       "L 106.167374 99.983133 \n",
       "L 106.027314 99.846687 \n",
       "L 105.046892 98.891566 \n",
       "L 104.906832 98.75512 \n",
       "L 103.92641 97.8 \n",
       "L 103.78635 97.663554 \n",
       "L 102.805928 96.708434 \n",
       "L 102.665868 96.571988 \n",
       "L 101.545386 96.571988 \n",
       "L 100.564964 95.616867 \n",
       "L 100.424904 95.480422 \n",
       "L 99.444482 94.525301 \n",
       "L 99.304422 94.388855 \n",
       "L 98.324 93.433735 \n",
       "L 98.18394 93.297289 \n",
       "L 97.203518 92.342169 \n",
       "L 97.063458 92.205723 \n",
       "L 96.083037 91.250602 \n",
       "L 95.942976 91.114157 \n",
       "L 94.962555 90.159036 \n",
       "L 94.822494 90.02259 \n",
       "L 93.842073 89.06747 \n",
       "L 93.702012 88.931024 \n",
       "L 92.58153 88.931024 \n",
       "L 91.601109 87.975904 \n",
       "L 91.461049 87.839458 \n",
       "L 90.480627 86.884337 \n",
       "L 90.340567 86.747892 \n",
       "L 89.360145 85.792771 \n",
       "L 89.220085 85.656325 \n",
       "L 88.239663 84.701205 \n",
       "L 88.099603 84.564759 \n",
       "L 87.119181 83.609639 \n",
       "L 86.979121 83.473193 \n",
       "L 85.998699 82.518072 \n",
       "L 85.858639 82.381627 \n",
       "L 84.878217 81.426506 \n",
       "L 84.738157 81.29006 \n",
       "L 83.617675 81.29006 \n",
       "L 82.637253 80.33494 \n",
       "L 82.497193 80.198494 \n",
       "L 81.516771 79.243373 \n",
       "L 81.376711 79.106928 \n",
       "L 80.39629 78.151807 \n",
       "L 80.256229 78.015361 \n",
       "L 79.275808 77.060241 \n",
       "L 79.135747 76.923795 \n",
       "L 78.155326 75.968675 \n",
       "L 78.015265 75.832229 \n",
       "L 77.034844 74.877108 \n",
       "L 76.894784 74.740663 \n",
       "L 75.914362 73.785542 \n",
       "L 75.774302 73.649096 \n",
       "L 74.65382 73.649096 \n",
       "L 73.673398 72.693976 \n",
       "L 73.533338 72.55753 \n",
       "L 72.552916 71.60241 \n",
       "L 72.412856 71.465964 \n",
       "L 71.432434 70.510843 \n",
       "L 71.292374 70.374398 \n",
       "L 70.311952 69.419277 \n",
       "L 70.171892 69.282831 \n",
       "L 69.19147 68.327711 \n",
       "L 69.05141 68.191265 \n",
       "L 68.070988 67.236145 \n",
       "L 67.930928 67.099699 \n",
       "L 66.950506 66.144578 \n",
       "L 66.810446 66.008133 \n",
       "L 65.689964 66.008133 \n",
       "L 64.709543 65.053012 \n",
       "L 64.569482 64.916566 \n",
       "L 63.589061 63.961446 \n",
       "L 63.449 63.825 \n",
       "L 62.468579 62.86988 \n",
       "L 62.328518 62.733434 \n",
       "L 61.348097 61.778313 \n",
       "L 61.208037 61.641867 \n",
       "L 60.227615 60.686747 \n",
       "L 60.087555 60.550301 \n",
       "L 59.107133 59.595181 \n",
       "L 58.967073 59.458735 \n",
       "L 57.986651 58.503614 \n",
       "L 57.846591 58.367169 \n",
       "L 56.866169 57.412048 \n",
       "L 56.726109 57.275602 \n",
       "L 55.605627 57.275602 \n",
       "L 54.625205 56.320482 \n",
       "L 54.485145 56.184036 \n",
       "L 53.504723 55.228916 \n",
       "L 53.364663 55.09247 \n",
       "L 52.384241 54.137349 \n",
       "L 52.244181 54.000904 \n",
       "L 51.263759 53.045783 \n",
       "L 51.123699 52.909337 \n",
       "L 50.143277 51.954217 \n",
       "L 50.003217 51.817771 \n",
       "L 49.022796 50.862651 \n",
       "L 48.882735 50.726205 \n",
       "L 47.902314 49.771084 \n",
       "L 47.762253 49.634639 \n",
       "L 46.641771 49.634639 \n",
       "L 45.66135 48.679518 \n",
       "L 45.52129 48.543072 \n",
       "L 44.540868 47.587952 \n",
       "L 44.400808 47.451506 \n",
       "L 43.420386 46.496386 \n",
       "L 43.280326 46.35994 \n",
       "L 42.299904 45.404819 \n",
       "L 42.159844 45.268373 \n",
       "L 42.159844 45.404819 \n",
       "L 42.159844 46.496386 \n",
       "L 42.159844 47.587952 \n",
       "L 42.159844 48.679518 \n",
       "L 42.159844 49.771084 \n",
       "L 42.159844 50.862651 \n",
       "L 42.159844 51.954217 \n",
       "L 42.159844 53.045783 \n",
       "L 42.159844 54.137349 \n",
       "L 42.159844 55.228916 \n",
       "L 42.159844 56.320482 \n",
       "L 42.159844 57.412048 \n",
       "L 42.159844 58.503614 \n",
       "L 42.159844 59.595181 \n",
       "L 42.159844 60.686747 \n",
       "L 42.159844 61.778313 \n",
       "L 42.159844 62.86988 \n",
       "L 42.159844 63.961446 \n",
       "L 42.159844 65.053012 \n",
       "L 42.159844 66.144578 \n",
       "L 42.159844 67.236145 \n",
       "L 42.159844 68.327711 \n",
       "L 42.159844 69.419277 \n",
       "L 42.159844 70.510843 \n",
       "L 42.159844 71.60241 \n",
       "L 42.159844 72.693976 \n",
       "L 42.159844 73.785542 \n",
       "L 42.159844 74.877108 \n",
       "L 42.159844 75.968675 \n",
       "L 42.159844 77.060241 \n",
       "L 42.159844 78.151807 \n",
       "L 42.159844 79.243373 \n",
       "L 42.159844 80.33494 \n",
       "L 42.159844 81.426506 \n",
       "L 42.159844 82.518072 \n",
       "L 42.159844 83.609639 \n",
       "L 42.159844 84.701205 \n",
       "L 42.159844 85.792771 \n",
       "L 42.159844 86.884337 \n",
       "L 42.159844 87.975904 \n",
       "L 42.159844 89.06747 \n",
       "L 42.159844 90.159036 \n",
       "L 42.159844 91.250602 \n",
       "L 42.159844 92.342169 \n",
       "L 42.159844 93.433735 \n",
       "L 42.159844 94.525301 \n",
       "L 42.159844 95.616867 \n",
       "L 42.159844 96.708434 \n",
       "L 42.159844 97.8 \n",
       "L 42.159844 98.891566 \n",
       "L 42.159844 99.983133 \n",
       "L 42.159844 101.074699 \n",
       "L 42.159844 102.166265 \n",
       "L 42.159844 103.257831 \n",
       "L 42.159844 104.349398 \n",
       "L 42.159844 105.440964 \n",
       "L 42.159844 106.53253 \n",
       "L 42.159844 107.624096 \n",
       "L 42.159844 108.715663 \n",
       "L 42.159844 109.807229 \n",
       "L 42.159844 110.898795 \n",
       "L 42.159844 111.990361 \n",
       "L 42.159844 113.081928 \n",
       "L 42.159844 114.173494 \n",
       "L 42.159844 115.26506 \n",
       "L 42.159844 116.356627 \n",
       "L 42.159844 117.448193 \n",
       "L 42.159844 118.539759 \n",
       "L 42.159844 119.631325 \n",
       "L 42.159844 120.722892 \n",
       "L 42.159844 121.814458 \n",
       "L 42.159844 122.906024 \n",
       "L 42.159844 123.99759 \n",
       "L 42.159844 125.089157 \n",
       "L 42.159844 126.180723 \n",
       "L 42.159844 127.272289 \n",
       "L 42.159844 128.363855 \n",
       "L 42.159844 129.455422 \n",
       "L 42.159844 130.546988 \n",
       "L 42.159844 131.638554 \n",
       "L 42.159844 132.73012 \n",
       "L 42.159844 133.821687 \n",
       "L 42.159844 134.913253 \n",
       "L 42.159844 136.004819 \n",
       "L 42.159844 137.096386 \n",
       "L 42.159844 138.187952 \n",
       "L 42.159844 139.279518 \n",
       "L 42.159844 140.371084 \n",
       "L 42.159844 141.462651 \n",
       "L 42.159844 142.554217 \n",
       "L 42.159844 143.645783 \n",
       "L 42.159844 144.737349 \n",
       "L 42.159844 145.828916 \n",
       "L 42.159844 146.920482 \n",
       "L 42.159844 148.012048 \n",
       "L 42.159844 149.103614 \n",
       "L 42.159844 150.195181 \n",
       "L 42.159844 151.286747 \n",
       "L 42.159844 152.378313 \n",
       "L 42.159844 153.46988 \n",
       "L 42.159844 154.561446 \n",
       "L 42.159844 155.653012 \n",
       "L 42.159844 156.744578 \n",
       "L 42.159844 157.836145 \n",
       "L 42.159844 158.927711 \n",
       "L 42.159844 160.019277 \n",
       "L 42.159844 161.110843 \n",
       "L 42.159844 162.20241 \n",
       "L 42.159844 163.293976 \n",
       "L 42.159844 164.385542 \n",
       "L 42.159844 165.477108 \n",
       "L 42.159844 166.568675 \n",
       "L 42.159844 167.660241 \n",
       "L 42.159844 168.751807 \n",
       "L 42.159844 169.843373 \n",
       "L 42.159844 170.93494 \n",
       "L 42.159844 172.026506 \n",
       "L 42.159844 173.118072 \n",
       "L 42.159844 174.209639 \n",
       "L 42.159844 175.301205 \n",
       "L 42.159844 176.392771 \n",
       "L 42.159844 177.484337 \n",
       "L 42.159844 178.575904 \n",
       "L 42.159844 179.66747 \n",
       "L 42.159844 180.759036 \n",
       "L 42.159844 181.850602 \n",
       "L 42.159844 182.942169 \n",
       "L 42.159844 184.033735 \n",
       "L 42.159844 185.125301 \n",
       "L 42.159844 186.216867 \n",
       "L 42.159844 187.308434 \n",
       "L 42.159844 188.4 \n",
       "L 42.159844 189.491566 \n",
       "L 42.159844 190.583133 \n",
       "L 42.159844 191.674699 \n",
       "L 42.159844 192.766265 \n",
       "L 42.159844 193.857831 \n",
       "L 42.159844 194.949398 \n",
       "L 42.159844 196.040964 \n",
       "L 42.159844 197.13253 \n",
       "L 42.159844 198.224096 \n",
       "L 42.159844 199.315663 \n",
       "L 42.159844 200.407229 \n",
       "L 42.159844 201.498795 \n",
       "L 42.159844 202.590361 \n",
       "L 42.159844 203.681928 \n",
       "L 42.159844 204.773494 \n",
       "L 42.159844 205.86506 \n",
       "L 42.159844 206.956627 \n",
       "L 42.159844 208.048193 \n",
       "L 42.159844 209.139759 \n",
       "L 42.159844 210.231325 \n",
       "L 42.159844 211.322892 \n",
       "L 42.159844 212.414458 \n",
       "L 42.159844 213.506024 \n",
       "L 42.159844 214.59759 \n",
       "L 42.159844 215.689157 \n",
       "L 42.159844 216.780723 \n",
       "L 42.159844 217.872289 \n",
       "L 42.159844 218.963855 \n",
       "L 42.159844 220.055422 \n",
       "L 42.159844 221.146988 \n",
       "L 42.159844 222.238554 \n",
       "L 42.159844 223.33012 \n",
       "L 42.159844 224.421687 \n",
       "L 42.159844 225.513253 \n",
       "L 42.159844 226.604819 \n",
       "L 42.159844 227.696386 \n",
       "L 42.159844 228.787952 \n",
       "L 42.159844 229.879518 \n",
       "L 42.159844 230.971084 \n",
       "L 42.159844 232.062651 \n",
       "L 42.159844 233.154217 \n",
       "L 42.159844 234.245783 \n",
       "L 42.159844 235.337349 \n",
       "L 42.159844 236.428916 \n",
       "L 42.159844 237.520482 \n",
       "L 42.159844 238.612048 \n",
       "L 42.159844 239.703614 \n",
       "L 42.159844 240.795181 \n",
       "L 42.159844 241.886747 \n",
       "L 42.159844 242.978313 \n",
       "L 42.159844 244.06988 \n",
       "L 42.159844 245.161446 \n",
       "L 42.159844 246.253012 \n",
       "L 42.159844 247.344578 \n",
       "L 42.159844 248.436145 \n",
       "L 42.159844 249.527711 \n",
       "L 42.159844 250.619277 \n",
       "L 42.159844 251.710843 \n",
       "L 42.159844 252.80241 \n",
       "L 42.159844 253.893976 \n",
       "L 42.159844 254.985542 \n",
       "L 42.159844 256.077108 \n",
       "L 42.159844 257.168675 \n",
       "L 42.159844 258.260241 \n",
       "L 42.159844 259.351807 \n",
       "L 42.159844 260.443373 \n",
       "L 42.159844 261.53494 \n",
       "L 42.159844 262.626506 \n",
       "L 42.159844 263.718072 \n",
       "L 42.159844 264.809639 \n",
       "L 42.159844 265.901205 \n",
       "L 42.159844 266.992771 \n",
       "L 42.159844 268.084337 \n",
       "L 42.159844 269.175904 \n",
       "L 42.159844 270.26747 \n",
       "L 42.159844 271.359036 \n",
       "L 42.159844 272.450602 \n",
       "L 42.159844 273.542169 \n",
       "L 42.159844 274.633735 \n",
       "L 42.159844 275.725301 \n",
       "L 42.159844 276.816867 \n",
       "L 42.159844 277.908434 \n",
       "L 42.159844 279 \n",
       "z\n",
       "\" style=\"fill:#1f77b4;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_2\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 128.436952 278.454217 \n",
       "L 128.997193 277.908434 \n",
       "L 128.997193 276.816867 \n",
       "L 128.997193 275.725301 \n",
       "L 128.997193 274.633735 \n",
       "L 128.997193 273.542169 \n",
       "L 128.997193 272.450602 \n",
       "L 128.997193 271.359036 \n",
       "L 128.997193 270.26747 \n",
       "L 128.997193 269.175904 \n",
       "L 129.557434 268.63012 \n",
       "L 130.117675 268.084337 \n",
       "L 130.117675 266.992771 \n",
       "L 130.117675 265.901205 \n",
       "L 130.117675 264.809639 \n",
       "L 130.117675 263.718072 \n",
       "L 130.117675 262.626506 \n",
       "L 130.117675 261.53494 \n",
       "L 130.117675 260.443373 \n",
       "L 130.677916 259.89759 \n",
       "L 131.238157 259.351807 \n",
       "L 131.238157 258.260241 \n",
       "L 131.238157 257.168675 \n",
       "L 131.238157 256.077108 \n",
       "L 131.238157 254.985542 \n",
       "L 131.238157 253.893976 \n",
       "L 131.238157 252.80241 \n",
       "L 131.238157 251.710843 \n",
       "L 131.238157 250.619277 \n",
       "L 131.798398 250.073494 \n",
       "L 132.358639 249.527711 \n",
       "L 132.358639 248.436145 \n",
       "L 132.358639 247.344578 \n",
       "L 132.358639 246.253012 \n",
       "L 132.358639 245.161446 \n",
       "L 132.358639 244.06988 \n",
       "L 132.358639 242.978313 \n",
       "L 132.358639 241.886747 \n",
       "L 132.358639 240.795181 \n",
       "L 132.91888 240.249398 \n",
       "L 133.479121 239.703614 \n",
       "L 133.479121 238.612048 \n",
       "L 133.479121 237.520482 \n",
       "L 133.479121 236.428916 \n",
       "L 133.479121 235.337349 \n",
       "L 133.479121 234.245783 \n",
       "L 133.479121 233.154217 \n",
       "L 133.479121 232.062651 \n",
       "L 133.479121 230.971084 \n",
       "L 134.039362 230.425301 \n",
       "L 134.599603 229.879518 \n",
       "L 134.599603 228.787952 \n",
       "L 134.599603 227.696386 \n",
       "L 134.599603 226.604819 \n",
       "L 134.599603 225.513253 \n",
       "L 134.599603 224.421687 \n",
       "L 134.599603 223.33012 \n",
       "L 134.599603 222.238554 \n",
       "L 134.599603 221.146988 \n",
       "L 135.159844 220.601205 \n",
       "L 135.720085 220.055422 \n",
       "L 135.720085 218.963855 \n",
       "L 135.720085 217.872289 \n",
       "L 135.720085 216.780723 \n",
       "L 135.720085 215.689157 \n",
       "L 135.720085 214.59759 \n",
       "L 135.720085 213.506024 \n",
       "L 135.720085 212.414458 \n",
       "L 135.720085 211.322892 \n",
       "L 136.280326 210.777108 \n",
       "L 136.840567 210.231325 \n",
       "L 136.840567 209.139759 \n",
       "L 136.840567 208.048193 \n",
       "L 136.840567 206.956627 \n",
       "L 136.840567 205.86506 \n",
       "L 136.840567 204.773494 \n",
       "L 136.840567 203.681928 \n",
       "L 136.840567 202.590361 \n",
       "L 136.840567 201.498795 \n",
       "L 137.400808 200.953012 \n",
       "L 137.961049 200.407229 \n",
       "L 137.961049 199.315663 \n",
       "L 137.961049 198.224096 \n",
       "L 137.961049 197.13253 \n",
       "L 137.961049 196.040964 \n",
       "L 137.961049 194.949398 \n",
       "L 137.961049 193.857831 \n",
       "L 137.961049 192.766265 \n",
       "L 138.52129 192.220482 \n",
       "L 139.08153 191.674699 \n",
       "L 139.08153 190.583133 \n",
       "L 139.08153 189.491566 \n",
       "L 139.08153 188.4 \n",
       "L 139.08153 187.308434 \n",
       "L 139.08153 186.216867 \n",
       "L 139.08153 185.125301 \n",
       "L 139.08153 184.033735 \n",
       "L 139.08153 182.942169 \n",
       "L 139.641771 182.396386 \n",
       "L 140.202012 181.850602 \n",
       "L 140.202012 180.759036 \n",
       "L 140.202012 179.66747 \n",
       "L 140.202012 178.575904 \n",
       "L 140.202012 177.484337 \n",
       "L 140.202012 176.392771 \n",
       "L 140.202012 175.301205 \n",
       "L 140.202012 174.209639 \n",
       "L 140.202012 173.118072 \n",
       "L 140.762253 172.572289 \n",
       "L 141.322494 172.026506 \n",
       "L 141.322494 170.93494 \n",
       "L 141.322494 169.843373 \n",
       "L 141.322494 168.751807 \n",
       "L 141.322494 167.660241 \n",
       "L 141.322494 166.568675 \n",
       "L 141.322494 165.477108 \n",
       "L 141.322494 164.385542 \n",
       "L 141.322494 163.293976 \n",
       "L 141.882735 162.748193 \n",
       "L 142.442976 162.20241 \n",
       "L 142.442976 161.110843 \n",
       "L 142.442976 160.019277 \n",
       "L 142.442976 158.927711 \n",
       "L 142.442976 157.836145 \n",
       "L 142.442976 156.744578 \n",
       "L 142.442976 155.653012 \n",
       "L 142.442976 154.561446 \n",
       "L 142.442976 153.46988 \n",
       "L 143.003217 152.924096 \n",
       "L 143.563458 152.378313 \n",
       "L 143.563458 151.286747 \n",
       "L 143.563458 150.195181 \n",
       "L 143.563458 149.103614 \n",
       "L 143.563458 148.012048 \n",
       "L 143.563458 146.920482 \n",
       "L 143.563458 145.828916 \n",
       "L 143.563458 144.737349 \n",
       "L 143.563458 143.645783 \n",
       "L 144.123699 143.1 \n",
       "L 144.68394 142.554217 \n",
       "L 144.68394 141.462651 \n",
       "L 144.68394 140.371084 \n",
       "L 144.68394 139.279518 \n",
       "L 144.68394 138.187952 \n",
       "L 144.68394 137.096386 \n",
       "L 144.68394 136.004819 \n",
       "L 144.68394 134.913253 \n",
       "L 144.68394 133.821687 \n",
       "L 144.40382 132.73012 \n",
       "L 144.123699 132.457229 \n",
       "L 143.283338 131.638554 \n",
       "L 143.003217 131.365663 \n",
       "L 142.162856 130.546988 \n",
       "L 141.882735 130.274096 \n",
       "L 141.042374 129.455422 \n",
       "L 140.762253 129.18253 \n",
       "L 139.921892 128.363855 \n",
       "L 139.641771 128.090964 \n",
       "L 138.52129 128.090964 \n",
       "L 137.680928 127.272289 \n",
       "L 137.400808 126.999398 \n",
       "L 136.560446 126.180723 \n",
       "L 136.280326 125.907831 \n",
       "L 135.439964 125.089157 \n",
       "L 135.159844 124.816265 \n",
       "L 134.319482 123.99759 \n",
       "L 134.039362 123.724699 \n",
       "L 133.199 122.906024 \n",
       "L 132.91888 122.633133 \n",
       "L 132.078518 121.814458 \n",
       "L 131.798398 121.541566 \n",
       "L 130.958037 120.722892 \n",
       "L 130.677916 120.45 \n",
       "L 129.837555 119.631325 \n",
       "L 129.557434 119.358434 \n",
       "L 128.436952 119.358434 \n",
       "L 127.596591 118.539759 \n",
       "L 127.31647 118.266867 \n",
       "L 126.476109 117.448193 \n",
       "L 126.195988 117.175301 \n",
       "L 125.355627 116.356627 \n",
       "L 125.075506 116.083735 \n",
       "L 124.235145 115.26506 \n",
       "L 123.955024 114.992169 \n",
       "L 123.114663 114.173494 \n",
       "L 122.834543 113.900602 \n",
       "L 121.994181 113.081928 \n",
       "L 121.714061 112.809036 \n",
       "L 120.873699 111.990361 \n",
       "L 120.593579 111.71747 \n",
       "L 119.473097 111.71747 \n",
       "L 118.632735 110.898795 \n",
       "L 118.352615 110.625904 \n",
       "L 117.512253 109.807229 \n",
       "L 117.232133 109.534337 \n",
       "L 116.391771 108.715663 \n",
       "L 116.111651 108.442771 \n",
       "L 115.27129 107.624096 \n",
       "L 114.991169 107.351205 \n",
       "L 114.150808 106.53253 \n",
       "L 113.870687 106.259639 \n",
       "L 113.030326 105.440964 \n",
       "L 112.750205 105.168072 \n",
       "L 111.909844 104.349398 \n",
       "L 111.629723 104.076506 \n",
       "L 110.509241 104.076506 \n",
       "L 109.66888 103.257831 \n",
       "L 109.388759 102.98494 \n",
       "L 108.548398 102.166265 \n",
       "L 108.268277 101.893373 \n",
       "L 107.427916 101.074699 \n",
       "L 107.147796 100.801807 \n",
       "L 106.307434 99.983133 \n",
       "L 106.027314 99.710241 \n",
       "L 105.186952 98.891566 \n",
       "L 104.906832 98.618675 \n",
       "L 104.06647 97.8 \n",
       "L 103.78635 97.527108 \n",
       "L 102.945988 96.708434 \n",
       "L 102.665868 96.435542 \n",
       "L 101.545386 96.435542 \n",
       "L 100.705024 95.616867 \n",
       "L 100.424904 95.343976 \n",
       "L 99.584543 94.525301 \n",
       "L 99.304422 94.25241 \n",
       "L 98.464061 93.433735 \n",
       "L 98.18394 93.160843 \n",
       "L 97.343579 92.342169 \n",
       "L 97.063458 92.069277 \n",
       "L 96.223097 91.250602 \n",
       "L 95.942976 90.977711 \n",
       "L 95.102615 90.159036 \n",
       "L 94.822494 89.886145 \n",
       "L 93.982133 89.06747 \n",
       "L 93.702012 88.794578 \n",
       "L 92.58153 88.794578 \n",
       "L 91.741169 87.975904 \n",
       "L 91.461049 87.703012 \n",
       "L 90.620687 86.884337 \n",
       "L 90.340567 86.611446 \n",
       "L 89.500205 85.792771 \n",
       "L 89.220085 85.51988 \n",
       "L 88.379723 84.701205 \n",
       "L 88.099603 84.428313 \n",
       "L 87.259241 83.609639 \n",
       "L 86.979121 83.336747 \n",
       "L 86.138759 82.518072 \n",
       "L 85.858639 82.245181 \n",
       "L 85.018277 81.426506 \n",
       "L 84.738157 81.153614 \n",
       "L 83.617675 81.153614 \n",
       "L 82.777314 80.33494 \n",
       "L 82.497193 80.062048 \n",
       "L 81.656832 79.243373 \n",
       "L 81.376711 78.970482 \n",
       "L 80.53635 78.151807 \n",
       "L 80.256229 77.878916 \n",
       "L 79.415868 77.060241 \n",
       "L 79.135747 76.787349 \n",
       "L 78.295386 75.968675 \n",
       "L 78.015265 75.695783 \n",
       "L 77.174904 74.877108 \n",
       "L 76.894784 74.604217 \n",
       "L 76.054422 73.785542 \n",
       "L 75.774302 73.512651 \n",
       "L 74.65382 73.512651 \n",
       "L 73.813458 72.693976 \n",
       "L 73.533338 72.421084 \n",
       "L 72.692976 71.60241 \n",
       "L 72.412856 71.329518 \n",
       "L 71.572494 70.510843 \n",
       "L 71.292374 70.237952 \n",
       "L 70.452012 69.419277 \n",
       "L 70.171892 69.146386 \n",
       "L 69.33153 68.327711 \n",
       "L 69.05141 68.054819 \n",
       "L 68.211049 67.236145 \n",
       "L 67.930928 66.963253 \n",
       "L 67.090567 66.144578 \n",
       "L 66.810446 65.871687 \n",
       "L 65.689964 65.871687 \n",
       "L 64.849603 65.053012 \n",
       "L 64.569482 64.78012 \n",
       "L 63.729121 63.961446 \n",
       "L 63.449 63.688554 \n",
       "L 62.608639 62.86988 \n",
       "L 62.328518 62.596988 \n",
       "L 61.488157 61.778313 \n",
       "L 61.208037 61.505422 \n",
       "L 60.367675 60.686747 \n",
       "L 60.087555 60.413855 \n",
       "L 59.247193 59.595181 \n",
       "L 58.967073 59.322289 \n",
       "L 58.126711 58.503614 \n",
       "L 57.846591 58.230723 \n",
       "L 57.006229 57.412048 \n",
       "L 56.726109 57.139157 \n",
       "L 55.605627 57.139157 \n",
       "L 54.765265 56.320482 \n",
       "L 54.485145 56.04759 \n",
       "L 53.644784 55.228916 \n",
       "L 53.364663 54.956024 \n",
       "L 52.524302 54.137349 \n",
       "L 52.244181 53.864458 \n",
       "L 51.40382 53.045783 \n",
       "L 51.123699 52.772892 \n",
       "L 50.283338 51.954217 \n",
       "L 50.003217 51.681325 \n",
       "L 49.162856 50.862651 \n",
       "L 48.882735 50.589759 \n",
       "L 48.042374 49.771084 \n",
       "L 47.762253 49.498193 \n",
       "L 46.641771 49.498193 \n",
       "L 45.80141 48.679518 \n",
       "L 45.52129 48.406627 \n",
       "L 44.680928 47.587952 \n",
       "L 44.400808 47.31506 \n",
       "L 43.560446 46.496386 \n",
       "L 43.280326 46.223494 \n",
       "L 42.439964 45.404819 \n",
       "L 42.159844 45.131928 \n",
       "L 42.159844 45.268373 \n",
       "L 42.299904 45.404819 \n",
       "L 43.280326 46.35994 \n",
       "L 43.420386 46.496386 \n",
       "L 44.400808 47.451506 \n",
       "L 44.540868 47.587952 \n",
       "L 45.52129 48.543072 \n",
       "L 45.66135 48.679518 \n",
       "L 46.641771 49.634639 \n",
       "L 47.762253 49.634639 \n",
       "L 47.902314 49.771084 \n",
       "L 48.882735 50.726205 \n",
       "L 49.022796 50.862651 \n",
       "L 50.003217 51.817771 \n",
       "L 50.143277 51.954217 \n",
       "L 51.123699 52.909337 \n",
       "L 51.263759 53.045783 \n",
       "L 52.244181 54.000904 \n",
       "L 52.384241 54.137349 \n",
       "L 53.364663 55.09247 \n",
       "L 53.504723 55.228916 \n",
       "L 54.485145 56.184036 \n",
       "L 54.625205 56.320482 \n",
       "L 55.605627 57.275602 \n",
       "L 56.726109 57.275602 \n",
       "L 56.866169 57.412048 \n",
       "L 57.846591 58.367169 \n",
       "L 57.986651 58.503614 \n",
       "L 58.967073 59.458735 \n",
       "L 59.107133 59.595181 \n",
       "L 60.087555 60.550301 \n",
       "L 60.227615 60.686747 \n",
       "L 61.208037 61.641867 \n",
       "L 61.348097 61.778313 \n",
       "L 62.328518 62.733434 \n",
       "L 62.468579 62.86988 \n",
       "L 63.449 63.825 \n",
       "L 63.589061 63.961446 \n",
       "L 64.569482 64.916566 \n",
       "L 64.709543 65.053012 \n",
       "L 65.689964 66.008133 \n",
       "L 66.810446 66.008133 \n",
       "L 66.950506 66.144578 \n",
       "L 67.930928 67.099699 \n",
       "L 68.070988 67.236145 \n",
       "L 69.05141 68.191265 \n",
       "L 69.19147 68.327711 \n",
       "L 70.171892 69.282831 \n",
       "L 70.311952 69.419277 \n",
       "L 71.292374 70.374398 \n",
       "L 71.432434 70.510843 \n",
       "L 72.412856 71.465964 \n",
       "L 72.552916 71.60241 \n",
       "L 73.533338 72.55753 \n",
       "L 73.673398 72.693976 \n",
       "L 74.65382 73.649096 \n",
       "L 75.774302 73.649096 \n",
       "L 75.914362 73.785542 \n",
       "L 76.894784 74.740663 \n",
       "L 77.034844 74.877108 \n",
       "L 78.015265 75.832229 \n",
       "L 78.155326 75.968675 \n",
       "L 79.135747 76.923795 \n",
       "L 79.275808 77.060241 \n",
       "L 80.256229 78.015361 \n",
       "L 80.39629 78.151807 \n",
       "L 81.376711 79.106928 \n",
       "L 81.516771 79.243373 \n",
       "L 82.497193 80.198494 \n",
       "L 82.637253 80.33494 \n",
       "L 83.617675 81.29006 \n",
       "L 84.738157 81.29006 \n",
       "L 84.878217 81.426506 \n",
       "L 85.858639 82.381627 \n",
       "L 85.998699 82.518072 \n",
       "L 86.979121 83.473193 \n",
       "L 87.119181 83.609639 \n",
       "L 88.099603 84.564759 \n",
       "L 88.239663 84.701205 \n",
       "L 89.220085 85.656325 \n",
       "L 89.360145 85.792771 \n",
       "L 90.340567 86.747892 \n",
       "L 90.480627 86.884337 \n",
       "L 91.461049 87.839458 \n",
       "L 91.601109 87.975904 \n",
       "L 92.58153 88.931024 \n",
       "L 93.702012 88.931024 \n",
       "L 93.842073 89.06747 \n",
       "L 94.822494 90.02259 \n",
       "L 94.962555 90.159036 \n",
       "L 95.942976 91.114157 \n",
       "L 96.083037 91.250602 \n",
       "L 97.063458 92.205723 \n",
       "L 97.203518 92.342169 \n",
       "L 98.18394 93.297289 \n",
       "L 98.324 93.433735 \n",
       "L 99.304422 94.388855 \n",
       "L 99.444482 94.525301 \n",
       "L 100.424904 95.480422 \n",
       "L 100.564964 95.616867 \n",
       "L 101.545386 96.571988 \n",
       "L 102.665868 96.571988 \n",
       "L 102.805928 96.708434 \n",
       "L 103.78635 97.663554 \n",
       "L 103.92641 97.8 \n",
       "L 104.906832 98.75512 \n",
       "L 105.046892 98.891566 \n",
       "L 106.027314 99.846687 \n",
       "L 106.167374 99.983133 \n",
       "L 107.147796 100.938253 \n",
       "L 107.287856 101.074699 \n",
       "L 108.268277 102.029819 \n",
       "L 108.408338 102.166265 \n",
       "L 109.388759 103.121386 \n",
       "L 109.52882 103.257831 \n",
       "L 110.509241 104.212952 \n",
       "L 111.629723 104.212952 \n",
       "L 111.769784 104.349398 \n",
       "L 112.750205 105.304518 \n",
       "L 112.890265 105.440964 \n",
       "L 113.870687 106.396084 \n",
       "L 114.010747 106.53253 \n",
       "L 114.991169 107.487651 \n",
       "L 115.131229 107.624096 \n",
       "L 116.111651 108.579217 \n",
       "L 116.251711 108.715663 \n",
       "L 117.232133 109.670783 \n",
       "L 117.372193 109.807229 \n",
       "L 118.352615 110.762349 \n",
       "L 118.492675 110.898795 \n",
       "L 119.473097 111.853916 \n",
       "L 120.593579 111.853916 \n",
       "L 120.733639 111.990361 \n",
       "L 121.714061 112.945482 \n",
       "L 121.854121 113.081928 \n",
       "L 122.834543 114.037048 \n",
       "L 122.974603 114.173494 \n",
       "L 123.955024 115.128614 \n",
       "L 124.095085 115.26506 \n",
       "L 125.075506 116.220181 \n",
       "L 125.215567 116.356627 \n",
       "L 126.195988 117.311747 \n",
       "L 126.336049 117.448193 \n",
       "L 127.31647 118.403313 \n",
       "L 127.45653 118.539759 \n",
       "L 128.436952 119.49488 \n",
       "L 129.557434 119.49488 \n",
       "L 129.697494 119.631325 \n",
       "L 130.677916 120.586446 \n",
       "L 130.817976 120.722892 \n",
       "L 131.798398 121.678012 \n",
       "L 131.938458 121.814458 \n",
       "L 132.91888 122.769578 \n",
       "L 133.05894 122.906024 \n",
       "L 134.039362 123.861145 \n",
       "L 134.179422 123.99759 \n",
       "L 135.159844 124.952711 \n",
       "L 135.299904 125.089157 \n",
       "L 136.280326 126.044277 \n",
       "L 136.420386 126.180723 \n",
       "L 137.400808 127.135843 \n",
       "L 137.540868 127.272289 \n",
       "L 138.52129 128.22741 \n",
       "L 139.641771 128.22741 \n",
       "L 139.781832 128.363855 \n",
       "L 140.762253 129.318976 \n",
       "L 140.902314 129.455422 \n",
       "L 141.882735 130.410542 \n",
       "L 142.022796 130.546988 \n",
       "L 143.003217 131.502108 \n",
       "L 143.143277 131.638554 \n",
       "L 144.123699 132.593675 \n",
       "L 144.263759 132.73012 \n",
       "L 144.40382 133.821687 \n",
       "L 144.40382 134.913253 \n",
       "L 144.40382 136.004819 \n",
       "L 144.40382 137.096386 \n",
       "L 144.40382 138.187952 \n",
       "L 144.40382 139.279518 \n",
       "L 144.40382 140.371084 \n",
       "L 144.40382 141.462651 \n",
       "L 144.40382 142.554217 \n",
       "L 144.123699 142.827108 \n",
       "L 143.283338 143.645783 \n",
       "L 143.283338 144.737349 \n",
       "L 143.283338 145.828916 \n",
       "L 143.283338 146.920482 \n",
       "L 143.283338 148.012048 \n",
       "L 143.283338 149.103614 \n",
       "L 143.283338 150.195181 \n",
       "L 143.283338 151.286747 \n",
       "L 143.283338 152.378313 \n",
       "L 143.003217 152.651205 \n",
       "L 142.162856 153.46988 \n",
       "L 142.162856 154.561446 \n",
       "L 142.162856 155.653012 \n",
       "L 142.162856 156.744578 \n",
       "L 142.162856 157.836145 \n",
       "L 142.162856 158.927711 \n",
       "L 142.162856 160.019277 \n",
       "L 142.162856 161.110843 \n",
       "L 142.162856 162.20241 \n",
       "L 141.882735 162.475301 \n",
       "L 141.042374 163.293976 \n",
       "L 141.042374 164.385542 \n",
       "L 141.042374 165.477108 \n",
       "L 141.042374 166.568675 \n",
       "L 141.042374 167.660241 \n",
       "L 141.042374 168.751807 \n",
       "L 141.042374 169.843373 \n",
       "L 141.042374 170.93494 \n",
       "L 141.042374 172.026506 \n",
       "L 140.762253 172.299398 \n",
       "L 139.921892 173.118072 \n",
       "L 139.921892 174.209639 \n",
       "L 139.921892 175.301205 \n",
       "L 139.921892 176.392771 \n",
       "L 139.921892 177.484337 \n",
       "L 139.921892 178.575904 \n",
       "L 139.921892 179.66747 \n",
       "L 139.921892 180.759036 \n",
       "L 139.921892 181.850602 \n",
       "L 139.641771 182.123494 \n",
       "L 138.80141 182.942169 \n",
       "L 138.80141 184.033735 \n",
       "L 138.80141 185.125301 \n",
       "L 138.80141 186.216867 \n",
       "L 138.80141 187.308434 \n",
       "L 138.80141 188.4 \n",
       "L 138.80141 189.491566 \n",
       "L 138.80141 190.583133 \n",
       "L 138.80141 191.674699 \n",
       "L 138.52129 191.94759 \n",
       "L 137.680928 192.766265 \n",
       "L 137.680928 193.857831 \n",
       "L 137.680928 194.949398 \n",
       "L 137.680928 196.040964 \n",
       "L 137.680928 197.13253 \n",
       "L 137.680928 198.224096 \n",
       "L 137.680928 199.315663 \n",
       "L 137.680928 200.407229 \n",
       "L 137.400808 200.68012 \n",
       "L 136.560446 201.498795 \n",
       "L 136.560446 202.590361 \n",
       "L 136.560446 203.681928 \n",
       "L 136.560446 204.773494 \n",
       "L 136.560446 205.86506 \n",
       "L 136.560446 206.956627 \n",
       "L 136.560446 208.048193 \n",
       "L 136.560446 209.139759 \n",
       "L 136.560446 210.231325 \n",
       "L 136.280326 210.504217 \n",
       "L 135.439964 211.322892 \n",
       "L 135.439964 212.414458 \n",
       "L 135.439964 213.506024 \n",
       "L 135.439964 214.59759 \n",
       "L 135.439964 215.689157 \n",
       "L 135.439964 216.780723 \n",
       "L 135.439964 217.872289 \n",
       "L 135.439964 218.963855 \n",
       "L 135.439964 220.055422 \n",
       "L 135.159844 220.328313 \n",
       "L 134.319482 221.146988 \n",
       "L 134.319482 222.238554 \n",
       "L 134.319482 223.33012 \n",
       "L 134.319482 224.421687 \n",
       "L 134.319482 225.513253 \n",
       "L 134.319482 226.604819 \n",
       "L 134.319482 227.696386 \n",
       "L 134.319482 228.787952 \n",
       "L 134.319482 229.879518 \n",
       "L 134.039362 230.15241 \n",
       "L 133.199 230.971084 \n",
       "L 133.199 232.062651 \n",
       "L 133.199 233.154217 \n",
       "L 133.199 234.245783 \n",
       "L 133.199 235.337349 \n",
       "L 133.199 236.428916 \n",
       "L 133.199 237.520482 \n",
       "L 133.199 238.612048 \n",
       "L 133.199 239.703614 \n",
       "L 132.91888 239.976506 \n",
       "L 132.078518 240.795181 \n",
       "L 132.078518 241.886747 \n",
       "L 132.078518 242.978313 \n",
       "L 132.078518 244.06988 \n",
       "L 132.078518 245.161446 \n",
       "L 132.078518 246.253012 \n",
       "L 132.078518 247.344578 \n",
       "L 132.078518 248.436145 \n",
       "L 132.078518 249.527711 \n",
       "L 131.798398 249.800602 \n",
       "L 130.958037 250.619277 \n",
       "L 130.958037 251.710843 \n",
       "L 130.958037 252.80241 \n",
       "L 130.958037 253.893976 \n",
       "L 130.958037 254.985542 \n",
       "L 130.958037 256.077108 \n",
       "L 130.958037 257.168675 \n",
       "L 130.958037 258.260241 \n",
       "L 130.958037 259.351807 \n",
       "L 130.677916 259.624699 \n",
       "L 129.837555 260.443373 \n",
       "L 129.837555 261.53494 \n",
       "L 129.837555 262.626506 \n",
       "L 129.837555 263.718072 \n",
       "L 129.837555 264.809639 \n",
       "L 129.837555 265.901205 \n",
       "L 129.837555 266.992771 \n",
       "L 129.837555 268.084337 \n",
       "L 129.557434 268.357229 \n",
       "L 128.717073 269.175904 \n",
       "L 128.717073 270.26747 \n",
       "L 128.717073 271.359036 \n",
       "L 128.717073 272.450602 \n",
       "L 128.717073 273.542169 \n",
       "L 128.717073 274.633735 \n",
       "L 128.717073 275.725301 \n",
       "L 128.717073 276.816867 \n",
       "L 128.717073 277.908434 \n",
       "L 128.436952 278.181325 \n",
       "L 127.596591 279 \n",
       "L 127.876711 279 \n",
       "z\n",
       "\" style=\"fill:#ff7f0e;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_3\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 128.436952 278.727108 \n",
       "L 129.277314 277.908434 \n",
       "L 129.277314 276.816867 \n",
       "L 129.277314 275.725301 \n",
       "L 129.277314 274.633735 \n",
       "L 129.277314 273.542169 \n",
       "L 129.277314 272.450602 \n",
       "L 129.277314 271.359036 \n",
       "L 129.277314 270.26747 \n",
       "L 129.277314 269.175904 \n",
       "L 129.557434 268.903012 \n",
       "L 130.397796 268.084337 \n",
       "L 130.397796 266.992771 \n",
       "L 130.397796 265.901205 \n",
       "L 130.397796 264.809639 \n",
       "L 130.397796 263.718072 \n",
       "L 130.397796 262.626506 \n",
       "L 130.397796 261.53494 \n",
       "L 130.397796 260.443373 \n",
       "L 130.677916 260.170482 \n",
       "L 131.518277 259.351807 \n",
       "L 131.518277 258.260241 \n",
       "L 131.518277 257.168675 \n",
       "L 131.518277 256.077108 \n",
       "L 131.518277 254.985542 \n",
       "L 131.518277 253.893976 \n",
       "L 131.518277 252.80241 \n",
       "L 131.518277 251.710843 \n",
       "L 131.518277 250.619277 \n",
       "L 131.798398 250.346386 \n",
       "L 132.638759 249.527711 \n",
       "L 132.638759 248.436145 \n",
       "L 132.638759 247.344578 \n",
       "L 132.638759 246.253012 \n",
       "L 132.638759 245.161446 \n",
       "L 132.638759 244.06988 \n",
       "L 132.638759 242.978313 \n",
       "L 132.638759 241.886747 \n",
       "L 132.638759 240.795181 \n",
       "L 132.91888 240.522289 \n",
       "L 133.759241 239.703614 \n",
       "L 133.759241 238.612048 \n",
       "L 133.759241 237.520482 \n",
       "L 133.759241 236.428916 \n",
       "L 133.759241 235.337349 \n",
       "L 133.759241 234.245783 \n",
       "L 133.759241 233.154217 \n",
       "L 133.759241 232.062651 \n",
       "L 133.759241 230.971084 \n",
       "L 134.039362 230.698193 \n",
       "L 134.879723 229.879518 \n",
       "L 134.879723 228.787952 \n",
       "L 134.879723 227.696386 \n",
       "L 134.879723 226.604819 \n",
       "L 134.879723 225.513253 \n",
       "L 134.879723 224.421687 \n",
       "L 134.879723 223.33012 \n",
       "L 134.879723 222.238554 \n",
       "L 134.879723 221.146988 \n",
       "L 135.159844 220.874096 \n",
       "L 136.000205 220.055422 \n",
       "L 136.000205 218.963855 \n",
       "L 136.000205 217.872289 \n",
       "L 136.000205 216.780723 \n",
       "L 136.000205 215.689157 \n",
       "L 136.000205 214.59759 \n",
       "L 136.000205 213.506024 \n",
       "L 136.000205 212.414458 \n",
       "L 136.000205 211.322892 \n",
       "L 136.280326 211.05 \n",
       "L 137.120687 210.231325 \n",
       "L 137.120687 209.139759 \n",
       "L 137.120687 208.048193 \n",
       "L 137.120687 206.956627 \n",
       "L 137.120687 205.86506 \n",
       "L 137.120687 204.773494 \n",
       "L 137.120687 203.681928 \n",
       "L 137.120687 202.590361 \n",
       "L 137.120687 201.498795 \n",
       "L 137.400808 201.225904 \n",
       "L 138.241169 200.407229 \n",
       "L 138.241169 199.315663 \n",
       "L 138.241169 198.224096 \n",
       "L 138.241169 197.13253 \n",
       "L 138.241169 196.040964 \n",
       "L 138.241169 194.949398 \n",
       "L 138.241169 193.857831 \n",
       "L 138.241169 192.766265 \n",
       "L 138.52129 192.493373 \n",
       "L 139.361651 191.674699 \n",
       "L 139.361651 190.583133 \n",
       "L 139.361651 189.491566 \n",
       "L 139.361651 188.4 \n",
       "L 139.361651 187.308434 \n",
       "L 139.361651 186.216867 \n",
       "L 139.361651 185.125301 \n",
       "L 139.361651 184.033735 \n",
       "L 139.361651 182.942169 \n",
       "L 139.641771 182.669277 \n",
       "L 140.482133 181.850602 \n",
       "L 140.482133 180.759036 \n",
       "L 140.482133 179.66747 \n",
       "L 140.482133 178.575904 \n",
       "L 140.482133 177.484337 \n",
       "L 140.482133 176.392771 \n",
       "L 140.482133 175.301205 \n",
       "L 140.482133 174.209639 \n",
       "L 140.482133 173.118072 \n",
       "L 140.762253 172.845181 \n",
       "L 141.602615 172.026506 \n",
       "L 141.602615 170.93494 \n",
       "L 141.602615 169.843373 \n",
       "L 141.602615 168.751807 \n",
       "L 141.602615 167.660241 \n",
       "L 141.602615 166.568675 \n",
       "L 141.602615 165.477108 \n",
       "L 141.602615 164.385542 \n",
       "L 141.602615 163.293976 \n",
       "L 141.882735 163.021084 \n",
       "L 142.723097 162.20241 \n",
       "L 142.723097 161.110843 \n",
       "L 142.723097 160.019277 \n",
       "L 142.723097 158.927711 \n",
       "L 142.723097 157.836145 \n",
       "L 142.723097 156.744578 \n",
       "L 142.723097 155.653012 \n",
       "L 142.723097 154.561446 \n",
       "L 142.723097 153.46988 \n",
       "L 143.003217 153.196988 \n",
       "L 143.843579 152.378313 \n",
       "L 143.843579 151.286747 \n",
       "L 143.843579 150.195181 \n",
       "L 143.843579 149.103614 \n",
       "L 143.843579 148.012048 \n",
       "L 143.843579 146.920482 \n",
       "L 143.843579 145.828916 \n",
       "L 143.843579 144.737349 \n",
       "L 143.843579 143.645783 \n",
       "L 144.123699 143.372892 \n",
       "L 144.964061 142.554217 \n",
       "L 144.964061 141.462651 \n",
       "L 144.964061 140.371084 \n",
       "L 144.964061 139.279518 \n",
       "L 144.964061 138.187952 \n",
       "L 144.964061 137.096386 \n",
       "L 144.964061 136.004819 \n",
       "L 144.964061 134.913253 \n",
       "L 144.964061 133.821687 \n",
       "L 144.54388 132.73012 \n",
       "L 144.123699 132.320783 \n",
       "L 143.423398 131.638554 \n",
       "L 143.003217 131.229217 \n",
       "L 142.302916 130.546988 \n",
       "L 141.882735 130.137651 \n",
       "L 141.182434 129.455422 \n",
       "L 140.762253 129.046084 \n",
       "L 140.061952 128.363855 \n",
       "L 139.641771 127.954518 \n",
       "L 138.52129 127.954518 \n",
       "L 137.820988 127.272289 \n",
       "L 137.400808 126.862952 \n",
       "L 136.700506 126.180723 \n",
       "L 136.280326 125.771386 \n",
       "L 135.580024 125.089157 \n",
       "L 135.159844 124.679819 \n",
       "L 134.459543 123.99759 \n",
       "L 134.039362 123.588253 \n",
       "L 133.339061 122.906024 \n",
       "L 132.91888 122.496687 \n",
       "L 132.218579 121.814458 \n",
       "L 131.798398 121.40512 \n",
       "L 131.098097 120.722892 \n",
       "L 130.677916 120.313554 \n",
       "L 129.977615 119.631325 \n",
       "L 129.557434 119.221988 \n",
       "L 128.436952 119.221988 \n",
       "L 127.736651 118.539759 \n",
       "L 127.31647 118.130422 \n",
       "L 126.616169 117.448193 \n",
       "L 126.195988 117.038855 \n",
       "L 125.495687 116.356627 \n",
       "L 125.075506 115.947289 \n",
       "L 124.375205 115.26506 \n",
       "L 123.955024 114.855723 \n",
       "L 123.254723 114.173494 \n",
       "L 122.834543 113.764157 \n",
       "L 122.134241 113.081928 \n",
       "L 121.714061 112.67259 \n",
       "L 121.013759 111.990361 \n",
       "L 120.593579 111.581024 \n",
       "L 119.473097 111.581024 \n",
       "L 118.772796 110.898795 \n",
       "L 118.352615 110.489458 \n",
       "L 117.652314 109.807229 \n",
       "L 117.232133 109.397892 \n",
       "L 116.531832 108.715663 \n",
       "L 116.111651 108.306325 \n",
       "L 115.41135 107.624096 \n",
       "L 114.991169 107.214759 \n",
       "L 114.290868 106.53253 \n",
       "L 113.870687 106.123193 \n",
       "L 113.170386 105.440964 \n",
       "L 112.750205 105.031627 \n",
       "L 112.049904 104.349398 \n",
       "L 111.629723 103.94006 \n",
       "L 110.509241 103.94006 \n",
       "L 109.80894 103.257831 \n",
       "L 109.388759 102.848494 \n",
       "L 108.688458 102.166265 \n",
       "L 108.268277 101.756928 \n",
       "L 107.567976 101.074699 \n",
       "L 107.147796 100.665361 \n",
       "L 106.447494 99.983133 \n",
       "L 106.027314 99.573795 \n",
       "L 105.327012 98.891566 \n",
       "L 104.906832 98.482229 \n",
       "L 104.20653 97.8 \n",
       "L 103.78635 97.390663 \n",
       "L 103.086049 96.708434 \n",
       "L 102.665868 96.299096 \n",
       "L 101.545386 96.299096 \n",
       "L 100.845085 95.616867 \n",
       "L 100.424904 95.20753 \n",
       "L 99.724603 94.525301 \n",
       "L 99.304422 94.115964 \n",
       "L 98.604121 93.433735 \n",
       "L 98.18394 93.024398 \n",
       "L 97.483639 92.342169 \n",
       "L 97.063458 91.932831 \n",
       "L 96.363157 91.250602 \n",
       "L 95.942976 90.841265 \n",
       "L 95.242675 90.159036 \n",
       "L 94.822494 89.749699 \n",
       "L 94.122193 89.06747 \n",
       "L 93.702012 88.658133 \n",
       "L 92.58153 88.658133 \n",
       "L 91.881229 87.975904 \n",
       "L 91.461049 87.566566 \n",
       "L 90.760747 86.884337 \n",
       "L 90.340567 86.475 \n",
       "L 89.640265 85.792771 \n",
       "L 89.220085 85.383434 \n",
       "L 88.519784 84.701205 \n",
       "L 88.099603 84.291867 \n",
       "L 87.399302 83.609639 \n",
       "L 86.979121 83.200301 \n",
       "L 86.27882 82.518072 \n",
       "L 85.858639 82.108735 \n",
       "L 85.158338 81.426506 \n",
       "L 84.738157 81.017169 \n",
       "L 83.617675 81.017169 \n",
       "L 82.917374 80.33494 \n",
       "L 82.497193 79.925602 \n",
       "L 81.796892 79.243373 \n",
       "L 81.376711 78.834036 \n",
       "L 80.67641 78.151807 \n",
       "L 80.256229 77.74247 \n",
       "L 79.555928 77.060241 \n",
       "L 79.135747 76.650904 \n",
       "L 78.435446 75.968675 \n",
       "L 78.015265 75.559337 \n",
       "L 77.314964 74.877108 \n",
       "L 76.894784 74.467771 \n",
       "L 76.194482 73.785542 \n",
       "L 75.774302 73.376205 \n",
       "L 74.65382 73.376205 \n",
       "L 73.953518 72.693976 \n",
       "L 73.533338 72.284639 \n",
       "L 72.833037 71.60241 \n",
       "L 72.412856 71.193072 \n",
       "L 71.712555 70.510843 \n",
       "L 71.292374 70.101506 \n",
       "L 70.592073 69.419277 \n",
       "L 70.171892 69.00994 \n",
       "L 69.471591 68.327711 \n",
       "L 69.05141 67.918373 \n",
       "L 68.351109 67.236145 \n",
       "L 67.930928 66.826807 \n",
       "L 67.230627 66.144578 \n",
       "L 66.810446 65.735241 \n",
       "L 65.689964 65.735241 \n",
       "L 64.989663 65.053012 \n",
       "L 64.569482 64.643675 \n",
       "L 63.869181 63.961446 \n",
       "L 63.449 63.552108 \n",
       "L 62.748699 62.86988 \n",
       "L 62.328518 62.460542 \n",
       "L 61.628217 61.778313 \n",
       "L 61.208037 61.368976 \n",
       "L 60.507735 60.686747 \n",
       "L 60.087555 60.27741 \n",
       "L 59.387253 59.595181 \n",
       "L 58.967073 59.185843 \n",
       "L 58.266771 58.503614 \n",
       "L 57.846591 58.094277 \n",
       "L 57.14629 57.412048 \n",
       "L 56.726109 57.002711 \n",
       "L 55.605627 57.002711 \n",
       "L 54.905326 56.320482 \n",
       "L 54.485145 55.911145 \n",
       "L 53.784844 55.228916 \n",
       "L 53.364663 54.819578 \n",
       "L 52.664362 54.137349 \n",
       "L 52.244181 53.728012 \n",
       "L 51.54388 53.045783 \n",
       "L 51.123699 52.636446 \n",
       "L 50.423398 51.954217 \n",
       "L 50.003217 51.54488 \n",
       "L 49.302916 50.862651 \n",
       "L 48.882735 50.453313 \n",
       "L 48.182434 49.771084 \n",
       "L 47.762253 49.361747 \n",
       "L 46.641771 49.361747 \n",
       "L 45.94147 48.679518 \n",
       "L 45.52129 48.270181 \n",
       "L 44.820988 47.587952 \n",
       "L 44.400808 47.178614 \n",
       "L 43.700506 46.496386 \n",
       "L 43.280326 46.087048 \n",
       "L 42.580024 45.404819 \n",
       "L 42.159844 44.995482 \n",
       "L 42.159844 45.131928 \n",
       "L 42.439964 45.404819 \n",
       "L 43.280326 46.223494 \n",
       "L 43.560446 46.496386 \n",
       "L 44.400808 47.31506 \n",
       "L 44.680928 47.587952 \n",
       "L 45.52129 48.406627 \n",
       "L 45.80141 48.679518 \n",
       "L 46.641771 49.498193 \n",
       "L 47.762253 49.498193 \n",
       "L 48.042374 49.771084 \n",
       "L 48.882735 50.589759 \n",
       "L 49.162856 50.862651 \n",
       "L 50.003217 51.681325 \n",
       "L 50.283338 51.954217 \n",
       "L 51.123699 52.772892 \n",
       "L 51.40382 53.045783 \n",
       "L 52.244181 53.864458 \n",
       "L 52.524302 54.137349 \n",
       "L 53.364663 54.956024 \n",
       "L 53.644784 55.228916 \n",
       "L 54.485145 56.04759 \n",
       "L 54.765265 56.320482 \n",
       "L 55.605627 57.139157 \n",
       "L 56.726109 57.139157 \n",
       "L 57.006229 57.412048 \n",
       "L 57.846591 58.230723 \n",
       "L 58.126711 58.503614 \n",
       "L 58.967073 59.322289 \n",
       "L 59.247193 59.595181 \n",
       "L 60.087555 60.413855 \n",
       "L 60.367675 60.686747 \n",
       "L 61.208037 61.505422 \n",
       "L 61.488157 61.778313 \n",
       "L 62.328518 62.596988 \n",
       "L 62.608639 62.86988 \n",
       "L 63.449 63.688554 \n",
       "L 63.729121 63.961446 \n",
       "L 64.569482 64.78012 \n",
       "L 64.849603 65.053012 \n",
       "L 65.689964 65.871687 \n",
       "L 66.810446 65.871687 \n",
       "L 67.090567 66.144578 \n",
       "L 67.930928 66.963253 \n",
       "L 68.211049 67.236145 \n",
       "L 69.05141 68.054819 \n",
       "L 69.33153 68.327711 \n",
       "L 70.171892 69.146386 \n",
       "L 70.452012 69.419277 \n",
       "L 71.292374 70.237952 \n",
       "L 71.572494 70.510843 \n",
       "L 72.412856 71.329518 \n",
       "L 72.692976 71.60241 \n",
       "L 73.533338 72.421084 \n",
       "L 73.813458 72.693976 \n",
       "L 74.65382 73.512651 \n",
       "L 75.774302 73.512651 \n",
       "L 76.054422 73.785542 \n",
       "L 76.894784 74.604217 \n",
       "L 77.174904 74.877108 \n",
       "L 78.015265 75.695783 \n",
       "L 78.295386 75.968675 \n",
       "L 79.135747 76.787349 \n",
       "L 79.415868 77.060241 \n",
       "L 80.256229 77.878916 \n",
       "L 80.53635 78.151807 \n",
       "L 81.376711 78.970482 \n",
       "L 81.656832 79.243373 \n",
       "L 82.497193 80.062048 \n",
       "L 82.777314 80.33494 \n",
       "L 83.617675 81.153614 \n",
       "L 84.738157 81.153614 \n",
       "L 85.018277 81.426506 \n",
       "L 85.858639 82.245181 \n",
       "L 86.138759 82.518072 \n",
       "L 86.979121 83.336747 \n",
       "L 87.259241 83.609639 \n",
       "L 88.099603 84.428313 \n",
       "L 88.379723 84.701205 \n",
       "L 89.220085 85.51988 \n",
       "L 89.500205 85.792771 \n",
       "L 90.340567 86.611446 \n",
       "L 90.620687 86.884337 \n",
       "L 91.461049 87.703012 \n",
       "L 91.741169 87.975904 \n",
       "L 92.58153 88.794578 \n",
       "L 93.702012 88.794578 \n",
       "L 93.982133 89.06747 \n",
       "L 94.822494 89.886145 \n",
       "L 95.102615 90.159036 \n",
       "L 95.942976 90.977711 \n",
       "L 96.223097 91.250602 \n",
       "L 97.063458 92.069277 \n",
       "L 97.343579 92.342169 \n",
       "L 98.18394 93.160843 \n",
       "L 98.464061 93.433735 \n",
       "L 99.304422 94.25241 \n",
       "L 99.584543 94.525301 \n",
       "L 100.424904 95.343976 \n",
       "L 100.705024 95.616867 \n",
       "L 101.545386 96.435542 \n",
       "L 102.665868 96.435542 \n",
       "L 102.945988 96.708434 \n",
       "L 103.78635 97.527108 \n",
       "L 104.06647 97.8 \n",
       "L 104.906832 98.618675 \n",
       "L 105.186952 98.891566 \n",
       "L 106.027314 99.710241 \n",
       "L 106.307434 99.983133 \n",
       "L 107.147796 100.801807 \n",
       "L 107.427916 101.074699 \n",
       "L 108.268277 101.893373 \n",
       "L 108.548398 102.166265 \n",
       "L 109.388759 102.98494 \n",
       "L 109.66888 103.257831 \n",
       "L 110.509241 104.076506 \n",
       "L 111.629723 104.076506 \n",
       "L 111.909844 104.349398 \n",
       "L 112.750205 105.168072 \n",
       "L 113.030326 105.440964 \n",
       "L 113.870687 106.259639 \n",
       "L 114.150808 106.53253 \n",
       "L 114.991169 107.351205 \n",
       "L 115.27129 107.624096 \n",
       "L 116.111651 108.442771 \n",
       "L 116.391771 108.715663 \n",
       "L 117.232133 109.534337 \n",
       "L 117.512253 109.807229 \n",
       "L 118.352615 110.625904 \n",
       "L 118.632735 110.898795 \n",
       "L 119.473097 111.71747 \n",
       "L 120.593579 111.71747 \n",
       "L 120.873699 111.990361 \n",
       "L 121.714061 112.809036 \n",
       "L 121.994181 113.081928 \n",
       "L 122.834543 113.900602 \n",
       "L 123.114663 114.173494 \n",
       "L 123.955024 114.992169 \n",
       "L 124.235145 115.26506 \n",
       "L 125.075506 116.083735 \n",
       "L 125.355627 116.356627 \n",
       "L 126.195988 117.175301 \n",
       "L 126.476109 117.448193 \n",
       "L 127.31647 118.266867 \n",
       "L 127.596591 118.539759 \n",
       "L 128.436952 119.358434 \n",
       "L 129.557434 119.358434 \n",
       "L 129.837555 119.631325 \n",
       "L 130.677916 120.45 \n",
       "L 130.958037 120.722892 \n",
       "L 131.798398 121.541566 \n",
       "L 132.078518 121.814458 \n",
       "L 132.91888 122.633133 \n",
       "L 133.199 122.906024 \n",
       "L 134.039362 123.724699 \n",
       "L 134.319482 123.99759 \n",
       "L 135.159844 124.816265 \n",
       "L 135.439964 125.089157 \n",
       "L 136.280326 125.907831 \n",
       "L 136.560446 126.180723 \n",
       "L 137.400808 126.999398 \n",
       "L 137.680928 127.272289 \n",
       "L 138.52129 128.090964 \n",
       "L 139.641771 128.090964 \n",
       "L 139.921892 128.363855 \n",
       "L 140.762253 129.18253 \n",
       "L 141.042374 129.455422 \n",
       "L 141.882735 130.274096 \n",
       "L 142.162856 130.546988 \n",
       "L 143.003217 131.365663 \n",
       "L 143.283338 131.638554 \n",
       "L 144.123699 132.457229 \n",
       "L 144.40382 132.73012 \n",
       "L 144.68394 133.821687 \n",
       "L 144.68394 134.913253 \n",
       "L 144.68394 136.004819 \n",
       "L 144.68394 137.096386 \n",
       "L 144.68394 138.187952 \n",
       "L 144.68394 139.279518 \n",
       "L 144.68394 140.371084 \n",
       "L 144.68394 141.462651 \n",
       "L 144.68394 142.554217 \n",
       "L 144.123699 143.1 \n",
       "L 143.563458 143.645783 \n",
       "L 143.563458 144.737349 \n",
       "L 143.563458 145.828916 \n",
       "L 143.563458 146.920482 \n",
       "L 143.563458 148.012048 \n",
       "L 143.563458 149.103614 \n",
       "L 143.563458 150.195181 \n",
       "L 143.563458 151.286747 \n",
       "L 143.563458 152.378313 \n",
       "L 143.003217 152.924096 \n",
       "L 142.442976 153.46988 \n",
       "L 142.442976 154.561446 \n",
       "L 142.442976 155.653012 \n",
       "L 142.442976 156.744578 \n",
       "L 142.442976 157.836145 \n",
       "L 142.442976 158.927711 \n",
       "L 142.442976 160.019277 \n",
       "L 142.442976 161.110843 \n",
       "L 142.442976 162.20241 \n",
       "L 141.882735 162.748193 \n",
       "L 141.322494 163.293976 \n",
       "L 141.322494 164.385542 \n",
       "L 141.322494 165.477108 \n",
       "L 141.322494 166.568675 \n",
       "L 141.322494 167.660241 \n",
       "L 141.322494 168.751807 \n",
       "L 141.322494 169.843373 \n",
       "L 141.322494 170.93494 \n",
       "L 141.322494 172.026506 \n",
       "L 140.762253 172.572289 \n",
       "L 140.202012 173.118072 \n",
       "L 140.202012 174.209639 \n",
       "L 140.202012 175.301205 \n",
       "L 140.202012 176.392771 \n",
       "L 140.202012 177.484337 \n",
       "L 140.202012 178.575904 \n",
       "L 140.202012 179.66747 \n",
       "L 140.202012 180.759036 \n",
       "L 140.202012 181.850602 \n",
       "L 139.641771 182.396386 \n",
       "L 139.08153 182.942169 \n",
       "L 139.08153 184.033735 \n",
       "L 139.08153 185.125301 \n",
       "L 139.08153 186.216867 \n",
       "L 139.08153 187.308434 \n",
       "L 139.08153 188.4 \n",
       "L 139.08153 189.491566 \n",
       "L 139.08153 190.583133 \n",
       "L 139.08153 191.674699 \n",
       "L 138.52129 192.220482 \n",
       "L 137.961049 192.766265 \n",
       "L 137.961049 193.857831 \n",
       "L 137.961049 194.949398 \n",
       "L 137.961049 196.040964 \n",
       "L 137.961049 197.13253 \n",
       "L 137.961049 198.224096 \n",
       "L 137.961049 199.315663 \n",
       "L 137.961049 200.407229 \n",
       "L 137.400808 200.953012 \n",
       "L 136.840567 201.498795 \n",
       "L 136.840567 202.590361 \n",
       "L 136.840567 203.681928 \n",
       "L 136.840567 204.773494 \n",
       "L 136.840567 205.86506 \n",
       "L 136.840567 206.956627 \n",
       "L 136.840567 208.048193 \n",
       "L 136.840567 209.139759 \n",
       "L 136.840567 210.231325 \n",
       "L 136.280326 210.777108 \n",
       "L 135.720085 211.322892 \n",
       "L 135.720085 212.414458 \n",
       "L 135.720085 213.506024 \n",
       "L 135.720085 214.59759 \n",
       "L 135.720085 215.689157 \n",
       "L 135.720085 216.780723 \n",
       "L 135.720085 217.872289 \n",
       "L 135.720085 218.963855 \n",
       "L 135.720085 220.055422 \n",
       "L 135.159844 220.601205 \n",
       "L 134.599603 221.146988 \n",
       "L 134.599603 222.238554 \n",
       "L 134.599603 223.33012 \n",
       "L 134.599603 224.421687 \n",
       "L 134.599603 225.513253 \n",
       "L 134.599603 226.604819 \n",
       "L 134.599603 227.696386 \n",
       "L 134.599603 228.787952 \n",
       "L 134.599603 229.879518 \n",
       "L 134.039362 230.425301 \n",
       "L 133.479121 230.971084 \n",
       "L 133.479121 232.062651 \n",
       "L 133.479121 233.154217 \n",
       "L 133.479121 234.245783 \n",
       "L 133.479121 235.337349 \n",
       "L 133.479121 236.428916 \n",
       "L 133.479121 237.520482 \n",
       "L 133.479121 238.612048 \n",
       "L 133.479121 239.703614 \n",
       "L 132.91888 240.249398 \n",
       "L 132.358639 240.795181 \n",
       "L 132.358639 241.886747 \n",
       "L 132.358639 242.978313 \n",
       "L 132.358639 244.06988 \n",
       "L 132.358639 245.161446 \n",
       "L 132.358639 246.253012 \n",
       "L 132.358639 247.344578 \n",
       "L 132.358639 248.436145 \n",
       "L 132.358639 249.527711 \n",
       "L 131.798398 250.073494 \n",
       "L 131.238157 250.619277 \n",
       "L 131.238157 251.710843 \n",
       "L 131.238157 252.80241 \n",
       "L 131.238157 253.893976 \n",
       "L 131.238157 254.985542 \n",
       "L 131.238157 256.077108 \n",
       "L 131.238157 257.168675 \n",
       "L 131.238157 258.260241 \n",
       "L 131.238157 259.351807 \n",
       "L 130.677916 259.89759 \n",
       "L 130.117675 260.443373 \n",
       "L 130.117675 261.53494 \n",
       "L 130.117675 262.626506 \n",
       "L 130.117675 263.718072 \n",
       "L 130.117675 264.809639 \n",
       "L 130.117675 265.901205 \n",
       "L 130.117675 266.992771 \n",
       "L 130.117675 268.084337 \n",
       "L 129.557434 268.63012 \n",
       "L 128.997193 269.175904 \n",
       "L 128.997193 270.26747 \n",
       "L 128.997193 271.359036 \n",
       "L 128.997193 272.450602 \n",
       "L 128.997193 273.542169 \n",
       "L 128.997193 274.633735 \n",
       "L 128.997193 275.725301 \n",
       "L 128.997193 276.816867 \n",
       "L 128.997193 277.908434 \n",
       "L 128.436952 278.454217 \n",
       "L 127.876711 279 \n",
       "L 128.156832 279 \n",
       "z\n",
       "\" style=\"fill:#d62728;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_4\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 128.436952 279 \n",
       "L 129.557434 279 \n",
       "L 130.677916 279 \n",
       "L 131.798398 279 \n",
       "L 132.91888 279 \n",
       "L 134.039362 279 \n",
       "L 135.159844 279 \n",
       "L 136.280326 279 \n",
       "L 137.400808 279 \n",
       "L 138.52129 279 \n",
       "L 139.641771 279 \n",
       "L 140.762253 279 \n",
       "L 141.882735 279 \n",
       "L 143.003217 279 \n",
       "L 144.123699 279 \n",
       "L 145.244181 279 \n",
       "L 146.364663 279 \n",
       "L 147.485145 279 \n",
       "L 148.605627 279 \n",
       "L 149.726109 279 \n",
       "L 150.846591 279 \n",
       "L 151.967073 279 \n",
       "L 153.087555 279 \n",
       "L 154.208037 279 \n",
       "L 155.328518 279 \n",
       "L 156.449 279 \n",
       "L 157.569482 279 \n",
       "L 158.689964 279 \n",
       "L 159.810446 279 \n",
       "L 160.930928 279 \n",
       "L 162.05141 279 \n",
       "L 163.171892 279 \n",
       "L 164.292374 279 \n",
       "L 165.412856 279 \n",
       "L 166.533338 279 \n",
       "L 167.65382 279 \n",
       "L 168.774302 279 \n",
       "L 169.894784 279 \n",
       "L 171.015265 279 \n",
       "L 172.135747 279 \n",
       "L 173.256229 279 \n",
       "L 174.376711 279 \n",
       "L 175.497193 279 \n",
       "L 176.617675 279 \n",
       "L 177.738157 279 \n",
       "L 178.858639 279 \n",
       "L 179.979121 279 \n",
       "L 181.099603 279 \n",
       "L 182.220085 279 \n",
       "L 183.340567 279 \n",
       "L 184.461049 279 \n",
       "L 185.58153 279 \n",
       "L 186.702012 279 \n",
       "L 187.822494 279 \n",
       "L 188.942976 279 \n",
       "L 190.063458 279 \n",
       "L 191.18394 279 \n",
       "L 192.304422 279 \n",
       "L 193.424904 279 \n",
       "L 194.545386 279 \n",
       "L 195.665868 279 \n",
       "L 196.78635 279 \n",
       "L 197.906832 279 \n",
       "L 199.027314 279 \n",
       "L 200.147796 279 \n",
       "L 201.268277 279 \n",
       "L 202.388759 279 \n",
       "L 203.509241 279 \n",
       "L 204.629723 279 \n",
       "L 205.750205 279 \n",
       "L 206.870687 279 \n",
       "L 207.991169 279 \n",
       "L 209.111651 279 \n",
       "L 210.232133 279 \n",
       "L 211.352615 279 \n",
       "L 212.473097 279 \n",
       "L 213.593579 279 \n",
       "L 214.714061 279 \n",
       "L 215.834543 279 \n",
       "L 216.955024 279 \n",
       "L 218.075506 279 \n",
       "L 219.195988 279 \n",
       "L 220.31647 279 \n",
       "L 221.436952 279 \n",
       "L 222.557434 279 \n",
       "L 223.677916 279 \n",
       "L 224.798398 279 \n",
       "L 225.91888 279 \n",
       "L 227.039362 279 \n",
       "L 228.159844 279 \n",
       "L 229.280326 279 \n",
       "L 230.400808 279 \n",
       "L 231.52129 279 \n",
       "L 232.641771 279 \n",
       "L 233.762253 279 \n",
       "L 234.882735 279 \n",
       "L 236.003217 279 \n",
       "L 237.123699 279 \n",
       "L 238.244181 279 \n",
       "L 239.364663 279 \n",
       "L 240.485145 279 \n",
       "L 241.605627 279 \n",
       "L 242.726109 279 \n",
       "L 243.846591 279 \n",
       "L 244.967073 279 \n",
       "L 246.087555 279 \n",
       "L 247.208037 279 \n",
       "L 248.328518 279 \n",
       "L 249.449 279 \n",
       "L 250.569482 279 \n",
       "L 251.689964 279 \n",
       "L 252.810446 279 \n",
       "L 253.930928 279 \n",
       "L 255.05141 279 \n",
       "L 256.171892 279 \n",
       "L 257.292374 279 \n",
       "L 258.412856 279 \n",
       "L 259.533338 279 \n",
       "L 260.65382 279 \n",
       "L 261.774302 279 \n",
       "L 262.894784 279 \n",
       "L 264.015265 279 \n",
       "L 265.135747 279 \n",
       "L 266.256229 279 \n",
       "L 267.376711 279 \n",
       "L 268.497193 279 \n",
       "L 269.617675 279 \n",
       "L 270.738157 279 \n",
       "L 271.858639 279 \n",
       "L 272.979121 279 \n",
       "L 274.099603 279 \n",
       "L 275.220085 279 \n",
       "L 276.340567 279 \n",
       "L 277.461049 279 \n",
       "L 278.58153 279 \n",
       "L 279.702012 279 \n",
       "L 280.822494 279 \n",
       "L 281.942976 279 \n",
       "L 283.063458 279 \n",
       "L 284.18394 279 \n",
       "L 285.304422 279 \n",
       "L 286.424904 279 \n",
       "L 287.545386 279 \n",
       "L 288.665868 279 \n",
       "L 289.78635 279 \n",
       "L 290.906832 279 \n",
       "L 292.027314 279 \n",
       "L 293.147796 279 \n",
       "L 294.268277 279 \n",
       "L 295.388759 279 \n",
       "L 296.509241 279 \n",
       "L 297.629723 279 \n",
       "L 298.750205 279 \n",
       "L 299.870687 279 \n",
       "L 300.991169 279 \n",
       "L 302.111651 279 \n",
       "L 303.232133 279 \n",
       "L 304.352615 279 \n",
       "L 305.473097 279 \n",
       "L 306.593579 279 \n",
       "L 307.714061 279 \n",
       "L 308.834543 279 \n",
       "L 309.955024 279 \n",
       "L 311.075506 279 \n",
       "L 312.195988 279 \n",
       "L 313.31647 279 \n",
       "L 314.436952 279 \n",
       "L 315.557434 279 \n",
       "L 316.677916 279 \n",
       "L 317.798398 279 \n",
       "L 318.91888 279 \n",
       "L 320.039362 279 \n",
       "L 321.159844 279 \n",
       "L 321.159844 277.908434 \n",
       "L 321.159844 276.816867 \n",
       "L 321.159844 275.725301 \n",
       "L 321.159844 274.633735 \n",
       "L 321.159844 273.542169 \n",
       "L 321.159844 272.450602 \n",
       "L 321.159844 271.359036 \n",
       "L 321.159844 270.26747 \n",
       "L 321.159844 269.175904 \n",
       "L 321.159844 268.084337 \n",
       "L 321.159844 266.992771 \n",
       "L 321.159844 265.901205 \n",
       "L 321.159844 264.809639 \n",
       "L 321.159844 263.718072 \n",
       "L 321.159844 262.626506 \n",
       "L 321.159844 261.53494 \n",
       "L 321.159844 260.443373 \n",
       "L 321.159844 259.351807 \n",
       "L 321.159844 258.260241 \n",
       "L 321.159844 257.168675 \n",
       "L 321.159844 256.077108 \n",
       "L 321.159844 254.985542 \n",
       "L 321.159844 253.893976 \n",
       "L 321.159844 252.80241 \n",
       "L 321.159844 251.710843 \n",
       "L 321.159844 250.619277 \n",
       "L 321.159844 249.527711 \n",
       "L 321.159844 248.436145 \n",
       "L 321.159844 247.344578 \n",
       "L 321.159844 246.253012 \n",
       "L 321.159844 245.161446 \n",
       "L 321.159844 244.06988 \n",
       "L 321.159844 242.978313 \n",
       "L 321.159844 241.886747 \n",
       "L 321.159844 240.795181 \n",
       "L 321.159844 239.703614 \n",
       "L 321.159844 238.612048 \n",
       "L 321.159844 237.520482 \n",
       "L 321.159844 236.428916 \n",
       "L 321.159844 235.337349 \n",
       "L 321.159844 234.245783 \n",
       "L 321.159844 233.154217 \n",
       "L 321.159844 232.062651 \n",
       "L 321.159844 230.971084 \n",
       "L 321.159844 229.879518 \n",
       "L 321.159844 228.787952 \n",
       "L 321.159844 227.696386 \n",
       "L 321.159844 226.604819 \n",
       "L 321.159844 225.513253 \n",
       "L 321.159844 224.421687 \n",
       "L 321.159844 223.33012 \n",
       "L 321.159844 222.238554 \n",
       "L 321.159844 221.146988 \n",
       "L 321.159844 220.055422 \n",
       "L 321.159844 218.963855 \n",
       "L 321.159844 217.872289 \n",
       "L 321.159844 216.780723 \n",
       "L 321.159844 215.689157 \n",
       "L 321.159844 214.59759 \n",
       "L 321.159844 213.506024 \n",
       "L 321.159844 212.414458 \n",
       "L 321.159844 211.322892 \n",
       "L 321.159844 210.231325 \n",
       "L 321.159844 209.139759 \n",
       "L 321.159844 208.048193 \n",
       "L 321.159844 206.956627 \n",
       "L 321.159844 205.86506 \n",
       "L 321.159844 204.773494 \n",
       "L 321.159844 203.681928 \n",
       "L 321.159844 202.590361 \n",
       "L 321.159844 201.498795 \n",
       "L 321.159844 200.407229 \n",
       "L 321.159844 199.315663 \n",
       "L 321.159844 198.224096 \n",
       "L 321.159844 197.13253 \n",
       "L 321.159844 196.040964 \n",
       "L 321.159844 194.949398 \n",
       "L 321.159844 193.857831 \n",
       "L 321.159844 192.766265 \n",
       "L 321.159844 191.674699 \n",
       "L 321.159844 190.583133 \n",
       "L 321.159844 189.491566 \n",
       "L 321.159844 188.4 \n",
       "L 321.159844 187.308434 \n",
       "L 321.159844 186.216867 \n",
       "L 321.159844 185.125301 \n",
       "L 321.159844 184.033735 \n",
       "L 321.159844 182.942169 \n",
       "L 321.159844 181.850602 \n",
       "L 321.159844 180.759036 \n",
       "L 321.159844 179.66747 \n",
       "L 321.159844 178.575904 \n",
       "L 321.159844 177.484337 \n",
       "L 321.159844 176.392771 \n",
       "L 321.159844 175.301205 \n",
       "L 321.159844 174.209639 \n",
       "L 321.159844 173.118072 \n",
       "L 321.159844 172.026506 \n",
       "L 321.159844 170.93494 \n",
       "L 321.159844 169.843373 \n",
       "L 321.159844 168.751807 \n",
       "L 321.159844 167.660241 \n",
       "L 321.159844 166.568675 \n",
       "L 321.159844 165.477108 \n",
       "L 321.159844 164.385542 \n",
       "L 321.159844 163.293976 \n",
       "L 321.159844 162.20241 \n",
       "L 321.159844 161.110843 \n",
       "L 321.159844 160.019277 \n",
       "L 321.159844 158.927711 \n",
       "L 321.159844 157.836145 \n",
       "L 321.159844 156.744578 \n",
       "L 321.159844 155.653012 \n",
       "L 321.159844 154.561446 \n",
       "L 321.159844 153.46988 \n",
       "L 321.159844 152.378313 \n",
       "L 321.159844 151.286747 \n",
       "L 321.159844 150.195181 \n",
       "L 321.159844 149.103614 \n",
       "L 321.159844 148.012048 \n",
       "L 321.159844 146.920482 \n",
       "L 321.159844 145.828916 \n",
       "L 321.159844 144.737349 \n",
       "L 321.159844 143.645783 \n",
       "L 321.159844 142.554217 \n",
       "L 321.159844 141.462651 \n",
       "L 321.159844 140.371084 \n",
       "L 321.159844 139.279518 \n",
       "L 321.159844 138.187952 \n",
       "L 321.159844 137.096386 \n",
       "L 321.159844 136.004819 \n",
       "L 321.159844 134.913253 \n",
       "L 321.159844 133.821687 \n",
       "L 321.159844 132.73012 \n",
       "L 321.159844 131.638554 \n",
       "L 321.159844 130.546988 \n",
       "L 321.159844 129.455422 \n",
       "L 321.159844 128.363855 \n",
       "L 321.159844 127.272289 \n",
       "L 321.159844 126.180723 \n",
       "L 321.159844 125.089157 \n",
       "L 321.159844 123.99759 \n",
       "L 321.159844 122.906024 \n",
       "L 321.159844 121.814458 \n",
       "L 321.159844 120.722892 \n",
       "L 321.159844 119.631325 \n",
       "L 321.159844 118.539759 \n",
       "L 321.159844 117.448193 \n",
       "L 321.159844 116.356627 \n",
       "L 321.159844 115.26506 \n",
       "L 321.159844 114.173494 \n",
       "L 321.159844 113.081928 \n",
       "L 321.159844 111.990361 \n",
       "L 321.159844 110.898795 \n",
       "L 321.159844 109.807229 \n",
       "L 321.159844 108.715663 \n",
       "L 321.159844 107.624096 \n",
       "L 321.159844 106.53253 \n",
       "L 321.159844 105.440964 \n",
       "L 321.159844 104.349398 \n",
       "L 321.159844 103.257831 \n",
       "L 321.159844 102.166265 \n",
       "L 321.159844 101.074699 \n",
       "L 321.159844 99.983133 \n",
       "L 321.159844 98.891566 \n",
       "L 321.159844 97.8 \n",
       "L 321.159844 96.708434 \n",
       "L 321.159844 95.616867 \n",
       "L 321.159844 94.525301 \n",
       "L 321.159844 93.433735 \n",
       "L 321.159844 92.342169 \n",
       "L 321.159844 91.250602 \n",
       "L 321.159844 90.159036 \n",
       "L 321.159844 89.06747 \n",
       "L 320.039362 90.159036 \n",
       "L 318.91888 90.159036 \n",
       "L 317.798398 90.159036 \n",
       "L 316.677916 90.159036 \n",
       "L 315.557434 91.250602 \n",
       "L 314.436952 91.250602 \n",
       "L 313.31647 91.250602 \n",
       "L 312.195988 91.250602 \n",
       "L 311.075506 92.342169 \n",
       "L 309.955024 92.342169 \n",
       "L 308.834543 92.342169 \n",
       "L 307.714061 92.342169 \n",
       "L 306.593579 93.433735 \n",
       "L 305.473097 93.433735 \n",
       "L 304.352615 93.433735 \n",
       "L 303.232133 93.433735 \n",
       "L 302.111651 94.525301 \n",
       "L 300.991169 94.525301 \n",
       "L 299.870687 94.525301 \n",
       "L 298.750205 95.616867 \n",
       "L 297.629723 95.616867 \n",
       "L 296.509241 95.616867 \n",
       "L 295.388759 95.616867 \n",
       "L 294.268277 96.708434 \n",
       "L 293.147796 96.708434 \n",
       "L 292.027314 96.708434 \n",
       "L 290.906832 96.708434 \n",
       "L 289.78635 97.8 \n",
       "L 288.665868 97.8 \n",
       "L 287.545386 97.8 \n",
       "L 286.424904 97.8 \n",
       "L 285.304422 98.891566 \n",
       "L 284.18394 98.891566 \n",
       "L 283.063458 98.891566 \n",
       "L 281.942976 98.891566 \n",
       "L 280.822494 99.983133 \n",
       "L 279.702012 99.983133 \n",
       "L 278.58153 99.983133 \n",
       "L 277.461049 99.983133 \n",
       "L 276.340567 101.074699 \n",
       "L 275.220085 101.074699 \n",
       "L 274.099603 101.074699 \n",
       "L 272.979121 101.074699 \n",
       "L 271.858639 102.166265 \n",
       "L 270.738157 102.166265 \n",
       "L 269.617675 102.166265 \n",
       "L 268.497193 102.166265 \n",
       "L 267.376711 103.257831 \n",
       "L 266.256229 103.257831 \n",
       "L 265.135747 103.257831 \n",
       "L 264.015265 104.349398 \n",
       "L 262.894784 104.349398 \n",
       "L 261.774302 104.349398 \n",
       "L 260.65382 104.349398 \n",
       "L 259.533338 105.440964 \n",
       "L 258.412856 105.440964 \n",
       "L 257.292374 105.440964 \n",
       "L 256.171892 105.440964 \n",
       "L 255.05141 106.53253 \n",
       "L 253.930928 106.53253 \n",
       "L 252.810446 106.53253 \n",
       "L 251.689964 106.53253 \n",
       "L 250.569482 107.624096 \n",
       "L 249.449 107.624096 \n",
       "L 248.328518 107.624096 \n",
       "L 247.208037 107.624096 \n",
       "L 246.087555 108.715663 \n",
       "L 244.967073 108.715663 \n",
       "L 243.846591 108.715663 \n",
       "L 242.726109 108.715663 \n",
       "L 241.605627 109.807229 \n",
       "L 240.485145 109.807229 \n",
       "L 239.364663 109.807229 \n",
       "L 238.244181 109.807229 \n",
       "L 237.123699 110.898795 \n",
       "L 236.003217 110.898795 \n",
       "L 234.882735 110.898795 \n",
       "L 233.762253 110.898795 \n",
       "L 232.641771 111.990361 \n",
       "L 231.52129 111.990361 \n",
       "L 230.400808 111.990361 \n",
       "L 229.280326 113.081928 \n",
       "L 228.159844 113.081928 \n",
       "L 227.039362 113.081928 \n",
       "L 225.91888 113.081928 \n",
       "L 224.798398 114.173494 \n",
       "L 223.677916 114.173494 \n",
       "L 222.557434 114.173494 \n",
       "L 221.436952 114.173494 \n",
       "L 220.31647 115.26506 \n",
       "L 219.195988 115.26506 \n",
       "L 218.075506 115.26506 \n",
       "L 216.955024 115.26506 \n",
       "L 215.834543 116.356627 \n",
       "L 214.714061 116.356627 \n",
       "L 213.593579 116.356627 \n",
       "L 212.473097 116.356627 \n",
       "L 211.352615 117.448193 \n",
       "L 210.232133 117.448193 \n",
       "L 209.111651 117.448193 \n",
       "L 207.991169 117.448193 \n",
       "L 206.870687 118.539759 \n",
       "L 205.750205 118.539759 \n",
       "L 204.629723 118.539759 \n",
       "L 203.509241 118.539759 \n",
       "L 202.388759 119.631325 \n",
       "L 201.268277 119.631325 \n",
       "L 200.147796 119.631325 \n",
       "L 199.027314 119.631325 \n",
       "L 197.906832 120.722892 \n",
       "L 196.78635 120.722892 \n",
       "L 195.665868 120.722892 \n",
       "L 194.545386 121.814458 \n",
       "L 193.424904 121.814458 \n",
       "L 192.304422 121.814458 \n",
       "L 191.18394 121.814458 \n",
       "L 190.063458 122.906024 \n",
       "L 188.942976 122.906024 \n",
       "L 187.822494 122.906024 \n",
       "L 186.702012 122.906024 \n",
       "L 185.58153 123.99759 \n",
       "L 184.461049 123.99759 \n",
       "L 183.340567 123.99759 \n",
       "L 182.220085 123.99759 \n",
       "L 181.099603 125.089157 \n",
       "L 179.979121 125.089157 \n",
       "L 178.858639 125.089157 \n",
       "L 177.738157 125.089157 \n",
       "L 176.617675 126.180723 \n",
       "L 175.497193 126.180723 \n",
       "L 174.376711 126.180723 \n",
       "L 173.256229 126.180723 \n",
       "L 172.135747 127.272289 \n",
       "L 171.015265 127.272289 \n",
       "L 169.894784 127.272289 \n",
       "L 168.774302 127.272289 \n",
       "L 167.65382 128.363855 \n",
       "L 166.533338 128.363855 \n",
       "L 165.412856 128.363855 \n",
       "L 164.292374 128.363855 \n",
       "L 163.171892 129.455422 \n",
       "L 162.05141 129.455422 \n",
       "L 160.930928 129.455422 \n",
       "L 159.810446 130.546988 \n",
       "L 158.689964 130.546988 \n",
       "L 157.569482 130.546988 \n",
       "L 156.449 130.546988 \n",
       "L 155.328518 131.638554 \n",
       "L 154.208037 131.638554 \n",
       "L 153.087555 131.638554 \n",
       "L 151.967073 131.638554 \n",
       "L 150.846591 132.73012 \n",
       "L 149.726109 132.73012 \n",
       "L 148.605627 132.73012 \n",
       "L 147.485145 132.73012 \n",
       "L 146.364663 133.821687 \n",
       "L 145.244181 133.821687 \n",
       "L 144.68394 132.73012 \n",
       "L 144.123699 132.184337 \n",
       "L 143.563458 131.638554 \n",
       "L 143.003217 131.092771 \n",
       "L 142.442976 130.546988 \n",
       "L 141.882735 130.001205 \n",
       "L 141.322494 129.455422 \n",
       "L 140.762253 128.909639 \n",
       "L 140.202012 128.363855 \n",
       "L 139.641771 127.818072 \n",
       "L 138.52129 127.818072 \n",
       "L 137.961049 127.272289 \n",
       "L 137.400808 126.726506 \n",
       "L 136.840567 126.180723 \n",
       "L 136.280326 125.63494 \n",
       "L 135.720085 125.089157 \n",
       "L 135.159844 124.543373 \n",
       "L 134.599603 123.99759 \n",
       "L 134.039362 123.451807 \n",
       "L 133.479121 122.906024 \n",
       "L 132.91888 122.360241 \n",
       "L 132.358639 121.814458 \n",
       "L 131.798398 121.268675 \n",
       "L 131.238157 120.722892 \n",
       "L 130.677916 120.177108 \n",
       "L 130.117675 119.631325 \n",
       "L 129.557434 119.085542 \n",
       "L 128.436952 119.085542 \n",
       "L 127.876711 118.539759 \n",
       "L 127.31647 117.993976 \n",
       "L 126.756229 117.448193 \n",
       "L 126.195988 116.90241 \n",
       "L 125.635747 116.356627 \n",
       "L 125.075506 115.810843 \n",
       "L 124.515265 115.26506 \n",
       "L 123.955024 114.719277 \n",
       "L 123.394784 114.173494 \n",
       "L 122.834543 113.627711 \n",
       "L 122.274302 113.081928 \n",
       "L 121.714061 112.536145 \n",
       "L 121.15382 111.990361 \n",
       "L 120.593579 111.444578 \n",
       "L 119.473097 111.444578 \n",
       "L 118.912856 110.898795 \n",
       "L 118.352615 110.353012 \n",
       "L 117.792374 109.807229 \n",
       "L 117.232133 109.261446 \n",
       "L 116.671892 108.715663 \n",
       "L 116.111651 108.16988 \n",
       "L 115.55141 107.624096 \n",
       "L 114.991169 107.078313 \n",
       "L 114.430928 106.53253 \n",
       "L 113.870687 105.986747 \n",
       "L 113.310446 105.440964 \n",
       "L 112.750205 104.895181 \n",
       "L 112.189964 104.349398 \n",
       "L 111.629723 103.803614 \n",
       "L 110.509241 103.803614 \n",
       "L 109.949 103.257831 \n",
       "L 109.388759 102.712048 \n",
       "L 108.828518 102.166265 \n",
       "L 108.268277 101.620482 \n",
       "L 107.708037 101.074699 \n",
       "L 107.147796 100.528916 \n",
       "L 106.587555 99.983133 \n",
       "L 106.027314 99.437349 \n",
       "L 105.467073 98.891566 \n",
       "L 104.906832 98.345783 \n",
       "L 104.346591 97.8 \n",
       "L 103.78635 97.254217 \n",
       "L 103.226109 96.708434 \n",
       "L 102.665868 96.162651 \n",
       "L 101.545386 96.162651 \n",
       "L 100.985145 95.616867 \n",
       "L 100.424904 95.071084 \n",
       "L 99.864663 94.525301 \n",
       "L 99.304422 93.979518 \n",
       "L 98.744181 93.433735 \n",
       "L 98.18394 92.887952 \n",
       "L 97.623699 92.342169 \n",
       "L 97.063458 91.796386 \n",
       "L 96.503217 91.250602 \n",
       "L 95.942976 90.704819 \n",
       "L 95.382735 90.159036 \n",
       "L 94.822494 89.613253 \n",
       "L 94.262253 89.06747 \n",
       "L 93.702012 88.521687 \n",
       "L 92.58153 88.521687 \n",
       "L 92.02129 87.975904 \n",
       "L 91.461049 87.43012 \n",
       "L 90.900808 86.884337 \n",
       "L 90.340567 86.338554 \n",
       "L 89.780326 85.792771 \n",
       "L 89.220085 85.246988 \n",
       "L 88.659844 84.701205 \n",
       "L 88.099603 84.155422 \n",
       "L 87.539362 83.609639 \n",
       "L 86.979121 83.063855 \n",
       "L 86.41888 82.518072 \n",
       "L 85.858639 81.972289 \n",
       "L 85.298398 81.426506 \n",
       "L 84.738157 80.880723 \n",
       "L 83.617675 80.880723 \n",
       "L 83.057434 80.33494 \n",
       "L 82.497193 79.789157 \n",
       "L 81.936952 79.243373 \n",
       "L 81.376711 78.69759 \n",
       "L 80.81647 78.151807 \n",
       "L 80.256229 77.606024 \n",
       "L 79.695988 77.060241 \n",
       "L 79.135747 76.514458 \n",
       "L 78.575506 75.968675 \n",
       "L 78.015265 75.422892 \n",
       "L 77.455024 74.877108 \n",
       "L 76.894784 74.331325 \n",
       "L 76.334543 73.785542 \n",
       "L 75.774302 73.239759 \n",
       "L 74.65382 73.239759 \n",
       "L 74.093579 72.693976 \n",
       "L 73.533338 72.148193 \n",
       "L 72.973097 71.60241 \n",
       "L 72.412856 71.056627 \n",
       "L 71.852615 70.510843 \n",
       "L 71.292374 69.96506 \n",
       "L 70.732133 69.419277 \n",
       "L 70.171892 68.873494 \n",
       "L 69.611651 68.327711 \n",
       "L 69.05141 67.781928 \n",
       "L 68.491169 67.236145 \n",
       "L 67.930928 66.690361 \n",
       "L 67.370687 66.144578 \n",
       "L 66.810446 65.598795 \n",
       "L 65.689964 65.598795 \n",
       "L 65.129723 65.053012 \n",
       "L 64.569482 64.507229 \n",
       "L 64.009241 63.961446 \n",
       "L 63.449 63.415663 \n",
       "L 62.888759 62.86988 \n",
       "L 62.328518 62.324096 \n",
       "L 61.768277 61.778313 \n",
       "L 61.208037 61.23253 \n",
       "L 60.647796 60.686747 \n",
       "L 60.087555 60.140964 \n",
       "L 59.527314 59.595181 \n",
       "L 58.967073 59.049398 \n",
       "L 58.406832 58.503614 \n",
       "L 57.846591 57.957831 \n",
       "L 57.28635 57.412048 \n",
       "L 56.726109 56.866265 \n",
       "L 55.605627 56.866265 \n",
       "L 55.045386 56.320482 \n",
       "L 54.485145 55.774699 \n",
       "L 53.924904 55.228916 \n",
       "L 53.364663 54.683133 \n",
       "L 52.804422 54.137349 \n",
       "L 52.244181 53.591566 \n",
       "L 51.68394 53.045783 \n",
       "L 51.123699 52.5 \n",
       "L 50.563458 51.954217 \n",
       "L 50.003217 51.408434 \n",
       "L 49.442976 50.862651 \n",
       "L 48.882735 50.316867 \n",
       "L 48.322494 49.771084 \n",
       "L 47.762253 49.225301 \n",
       "L 46.641771 49.225301 \n",
       "L 46.08153 48.679518 \n",
       "L 45.52129 48.133735 \n",
       "L 44.961049 47.587952 \n",
       "L 44.400808 47.042169 \n",
       "L 43.840567 46.496386 \n",
       "L 43.280326 45.950602 \n",
       "L 42.720085 45.404819 \n",
       "L 42.159844 44.859036 \n",
       "L 42.159844 44.995482 \n",
       "L 42.580024 45.404819 \n",
       "L 43.280326 46.087048 \n",
       "L 43.700506 46.496386 \n",
       "L 44.400808 47.178614 \n",
       "L 44.820988 47.587952 \n",
       "L 45.52129 48.270181 \n",
       "L 45.94147 48.679518 \n",
       "L 46.641771 49.361747 \n",
       "L 47.762253 49.361747 \n",
       "L 48.182434 49.771084 \n",
       "L 48.882735 50.453313 \n",
       "L 49.302916 50.862651 \n",
       "L 50.003217 51.54488 \n",
       "L 50.423398 51.954217 \n",
       "L 51.123699 52.636446 \n",
       "L 51.54388 53.045783 \n",
       "L 52.244181 53.728012 \n",
       "L 52.664362 54.137349 \n",
       "L 53.364663 54.819578 \n",
       "L 53.784844 55.228916 \n",
       "L 54.485145 55.911145 \n",
       "L 54.905326 56.320482 \n",
       "L 55.605627 57.002711 \n",
       "L 56.726109 57.002711 \n",
       "L 57.14629 57.412048 \n",
       "L 57.846591 58.094277 \n",
       "L 58.266771 58.503614 \n",
       "L 58.967073 59.185843 \n",
       "L 59.387253 59.595181 \n",
       "L 60.087555 60.27741 \n",
       "L 60.507735 60.686747 \n",
       "L 61.208037 61.368976 \n",
       "L 61.628217 61.778313 \n",
       "L 62.328518 62.460542 \n",
       "L 62.748699 62.86988 \n",
       "L 63.449 63.552108 \n",
       "L 63.869181 63.961446 \n",
       "L 64.569482 64.643675 \n",
       "L 64.989663 65.053012 \n",
       "L 65.689964 65.735241 \n",
       "L 66.810446 65.735241 \n",
       "L 67.230627 66.144578 \n",
       "L 67.930928 66.826807 \n",
       "L 68.351109 67.236145 \n",
       "L 69.05141 67.918373 \n",
       "L 69.471591 68.327711 \n",
       "L 70.171892 69.00994 \n",
       "L 70.592073 69.419277 \n",
       "L 71.292374 70.101506 \n",
       "L 71.712555 70.510843 \n",
       "L 72.412856 71.193072 \n",
       "L 72.833037 71.60241 \n",
       "L 73.533338 72.284639 \n",
       "L 73.953518 72.693976 \n",
       "L 74.65382 73.376205 \n",
       "L 75.774302 73.376205 \n",
       "L 76.194482 73.785542 \n",
       "L 76.894784 74.467771 \n",
       "L 77.314964 74.877108 \n",
       "L 78.015265 75.559337 \n",
       "L 78.435446 75.968675 \n",
       "L 79.135747 76.650904 \n",
       "L 79.555928 77.060241 \n",
       "L 80.256229 77.74247 \n",
       "L 80.67641 78.151807 \n",
       "L 81.376711 78.834036 \n",
       "L 81.796892 79.243373 \n",
       "L 82.497193 79.925602 \n",
       "L 82.917374 80.33494 \n",
       "L 83.617675 81.017169 \n",
       "L 84.738157 81.017169 \n",
       "L 85.158338 81.426506 \n",
       "L 85.858639 82.108735 \n",
       "L 86.27882 82.518072 \n",
       "L 86.979121 83.200301 \n",
       "L 87.399302 83.609639 \n",
       "L 88.099603 84.291867 \n",
       "L 88.519784 84.701205 \n",
       "L 89.220085 85.383434 \n",
       "L 89.640265 85.792771 \n",
       "L 90.340567 86.475 \n",
       "L 90.760747 86.884337 \n",
       "L 91.461049 87.566566 \n",
       "L 91.881229 87.975904 \n",
       "L 92.58153 88.658133 \n",
       "L 93.702012 88.658133 \n",
       "L 94.122193 89.06747 \n",
       "L 94.822494 89.749699 \n",
       "L 95.242675 90.159036 \n",
       "L 95.942976 90.841265 \n",
       "L 96.363157 91.250602 \n",
       "L 97.063458 91.932831 \n",
       "L 97.483639 92.342169 \n",
       "L 98.18394 93.024398 \n",
       "L 98.604121 93.433735 \n",
       "L 99.304422 94.115964 \n",
       "L 99.724603 94.525301 \n",
       "L 100.424904 95.20753 \n",
       "L 100.845085 95.616867 \n",
       "L 101.545386 96.299096 \n",
       "L 102.665868 96.299096 \n",
       "L 103.086049 96.708434 \n",
       "L 103.78635 97.390663 \n",
       "L 104.20653 97.8 \n",
       "L 104.906832 98.482229 \n",
       "L 105.327012 98.891566 \n",
       "L 106.027314 99.573795 \n",
       "L 106.447494 99.983133 \n",
       "L 107.147796 100.665361 \n",
       "L 107.567976 101.074699 \n",
       "L 108.268277 101.756928 \n",
       "L 108.688458 102.166265 \n",
       "L 109.388759 102.848494 \n",
       "L 109.80894 103.257831 \n",
       "L 110.509241 103.94006 \n",
       "L 111.629723 103.94006 \n",
       "L 112.049904 104.349398 \n",
       "L 112.750205 105.031627 \n",
       "L 113.170386 105.440964 \n",
       "L 113.870687 106.123193 \n",
       "L 114.290868 106.53253 \n",
       "L 114.991169 107.214759 \n",
       "L 115.41135 107.624096 \n",
       "L 116.111651 108.306325 \n",
       "L 116.531832 108.715663 \n",
       "L 117.232133 109.397892 \n",
       "L 117.652314 109.807229 \n",
       "L 118.352615 110.489458 \n",
       "L 118.772796 110.898795 \n",
       "L 119.473097 111.581024 \n",
       "L 120.593579 111.581024 \n",
       "L 121.013759 111.990361 \n",
       "L 121.714061 112.67259 \n",
       "L 122.134241 113.081928 \n",
       "L 122.834543 113.764157 \n",
       "L 123.254723 114.173494 \n",
       "L 123.955024 114.855723 \n",
       "L 124.375205 115.26506 \n",
       "L 125.075506 115.947289 \n",
       "L 125.495687 116.356627 \n",
       "L 126.195988 117.038855 \n",
       "L 126.616169 117.448193 \n",
       "L 127.31647 118.130422 \n",
       "L 127.736651 118.539759 \n",
       "L 128.436952 119.221988 \n",
       "L 129.557434 119.221988 \n",
       "L 129.977615 119.631325 \n",
       "L 130.677916 120.313554 \n",
       "L 131.098097 120.722892 \n",
       "L 131.798398 121.40512 \n",
       "L 132.218579 121.814458 \n",
       "L 132.91888 122.496687 \n",
       "L 133.339061 122.906024 \n",
       "L 134.039362 123.588253 \n",
       "L 134.459543 123.99759 \n",
       "L 135.159844 124.679819 \n",
       "L 135.580024 125.089157 \n",
       "L 136.280326 125.771386 \n",
       "L 136.700506 126.180723 \n",
       "L 137.400808 126.862952 \n",
       "L 137.820988 127.272289 \n",
       "L 138.52129 127.954518 \n",
       "L 139.641771 127.954518 \n",
       "L 140.061952 128.363855 \n",
       "L 140.762253 129.046084 \n",
       "L 141.182434 129.455422 \n",
       "L 141.882735 130.137651 \n",
       "L 142.302916 130.546988 \n",
       "L 143.003217 131.229217 \n",
       "L 143.423398 131.638554 \n",
       "L 144.123699 132.320783 \n",
       "L 144.54388 132.73012 \n",
       "L 144.964061 133.821687 \n",
       "L 144.964061 134.913253 \n",
       "L 144.964061 136.004819 \n",
       "L 144.964061 137.096386 \n",
       "L 144.964061 138.187952 \n",
       "L 144.964061 139.279518 \n",
       "L 144.964061 140.371084 \n",
       "L 144.964061 141.462651 \n",
       "L 144.964061 142.554217 \n",
       "L 144.123699 143.372892 \n",
       "L 143.843579 143.645783 \n",
       "L 143.843579 144.737349 \n",
       "L 143.843579 145.828916 \n",
       "L 143.843579 146.920482 \n",
       "L 143.843579 148.012048 \n",
       "L 143.843579 149.103614 \n",
       "L 143.843579 150.195181 \n",
       "L 143.843579 151.286747 \n",
       "L 143.843579 152.378313 \n",
       "L 143.003217 153.196988 \n",
       "L 142.723097 153.46988 \n",
       "L 142.723097 154.561446 \n",
       "L 142.723097 155.653012 \n",
       "L 142.723097 156.744578 \n",
       "L 142.723097 157.836145 \n",
       "L 142.723097 158.927711 \n",
       "L 142.723097 160.019277 \n",
       "L 142.723097 161.110843 \n",
       "L 142.723097 162.20241 \n",
       "L 141.882735 163.021084 \n",
       "L 141.602615 163.293976 \n",
       "L 141.602615 164.385542 \n",
       "L 141.602615 165.477108 \n",
       "L 141.602615 166.568675 \n",
       "L 141.602615 167.660241 \n",
       "L 141.602615 168.751807 \n",
       "L 141.602615 169.843373 \n",
       "L 141.602615 170.93494 \n",
       "L 141.602615 172.026506 \n",
       "L 140.762253 172.845181 \n",
       "L 140.482133 173.118072 \n",
       "L 140.482133 174.209639 \n",
       "L 140.482133 175.301205 \n",
       "L 140.482133 176.392771 \n",
       "L 140.482133 177.484337 \n",
       "L 140.482133 178.575904 \n",
       "L 140.482133 179.66747 \n",
       "L 140.482133 180.759036 \n",
       "L 140.482133 181.850602 \n",
       "L 139.641771 182.669277 \n",
       "L 139.361651 182.942169 \n",
       "L 139.361651 184.033735 \n",
       "L 139.361651 185.125301 \n",
       "L 139.361651 186.216867 \n",
       "L 139.361651 187.308434 \n",
       "L 139.361651 188.4 \n",
       "L 139.361651 189.491566 \n",
       "L 139.361651 190.583133 \n",
       "L 139.361651 191.674699 \n",
       "L 138.52129 192.493373 \n",
       "L 138.241169 192.766265 \n",
       "L 138.241169 193.857831 \n",
       "L 138.241169 194.949398 \n",
       "L 138.241169 196.040964 \n",
       "L 138.241169 197.13253 \n",
       "L 138.241169 198.224096 \n",
       "L 138.241169 199.315663 \n",
       "L 138.241169 200.407229 \n",
       "L 137.400808 201.225904 \n",
       "L 137.120687 201.498795 \n",
       "L 137.120687 202.590361 \n",
       "L 137.120687 203.681928 \n",
       "L 137.120687 204.773494 \n",
       "L 137.120687 205.86506 \n",
       "L 137.120687 206.956627 \n",
       "L 137.120687 208.048193 \n",
       "L 137.120687 209.139759 \n",
       "L 137.120687 210.231325 \n",
       "L 136.280326 211.05 \n",
       "L 136.000205 211.322892 \n",
       "L 136.000205 212.414458 \n",
       "L 136.000205 213.506024 \n",
       "L 136.000205 214.59759 \n",
       "L 136.000205 215.689157 \n",
       "L 136.000205 216.780723 \n",
       "L 136.000205 217.872289 \n",
       "L 136.000205 218.963855 \n",
       "L 136.000205 220.055422 \n",
       "L 135.159844 220.874096 \n",
       "L 134.879723 221.146988 \n",
       "L 134.879723 222.238554 \n",
       "L 134.879723 223.33012 \n",
       "L 134.879723 224.421687 \n",
       "L 134.879723 225.513253 \n",
       "L 134.879723 226.604819 \n",
       "L 134.879723 227.696386 \n",
       "L 134.879723 228.787952 \n",
       "L 134.879723 229.879518 \n",
       "L 134.039362 230.698193 \n",
       "L 133.759241 230.971084 \n",
       "L 133.759241 232.062651 \n",
       "L 133.759241 233.154217 \n",
       "L 133.759241 234.245783 \n",
       "L 133.759241 235.337349 \n",
       "L 133.759241 236.428916 \n",
       "L 133.759241 237.520482 \n",
       "L 133.759241 238.612048 \n",
       "L 133.759241 239.703614 \n",
       "L 132.91888 240.522289 \n",
       "L 132.638759 240.795181 \n",
       "L 132.638759 241.886747 \n",
       "L 132.638759 242.978313 \n",
       "L 132.638759 244.06988 \n",
       "L 132.638759 245.161446 \n",
       "L 132.638759 246.253012 \n",
       "L 132.638759 247.344578 \n",
       "L 132.638759 248.436145 \n",
       "L 132.638759 249.527711 \n",
       "L 131.798398 250.346386 \n",
       "L 131.518277 250.619277 \n",
       "L 131.518277 251.710843 \n",
       "L 131.518277 252.80241 \n",
       "L 131.518277 253.893976 \n",
       "L 131.518277 254.985542 \n",
       "L 131.518277 256.077108 \n",
       "L 131.518277 257.168675 \n",
       "L 131.518277 258.260241 \n",
       "L 131.518277 259.351807 \n",
       "L 130.677916 260.170482 \n",
       "L 130.397796 260.443373 \n",
       "L 130.397796 261.53494 \n",
       "L 130.397796 262.626506 \n",
       "L 130.397796 263.718072 \n",
       "L 130.397796 264.809639 \n",
       "L 130.397796 265.901205 \n",
       "L 130.397796 266.992771 \n",
       "L 130.397796 268.084337 \n",
       "L 129.557434 268.903012 \n",
       "L 129.277314 269.175904 \n",
       "L 129.277314 270.26747 \n",
       "L 129.277314 271.359036 \n",
       "L 129.277314 272.450602 \n",
       "L 129.277314 273.542169 \n",
       "L 129.277314 274.633735 \n",
       "L 129.277314 275.725301 \n",
       "L 129.277314 276.816867 \n",
       "L 129.277314 277.908434 \n",
       "L 128.436952 278.727108 \n",
       "L 128.156832 279 \n",
       "z\n",
       "\" style=\"fill:#9467bd;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_5\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 145.244181 133.821687 \n",
       "L 146.364663 133.821687 \n",
       "L 147.485145 132.73012 \n",
       "L 148.605627 132.73012 \n",
       "L 149.726109 132.73012 \n",
       "L 150.846591 132.73012 \n",
       "L 151.967073 131.638554 \n",
       "L 153.087555 131.638554 \n",
       "L 154.208037 131.638554 \n",
       "L 155.328518 131.638554 \n",
       "L 156.449 130.546988 \n",
       "L 157.569482 130.546988 \n",
       "L 158.689964 130.546988 \n",
       "L 159.810446 130.546988 \n",
       "L 160.930928 129.455422 \n",
       "L 162.05141 129.455422 \n",
       "L 163.171892 129.455422 \n",
       "L 164.292374 128.363855 \n",
       "L 165.412856 128.363855 \n",
       "L 166.533338 128.363855 \n",
       "L 167.65382 128.363855 \n",
       "L 168.774302 127.272289 \n",
       "L 169.894784 127.272289 \n",
       "L 171.015265 127.272289 \n",
       "L 172.135747 127.272289 \n",
       "L 173.256229 126.180723 \n",
       "L 174.376711 126.180723 \n",
       "L 175.497193 126.180723 \n",
       "L 176.617675 126.180723 \n",
       "L 177.738157 125.089157 \n",
       "L 178.858639 125.089157 \n",
       "L 179.979121 125.089157 \n",
       "L 181.099603 125.089157 \n",
       "L 182.220085 123.99759 \n",
       "L 183.340567 123.99759 \n",
       "L 184.461049 123.99759 \n",
       "L 185.58153 123.99759 \n",
       "L 186.702012 122.906024 \n",
       "L 187.822494 122.906024 \n",
       "L 188.942976 122.906024 \n",
       "L 190.063458 122.906024 \n",
       "L 191.18394 121.814458 \n",
       "L 192.304422 121.814458 \n",
       "L 193.424904 121.814458 \n",
       "L 194.545386 121.814458 \n",
       "L 195.665868 120.722892 \n",
       "L 196.78635 120.722892 \n",
       "L 197.906832 120.722892 \n",
       "L 199.027314 119.631325 \n",
       "L 200.147796 119.631325 \n",
       "L 201.268277 119.631325 \n",
       "L 202.388759 119.631325 \n",
       "L 203.509241 118.539759 \n",
       "L 204.629723 118.539759 \n",
       "L 205.750205 118.539759 \n",
       "L 206.870687 118.539759 \n",
       "L 207.991169 117.448193 \n",
       "L 209.111651 117.448193 \n",
       "L 210.232133 117.448193 \n",
       "L 211.352615 117.448193 \n",
       "L 212.473097 116.356627 \n",
       "L 213.593579 116.356627 \n",
       "L 214.714061 116.356627 \n",
       "L 215.834543 116.356627 \n",
       "L 216.955024 115.26506 \n",
       "L 218.075506 115.26506 \n",
       "L 219.195988 115.26506 \n",
       "L 220.31647 115.26506 \n",
       "L 221.436952 114.173494 \n",
       "L 222.557434 114.173494 \n",
       "L 223.677916 114.173494 \n",
       "L 224.798398 114.173494 \n",
       "L 225.91888 113.081928 \n",
       "L 227.039362 113.081928 \n",
       "L 228.159844 113.081928 \n",
       "L 229.280326 113.081928 \n",
       "L 230.400808 111.990361 \n",
       "L 231.52129 111.990361 \n",
       "L 232.641771 111.990361 \n",
       "L 233.762253 110.898795 \n",
       "L 234.882735 110.898795 \n",
       "L 236.003217 110.898795 \n",
       "L 237.123699 110.898795 \n",
       "L 238.244181 109.807229 \n",
       "L 239.364663 109.807229 \n",
       "L 240.485145 109.807229 \n",
       "L 241.605627 109.807229 \n",
       "L 242.726109 108.715663 \n",
       "L 243.846591 108.715663 \n",
       "L 244.967073 108.715663 \n",
       "L 246.087555 108.715663 \n",
       "L 247.208037 107.624096 \n",
       "L 248.328518 107.624096 \n",
       "L 249.449 107.624096 \n",
       "L 250.569482 107.624096 \n",
       "L 251.689964 106.53253 \n",
       "L 252.810446 106.53253 \n",
       "L 253.930928 106.53253 \n",
       "L 255.05141 106.53253 \n",
       "L 256.171892 105.440964 \n",
       "L 257.292374 105.440964 \n",
       "L 258.412856 105.440964 \n",
       "L 259.533338 105.440964 \n",
       "L 260.65382 104.349398 \n",
       "L 261.774302 104.349398 \n",
       "L 262.894784 104.349398 \n",
       "L 264.015265 104.349398 \n",
       "L 265.135747 103.257831 \n",
       "L 266.256229 103.257831 \n",
       "L 267.376711 103.257831 \n",
       "L 268.497193 102.166265 \n",
       "L 269.617675 102.166265 \n",
       "L 270.738157 102.166265 \n",
       "L 271.858639 102.166265 \n",
       "L 272.979121 101.074699 \n",
       "L 274.099603 101.074699 \n",
       "L 275.220085 101.074699 \n",
       "L 276.340567 101.074699 \n",
       "L 277.461049 99.983133 \n",
       "L 278.58153 99.983133 \n",
       "L 279.702012 99.983133 \n",
       "L 280.822494 99.983133 \n",
       "L 281.942976 98.891566 \n",
       "L 283.063458 98.891566 \n",
       "L 284.18394 98.891566 \n",
       "L 285.304422 98.891566 \n",
       "L 286.424904 97.8 \n",
       "L 287.545386 97.8 \n",
       "L 288.665868 97.8 \n",
       "L 289.78635 97.8 \n",
       "L 290.906832 96.708434 \n",
       "L 292.027314 96.708434 \n",
       "L 293.147796 96.708434 \n",
       "L 294.268277 96.708434 \n",
       "L 295.388759 95.616867 \n",
       "L 296.509241 95.616867 \n",
       "L 297.629723 95.616867 \n",
       "L 298.750205 95.616867 \n",
       "L 299.870687 94.525301 \n",
       "L 300.991169 94.525301 \n",
       "L 302.111651 94.525301 \n",
       "L 303.232133 93.433735 \n",
       "L 304.352615 93.433735 \n",
       "L 305.473097 93.433735 \n",
       "L 306.593579 93.433735 \n",
       "L 307.714061 92.342169 \n",
       "L 308.834543 92.342169 \n",
       "L 309.955024 92.342169 \n",
       "L 311.075506 92.342169 \n",
       "L 312.195988 91.250602 \n",
       "L 313.31647 91.250602 \n",
       "L 314.436952 91.250602 \n",
       "L 315.557434 91.250602 \n",
       "L 316.677916 90.159036 \n",
       "L 317.798398 90.159036 \n",
       "L 318.91888 90.159036 \n",
       "L 320.039362 90.159036 \n",
       "L 321.159844 89.06747 \n",
       "L 321.159844 88.794578 \n",
       "L 320.879723 89.06747 \n",
       "L 320.039362 89.886145 \n",
       "L 318.91888 89.886145 \n",
       "L 317.798398 89.886145 \n",
       "L 316.677916 89.886145 \n",
       "L 316.397796 90.159036 \n",
       "L 315.557434 90.977711 \n",
       "L 314.436952 90.977711 \n",
       "L 313.31647 90.977711 \n",
       "L 312.195988 90.977711 \n",
       "L 311.915868 91.250602 \n",
       "L 311.075506 92.069277 \n",
       "L 309.955024 92.069277 \n",
       "L 308.834543 92.069277 \n",
       "L 307.714061 92.069277 \n",
       "L 307.43394 92.342169 \n",
       "L 306.593579 93.160843 \n",
       "L 305.473097 93.160843 \n",
       "L 304.352615 93.160843 \n",
       "L 303.232133 93.160843 \n",
       "L 302.952012 93.433735 \n",
       "L 302.111651 94.25241 \n",
       "L 300.991169 94.25241 \n",
       "L 299.870687 94.25241 \n",
       "L 299.590567 94.525301 \n",
       "L 298.750205 95.343976 \n",
       "L 297.629723 95.343976 \n",
       "L 296.509241 95.343976 \n",
       "L 295.388759 95.343976 \n",
       "L 295.108639 95.616867 \n",
       "L 294.268277 96.435542 \n",
       "L 293.147796 96.435542 \n",
       "L 292.027314 96.435542 \n",
       "L 290.906832 96.435542 \n",
       "L 290.626711 96.708434 \n",
       "L 289.78635 97.527108 \n",
       "L 288.665868 97.527108 \n",
       "L 287.545386 97.527108 \n",
       "L 286.424904 97.527108 \n",
       "L 286.144784 97.8 \n",
       "L 285.304422 98.618675 \n",
       "L 284.18394 98.618675 \n",
       "L 283.063458 98.618675 \n",
       "L 281.942976 98.618675 \n",
       "L 281.662856 98.891566 \n",
       "L 280.822494 99.710241 \n",
       "L 279.702012 99.710241 \n",
       "L 278.58153 99.710241 \n",
       "L 277.461049 99.710241 \n",
       "L 277.180928 99.983133 \n",
       "L 276.340567 100.801807 \n",
       "L 275.220085 100.801807 \n",
       "L 274.099603 100.801807 \n",
       "L 272.979121 100.801807 \n",
       "L 272.699 101.074699 \n",
       "L 271.858639 101.893373 \n",
       "L 270.738157 101.893373 \n",
       "L 269.617675 101.893373 \n",
       "L 268.497193 101.893373 \n",
       "L 268.217073 102.166265 \n",
       "L 267.376711 102.98494 \n",
       "L 266.256229 102.98494 \n",
       "L 265.135747 102.98494 \n",
       "L 264.855627 103.257831 \n",
       "L 264.015265 104.076506 \n",
       "L 262.894784 104.076506 \n",
       "L 261.774302 104.076506 \n",
       "L 260.65382 104.076506 \n",
       "L 260.373699 104.349398 \n",
       "L 259.533338 105.168072 \n",
       "L 258.412856 105.168072 \n",
       "L 257.292374 105.168072 \n",
       "L 256.171892 105.168072 \n",
       "L 255.891771 105.440964 \n",
       "L 255.05141 106.259639 \n",
       "L 253.930928 106.259639 \n",
       "L 252.810446 106.259639 \n",
       "L 251.689964 106.259639 \n",
       "L 251.409844 106.53253 \n",
       "L 250.569482 107.351205 \n",
       "L 249.449 107.351205 \n",
       "L 248.328518 107.351205 \n",
       "L 247.208037 107.351205 \n",
       "L 246.927916 107.624096 \n",
       "L 246.087555 108.442771 \n",
       "L 244.967073 108.442771 \n",
       "L 243.846591 108.442771 \n",
       "L 242.726109 108.442771 \n",
       "L 242.445988 108.715663 \n",
       "L 241.605627 109.534337 \n",
       "L 240.485145 109.534337 \n",
       "L 239.364663 109.534337 \n",
       "L 238.244181 109.534337 \n",
       "L 237.964061 109.807229 \n",
       "L 237.123699 110.625904 \n",
       "L 236.003217 110.625904 \n",
       "L 234.882735 110.625904 \n",
       "L 233.762253 110.625904 \n",
       "L 233.482133 110.898795 \n",
       "L 232.641771 111.71747 \n",
       "L 231.52129 111.71747 \n",
       "L 230.400808 111.71747 \n",
       "L 230.120687 111.990361 \n",
       "L 229.280326 112.809036 \n",
       "L 228.159844 112.809036 \n",
       "L 227.039362 112.809036 \n",
       "L 225.91888 112.809036 \n",
       "L 225.638759 113.081928 \n",
       "L 224.798398 113.900602 \n",
       "L 223.677916 113.900602 \n",
       "L 222.557434 113.900602 \n",
       "L 221.436952 113.900602 \n",
       "L 221.156832 114.173494 \n",
       "L 220.31647 114.992169 \n",
       "L 219.195988 114.992169 \n",
       "L 218.075506 114.992169 \n",
       "L 216.955024 114.992169 \n",
       "L 216.674904 115.26506 \n",
       "L 215.834543 116.083735 \n",
       "L 214.714061 116.083735 \n",
       "L 213.593579 116.083735 \n",
       "L 212.473097 116.083735 \n",
       "L 212.192976 116.356627 \n",
       "L 211.352615 117.175301 \n",
       "L 210.232133 117.175301 \n",
       "L 209.111651 117.175301 \n",
       "L 207.991169 117.175301 \n",
       "L 207.711049 117.448193 \n",
       "L 206.870687 118.266867 \n",
       "L 205.750205 118.266867 \n",
       "L 204.629723 118.266867 \n",
       "L 203.509241 118.266867 \n",
       "L 203.229121 118.539759 \n",
       "L 202.388759 119.358434 \n",
       "L 201.268277 119.358434 \n",
       "L 200.147796 119.358434 \n",
       "L 199.027314 119.358434 \n",
       "L 198.747193 119.631325 \n",
       "L 197.906832 120.45 \n",
       "L 196.78635 120.45 \n",
       "L 195.665868 120.45 \n",
       "L 195.385747 120.722892 \n",
       "L 194.545386 121.541566 \n",
       "L 193.424904 121.541566 \n",
       "L 192.304422 121.541566 \n",
       "L 191.18394 121.541566 \n",
       "L 190.90382 121.814458 \n",
       "L 190.063458 122.633133 \n",
       "L 188.942976 122.633133 \n",
       "L 187.822494 122.633133 \n",
       "L 186.702012 122.633133 \n",
       "L 186.421892 122.906024 \n",
       "L 185.58153 123.724699 \n",
       "L 184.461049 123.724699 \n",
       "L 183.340567 123.724699 \n",
       "L 182.220085 123.724699 \n",
       "L 181.939964 123.99759 \n",
       "L 181.099603 124.816265 \n",
       "L 179.979121 124.816265 \n",
       "L 178.858639 124.816265 \n",
       "L 177.738157 124.816265 \n",
       "L 177.458037 125.089157 \n",
       "L 176.617675 125.907831 \n",
       "L 175.497193 125.907831 \n",
       "L 174.376711 125.907831 \n",
       "L 173.256229 125.907831 \n",
       "L 172.976109 126.180723 \n",
       "L 172.135747 126.999398 \n",
       "L 171.015265 126.999398 \n",
       "L 169.894784 126.999398 \n",
       "L 168.774302 126.999398 \n",
       "L 168.494181 127.272289 \n",
       "L 167.65382 128.090964 \n",
       "L 166.533338 128.090964 \n",
       "L 165.412856 128.090964 \n",
       "L 164.292374 128.090964 \n",
       "L 164.012253 128.363855 \n",
       "L 163.171892 129.18253 \n",
       "L 162.05141 129.18253 \n",
       "L 160.930928 129.18253 \n",
       "L 160.650808 129.455422 \n",
       "L 159.810446 130.274096 \n",
       "L 158.689964 130.274096 \n",
       "L 157.569482 130.274096 \n",
       "L 156.449 130.274096 \n",
       "L 156.16888 130.546988 \n",
       "L 155.328518 131.365663 \n",
       "L 154.208037 131.365663 \n",
       "L 153.087555 131.365663 \n",
       "L 151.967073 131.365663 \n",
       "L 151.686952 131.638554 \n",
       "L 150.846591 132.457229 \n",
       "L 149.726109 132.457229 \n",
       "L 148.605627 132.457229 \n",
       "L 147.485145 132.457229 \n",
       "L 147.205024 132.73012 \n",
       "L 146.364663 133.548795 \n",
       "L 145.244181 133.548795 \n",
       "L 144.824 132.73012 \n",
       "L 144.123699 132.047892 \n",
       "L 143.703518 131.638554 \n",
       "L 143.003217 130.956325 \n",
       "L 142.583037 130.546988 \n",
       "L 141.882735 129.864759 \n",
       "L 141.462555 129.455422 \n",
       "L 140.762253 128.773193 \n",
       "L 140.342073 128.363855 \n",
       "L 139.641771 127.681627 \n",
       "L 138.52129 127.681627 \n",
       "L 138.101109 127.272289 \n",
       "L 137.400808 126.59006 \n",
       "L 136.980627 126.180723 \n",
       "L 136.280326 125.498494 \n",
       "L 135.860145 125.089157 \n",
       "L 135.159844 124.406928 \n",
       "L 134.739663 123.99759 \n",
       "L 134.039362 123.315361 \n",
       "L 133.619181 122.906024 \n",
       "L 132.91888 122.223795 \n",
       "L 132.498699 121.814458 \n",
       "L 131.798398 121.132229 \n",
       "L 131.378217 120.722892 \n",
       "L 130.677916 120.040663 \n",
       "L 130.257735 119.631325 \n",
       "L 129.557434 118.949096 \n",
       "L 128.436952 118.949096 \n",
       "L 128.016771 118.539759 \n",
       "L 127.31647 117.85753 \n",
       "L 126.89629 117.448193 \n",
       "L 126.195988 116.765964 \n",
       "L 125.775808 116.356627 \n",
       "L 125.075506 115.674398 \n",
       "L 124.655326 115.26506 \n",
       "L 123.955024 114.582831 \n",
       "L 123.534844 114.173494 \n",
       "L 122.834543 113.491265 \n",
       "L 122.414362 113.081928 \n",
       "L 121.714061 112.399699 \n",
       "L 121.29388 111.990361 \n",
       "L 120.593579 111.308133 \n",
       "L 119.473097 111.308133 \n",
       "L 119.052916 110.898795 \n",
       "L 118.352615 110.216566 \n",
       "L 117.932434 109.807229 \n",
       "L 117.232133 109.125 \n",
       "L 116.811952 108.715663 \n",
       "L 116.111651 108.033434 \n",
       "L 115.69147 107.624096 \n",
       "L 114.991169 106.941867 \n",
       "L 114.570988 106.53253 \n",
       "L 113.870687 105.850301 \n",
       "L 113.450506 105.440964 \n",
       "L 112.750205 104.758735 \n",
       "L 112.330024 104.349398 \n",
       "L 111.629723 103.667169 \n",
       "L 110.509241 103.667169 \n",
       "L 110.089061 103.257831 \n",
       "L 109.388759 102.575602 \n",
       "L 108.968579 102.166265 \n",
       "L 108.268277 101.484036 \n",
       "L 107.848097 101.074699 \n",
       "L 107.147796 100.39247 \n",
       "L 106.727615 99.983133 \n",
       "L 106.027314 99.300904 \n",
       "L 105.607133 98.891566 \n",
       "L 104.906832 98.209337 \n",
       "L 104.486651 97.8 \n",
       "L 103.78635 97.117771 \n",
       "L 103.366169 96.708434 \n",
       "L 102.665868 96.026205 \n",
       "L 101.545386 96.026205 \n",
       "L 101.125205 95.616867 \n",
       "L 100.424904 94.934639 \n",
       "L 100.004723 94.525301 \n",
       "L 99.304422 93.843072 \n",
       "L 98.884241 93.433735 \n",
       "L 98.18394 92.751506 \n",
       "L 97.763759 92.342169 \n",
       "L 97.063458 91.65994 \n",
       "L 96.643277 91.250602 \n",
       "L 95.942976 90.568373 \n",
       "L 95.522796 90.159036 \n",
       "L 94.822494 89.476807 \n",
       "L 94.402314 89.06747 \n",
       "L 93.702012 88.385241 \n",
       "L 92.58153 88.385241 \n",
       "L 92.16135 87.975904 \n",
       "L 91.461049 87.293675 \n",
       "L 91.040868 86.884337 \n",
       "L 90.340567 86.202108 \n",
       "L 89.920386 85.792771 \n",
       "L 89.220085 85.110542 \n",
       "L 88.799904 84.701205 \n",
       "L 88.099603 84.018976 \n",
       "L 87.679422 83.609639 \n",
       "L 86.979121 82.92741 \n",
       "L 86.55894 82.518072 \n",
       "L 85.858639 81.835843 \n",
       "L 85.438458 81.426506 \n",
       "L 84.738157 80.744277 \n",
       "L 83.617675 80.744277 \n",
       "L 83.197494 80.33494 \n",
       "L 82.497193 79.652711 \n",
       "L 82.077012 79.243373 \n",
       "L 81.376711 78.561145 \n",
       "L 80.95653 78.151807 \n",
       "L 80.256229 77.469578 \n",
       "L 79.836049 77.060241 \n",
       "L 79.135747 76.378012 \n",
       "L 78.715567 75.968675 \n",
       "L 78.015265 75.286446 \n",
       "L 77.595085 74.877108 \n",
       "L 76.894784 74.19488 \n",
       "L 76.474603 73.785542 \n",
       "L 75.774302 73.103313 \n",
       "L 74.65382 73.103313 \n",
       "L 74.233639 72.693976 \n",
       "L 73.533338 72.011747 \n",
       "L 73.113157 71.60241 \n",
       "L 72.412856 70.920181 \n",
       "L 71.992675 70.510843 \n",
       "L 71.292374 69.828614 \n",
       "L 70.872193 69.419277 \n",
       "L 70.171892 68.737048 \n",
       "L 69.751711 68.327711 \n",
       "L 69.05141 67.645482 \n",
       "L 68.631229 67.236145 \n",
       "L 67.930928 66.553916 \n",
       "L 67.510747 66.144578 \n",
       "L 66.810446 65.462349 \n",
       "L 65.689964 65.462349 \n",
       "L 65.269784 65.053012 \n",
       "L 64.569482 64.370783 \n",
       "L 64.149302 63.961446 \n",
       "L 63.449 63.279217 \n",
       "L 63.02882 62.86988 \n",
       "L 62.328518 62.187651 \n",
       "L 61.908338 61.778313 \n",
       "L 61.208037 61.096084 \n",
       "L 60.787856 60.686747 \n",
       "L 60.087555 60.004518 \n",
       "L 59.667374 59.595181 \n",
       "L 58.967073 58.912952 \n",
       "L 58.546892 58.503614 \n",
       "L 57.846591 57.821386 \n",
       "L 57.42641 57.412048 \n",
       "L 56.726109 56.729819 \n",
       "L 55.605627 56.729819 \n",
       "L 55.185446 56.320482 \n",
       "L 54.485145 55.638253 \n",
       "L 54.064964 55.228916 \n",
       "L 53.364663 54.546687 \n",
       "L 52.944482 54.137349 \n",
       "L 52.244181 53.45512 \n",
       "L 51.824 53.045783 \n",
       "L 51.123699 52.363554 \n",
       "L 50.703518 51.954217 \n",
       "L 50.003217 51.271988 \n",
       "L 49.583037 50.862651 \n",
       "L 48.882735 50.180422 \n",
       "L 48.462555 49.771084 \n",
       "L 47.762253 49.088855 \n",
       "L 46.641771 49.088855 \n",
       "L 46.221591 48.679518 \n",
       "L 45.52129 47.997289 \n",
       "L 45.101109 47.587952 \n",
       "L 44.400808 46.905723 \n",
       "L 43.980627 46.496386 \n",
       "L 43.280326 45.814157 \n",
       "L 42.860145 45.404819 \n",
       "L 42.159844 44.72259 \n",
       "L 42.159844 44.859036 \n",
       "L 42.720085 45.404819 \n",
       "L 43.280326 45.950602 \n",
       "L 43.840567 46.496386 \n",
       "L 44.400808 47.042169 \n",
       "L 44.961049 47.587952 \n",
       "L 45.52129 48.133735 \n",
       "L 46.08153 48.679518 \n",
       "L 46.641771 49.225301 \n",
       "L 47.762253 49.225301 \n",
       "L 48.322494 49.771084 \n",
       "L 48.882735 50.316867 \n",
       "L 49.442976 50.862651 \n",
       "L 50.003217 51.408434 \n",
       "L 50.563458 51.954217 \n",
       "L 51.123699 52.5 \n",
       "L 51.68394 53.045783 \n",
       "L 52.244181 53.591566 \n",
       "L 52.804422 54.137349 \n",
       "L 53.364663 54.683133 \n",
       "L 53.924904 55.228916 \n",
       "L 54.485145 55.774699 \n",
       "L 55.045386 56.320482 \n",
       "L 55.605627 56.866265 \n",
       "L 56.726109 56.866265 \n",
       "L 57.28635 57.412048 \n",
       "L 57.846591 57.957831 \n",
       "L 58.406832 58.503614 \n",
       "L 58.967073 59.049398 \n",
       "L 59.527314 59.595181 \n",
       "L 60.087555 60.140964 \n",
       "L 60.647796 60.686747 \n",
       "L 61.208037 61.23253 \n",
       "L 61.768277 61.778313 \n",
       "L 62.328518 62.324096 \n",
       "L 62.888759 62.86988 \n",
       "L 63.449 63.415663 \n",
       "L 64.009241 63.961446 \n",
       "L 64.569482 64.507229 \n",
       "L 65.129723 65.053012 \n",
       "L 65.689964 65.598795 \n",
       "L 66.810446 65.598795 \n",
       "L 67.370687 66.144578 \n",
       "L 67.930928 66.690361 \n",
       "L 68.491169 67.236145 \n",
       "L 69.05141 67.781928 \n",
       "L 69.611651 68.327711 \n",
       "L 70.171892 68.873494 \n",
       "L 70.732133 69.419277 \n",
       "L 71.292374 69.96506 \n",
       "L 71.852615 70.510843 \n",
       "L 72.412856 71.056627 \n",
       "L 72.973097 71.60241 \n",
       "L 73.533338 72.148193 \n",
       "L 74.093579 72.693976 \n",
       "L 74.65382 73.239759 \n",
       "L 75.774302 73.239759 \n",
       "L 76.334543 73.785542 \n",
       "L 76.894784 74.331325 \n",
       "L 77.455024 74.877108 \n",
       "L 78.015265 75.422892 \n",
       "L 78.575506 75.968675 \n",
       "L 79.135747 76.514458 \n",
       "L 79.695988 77.060241 \n",
       "L 80.256229 77.606024 \n",
       "L 80.81647 78.151807 \n",
       "L 81.376711 78.69759 \n",
       "L 81.936952 79.243373 \n",
       "L 82.497193 79.789157 \n",
       "L 83.057434 80.33494 \n",
       "L 83.617675 80.880723 \n",
       "L 84.738157 80.880723 \n",
       "L 85.298398 81.426506 \n",
       "L 85.858639 81.972289 \n",
       "L 86.41888 82.518072 \n",
       "L 86.979121 83.063855 \n",
       "L 87.539362 83.609639 \n",
       "L 88.099603 84.155422 \n",
       "L 88.659844 84.701205 \n",
       "L 89.220085 85.246988 \n",
       "L 89.780326 85.792771 \n",
       "L 90.340567 86.338554 \n",
       "L 90.900808 86.884337 \n",
       "L 91.461049 87.43012 \n",
       "L 92.02129 87.975904 \n",
       "L 92.58153 88.521687 \n",
       "L 93.702012 88.521687 \n",
       "L 94.262253 89.06747 \n",
       "L 94.822494 89.613253 \n",
       "L 95.382735 90.159036 \n",
       "L 95.942976 90.704819 \n",
       "L 96.503217 91.250602 \n",
       "L 97.063458 91.796386 \n",
       "L 97.623699 92.342169 \n",
       "L 98.18394 92.887952 \n",
       "L 98.744181 93.433735 \n",
       "L 99.304422 93.979518 \n",
       "L 99.864663 94.525301 \n",
       "L 100.424904 95.071084 \n",
       "L 100.985145 95.616867 \n",
       "L 101.545386 96.162651 \n",
       "L 102.665868 96.162651 \n",
       "L 103.226109 96.708434 \n",
       "L 103.78635 97.254217 \n",
       "L 104.346591 97.8 \n",
       "L 104.906832 98.345783 \n",
       "L 105.467073 98.891566 \n",
       "L 106.027314 99.437349 \n",
       "L 106.587555 99.983133 \n",
       "L 107.147796 100.528916 \n",
       "L 107.708037 101.074699 \n",
       "L 108.268277 101.620482 \n",
       "L 108.828518 102.166265 \n",
       "L 109.388759 102.712048 \n",
       "L 109.949 103.257831 \n",
       "L 110.509241 103.803614 \n",
       "L 111.629723 103.803614 \n",
       "L 112.189964 104.349398 \n",
       "L 112.750205 104.895181 \n",
       "L 113.310446 105.440964 \n",
       "L 113.870687 105.986747 \n",
       "L 114.430928 106.53253 \n",
       "L 114.991169 107.078313 \n",
       "L 115.55141 107.624096 \n",
       "L 116.111651 108.16988 \n",
       "L 116.671892 108.715663 \n",
       "L 117.232133 109.261446 \n",
       "L 117.792374 109.807229 \n",
       "L 118.352615 110.353012 \n",
       "L 118.912856 110.898795 \n",
       "L 119.473097 111.444578 \n",
       "L 120.593579 111.444578 \n",
       "L 121.15382 111.990361 \n",
       "L 121.714061 112.536145 \n",
       "L 122.274302 113.081928 \n",
       "L 122.834543 113.627711 \n",
       "L 123.394784 114.173494 \n",
       "L 123.955024 114.719277 \n",
       "L 124.515265 115.26506 \n",
       "L 125.075506 115.810843 \n",
       "L 125.635747 116.356627 \n",
       "L 126.195988 116.90241 \n",
       "L 126.756229 117.448193 \n",
       "L 127.31647 117.993976 \n",
       "L 127.876711 118.539759 \n",
       "L 128.436952 119.085542 \n",
       "L 129.557434 119.085542 \n",
       "L 130.117675 119.631325 \n",
       "L 130.677916 120.177108 \n",
       "L 131.238157 120.722892 \n",
       "L 131.798398 121.268675 \n",
       "L 132.358639 121.814458 \n",
       "L 132.91888 122.360241 \n",
       "L 133.479121 122.906024 \n",
       "L 134.039362 123.451807 \n",
       "L 134.599603 123.99759 \n",
       "L 135.159844 124.543373 \n",
       "L 135.720085 125.089157 \n",
       "L 136.280326 125.63494 \n",
       "L 136.840567 126.180723 \n",
       "L 137.400808 126.726506 \n",
       "L 137.961049 127.272289 \n",
       "L 138.52129 127.818072 \n",
       "L 139.641771 127.818072 \n",
       "L 140.202012 128.363855 \n",
       "L 140.762253 128.909639 \n",
       "L 141.322494 129.455422 \n",
       "L 141.882735 130.001205 \n",
       "L 142.442976 130.546988 \n",
       "L 143.003217 131.092771 \n",
       "L 143.563458 131.638554 \n",
       "L 144.123699 132.184337 \n",
       "L 144.68394 132.73012 \n",
       "z\n",
       "\" style=\"fill:#8c564b;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_6\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 145.244181 133.548795 \n",
       "L 146.364663 133.548795 \n",
       "L 147.205024 132.73012 \n",
       "L 147.485145 132.457229 \n",
       "L 148.605627 132.457229 \n",
       "L 149.726109 132.457229 \n",
       "L 150.846591 132.457229 \n",
       "L 151.686952 131.638554 \n",
       "L 151.967073 131.365663 \n",
       "L 153.087555 131.365663 \n",
       "L 154.208037 131.365663 \n",
       "L 155.328518 131.365663 \n",
       "L 156.16888 130.546988 \n",
       "L 156.449 130.274096 \n",
       "L 157.569482 130.274096 \n",
       "L 158.689964 130.274096 \n",
       "L 159.810446 130.274096 \n",
       "L 160.650808 129.455422 \n",
       "L 160.930928 129.18253 \n",
       "L 162.05141 129.18253 \n",
       "L 163.171892 129.18253 \n",
       "L 164.012253 128.363855 \n",
       "L 164.292374 128.090964 \n",
       "L 165.412856 128.090964 \n",
       "L 166.533338 128.090964 \n",
       "L 167.65382 128.090964 \n",
       "L 168.494181 127.272289 \n",
       "L 168.774302 126.999398 \n",
       "L 169.894784 126.999398 \n",
       "L 171.015265 126.999398 \n",
       "L 172.135747 126.999398 \n",
       "L 172.976109 126.180723 \n",
       "L 173.256229 125.907831 \n",
       "L 174.376711 125.907831 \n",
       "L 175.497193 125.907831 \n",
       "L 176.617675 125.907831 \n",
       "L 177.458037 125.089157 \n",
       "L 177.738157 124.816265 \n",
       "L 178.858639 124.816265 \n",
       "L 179.979121 124.816265 \n",
       "L 181.099603 124.816265 \n",
       "L 181.939964 123.99759 \n",
       "L 182.220085 123.724699 \n",
       "L 183.340567 123.724699 \n",
       "L 184.461049 123.724699 \n",
       "L 185.58153 123.724699 \n",
       "L 186.421892 122.906024 \n",
       "L 186.702012 122.633133 \n",
       "L 187.822494 122.633133 \n",
       "L 188.942976 122.633133 \n",
       "L 190.063458 122.633133 \n",
       "L 190.90382 121.814458 \n",
       "L 191.18394 121.541566 \n",
       "L 192.304422 121.541566 \n",
       "L 193.424904 121.541566 \n",
       "L 194.545386 121.541566 \n",
       "L 195.385747 120.722892 \n",
       "L 195.665868 120.45 \n",
       "L 196.78635 120.45 \n",
       "L 197.906832 120.45 \n",
       "L 198.747193 119.631325 \n",
       "L 199.027314 119.358434 \n",
       "L 200.147796 119.358434 \n",
       "L 201.268277 119.358434 \n",
       "L 202.388759 119.358434 \n",
       "L 203.229121 118.539759 \n",
       "L 203.509241 118.266867 \n",
       "L 204.629723 118.266867 \n",
       "L 205.750205 118.266867 \n",
       "L 206.870687 118.266867 \n",
       "L 207.711049 117.448193 \n",
       "L 207.991169 117.175301 \n",
       "L 209.111651 117.175301 \n",
       "L 210.232133 117.175301 \n",
       "L 211.352615 117.175301 \n",
       "L 212.192976 116.356627 \n",
       "L 212.473097 116.083735 \n",
       "L 213.593579 116.083735 \n",
       "L 214.714061 116.083735 \n",
       "L 215.834543 116.083735 \n",
       "L 216.674904 115.26506 \n",
       "L 216.955024 114.992169 \n",
       "L 218.075506 114.992169 \n",
       "L 219.195988 114.992169 \n",
       "L 220.31647 114.992169 \n",
       "L 221.156832 114.173494 \n",
       "L 221.436952 113.900602 \n",
       "L 222.557434 113.900602 \n",
       "L 223.677916 113.900602 \n",
       "L 224.798398 113.900602 \n",
       "L 225.638759 113.081928 \n",
       "L 225.91888 112.809036 \n",
       "L 227.039362 112.809036 \n",
       "L 228.159844 112.809036 \n",
       "L 229.280326 112.809036 \n",
       "L 230.120687 111.990361 \n",
       "L 230.400808 111.71747 \n",
       "L 231.52129 111.71747 \n",
       "L 232.641771 111.71747 \n",
       "L 233.482133 110.898795 \n",
       "L 233.762253 110.625904 \n",
       "L 234.882735 110.625904 \n",
       "L 236.003217 110.625904 \n",
       "L 237.123699 110.625904 \n",
       "L 237.964061 109.807229 \n",
       "L 238.244181 109.534337 \n",
       "L 239.364663 109.534337 \n",
       "L 240.485145 109.534337 \n",
       "L 241.605627 109.534337 \n",
       "L 242.445988 108.715663 \n",
       "L 242.726109 108.442771 \n",
       "L 243.846591 108.442771 \n",
       "L 244.967073 108.442771 \n",
       "L 246.087555 108.442771 \n",
       "L 246.927916 107.624096 \n",
       "L 247.208037 107.351205 \n",
       "L 248.328518 107.351205 \n",
       "L 249.449 107.351205 \n",
       "L 250.569482 107.351205 \n",
       "L 251.409844 106.53253 \n",
       "L 251.689964 106.259639 \n",
       "L 252.810446 106.259639 \n",
       "L 253.930928 106.259639 \n",
       "L 255.05141 106.259639 \n",
       "L 255.891771 105.440964 \n",
       "L 256.171892 105.168072 \n",
       "L 257.292374 105.168072 \n",
       "L 258.412856 105.168072 \n",
       "L 259.533338 105.168072 \n",
       "L 260.373699 104.349398 \n",
       "L 260.65382 104.076506 \n",
       "L 261.774302 104.076506 \n",
       "L 262.894784 104.076506 \n",
       "L 264.015265 104.076506 \n",
       "L 264.855627 103.257831 \n",
       "L 265.135747 102.98494 \n",
       "L 266.256229 102.98494 \n",
       "L 267.376711 102.98494 \n",
       "L 268.217073 102.166265 \n",
       "L 268.497193 101.893373 \n",
       "L 269.617675 101.893373 \n",
       "L 270.738157 101.893373 \n",
       "L 271.858639 101.893373 \n",
       "L 272.699 101.074699 \n",
       "L 272.979121 100.801807 \n",
       "L 274.099603 100.801807 \n",
       "L 275.220085 100.801807 \n",
       "L 276.340567 100.801807 \n",
       "L 277.180928 99.983133 \n",
       "L 277.461049 99.710241 \n",
       "L 278.58153 99.710241 \n",
       "L 279.702012 99.710241 \n",
       "L 280.822494 99.710241 \n",
       "L 281.662856 98.891566 \n",
       "L 281.942976 98.618675 \n",
       "L 283.063458 98.618675 \n",
       "L 284.18394 98.618675 \n",
       "L 285.304422 98.618675 \n",
       "L 286.144784 97.8 \n",
       "L 286.424904 97.527108 \n",
       "L 287.545386 97.527108 \n",
       "L 288.665868 97.527108 \n",
       "L 289.78635 97.527108 \n",
       "L 290.626711 96.708434 \n",
       "L 290.906832 96.435542 \n",
       "L 292.027314 96.435542 \n",
       "L 293.147796 96.435542 \n",
       "L 294.268277 96.435542 \n",
       "L 295.108639 95.616867 \n",
       "L 295.388759 95.343976 \n",
       "L 296.509241 95.343976 \n",
       "L 297.629723 95.343976 \n",
       "L 298.750205 95.343976 \n",
       "L 299.590567 94.525301 \n",
       "L 299.870687 94.25241 \n",
       "L 300.991169 94.25241 \n",
       "L 302.111651 94.25241 \n",
       "L 302.952012 93.433735 \n",
       "L 303.232133 93.160843 \n",
       "L 304.352615 93.160843 \n",
       "L 305.473097 93.160843 \n",
       "L 306.593579 93.160843 \n",
       "L 307.43394 92.342169 \n",
       "L 307.714061 92.069277 \n",
       "L 308.834543 92.069277 \n",
       "L 309.955024 92.069277 \n",
       "L 311.075506 92.069277 \n",
       "L 311.915868 91.250602 \n",
       "L 312.195988 90.977711 \n",
       "L 313.31647 90.977711 \n",
       "L 314.436952 90.977711 \n",
       "L 315.557434 90.977711 \n",
       "L 316.397796 90.159036 \n",
       "L 316.677916 89.886145 \n",
       "L 317.798398 89.886145 \n",
       "L 318.91888 89.886145 \n",
       "L 320.039362 89.886145 \n",
       "L 320.879723 89.06747 \n",
       "L 321.159844 88.794578 \n",
       "L 321.159844 88.521687 \n",
       "L 320.599603 89.06747 \n",
       "L 320.039362 89.613253 \n",
       "L 318.91888 89.613253 \n",
       "L 317.798398 89.613253 \n",
       "L 316.677916 89.613253 \n",
       "L 316.117675 90.159036 \n",
       "L 315.557434 90.704819 \n",
       "L 314.436952 90.704819 \n",
       "L 313.31647 90.704819 \n",
       "L 312.195988 90.704819 \n",
       "L 311.635747 91.250602 \n",
       "L 311.075506 91.796386 \n",
       "L 309.955024 91.796386 \n",
       "L 308.834543 91.796386 \n",
       "L 307.714061 91.796386 \n",
       "L 307.15382 92.342169 \n",
       "L 306.593579 92.887952 \n",
       "L 305.473097 92.887952 \n",
       "L 304.352615 92.887952 \n",
       "L 303.232133 92.887952 \n",
       "L 302.671892 93.433735 \n",
       "L 302.111651 93.979518 \n",
       "L 300.991169 93.979518 \n",
       "L 299.870687 93.979518 \n",
       "L 299.310446 94.525301 \n",
       "L 298.750205 95.071084 \n",
       "L 297.629723 95.071084 \n",
       "L 296.509241 95.071084 \n",
       "L 295.388759 95.071084 \n",
       "L 294.828518 95.616867 \n",
       "L 294.268277 96.162651 \n",
       "L 293.147796 96.162651 \n",
       "L 292.027314 96.162651 \n",
       "L 290.906832 96.162651 \n",
       "L 290.346591 96.708434 \n",
       "L 289.78635 97.254217 \n",
       "L 288.665868 97.254217 \n",
       "L 287.545386 97.254217 \n",
       "L 286.424904 97.254217 \n",
       "L 285.864663 97.8 \n",
       "L 285.304422 98.345783 \n",
       "L 284.18394 98.345783 \n",
       "L 283.063458 98.345783 \n",
       "L 281.942976 98.345783 \n",
       "L 281.382735 98.891566 \n",
       "L 280.822494 99.437349 \n",
       "L 279.702012 99.437349 \n",
       "L 278.58153 99.437349 \n",
       "L 277.461049 99.437349 \n",
       "L 276.900808 99.983133 \n",
       "L 276.340567 100.528916 \n",
       "L 275.220085 100.528916 \n",
       "L 274.099603 100.528916 \n",
       "L 272.979121 100.528916 \n",
       "L 272.41888 101.074699 \n",
       "L 271.858639 101.620482 \n",
       "L 270.738157 101.620482 \n",
       "L 269.617675 101.620482 \n",
       "L 268.497193 101.620482 \n",
       "L 267.936952 102.166265 \n",
       "L 267.376711 102.712048 \n",
       "L 266.256229 102.712048 \n",
       "L 265.135747 102.712048 \n",
       "L 264.575506 103.257831 \n",
       "L 264.015265 103.803614 \n",
       "L 262.894784 103.803614 \n",
       "L 261.774302 103.803614 \n",
       "L 260.65382 103.803614 \n",
       "L 260.093579 104.349398 \n",
       "L 259.533338 104.895181 \n",
       "L 258.412856 104.895181 \n",
       "L 257.292374 104.895181 \n",
       "L 256.171892 104.895181 \n",
       "L 255.611651 105.440964 \n",
       "L 255.05141 105.986747 \n",
       "L 253.930928 105.986747 \n",
       "L 252.810446 105.986747 \n",
       "L 251.689964 105.986747 \n",
       "L 251.129723 106.53253 \n",
       "L 250.569482 107.078313 \n",
       "L 249.449 107.078313 \n",
       "L 248.328518 107.078313 \n",
       "L 247.208037 107.078313 \n",
       "L 246.647796 107.624096 \n",
       "L 246.087555 108.16988 \n",
       "L 244.967073 108.16988 \n",
       "L 243.846591 108.16988 \n",
       "L 242.726109 108.16988 \n",
       "L 242.165868 108.715663 \n",
       "L 241.605627 109.261446 \n",
       "L 240.485145 109.261446 \n",
       "L 239.364663 109.261446 \n",
       "L 238.244181 109.261446 \n",
       "L 237.68394 109.807229 \n",
       "L 237.123699 110.353012 \n",
       "L 236.003217 110.353012 \n",
       "L 234.882735 110.353012 \n",
       "L 233.762253 110.353012 \n",
       "L 233.202012 110.898795 \n",
       "L 232.641771 111.444578 \n",
       "L 231.52129 111.444578 \n",
       "L 230.400808 111.444578 \n",
       "L 229.840567 111.990361 \n",
       "L 229.280326 112.536145 \n",
       "L 228.159844 112.536145 \n",
       "L 227.039362 112.536145 \n",
       "L 225.91888 112.536145 \n",
       "L 225.358639 113.081928 \n",
       "L 224.798398 113.627711 \n",
       "L 223.677916 113.627711 \n",
       "L 222.557434 113.627711 \n",
       "L 221.436952 113.627711 \n",
       "L 220.876711 114.173494 \n",
       "L 220.31647 114.719277 \n",
       "L 219.195988 114.719277 \n",
       "L 218.075506 114.719277 \n",
       "L 216.955024 114.719277 \n",
       "L 216.394784 115.26506 \n",
       "L 215.834543 115.810843 \n",
       "L 214.714061 115.810843 \n",
       "L 213.593579 115.810843 \n",
       "L 212.473097 115.810843 \n",
       "L 211.912856 116.356627 \n",
       "L 211.352615 116.90241 \n",
       "L 210.232133 116.90241 \n",
       "L 209.111651 116.90241 \n",
       "L 207.991169 116.90241 \n",
       "L 207.430928 117.448193 \n",
       "L 206.870687 117.993976 \n",
       "L 205.750205 117.993976 \n",
       "L 204.629723 117.993976 \n",
       "L 203.509241 117.993976 \n",
       "L 202.949 118.539759 \n",
       "L 202.388759 119.085542 \n",
       "L 201.268277 119.085542 \n",
       "L 200.147796 119.085542 \n",
       "L 199.027314 119.085542 \n",
       "L 198.467073 119.631325 \n",
       "L 197.906832 120.177108 \n",
       "L 196.78635 120.177108 \n",
       "L 195.665868 120.177108 \n",
       "L 195.105627 120.722892 \n",
       "L 194.545386 121.268675 \n",
       "L 193.424904 121.268675 \n",
       "L 192.304422 121.268675 \n",
       "L 191.18394 121.268675 \n",
       "L 190.623699 121.814458 \n",
       "L 190.063458 122.360241 \n",
       "L 188.942976 122.360241 \n",
       "L 187.822494 122.360241 \n",
       "L 186.702012 122.360241 \n",
       "L 186.141771 122.906024 \n",
       "L 185.58153 123.451807 \n",
       "L 184.461049 123.451807 \n",
       "L 183.340567 123.451807 \n",
       "L 182.220085 123.451807 \n",
       "L 181.659844 123.99759 \n",
       "L 181.099603 124.543373 \n",
       "L 179.979121 124.543373 \n",
       "L 178.858639 124.543373 \n",
       "L 177.738157 124.543373 \n",
       "L 177.177916 125.089157 \n",
       "L 176.617675 125.63494 \n",
       "L 175.497193 125.63494 \n",
       "L 174.376711 125.63494 \n",
       "L 173.256229 125.63494 \n",
       "L 172.695988 126.180723 \n",
       "L 172.135747 126.726506 \n",
       "L 171.015265 126.726506 \n",
       "L 169.894784 126.726506 \n",
       "L 168.774302 126.726506 \n",
       "L 168.214061 127.272289 \n",
       "L 167.65382 127.818072 \n",
       "L 166.533338 127.818072 \n",
       "L 165.412856 127.818072 \n",
       "L 164.292374 127.818072 \n",
       "L 163.732133 128.363855 \n",
       "L 163.171892 128.909639 \n",
       "L 162.05141 128.909639 \n",
       "L 160.930928 128.909639 \n",
       "L 160.370687 129.455422 \n",
       "L 159.810446 130.001205 \n",
       "L 158.689964 130.001205 \n",
       "L 157.569482 130.001205 \n",
       "L 156.449 130.001205 \n",
       "L 155.888759 130.546988 \n",
       "L 155.328518 131.092771 \n",
       "L 154.208037 131.092771 \n",
       "L 153.087555 131.092771 \n",
       "L 151.967073 131.092771 \n",
       "L 151.406832 131.638554 \n",
       "L 150.846591 132.184337 \n",
       "L 149.726109 132.184337 \n",
       "L 148.605627 132.184337 \n",
       "L 147.485145 132.184337 \n",
       "L 146.924904 132.73012 \n",
       "L 146.364663 133.275904 \n",
       "L 145.244181 133.275904 \n",
       "L 144.964061 132.73012 \n",
       "L 144.123699 131.911446 \n",
       "L 143.843579 131.638554 \n",
       "L 143.003217 130.81988 \n",
       "L 142.723097 130.546988 \n",
       "L 141.882735 129.728313 \n",
       "L 141.602615 129.455422 \n",
       "L 140.762253 128.636747 \n",
       "L 140.482133 128.363855 \n",
       "L 139.641771 127.545181 \n",
       "L 138.52129 127.545181 \n",
       "L 138.241169 127.272289 \n",
       "L 137.400808 126.453614 \n",
       "L 137.120687 126.180723 \n",
       "L 136.280326 125.362048 \n",
       "L 136.000205 125.089157 \n",
       "L 135.159844 124.270482 \n",
       "L 134.879723 123.99759 \n",
       "L 134.039362 123.178916 \n",
       "L 133.759241 122.906024 \n",
       "L 132.91888 122.087349 \n",
       "L 132.638759 121.814458 \n",
       "L 131.798398 120.995783 \n",
       "L 131.518277 120.722892 \n",
       "L 130.677916 119.904217 \n",
       "L 130.397796 119.631325 \n",
       "L 129.557434 118.812651 \n",
       "L 128.436952 118.812651 \n",
       "L 128.156832 118.539759 \n",
       "L 127.31647 117.721084 \n",
       "L 127.03635 117.448193 \n",
       "L 126.195988 116.629518 \n",
       "L 125.915868 116.356627 \n",
       "L 125.075506 115.537952 \n",
       "L 124.795386 115.26506 \n",
       "L 123.955024 114.446386 \n",
       "L 123.674904 114.173494 \n",
       "L 122.834543 113.354819 \n",
       "L 122.554422 113.081928 \n",
       "L 121.714061 112.263253 \n",
       "L 121.43394 111.990361 \n",
       "L 120.593579 111.171687 \n",
       "L 119.473097 111.171687 \n",
       "L 119.192976 110.898795 \n",
       "L 118.352615 110.08012 \n",
       "L 118.072494 109.807229 \n",
       "L 117.232133 108.988554 \n",
       "L 116.952012 108.715663 \n",
       "L 116.111651 107.896988 \n",
       "L 115.83153 107.624096 \n",
       "L 114.991169 106.805422 \n",
       "L 114.711049 106.53253 \n",
       "L 113.870687 105.713855 \n",
       "L 113.590567 105.440964 \n",
       "L 112.750205 104.622289 \n",
       "L 112.470085 104.349398 \n",
       "L 111.629723 103.530723 \n",
       "L 110.509241 103.530723 \n",
       "L 110.229121 103.257831 \n",
       "L 109.388759 102.439157 \n",
       "L 109.108639 102.166265 \n",
       "L 108.268277 101.34759 \n",
       "L 107.988157 101.074699 \n",
       "L 107.147796 100.256024 \n",
       "L 106.867675 99.983133 \n",
       "L 106.027314 99.164458 \n",
       "L 105.747193 98.891566 \n",
       "L 104.906832 98.072892 \n",
       "L 104.626711 97.8 \n",
       "L 103.78635 96.981325 \n",
       "L 103.506229 96.708434 \n",
       "L 102.665868 95.889759 \n",
       "L 101.545386 95.889759 \n",
       "L 101.265265 95.616867 \n",
       "L 100.424904 94.798193 \n",
       "L 100.144784 94.525301 \n",
       "L 99.304422 93.706627 \n",
       "L 99.024302 93.433735 \n",
       "L 98.18394 92.61506 \n",
       "L 97.90382 92.342169 \n",
       "L 97.063458 91.523494 \n",
       "L 96.783338 91.250602 \n",
       "L 95.942976 90.431928 \n",
       "L 95.662856 90.159036 \n",
       "L 94.822494 89.340361 \n",
       "L 94.542374 89.06747 \n",
       "L 93.702012 88.248795 \n",
       "L 92.58153 88.248795 \n",
       "L 92.30141 87.975904 \n",
       "L 91.461049 87.157229 \n",
       "L 91.180928 86.884337 \n",
       "L 90.340567 86.065663 \n",
       "L 90.060446 85.792771 \n",
       "L 89.220085 84.974096 \n",
       "L 88.939964 84.701205 \n",
       "L 88.099603 83.88253 \n",
       "L 87.819482 83.609639 \n",
       "L 86.979121 82.790964 \n",
       "L 86.699 82.518072 \n",
       "L 85.858639 81.699398 \n",
       "L 85.578518 81.426506 \n",
       "L 84.738157 80.607831 \n",
       "L 83.617675 80.607831 \n",
       "L 83.337555 80.33494 \n",
       "L 82.497193 79.516265 \n",
       "L 82.217073 79.243373 \n",
       "L 81.376711 78.424699 \n",
       "L 81.096591 78.151807 \n",
       "L 80.256229 77.333133 \n",
       "L 79.976109 77.060241 \n",
       "L 79.135747 76.241566 \n",
       "L 78.855627 75.968675 \n",
       "L 78.015265 75.15 \n",
       "L 77.735145 74.877108 \n",
       "L 76.894784 74.058434 \n",
       "L 76.614663 73.785542 \n",
       "L 75.774302 72.966867 \n",
       "L 74.65382 72.966867 \n",
       "L 74.373699 72.693976 \n",
       "L 73.533338 71.875301 \n",
       "L 73.253217 71.60241 \n",
       "L 72.412856 70.783735 \n",
       "L 72.132735 70.510843 \n",
       "L 71.292374 69.692169 \n",
       "L 71.012253 69.419277 \n",
       "L 70.171892 68.600602 \n",
       "L 69.891771 68.327711 \n",
       "L 69.05141 67.509036 \n",
       "L 68.77129 67.236145 \n",
       "L 67.930928 66.41747 \n",
       "L 67.650808 66.144578 \n",
       "L 66.810446 65.325904 \n",
       "L 65.689964 65.325904 \n",
       "L 65.409844 65.053012 \n",
       "L 64.569482 64.234337 \n",
       "L 64.289362 63.961446 \n",
       "L 63.449 63.142771 \n",
       "L 63.16888 62.86988 \n",
       "L 62.328518 62.051205 \n",
       "L 62.048398 61.778313 \n",
       "L 61.208037 60.959639 \n",
       "L 60.927916 60.686747 \n",
       "L 60.087555 59.868072 \n",
       "L 59.807434 59.595181 \n",
       "L 58.967073 58.776506 \n",
       "L 58.686952 58.503614 \n",
       "L 57.846591 57.68494 \n",
       "L 57.56647 57.412048 \n",
       "L 56.726109 56.593373 \n",
       "L 55.605627 56.593373 \n",
       "L 55.325506 56.320482 \n",
       "L 54.485145 55.501807 \n",
       "L 54.205024 55.228916 \n",
       "L 53.364663 54.410241 \n",
       "L 53.084543 54.137349 \n",
       "L 52.244181 53.318675 \n",
       "L 51.964061 53.045783 \n",
       "L 51.123699 52.227108 \n",
       "L 50.843579 51.954217 \n",
       "L 50.003217 51.135542 \n",
       "L 49.723097 50.862651 \n",
       "L 48.882735 50.043976 \n",
       "L 48.602615 49.771084 \n",
       "L 47.762253 48.95241 \n",
       "L 46.641771 48.95241 \n",
       "L 46.361651 48.679518 \n",
       "L 45.52129 47.860843 \n",
       "L 45.241169 47.587952 \n",
       "L 44.400808 46.769277 \n",
       "L 44.120687 46.496386 \n",
       "L 43.280326 45.677711 \n",
       "L 43.000205 45.404819 \n",
       "L 42.159844 44.586145 \n",
       "L 42.159844 44.72259 \n",
       "L 42.860145 45.404819 \n",
       "L 43.280326 45.814157 \n",
       "L 43.980627 46.496386 \n",
       "L 44.400808 46.905723 \n",
       "L 45.101109 47.587952 \n",
       "L 45.52129 47.997289 \n",
       "L 46.221591 48.679518 \n",
       "L 46.641771 49.088855 \n",
       "L 47.762253 49.088855 \n",
       "L 48.462555 49.771084 \n",
       "L 48.882735 50.180422 \n",
       "L 49.583037 50.862651 \n",
       "L 50.003217 51.271988 \n",
       "L 50.703518 51.954217 \n",
       "L 51.123699 52.363554 \n",
       "L 51.824 53.045783 \n",
       "L 52.244181 53.45512 \n",
       "L 52.944482 54.137349 \n",
       "L 53.364663 54.546687 \n",
       "L 54.064964 55.228916 \n",
       "L 54.485145 55.638253 \n",
       "L 55.185446 56.320482 \n",
       "L 55.605627 56.729819 \n",
       "L 56.726109 56.729819 \n",
       "L 57.42641 57.412048 \n",
       "L 57.846591 57.821386 \n",
       "L 58.546892 58.503614 \n",
       "L 58.967073 58.912952 \n",
       "L 59.667374 59.595181 \n",
       "L 60.087555 60.004518 \n",
       "L 60.787856 60.686747 \n",
       "L 61.208037 61.096084 \n",
       "L 61.908338 61.778313 \n",
       "L 62.328518 62.187651 \n",
       "L 63.02882 62.86988 \n",
       "L 63.449 63.279217 \n",
       "L 64.149302 63.961446 \n",
       "L 64.569482 64.370783 \n",
       "L 65.269784 65.053012 \n",
       "L 65.689964 65.462349 \n",
       "L 66.810446 65.462349 \n",
       "L 67.510747 66.144578 \n",
       "L 67.930928 66.553916 \n",
       "L 68.631229 67.236145 \n",
       "L 69.05141 67.645482 \n",
       "L 69.751711 68.327711 \n",
       "L 70.171892 68.737048 \n",
       "L 70.872193 69.419277 \n",
       "L 71.292374 69.828614 \n",
       "L 71.992675 70.510843 \n",
       "L 72.412856 70.920181 \n",
       "L 73.113157 71.60241 \n",
       "L 73.533338 72.011747 \n",
       "L 74.233639 72.693976 \n",
       "L 74.65382 73.103313 \n",
       "L 75.774302 73.103313 \n",
       "L 76.474603 73.785542 \n",
       "L 76.894784 74.19488 \n",
       "L 77.595085 74.877108 \n",
       "L 78.015265 75.286446 \n",
       "L 78.715567 75.968675 \n",
       "L 79.135747 76.378012 \n",
       "L 79.836049 77.060241 \n",
       "L 80.256229 77.469578 \n",
       "L 80.95653 78.151807 \n",
       "L 81.376711 78.561145 \n",
       "L 82.077012 79.243373 \n",
       "L 82.497193 79.652711 \n",
       "L 83.197494 80.33494 \n",
       "L 83.617675 80.744277 \n",
       "L 84.738157 80.744277 \n",
       "L 85.438458 81.426506 \n",
       "L 85.858639 81.835843 \n",
       "L 86.55894 82.518072 \n",
       "L 86.979121 82.92741 \n",
       "L 87.679422 83.609639 \n",
       "L 88.099603 84.018976 \n",
       "L 88.799904 84.701205 \n",
       "L 89.220085 85.110542 \n",
       "L 89.920386 85.792771 \n",
       "L 90.340567 86.202108 \n",
       "L 91.040868 86.884337 \n",
       "L 91.461049 87.293675 \n",
       "L 92.16135 87.975904 \n",
       "L 92.58153 88.385241 \n",
       "L 93.702012 88.385241 \n",
       "L 94.402314 89.06747 \n",
       "L 94.822494 89.476807 \n",
       "L 95.522796 90.159036 \n",
       "L 95.942976 90.568373 \n",
       "L 96.643277 91.250602 \n",
       "L 97.063458 91.65994 \n",
       "L 97.763759 92.342169 \n",
       "L 98.18394 92.751506 \n",
       "L 98.884241 93.433735 \n",
       "L 99.304422 93.843072 \n",
       "L 100.004723 94.525301 \n",
       "L 100.424904 94.934639 \n",
       "L 101.125205 95.616867 \n",
       "L 101.545386 96.026205 \n",
       "L 102.665868 96.026205 \n",
       "L 103.366169 96.708434 \n",
       "L 103.78635 97.117771 \n",
       "L 104.486651 97.8 \n",
       "L 104.906832 98.209337 \n",
       "L 105.607133 98.891566 \n",
       "L 106.027314 99.300904 \n",
       "L 106.727615 99.983133 \n",
       "L 107.147796 100.39247 \n",
       "L 107.848097 101.074699 \n",
       "L 108.268277 101.484036 \n",
       "L 108.968579 102.166265 \n",
       "L 109.388759 102.575602 \n",
       "L 110.089061 103.257831 \n",
       "L 110.509241 103.667169 \n",
       "L 111.629723 103.667169 \n",
       "L 112.330024 104.349398 \n",
       "L 112.750205 104.758735 \n",
       "L 113.450506 105.440964 \n",
       "L 113.870687 105.850301 \n",
       "L 114.570988 106.53253 \n",
       "L 114.991169 106.941867 \n",
       "L 115.69147 107.624096 \n",
       "L 116.111651 108.033434 \n",
       "L 116.811952 108.715663 \n",
       "L 117.232133 109.125 \n",
       "L 117.932434 109.807229 \n",
       "L 118.352615 110.216566 \n",
       "L 119.052916 110.898795 \n",
       "L 119.473097 111.308133 \n",
       "L 120.593579 111.308133 \n",
       "L 121.29388 111.990361 \n",
       "L 121.714061 112.399699 \n",
       "L 122.414362 113.081928 \n",
       "L 122.834543 113.491265 \n",
       "L 123.534844 114.173494 \n",
       "L 123.955024 114.582831 \n",
       "L 124.655326 115.26506 \n",
       "L 125.075506 115.674398 \n",
       "L 125.775808 116.356627 \n",
       "L 126.195988 116.765964 \n",
       "L 126.89629 117.448193 \n",
       "L 127.31647 117.85753 \n",
       "L 128.016771 118.539759 \n",
       "L 128.436952 118.949096 \n",
       "L 129.557434 118.949096 \n",
       "L 130.257735 119.631325 \n",
       "L 130.677916 120.040663 \n",
       "L 131.378217 120.722892 \n",
       "L 131.798398 121.132229 \n",
       "L 132.498699 121.814458 \n",
       "L 132.91888 122.223795 \n",
       "L 133.619181 122.906024 \n",
       "L 134.039362 123.315361 \n",
       "L 134.739663 123.99759 \n",
       "L 135.159844 124.406928 \n",
       "L 135.860145 125.089157 \n",
       "L 136.280326 125.498494 \n",
       "L 136.980627 126.180723 \n",
       "L 137.400808 126.59006 \n",
       "L 138.101109 127.272289 \n",
       "L 138.52129 127.681627 \n",
       "L 139.641771 127.681627 \n",
       "L 140.342073 128.363855 \n",
       "L 140.762253 128.773193 \n",
       "L 141.462555 129.455422 \n",
       "L 141.882735 129.864759 \n",
       "L 142.583037 130.546988 \n",
       "L 143.003217 130.956325 \n",
       "L 143.703518 131.638554 \n",
       "L 144.123699 132.047892 \n",
       "L 144.824 132.73012 \n",
       "z\n",
       "\" style=\"fill:#e377c2;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_7\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 145.244181 133.275904 \n",
       "L 146.364663 133.275904 \n",
       "L 146.924904 132.73012 \n",
       "L 147.485145 132.184337 \n",
       "L 148.605627 132.184337 \n",
       "L 149.726109 132.184337 \n",
       "L 150.846591 132.184337 \n",
       "L 151.406832 131.638554 \n",
       "L 151.967073 131.092771 \n",
       "L 153.087555 131.092771 \n",
       "L 154.208037 131.092771 \n",
       "L 155.328518 131.092771 \n",
       "L 155.888759 130.546988 \n",
       "L 156.449 130.001205 \n",
       "L 157.569482 130.001205 \n",
       "L 158.689964 130.001205 \n",
       "L 159.810446 130.001205 \n",
       "L 160.370687 129.455422 \n",
       "L 160.930928 128.909639 \n",
       "L 162.05141 128.909639 \n",
       "L 163.171892 128.909639 \n",
       "L 163.732133 128.363855 \n",
       "L 164.292374 127.818072 \n",
       "L 165.412856 127.818072 \n",
       "L 166.533338 127.818072 \n",
       "L 167.65382 127.818072 \n",
       "L 168.214061 127.272289 \n",
       "L 168.774302 126.726506 \n",
       "L 169.894784 126.726506 \n",
       "L 171.015265 126.726506 \n",
       "L 172.135747 126.726506 \n",
       "L 172.695988 126.180723 \n",
       "L 173.256229 125.63494 \n",
       "L 174.376711 125.63494 \n",
       "L 175.497193 125.63494 \n",
       "L 176.617675 125.63494 \n",
       "L 177.177916 125.089157 \n",
       "L 177.738157 124.543373 \n",
       "L 178.858639 124.543373 \n",
       "L 179.979121 124.543373 \n",
       "L 181.099603 124.543373 \n",
       "L 181.659844 123.99759 \n",
       "L 182.220085 123.451807 \n",
       "L 183.340567 123.451807 \n",
       "L 184.461049 123.451807 \n",
       "L 185.58153 123.451807 \n",
       "L 186.141771 122.906024 \n",
       "L 186.702012 122.360241 \n",
       "L 187.822494 122.360241 \n",
       "L 188.942976 122.360241 \n",
       "L 190.063458 122.360241 \n",
       "L 190.623699 121.814458 \n",
       "L 191.18394 121.268675 \n",
       "L 192.304422 121.268675 \n",
       "L 193.424904 121.268675 \n",
       "L 194.545386 121.268675 \n",
       "L 195.105627 120.722892 \n",
       "L 195.665868 120.177108 \n",
       "L 196.78635 120.177108 \n",
       "L 197.906832 120.177108 \n",
       "L 198.467073 119.631325 \n",
       "L 199.027314 119.085542 \n",
       "L 200.147796 119.085542 \n",
       "L 201.268277 119.085542 \n",
       "L 202.388759 119.085542 \n",
       "L 202.949 118.539759 \n",
       "L 203.509241 117.993976 \n",
       "L 204.629723 117.993976 \n",
       "L 205.750205 117.993976 \n",
       "L 206.870687 117.993976 \n",
       "L 207.430928 117.448193 \n",
       "L 207.991169 116.90241 \n",
       "L 209.111651 116.90241 \n",
       "L 210.232133 116.90241 \n",
       "L 211.352615 116.90241 \n",
       "L 211.912856 116.356627 \n",
       "L 212.473097 115.810843 \n",
       "L 213.593579 115.810843 \n",
       "L 214.714061 115.810843 \n",
       "L 215.834543 115.810843 \n",
       "L 216.394784 115.26506 \n",
       "L 216.955024 114.719277 \n",
       "L 218.075506 114.719277 \n",
       "L 219.195988 114.719277 \n",
       "L 220.31647 114.719277 \n",
       "L 220.876711 114.173494 \n",
       "L 221.436952 113.627711 \n",
       "L 222.557434 113.627711 \n",
       "L 223.677916 113.627711 \n",
       "L 224.798398 113.627711 \n",
       "L 225.358639 113.081928 \n",
       "L 225.91888 112.536145 \n",
       "L 227.039362 112.536145 \n",
       "L 228.159844 112.536145 \n",
       "L 229.280326 112.536145 \n",
       "L 229.840567 111.990361 \n",
       "L 230.400808 111.444578 \n",
       "L 231.52129 111.444578 \n",
       "L 232.641771 111.444578 \n",
       "L 233.202012 110.898795 \n",
       "L 233.762253 110.353012 \n",
       "L 234.882735 110.353012 \n",
       "L 236.003217 110.353012 \n",
       "L 237.123699 110.353012 \n",
       "L 237.68394 109.807229 \n",
       "L 238.244181 109.261446 \n",
       "L 239.364663 109.261446 \n",
       "L 240.485145 109.261446 \n",
       "L 241.605627 109.261446 \n",
       "L 242.165868 108.715663 \n",
       "L 242.726109 108.16988 \n",
       "L 243.846591 108.16988 \n",
       "L 244.967073 108.16988 \n",
       "L 246.087555 108.16988 \n",
       "L 246.647796 107.624096 \n",
       "L 247.208037 107.078313 \n",
       "L 248.328518 107.078313 \n",
       "L 249.449 107.078313 \n",
       "L 250.569482 107.078313 \n",
       "L 251.129723 106.53253 \n",
       "L 251.689964 105.986747 \n",
       "L 252.810446 105.986747 \n",
       "L 253.930928 105.986747 \n",
       "L 255.05141 105.986747 \n",
       "L 255.611651 105.440964 \n",
       "L 256.171892 104.895181 \n",
       "L 257.292374 104.895181 \n",
       "L 258.412856 104.895181 \n",
       "L 259.533338 104.895181 \n",
       "L 260.093579 104.349398 \n",
       "L 260.65382 103.803614 \n",
       "L 261.774302 103.803614 \n",
       "L 262.894784 103.803614 \n",
       "L 264.015265 103.803614 \n",
       "L 264.575506 103.257831 \n",
       "L 265.135747 102.712048 \n",
       "L 266.256229 102.712048 \n",
       "L 267.376711 102.712048 \n",
       "L 267.936952 102.166265 \n",
       "L 268.497193 101.620482 \n",
       "L 269.617675 101.620482 \n",
       "L 270.738157 101.620482 \n",
       "L 271.858639 101.620482 \n",
       "L 272.41888 101.074699 \n",
       "L 272.979121 100.528916 \n",
       "L 274.099603 100.528916 \n",
       "L 275.220085 100.528916 \n",
       "L 276.340567 100.528916 \n",
       "L 276.900808 99.983133 \n",
       "L 277.461049 99.437349 \n",
       "L 278.58153 99.437349 \n",
       "L 279.702012 99.437349 \n",
       "L 280.822494 99.437349 \n",
       "L 281.382735 98.891566 \n",
       "L 281.942976 98.345783 \n",
       "L 283.063458 98.345783 \n",
       "L 284.18394 98.345783 \n",
       "L 285.304422 98.345783 \n",
       "L 285.864663 97.8 \n",
       "L 286.424904 97.254217 \n",
       "L 287.545386 97.254217 \n",
       "L 288.665868 97.254217 \n",
       "L 289.78635 97.254217 \n",
       "L 290.346591 96.708434 \n",
       "L 290.906832 96.162651 \n",
       "L 292.027314 96.162651 \n",
       "L 293.147796 96.162651 \n",
       "L 294.268277 96.162651 \n",
       "L 294.828518 95.616867 \n",
       "L 295.388759 95.071084 \n",
       "L 296.509241 95.071084 \n",
       "L 297.629723 95.071084 \n",
       "L 298.750205 95.071084 \n",
       "L 299.310446 94.525301 \n",
       "L 299.870687 93.979518 \n",
       "L 300.991169 93.979518 \n",
       "L 302.111651 93.979518 \n",
       "L 302.671892 93.433735 \n",
       "L 303.232133 92.887952 \n",
       "L 304.352615 92.887952 \n",
       "L 305.473097 92.887952 \n",
       "L 306.593579 92.887952 \n",
       "L 307.15382 92.342169 \n",
       "L 307.714061 91.796386 \n",
       "L 308.834543 91.796386 \n",
       "L 309.955024 91.796386 \n",
       "L 311.075506 91.796386 \n",
       "L 311.635747 91.250602 \n",
       "L 312.195988 90.704819 \n",
       "L 313.31647 90.704819 \n",
       "L 314.436952 90.704819 \n",
       "L 315.557434 90.704819 \n",
       "L 316.117675 90.159036 \n",
       "L 316.677916 89.613253 \n",
       "L 317.798398 89.613253 \n",
       "L 318.91888 89.613253 \n",
       "L 320.039362 89.613253 \n",
       "L 320.599603 89.06747 \n",
       "L 321.159844 88.521687 \n",
       "L 321.159844 88.248795 \n",
       "L 320.319482 89.06747 \n",
       "L 320.039362 89.340361 \n",
       "L 318.91888 89.340361 \n",
       "L 317.798398 89.340361 \n",
       "L 316.677916 89.340361 \n",
       "L 315.837555 90.159036 \n",
       "L 315.557434 90.431928 \n",
       "L 314.436952 90.431928 \n",
       "L 313.31647 90.431928 \n",
       "L 312.195988 90.431928 \n",
       "L 311.355627 91.250602 \n",
       "L 311.075506 91.523494 \n",
       "L 309.955024 91.523494 \n",
       "L 308.834543 91.523494 \n",
       "L 307.714061 91.523494 \n",
       "L 306.873699 92.342169 \n",
       "L 306.593579 92.61506 \n",
       "L 305.473097 92.61506 \n",
       "L 304.352615 92.61506 \n",
       "L 303.232133 92.61506 \n",
       "L 302.391771 93.433735 \n",
       "L 302.111651 93.706627 \n",
       "L 300.991169 93.706627 \n",
       "L 299.870687 93.706627 \n",
       "L 299.030326 94.525301 \n",
       "L 298.750205 94.798193 \n",
       "L 297.629723 94.798193 \n",
       "L 296.509241 94.798193 \n",
       "L 295.388759 94.798193 \n",
       "L 294.548398 95.616867 \n",
       "L 294.268277 95.889759 \n",
       "L 293.147796 95.889759 \n",
       "L 292.027314 95.889759 \n",
       "L 290.906832 95.889759 \n",
       "L 290.06647 96.708434 \n",
       "L 289.78635 96.981325 \n",
       "L 288.665868 96.981325 \n",
       "L 287.545386 96.981325 \n",
       "L 286.424904 96.981325 \n",
       "L 285.584543 97.8 \n",
       "L 285.304422 98.072892 \n",
       "L 284.18394 98.072892 \n",
       "L 283.063458 98.072892 \n",
       "L 281.942976 98.072892 \n",
       "L 281.102615 98.891566 \n",
       "L 280.822494 99.164458 \n",
       "L 279.702012 99.164458 \n",
       "L 278.58153 99.164458 \n",
       "L 277.461049 99.164458 \n",
       "L 276.620687 99.983133 \n",
       "L 276.340567 100.256024 \n",
       "L 275.220085 100.256024 \n",
       "L 274.099603 100.256024 \n",
       "L 272.979121 100.256024 \n",
       "L 272.138759 101.074699 \n",
       "L 271.858639 101.34759 \n",
       "L 270.738157 101.34759 \n",
       "L 269.617675 101.34759 \n",
       "L 268.497193 101.34759 \n",
       "L 267.656832 102.166265 \n",
       "L 267.376711 102.439157 \n",
       "L 266.256229 102.439157 \n",
       "L 265.135747 102.439157 \n",
       "L 264.295386 103.257831 \n",
       "L 264.015265 103.530723 \n",
       "L 262.894784 103.530723 \n",
       "L 261.774302 103.530723 \n",
       "L 260.65382 103.530723 \n",
       "L 259.813458 104.349398 \n",
       "L 259.533338 104.622289 \n",
       "L 258.412856 104.622289 \n",
       "L 257.292374 104.622289 \n",
       "L 256.171892 104.622289 \n",
       "L 255.33153 105.440964 \n",
       "L 255.05141 105.713855 \n",
       "L 253.930928 105.713855 \n",
       "L 252.810446 105.713855 \n",
       "L 251.689964 105.713855 \n",
       "L 250.849603 106.53253 \n",
       "L 250.569482 106.805422 \n",
       "L 249.449 106.805422 \n",
       "L 248.328518 106.805422 \n",
       "L 247.208037 106.805422 \n",
       "L 246.367675 107.624096 \n",
       "L 246.087555 107.896988 \n",
       "L 244.967073 107.896988 \n",
       "L 243.846591 107.896988 \n",
       "L 242.726109 107.896988 \n",
       "L 241.885747 108.715663 \n",
       "L 241.605627 108.988554 \n",
       "L 240.485145 108.988554 \n",
       "L 239.364663 108.988554 \n",
       "L 238.244181 108.988554 \n",
       "L 237.40382 109.807229 \n",
       "L 237.123699 110.08012 \n",
       "L 236.003217 110.08012 \n",
       "L 234.882735 110.08012 \n",
       "L 233.762253 110.08012 \n",
       "L 232.921892 110.898795 \n",
       "L 232.641771 111.171687 \n",
       "L 231.52129 111.171687 \n",
       "L 230.400808 111.171687 \n",
       "L 229.560446 111.990361 \n",
       "L 229.280326 112.263253 \n",
       "L 228.159844 112.263253 \n",
       "L 227.039362 112.263253 \n",
       "L 225.91888 112.263253 \n",
       "L 225.078518 113.081928 \n",
       "L 224.798398 113.354819 \n",
       "L 223.677916 113.354819 \n",
       "L 222.557434 113.354819 \n",
       "L 221.436952 113.354819 \n",
       "L 220.596591 114.173494 \n",
       "L 220.31647 114.446386 \n",
       "L 219.195988 114.446386 \n",
       "L 218.075506 114.446386 \n",
       "L 216.955024 114.446386 \n",
       "L 216.114663 115.26506 \n",
       "L 215.834543 115.537952 \n",
       "L 214.714061 115.537952 \n",
       "L 213.593579 115.537952 \n",
       "L 212.473097 115.537952 \n",
       "L 211.632735 116.356627 \n",
       "L 211.352615 116.629518 \n",
       "L 210.232133 116.629518 \n",
       "L 209.111651 116.629518 \n",
       "L 207.991169 116.629518 \n",
       "L 207.150808 117.448193 \n",
       "L 206.870687 117.721084 \n",
       "L 205.750205 117.721084 \n",
       "L 204.629723 117.721084 \n",
       "L 203.509241 117.721084 \n",
       "L 202.66888 118.539759 \n",
       "L 202.388759 118.812651 \n",
       "L 201.268277 118.812651 \n",
       "L 200.147796 118.812651 \n",
       "L 199.027314 118.812651 \n",
       "L 198.186952 119.631325 \n",
       "L 197.906832 119.904217 \n",
       "L 196.78635 119.904217 \n",
       "L 195.665868 119.904217 \n",
       "L 194.825506 120.722892 \n",
       "L 194.545386 120.995783 \n",
       "L 193.424904 120.995783 \n",
       "L 192.304422 120.995783 \n",
       "L 191.18394 120.995783 \n",
       "L 190.343579 121.814458 \n",
       "L 190.063458 122.087349 \n",
       "L 188.942976 122.087349 \n",
       "L 187.822494 122.087349 \n",
       "L 186.702012 122.087349 \n",
       "L 185.861651 122.906024 \n",
       "L 185.58153 123.178916 \n",
       "L 184.461049 123.178916 \n",
       "L 183.340567 123.178916 \n",
       "L 182.220085 123.178916 \n",
       "L 181.379723 123.99759 \n",
       "L 181.099603 124.270482 \n",
       "L 179.979121 124.270482 \n",
       "L 178.858639 124.270482 \n",
       "L 177.738157 124.270482 \n",
       "L 176.897796 125.089157 \n",
       "L 176.617675 125.362048 \n",
       "L 175.497193 125.362048 \n",
       "L 174.376711 125.362048 \n",
       "L 173.256229 125.362048 \n",
       "L 172.415868 126.180723 \n",
       "L 172.135747 126.453614 \n",
       "L 171.015265 126.453614 \n",
       "L 169.894784 126.453614 \n",
       "L 168.774302 126.453614 \n",
       "L 167.93394 127.272289 \n",
       "L 167.65382 127.545181 \n",
       "L 166.533338 127.545181 \n",
       "L 165.412856 127.545181 \n",
       "L 164.292374 127.545181 \n",
       "L 163.452012 128.363855 \n",
       "L 163.171892 128.636747 \n",
       "L 162.05141 128.636747 \n",
       "L 160.930928 128.636747 \n",
       "L 160.090567 129.455422 \n",
       "L 159.810446 129.728313 \n",
       "L 158.689964 129.728313 \n",
       "L 157.569482 129.728313 \n",
       "L 156.449 129.728313 \n",
       "L 155.608639 130.546988 \n",
       "L 155.328518 130.81988 \n",
       "L 154.208037 130.81988 \n",
       "L 153.087555 130.81988 \n",
       "L 151.967073 130.81988 \n",
       "L 151.126711 131.638554 \n",
       "L 150.846591 131.911446 \n",
       "L 149.726109 131.911446 \n",
       "L 148.605627 131.911446 \n",
       "L 147.485145 131.911446 \n",
       "L 146.644784 132.73012 \n",
       "L 146.364663 133.003012 \n",
       "L 145.244181 133.003012 \n",
       "L 145.104121 132.73012 \n",
       "L 144.123699 131.775 \n",
       "L 143.983639 131.638554 \n",
       "L 143.003217 130.683434 \n",
       "L 142.863157 130.546988 \n",
       "L 141.882735 129.591867 \n",
       "L 141.742675 129.455422 \n",
       "L 140.762253 128.500301 \n",
       "L 140.622193 128.363855 \n",
       "L 139.641771 127.408735 \n",
       "L 138.52129 127.408735 \n",
       "L 138.381229 127.272289 \n",
       "L 137.400808 126.317169 \n",
       "L 137.260747 126.180723 \n",
       "L 136.280326 125.225602 \n",
       "L 136.140265 125.089157 \n",
       "L 135.159844 124.134036 \n",
       "L 135.019784 123.99759 \n",
       "L 134.039362 123.04247 \n",
       "L 133.899302 122.906024 \n",
       "L 132.91888 121.950904 \n",
       "L 132.77882 121.814458 \n",
       "L 131.798398 120.859337 \n",
       "L 131.658338 120.722892 \n",
       "L 130.677916 119.767771 \n",
       "L 130.537856 119.631325 \n",
       "L 129.557434 118.676205 \n",
       "L 128.436952 118.676205 \n",
       "L 128.296892 118.539759 \n",
       "L 127.31647 117.584639 \n",
       "L 127.17641 117.448193 \n",
       "L 126.195988 116.493072 \n",
       "L 126.055928 116.356627 \n",
       "L 125.075506 115.401506 \n",
       "L 124.935446 115.26506 \n",
       "L 123.955024 114.30994 \n",
       "L 123.814964 114.173494 \n",
       "L 122.834543 113.218373 \n",
       "L 122.694482 113.081928 \n",
       "L 121.714061 112.126807 \n",
       "L 121.574 111.990361 \n",
       "L 120.593579 111.035241 \n",
       "L 119.473097 111.035241 \n",
       "L 119.333037 110.898795 \n",
       "L 118.352615 109.943675 \n",
       "L 118.212555 109.807229 \n",
       "L 117.232133 108.852108 \n",
       "L 117.092073 108.715663 \n",
       "L 116.111651 107.760542 \n",
       "L 115.971591 107.624096 \n",
       "L 114.991169 106.668976 \n",
       "L 114.851109 106.53253 \n",
       "L 113.870687 105.57741 \n",
       "L 113.730627 105.440964 \n",
       "L 112.750205 104.485843 \n",
       "L 112.610145 104.349398 \n",
       "L 111.629723 103.394277 \n",
       "L 110.509241 103.394277 \n",
       "L 110.369181 103.257831 \n",
       "L 109.388759 102.302711 \n",
       "L 109.248699 102.166265 \n",
       "L 108.268277 101.211145 \n",
       "L 108.128217 101.074699 \n",
       "L 107.147796 100.119578 \n",
       "L 107.007735 99.983133 \n",
       "L 106.027314 99.028012 \n",
       "L 105.887253 98.891566 \n",
       "L 104.906832 97.936446 \n",
       "L 104.766771 97.8 \n",
       "L 103.78635 96.84488 \n",
       "L 103.64629 96.708434 \n",
       "L 102.665868 95.753313 \n",
       "L 101.545386 95.753313 \n",
       "L 101.405326 95.616867 \n",
       "L 100.424904 94.661747 \n",
       "L 100.284844 94.525301 \n",
       "L 99.304422 93.570181 \n",
       "L 99.164362 93.433735 \n",
       "L 98.18394 92.478614 \n",
       "L 98.04388 92.342169 \n",
       "L 97.063458 91.387048 \n",
       "L 96.923398 91.250602 \n",
       "L 95.942976 90.295482 \n",
       "L 95.802916 90.159036 \n",
       "L 94.822494 89.203916 \n",
       "L 94.682434 89.06747 \n",
       "L 93.702012 88.112349 \n",
       "L 92.58153 88.112349 \n",
       "L 92.44147 87.975904 \n",
       "L 91.461049 87.020783 \n",
       "L 91.320988 86.884337 \n",
       "L 90.340567 85.929217 \n",
       "L 90.200506 85.792771 \n",
       "L 89.220085 84.837651 \n",
       "L 89.080024 84.701205 \n",
       "L 88.099603 83.746084 \n",
       "L 87.959543 83.609639 \n",
       "L 86.979121 82.654518 \n",
       "L 86.839061 82.518072 \n",
       "L 85.858639 81.562952 \n",
       "L 85.718579 81.426506 \n",
       "L 84.738157 80.471386 \n",
       "L 83.617675 80.471386 \n",
       "L 83.477615 80.33494 \n",
       "L 82.497193 79.379819 \n",
       "L 82.357133 79.243373 \n",
       "L 81.376711 78.288253 \n",
       "L 81.236651 78.151807 \n",
       "L 80.256229 77.196687 \n",
       "L 80.116169 77.060241 \n",
       "L 79.135747 76.10512 \n",
       "L 78.995687 75.968675 \n",
       "L 78.015265 75.013554 \n",
       "L 77.875205 74.877108 \n",
       "L 76.894784 73.921988 \n",
       "L 76.754723 73.785542 \n",
       "L 75.774302 72.830422 \n",
       "L 74.65382 72.830422 \n",
       "L 74.513759 72.693976 \n",
       "L 73.533338 71.738855 \n",
       "L 73.393277 71.60241 \n",
       "L 72.412856 70.647289 \n",
       "L 72.272796 70.510843 \n",
       "L 71.292374 69.555723 \n",
       "L 71.152314 69.419277 \n",
       "L 70.171892 68.464157 \n",
       "L 70.031832 68.327711 \n",
       "L 69.05141 67.37259 \n",
       "L 68.91135 67.236145 \n",
       "L 67.930928 66.281024 \n",
       "L 67.790868 66.144578 \n",
       "L 66.810446 65.189458 \n",
       "L 65.689964 65.189458 \n",
       "L 65.549904 65.053012 \n",
       "L 64.569482 64.097892 \n",
       "L 64.429422 63.961446 \n",
       "L 63.449 63.006325 \n",
       "L 63.30894 62.86988 \n",
       "L 62.328518 61.914759 \n",
       "L 62.188458 61.778313 \n",
       "L 61.208037 60.823193 \n",
       "L 61.067976 60.686747 \n",
       "L 60.087555 59.731627 \n",
       "L 59.947494 59.595181 \n",
       "L 58.967073 58.64006 \n",
       "L 58.827012 58.503614 \n",
       "L 57.846591 57.548494 \n",
       "L 57.70653 57.412048 \n",
       "L 56.726109 56.456928 \n",
       "L 55.605627 56.456928 \n",
       "L 55.465567 56.320482 \n",
       "L 54.485145 55.365361 \n",
       "L 54.345085 55.228916 \n",
       "L 53.364663 54.273795 \n",
       "L 53.224603 54.137349 \n",
       "L 52.244181 53.182229 \n",
       "L 52.104121 53.045783 \n",
       "L 51.123699 52.090663 \n",
       "L 50.983639 51.954217 \n",
       "L 50.003217 50.999096 \n",
       "L 49.863157 50.862651 \n",
       "L 48.882735 49.90753 \n",
       "L 48.742675 49.771084 \n",
       "L 47.762253 48.815964 \n",
       "L 46.641771 48.815964 \n",
       "L 46.501711 48.679518 \n",
       "L 45.52129 47.724398 \n",
       "L 45.381229 47.587952 \n",
       "L 44.400808 46.632831 \n",
       "L 44.260747 46.496386 \n",
       "L 43.280326 45.541265 \n",
       "L 43.140265 45.404819 \n",
       "L 42.159844 44.449699 \n",
       "L 42.159844 44.586145 \n",
       "L 43.000205 45.404819 \n",
       "L 43.280326 45.677711 \n",
       "L 44.120687 46.496386 \n",
       "L 44.400808 46.769277 \n",
       "L 45.241169 47.587952 \n",
       "L 45.52129 47.860843 \n",
       "L 46.361651 48.679518 \n",
       "L 46.641771 48.95241 \n",
       "L 47.762253 48.95241 \n",
       "L 48.602615 49.771084 \n",
       "L 48.882735 50.043976 \n",
       "L 49.723097 50.862651 \n",
       "L 50.003217 51.135542 \n",
       "L 50.843579 51.954217 \n",
       "L 51.123699 52.227108 \n",
       "L 51.964061 53.045783 \n",
       "L 52.244181 53.318675 \n",
       "L 53.084543 54.137349 \n",
       "L 53.364663 54.410241 \n",
       "L 54.205024 55.228916 \n",
       "L 54.485145 55.501807 \n",
       "L 55.325506 56.320482 \n",
       "L 55.605627 56.593373 \n",
       "L 56.726109 56.593373 \n",
       "L 57.56647 57.412048 \n",
       "L 57.846591 57.68494 \n",
       "L 58.686952 58.503614 \n",
       "L 58.967073 58.776506 \n",
       "L 59.807434 59.595181 \n",
       "L 60.087555 59.868072 \n",
       "L 60.927916 60.686747 \n",
       "L 61.208037 60.959639 \n",
       "L 62.048398 61.778313 \n",
       "L 62.328518 62.051205 \n",
       "L 63.16888 62.86988 \n",
       "L 63.449 63.142771 \n",
       "L 64.289362 63.961446 \n",
       "L 64.569482 64.234337 \n",
       "L 65.409844 65.053012 \n",
       "L 65.689964 65.325904 \n",
       "L 66.810446 65.325904 \n",
       "L 67.650808 66.144578 \n",
       "L 67.930928 66.41747 \n",
       "L 68.77129 67.236145 \n",
       "L 69.05141 67.509036 \n",
       "L 69.891771 68.327711 \n",
       "L 70.171892 68.600602 \n",
       "L 71.012253 69.419277 \n",
       "L 71.292374 69.692169 \n",
       "L 72.132735 70.510843 \n",
       "L 72.412856 70.783735 \n",
       "L 73.253217 71.60241 \n",
       "L 73.533338 71.875301 \n",
       "L 74.373699 72.693976 \n",
       "L 74.65382 72.966867 \n",
       "L 75.774302 72.966867 \n",
       "L 76.614663 73.785542 \n",
       "L 76.894784 74.058434 \n",
       "L 77.735145 74.877108 \n",
       "L 78.015265 75.15 \n",
       "L 78.855627 75.968675 \n",
       "L 79.135747 76.241566 \n",
       "L 79.976109 77.060241 \n",
       "L 80.256229 77.333133 \n",
       "L 81.096591 78.151807 \n",
       "L 81.376711 78.424699 \n",
       "L 82.217073 79.243373 \n",
       "L 82.497193 79.516265 \n",
       "L 83.337555 80.33494 \n",
       "L 83.617675 80.607831 \n",
       "L 84.738157 80.607831 \n",
       "L 85.578518 81.426506 \n",
       "L 85.858639 81.699398 \n",
       "L 86.699 82.518072 \n",
       "L 86.979121 82.790964 \n",
       "L 87.819482 83.609639 \n",
       "L 88.099603 83.88253 \n",
       "L 88.939964 84.701205 \n",
       "L 89.220085 84.974096 \n",
       "L 90.060446 85.792771 \n",
       "L 90.340567 86.065663 \n",
       "L 91.180928 86.884337 \n",
       "L 91.461049 87.157229 \n",
       "L 92.30141 87.975904 \n",
       "L 92.58153 88.248795 \n",
       "L 93.702012 88.248795 \n",
       "L 94.542374 89.06747 \n",
       "L 94.822494 89.340361 \n",
       "L 95.662856 90.159036 \n",
       "L 95.942976 90.431928 \n",
       "L 96.783338 91.250602 \n",
       "L 97.063458 91.523494 \n",
       "L 97.90382 92.342169 \n",
       "L 98.18394 92.61506 \n",
       "L 99.024302 93.433735 \n",
       "L 99.304422 93.706627 \n",
       "L 100.144784 94.525301 \n",
       "L 100.424904 94.798193 \n",
       "L 101.265265 95.616867 \n",
       "L 101.545386 95.889759 \n",
       "L 102.665868 95.889759 \n",
       "L 103.506229 96.708434 \n",
       "L 103.78635 96.981325 \n",
       "L 104.626711 97.8 \n",
       "L 104.906832 98.072892 \n",
       "L 105.747193 98.891566 \n",
       "L 106.027314 99.164458 \n",
       "L 106.867675 99.983133 \n",
       "L 107.147796 100.256024 \n",
       "L 107.988157 101.074699 \n",
       "L 108.268277 101.34759 \n",
       "L 109.108639 102.166265 \n",
       "L 109.388759 102.439157 \n",
       "L 110.229121 103.257831 \n",
       "L 110.509241 103.530723 \n",
       "L 111.629723 103.530723 \n",
       "L 112.470085 104.349398 \n",
       "L 112.750205 104.622289 \n",
       "L 113.590567 105.440964 \n",
       "L 113.870687 105.713855 \n",
       "L 114.711049 106.53253 \n",
       "L 114.991169 106.805422 \n",
       "L 115.83153 107.624096 \n",
       "L 116.111651 107.896988 \n",
       "L 116.952012 108.715663 \n",
       "L 117.232133 108.988554 \n",
       "L 118.072494 109.807229 \n",
       "L 118.352615 110.08012 \n",
       "L 119.192976 110.898795 \n",
       "L 119.473097 111.171687 \n",
       "L 120.593579 111.171687 \n",
       "L 121.43394 111.990361 \n",
       "L 121.714061 112.263253 \n",
       "L 122.554422 113.081928 \n",
       "L 122.834543 113.354819 \n",
       "L 123.674904 114.173494 \n",
       "L 123.955024 114.446386 \n",
       "L 124.795386 115.26506 \n",
       "L 125.075506 115.537952 \n",
       "L 125.915868 116.356627 \n",
       "L 126.195988 116.629518 \n",
       "L 127.03635 117.448193 \n",
       "L 127.31647 117.721084 \n",
       "L 128.156832 118.539759 \n",
       "L 128.436952 118.812651 \n",
       "L 129.557434 118.812651 \n",
       "L 130.397796 119.631325 \n",
       "L 130.677916 119.904217 \n",
       "L 131.518277 120.722892 \n",
       "L 131.798398 120.995783 \n",
       "L 132.638759 121.814458 \n",
       "L 132.91888 122.087349 \n",
       "L 133.759241 122.906024 \n",
       "L 134.039362 123.178916 \n",
       "L 134.879723 123.99759 \n",
       "L 135.159844 124.270482 \n",
       "L 136.000205 125.089157 \n",
       "L 136.280326 125.362048 \n",
       "L 137.120687 126.180723 \n",
       "L 137.400808 126.453614 \n",
       "L 138.241169 127.272289 \n",
       "L 138.52129 127.545181 \n",
       "L 139.641771 127.545181 \n",
       "L 140.482133 128.363855 \n",
       "L 140.762253 128.636747 \n",
       "L 141.602615 129.455422 \n",
       "L 141.882735 129.728313 \n",
       "L 142.723097 130.546988 \n",
       "L 143.003217 130.81988 \n",
       "L 143.843579 131.638554 \n",
       "L 144.123699 131.911446 \n",
       "L 144.964061 132.73012 \n",
       "z\n",
       "\" style=\"fill:#bcbd22;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_8\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 145.244181 133.003012 \n",
       "L 146.364663 133.003012 \n",
       "L 146.644784 132.73012 \n",
       "L 147.485145 131.911446 \n",
       "L 148.605627 131.911446 \n",
       "L 149.726109 131.911446 \n",
       "L 150.846591 131.911446 \n",
       "L 151.126711 131.638554 \n",
       "L 151.967073 130.81988 \n",
       "L 153.087555 130.81988 \n",
       "L 154.208037 130.81988 \n",
       "L 155.328518 130.81988 \n",
       "L 155.608639 130.546988 \n",
       "L 156.449 129.728313 \n",
       "L 157.569482 129.728313 \n",
       "L 158.689964 129.728313 \n",
       "L 159.810446 129.728313 \n",
       "L 160.090567 129.455422 \n",
       "L 160.930928 128.636747 \n",
       "L 162.05141 128.636747 \n",
       "L 163.171892 128.636747 \n",
       "L 163.452012 128.363855 \n",
       "L 164.292374 127.545181 \n",
       "L 165.412856 127.545181 \n",
       "L 166.533338 127.545181 \n",
       "L 167.65382 127.545181 \n",
       "L 167.93394 127.272289 \n",
       "L 168.774302 126.453614 \n",
       "L 169.894784 126.453614 \n",
       "L 171.015265 126.453614 \n",
       "L 172.135747 126.453614 \n",
       "L 172.415868 126.180723 \n",
       "L 173.256229 125.362048 \n",
       "L 174.376711 125.362048 \n",
       "L 175.497193 125.362048 \n",
       "L 176.617675 125.362048 \n",
       "L 176.897796 125.089157 \n",
       "L 177.738157 124.270482 \n",
       "L 178.858639 124.270482 \n",
       "L 179.979121 124.270482 \n",
       "L 181.099603 124.270482 \n",
       "L 181.379723 123.99759 \n",
       "L 182.220085 123.178916 \n",
       "L 183.340567 123.178916 \n",
       "L 184.461049 123.178916 \n",
       "L 185.58153 123.178916 \n",
       "L 185.861651 122.906024 \n",
       "L 186.702012 122.087349 \n",
       "L 187.822494 122.087349 \n",
       "L 188.942976 122.087349 \n",
       "L 190.063458 122.087349 \n",
       "L 190.343579 121.814458 \n",
       "L 191.18394 120.995783 \n",
       "L 192.304422 120.995783 \n",
       "L 193.424904 120.995783 \n",
       "L 194.545386 120.995783 \n",
       "L 194.825506 120.722892 \n",
       "L 195.665868 119.904217 \n",
       "L 196.78635 119.904217 \n",
       "L 197.906832 119.904217 \n",
       "L 198.186952 119.631325 \n",
       "L 199.027314 118.812651 \n",
       "L 200.147796 118.812651 \n",
       "L 201.268277 118.812651 \n",
       "L 202.388759 118.812651 \n",
       "L 202.66888 118.539759 \n",
       "L 203.509241 117.721084 \n",
       "L 204.629723 117.721084 \n",
       "L 205.750205 117.721084 \n",
       "L 206.870687 117.721084 \n",
       "L 207.150808 117.448193 \n",
       "L 207.991169 116.629518 \n",
       "L 209.111651 116.629518 \n",
       "L 210.232133 116.629518 \n",
       "L 211.352615 116.629518 \n",
       "L 211.632735 116.356627 \n",
       "L 212.473097 115.537952 \n",
       "L 213.593579 115.537952 \n",
       "L 214.714061 115.537952 \n",
       "L 215.834543 115.537952 \n",
       "L 216.114663 115.26506 \n",
       "L 216.955024 114.446386 \n",
       "L 218.075506 114.446386 \n",
       "L 219.195988 114.446386 \n",
       "L 220.31647 114.446386 \n",
       "L 220.596591 114.173494 \n",
       "L 221.436952 113.354819 \n",
       "L 222.557434 113.354819 \n",
       "L 223.677916 113.354819 \n",
       "L 224.798398 113.354819 \n",
       "L 225.078518 113.081928 \n",
       "L 225.91888 112.263253 \n",
       "L 227.039362 112.263253 \n",
       "L 228.159844 112.263253 \n",
       "L 229.280326 112.263253 \n",
       "L 229.560446 111.990361 \n",
       "L 230.400808 111.171687 \n",
       "L 231.52129 111.171687 \n",
       "L 232.641771 111.171687 \n",
       "L 232.921892 110.898795 \n",
       "L 233.762253 110.08012 \n",
       "L 234.882735 110.08012 \n",
       "L 236.003217 110.08012 \n",
       "L 237.123699 110.08012 \n",
       "L 237.40382 109.807229 \n",
       "L 238.244181 108.988554 \n",
       "L 239.364663 108.988554 \n",
       "L 240.485145 108.988554 \n",
       "L 241.605627 108.988554 \n",
       "L 241.885747 108.715663 \n",
       "L 242.726109 107.896988 \n",
       "L 243.846591 107.896988 \n",
       "L 244.967073 107.896988 \n",
       "L 246.087555 107.896988 \n",
       "L 246.367675 107.624096 \n",
       "L 247.208037 106.805422 \n",
       "L 248.328518 106.805422 \n",
       "L 249.449 106.805422 \n",
       "L 250.569482 106.805422 \n",
       "L 250.849603 106.53253 \n",
       "L 251.689964 105.713855 \n",
       "L 252.810446 105.713855 \n",
       "L 253.930928 105.713855 \n",
       "L 255.05141 105.713855 \n",
       "L 255.33153 105.440964 \n",
       "L 256.171892 104.622289 \n",
       "L 257.292374 104.622289 \n",
       "L 258.412856 104.622289 \n",
       "L 259.533338 104.622289 \n",
       "L 259.813458 104.349398 \n",
       "L 260.65382 103.530723 \n",
       "L 261.774302 103.530723 \n",
       "L 262.894784 103.530723 \n",
       "L 264.015265 103.530723 \n",
       "L 264.295386 103.257831 \n",
       "L 265.135747 102.439157 \n",
       "L 266.256229 102.439157 \n",
       "L 267.376711 102.439157 \n",
       "L 267.656832 102.166265 \n",
       "L 268.497193 101.34759 \n",
       "L 269.617675 101.34759 \n",
       "L 270.738157 101.34759 \n",
       "L 271.858639 101.34759 \n",
       "L 272.138759 101.074699 \n",
       "L 272.979121 100.256024 \n",
       "L 274.099603 100.256024 \n",
       "L 275.220085 100.256024 \n",
       "L 276.340567 100.256024 \n",
       "L 276.620687 99.983133 \n",
       "L 277.461049 99.164458 \n",
       "L 278.58153 99.164458 \n",
       "L 279.702012 99.164458 \n",
       "L 280.822494 99.164458 \n",
       "L 281.102615 98.891566 \n",
       "L 281.942976 98.072892 \n",
       "L 283.063458 98.072892 \n",
       "L 284.18394 98.072892 \n",
       "L 285.304422 98.072892 \n",
       "L 285.584543 97.8 \n",
       "L 286.424904 96.981325 \n",
       "L 287.545386 96.981325 \n",
       "L 288.665868 96.981325 \n",
       "L 289.78635 96.981325 \n",
       "L 290.06647 96.708434 \n",
       "L 290.906832 95.889759 \n",
       "L 292.027314 95.889759 \n",
       "L 293.147796 95.889759 \n",
       "L 294.268277 95.889759 \n",
       "L 294.548398 95.616867 \n",
       "L 295.388759 94.798193 \n",
       "L 296.509241 94.798193 \n",
       "L 297.629723 94.798193 \n",
       "L 298.750205 94.798193 \n",
       "L 299.030326 94.525301 \n",
       "L 299.870687 93.706627 \n",
       "L 300.991169 93.706627 \n",
       "L 302.111651 93.706627 \n",
       "L 302.391771 93.433735 \n",
       "L 303.232133 92.61506 \n",
       "L 304.352615 92.61506 \n",
       "L 305.473097 92.61506 \n",
       "L 306.593579 92.61506 \n",
       "L 306.873699 92.342169 \n",
       "L 307.714061 91.523494 \n",
       "L 308.834543 91.523494 \n",
       "L 309.955024 91.523494 \n",
       "L 311.075506 91.523494 \n",
       "L 311.355627 91.250602 \n",
       "L 312.195988 90.431928 \n",
       "L 313.31647 90.431928 \n",
       "L 314.436952 90.431928 \n",
       "L 315.557434 90.431928 \n",
       "L 315.837555 90.159036 \n",
       "L 316.677916 89.340361 \n",
       "L 317.798398 89.340361 \n",
       "L 318.91888 89.340361 \n",
       "L 320.039362 89.340361 \n",
       "L 320.319482 89.06747 \n",
       "L 321.159844 88.248795 \n",
       "L 321.159844 87.975904 \n",
       "L 321.159844 86.884337 \n",
       "L 321.159844 85.792771 \n",
       "L 321.159844 84.701205 \n",
       "L 321.159844 83.609639 \n",
       "L 321.159844 82.518072 \n",
       "L 321.159844 81.426506 \n",
       "L 321.159844 80.33494 \n",
       "L 321.159844 79.243373 \n",
       "L 321.159844 78.151807 \n",
       "L 321.159844 77.060241 \n",
       "L 321.159844 75.968675 \n",
       "L 321.159844 74.877108 \n",
       "L 321.159844 73.785542 \n",
       "L 321.159844 72.693976 \n",
       "L 321.159844 71.60241 \n",
       "L 321.159844 70.510843 \n",
       "L 321.159844 69.419277 \n",
       "L 321.159844 68.327711 \n",
       "L 321.159844 67.236145 \n",
       "L 321.159844 66.144578 \n",
       "L 321.159844 65.053012 \n",
       "L 321.159844 63.961446 \n",
       "L 321.159844 62.86988 \n",
       "L 321.159844 61.778313 \n",
       "L 321.159844 60.686747 \n",
       "L 321.159844 59.595181 \n",
       "L 321.159844 58.503614 \n",
       "L 321.159844 57.412048 \n",
       "L 321.159844 56.320482 \n",
       "L 321.159844 55.228916 \n",
       "L 321.159844 54.137349 \n",
       "L 321.159844 53.045783 \n",
       "L 321.159844 51.954217 \n",
       "L 321.159844 50.862651 \n",
       "L 321.159844 49.771084 \n",
       "L 321.159844 48.679518 \n",
       "L 321.159844 47.587952 \n",
       "L 321.159844 46.496386 \n",
       "L 321.159844 45.404819 \n",
       "L 321.159844 44.313253 \n",
       "L 321.159844 43.221687 \n",
       "L 321.159844 42.13012 \n",
       "L 321.159844 41.038554 \n",
       "L 321.159844 39.946988 \n",
       "L 321.159844 38.855422 \n",
       "L 321.159844 37.763855 \n",
       "L 321.159844 36.672289 \n",
       "L 321.159844 35.580723 \n",
       "L 321.159844 34.489157 \n",
       "L 321.159844 33.39759 \n",
       "L 321.159844 32.306024 \n",
       "L 321.159844 31.214458 \n",
       "L 321.159844 30.122892 \n",
       "L 321.159844 29.031325 \n",
       "L 321.159844 27.939759 \n",
       "L 321.159844 26.848193 \n",
       "L 321.159844 25.756627 \n",
       "L 321.159844 24.66506 \n",
       "L 321.159844 23.573494 \n",
       "L 321.159844 22.481928 \n",
       "L 321.159844 21.390361 \n",
       "L 321.159844 20.298795 \n",
       "L 321.159844 19.207229 \n",
       "L 321.159844 18.115663 \n",
       "L 321.159844 17.024096 \n",
       "L 321.159844 15.93253 \n",
       "L 321.159844 14.840964 \n",
       "L 321.159844 13.749398 \n",
       "L 321.159844 12.657831 \n",
       "L 321.159844 11.566265 \n",
       "L 321.159844 10.474699 \n",
       "L 321.159844 9.383133 \n",
       "L 321.159844 8.291566 \n",
       "L 321.159844 7.2 \n",
       "L 320.039362 7.2 \n",
       "L 318.91888 7.2 \n",
       "L 317.798398 7.2 \n",
       "L 316.677916 7.2 \n",
       "L 315.557434 7.2 \n",
       "L 314.436952 7.2 \n",
       "L 313.31647 7.2 \n",
       "L 312.195988 7.2 \n",
       "L 311.075506 7.2 \n",
       "L 309.955024 7.2 \n",
       "L 308.834543 7.2 \n",
       "L 307.714061 7.2 \n",
       "L 306.593579 7.2 \n",
       "L 305.473097 7.2 \n",
       "L 304.352615 7.2 \n",
       "L 303.232133 7.2 \n",
       "L 302.111651 7.2 \n",
       "L 300.991169 7.2 \n",
       "L 299.870687 7.2 \n",
       "L 298.750205 7.2 \n",
       "L 297.629723 7.2 \n",
       "L 296.509241 7.2 \n",
       "L 295.388759 7.2 \n",
       "L 294.268277 7.2 \n",
       "L 293.147796 7.2 \n",
       "L 292.027314 7.2 \n",
       "L 290.906832 7.2 \n",
       "L 289.78635 7.2 \n",
       "L 288.665868 7.2 \n",
       "L 287.545386 7.2 \n",
       "L 286.424904 7.2 \n",
       "L 285.304422 7.2 \n",
       "L 284.18394 7.2 \n",
       "L 283.063458 7.2 \n",
       "L 281.942976 7.2 \n",
       "L 280.822494 7.2 \n",
       "L 279.702012 7.2 \n",
       "L 278.58153 7.2 \n",
       "L 277.461049 7.2 \n",
       "L 276.340567 7.2 \n",
       "L 275.220085 7.2 \n",
       "L 274.099603 7.2 \n",
       "L 272.979121 7.2 \n",
       "L 271.858639 7.2 \n",
       "L 270.738157 7.2 \n",
       "L 269.617675 7.2 \n",
       "L 268.497193 7.2 \n",
       "L 267.376711 7.2 \n",
       "L 266.256229 7.2 \n",
       "L 265.135747 7.2 \n",
       "L 264.015265 7.2 \n",
       "L 262.894784 7.2 \n",
       "L 261.774302 7.2 \n",
       "L 260.65382 7.2 \n",
       "L 259.533338 7.2 \n",
       "L 258.412856 7.2 \n",
       "L 257.292374 7.2 \n",
       "L 256.171892 7.2 \n",
       "L 255.05141 7.2 \n",
       "L 253.930928 7.2 \n",
       "L 252.810446 7.2 \n",
       "L 251.689964 7.2 \n",
       "L 250.569482 7.2 \n",
       "L 249.449 7.2 \n",
       "L 248.328518 7.2 \n",
       "L 247.208037 7.2 \n",
       "L 246.087555 7.2 \n",
       "L 244.967073 7.2 \n",
       "L 243.846591 7.2 \n",
       "L 242.726109 7.2 \n",
       "L 241.605627 7.2 \n",
       "L 240.485145 7.2 \n",
       "L 239.364663 7.2 \n",
       "L 238.244181 7.2 \n",
       "L 237.123699 7.2 \n",
       "L 236.003217 7.2 \n",
       "L 234.882735 7.2 \n",
       "L 233.762253 7.2 \n",
       "L 232.641771 7.2 \n",
       "L 231.52129 7.2 \n",
       "L 230.400808 7.2 \n",
       "L 229.280326 7.2 \n",
       "L 228.159844 7.2 \n",
       "L 227.039362 7.2 \n",
       "L 225.91888 7.2 \n",
       "L 224.798398 7.2 \n",
       "L 223.677916 7.2 \n",
       "L 222.557434 7.2 \n",
       "L 221.436952 7.2 \n",
       "L 220.31647 7.2 \n",
       "L 219.195988 7.2 \n",
       "L 218.075506 7.2 \n",
       "L 216.955024 7.2 \n",
       "L 215.834543 7.2 \n",
       "L 214.714061 7.2 \n",
       "L 213.593579 7.2 \n",
       "L 212.473097 7.2 \n",
       "L 211.352615 7.2 \n",
       "L 210.232133 7.2 \n",
       "L 209.111651 7.2 \n",
       "L 207.991169 7.2 \n",
       "L 206.870687 7.2 \n",
       "L 205.750205 7.2 \n",
       "L 204.629723 7.2 \n",
       "L 203.509241 7.2 \n",
       "L 202.388759 7.2 \n",
       "L 201.268277 7.2 \n",
       "L 200.147796 7.2 \n",
       "L 199.027314 7.2 \n",
       "L 197.906832 7.2 \n",
       "L 196.78635 7.2 \n",
       "L 195.665868 7.2 \n",
       "L 194.545386 7.2 \n",
       "L 193.424904 7.2 \n",
       "L 192.304422 7.2 \n",
       "L 191.18394 7.2 \n",
       "L 190.063458 7.2 \n",
       "L 188.942976 7.2 \n",
       "L 187.822494 7.2 \n",
       "L 186.702012 7.2 \n",
       "L 185.58153 7.2 \n",
       "L 184.461049 7.2 \n",
       "L 183.340567 7.2 \n",
       "L 182.220085 7.2 \n",
       "L 181.099603 7.2 \n",
       "L 179.979121 7.2 \n",
       "L 178.858639 7.2 \n",
       "L 177.738157 7.2 \n",
       "L 176.617675 7.2 \n",
       "L 175.497193 7.2 \n",
       "L 174.376711 7.2 \n",
       "L 173.256229 7.2 \n",
       "L 172.135747 7.2 \n",
       "L 171.015265 7.2 \n",
       "L 169.894784 7.2 \n",
       "L 168.774302 7.2 \n",
       "L 167.65382 7.2 \n",
       "L 166.533338 7.2 \n",
       "L 165.412856 7.2 \n",
       "L 164.292374 7.2 \n",
       "L 163.171892 7.2 \n",
       "L 162.05141 7.2 \n",
       "L 160.930928 7.2 \n",
       "L 159.810446 7.2 \n",
       "L 158.689964 7.2 \n",
       "L 157.569482 7.2 \n",
       "L 156.449 7.2 \n",
       "L 155.328518 7.2 \n",
       "L 154.208037 7.2 \n",
       "L 153.087555 7.2 \n",
       "L 151.967073 7.2 \n",
       "L 150.846591 7.2 \n",
       "L 149.726109 7.2 \n",
       "L 148.605627 7.2 \n",
       "L 147.485145 7.2 \n",
       "L 146.364663 7.2 \n",
       "L 145.244181 7.2 \n",
       "L 144.123699 7.2 \n",
       "L 143.003217 7.2 \n",
       "L 141.882735 7.2 \n",
       "L 140.762253 7.2 \n",
       "L 139.641771 7.2 \n",
       "L 138.52129 7.2 \n",
       "L 137.400808 7.2 \n",
       "L 136.280326 7.2 \n",
       "L 135.159844 7.2 \n",
       "L 134.039362 7.2 \n",
       "L 132.91888 7.2 \n",
       "L 131.798398 7.2 \n",
       "L 130.677916 7.2 \n",
       "L 129.557434 7.2 \n",
       "L 128.436952 7.2 \n",
       "L 127.31647 7.2 \n",
       "L 126.195988 7.2 \n",
       "L 125.075506 7.2 \n",
       "L 123.955024 7.2 \n",
       "L 122.834543 7.2 \n",
       "L 121.714061 7.2 \n",
       "L 120.593579 7.2 \n",
       "L 119.473097 7.2 \n",
       "L 118.352615 7.2 \n",
       "L 117.232133 7.2 \n",
       "L 116.111651 7.2 \n",
       "L 114.991169 7.2 \n",
       "L 113.870687 7.2 \n",
       "L 112.750205 7.2 \n",
       "L 111.629723 7.2 \n",
       "L 110.509241 7.2 \n",
       "L 109.388759 7.2 \n",
       "L 108.268277 7.2 \n",
       "L 107.147796 7.2 \n",
       "L 106.027314 7.2 \n",
       "L 104.906832 7.2 \n",
       "L 103.78635 7.2 \n",
       "L 102.665868 7.2 \n",
       "L 101.545386 7.2 \n",
       "L 100.424904 7.2 \n",
       "L 99.304422 7.2 \n",
       "L 98.18394 7.2 \n",
       "L 97.063458 7.2 \n",
       "L 95.942976 7.2 \n",
       "L 94.822494 7.2 \n",
       "L 93.702012 7.2 \n",
       "L 92.58153 7.2 \n",
       "L 91.461049 7.2 \n",
       "L 90.340567 7.2 \n",
       "L 89.220085 7.2 \n",
       "L 88.099603 7.2 \n",
       "L 86.979121 7.2 \n",
       "L 85.858639 7.2 \n",
       "L 84.738157 7.2 \n",
       "L 83.617675 7.2 \n",
       "L 82.497193 7.2 \n",
       "L 81.376711 7.2 \n",
       "L 80.256229 7.2 \n",
       "L 79.135747 7.2 \n",
       "L 78.015265 7.2 \n",
       "L 76.894784 7.2 \n",
       "L 75.774302 7.2 \n",
       "L 74.65382 7.2 \n",
       "L 73.533338 7.2 \n",
       "L 72.412856 7.2 \n",
       "L 71.292374 7.2 \n",
       "L 70.171892 7.2 \n",
       "L 69.05141 7.2 \n",
       "L 67.930928 7.2 \n",
       "L 66.810446 7.2 \n",
       "L 65.689964 7.2 \n",
       "L 64.569482 7.2 \n",
       "L 63.449 7.2 \n",
       "L 62.328518 7.2 \n",
       "L 61.208037 7.2 \n",
       "L 60.087555 7.2 \n",
       "L 58.967073 7.2 \n",
       "L 57.846591 7.2 \n",
       "L 56.726109 7.2 \n",
       "L 55.605627 7.2 \n",
       "L 54.485145 7.2 \n",
       "L 53.364663 7.2 \n",
       "L 52.244181 7.2 \n",
       "L 51.123699 7.2 \n",
       "L 50.003217 7.2 \n",
       "L 48.882735 7.2 \n",
       "L 47.762253 7.2 \n",
       "L 46.641771 7.2 \n",
       "L 45.52129 7.2 \n",
       "L 44.400808 7.2 \n",
       "L 43.280326 7.2 \n",
       "L 42.159844 7.2 \n",
       "L 42.159844 8.291566 \n",
       "L 42.159844 9.383133 \n",
       "L 42.159844 10.474699 \n",
       "L 42.159844 11.566265 \n",
       "L 42.159844 12.657831 \n",
       "L 42.159844 13.749398 \n",
       "L 42.159844 14.840964 \n",
       "L 42.159844 15.93253 \n",
       "L 42.159844 17.024096 \n",
       "L 42.159844 18.115663 \n",
       "L 42.159844 19.207229 \n",
       "L 42.159844 20.298795 \n",
       "L 42.159844 21.390361 \n",
       "L 42.159844 22.481928 \n",
       "L 42.159844 23.573494 \n",
       "L 42.159844 24.66506 \n",
       "L 42.159844 25.756627 \n",
       "L 42.159844 26.848193 \n",
       "L 42.159844 27.939759 \n",
       "L 42.159844 29.031325 \n",
       "L 42.159844 30.122892 \n",
       "L 42.159844 31.214458 \n",
       "L 42.159844 32.306024 \n",
       "L 42.159844 33.39759 \n",
       "L 42.159844 34.489157 \n",
       "L 42.159844 35.580723 \n",
       "L 42.159844 36.672289 \n",
       "L 42.159844 37.763855 \n",
       "L 42.159844 38.855422 \n",
       "L 42.159844 39.946988 \n",
       "L 42.159844 41.038554 \n",
       "L 42.159844 42.13012 \n",
       "L 42.159844 43.221687 \n",
       "L 42.159844 44.313253 \n",
       "L 42.159844 44.449699 \n",
       "L 43.140265 45.404819 \n",
       "L 43.280326 45.541265 \n",
       "L 44.260747 46.496386 \n",
       "L 44.400808 46.632831 \n",
       "L 45.381229 47.587952 \n",
       "L 45.52129 47.724398 \n",
       "L 46.501711 48.679518 \n",
       "L 46.641771 48.815964 \n",
       "L 47.762253 48.815964 \n",
       "L 48.742675 49.771084 \n",
       "L 48.882735 49.90753 \n",
       "L 49.863157 50.862651 \n",
       "L 50.003217 50.999096 \n",
       "L 50.983639 51.954217 \n",
       "L 51.123699 52.090663 \n",
       "L 52.104121 53.045783 \n",
       "L 52.244181 53.182229 \n",
       "L 53.224603 54.137349 \n",
       "L 53.364663 54.273795 \n",
       "L 54.345085 55.228916 \n",
       "L 54.485145 55.365361 \n",
       "L 55.465567 56.320482 \n",
       "L 55.605627 56.456928 \n",
       "L 56.726109 56.456928 \n",
       "L 57.70653 57.412048 \n",
       "L 57.846591 57.548494 \n",
       "L 58.827012 58.503614 \n",
       "L 58.967073 58.64006 \n",
       "L 59.947494 59.595181 \n",
       "L 60.087555 59.731627 \n",
       "L 61.067976 60.686747 \n",
       "L 61.208037 60.823193 \n",
       "L 62.188458 61.778313 \n",
       "L 62.328518 61.914759 \n",
       "L 63.30894 62.86988 \n",
       "L 63.449 63.006325 \n",
       "L 64.429422 63.961446 \n",
       "L 64.569482 64.097892 \n",
       "L 65.549904 65.053012 \n",
       "L 65.689964 65.189458 \n",
       "L 66.810446 65.189458 \n",
       "L 67.790868 66.144578 \n",
       "L 67.930928 66.281024 \n",
       "L 68.91135 67.236145 \n",
       "L 69.05141 67.37259 \n",
       "L 70.031832 68.327711 \n",
       "L 70.171892 68.464157 \n",
       "L 71.152314 69.419277 \n",
       "L 71.292374 69.555723 \n",
       "L 72.272796 70.510843 \n",
       "L 72.412856 70.647289 \n",
       "L 73.393277 71.60241 \n",
       "L 73.533338 71.738855 \n",
       "L 74.513759 72.693976 \n",
       "L 74.65382 72.830422 \n",
       "L 75.774302 72.830422 \n",
       "L 76.754723 73.785542 \n",
       "L 76.894784 73.921988 \n",
       "L 77.875205 74.877108 \n",
       "L 78.015265 75.013554 \n",
       "L 78.995687 75.968675 \n",
       "L 79.135747 76.10512 \n",
       "L 80.116169 77.060241 \n",
       "L 80.256229 77.196687 \n",
       "L 81.236651 78.151807 \n",
       "L 81.376711 78.288253 \n",
       "L 82.357133 79.243373 \n",
       "L 82.497193 79.379819 \n",
       "L 83.477615 80.33494 \n",
       "L 83.617675 80.471386 \n",
       "L 84.738157 80.471386 \n",
       "L 85.718579 81.426506 \n",
       "L 85.858639 81.562952 \n",
       "L 86.839061 82.518072 \n",
       "L 86.979121 82.654518 \n",
       "L 87.959543 83.609639 \n",
       "L 88.099603 83.746084 \n",
       "L 89.080024 84.701205 \n",
       "L 89.220085 84.837651 \n",
       "L 90.200506 85.792771 \n",
       "L 90.340567 85.929217 \n",
       "L 91.320988 86.884337 \n",
       "L 91.461049 87.020783 \n",
       "L 92.44147 87.975904 \n",
       "L 92.58153 88.112349 \n",
       "L 93.702012 88.112349 \n",
       "L 94.682434 89.06747 \n",
       "L 94.822494 89.203916 \n",
       "L 95.802916 90.159036 \n",
       "L 95.942976 90.295482 \n",
       "L 96.923398 91.250602 \n",
       "L 97.063458 91.387048 \n",
       "L 98.04388 92.342169 \n",
       "L 98.18394 92.478614 \n",
       "L 99.164362 93.433735 \n",
       "L 99.304422 93.570181 \n",
       "L 100.284844 94.525301 \n",
       "L 100.424904 94.661747 \n",
       "L 101.405326 95.616867 \n",
       "L 101.545386 95.753313 \n",
       "L 102.665868 95.753313 \n",
       "L 103.64629 96.708434 \n",
       "L 103.78635 96.84488 \n",
       "L 104.766771 97.8 \n",
       "L 104.906832 97.936446 \n",
       "L 105.887253 98.891566 \n",
       "L 106.027314 99.028012 \n",
       "L 107.007735 99.983133 \n",
       "L 107.147796 100.119578 \n",
       "L 108.128217 101.074699 \n",
       "L 108.268277 101.211145 \n",
       "L 109.248699 102.166265 \n",
       "L 109.388759 102.302711 \n",
       "L 110.369181 103.257831 \n",
       "L 110.509241 103.394277 \n",
       "L 111.629723 103.394277 \n",
       "L 112.610145 104.349398 \n",
       "L 112.750205 104.485843 \n",
       "L 113.730627 105.440964 \n",
       "L 113.870687 105.57741 \n",
       "L 114.851109 106.53253 \n",
       "L 114.991169 106.668976 \n",
       "L 115.971591 107.624096 \n",
       "L 116.111651 107.760542 \n",
       "L 117.092073 108.715663 \n",
       "L 117.232133 108.852108 \n",
       "L 118.212555 109.807229 \n",
       "L 118.352615 109.943675 \n",
       "L 119.333037 110.898795 \n",
       "L 119.473097 111.035241 \n",
       "L 120.593579 111.035241 \n",
       "L 121.574 111.990361 \n",
       "L 121.714061 112.126807 \n",
       "L 122.694482 113.081928 \n",
       "L 122.834543 113.218373 \n",
       "L 123.814964 114.173494 \n",
       "L 123.955024 114.30994 \n",
       "L 124.935446 115.26506 \n",
       "L 125.075506 115.401506 \n",
       "L 126.055928 116.356627 \n",
       "L 126.195988 116.493072 \n",
       "L 127.17641 117.448193 \n",
       "L 127.31647 117.584639 \n",
       "L 128.296892 118.539759 \n",
       "L 128.436952 118.676205 \n",
       "L 129.557434 118.676205 \n",
       "L 130.537856 119.631325 \n",
       "L 130.677916 119.767771 \n",
       "L 131.658338 120.722892 \n",
       "L 131.798398 120.859337 \n",
       "L 132.77882 121.814458 \n",
       "L 132.91888 121.950904 \n",
       "L 133.899302 122.906024 \n",
       "L 134.039362 123.04247 \n",
       "L 135.019784 123.99759 \n",
       "L 135.159844 124.134036 \n",
       "L 136.140265 125.089157 \n",
       "L 136.280326 125.225602 \n",
       "L 137.260747 126.180723 \n",
       "L 137.400808 126.317169 \n",
       "L 138.381229 127.272289 \n",
       "L 138.52129 127.408735 \n",
       "L 139.641771 127.408735 \n",
       "L 140.622193 128.363855 \n",
       "L 140.762253 128.500301 \n",
       "L 141.742675 129.455422 \n",
       "L 141.882735 129.591867 \n",
       "L 142.863157 130.546988 \n",
       "L 143.003217 130.683434 \n",
       "L 143.983639 131.638554 \n",
       "L 144.123699 131.775 \n",
       "L 145.104121 132.73012 \n",
       "z\n",
       "\" style=\"fill:#17becf;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_9\">\n",
       "    <defs>\n",
       "     <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"C0_0_2e3b908ff7\"/>\n",
       "    </defs>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"284.555324\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"271.419731\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"91.899957\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"52.493177\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"91.899957\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"196.984703\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"153.199392\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"216.502778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"161.956454\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"175.092047\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"101.155556\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"236.391482\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"166.334985\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"271.419731\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"56.871708\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"240.770013\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"140.063799\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"267.0412\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"245.148544\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"245.148544\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"267.0412\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"297.690917\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.250239\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"226.988889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"196.984703\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"310.82651\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"153.199392\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"288.933855\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.250239\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"302.069448\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"161.956454\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"240.770013\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"236.391482\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"101.155556\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"170.713516\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"302.069448\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6b34b769b6)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_2e3b908ff7\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_1\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 45.404819 \n",
       "L 46.641771 49.771084 \n",
       "L 47.762253 49.771084 \n",
       "L 55.605627 57.412048 \n",
       "L 56.726109 57.412048 \n",
       "L 65.689964 66.144578 \n",
       "L 66.810446 66.144578 \n",
       "L 74.65382 73.785542 \n",
       "L 75.774302 73.785542 \n",
       "L 83.617675 81.426506 \n",
       "L 84.738157 81.426506 \n",
       "L 92.58153 89.06747 \n",
       "L 93.702012 89.06747 \n",
       "L 101.545386 96.708434 \n",
       "L 102.665868 96.708434 \n",
       "L 110.509241 104.349398 \n",
       "L 111.629723 104.349398 \n",
       "L 119.473097 111.990361 \n",
       "L 120.593579 111.990361 \n",
       "L 128.436952 119.631325 \n",
       "L 129.557434 119.631325 \n",
       "L 138.52129 128.363855 \n",
       "L 139.641771 128.363855 \n",
       "L 144.123699 132.73012 \n",
       "L 144.123699 142.554217 \n",
       "L 143.003217 143.645783 \n",
       "L 143.003217 152.378313 \n",
       "L 141.882735 153.46988 \n",
       "L 141.882735 162.20241 \n",
       "L 140.762253 163.293976 \n",
       "L 140.762253 172.026506 \n",
       "L 139.641771 173.118072 \n",
       "L 139.641771 181.850602 \n",
       "L 138.52129 182.942169 \n",
       "L 138.52129 191.674699 \n",
       "L 137.400808 192.766265 \n",
       "L 137.400808 200.407229 \n",
       "L 136.280326 201.498795 \n",
       "L 136.280326 210.231325 \n",
       "L 135.159844 211.322892 \n",
       "L 135.159844 220.055422 \n",
       "L 134.039362 221.146988 \n",
       "L 134.039362 229.879518 \n",
       "L 132.91888 230.971084 \n",
       "L 132.91888 239.703614 \n",
       "L 131.798398 240.795181 \n",
       "L 131.798398 249.527711 \n",
       "L 130.677916 250.619277 \n",
       "L 130.677916 259.351807 \n",
       "L 129.557434 260.443373 \n",
       "L 129.557434 268.084337 \n",
       "L 128.436952 269.175904 \n",
       "L 128.436952 277.908434 \n",
       "L 127.31647 279 \n",
       "L 127.31647 279 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_2\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 45.268373 \n",
       "L 46.641771 49.634639 \n",
       "L 47.762253 49.634639 \n",
       "L 55.605627 57.275602 \n",
       "L 56.726109 57.275602 \n",
       "L 65.689964 66.008133 \n",
       "L 66.810446 66.008133 \n",
       "L 74.65382 73.649096 \n",
       "L 75.774302 73.649096 \n",
       "L 83.617675 81.29006 \n",
       "L 84.738157 81.29006 \n",
       "L 92.58153 88.931024 \n",
       "L 93.702012 88.931024 \n",
       "L 101.545386 96.571988 \n",
       "L 102.665868 96.571988 \n",
       "L 110.509241 104.212952 \n",
       "L 111.629723 104.212952 \n",
       "L 119.473097 111.853916 \n",
       "L 120.593579 111.853916 \n",
       "L 128.436952 119.49488 \n",
       "L 129.557434 119.49488 \n",
       "L 138.52129 128.22741 \n",
       "L 139.641771 128.22741 \n",
       "L 144.263759 132.73012 \n",
       "L 144.40382 133.821687 \n",
       "L 144.40382 142.554217 \n",
       "L 143.283338 143.645783 \n",
       "L 143.283338 152.378313 \n",
       "L 142.162856 153.46988 \n",
       "L 142.162856 162.20241 \n",
       "L 141.042374 163.293976 \n",
       "L 141.042374 172.026506 \n",
       "L 139.921892 173.118072 \n",
       "L 139.921892 181.850602 \n",
       "L 138.80141 182.942169 \n",
       "L 138.80141 191.674699 \n",
       "L 137.680928 192.766265 \n",
       "L 137.680928 200.407229 \n",
       "L 136.560446 201.498795 \n",
       "L 136.560446 210.231325 \n",
       "L 135.439964 211.322892 \n",
       "L 135.439964 220.055422 \n",
       "L 134.319482 221.146988 \n",
       "L 134.319482 229.879518 \n",
       "L 133.199 230.971084 \n",
       "L 133.199 239.703614 \n",
       "L 132.078518 240.795181 \n",
       "L 132.078518 249.527711 \n",
       "L 130.958037 250.619277 \n",
       "L 130.958037 259.351807 \n",
       "L 129.837555 260.443373 \n",
       "L 129.837555 268.084337 \n",
       "L 128.717073 269.175904 \n",
       "L 128.717073 277.908434 \n",
       "L 127.596591 279 \n",
       "L 127.596591 279 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_3\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 45.131928 \n",
       "L 46.641771 49.498193 \n",
       "L 47.762253 49.498193 \n",
       "L 55.605627 57.139157 \n",
       "L 56.726109 57.139157 \n",
       "L 65.689964 65.871687 \n",
       "L 66.810446 65.871687 \n",
       "L 74.65382 73.512651 \n",
       "L 75.774302 73.512651 \n",
       "L 83.617675 81.153614 \n",
       "L 84.738157 81.153614 \n",
       "L 92.58153 88.794578 \n",
       "L 93.702012 88.794578 \n",
       "L 101.545386 96.435542 \n",
       "L 102.665868 96.435542 \n",
       "L 110.509241 104.076506 \n",
       "L 111.629723 104.076506 \n",
       "L 119.473097 111.71747 \n",
       "L 120.593579 111.71747 \n",
       "L 128.436952 119.358434 \n",
       "L 129.557434 119.358434 \n",
       "L 138.52129 128.090964 \n",
       "L 139.641771 128.090964 \n",
       "L 144.40382 132.73012 \n",
       "L 144.68394 133.821687 \n",
       "L 144.68394 142.554217 \n",
       "L 143.563458 143.645783 \n",
       "L 143.563458 152.378313 \n",
       "L 142.442976 153.46988 \n",
       "L 142.442976 162.20241 \n",
       "L 141.322494 163.293976 \n",
       "L 141.322494 172.026506 \n",
       "L 140.202012 173.118072 \n",
       "L 140.202012 181.850602 \n",
       "L 139.08153 182.942169 \n",
       "L 139.08153 191.674699 \n",
       "L 137.961049 192.766265 \n",
       "L 137.961049 200.407229 \n",
       "L 136.840567 201.498795 \n",
       "L 136.840567 210.231325 \n",
       "L 135.720085 211.322892 \n",
       "L 135.720085 220.055422 \n",
       "L 134.599603 221.146988 \n",
       "L 134.599603 229.879518 \n",
       "L 133.479121 230.971084 \n",
       "L 133.479121 239.703614 \n",
       "L 132.358639 240.795181 \n",
       "L 132.358639 249.527711 \n",
       "L 131.238157 250.619277 \n",
       "L 131.238157 259.351807 \n",
       "L 130.117675 260.443373 \n",
       "L 130.117675 268.084337 \n",
       "L 128.997193 269.175904 \n",
       "L 128.997193 277.908434 \n",
       "L 127.876711 279 \n",
       "L 127.876711 279 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_4\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 44.995482 \n",
       "L 46.641771 49.361747 \n",
       "L 47.762253 49.361747 \n",
       "L 55.605627 57.002711 \n",
       "L 56.726109 57.002711 \n",
       "L 65.689964 65.735241 \n",
       "L 66.810446 65.735241 \n",
       "L 74.65382 73.376205 \n",
       "L 75.774302 73.376205 \n",
       "L 83.617675 81.017169 \n",
       "L 84.738157 81.017169 \n",
       "L 92.58153 88.658133 \n",
       "L 93.702012 88.658133 \n",
       "L 101.545386 96.299096 \n",
       "L 102.665868 96.299096 \n",
       "L 110.509241 103.94006 \n",
       "L 111.629723 103.94006 \n",
       "L 119.473097 111.581024 \n",
       "L 120.593579 111.581024 \n",
       "L 128.436952 119.221988 \n",
       "L 129.557434 119.221988 \n",
       "L 138.52129 127.954518 \n",
       "L 139.641771 127.954518 \n",
       "L 144.54388 132.73012 \n",
       "L 144.964061 133.821687 \n",
       "L 144.964061 142.554217 \n",
       "L 143.843579 143.645783 \n",
       "L 143.843579 152.378313 \n",
       "L 142.723097 153.46988 \n",
       "L 142.723097 162.20241 \n",
       "L 141.602615 163.293976 \n",
       "L 141.602615 172.026506 \n",
       "L 140.482133 173.118072 \n",
       "L 140.482133 181.850602 \n",
       "L 139.361651 182.942169 \n",
       "L 139.361651 191.674699 \n",
       "L 138.241169 192.766265 \n",
       "L 138.241169 200.407229 \n",
       "L 137.120687 201.498795 \n",
       "L 137.120687 210.231325 \n",
       "L 136.000205 211.322892 \n",
       "L 136.000205 220.055422 \n",
       "L 134.879723 221.146988 \n",
       "L 134.879723 229.879518 \n",
       "L 133.759241 230.971084 \n",
       "L 133.759241 239.703614 \n",
       "L 132.638759 240.795181 \n",
       "L 132.638759 249.527711 \n",
       "L 131.518277 250.619277 \n",
       "L 131.518277 259.351807 \n",
       "L 130.397796 260.443373 \n",
       "L 130.397796 268.084337 \n",
       "L 129.277314 269.175904 \n",
       "L 129.277314 277.908434 \n",
       "L 128.156832 279 \n",
       "L 128.156832 279 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_5\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 44.859036 \n",
       "L 46.641771 49.225301 \n",
       "L 47.762253 49.225301 \n",
       "L 55.605627 56.866265 \n",
       "L 56.726109 56.866265 \n",
       "L 65.689964 65.598795 \n",
       "L 66.810446 65.598795 \n",
       "L 74.65382 73.239759 \n",
       "L 75.774302 73.239759 \n",
       "L 83.617675 80.880723 \n",
       "L 84.738157 80.880723 \n",
       "L 92.58153 88.521687 \n",
       "L 93.702012 88.521687 \n",
       "L 101.545386 96.162651 \n",
       "L 102.665868 96.162651 \n",
       "L 110.509241 103.803614 \n",
       "L 111.629723 103.803614 \n",
       "L 119.473097 111.444578 \n",
       "L 120.593579 111.444578 \n",
       "L 128.436952 119.085542 \n",
       "L 129.557434 119.085542 \n",
       "L 138.52129 127.818072 \n",
       "L 139.641771 127.818072 \n",
       "L 144.68394 132.73012 \n",
       "L 145.244181 133.821687 \n",
       "L 146.364663 133.821687 \n",
       "L 147.485145 132.73012 \n",
       "L 150.846591 132.73012 \n",
       "L 151.967073 131.638554 \n",
       "L 155.328518 131.638554 \n",
       "L 156.449 130.546988 \n",
       "L 159.810446 130.546988 \n",
       "L 160.930928 129.455422 \n",
       "L 163.171892 129.455422 \n",
       "L 164.292374 128.363855 \n",
       "L 167.65382 128.363855 \n",
       "L 168.774302 127.272289 \n",
       "L 172.135747 127.272289 \n",
       "L 173.256229 126.180723 \n",
       "L 176.617675 126.180723 \n",
       "L 177.738157 125.089157 \n",
       "L 181.099603 125.089157 \n",
       "L 182.220085 123.99759 \n",
       "L 185.58153 123.99759 \n",
       "L 186.702012 122.906024 \n",
       "L 190.063458 122.906024 \n",
       "L 191.18394 121.814458 \n",
       "L 194.545386 121.814458 \n",
       "L 195.665868 120.722892 \n",
       "L 197.906832 120.722892 \n",
       "L 199.027314 119.631325 \n",
       "L 202.388759 119.631325 \n",
       "L 203.509241 118.539759 \n",
       "L 206.870687 118.539759 \n",
       "L 207.991169 117.448193 \n",
       "L 211.352615 117.448193 \n",
       "L 212.473097 116.356627 \n",
       "L 215.834543 116.356627 \n",
       "L 216.955024 115.26506 \n",
       "L 220.31647 115.26506 \n",
       "L 221.436952 114.173494 \n",
       "L 224.798398 114.173494 \n",
       "L 225.91888 113.081928 \n",
       "L 229.280326 113.081928 \n",
       "L 230.400808 111.990361 \n",
       "L 232.641771 111.990361 \n",
       "L 233.762253 110.898795 \n",
       "L 237.123699 110.898795 \n",
       "L 238.244181 109.807229 \n",
       "L 241.605627 109.807229 \n",
       "L 242.726109 108.715663 \n",
       "L 246.087555 108.715663 \n",
       "L 247.208037 107.624096 \n",
       "L 250.569482 107.624096 \n",
       "L 251.689964 106.53253 \n",
       "L 255.05141 106.53253 \n",
       "L 256.171892 105.440964 \n",
       "L 259.533338 105.440964 \n",
       "L 260.65382 104.349398 \n",
       "L 264.015265 104.349398 \n",
       "L 265.135747 103.257831 \n",
       "L 267.376711 103.257831 \n",
       "L 268.497193 102.166265 \n",
       "L 271.858639 102.166265 \n",
       "L 272.979121 101.074699 \n",
       "L 276.340567 101.074699 \n",
       "L 277.461049 99.983133 \n",
       "L 280.822494 99.983133 \n",
       "L 281.942976 98.891566 \n",
       "L 285.304422 98.891566 \n",
       "L 286.424904 97.8 \n",
       "L 289.78635 97.8 \n",
       "L 290.906832 96.708434 \n",
       "L 294.268277 96.708434 \n",
       "L 295.388759 95.616867 \n",
       "L 298.750205 95.616867 \n",
       "L 299.870687 94.525301 \n",
       "L 302.111651 94.525301 \n",
       "L 303.232133 93.433735 \n",
       "L 306.593579 93.433735 \n",
       "L 307.714061 92.342169 \n",
       "L 311.075506 92.342169 \n",
       "L 312.195988 91.250602 \n",
       "L 315.557434 91.250602 \n",
       "L 316.677916 90.159036 \n",
       "L 320.039362 90.159036 \n",
       "L 321.159844 89.06747 \n",
       "L 321.159844 89.06747 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_6\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 44.72259 \n",
       "L 46.641771 49.088855 \n",
       "L 47.762253 49.088855 \n",
       "L 55.605627 56.729819 \n",
       "L 56.726109 56.729819 \n",
       "L 65.689964 65.462349 \n",
       "L 66.810446 65.462349 \n",
       "L 74.65382 73.103313 \n",
       "L 75.774302 73.103313 \n",
       "L 83.617675 80.744277 \n",
       "L 84.738157 80.744277 \n",
       "L 92.58153 88.385241 \n",
       "L 93.702012 88.385241 \n",
       "L 101.545386 96.026205 \n",
       "L 102.665868 96.026205 \n",
       "L 110.509241 103.667169 \n",
       "L 111.629723 103.667169 \n",
       "L 119.473097 111.308133 \n",
       "L 120.593579 111.308133 \n",
       "L 128.436952 118.949096 \n",
       "L 129.557434 118.949096 \n",
       "L 138.52129 127.681627 \n",
       "L 139.641771 127.681627 \n",
       "L 144.824 132.73012 \n",
       "L 145.244181 133.548795 \n",
       "L 146.364663 133.548795 \n",
       "L 147.485145 132.457229 \n",
       "L 150.846591 132.457229 \n",
       "L 151.967073 131.365663 \n",
       "L 155.328518 131.365663 \n",
       "L 156.449 130.274096 \n",
       "L 159.810446 130.274096 \n",
       "L 160.930928 129.18253 \n",
       "L 163.171892 129.18253 \n",
       "L 164.292374 128.090964 \n",
       "L 167.65382 128.090964 \n",
       "L 168.774302 126.999398 \n",
       "L 172.135747 126.999398 \n",
       "L 173.256229 125.907831 \n",
       "L 176.617675 125.907831 \n",
       "L 177.738157 124.816265 \n",
       "L 181.099603 124.816265 \n",
       "L 182.220085 123.724699 \n",
       "L 185.58153 123.724699 \n",
       "L 186.702012 122.633133 \n",
       "L 190.063458 122.633133 \n",
       "L 191.18394 121.541566 \n",
       "L 194.545386 121.541566 \n",
       "L 195.665868 120.45 \n",
       "L 197.906832 120.45 \n",
       "L 199.027314 119.358434 \n",
       "L 202.388759 119.358434 \n",
       "L 203.509241 118.266867 \n",
       "L 206.870687 118.266867 \n",
       "L 207.991169 117.175301 \n",
       "L 211.352615 117.175301 \n",
       "L 212.473097 116.083735 \n",
       "L 215.834543 116.083735 \n",
       "L 216.955024 114.992169 \n",
       "L 220.31647 114.992169 \n",
       "L 221.436952 113.900602 \n",
       "L 224.798398 113.900602 \n",
       "L 225.91888 112.809036 \n",
       "L 229.280326 112.809036 \n",
       "L 230.400808 111.71747 \n",
       "L 232.641771 111.71747 \n",
       "L 233.762253 110.625904 \n",
       "L 237.123699 110.625904 \n",
       "L 238.244181 109.534337 \n",
       "L 241.605627 109.534337 \n",
       "L 242.726109 108.442771 \n",
       "L 246.087555 108.442771 \n",
       "L 247.208037 107.351205 \n",
       "L 250.569482 107.351205 \n",
       "L 251.689964 106.259639 \n",
       "L 255.05141 106.259639 \n",
       "L 256.171892 105.168072 \n",
       "L 259.533338 105.168072 \n",
       "L 260.65382 104.076506 \n",
       "L 264.015265 104.076506 \n",
       "L 265.135747 102.98494 \n",
       "L 267.376711 102.98494 \n",
       "L 268.497193 101.893373 \n",
       "L 271.858639 101.893373 \n",
       "L 272.979121 100.801807 \n",
       "L 276.340567 100.801807 \n",
       "L 277.461049 99.710241 \n",
       "L 280.822494 99.710241 \n",
       "L 281.942976 98.618675 \n",
       "L 285.304422 98.618675 \n",
       "L 286.424904 97.527108 \n",
       "L 289.78635 97.527108 \n",
       "L 290.906832 96.435542 \n",
       "L 294.268277 96.435542 \n",
       "L 295.388759 95.343976 \n",
       "L 298.750205 95.343976 \n",
       "L 299.870687 94.25241 \n",
       "L 302.111651 94.25241 \n",
       "L 303.232133 93.160843 \n",
       "L 306.593579 93.160843 \n",
       "L 307.714061 92.069277 \n",
       "L 311.075506 92.069277 \n",
       "L 312.195988 90.977711 \n",
       "L 315.557434 90.977711 \n",
       "L 316.677916 89.886145 \n",
       "L 320.039362 89.886145 \n",
       "L 321.159844 88.794578 \n",
       "L 321.159844 88.794578 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_7\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 44.586145 \n",
       "L 46.641771 48.95241 \n",
       "L 47.762253 48.95241 \n",
       "L 55.605627 56.593373 \n",
       "L 56.726109 56.593373 \n",
       "L 65.689964 65.325904 \n",
       "L 66.810446 65.325904 \n",
       "L 74.65382 72.966867 \n",
       "L 75.774302 72.966867 \n",
       "L 83.617675 80.607831 \n",
       "L 84.738157 80.607831 \n",
       "L 92.58153 88.248795 \n",
       "L 93.702012 88.248795 \n",
       "L 101.545386 95.889759 \n",
       "L 102.665868 95.889759 \n",
       "L 110.509241 103.530723 \n",
       "L 111.629723 103.530723 \n",
       "L 119.473097 111.171687 \n",
       "L 120.593579 111.171687 \n",
       "L 128.436952 118.812651 \n",
       "L 129.557434 118.812651 \n",
       "L 138.52129 127.545181 \n",
       "L 139.641771 127.545181 \n",
       "L 144.964061 132.73012 \n",
       "L 145.244181 133.275904 \n",
       "L 146.364663 133.275904 \n",
       "L 147.485145 132.184337 \n",
       "L 150.846591 132.184337 \n",
       "L 151.967073 131.092771 \n",
       "L 155.328518 131.092771 \n",
       "L 156.449 130.001205 \n",
       "L 159.810446 130.001205 \n",
       "L 160.930928 128.909639 \n",
       "L 163.171892 128.909639 \n",
       "L 164.292374 127.818072 \n",
       "L 167.65382 127.818072 \n",
       "L 168.774302 126.726506 \n",
       "L 172.135747 126.726506 \n",
       "L 173.256229 125.63494 \n",
       "L 176.617675 125.63494 \n",
       "L 177.738157 124.543373 \n",
       "L 181.099603 124.543373 \n",
       "L 182.220085 123.451807 \n",
       "L 185.58153 123.451807 \n",
       "L 186.702012 122.360241 \n",
       "L 190.063458 122.360241 \n",
       "L 191.18394 121.268675 \n",
       "L 194.545386 121.268675 \n",
       "L 195.665868 120.177108 \n",
       "L 197.906832 120.177108 \n",
       "L 199.027314 119.085542 \n",
       "L 202.388759 119.085542 \n",
       "L 203.509241 117.993976 \n",
       "L 206.870687 117.993976 \n",
       "L 207.991169 116.90241 \n",
       "L 211.352615 116.90241 \n",
       "L 212.473097 115.810843 \n",
       "L 215.834543 115.810843 \n",
       "L 216.955024 114.719277 \n",
       "L 220.31647 114.719277 \n",
       "L 221.436952 113.627711 \n",
       "L 224.798398 113.627711 \n",
       "L 225.91888 112.536145 \n",
       "L 229.280326 112.536145 \n",
       "L 230.400808 111.444578 \n",
       "L 232.641771 111.444578 \n",
       "L 233.762253 110.353012 \n",
       "L 237.123699 110.353012 \n",
       "L 238.244181 109.261446 \n",
       "L 241.605627 109.261446 \n",
       "L 242.726109 108.16988 \n",
       "L 246.087555 108.16988 \n",
       "L 247.208037 107.078313 \n",
       "L 250.569482 107.078313 \n",
       "L 251.689964 105.986747 \n",
       "L 255.05141 105.986747 \n",
       "L 256.171892 104.895181 \n",
       "L 259.533338 104.895181 \n",
       "L 260.65382 103.803614 \n",
       "L 264.015265 103.803614 \n",
       "L 265.135747 102.712048 \n",
       "L 267.376711 102.712048 \n",
       "L 268.497193 101.620482 \n",
       "L 271.858639 101.620482 \n",
       "L 272.979121 100.528916 \n",
       "L 276.340567 100.528916 \n",
       "L 277.461049 99.437349 \n",
       "L 280.822494 99.437349 \n",
       "L 281.942976 98.345783 \n",
       "L 285.304422 98.345783 \n",
       "L 286.424904 97.254217 \n",
       "L 289.78635 97.254217 \n",
       "L 290.906832 96.162651 \n",
       "L 294.268277 96.162651 \n",
       "L 295.388759 95.071084 \n",
       "L 298.750205 95.071084 \n",
       "L 299.870687 93.979518 \n",
       "L 302.111651 93.979518 \n",
       "L 303.232133 92.887952 \n",
       "L 306.593579 92.887952 \n",
       "L 307.714061 91.796386 \n",
       "L 311.075506 91.796386 \n",
       "L 312.195988 90.704819 \n",
       "L 315.557434 90.704819 \n",
       "L 316.677916 89.613253 \n",
       "L 320.039362 89.613253 \n",
       "L 321.159844 88.521687 \n",
       "L 321.159844 88.521687 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_8\">\n",
       "    <path clip-path=\"url(#p6b34b769b6)\" d=\"M 42.159844 44.449699 \n",
       "L 46.641771 48.815964 \n",
       "L 47.762253 48.815964 \n",
       "L 55.605627 56.456928 \n",
       "L 56.726109 56.456928 \n",
       "L 65.689964 65.189458 \n",
       "L 66.810446 65.189458 \n",
       "L 74.65382 72.830422 \n",
       "L 75.774302 72.830422 \n",
       "L 83.617675 80.471386 \n",
       "L 84.738157 80.471386 \n",
       "L 92.58153 88.112349 \n",
       "L 93.702012 88.112349 \n",
       "L 101.545386 95.753313 \n",
       "L 102.665868 95.753313 \n",
       "L 110.509241 103.394277 \n",
       "L 111.629723 103.394277 \n",
       "L 119.473097 111.035241 \n",
       "L 120.593579 111.035241 \n",
       "L 128.436952 118.676205 \n",
       "L 129.557434 118.676205 \n",
       "L 138.52129 127.408735 \n",
       "L 139.641771 127.408735 \n",
       "L 145.244181 133.003012 \n",
       "L 146.364663 133.003012 \n",
       "L 147.485145 131.911446 \n",
       "L 150.846591 131.911446 \n",
       "L 151.967073 130.81988 \n",
       "L 155.328518 130.81988 \n",
       "L 156.449 129.728313 \n",
       "L 159.810446 129.728313 \n",
       "L 160.930928 128.636747 \n",
       "L 163.171892 128.636747 \n",
       "L 164.292374 127.545181 \n",
       "L 167.65382 127.545181 \n",
       "L 168.774302 126.453614 \n",
       "L 172.135747 126.453614 \n",
       "L 173.256229 125.362048 \n",
       "L 176.617675 125.362048 \n",
       "L 177.738157 124.270482 \n",
       "L 181.099603 124.270482 \n",
       "L 182.220085 123.178916 \n",
       "L 185.58153 123.178916 \n",
       "L 186.702012 122.087349 \n",
       "L 190.063458 122.087349 \n",
       "L 191.18394 120.995783 \n",
       "L 194.545386 120.995783 \n",
       "L 195.665868 119.904217 \n",
       "L 197.906832 119.904217 \n",
       "L 199.027314 118.812651 \n",
       "L 202.388759 118.812651 \n",
       "L 203.509241 117.721084 \n",
       "L 206.870687 117.721084 \n",
       "L 207.991169 116.629518 \n",
       "L 211.352615 116.629518 \n",
       "L 212.473097 115.537952 \n",
       "L 215.834543 115.537952 \n",
       "L 216.955024 114.446386 \n",
       "L 220.31647 114.446386 \n",
       "L 221.436952 113.354819 \n",
       "L 224.798398 113.354819 \n",
       "L 225.91888 112.263253 \n",
       "L 229.280326 112.263253 \n",
       "L 230.400808 111.171687 \n",
       "L 232.641771 111.171687 \n",
       "L 233.762253 110.08012 \n",
       "L 237.123699 110.08012 \n",
       "L 238.244181 108.988554 \n",
       "L 241.605627 108.988554 \n",
       "L 242.726109 107.896988 \n",
       "L 246.087555 107.896988 \n",
       "L 247.208037 106.805422 \n",
       "L 250.569482 106.805422 \n",
       "L 251.689964 105.713855 \n",
       "L 255.05141 105.713855 \n",
       "L 256.171892 104.622289 \n",
       "L 259.533338 104.622289 \n",
       "L 260.65382 103.530723 \n",
       "L 264.015265 103.530723 \n",
       "L 265.135747 102.439157 \n",
       "L 267.376711 102.439157 \n",
       "L 268.497193 101.34759 \n",
       "L 271.858639 101.34759 \n",
       "L 272.979121 100.256024 \n",
       "L 276.340567 100.256024 \n",
       "L 277.461049 99.164458 \n",
       "L 280.822494 99.164458 \n",
       "L 281.942976 98.072892 \n",
       "L 285.304422 98.072892 \n",
       "L 286.424904 96.981325 \n",
       "L 289.78635 96.981325 \n",
       "L 290.906832 95.889759 \n",
       "L 294.268277 95.889759 \n",
       "L 295.388759 94.798193 \n",
       "L 298.750205 94.798193 \n",
       "L 299.870687 93.706627 \n",
       "L 302.111651 93.706627 \n",
       "L 303.232133 92.61506 \n",
       "L 306.593579 92.61506 \n",
       "L 307.714061 91.523494 \n",
       "L 311.075506 91.523494 \n",
       "L 312.195988 90.431928 \n",
       "L 315.557434 90.431928 \n",
       "L 316.677916 89.340361 \n",
       "L 320.039362 89.340361 \n",
       "L 321.159844 88.248795 \n",
       "L 321.159844 88.248795 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_9\"/>\n",
       "   <g id=\"patch_3\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 42.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_4\">\n",
       "    <path d=\"M 321.159844 279 \n",
       "L 321.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_5\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 321.159844 279 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_6\">\n",
       "    <path d=\"M 42.159844 7.2 \n",
       "L 321.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"legend_1\">\n",
       "    <g id=\"line2d_25\"/>\n",
       "    <g id=\"line2d_26\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"mdec9be64f4\" style=\"stroke:#1f77b4;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#1f77b4;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#mdec9be64f4\" y=\"19.857812\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_15\">\n",
       "     <!-- setosa -->\n",
       "     <defs>\n",
       "      <path d=\"M 3.078125 15.484375 \n",
       "L 11.765625 16.84375 \n",
       "Q 12.5 11.625 15.84375 8.84375 \n",
       "Q 19.1875 6.0625 25.203125 6.0625 \n",
       "Q 31.25 6.0625 34.171875 8.515625 \n",
       "Q 37.109375 10.984375 37.109375 14.3125 \n",
       "Q 37.109375 17.28125 34.515625 19 \n",
       "Q 32.71875 20.171875 25.53125 21.96875 \n",
       "Q 15.875 24.421875 12.140625 26.203125 \n",
       "Q 8.40625 27.984375 6.46875 31.125 \n",
       "Q 4.546875 34.28125 4.546875 38.09375 \n",
       "Q 4.546875 41.546875 6.125 44.5 \n",
       "Q 7.71875 47.46875 10.453125 49.421875 \n",
       "Q 12.5 50.921875 16.03125 51.96875 \n",
       "Q 19.578125 53.03125 23.640625 53.03125 \n",
       "Q 29.734375 53.03125 34.34375 51.265625 \n",
       "Q 38.96875 49.515625 41.15625 46.5 \n",
       "Q 43.359375 43.5 44.1875 38.484375 \n",
       "L 35.59375 37.3125 \n",
       "Q 35.015625 41.3125 32.203125 43.546875 \n",
       "Q 29.390625 45.796875 24.265625 45.796875 \n",
       "Q 18.21875 45.796875 15.625 43.796875 \n",
       "Q 13.03125 41.796875 13.03125 39.109375 \n",
       "Q 13.03125 37.40625 14.109375 36.03125 \n",
       "Q 15.1875 34.625 17.484375 33.6875 \n",
       "Q 18.796875 33.203125 25.25 31.453125 \n",
       "Q 34.578125 28.953125 38.25 27.359375 \n",
       "Q 41.9375 25.78125 44.03125 22.75 \n",
       "Q 46.140625 19.734375 46.140625 15.234375 \n",
       "Q 46.140625 10.84375 43.578125 6.953125 \n",
       "Q 41.015625 3.078125 36.171875 0.953125 \n",
       "Q 31.34375 -1.171875 25.25 -1.171875 \n",
       "Q 15.140625 -1.171875 9.84375 3.03125 \n",
       "Q 4.546875 7.234375 3.078125 15.484375 \n",
       "z\n",
       "\" id=\"ArialMT-115\"/>\n",
       "      <path d=\"M 3.328125 25.921875 \n",
       "Q 3.328125 40.328125 11.328125 47.265625 \n",
       "Q 18.015625 53.03125 27.640625 53.03125 \n",
       "Q 38.328125 53.03125 45.109375 46.015625 \n",
       "Q 51.90625 39.015625 51.90625 26.65625 \n",
       "Q 51.90625 16.65625 48.90625 10.90625 \n",
       "Q 45.90625 5.171875 40.15625 2 \n",
       "Q 34.421875 -1.171875 27.640625 -1.171875 \n",
       "Q 16.75 -1.171875 10.03125 5.8125 \n",
       "Q 3.328125 12.796875 3.328125 25.921875 \n",
       "z\n",
       "M 12.359375 25.921875 \n",
       "Q 12.359375 15.96875 16.703125 11.015625 \n",
       "Q 21.046875 6.0625 27.640625 6.0625 \n",
       "Q 34.1875 6.0625 38.53125 11.03125 \n",
       "Q 42.875 16.015625 42.875 26.21875 \n",
       "Q 42.875 35.84375 38.5 40.796875 \n",
       "Q 34.125 45.75 27.640625 45.75 \n",
       "Q 21.046875 45.75 16.703125 40.8125 \n",
       "Q 12.359375 35.890625 12.359375 25.921875 \n",
       "z\n",
       "\" id=\"ArialMT-111\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 23.357812)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"105.615234\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"133.398438\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"189.013672\" xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"239.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"line2d_27\"/>\n",
       "    <g id=\"line2d_28\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"m71308f9d4a\" style=\"stroke:#8c564b;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#8c564b;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#m71308f9d4a\" y=\"34.003125\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_16\">\n",
       "     <!-- versicolor -->\n",
       "     <defs>\n",
       "      <path d=\"M 21 0 \n",
       "L 1.265625 51.859375 \n",
       "L 10.546875 51.859375 \n",
       "L 21.6875 20.796875 \n",
       "Q 23.484375 15.765625 25 10.359375 \n",
       "Q 26.171875 14.453125 28.265625 20.21875 \n",
       "L 39.796875 51.859375 \n",
       "L 48.828125 51.859375 \n",
       "L 29.203125 0 \n",
       "z\n",
       "\" id=\"ArialMT-118\"/>\n",
       "      <path d=\"M 6.5 0 \n",
       "L 6.5 51.859375 \n",
       "L 14.40625 51.859375 \n",
       "L 14.40625 44 \n",
       "Q 17.4375 49.515625 20 51.265625 \n",
       "Q 22.5625 53.03125 25.640625 53.03125 \n",
       "Q 30.078125 53.03125 34.671875 50.203125 \n",
       "L 31.640625 42.046875 \n",
       "Q 28.421875 43.953125 25.203125 43.953125 \n",
       "Q 22.3125 43.953125 20.015625 42.21875 \n",
       "Q 17.71875 40.484375 16.75 37.40625 \n",
       "Q 15.28125 32.71875 15.28125 27.15625 \n",
       "L 15.28125 0 \n",
       "z\n",
       "\" id=\"ArialMT-114\"/>\n",
       "      <path d=\"M 40.4375 19 \n",
       "L 49.078125 17.875 \n",
       "Q 47.65625 8.9375 41.8125 3.875 \n",
       "Q 35.984375 -1.171875 27.484375 -1.171875 \n",
       "Q 16.84375 -1.171875 10.375 5.78125 \n",
       "Q 3.90625 12.75 3.90625 25.734375 \n",
       "Q 3.90625 34.125 6.6875 40.421875 \n",
       "Q 9.46875 46.734375 15.15625 49.875 \n",
       "Q 20.84375 53.03125 27.546875 53.03125 \n",
       "Q 35.984375 53.03125 41.359375 48.75 \n",
       "Q 46.734375 44.484375 48.25 36.625 \n",
       "L 39.703125 35.296875 \n",
       "Q 38.484375 40.53125 35.375 43.15625 \n",
       "Q 32.28125 45.796875 27.875 45.796875 \n",
       "Q 21.234375 45.796875 17.078125 41.03125 \n",
       "Q 12.9375 36.28125 12.9375 25.984375 \n",
       "Q 12.9375 15.53125 16.9375 10.796875 \n",
       "Q 20.953125 6.0625 27.390625 6.0625 \n",
       "Q 32.5625 6.0625 36.03125 9.234375 \n",
       "Q 39.5 12.40625 40.4375 19 \n",
       "z\n",
       "\" id=\"ArialMT-99\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 37.503125)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-118\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"105.615234\" xlink:href=\"#ArialMT-114\"/>\n",
       "      <use x=\"138.916016\" xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"188.916016\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"211.132812\" xlink:href=\"#ArialMT-99\"/>\n",
       "      <use x=\"261.132812\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"316.748047\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"338.964844\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"394.580078\" xlink:href=\"#ArialMT-114\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"line2d_29\"/>\n",
       "    <g id=\"line2d_30\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"m1791aa6d01\" style=\"stroke:#17becf;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#17becf;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#m1791aa6d01\" y=\"48.148437\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_17\">\n",
       "     <!-- virginica -->\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 51.648437)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-118\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"72.216797\" xlink:href=\"#ArialMT-114\"/>\n",
       "      <use x=\"105.517578\" xlink:href=\"#ArialMT-103\"/>\n",
       "      <use x=\"161.132812\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"183.349609\" xlink:href=\"#ArialMT-110\"/>\n",
       "      <use x=\"238.964844\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"261.181641\" xlink:href=\"#ArialMT-99\"/>\n",
       "      <use x=\"311.181641\" xlink:href=\"#ArialMT-97\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "  </g>\n",
       " </g>\n",
       " <defs>\n",
       "  <clipPath id=\"p6b34b769b6\">\n",
       "   <rect height=\"271.8\" width=\"279\" x=\"42.159844\" y=\"7.2\"/>\n",
       "  </clipPath>\n",
       " </defs>\n",
       "</svg>\n"
      ],
      "text/plain": [
       "<Figure size 360x360 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "from sklearn.svm import LinearSVC\n",
    "\n",
    "cls = LinearSVC()\n",
    "cls.fit(X2, Y)\n",
    "plot_boundary(cls, X2, Y)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "And for a decision tree, the decision boundary has the characteristic shape with right angles."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/svg+xml": [
       "<?xml version=\"1.0\" encoding=\"utf-8\" standalone=\"no\"?>\n",
       "<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n",
       "  \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n",
       "<!-- Created with matplotlib (https://matplotlib.org/) -->\n",
       "<svg height=\"316.534063pt\" version=\"1.1\" viewBox=\"0 0 328.359844 316.534063\" width=\"328.359844pt\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n",
       " <defs>\n",
       "  <style type=\"text/css\">\n",
       "*{stroke-linecap:butt;stroke-linejoin:round;}\n",
       "  </style>\n",
       " </defs>\n",
       " <g id=\"figure_1\">\n",
       "  <g id=\"patch_1\">\n",
       "   <path d=\"M 0 316.534063 \n",
       "L 328.359844 316.534063 \n",
       "L 328.359844 0 \n",
       "L 0 0 \n",
       "z\n",
       "\" style=\"fill:none;\"/>\n",
       "  </g>\n",
       "  <g id=\"axes_1\">\n",
       "   <g id=\"patch_2\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 321.159844 279 \n",
       "L 321.159844 7.2 \n",
       "L 42.159844 7.2 \n",
       "z\n",
       "\" style=\"fill:#eaeaf2;\"/>\n",
       "   </g>\n",
       "   <g id=\"matplotlib.axis_1\">\n",
       "    <g id=\"xtick_1\">\n",
       "     <g id=\"line2d_1\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 52.493177 279 \n",
       "L 52.493177 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_2\"/>\n",
       "     <g id=\"text_1\">\n",
       "      <!-- 1 -->\n",
       "      <defs>\n",
       "       <path d=\"M 37.25 0 \n",
       "L 28.46875 0 \n",
       "L 28.46875 56 \n",
       "Q 25.296875 52.984375 20.140625 49.953125 \n",
       "Q 14.984375 46.921875 10.890625 45.40625 \n",
       "L 10.890625 53.90625 \n",
       "Q 18.265625 57.375 23.78125 62.296875 \n",
       "Q 29.296875 67.234375 31.59375 71.875 \n",
       "L 37.25 71.875 \n",
       "z\n",
       "\" id=\"ArialMT-49\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(49.712708 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_2\">\n",
       "     <g id=\"line2d_3\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 96.278488 279 \n",
       "L 96.278488 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_4\"/>\n",
       "     <g id=\"text_2\">\n",
       "      <!-- 2 -->\n",
       "      <defs>\n",
       "       <path d=\"M 50.34375 8.453125 \n",
       "L 50.34375 0 \n",
       "L 3.03125 0 \n",
       "Q 2.9375 3.171875 4.046875 6.109375 \n",
       "Q 5.859375 10.9375 9.828125 15.625 \n",
       "Q 13.8125 20.3125 21.34375 26.46875 \n",
       "Q 33.015625 36.03125 37.109375 41.625 \n",
       "Q 41.21875 47.21875 41.21875 52.203125 \n",
       "Q 41.21875 57.421875 37.46875 61 \n",
       "Q 33.734375 64.59375 27.734375 64.59375 \n",
       "Q 21.390625 64.59375 17.578125 60.78125 \n",
       "Q 13.765625 56.984375 13.71875 50.25 \n",
       "L 4.6875 51.171875 \n",
       "Q 5.609375 61.28125 11.65625 66.578125 \n",
       "Q 17.71875 71.875 27.9375 71.875 \n",
       "Q 38.234375 71.875 44.234375 66.15625 \n",
       "Q 50.25 60.453125 50.25 52 \n",
       "Q 50.25 47.703125 48.484375 43.546875 \n",
       "Q 46.734375 39.40625 42.65625 34.8125 \n",
       "Q 38.578125 30.21875 29.109375 22.21875 \n",
       "Q 21.1875 15.578125 18.9375 13.203125 \n",
       "Q 16.703125 10.84375 15.234375 8.453125 \n",
       "z\n",
       "\" id=\"ArialMT-50\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(93.498019 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_3\">\n",
       "     <g id=\"line2d_5\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 140.063799 279 \n",
       "L 140.063799 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_6\"/>\n",
       "     <g id=\"text_3\">\n",
       "      <!-- 3 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.203125 18.890625 \n",
       "L 12.984375 20.0625 \n",
       "Q 14.5 12.59375 18.140625 9.296875 \n",
       "Q 21.78125 6 27 6 \n",
       "Q 33.203125 6 37.46875 10.296875 \n",
       "Q 41.75 14.59375 41.75 20.953125 \n",
       "Q 41.75 27 37.796875 30.921875 \n",
       "Q 33.84375 34.859375 27.734375 34.859375 \n",
       "Q 25.25 34.859375 21.53125 33.890625 \n",
       "L 22.515625 41.609375 \n",
       "Q 23.390625 41.5 23.921875 41.5 \n",
       "Q 29.546875 41.5 34.03125 44.421875 \n",
       "Q 38.53125 47.359375 38.53125 53.46875 \n",
       "Q 38.53125 58.296875 35.25 61.46875 \n",
       "Q 31.984375 64.65625 26.8125 64.65625 \n",
       "Q 21.6875 64.65625 18.265625 61.421875 \n",
       "Q 14.84375 58.203125 13.875 51.765625 \n",
       "L 5.078125 53.328125 \n",
       "Q 6.6875 62.15625 12.390625 67.015625 \n",
       "Q 18.109375 71.875 26.609375 71.875 \n",
       "Q 32.46875 71.875 37.390625 69.359375 \n",
       "Q 42.328125 66.84375 44.9375 62.5 \n",
       "Q 47.5625 58.15625 47.5625 53.265625 \n",
       "Q 47.5625 48.640625 45.0625 44.828125 \n",
       "Q 42.578125 41.015625 37.703125 38.765625 \n",
       "Q 44.046875 37.3125 47.5625 32.6875 \n",
       "Q 51.078125 28.078125 51.078125 21.140625 \n",
       "Q 51.078125 11.765625 44.234375 5.25 \n",
       "Q 37.40625 -1.265625 26.953125 -1.265625 \n",
       "Q 17.53125 -1.265625 11.296875 4.34375 \n",
       "Q 5.078125 9.96875 4.203125 18.890625 \n",
       "z\n",
       "\" id=\"ArialMT-51\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(137.28333 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-51\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_4\">\n",
       "     <g id=\"line2d_7\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 183.849109 279 \n",
       "L 183.849109 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_8\"/>\n",
       "     <g id=\"text_4\">\n",
       "      <!-- 4 -->\n",
       "      <defs>\n",
       "       <path d=\"M 32.328125 0 \n",
       "L 32.328125 17.140625 \n",
       "L 1.265625 17.140625 \n",
       "L 1.265625 25.203125 \n",
       "L 33.9375 71.578125 \n",
       "L 41.109375 71.578125 \n",
       "L 41.109375 25.203125 \n",
       "L 50.78125 25.203125 \n",
       "L 50.78125 17.140625 \n",
       "L 41.109375 17.140625 \n",
       "L 41.109375 0 \n",
       "z\n",
       "M 32.328125 25.203125 \n",
       "L 32.328125 57.46875 \n",
       "L 9.90625 25.203125 \n",
       "z\n",
       "\" id=\"ArialMT-52\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(181.068641 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-52\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_5\">\n",
       "     <g id=\"line2d_9\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 227.63442 279 \n",
       "L 227.63442 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_10\"/>\n",
       "     <g id=\"text_5\">\n",
       "      <!-- 5 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.15625 18.75 \n",
       "L 13.375 19.53125 \n",
       "Q 14.40625 12.796875 18.140625 9.390625 \n",
       "Q 21.875 6 27.15625 6 \n",
       "Q 33.5 6 37.890625 10.78125 \n",
       "Q 42.28125 15.578125 42.28125 23.484375 \n",
       "Q 42.28125 31 38.0625 35.34375 \n",
       "Q 33.84375 39.703125 27 39.703125 \n",
       "Q 22.75 39.703125 19.328125 37.765625 \n",
       "Q 15.921875 35.84375 13.96875 32.765625 \n",
       "L 5.71875 33.84375 \n",
       "L 12.640625 70.609375 \n",
       "L 48.25 70.609375 \n",
       "L 48.25 62.203125 \n",
       "L 19.671875 62.203125 \n",
       "L 15.828125 42.96875 \n",
       "Q 22.265625 47.46875 29.34375 47.46875 \n",
       "Q 38.71875 47.46875 45.15625 40.96875 \n",
       "Q 51.609375 34.46875 51.609375 24.265625 \n",
       "Q 51.609375 14.546875 45.953125 7.46875 \n",
       "Q 39.0625 -1.21875 27.15625 -1.21875 \n",
       "Q 17.390625 -1.21875 11.203125 4.25 \n",
       "Q 5.03125 9.71875 4.15625 18.75 \n",
       "z\n",
       "\" id=\"ArialMT-53\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(224.853951 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_6\">\n",
       "     <g id=\"line2d_11\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 271.419731 279 \n",
       "L 271.419731 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_12\"/>\n",
       "     <g id=\"text_6\">\n",
       "      <!-- 6 -->\n",
       "      <defs>\n",
       "       <path d=\"M 49.75 54.046875 \n",
       "L 41.015625 53.375 \n",
       "Q 39.84375 58.546875 37.703125 60.890625 \n",
       "Q 34.125 64.65625 28.90625 64.65625 \n",
       "Q 24.703125 64.65625 21.53125 62.3125 \n",
       "Q 17.390625 59.28125 14.984375 53.46875 \n",
       "Q 12.59375 47.65625 12.5 36.921875 \n",
       "Q 15.671875 41.75 20.265625 44.09375 \n",
       "Q 24.859375 46.4375 29.890625 46.4375 \n",
       "Q 38.671875 46.4375 44.84375 39.96875 \n",
       "Q 51.03125 33.5 51.03125 23.25 \n",
       "Q 51.03125 16.5 48.125 10.71875 \n",
       "Q 45.21875 4.9375 40.140625 1.859375 \n",
       "Q 35.0625 -1.21875 28.609375 -1.21875 \n",
       "Q 17.625 -1.21875 10.6875 6.859375 \n",
       "Q 3.765625 14.9375 3.765625 33.5 \n",
       "Q 3.765625 54.25 11.421875 63.671875 \n",
       "Q 18.109375 71.875 29.4375 71.875 \n",
       "Q 37.890625 71.875 43.28125 67.140625 \n",
       "Q 48.6875 62.40625 49.75 54.046875 \n",
       "z\n",
       "M 13.875 23.1875 \n",
       "Q 13.875 18.65625 15.796875 14.5 \n",
       "Q 17.71875 10.359375 21.1875 8.171875 \n",
       "Q 24.65625 6 28.46875 6 \n",
       "Q 34.03125 6 38.03125 10.484375 \n",
       "Q 42.046875 14.984375 42.046875 22.703125 \n",
       "Q 42.046875 30.125 38.078125 34.390625 \n",
       "Q 34.125 38.671875 28.125 38.671875 \n",
       "Q 22.171875 38.671875 18.015625 34.390625 \n",
       "Q 13.875 30.125 13.875 23.1875 \n",
       "z\n",
       "\" id=\"ArialMT-54\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(268.639262 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-54\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_7\">\n",
       "     <g id=\"line2d_13\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 315.205041 279 \n",
       "L 315.205041 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_14\"/>\n",
       "     <g id=\"text_7\">\n",
       "      <!-- 7 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.734375 62.203125 \n",
       "L 4.734375 70.65625 \n",
       "L 51.078125 70.65625 \n",
       "L 51.078125 63.8125 \n",
       "Q 44.234375 56.546875 37.515625 44.484375 \n",
       "Q 30.8125 32.421875 27.15625 19.671875 \n",
       "Q 24.515625 10.6875 23.78125 0 \n",
       "L 14.75 0 \n",
       "Q 14.890625 8.453125 18.0625 20.40625 \n",
       "Q 21.234375 32.375 27.171875 43.484375 \n",
       "Q 33.109375 54.59375 39.796875 62.203125 \n",
       "z\n",
       "\" id=\"ArialMT-55\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(312.424573 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-55\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_8\">\n",
       "     <!-- petal_length -->\n",
       "     <defs>\n",
       "      <path d=\"M 6.59375 -19.875 \n",
       "L 6.59375 51.859375 \n",
       "L 14.59375 51.859375 \n",
       "L 14.59375 45.125 \n",
       "Q 17.4375 49.078125 21 51.046875 \n",
       "Q 24.5625 53.03125 29.640625 53.03125 \n",
       "Q 36.28125 53.03125 41.359375 49.609375 \n",
       "Q 46.4375 46.1875 49.015625 39.953125 \n",
       "Q 51.609375 33.734375 51.609375 26.3125 \n",
       "Q 51.609375 18.359375 48.75 11.984375 \n",
       "Q 45.90625 5.609375 40.453125 2.21875 \n",
       "Q 35.015625 -1.171875 29 -1.171875 \n",
       "Q 24.609375 -1.171875 21.109375 0.6875 \n",
       "Q 17.625 2.546875 15.375 5.375 \n",
       "L 15.375 -19.875 \n",
       "z\n",
       "M 14.546875 25.640625 \n",
       "Q 14.546875 15.625 18.59375 10.84375 \n",
       "Q 22.65625 6.0625 28.421875 6.0625 \n",
       "Q 34.28125 6.0625 38.453125 11.015625 \n",
       "Q 42.625 15.96875 42.625 26.375 \n",
       "Q 42.625 36.28125 38.546875 41.203125 \n",
       "Q 34.46875 46.140625 28.8125 46.140625 \n",
       "Q 23.1875 46.140625 18.859375 40.890625 \n",
       "Q 14.546875 35.640625 14.546875 25.640625 \n",
       "z\n",
       "\" id=\"ArialMT-112\"/>\n",
       "      <path d=\"M 42.09375 16.703125 \n",
       "L 51.171875 15.578125 \n",
       "Q 49.03125 7.625 43.21875 3.21875 \n",
       "Q 37.40625 -1.171875 28.375 -1.171875 \n",
       "Q 17 -1.171875 10.328125 5.828125 \n",
       "Q 3.65625 12.84375 3.65625 25.484375 \n",
       "Q 3.65625 38.578125 10.390625 45.796875 \n",
       "Q 17.140625 53.03125 27.875 53.03125 \n",
       "Q 38.28125 53.03125 44.875 45.953125 \n",
       "Q 51.46875 38.875 51.46875 26.03125 \n",
       "Q 51.46875 25.25 51.421875 23.6875 \n",
       "L 12.75 23.6875 \n",
       "Q 13.234375 15.140625 17.578125 10.59375 \n",
       "Q 21.921875 6.0625 28.421875 6.0625 \n",
       "Q 33.25 6.0625 36.671875 8.59375 \n",
       "Q 40.09375 11.140625 42.09375 16.703125 \n",
       "z\n",
       "M 13.234375 30.90625 \n",
       "L 42.1875 30.90625 \n",
       "Q 41.609375 37.453125 38.875 40.71875 \n",
       "Q 34.671875 45.796875 27.984375 45.796875 \n",
       "Q 21.921875 45.796875 17.796875 41.75 \n",
       "Q 13.671875 37.703125 13.234375 30.90625 \n",
       "z\n",
       "\" id=\"ArialMT-101\"/>\n",
       "      <path d=\"M 25.78125 7.859375 \n",
       "L 27.046875 0.09375 \n",
       "Q 23.34375 -0.6875 20.40625 -0.6875 \n",
       "Q 15.625 -0.6875 12.984375 0.828125 \n",
       "Q 10.359375 2.34375 9.28125 4.8125 \n",
       "Q 8.203125 7.28125 8.203125 15.1875 \n",
       "L 8.203125 45.015625 \n",
       "L 1.765625 45.015625 \n",
       "L 1.765625 51.859375 \n",
       "L 8.203125 51.859375 \n",
       "L 8.203125 64.703125 \n",
       "L 16.9375 69.96875 \n",
       "L 16.9375 51.859375 \n",
       "L 25.78125 51.859375 \n",
       "L 25.78125 45.015625 \n",
       "L 16.9375 45.015625 \n",
       "L 16.9375 14.703125 \n",
       "Q 16.9375 10.9375 17.40625 9.859375 \n",
       "Q 17.875 8.796875 18.921875 8.15625 \n",
       "Q 19.96875 7.515625 21.921875 7.515625 \n",
       "Q 23.390625 7.515625 25.78125 7.859375 \n",
       "z\n",
       "\" id=\"ArialMT-116\"/>\n",
       "      <path d=\"M 40.4375 6.390625 \n",
       "Q 35.546875 2.25 31.03125 0.53125 \n",
       "Q 26.515625 -1.171875 21.34375 -1.171875 \n",
       "Q 12.796875 -1.171875 8.203125 3 \n",
       "Q 3.609375 7.171875 3.609375 13.671875 \n",
       "Q 3.609375 17.484375 5.34375 20.625 \n",
       "Q 7.078125 23.78125 9.890625 25.6875 \n",
       "Q 12.703125 27.59375 16.21875 28.5625 \n",
       "Q 18.796875 29.25 24.03125 29.890625 \n",
       "Q 34.671875 31.15625 39.703125 32.90625 \n",
       "Q 39.75 34.71875 39.75 35.203125 \n",
       "Q 39.75 40.578125 37.25 42.78125 \n",
       "Q 33.890625 45.75 27.25 45.75 \n",
       "Q 21.046875 45.75 18.09375 43.578125 \n",
       "Q 15.140625 41.40625 13.71875 35.890625 \n",
       "L 5.125 37.0625 \n",
       "Q 6.296875 42.578125 8.984375 45.96875 \n",
       "Q 11.671875 49.359375 16.75 51.1875 \n",
       "Q 21.828125 53.03125 28.515625 53.03125 \n",
       "Q 35.15625 53.03125 39.296875 51.46875 \n",
       "Q 43.453125 49.90625 45.40625 47.53125 \n",
       "Q 47.359375 45.171875 48.140625 41.546875 \n",
       "Q 48.578125 39.3125 48.578125 33.453125 \n",
       "L 48.578125 21.734375 \n",
       "Q 48.578125 9.46875 49.140625 6.21875 \n",
       "Q 49.703125 2.984375 51.375 0 \n",
       "L 42.1875 0 \n",
       "Q 40.828125 2.734375 40.4375 6.390625 \n",
       "z\n",
       "M 39.703125 26.03125 \n",
       "Q 34.90625 24.078125 25.34375 22.703125 \n",
       "Q 19.921875 21.921875 17.671875 20.9375 \n",
       "Q 15.4375 19.96875 14.203125 18.09375 \n",
       "Q 12.984375 16.21875 12.984375 13.921875 \n",
       "Q 12.984375 10.40625 15.640625 8.0625 \n",
       "Q 18.3125 5.71875 23.4375 5.71875 \n",
       "Q 28.515625 5.71875 32.46875 7.9375 \n",
       "Q 36.421875 10.15625 38.28125 14.015625 \n",
       "Q 39.703125 17 39.703125 22.796875 \n",
       "z\n",
       "\" id=\"ArialMT-97\"/>\n",
       "      <path d=\"M 6.390625 0 \n",
       "L 6.390625 71.578125 \n",
       "L 15.1875 71.578125 \n",
       "L 15.1875 0 \n",
       "z\n",
       "\" id=\"ArialMT-108\"/>\n",
       "      <path d=\"M -1.515625 -19.875 \n",
       "L -1.515625 -13.53125 \n",
       "L 56.734375 -13.53125 \n",
       "L 56.734375 -19.875 \n",
       "z\n",
       "\" id=\"ArialMT-95\"/>\n",
       "      <path d=\"M 6.59375 0 \n",
       "L 6.59375 51.859375 \n",
       "L 14.5 51.859375 \n",
       "L 14.5 44.484375 \n",
       "Q 20.21875 53.03125 31 53.03125 \n",
       "Q 35.6875 53.03125 39.625 51.34375 \n",
       "Q 43.5625 49.65625 45.515625 46.921875 \n",
       "Q 47.46875 44.1875 48.25 40.4375 \n",
       "Q 48.734375 37.984375 48.734375 31.890625 \n",
       "L 48.734375 0 \n",
       "L 39.9375 0 \n",
       "L 39.9375 31.546875 \n",
       "Q 39.9375 36.921875 38.90625 39.578125 \n",
       "Q 37.890625 42.234375 35.28125 43.8125 \n",
       "Q 32.671875 45.40625 29.15625 45.40625 \n",
       "Q 23.53125 45.40625 19.453125 41.84375 \n",
       "Q 15.375 38.28125 15.375 28.328125 \n",
       "L 15.375 0 \n",
       "z\n",
       "\" id=\"ArialMT-110\"/>\n",
       "      <path d=\"M 4.984375 -4.296875 \n",
       "L 13.53125 -5.5625 \n",
       "Q 14.0625 -9.515625 16.5 -11.328125 \n",
       "Q 19.78125 -13.765625 25.4375 -13.765625 \n",
       "Q 31.546875 -13.765625 34.859375 -11.328125 \n",
       "Q 38.1875 -8.890625 39.359375 -4.5 \n",
       "Q 40.046875 -1.8125 39.984375 6.78125 \n",
       "Q 34.234375 0 25.640625 0 \n",
       "Q 14.9375 0 9.078125 7.71875 \n",
       "Q 3.21875 15.4375 3.21875 26.21875 \n",
       "Q 3.21875 33.640625 5.90625 39.90625 \n",
       "Q 8.59375 46.1875 13.6875 49.609375 \n",
       "Q 18.796875 53.03125 25.6875 53.03125 \n",
       "Q 34.859375 53.03125 40.828125 45.609375 \n",
       "L 40.828125 51.859375 \n",
       "L 48.921875 51.859375 \n",
       "L 48.921875 7.03125 \n",
       "Q 48.921875 -5.078125 46.453125 -10.125 \n",
       "Q 44 -15.1875 38.640625 -18.109375 \n",
       "Q 33.296875 -21.046875 25.484375 -21.046875 \n",
       "Q 16.21875 -21.046875 10.5 -16.875 \n",
       "Q 4.78125 -12.703125 4.984375 -4.296875 \n",
       "z\n",
       "M 12.25 26.859375 \n",
       "Q 12.25 16.65625 16.296875 11.96875 \n",
       "Q 20.359375 7.28125 26.46875 7.28125 \n",
       "Q 32.515625 7.28125 36.609375 11.9375 \n",
       "Q 40.71875 16.609375 40.71875 26.5625 \n",
       "Q 40.71875 36.078125 36.5 40.90625 \n",
       "Q 32.28125 45.75 26.3125 45.75 \n",
       "Q 20.453125 45.75 16.34375 40.984375 \n",
       "Q 12.25 36.234375 12.25 26.859375 \n",
       "z\n",
       "\" id=\"ArialMT-103\"/>\n",
       "      <path d=\"M 6.59375 0 \n",
       "L 6.59375 71.578125 \n",
       "L 15.375 71.578125 \n",
       "L 15.375 45.90625 \n",
       "Q 21.53125 53.03125 30.90625 53.03125 \n",
       "Q 36.671875 53.03125 40.921875 50.75 \n",
       "Q 45.171875 48.484375 47 44.484375 \n",
       "Q 48.828125 40.484375 48.828125 32.859375 \n",
       "L 48.828125 0 \n",
       "L 40.046875 0 \n",
       "L 40.046875 32.859375 \n",
       "Q 40.046875 39.453125 37.1875 42.453125 \n",
       "Q 34.328125 45.453125 29.109375 45.453125 \n",
       "Q 25.203125 45.453125 21.75 43.421875 \n",
       "Q 18.3125 41.40625 16.84375 37.9375 \n",
       "Q 15.375 34.46875 15.375 28.375 \n",
       "L 15.375 0 \n",
       "z\n",
       "\" id=\"ArialMT-104\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(151.691719 307.018906)scale(0.11 -0.11)\">\n",
       "      <use xlink:href=\"#ArialMT-112\"/>\n",
       "      <use x=\"55.615234\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"111.230469\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"139.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "      <use x=\"194.628906\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"216.845703\" xlink:href=\"#ArialMT-95\"/>\n",
       "      <use x=\"272.460938\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"294.677734\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"350.292969\" xlink:href=\"#ArialMT-110\"/>\n",
       "      <use x=\"405.908203\" xlink:href=\"#ArialMT-103\"/>\n",
       "      <use x=\"461.523438\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"489.306641\" xlink:href=\"#ArialMT-104\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"matplotlib.axis_2\">\n",
       "    <g id=\"ytick_1\">\n",
       "     <g id=\"line2d_15\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 226.988889 \n",
       "L 321.159844 226.988889 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_16\"/>\n",
       "     <g id=\"text_9\">\n",
       "      <!-- 0.5 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.15625 35.296875 \n",
       "Q 4.15625 48 6.765625 55.734375 \n",
       "Q 9.375 63.484375 14.515625 67.671875 \n",
       "Q 19.671875 71.875 27.484375 71.875 \n",
       "Q 33.25 71.875 37.59375 69.546875 \n",
       "Q 41.9375 67.234375 44.765625 62.859375 \n",
       "Q 47.609375 58.5 49.21875 52.21875 \n",
       "Q 50.828125 45.953125 50.828125 35.296875 \n",
       "Q 50.828125 22.703125 48.234375 14.96875 \n",
       "Q 45.65625 7.234375 40.5 3 \n",
       "Q 35.359375 -1.21875 27.484375 -1.21875 \n",
       "Q 17.140625 -1.21875 11.234375 6.203125 \n",
       "Q 4.15625 15.140625 4.15625 35.296875 \n",
       "z\n",
       "M 13.1875 35.296875 \n",
       "Q 13.1875 17.671875 17.3125 11.828125 \n",
       "Q 21.4375 6 27.484375 6 \n",
       "Q 33.546875 6 37.671875 11.859375 \n",
       "Q 41.796875 17.71875 41.796875 35.296875 \n",
       "Q 41.796875 52.984375 37.671875 58.78125 \n",
       "Q 33.546875 64.59375 27.390625 64.59375 \n",
       "Q 21.34375 64.59375 17.71875 59.46875 \n",
       "Q 13.1875 52.9375 13.1875 35.296875 \n",
       "z\n",
       "\" id=\"ArialMT-48\"/>\n",
       "       <path d=\"M 9.078125 0 \n",
       "L 9.078125 10.015625 \n",
       "L 19.09375 10.015625 \n",
       "L 19.09375 0 \n",
       "z\n",
       "\" id=\"ArialMT-46\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 230.567795)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-48\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_2\">\n",
       "     <g id=\"line2d_17\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 174.558333 \n",
       "L 321.159844 174.558333 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_18\"/>\n",
       "     <g id=\"text_10\">\n",
       "      <!-- 1.0 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 178.13724)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_3\">\n",
       "     <g id=\"line2d_19\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 122.127778 \n",
       "L 321.159844 122.127778 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_20\"/>\n",
       "     <g id=\"text_11\">\n",
       "      <!-- 1.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 125.706684)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_4\">\n",
       "     <g id=\"line2d_21\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 69.697222 \n",
       "L 321.159844 69.697222 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_22\"/>\n",
       "     <g id=\"text_12\">\n",
       "      <!-- 2.0 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 73.276128)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_5\">\n",
       "     <g id=\"line2d_23\">\n",
       "      <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 17.266667 \n",
       "L 321.159844 17.266667 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_24\"/>\n",
       "     <g id=\"text_13\">\n",
       "      <!-- 2.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 20.845573)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_14\">\n",
       "     <!-- petal_width -->\n",
       "     <defs>\n",
       "      <path d=\"M 16.15625 0 \n",
       "L 0.296875 51.859375 \n",
       "L 9.375 51.859375 \n",
       "L 17.625 21.921875 \n",
       "L 20.703125 10.796875 \n",
       "Q 20.90625 11.625 23.390625 21.484375 \n",
       "L 31.640625 51.859375 \n",
       "L 40.671875 51.859375 \n",
       "L 48.4375 21.78125 \n",
       "L 51.03125 11.859375 \n",
       "L 54 21.875 \n",
       "L 62.890625 51.859375 \n",
       "L 71.4375 51.859375 \n",
       "L 55.21875 0 \n",
       "L 46.09375 0 \n",
       "L 37.84375 31.0625 \n",
       "L 35.84375 39.890625 \n",
       "L 25.34375 0 \n",
       "z\n",
       "\" id=\"ArialMT-119\"/>\n",
       "      <path d=\"M 6.640625 61.46875 \n",
       "L 6.640625 71.578125 \n",
       "L 15.4375 71.578125 \n",
       "L 15.4375 61.46875 \n",
       "z\n",
       "M 6.640625 0 \n",
       "L 6.640625 51.859375 \n",
       "L 15.4375 51.859375 \n",
       "L 15.4375 0 \n",
       "z\n",
       "\" id=\"ArialMT-105\"/>\n",
       "      <path d=\"M 40.234375 0 \n",
       "L 40.234375 6.546875 \n",
       "Q 35.296875 -1.171875 25.734375 -1.171875 \n",
       "Q 19.53125 -1.171875 14.328125 2.25 \n",
       "Q 9.125 5.671875 6.265625 11.796875 \n",
       "Q 3.421875 17.921875 3.421875 25.875 \n",
       "Q 3.421875 33.640625 6 39.96875 \n",
       "Q 8.59375 46.296875 13.765625 49.65625 \n",
       "Q 18.953125 53.03125 25.34375 53.03125 \n",
       "Q 30.03125 53.03125 33.6875 51.046875 \n",
       "Q 37.359375 49.078125 39.65625 45.90625 \n",
       "L 39.65625 71.578125 \n",
       "L 48.390625 71.578125 \n",
       "L 48.390625 0 \n",
       "z\n",
       "M 12.453125 25.875 \n",
       "Q 12.453125 15.921875 16.640625 10.984375 \n",
       "Q 20.84375 6.0625 26.5625 6.0625 \n",
       "Q 32.328125 6.0625 36.34375 10.765625 \n",
       "Q 40.375 15.484375 40.375 25.140625 \n",
       "Q 40.375 35.796875 36.265625 40.765625 \n",
       "Q 32.171875 45.75 26.171875 45.75 \n",
       "Q 20.3125 45.75 16.375 40.96875 \n",
       "Q 12.453125 36.1875 12.453125 25.875 \n",
       "z\n",
       "\" id=\"ArialMT-100\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(15.073594 170.923125)rotate(-90)scale(0.11 -0.11)\">\n",
       "      <use xlink:href=\"#ArialMT-112\"/>\n",
       "      <use x=\"55.615234\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"111.230469\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"139.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "      <use x=\"194.628906\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"216.845703\" xlink:href=\"#ArialMT-95\"/>\n",
       "      <use x=\"272.460938\" xlink:href=\"#ArialMT-119\"/>\n",
       "      <use x=\"344.677734\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"366.894531\" xlink:href=\"#ArialMT-100\"/>\n",
       "      <use x=\"422.509766\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"450.292969\" xlink:href=\"#ArialMT-104\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_1\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 43.280326 279 \n",
       "L 44.400808 279 \n",
       "L 45.52129 279 \n",
       "L 46.641771 279 \n",
       "L 47.762253 279 \n",
       "L 48.882735 279 \n",
       "L 50.003217 279 \n",
       "L 51.123699 279 \n",
       "L 52.244181 279 \n",
       "L 53.364663 279 \n",
       "L 54.485145 279 \n",
       "L 55.605627 279 \n",
       "L 56.726109 279 \n",
       "L 57.846591 279 \n",
       "L 58.967073 279 \n",
       "L 60.087555 279 \n",
       "L 61.208037 279 \n",
       "L 62.328518 279 \n",
       "L 63.449 279 \n",
       "L 64.569482 279 \n",
       "L 65.689964 279 \n",
       "L 66.810446 279 \n",
       "L 67.930928 279 \n",
       "L 69.05141 279 \n",
       "L 70.171892 279 \n",
       "L 71.292374 279 \n",
       "L 72.412856 279 \n",
       "L 73.533338 279 \n",
       "L 74.65382 279 \n",
       "L 75.774302 279 \n",
       "L 76.894784 279 \n",
       "L 78.015265 279 \n",
       "L 79.135747 279 \n",
       "L 80.256229 279 \n",
       "L 81.376711 279 \n",
       "L 82.497193 279 \n",
       "L 83.617675 279 \n",
       "L 84.738157 279 \n",
       "L 85.858639 279 \n",
       "L 86.979121 279 \n",
       "L 88.099603 279 \n",
       "L 89.220085 279 \n",
       "L 90.340567 279 \n",
       "L 91.461049 279 \n",
       "L 92.58153 279 \n",
       "L 93.702012 279 \n",
       "L 94.822494 279 \n",
       "L 95.942976 279 \n",
       "L 97.063458 279 \n",
       "L 98.18394 279 \n",
       "L 99.304422 279 \n",
       "L 100.424904 279 \n",
       "L 101.545386 279 \n",
       "L 102.665868 279 \n",
       "L 103.78635 279 \n",
       "L 104.906832 279 \n",
       "L 106.027314 279 \n",
       "L 107.147796 279 \n",
       "L 108.268277 279 \n",
       "L 109.388759 279 \n",
       "L 110.509241 279 \n",
       "L 111.629723 279 \n",
       "L 112.750205 279 \n",
       "L 113.870687 279 \n",
       "L 114.991169 279 \n",
       "L 116.111651 279 \n",
       "L 117.232133 279 \n",
       "L 118.352615 279 \n",
       "L 119.473097 279 \n",
       "L 120.593579 279 \n",
       "L 121.714061 279 \n",
       "L 122.834543 279 \n",
       "L 123.955024 279 \n",
       "L 125.075506 279 \n",
       "L 126.195988 279 \n",
       "L 127.31647 279 \n",
       "L 128.436952 279 \n",
       "L 129.557434 279 \n",
       "L 130.677916 279 \n",
       "L 131.798398 279 \n",
       "L 132.91888 279 \n",
       "L 134.039362 279 \n",
       "L 135.159844 279 \n",
       "L 136.280326 279 \n",
       "L 137.400808 279 \n",
       "L 138.52129 279 \n",
       "L 139.641771 279 \n",
       "L 140.762253 279 \n",
       "L 141.882735 279 \n",
       "L 143.003217 279 \n",
       "L 144.123699 279 \n",
       "L 145.244181 279 \n",
       "L 146.364663 279 \n",
       "L 147.485145 279 \n",
       "L 148.605627 279 \n",
       "L 149.726109 279 \n",
       "L 150.846591 279 \n",
       "L 151.967073 279 \n",
       "L 153.087555 279 \n",
       "L 154.208037 279 \n",
       "L 155.328518 279 \n",
       "L 156.449 279 \n",
       "L 157.569482 279 \n",
       "L 158.689964 279 \n",
       "L 159.810446 279 \n",
       "L 160.930928 279 \n",
       "L 162.05141 279 \n",
       "L 163.171892 279 \n",
       "L 164.292374 279 \n",
       "L 165.412856 279 \n",
       "L 166.533338 279 \n",
       "L 167.65382 279 \n",
       "L 168.774302 279 \n",
       "L 169.894784 279 \n",
       "L 171.015265 279 \n",
       "L 172.135747 279 \n",
       "L 173.256229 279 \n",
       "L 174.376711 279 \n",
       "L 175.497193 279 \n",
       "L 176.617675 279 \n",
       "L 177.738157 279 \n",
       "L 178.858639 279 \n",
       "L 179.979121 279 \n",
       "L 181.099603 279 \n",
       "L 182.220085 279 \n",
       "L 183.340567 279 \n",
       "L 184.461049 279 \n",
       "L 185.58153 279 \n",
       "L 186.702012 279 \n",
       "L 187.822494 279 \n",
       "L 188.942976 279 \n",
       "L 190.063458 279 \n",
       "L 191.18394 279 \n",
       "L 192.304422 279 \n",
       "L 193.424904 279 \n",
       "L 194.545386 279 \n",
       "L 195.665868 279 \n",
       "L 196.78635 279 \n",
       "L 197.906832 279 \n",
       "L 199.027314 279 \n",
       "L 200.147796 279 \n",
       "L 201.268277 279 \n",
       "L 202.388759 279 \n",
       "L 203.509241 279 \n",
       "L 204.629723 279 \n",
       "L 205.750205 279 \n",
       "L 206.870687 279 \n",
       "L 207.991169 279 \n",
       "L 209.111651 279 \n",
       "L 210.232133 279 \n",
       "L 211.352615 279 \n",
       "L 212.473097 279 \n",
       "L 213.593579 279 \n",
       "L 214.714061 279 \n",
       "L 215.834543 279 \n",
       "L 216.955024 279 \n",
       "L 218.075506 279 \n",
       "L 219.195988 279 \n",
       "L 220.31647 279 \n",
       "L 221.436952 279 \n",
       "L 222.557434 279 \n",
       "L 223.677916 279 \n",
       "L 224.798398 279 \n",
       "L 225.91888 279 \n",
       "L 227.039362 279 \n",
       "L 228.159844 279 \n",
       "L 229.280326 279 \n",
       "L 230.400808 279 \n",
       "L 231.52129 279 \n",
       "L 232.641771 279 \n",
       "L 233.762253 279 \n",
       "L 234.882735 279 \n",
       "L 236.003217 279 \n",
       "L 237.123699 279 \n",
       "L 238.244181 279 \n",
       "L 239.364663 279 \n",
       "L 240.485145 279 \n",
       "L 241.605627 279 \n",
       "L 242.726109 279 \n",
       "L 243.846591 279 \n",
       "L 244.967073 279 \n",
       "L 246.087555 279 \n",
       "L 247.208037 279 \n",
       "L 248.328518 279 \n",
       "L 249.449 279 \n",
       "L 250.569482 279 \n",
       "L 251.689964 279 \n",
       "L 252.810446 279 \n",
       "L 253.930928 279 \n",
       "L 255.05141 279 \n",
       "L 256.171892 279 \n",
       "L 257.292374 279 \n",
       "L 258.412856 279 \n",
       "L 259.533338 279 \n",
       "L 260.65382 279 \n",
       "L 261.774302 279 \n",
       "L 262.894784 279 \n",
       "L 264.015265 279 \n",
       "L 265.135747 279 \n",
       "L 266.256229 279 \n",
       "L 267.376711 279 \n",
       "L 268.497193 279 \n",
       "L 269.617675 279 \n",
       "L 270.738157 279 \n",
       "L 271.858639 279 \n",
       "L 272.979121 279 \n",
       "L 274.099603 279 \n",
       "L 275.220085 279 \n",
       "L 276.340567 279 \n",
       "L 277.461049 279 \n",
       "L 278.58153 279 \n",
       "L 279.702012 279 \n",
       "L 280.822494 279 \n",
       "L 281.942976 279 \n",
       "L 283.063458 279 \n",
       "L 284.18394 279 \n",
       "L 285.304422 279 \n",
       "L 286.424904 279 \n",
       "L 287.545386 279 \n",
       "L 288.665868 279 \n",
       "L 289.78635 279 \n",
       "L 290.906832 279 \n",
       "L 292.027314 279 \n",
       "L 293.147796 279 \n",
       "L 294.268277 279 \n",
       "L 295.388759 279 \n",
       "L 296.509241 279 \n",
       "L 297.629723 279 \n",
       "L 298.750205 279 \n",
       "L 299.870687 279 \n",
       "L 300.991169 279 \n",
       "L 302.111651 279 \n",
       "L 303.232133 279 \n",
       "L 304.352615 279 \n",
       "L 305.473097 279 \n",
       "L 306.593579 279 \n",
       "L 307.714061 279 \n",
       "L 308.834543 279 \n",
       "L 309.955024 279 \n",
       "L 311.075506 279 \n",
       "L 312.195988 279 \n",
       "L 313.31647 279 \n",
       "L 314.436952 279 \n",
       "L 315.557434 279 \n",
       "L 316.677916 279 \n",
       "L 317.798398 279 \n",
       "L 318.91888 279 \n",
       "L 320.039362 279 \n",
       "L 321.159844 279 \n",
       "L 321.159844 277.908434 \n",
       "L 321.159844 276.816867 \n",
       "L 321.159844 275.725301 \n",
       "L 321.159844 274.633735 \n",
       "L 321.159844 273.542169 \n",
       "L 321.159844 272.450602 \n",
       "L 321.159844 271.359036 \n",
       "L 321.159844 270.26747 \n",
       "L 321.159844 269.175904 \n",
       "L 321.159844 268.084337 \n",
       "L 321.159844 266.992771 \n",
       "L 321.159844 265.901205 \n",
       "L 321.159844 264.809639 \n",
       "L 321.159844 263.718072 \n",
       "L 321.159844 262.626506 \n",
       "L 321.159844 261.53494 \n",
       "L 321.159844 260.443373 \n",
       "L 321.159844 259.351807 \n",
       "L 321.159844 258.260241 \n",
       "L 321.159844 257.168675 \n",
       "L 321.159844 256.077108 \n",
       "L 321.159844 254.985542 \n",
       "L 321.159844 253.893976 \n",
       "L 321.159844 252.80241 \n",
       "L 321.159844 251.710843 \n",
       "L 321.159844 250.619277 \n",
       "L 321.159844 249.527711 \n",
       "L 321.159844 248.436145 \n",
       "L 321.159844 247.344578 \n",
       "L 321.159844 246.253012 \n",
       "L 321.159844 245.161446 \n",
       "L 321.159844 244.06988 \n",
       "L 321.159844 242.978313 \n",
       "L 321.159844 241.886747 \n",
       "L 321.159844 240.795181 \n",
       "L 321.159844 239.703614 \n",
       "L 321.159844 238.612048 \n",
       "L 321.159844 237.520482 \n",
       "L 321.159844 236.428916 \n",
       "L 321.159844 235.337349 \n",
       "L 321.159844 234.245783 \n",
       "L 321.159844 233.154217 \n",
       "L 321.159844 232.062651 \n",
       "L 321.159844 230.971084 \n",
       "L 321.159844 229.879518 \n",
       "L 321.159844 228.787952 \n",
       "L 321.159844 227.696386 \n",
       "L 321.159844 226.604819 \n",
       "L 321.159844 225.513253 \n",
       "L 321.159844 224.421687 \n",
       "L 321.159844 223.33012 \n",
       "L 321.159844 222.238554 \n",
       "L 321.159844 221.146988 \n",
       "L 321.159844 220.055422 \n",
       "L 321.159844 218.963855 \n",
       "L 321.159844 217.872289 \n",
       "L 321.159844 216.780723 \n",
       "L 321.159844 215.689157 \n",
       "L 321.159844 214.59759 \n",
       "L 321.159844 213.506024 \n",
       "L 321.159844 212.414458 \n",
       "L 321.159844 211.322892 \n",
       "L 321.159844 210.231325 \n",
       "L 321.159844 209.139759 \n",
       "L 321.159844 208.048193 \n",
       "L 321.159844 206.956627 \n",
       "L 321.159844 205.86506 \n",
       "L 321.159844 204.773494 \n",
       "L 321.159844 203.681928 \n",
       "L 321.159844 202.590361 \n",
       "L 321.159844 201.498795 \n",
       "L 321.159844 200.407229 \n",
       "L 321.159844 199.315663 \n",
       "L 321.159844 198.224096 \n",
       "L 321.159844 197.13253 \n",
       "L 321.159844 196.040964 \n",
       "L 321.159844 195.904518 \n",
       "L 320.039362 195.904518 \n",
       "L 318.91888 195.904518 \n",
       "L 317.798398 195.904518 \n",
       "L 316.677916 195.904518 \n",
       "L 315.557434 195.904518 \n",
       "L 314.436952 195.904518 \n",
       "L 313.31647 195.904518 \n",
       "L 312.195988 195.904518 \n",
       "L 311.075506 195.904518 \n",
       "L 309.955024 195.904518 \n",
       "L 308.834543 195.904518 \n",
       "L 307.714061 195.904518 \n",
       "L 306.593579 195.904518 \n",
       "L 305.473097 195.904518 \n",
       "L 304.352615 195.904518 \n",
       "L 303.232133 195.904518 \n",
       "L 302.111651 195.904518 \n",
       "L 300.991169 195.904518 \n",
       "L 299.870687 195.904518 \n",
       "L 298.750205 195.904518 \n",
       "L 297.629723 195.904518 \n",
       "L 296.509241 195.904518 \n",
       "L 295.388759 195.904518 \n",
       "L 294.268277 195.904518 \n",
       "L 293.147796 195.904518 \n",
       "L 292.027314 195.904518 \n",
       "L 290.906832 195.904518 \n",
       "L 289.78635 195.904518 \n",
       "L 288.665868 195.904518 \n",
       "L 287.545386 195.904518 \n",
       "L 286.424904 195.904518 \n",
       "L 285.304422 195.904518 \n",
       "L 284.18394 195.904518 \n",
       "L 283.063458 195.904518 \n",
       "L 281.942976 195.904518 \n",
       "L 280.822494 195.904518 \n",
       "L 279.702012 195.904518 \n",
       "L 278.58153 195.904518 \n",
       "L 277.461049 195.904518 \n",
       "L 276.340567 195.904518 \n",
       "L 275.220085 195.904518 \n",
       "L 274.099603 195.904518 \n",
       "L 272.979121 195.904518 \n",
       "L 271.858639 195.904518 \n",
       "L 270.738157 195.904518 \n",
       "L 269.617675 195.904518 \n",
       "L 268.497193 195.904518 \n",
       "L 267.376711 195.904518 \n",
       "L 266.256229 195.904518 \n",
       "L 265.135747 195.904518 \n",
       "L 264.015265 195.904518 \n",
       "L 262.894784 195.904518 \n",
       "L 261.774302 195.904518 \n",
       "L 260.65382 195.904518 \n",
       "L 259.533338 195.904518 \n",
       "L 258.412856 195.904518 \n",
       "L 257.292374 195.904518 \n",
       "L 256.171892 195.904518 \n",
       "L 255.05141 195.904518 \n",
       "L 253.930928 195.904518 \n",
       "L 252.810446 195.904518 \n",
       "L 251.689964 195.904518 \n",
       "L 250.569482 195.904518 \n",
       "L 249.449 195.904518 \n",
       "L 248.328518 195.904518 \n",
       "L 247.208037 195.904518 \n",
       "L 246.087555 195.904518 \n",
       "L 244.967073 195.904518 \n",
       "L 243.846591 195.904518 \n",
       "L 242.726109 195.904518 \n",
       "L 241.605627 195.904518 \n",
       "L 240.485145 195.904518 \n",
       "L 239.364663 195.904518 \n",
       "L 238.244181 195.904518 \n",
       "L 237.123699 195.904518 \n",
       "L 236.003217 195.904518 \n",
       "L 234.882735 195.904518 \n",
       "L 233.762253 195.904518 \n",
       "L 232.641771 195.904518 \n",
       "L 231.52129 195.904518 \n",
       "L 230.400808 195.904518 \n",
       "L 229.280326 195.904518 \n",
       "L 228.159844 195.904518 \n",
       "L 227.039362 195.904518 \n",
       "L 225.91888 195.904518 \n",
       "L 224.798398 195.768072 \n",
       "L 223.677916 195.768072 \n",
       "L 222.557434 195.768072 \n",
       "L 221.436952 195.768072 \n",
       "L 220.31647 195.768072 \n",
       "L 219.195988 195.768072 \n",
       "L 218.075506 195.768072 \n",
       "L 216.955024 195.768072 \n",
       "L 215.834543 195.768072 \n",
       "L 214.714061 195.768072 \n",
       "L 213.593579 195.768072 \n",
       "L 212.473097 195.768072 \n",
       "L 211.352615 195.768072 \n",
       "L 210.232133 195.768072 \n",
       "L 209.111651 195.768072 \n",
       "L 207.991169 195.768072 \n",
       "L 206.870687 195.768072 \n",
       "L 205.750205 195.768072 \n",
       "L 204.629723 195.768072 \n",
       "L 203.509241 195.768072 \n",
       "L 202.388759 195.768072 \n",
       "L 201.268277 195.768072 \n",
       "L 200.147796 195.768072 \n",
       "L 199.027314 195.768072 \n",
       "L 197.906832 195.768072 \n",
       "L 196.78635 195.768072 \n",
       "L 195.665868 195.768072 \n",
       "L 194.545386 195.768072 \n",
       "L 193.424904 195.768072 \n",
       "L 192.304422 195.768072 \n",
       "L 191.18394 195.768072 \n",
       "L 190.063458 195.768072 \n",
       "L 188.942976 195.768072 \n",
       "L 187.822494 195.768072 \n",
       "L 186.702012 195.768072 \n",
       "L 185.58153 195.768072 \n",
       "L 184.461049 195.768072 \n",
       "L 183.340567 195.768072 \n",
       "L 182.220085 195.768072 \n",
       "L 181.099603 195.768072 \n",
       "L 179.979121 195.768072 \n",
       "L 178.858639 195.768072 \n",
       "L 177.738157 195.768072 \n",
       "L 176.617675 195.768072 \n",
       "L 175.497193 195.768072 \n",
       "L 174.376711 195.768072 \n",
       "L 173.256229 195.768072 \n",
       "L 172.135747 195.768072 \n",
       "L 171.015265 195.768072 \n",
       "L 169.894784 195.768072 \n",
       "L 168.774302 195.768072 \n",
       "L 167.65382 195.768072 \n",
       "L 166.533338 195.768072 \n",
       "L 165.412856 195.768072 \n",
       "L 164.292374 195.768072 \n",
       "L 163.171892 195.768072 \n",
       "L 162.05141 195.768072 \n",
       "L 160.930928 195.768072 \n",
       "L 159.810446 195.768072 \n",
       "L 158.689964 195.768072 \n",
       "L 157.569482 195.768072 \n",
       "L 156.449 195.768072 \n",
       "L 155.328518 195.768072 \n",
       "L 154.208037 195.768072 \n",
       "L 153.087555 195.768072 \n",
       "L 151.967073 195.768072 \n",
       "L 150.846591 195.768072 \n",
       "L 149.726109 195.768072 \n",
       "L 148.605627 195.768072 \n",
       "L 147.485145 195.768072 \n",
       "L 146.364663 195.768072 \n",
       "L 145.244181 195.768072 \n",
       "L 144.123699 195.768072 \n",
       "L 143.003217 195.768072 \n",
       "L 141.882735 195.768072 \n",
       "L 140.762253 195.768072 \n",
       "L 139.641771 195.768072 \n",
       "L 138.52129 195.768072 \n",
       "L 137.400808 195.768072 \n",
       "L 136.280326 195.768072 \n",
       "L 135.159844 195.768072 \n",
       "L 134.039362 195.768072 \n",
       "L 132.91888 195.768072 \n",
       "L 131.798398 195.768072 \n",
       "L 130.677916 195.768072 \n",
       "L 129.557434 195.768072 \n",
       "L 128.436952 195.768072 \n",
       "L 127.31647 195.768072 \n",
       "L 126.195988 195.768072 \n",
       "L 125.075506 195.768072 \n",
       "L 123.955024 195.768072 \n",
       "L 122.834543 195.768072 \n",
       "L 121.714061 195.768072 \n",
       "L 120.593579 195.768072 \n",
       "L 119.473097 195.768072 \n",
       "L 118.352615 195.768072 \n",
       "L 117.232133 195.768072 \n",
       "L 116.111651 195.768072 \n",
       "L 114.991169 195.768072 \n",
       "L 113.870687 195.768072 \n",
       "L 112.750205 195.768072 \n",
       "L 111.629723 195.768072 \n",
       "L 110.509241 195.768072 \n",
       "L 109.388759 195.768072 \n",
       "L 108.268277 195.768072 \n",
       "L 107.147796 195.768072 \n",
       "L 106.027314 195.768072 \n",
       "L 104.906832 195.768072 \n",
       "L 103.78635 195.768072 \n",
       "L 102.665868 195.768072 \n",
       "L 101.545386 195.768072 \n",
       "L 100.424904 195.768072 \n",
       "L 99.304422 195.768072 \n",
       "L 98.18394 195.768072 \n",
       "L 97.063458 195.768072 \n",
       "L 95.942976 195.768072 \n",
       "L 94.822494 195.768072 \n",
       "L 93.702012 195.768072 \n",
       "L 92.58153 195.768072 \n",
       "L 91.461049 195.768072 \n",
       "L 90.340567 195.768072 \n",
       "L 89.220085 195.768072 \n",
       "L 88.099603 195.768072 \n",
       "L 86.979121 195.768072 \n",
       "L 85.858639 195.768072 \n",
       "L 84.738157 195.768072 \n",
       "L 83.617675 195.768072 \n",
       "L 82.497193 195.768072 \n",
       "L 81.376711 195.768072 \n",
       "L 80.256229 195.768072 \n",
       "L 79.135747 195.768072 \n",
       "L 78.015265 195.768072 \n",
       "L 76.894784 195.768072 \n",
       "L 75.774302 195.768072 \n",
       "L 74.65382 195.768072 \n",
       "L 73.533338 195.768072 \n",
       "L 72.412856 195.768072 \n",
       "L 71.292374 195.768072 \n",
       "L 70.171892 195.768072 \n",
       "L 69.05141 195.768072 \n",
       "L 67.930928 195.768072 \n",
       "L 66.810446 195.768072 \n",
       "L 65.689964 195.768072 \n",
       "L 64.569482 195.768072 \n",
       "L 63.449 195.768072 \n",
       "L 62.328518 195.768072 \n",
       "L 61.208037 195.768072 \n",
       "L 60.087555 195.768072 \n",
       "L 58.967073 195.768072 \n",
       "L 57.846591 195.768072 \n",
       "L 56.726109 195.768072 \n",
       "L 55.605627 195.768072 \n",
       "L 54.485145 195.768072 \n",
       "L 53.364663 195.768072 \n",
       "L 52.244181 195.768072 \n",
       "L 51.123699 195.768072 \n",
       "L 50.003217 195.768072 \n",
       "L 48.882735 195.768072 \n",
       "L 47.762253 195.768072 \n",
       "L 46.641771 195.768072 \n",
       "L 45.52129 195.768072 \n",
       "L 44.400808 195.768072 \n",
       "L 43.280326 195.768072 \n",
       "L 42.159844 195.768072 \n",
       "L 42.159844 196.040964 \n",
       "L 42.159844 197.13253 \n",
       "L 42.159844 198.224096 \n",
       "L 42.159844 199.315663 \n",
       "L 42.159844 200.407229 \n",
       "L 42.159844 201.498795 \n",
       "L 42.159844 202.590361 \n",
       "L 42.159844 203.681928 \n",
       "L 42.159844 204.773494 \n",
       "L 42.159844 205.86506 \n",
       "L 42.159844 206.956627 \n",
       "L 42.159844 208.048193 \n",
       "L 42.159844 209.139759 \n",
       "L 42.159844 210.231325 \n",
       "L 42.159844 211.322892 \n",
       "L 42.159844 212.414458 \n",
       "L 42.159844 213.506024 \n",
       "L 42.159844 214.59759 \n",
       "L 42.159844 215.689157 \n",
       "L 42.159844 216.780723 \n",
       "L 42.159844 217.872289 \n",
       "L 42.159844 218.963855 \n",
       "L 42.159844 220.055422 \n",
       "L 42.159844 221.146988 \n",
       "L 42.159844 222.238554 \n",
       "L 42.159844 223.33012 \n",
       "L 42.159844 224.421687 \n",
       "L 42.159844 225.513253 \n",
       "L 42.159844 226.604819 \n",
       "L 42.159844 227.696386 \n",
       "L 42.159844 228.787952 \n",
       "L 42.159844 229.879518 \n",
       "L 42.159844 230.971084 \n",
       "L 42.159844 232.062651 \n",
       "L 42.159844 233.154217 \n",
       "L 42.159844 234.245783 \n",
       "L 42.159844 235.337349 \n",
       "L 42.159844 236.428916 \n",
       "L 42.159844 237.520482 \n",
       "L 42.159844 238.612048 \n",
       "L 42.159844 239.703614 \n",
       "L 42.159844 240.795181 \n",
       "L 42.159844 241.886747 \n",
       "L 42.159844 242.978313 \n",
       "L 42.159844 244.06988 \n",
       "L 42.159844 245.161446 \n",
       "L 42.159844 246.253012 \n",
       "L 42.159844 247.344578 \n",
       "L 42.159844 248.436145 \n",
       "L 42.159844 249.527711 \n",
       "L 42.159844 250.619277 \n",
       "L 42.159844 251.710843 \n",
       "L 42.159844 252.80241 \n",
       "L 42.159844 253.893976 \n",
       "L 42.159844 254.985542 \n",
       "L 42.159844 256.077108 \n",
       "L 42.159844 257.168675 \n",
       "L 42.159844 258.260241 \n",
       "L 42.159844 259.351807 \n",
       "L 42.159844 260.443373 \n",
       "L 42.159844 261.53494 \n",
       "L 42.159844 262.626506 \n",
       "L 42.159844 263.718072 \n",
       "L 42.159844 264.809639 \n",
       "L 42.159844 265.901205 \n",
       "L 42.159844 266.992771 \n",
       "L 42.159844 268.084337 \n",
       "L 42.159844 269.175904 \n",
       "L 42.159844 270.26747 \n",
       "L 42.159844 271.359036 \n",
       "L 42.159844 272.450602 \n",
       "L 42.159844 273.542169 \n",
       "L 42.159844 274.633735 \n",
       "L 42.159844 275.725301 \n",
       "L 42.159844 276.816867 \n",
       "L 42.159844 277.908434 \n",
       "L 42.159844 279 \n",
       "z\n",
       "\" style=\"fill:#1f77b4;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_2\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 43.280326 195.768072 \n",
       "L 44.400808 195.768072 \n",
       "L 45.52129 195.768072 \n",
       "L 46.641771 195.768072 \n",
       "L 47.762253 195.768072 \n",
       "L 48.882735 195.768072 \n",
       "L 50.003217 195.768072 \n",
       "L 51.123699 195.768072 \n",
       "L 52.244181 195.768072 \n",
       "L 53.364663 195.768072 \n",
       "L 54.485145 195.768072 \n",
       "L 55.605627 195.768072 \n",
       "L 56.726109 195.768072 \n",
       "L 57.846591 195.768072 \n",
       "L 58.967073 195.768072 \n",
       "L 60.087555 195.768072 \n",
       "L 61.208037 195.768072 \n",
       "L 62.328518 195.768072 \n",
       "L 63.449 195.768072 \n",
       "L 64.569482 195.768072 \n",
       "L 65.689964 195.768072 \n",
       "L 66.810446 195.768072 \n",
       "L 67.930928 195.768072 \n",
       "L 69.05141 195.768072 \n",
       "L 70.171892 195.768072 \n",
       "L 71.292374 195.768072 \n",
       "L 72.412856 195.768072 \n",
       "L 73.533338 195.768072 \n",
       "L 74.65382 195.768072 \n",
       "L 75.774302 195.768072 \n",
       "L 76.894784 195.768072 \n",
       "L 78.015265 195.768072 \n",
       "L 79.135747 195.768072 \n",
       "L 80.256229 195.768072 \n",
       "L 81.376711 195.768072 \n",
       "L 82.497193 195.768072 \n",
       "L 83.617675 195.768072 \n",
       "L 84.738157 195.768072 \n",
       "L 85.858639 195.768072 \n",
       "L 86.979121 195.768072 \n",
       "L 88.099603 195.768072 \n",
       "L 89.220085 195.768072 \n",
       "L 90.340567 195.768072 \n",
       "L 91.461049 195.768072 \n",
       "L 92.58153 195.768072 \n",
       "L 93.702012 195.768072 \n",
       "L 94.822494 195.768072 \n",
       "L 95.942976 195.768072 \n",
       "L 97.063458 195.768072 \n",
       "L 98.18394 195.768072 \n",
       "L 99.304422 195.768072 \n",
       "L 100.424904 195.768072 \n",
       "L 101.545386 195.768072 \n",
       "L 102.665868 195.768072 \n",
       "L 103.78635 195.768072 \n",
       "L 104.906832 195.768072 \n",
       "L 106.027314 195.768072 \n",
       "L 107.147796 195.768072 \n",
       "L 108.268277 195.768072 \n",
       "L 109.388759 195.768072 \n",
       "L 110.509241 195.768072 \n",
       "L 111.629723 195.768072 \n",
       "L 112.750205 195.768072 \n",
       "L 113.870687 195.768072 \n",
       "L 114.991169 195.768072 \n",
       "L 116.111651 195.768072 \n",
       "L 117.232133 195.768072 \n",
       "L 118.352615 195.768072 \n",
       "L 119.473097 195.768072 \n",
       "L 120.593579 195.768072 \n",
       "L 121.714061 195.768072 \n",
       "L 122.834543 195.768072 \n",
       "L 123.955024 195.768072 \n",
       "L 125.075506 195.768072 \n",
       "L 126.195988 195.768072 \n",
       "L 127.31647 195.768072 \n",
       "L 128.436952 195.768072 \n",
       "L 129.557434 195.768072 \n",
       "L 130.677916 195.768072 \n",
       "L 131.798398 195.768072 \n",
       "L 132.91888 195.768072 \n",
       "L 134.039362 195.768072 \n",
       "L 135.159844 195.768072 \n",
       "L 136.280326 195.768072 \n",
       "L 137.400808 195.768072 \n",
       "L 138.52129 195.768072 \n",
       "L 139.641771 195.768072 \n",
       "L 140.762253 195.768072 \n",
       "L 141.882735 195.768072 \n",
       "L 143.003217 195.768072 \n",
       "L 144.123699 195.768072 \n",
       "L 145.244181 195.768072 \n",
       "L 146.364663 195.768072 \n",
       "L 147.485145 195.768072 \n",
       "L 148.605627 195.768072 \n",
       "L 149.726109 195.768072 \n",
       "L 150.846591 195.768072 \n",
       "L 151.967073 195.768072 \n",
       "L 153.087555 195.768072 \n",
       "L 154.208037 195.768072 \n",
       "L 155.328518 195.768072 \n",
       "L 156.449 195.768072 \n",
       "L 157.569482 195.768072 \n",
       "L 158.689964 195.768072 \n",
       "L 159.810446 195.768072 \n",
       "L 160.930928 195.768072 \n",
       "L 162.05141 195.768072 \n",
       "L 163.171892 195.768072 \n",
       "L 164.292374 195.768072 \n",
       "L 165.412856 195.768072 \n",
       "L 166.533338 195.768072 \n",
       "L 167.65382 195.768072 \n",
       "L 168.774302 195.768072 \n",
       "L 169.894784 195.768072 \n",
       "L 171.015265 195.768072 \n",
       "L 172.135747 195.768072 \n",
       "L 173.256229 195.768072 \n",
       "L 174.376711 195.768072 \n",
       "L 175.497193 195.768072 \n",
       "L 176.617675 195.768072 \n",
       "L 177.738157 195.768072 \n",
       "L 178.858639 195.768072 \n",
       "L 179.979121 195.768072 \n",
       "L 181.099603 195.768072 \n",
       "L 182.220085 195.768072 \n",
       "L 183.340567 195.768072 \n",
       "L 184.461049 195.768072 \n",
       "L 185.58153 195.768072 \n",
       "L 186.702012 195.768072 \n",
       "L 187.822494 195.768072 \n",
       "L 188.942976 195.768072 \n",
       "L 190.063458 195.768072 \n",
       "L 191.18394 195.768072 \n",
       "L 192.304422 195.768072 \n",
       "L 193.424904 195.768072 \n",
       "L 194.545386 195.768072 \n",
       "L 195.665868 195.768072 \n",
       "L 196.78635 195.768072 \n",
       "L 197.906832 195.768072 \n",
       "L 199.027314 195.768072 \n",
       "L 200.147796 195.768072 \n",
       "L 201.268277 195.768072 \n",
       "L 202.388759 195.768072 \n",
       "L 203.509241 195.768072 \n",
       "L 204.629723 195.768072 \n",
       "L 205.750205 195.768072 \n",
       "L 206.870687 195.768072 \n",
       "L 207.991169 195.768072 \n",
       "L 209.111651 195.768072 \n",
       "L 210.232133 195.768072 \n",
       "L 211.352615 195.768072 \n",
       "L 212.473097 195.768072 \n",
       "L 213.593579 195.768072 \n",
       "L 214.714061 195.768072 \n",
       "L 215.834543 195.768072 \n",
       "L 216.955024 195.768072 \n",
       "L 218.075506 195.768072 \n",
       "L 219.195988 195.768072 \n",
       "L 220.31647 195.768072 \n",
       "L 221.436952 195.768072 \n",
       "L 222.557434 195.768072 \n",
       "L 223.677916 195.768072 \n",
       "L 224.798398 195.768072 \n",
       "L 225.91888 195.904518 \n",
       "L 227.039362 195.904518 \n",
       "L 228.159844 195.904518 \n",
       "L 229.280326 195.904518 \n",
       "L 230.400808 195.904518 \n",
       "L 231.52129 195.904518 \n",
       "L 232.641771 195.904518 \n",
       "L 233.762253 195.904518 \n",
       "L 234.882735 195.904518 \n",
       "L 236.003217 195.904518 \n",
       "L 237.123699 195.904518 \n",
       "L 238.244181 195.904518 \n",
       "L 239.364663 195.904518 \n",
       "L 240.485145 195.904518 \n",
       "L 241.605627 195.904518 \n",
       "L 242.726109 195.904518 \n",
       "L 243.846591 195.904518 \n",
       "L 244.967073 195.904518 \n",
       "L 246.087555 195.904518 \n",
       "L 247.208037 195.904518 \n",
       "L 248.328518 195.904518 \n",
       "L 249.449 195.904518 \n",
       "L 250.569482 195.904518 \n",
       "L 251.689964 195.904518 \n",
       "L 252.810446 195.904518 \n",
       "L 253.930928 195.904518 \n",
       "L 255.05141 195.904518 \n",
       "L 256.171892 195.904518 \n",
       "L 257.292374 195.904518 \n",
       "L 258.412856 195.904518 \n",
       "L 259.533338 195.904518 \n",
       "L 260.65382 195.904518 \n",
       "L 261.774302 195.904518 \n",
       "L 262.894784 195.904518 \n",
       "L 264.015265 195.904518 \n",
       "L 265.135747 195.904518 \n",
       "L 266.256229 195.904518 \n",
       "L 267.376711 195.904518 \n",
       "L 268.497193 195.904518 \n",
       "L 269.617675 195.904518 \n",
       "L 270.738157 195.904518 \n",
       "L 271.858639 195.904518 \n",
       "L 272.979121 195.904518 \n",
       "L 274.099603 195.904518 \n",
       "L 275.220085 195.904518 \n",
       "L 276.340567 195.904518 \n",
       "L 277.461049 195.904518 \n",
       "L 278.58153 195.904518 \n",
       "L 279.702012 195.904518 \n",
       "L 280.822494 195.904518 \n",
       "L 281.942976 195.904518 \n",
       "L 283.063458 195.904518 \n",
       "L 284.18394 195.904518 \n",
       "L 285.304422 195.904518 \n",
       "L 286.424904 195.904518 \n",
       "L 287.545386 195.904518 \n",
       "L 288.665868 195.904518 \n",
       "L 289.78635 195.904518 \n",
       "L 290.906832 195.904518 \n",
       "L 292.027314 195.904518 \n",
       "L 293.147796 195.904518 \n",
       "L 294.268277 195.904518 \n",
       "L 295.388759 195.904518 \n",
       "L 296.509241 195.904518 \n",
       "L 297.629723 195.904518 \n",
       "L 298.750205 195.904518 \n",
       "L 299.870687 195.904518 \n",
       "L 300.991169 195.904518 \n",
       "L 302.111651 195.904518 \n",
       "L 303.232133 195.904518 \n",
       "L 304.352615 195.904518 \n",
       "L 305.473097 195.904518 \n",
       "L 306.593579 195.904518 \n",
       "L 307.714061 195.904518 \n",
       "L 308.834543 195.904518 \n",
       "L 309.955024 195.904518 \n",
       "L 311.075506 195.904518 \n",
       "L 312.195988 195.904518 \n",
       "L 313.31647 195.904518 \n",
       "L 314.436952 195.904518 \n",
       "L 315.557434 195.904518 \n",
       "L 316.677916 195.904518 \n",
       "L 317.798398 195.904518 \n",
       "L 318.91888 195.904518 \n",
       "L 320.039362 195.904518 \n",
       "L 321.159844 195.904518 \n",
       "L 321.159844 195.768072 \n",
       "L 320.039362 195.768072 \n",
       "L 318.91888 195.768072 \n",
       "L 317.798398 195.768072 \n",
       "L 316.677916 195.768072 \n",
       "L 315.557434 195.768072 \n",
       "L 314.436952 195.768072 \n",
       "L 313.31647 195.768072 \n",
       "L 312.195988 195.768072 \n",
       "L 311.075506 195.768072 \n",
       "L 309.955024 195.768072 \n",
       "L 308.834543 195.768072 \n",
       "L 307.714061 195.768072 \n",
       "L 306.593579 195.768072 \n",
       "L 305.473097 195.768072 \n",
       "L 304.352615 195.768072 \n",
       "L 303.232133 195.768072 \n",
       "L 302.111651 195.768072 \n",
       "L 300.991169 195.768072 \n",
       "L 299.870687 195.768072 \n",
       "L 298.750205 195.768072 \n",
       "L 297.629723 195.768072 \n",
       "L 296.509241 195.768072 \n",
       "L 295.388759 195.768072 \n",
       "L 294.268277 195.768072 \n",
       "L 293.147796 195.768072 \n",
       "L 292.027314 195.768072 \n",
       "L 290.906832 195.768072 \n",
       "L 289.78635 195.768072 \n",
       "L 288.665868 195.768072 \n",
       "L 287.545386 195.768072 \n",
       "L 286.424904 195.768072 \n",
       "L 285.304422 195.768072 \n",
       "L 284.18394 195.768072 \n",
       "L 283.063458 195.768072 \n",
       "L 281.942976 195.768072 \n",
       "L 280.822494 195.768072 \n",
       "L 279.702012 195.768072 \n",
       "L 278.58153 195.768072 \n",
       "L 277.461049 195.768072 \n",
       "L 276.340567 195.768072 \n",
       "L 275.220085 195.768072 \n",
       "L 274.099603 195.768072 \n",
       "L 272.979121 195.768072 \n",
       "L 271.858639 195.768072 \n",
       "L 270.738157 195.768072 \n",
       "L 269.617675 195.768072 \n",
       "L 268.497193 195.768072 \n",
       "L 267.376711 195.768072 \n",
       "L 266.256229 195.768072 \n",
       "L 265.135747 195.768072 \n",
       "L 264.015265 195.768072 \n",
       "L 262.894784 195.768072 \n",
       "L 261.774302 195.768072 \n",
       "L 260.65382 195.768072 \n",
       "L 259.533338 195.768072 \n",
       "L 258.412856 195.768072 \n",
       "L 257.292374 195.768072 \n",
       "L 256.171892 195.768072 \n",
       "L 255.05141 195.768072 \n",
       "L 253.930928 195.768072 \n",
       "L 252.810446 195.768072 \n",
       "L 251.689964 195.768072 \n",
       "L 250.569482 195.768072 \n",
       "L 249.449 195.768072 \n",
       "L 248.328518 195.768072 \n",
       "L 247.208037 195.768072 \n",
       "L 246.087555 195.768072 \n",
       "L 244.967073 195.768072 \n",
       "L 243.846591 195.768072 \n",
       "L 242.726109 195.768072 \n",
       "L 241.605627 195.768072 \n",
       "L 240.485145 195.768072 \n",
       "L 239.364663 195.768072 \n",
       "L 238.244181 195.768072 \n",
       "L 237.123699 195.768072 \n",
       "L 236.003217 195.768072 \n",
       "L 234.882735 195.768072 \n",
       "L 233.762253 195.768072 \n",
       "L 232.641771 195.768072 \n",
       "L 231.52129 195.768072 \n",
       "L 230.400808 195.768072 \n",
       "L 229.280326 195.768072 \n",
       "L 228.159844 195.768072 \n",
       "L 227.039362 195.768072 \n",
       "L 225.91888 195.768072 \n",
       "L 224.798398 195.495181 \n",
       "L 223.677916 195.495181 \n",
       "L 222.557434 195.495181 \n",
       "L 221.436952 195.495181 \n",
       "L 220.31647 195.495181 \n",
       "L 219.195988 195.495181 \n",
       "L 218.075506 195.495181 \n",
       "L 216.955024 195.495181 \n",
       "L 215.834543 195.495181 \n",
       "L 214.714061 195.495181 \n",
       "L 213.593579 195.495181 \n",
       "L 212.473097 195.495181 \n",
       "L 211.352615 195.495181 \n",
       "L 210.232133 195.495181 \n",
       "L 209.111651 195.495181 \n",
       "L 207.991169 195.495181 \n",
       "L 206.870687 195.495181 \n",
       "L 205.750205 195.495181 \n",
       "L 204.629723 195.495181 \n",
       "L 203.509241 195.495181 \n",
       "L 202.388759 195.495181 \n",
       "L 201.268277 195.495181 \n",
       "L 200.147796 195.495181 \n",
       "L 199.027314 195.495181 \n",
       "L 197.906832 195.495181 \n",
       "L 196.78635 195.495181 \n",
       "L 195.665868 195.495181 \n",
       "L 194.545386 195.495181 \n",
       "L 193.424904 195.495181 \n",
       "L 192.304422 195.495181 \n",
       "L 191.18394 195.495181 \n",
       "L 190.063458 195.495181 \n",
       "L 188.942976 195.495181 \n",
       "L 187.822494 195.495181 \n",
       "L 186.702012 195.495181 \n",
       "L 185.58153 195.495181 \n",
       "L 184.461049 195.495181 \n",
       "L 183.340567 195.495181 \n",
       "L 182.220085 195.495181 \n",
       "L 181.099603 195.495181 \n",
       "L 179.979121 195.495181 \n",
       "L 178.858639 195.495181 \n",
       "L 177.738157 195.495181 \n",
       "L 176.617675 195.495181 \n",
       "L 175.497193 195.495181 \n",
       "L 174.376711 195.495181 \n",
       "L 173.256229 195.495181 \n",
       "L 172.135747 195.495181 \n",
       "L 171.015265 195.495181 \n",
       "L 169.894784 195.495181 \n",
       "L 168.774302 195.495181 \n",
       "L 167.65382 195.495181 \n",
       "L 166.533338 195.495181 \n",
       "L 165.412856 195.495181 \n",
       "L 164.292374 195.495181 \n",
       "L 163.171892 195.495181 \n",
       "L 162.05141 195.495181 \n",
       "L 160.930928 195.495181 \n",
       "L 159.810446 195.495181 \n",
       "L 158.689964 195.495181 \n",
       "L 157.569482 195.495181 \n",
       "L 156.449 195.495181 \n",
       "L 155.328518 195.495181 \n",
       "L 154.208037 195.495181 \n",
       "L 153.087555 195.495181 \n",
       "L 151.967073 195.495181 \n",
       "L 150.846591 195.495181 \n",
       "L 149.726109 195.495181 \n",
       "L 148.605627 195.495181 \n",
       "L 147.485145 195.495181 \n",
       "L 146.364663 195.495181 \n",
       "L 145.244181 195.495181 \n",
       "L 144.123699 195.495181 \n",
       "L 143.003217 195.495181 \n",
       "L 141.882735 195.495181 \n",
       "L 140.762253 195.495181 \n",
       "L 139.641771 195.495181 \n",
       "L 138.52129 195.495181 \n",
       "L 137.400808 195.495181 \n",
       "L 136.280326 195.495181 \n",
       "L 135.159844 195.495181 \n",
       "L 134.039362 195.495181 \n",
       "L 132.91888 195.495181 \n",
       "L 131.798398 195.495181 \n",
       "L 130.677916 195.495181 \n",
       "L 129.557434 195.495181 \n",
       "L 128.436952 195.495181 \n",
       "L 127.31647 195.495181 \n",
       "L 126.195988 195.495181 \n",
       "L 125.075506 195.495181 \n",
       "L 123.955024 195.495181 \n",
       "L 122.834543 195.495181 \n",
       "L 121.714061 195.495181 \n",
       "L 120.593579 195.495181 \n",
       "L 119.473097 195.495181 \n",
       "L 118.352615 195.495181 \n",
       "L 117.232133 195.495181 \n",
       "L 116.111651 195.495181 \n",
       "L 114.991169 195.495181 \n",
       "L 113.870687 195.495181 \n",
       "L 112.750205 195.495181 \n",
       "L 111.629723 195.495181 \n",
       "L 110.509241 195.495181 \n",
       "L 109.388759 195.495181 \n",
       "L 108.268277 195.495181 \n",
       "L 107.147796 195.495181 \n",
       "L 106.027314 195.495181 \n",
       "L 104.906832 195.495181 \n",
       "L 103.78635 195.495181 \n",
       "L 102.665868 195.495181 \n",
       "L 101.545386 195.495181 \n",
       "L 100.424904 195.495181 \n",
       "L 99.304422 195.495181 \n",
       "L 98.18394 195.495181 \n",
       "L 97.063458 195.495181 \n",
       "L 95.942976 195.495181 \n",
       "L 94.822494 195.495181 \n",
       "L 93.702012 195.495181 \n",
       "L 92.58153 195.495181 \n",
       "L 91.461049 195.495181 \n",
       "L 90.340567 195.495181 \n",
       "L 89.220085 195.495181 \n",
       "L 88.099603 195.495181 \n",
       "L 86.979121 195.495181 \n",
       "L 85.858639 195.495181 \n",
       "L 84.738157 195.495181 \n",
       "L 83.617675 195.495181 \n",
       "L 82.497193 195.495181 \n",
       "L 81.376711 195.495181 \n",
       "L 80.256229 195.495181 \n",
       "L 79.135747 195.495181 \n",
       "L 78.015265 195.495181 \n",
       "L 76.894784 195.495181 \n",
       "L 75.774302 195.495181 \n",
       "L 74.65382 195.495181 \n",
       "L 73.533338 195.495181 \n",
       "L 72.412856 195.495181 \n",
       "L 71.292374 195.495181 \n",
       "L 70.171892 195.495181 \n",
       "L 69.05141 195.495181 \n",
       "L 67.930928 195.495181 \n",
       "L 66.810446 195.495181 \n",
       "L 65.689964 195.495181 \n",
       "L 64.569482 195.495181 \n",
       "L 63.449 195.495181 \n",
       "L 62.328518 195.495181 \n",
       "L 61.208037 195.495181 \n",
       "L 60.087555 195.495181 \n",
       "L 58.967073 195.495181 \n",
       "L 57.846591 195.495181 \n",
       "L 56.726109 195.495181 \n",
       "L 55.605627 195.495181 \n",
       "L 54.485145 195.495181 \n",
       "L 53.364663 195.495181 \n",
       "L 52.244181 195.495181 \n",
       "L 51.123699 195.495181 \n",
       "L 50.003217 195.495181 \n",
       "L 48.882735 195.495181 \n",
       "L 47.762253 195.495181 \n",
       "L 46.641771 195.495181 \n",
       "L 45.52129 195.495181 \n",
       "L 44.400808 195.495181 \n",
       "L 43.280326 195.495181 \n",
       "L 42.159844 195.495181 \n",
       "L 42.159844 195.768072 \n",
       "z\n",
       "\" style=\"fill:#ff7f0e;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_3\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 43.280326 195.495181 \n",
       "L 44.400808 195.495181 \n",
       "L 45.52129 195.495181 \n",
       "L 46.641771 195.495181 \n",
       "L 47.762253 195.495181 \n",
       "L 48.882735 195.495181 \n",
       "L 50.003217 195.495181 \n",
       "L 51.123699 195.495181 \n",
       "L 52.244181 195.495181 \n",
       "L 53.364663 195.495181 \n",
       "L 54.485145 195.495181 \n",
       "L 55.605627 195.495181 \n",
       "L 56.726109 195.495181 \n",
       "L 57.846591 195.495181 \n",
       "L 58.967073 195.495181 \n",
       "L 60.087555 195.495181 \n",
       "L 61.208037 195.495181 \n",
       "L 62.328518 195.495181 \n",
       "L 63.449 195.495181 \n",
       "L 64.569482 195.495181 \n",
       "L 65.689964 195.495181 \n",
       "L 66.810446 195.495181 \n",
       "L 67.930928 195.495181 \n",
       "L 69.05141 195.495181 \n",
       "L 70.171892 195.495181 \n",
       "L 71.292374 195.495181 \n",
       "L 72.412856 195.495181 \n",
       "L 73.533338 195.495181 \n",
       "L 74.65382 195.495181 \n",
       "L 75.774302 195.495181 \n",
       "L 76.894784 195.495181 \n",
       "L 78.015265 195.495181 \n",
       "L 79.135747 195.495181 \n",
       "L 80.256229 195.495181 \n",
       "L 81.376711 195.495181 \n",
       "L 82.497193 195.495181 \n",
       "L 83.617675 195.495181 \n",
       "L 84.738157 195.495181 \n",
       "L 85.858639 195.495181 \n",
       "L 86.979121 195.495181 \n",
       "L 88.099603 195.495181 \n",
       "L 89.220085 195.495181 \n",
       "L 90.340567 195.495181 \n",
       "L 91.461049 195.495181 \n",
       "L 92.58153 195.495181 \n",
       "L 93.702012 195.495181 \n",
       "L 94.822494 195.495181 \n",
       "L 95.942976 195.495181 \n",
       "L 97.063458 195.495181 \n",
       "L 98.18394 195.495181 \n",
       "L 99.304422 195.495181 \n",
       "L 100.424904 195.495181 \n",
       "L 101.545386 195.495181 \n",
       "L 102.665868 195.495181 \n",
       "L 103.78635 195.495181 \n",
       "L 104.906832 195.495181 \n",
       "L 106.027314 195.495181 \n",
       "L 107.147796 195.495181 \n",
       "L 108.268277 195.495181 \n",
       "L 109.388759 195.495181 \n",
       "L 110.509241 195.495181 \n",
       "L 111.629723 195.495181 \n",
       "L 112.750205 195.495181 \n",
       "L 113.870687 195.495181 \n",
       "L 114.991169 195.495181 \n",
       "L 116.111651 195.495181 \n",
       "L 117.232133 195.495181 \n",
       "L 118.352615 195.495181 \n",
       "L 119.473097 195.495181 \n",
       "L 120.593579 195.495181 \n",
       "L 121.714061 195.495181 \n",
       "L 122.834543 195.495181 \n",
       "L 123.955024 195.495181 \n",
       "L 125.075506 195.495181 \n",
       "L 126.195988 195.495181 \n",
       "L 127.31647 195.495181 \n",
       "L 128.436952 195.495181 \n",
       "L 129.557434 195.495181 \n",
       "L 130.677916 195.495181 \n",
       "L 131.798398 195.495181 \n",
       "L 132.91888 195.495181 \n",
       "L 134.039362 195.495181 \n",
       "L 135.159844 195.495181 \n",
       "L 136.280326 195.495181 \n",
       "L 137.400808 195.495181 \n",
       "L 138.52129 195.495181 \n",
       "L 139.641771 195.495181 \n",
       "L 140.762253 195.495181 \n",
       "L 141.882735 195.495181 \n",
       "L 143.003217 195.495181 \n",
       "L 144.123699 195.495181 \n",
       "L 145.244181 195.495181 \n",
       "L 146.364663 195.495181 \n",
       "L 147.485145 195.495181 \n",
       "L 148.605627 195.495181 \n",
       "L 149.726109 195.495181 \n",
       "L 150.846591 195.495181 \n",
       "L 151.967073 195.495181 \n",
       "L 153.087555 195.495181 \n",
       "L 154.208037 195.495181 \n",
       "L 155.328518 195.495181 \n",
       "L 156.449 195.495181 \n",
       "L 157.569482 195.495181 \n",
       "L 158.689964 195.495181 \n",
       "L 159.810446 195.495181 \n",
       "L 160.930928 195.495181 \n",
       "L 162.05141 195.495181 \n",
       "L 163.171892 195.495181 \n",
       "L 164.292374 195.495181 \n",
       "L 165.412856 195.495181 \n",
       "L 166.533338 195.495181 \n",
       "L 167.65382 195.495181 \n",
       "L 168.774302 195.495181 \n",
       "L 169.894784 195.495181 \n",
       "L 171.015265 195.495181 \n",
       "L 172.135747 195.495181 \n",
       "L 173.256229 195.495181 \n",
       "L 174.376711 195.495181 \n",
       "L 175.497193 195.495181 \n",
       "L 176.617675 195.495181 \n",
       "L 177.738157 195.495181 \n",
       "L 178.858639 195.495181 \n",
       "L 179.979121 195.495181 \n",
       "L 181.099603 195.495181 \n",
       "L 182.220085 195.495181 \n",
       "L 183.340567 195.495181 \n",
       "L 184.461049 195.495181 \n",
       "L 185.58153 195.495181 \n",
       "L 186.702012 195.495181 \n",
       "L 187.822494 195.495181 \n",
       "L 188.942976 195.495181 \n",
       "L 190.063458 195.495181 \n",
       "L 191.18394 195.495181 \n",
       "L 192.304422 195.495181 \n",
       "L 193.424904 195.495181 \n",
       "L 194.545386 195.495181 \n",
       "L 195.665868 195.495181 \n",
       "L 196.78635 195.495181 \n",
       "L 197.906832 195.495181 \n",
       "L 199.027314 195.495181 \n",
       "L 200.147796 195.495181 \n",
       "L 201.268277 195.495181 \n",
       "L 202.388759 195.495181 \n",
       "L 203.509241 195.495181 \n",
       "L 204.629723 195.495181 \n",
       "L 205.750205 195.495181 \n",
       "L 206.870687 195.495181 \n",
       "L 207.991169 195.495181 \n",
       "L 209.111651 195.495181 \n",
       "L 210.232133 195.495181 \n",
       "L 211.352615 195.495181 \n",
       "L 212.473097 195.495181 \n",
       "L 213.593579 195.495181 \n",
       "L 214.714061 195.495181 \n",
       "L 215.834543 195.495181 \n",
       "L 216.955024 195.495181 \n",
       "L 218.075506 195.495181 \n",
       "L 219.195988 195.495181 \n",
       "L 220.31647 195.495181 \n",
       "L 221.436952 195.495181 \n",
       "L 222.557434 195.495181 \n",
       "L 223.677916 195.495181 \n",
       "L 224.798398 195.495181 \n",
       "L 225.91888 195.768072 \n",
       "L 227.039362 195.768072 \n",
       "L 228.159844 195.768072 \n",
       "L 229.280326 195.768072 \n",
       "L 230.400808 195.768072 \n",
       "L 231.52129 195.768072 \n",
       "L 232.641771 195.768072 \n",
       "L 233.762253 195.768072 \n",
       "L 234.882735 195.768072 \n",
       "L 236.003217 195.768072 \n",
       "L 237.123699 195.768072 \n",
       "L 238.244181 195.768072 \n",
       "L 239.364663 195.768072 \n",
       "L 240.485145 195.768072 \n",
       "L 241.605627 195.768072 \n",
       "L 242.726109 195.768072 \n",
       "L 243.846591 195.768072 \n",
       "L 244.967073 195.768072 \n",
       "L 246.087555 195.768072 \n",
       "L 247.208037 195.768072 \n",
       "L 248.328518 195.768072 \n",
       "L 249.449 195.768072 \n",
       "L 250.569482 195.768072 \n",
       "L 251.689964 195.768072 \n",
       "L 252.810446 195.768072 \n",
       "L 253.930928 195.768072 \n",
       "L 255.05141 195.768072 \n",
       "L 256.171892 195.768072 \n",
       "L 257.292374 195.768072 \n",
       "L 258.412856 195.768072 \n",
       "L 259.533338 195.768072 \n",
       "L 260.65382 195.768072 \n",
       "L 261.774302 195.768072 \n",
       "L 262.894784 195.768072 \n",
       "L 264.015265 195.768072 \n",
       "L 265.135747 195.768072 \n",
       "L 266.256229 195.768072 \n",
       "L 267.376711 195.768072 \n",
       "L 268.497193 195.768072 \n",
       "L 269.617675 195.768072 \n",
       "L 270.738157 195.768072 \n",
       "L 271.858639 195.768072 \n",
       "L 272.979121 195.768072 \n",
       "L 274.099603 195.768072 \n",
       "L 275.220085 195.768072 \n",
       "L 276.340567 195.768072 \n",
       "L 277.461049 195.768072 \n",
       "L 278.58153 195.768072 \n",
       "L 279.702012 195.768072 \n",
       "L 280.822494 195.768072 \n",
       "L 281.942976 195.768072 \n",
       "L 283.063458 195.768072 \n",
       "L 284.18394 195.768072 \n",
       "L 285.304422 195.768072 \n",
       "L 286.424904 195.768072 \n",
       "L 287.545386 195.768072 \n",
       "L 288.665868 195.768072 \n",
       "L 289.78635 195.768072 \n",
       "L 290.906832 195.768072 \n",
       "L 292.027314 195.768072 \n",
       "L 293.147796 195.768072 \n",
       "L 294.268277 195.768072 \n",
       "L 295.388759 195.768072 \n",
       "L 296.509241 195.768072 \n",
       "L 297.629723 195.768072 \n",
       "L 298.750205 195.768072 \n",
       "L 299.870687 195.768072 \n",
       "L 300.991169 195.768072 \n",
       "L 302.111651 195.768072 \n",
       "L 303.232133 195.768072 \n",
       "L 304.352615 195.768072 \n",
       "L 305.473097 195.768072 \n",
       "L 306.593579 195.768072 \n",
       "L 307.714061 195.768072 \n",
       "L 308.834543 195.768072 \n",
       "L 309.955024 195.768072 \n",
       "L 311.075506 195.768072 \n",
       "L 312.195988 195.768072 \n",
       "L 313.31647 195.768072 \n",
       "L 314.436952 195.768072 \n",
       "L 315.557434 195.768072 \n",
       "L 316.677916 195.768072 \n",
       "L 317.798398 195.768072 \n",
       "L 318.91888 195.768072 \n",
       "L 320.039362 195.768072 \n",
       "L 321.159844 195.768072 \n",
       "L 321.159844 195.631627 \n",
       "L 320.039362 195.631627 \n",
       "L 318.91888 195.631627 \n",
       "L 317.798398 195.631627 \n",
       "L 316.677916 195.631627 \n",
       "L 315.557434 195.631627 \n",
       "L 314.436952 195.631627 \n",
       "L 313.31647 195.631627 \n",
       "L 312.195988 195.631627 \n",
       "L 311.075506 195.631627 \n",
       "L 309.955024 195.631627 \n",
       "L 308.834543 195.631627 \n",
       "L 307.714061 195.631627 \n",
       "L 306.593579 195.631627 \n",
       "L 305.473097 195.631627 \n",
       "L 304.352615 195.631627 \n",
       "L 303.232133 195.631627 \n",
       "L 302.111651 195.631627 \n",
       "L 300.991169 195.631627 \n",
       "L 299.870687 195.631627 \n",
       "L 298.750205 195.631627 \n",
       "L 297.629723 195.631627 \n",
       "L 296.509241 195.631627 \n",
       "L 295.388759 195.631627 \n",
       "L 294.268277 195.631627 \n",
       "L 293.147796 195.631627 \n",
       "L 292.027314 195.631627 \n",
       "L 290.906832 195.631627 \n",
       "L 289.78635 195.631627 \n",
       "L 288.665868 195.631627 \n",
       "L 287.545386 195.631627 \n",
       "L 286.424904 195.631627 \n",
       "L 285.304422 195.631627 \n",
       "L 284.18394 195.631627 \n",
       "L 283.063458 195.631627 \n",
       "L 281.942976 195.631627 \n",
       "L 280.822494 195.631627 \n",
       "L 279.702012 195.631627 \n",
       "L 278.58153 195.631627 \n",
       "L 277.461049 195.631627 \n",
       "L 276.340567 195.631627 \n",
       "L 275.220085 195.631627 \n",
       "L 274.099603 195.631627 \n",
       "L 272.979121 195.631627 \n",
       "L 271.858639 195.631627 \n",
       "L 270.738157 195.631627 \n",
       "L 269.617675 195.631627 \n",
       "L 268.497193 195.631627 \n",
       "L 267.376711 195.631627 \n",
       "L 266.256229 195.631627 \n",
       "L 265.135747 195.631627 \n",
       "L 264.015265 195.631627 \n",
       "L 262.894784 195.631627 \n",
       "L 261.774302 195.631627 \n",
       "L 260.65382 195.631627 \n",
       "L 259.533338 195.631627 \n",
       "L 258.412856 195.631627 \n",
       "L 257.292374 195.631627 \n",
       "L 256.171892 195.631627 \n",
       "L 255.05141 195.631627 \n",
       "L 253.930928 195.631627 \n",
       "L 252.810446 195.631627 \n",
       "L 251.689964 195.631627 \n",
       "L 250.569482 195.631627 \n",
       "L 249.449 195.631627 \n",
       "L 248.328518 195.631627 \n",
       "L 247.208037 195.631627 \n",
       "L 246.087555 195.631627 \n",
       "L 244.967073 195.631627 \n",
       "L 243.846591 195.631627 \n",
       "L 242.726109 195.631627 \n",
       "L 241.605627 195.631627 \n",
       "L 240.485145 195.631627 \n",
       "L 239.364663 195.631627 \n",
       "L 238.244181 195.631627 \n",
       "L 237.123699 195.631627 \n",
       "L 236.003217 195.631627 \n",
       "L 234.882735 195.631627 \n",
       "L 233.762253 195.631627 \n",
       "L 232.641771 195.631627 \n",
       "L 231.52129 195.631627 \n",
       "L 230.400808 195.631627 \n",
       "L 229.280326 195.631627 \n",
       "L 228.159844 195.631627 \n",
       "L 227.039362 195.631627 \n",
       "L 225.91888 195.631627 \n",
       "L 224.798398 195.222289 \n",
       "L 223.677916 195.222289 \n",
       "L 222.557434 195.222289 \n",
       "L 221.436952 195.222289 \n",
       "L 220.31647 195.222289 \n",
       "L 219.195988 195.222289 \n",
       "L 218.075506 195.222289 \n",
       "L 216.955024 195.222289 \n",
       "L 215.834543 195.222289 \n",
       "L 214.714061 195.222289 \n",
       "L 213.593579 195.222289 \n",
       "L 212.473097 195.222289 \n",
       "L 211.352615 195.222289 \n",
       "L 210.232133 195.222289 \n",
       "L 209.111651 195.222289 \n",
       "L 207.991169 195.222289 \n",
       "L 206.870687 195.222289 \n",
       "L 205.750205 195.222289 \n",
       "L 204.629723 195.222289 \n",
       "L 203.509241 195.222289 \n",
       "L 202.388759 195.222289 \n",
       "L 201.268277 195.222289 \n",
       "L 200.147796 195.222289 \n",
       "L 199.027314 195.222289 \n",
       "L 197.906832 195.222289 \n",
       "L 196.78635 195.222289 \n",
       "L 195.665868 195.222289 \n",
       "L 194.545386 195.222289 \n",
       "L 193.424904 195.222289 \n",
       "L 192.304422 195.222289 \n",
       "L 191.18394 195.222289 \n",
       "L 190.063458 195.222289 \n",
       "L 188.942976 195.222289 \n",
       "L 187.822494 195.222289 \n",
       "L 186.702012 195.222289 \n",
       "L 185.58153 195.222289 \n",
       "L 184.461049 195.222289 \n",
       "L 183.340567 195.222289 \n",
       "L 182.220085 195.222289 \n",
       "L 181.099603 195.222289 \n",
       "L 179.979121 195.222289 \n",
       "L 178.858639 195.222289 \n",
       "L 177.738157 195.222289 \n",
       "L 176.617675 195.222289 \n",
       "L 175.497193 195.222289 \n",
       "L 174.376711 195.222289 \n",
       "L 173.256229 195.222289 \n",
       "L 172.135747 195.222289 \n",
       "L 171.015265 195.222289 \n",
       "L 169.894784 195.222289 \n",
       "L 168.774302 195.222289 \n",
       "L 167.65382 195.222289 \n",
       "L 166.533338 195.222289 \n",
       "L 165.412856 195.222289 \n",
       "L 164.292374 195.222289 \n",
       "L 163.171892 195.222289 \n",
       "L 162.05141 195.222289 \n",
       "L 160.930928 195.222289 \n",
       "L 159.810446 195.222289 \n",
       "L 158.689964 195.222289 \n",
       "L 157.569482 195.222289 \n",
       "L 156.449 195.222289 \n",
       "L 155.328518 195.222289 \n",
       "L 154.208037 195.222289 \n",
       "L 153.087555 195.222289 \n",
       "L 151.967073 195.222289 \n",
       "L 150.846591 195.222289 \n",
       "L 149.726109 195.222289 \n",
       "L 148.605627 195.222289 \n",
       "L 147.485145 195.222289 \n",
       "L 146.364663 195.222289 \n",
       "L 145.244181 195.222289 \n",
       "L 144.123699 195.222289 \n",
       "L 143.003217 195.222289 \n",
       "L 141.882735 195.222289 \n",
       "L 140.762253 195.222289 \n",
       "L 139.641771 195.222289 \n",
       "L 138.52129 195.222289 \n",
       "L 137.400808 195.222289 \n",
       "L 136.280326 195.222289 \n",
       "L 135.159844 195.222289 \n",
       "L 134.039362 195.222289 \n",
       "L 132.91888 195.222289 \n",
       "L 131.798398 195.222289 \n",
       "L 130.677916 195.222289 \n",
       "L 129.557434 195.222289 \n",
       "L 128.436952 195.222289 \n",
       "L 127.31647 195.222289 \n",
       "L 126.195988 195.222289 \n",
       "L 125.075506 195.222289 \n",
       "L 123.955024 195.222289 \n",
       "L 122.834543 195.222289 \n",
       "L 121.714061 195.222289 \n",
       "L 120.593579 195.222289 \n",
       "L 119.473097 195.222289 \n",
       "L 118.352615 195.222289 \n",
       "L 117.232133 195.222289 \n",
       "L 116.111651 195.222289 \n",
       "L 114.991169 195.222289 \n",
       "L 113.870687 195.222289 \n",
       "L 112.750205 195.222289 \n",
       "L 111.629723 195.222289 \n",
       "L 110.509241 195.222289 \n",
       "L 109.388759 195.222289 \n",
       "L 108.268277 195.222289 \n",
       "L 107.147796 195.222289 \n",
       "L 106.027314 195.222289 \n",
       "L 104.906832 195.222289 \n",
       "L 103.78635 195.222289 \n",
       "L 102.665868 195.222289 \n",
       "L 101.545386 195.222289 \n",
       "L 100.424904 195.222289 \n",
       "L 99.304422 195.222289 \n",
       "L 98.18394 195.222289 \n",
       "L 97.063458 195.222289 \n",
       "L 95.942976 195.222289 \n",
       "L 94.822494 195.222289 \n",
       "L 93.702012 195.222289 \n",
       "L 92.58153 195.222289 \n",
       "L 91.461049 195.222289 \n",
       "L 90.340567 195.222289 \n",
       "L 89.220085 195.222289 \n",
       "L 88.099603 195.222289 \n",
       "L 86.979121 195.222289 \n",
       "L 85.858639 195.222289 \n",
       "L 84.738157 195.222289 \n",
       "L 83.617675 195.222289 \n",
       "L 82.497193 195.222289 \n",
       "L 81.376711 195.222289 \n",
       "L 80.256229 195.222289 \n",
       "L 79.135747 195.222289 \n",
       "L 78.015265 195.222289 \n",
       "L 76.894784 195.222289 \n",
       "L 75.774302 195.222289 \n",
       "L 74.65382 195.222289 \n",
       "L 73.533338 195.222289 \n",
       "L 72.412856 195.222289 \n",
       "L 71.292374 195.222289 \n",
       "L 70.171892 195.222289 \n",
       "L 69.05141 195.222289 \n",
       "L 67.930928 195.222289 \n",
       "L 66.810446 195.222289 \n",
       "L 65.689964 195.222289 \n",
       "L 64.569482 195.222289 \n",
       "L 63.449 195.222289 \n",
       "L 62.328518 195.222289 \n",
       "L 61.208037 195.222289 \n",
       "L 60.087555 195.222289 \n",
       "L 58.967073 195.222289 \n",
       "L 57.846591 195.222289 \n",
       "L 56.726109 195.222289 \n",
       "L 55.605627 195.222289 \n",
       "L 54.485145 195.222289 \n",
       "L 53.364663 195.222289 \n",
       "L 52.244181 195.222289 \n",
       "L 51.123699 195.222289 \n",
       "L 50.003217 195.222289 \n",
       "L 48.882735 195.222289 \n",
       "L 47.762253 195.222289 \n",
       "L 46.641771 195.222289 \n",
       "L 45.52129 195.222289 \n",
       "L 44.400808 195.222289 \n",
       "L 43.280326 195.222289 \n",
       "L 42.159844 195.222289 \n",
       "L 42.159844 195.495181 \n",
       "z\n",
       "\" style=\"fill:#d62728;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_4\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 43.280326 195.222289 \n",
       "L 44.400808 195.222289 \n",
       "L 45.52129 195.222289 \n",
       "L 46.641771 195.222289 \n",
       "L 47.762253 195.222289 \n",
       "L 48.882735 195.222289 \n",
       "L 50.003217 195.222289 \n",
       "L 51.123699 195.222289 \n",
       "L 52.244181 195.222289 \n",
       "L 53.364663 195.222289 \n",
       "L 54.485145 195.222289 \n",
       "L 55.605627 195.222289 \n",
       "L 56.726109 195.222289 \n",
       "L 57.846591 195.222289 \n",
       "L 58.967073 195.222289 \n",
       "L 60.087555 195.222289 \n",
       "L 61.208037 195.222289 \n",
       "L 62.328518 195.222289 \n",
       "L 63.449 195.222289 \n",
       "L 64.569482 195.222289 \n",
       "L 65.689964 195.222289 \n",
       "L 66.810446 195.222289 \n",
       "L 67.930928 195.222289 \n",
       "L 69.05141 195.222289 \n",
       "L 70.171892 195.222289 \n",
       "L 71.292374 195.222289 \n",
       "L 72.412856 195.222289 \n",
       "L 73.533338 195.222289 \n",
       "L 74.65382 195.222289 \n",
       "L 75.774302 195.222289 \n",
       "L 76.894784 195.222289 \n",
       "L 78.015265 195.222289 \n",
       "L 79.135747 195.222289 \n",
       "L 80.256229 195.222289 \n",
       "L 81.376711 195.222289 \n",
       "L 82.497193 195.222289 \n",
       "L 83.617675 195.222289 \n",
       "L 84.738157 195.222289 \n",
       "L 85.858639 195.222289 \n",
       "L 86.979121 195.222289 \n",
       "L 88.099603 195.222289 \n",
       "L 89.220085 195.222289 \n",
       "L 90.340567 195.222289 \n",
       "L 91.461049 195.222289 \n",
       "L 92.58153 195.222289 \n",
       "L 93.702012 195.222289 \n",
       "L 94.822494 195.222289 \n",
       "L 95.942976 195.222289 \n",
       "L 97.063458 195.222289 \n",
       "L 98.18394 195.222289 \n",
       "L 99.304422 195.222289 \n",
       "L 100.424904 195.222289 \n",
       "L 101.545386 195.222289 \n",
       "L 102.665868 195.222289 \n",
       "L 103.78635 195.222289 \n",
       "L 104.906832 195.222289 \n",
       "L 106.027314 195.222289 \n",
       "L 107.147796 195.222289 \n",
       "L 108.268277 195.222289 \n",
       "L 109.388759 195.222289 \n",
       "L 110.509241 195.222289 \n",
       "L 111.629723 195.222289 \n",
       "L 112.750205 195.222289 \n",
       "L 113.870687 195.222289 \n",
       "L 114.991169 195.222289 \n",
       "L 116.111651 195.222289 \n",
       "L 117.232133 195.222289 \n",
       "L 118.352615 195.222289 \n",
       "L 119.473097 195.222289 \n",
       "L 120.593579 195.222289 \n",
       "L 121.714061 195.222289 \n",
       "L 122.834543 195.222289 \n",
       "L 123.955024 195.222289 \n",
       "L 125.075506 195.222289 \n",
       "L 126.195988 195.222289 \n",
       "L 127.31647 195.222289 \n",
       "L 128.436952 195.222289 \n",
       "L 129.557434 195.222289 \n",
       "L 130.677916 195.222289 \n",
       "L 131.798398 195.222289 \n",
       "L 132.91888 195.222289 \n",
       "L 134.039362 195.222289 \n",
       "L 135.159844 195.222289 \n",
       "L 136.280326 195.222289 \n",
       "L 137.400808 195.222289 \n",
       "L 138.52129 195.222289 \n",
       "L 139.641771 195.222289 \n",
       "L 140.762253 195.222289 \n",
       "L 141.882735 195.222289 \n",
       "L 143.003217 195.222289 \n",
       "L 144.123699 195.222289 \n",
       "L 145.244181 195.222289 \n",
       "L 146.364663 195.222289 \n",
       "L 147.485145 195.222289 \n",
       "L 148.605627 195.222289 \n",
       "L 149.726109 195.222289 \n",
       "L 150.846591 195.222289 \n",
       "L 151.967073 195.222289 \n",
       "L 153.087555 195.222289 \n",
       "L 154.208037 195.222289 \n",
       "L 155.328518 195.222289 \n",
       "L 156.449 195.222289 \n",
       "L 157.569482 195.222289 \n",
       "L 158.689964 195.222289 \n",
       "L 159.810446 195.222289 \n",
       "L 160.930928 195.222289 \n",
       "L 162.05141 195.222289 \n",
       "L 163.171892 195.222289 \n",
       "L 164.292374 195.222289 \n",
       "L 165.412856 195.222289 \n",
       "L 166.533338 195.222289 \n",
       "L 167.65382 195.222289 \n",
       "L 168.774302 195.222289 \n",
       "L 169.894784 195.222289 \n",
       "L 171.015265 195.222289 \n",
       "L 172.135747 195.222289 \n",
       "L 173.256229 195.222289 \n",
       "L 174.376711 195.222289 \n",
       "L 175.497193 195.222289 \n",
       "L 176.617675 195.222289 \n",
       "L 177.738157 195.222289 \n",
       "L 178.858639 195.222289 \n",
       "L 179.979121 195.222289 \n",
       "L 181.099603 195.222289 \n",
       "L 182.220085 195.222289 \n",
       "L 183.340567 195.222289 \n",
       "L 184.461049 195.222289 \n",
       "L 185.58153 195.222289 \n",
       "L 186.702012 195.222289 \n",
       "L 187.822494 195.222289 \n",
       "L 188.942976 195.222289 \n",
       "L 190.063458 195.222289 \n",
       "L 191.18394 195.222289 \n",
       "L 192.304422 195.222289 \n",
       "L 193.424904 195.222289 \n",
       "L 194.545386 195.222289 \n",
       "L 195.665868 195.222289 \n",
       "L 196.78635 195.222289 \n",
       "L 197.906832 195.222289 \n",
       "L 199.027314 195.222289 \n",
       "L 200.147796 195.222289 \n",
       "L 201.268277 195.222289 \n",
       "L 202.388759 195.222289 \n",
       "L 203.509241 195.222289 \n",
       "L 204.629723 195.222289 \n",
       "L 205.750205 195.222289 \n",
       "L 206.870687 195.222289 \n",
       "L 207.991169 195.222289 \n",
       "L 209.111651 195.222289 \n",
       "L 210.232133 195.222289 \n",
       "L 211.352615 195.222289 \n",
       "L 212.473097 195.222289 \n",
       "L 213.593579 195.222289 \n",
       "L 214.714061 195.222289 \n",
       "L 215.834543 195.222289 \n",
       "L 216.955024 195.222289 \n",
       "L 218.075506 195.222289 \n",
       "L 219.195988 195.222289 \n",
       "L 220.31647 195.222289 \n",
       "L 221.436952 195.222289 \n",
       "L 222.557434 195.222289 \n",
       "L 223.677916 195.222289 \n",
       "L 224.798398 195.222289 \n",
       "L 225.91888 195.631627 \n",
       "L 227.039362 195.631627 \n",
       "L 228.159844 195.631627 \n",
       "L 229.280326 195.631627 \n",
       "L 230.400808 195.631627 \n",
       "L 231.52129 195.631627 \n",
       "L 232.641771 195.631627 \n",
       "L 233.762253 195.631627 \n",
       "L 234.882735 195.631627 \n",
       "L 236.003217 195.631627 \n",
       "L 237.123699 195.631627 \n",
       "L 238.244181 195.631627 \n",
       "L 239.364663 195.631627 \n",
       "L 240.485145 195.631627 \n",
       "L 241.605627 195.631627 \n",
       "L 242.726109 195.631627 \n",
       "L 243.846591 195.631627 \n",
       "L 244.967073 195.631627 \n",
       "L 246.087555 195.631627 \n",
       "L 247.208037 195.631627 \n",
       "L 248.328518 195.631627 \n",
       "L 249.449 195.631627 \n",
       "L 250.569482 195.631627 \n",
       "L 251.689964 195.631627 \n",
       "L 252.810446 195.631627 \n",
       "L 253.930928 195.631627 \n",
       "L 255.05141 195.631627 \n",
       "L 256.171892 195.631627 \n",
       "L 257.292374 195.631627 \n",
       "L 258.412856 195.631627 \n",
       "L 259.533338 195.631627 \n",
       "L 260.65382 195.631627 \n",
       "L 261.774302 195.631627 \n",
       "L 262.894784 195.631627 \n",
       "L 264.015265 195.631627 \n",
       "L 265.135747 195.631627 \n",
       "L 266.256229 195.631627 \n",
       "L 267.376711 195.631627 \n",
       "L 268.497193 195.631627 \n",
       "L 269.617675 195.631627 \n",
       "L 270.738157 195.631627 \n",
       "L 271.858639 195.631627 \n",
       "L 272.979121 195.631627 \n",
       "L 274.099603 195.631627 \n",
       "L 275.220085 195.631627 \n",
       "L 276.340567 195.631627 \n",
       "L 277.461049 195.631627 \n",
       "L 278.58153 195.631627 \n",
       "L 279.702012 195.631627 \n",
       "L 280.822494 195.631627 \n",
       "L 281.942976 195.631627 \n",
       "L 283.063458 195.631627 \n",
       "L 284.18394 195.631627 \n",
       "L 285.304422 195.631627 \n",
       "L 286.424904 195.631627 \n",
       "L 287.545386 195.631627 \n",
       "L 288.665868 195.631627 \n",
       "L 289.78635 195.631627 \n",
       "L 290.906832 195.631627 \n",
       "L 292.027314 195.631627 \n",
       "L 293.147796 195.631627 \n",
       "L 294.268277 195.631627 \n",
       "L 295.388759 195.631627 \n",
       "L 296.509241 195.631627 \n",
       "L 297.629723 195.631627 \n",
       "L 298.750205 195.631627 \n",
       "L 299.870687 195.631627 \n",
       "L 300.991169 195.631627 \n",
       "L 302.111651 195.631627 \n",
       "L 303.232133 195.631627 \n",
       "L 304.352615 195.631627 \n",
       "L 305.473097 195.631627 \n",
       "L 306.593579 195.631627 \n",
       "L 307.714061 195.631627 \n",
       "L 308.834543 195.631627 \n",
       "L 309.955024 195.631627 \n",
       "L 311.075506 195.631627 \n",
       "L 312.195988 195.631627 \n",
       "L 313.31647 195.631627 \n",
       "L 314.436952 195.631627 \n",
       "L 315.557434 195.631627 \n",
       "L 316.677916 195.631627 \n",
       "L 317.798398 195.631627 \n",
       "L 318.91888 195.631627 \n",
       "L 320.039362 195.631627 \n",
       "L 321.159844 195.631627 \n",
       "L 321.159844 195.495181 \n",
       "L 320.039362 195.495181 \n",
       "L 318.91888 195.495181 \n",
       "L 317.798398 195.495181 \n",
       "L 316.677916 195.495181 \n",
       "L 315.557434 195.495181 \n",
       "L 314.436952 195.495181 \n",
       "L 313.31647 195.495181 \n",
       "L 312.195988 195.495181 \n",
       "L 311.075506 195.495181 \n",
       "L 309.955024 195.495181 \n",
       "L 308.834543 195.495181 \n",
       "L 307.714061 195.495181 \n",
       "L 306.593579 195.495181 \n",
       "L 305.473097 195.495181 \n",
       "L 304.352615 195.495181 \n",
       "L 303.232133 195.495181 \n",
       "L 302.111651 195.495181 \n",
       "L 300.991169 195.495181 \n",
       "L 299.870687 195.495181 \n",
       "L 298.750205 195.495181 \n",
       "L 297.629723 195.495181 \n",
       "L 296.509241 195.495181 \n",
       "L 295.388759 195.495181 \n",
       "L 294.268277 195.495181 \n",
       "L 293.147796 195.495181 \n",
       "L 292.027314 195.495181 \n",
       "L 290.906832 195.495181 \n",
       "L 289.78635 195.495181 \n",
       "L 288.665868 195.495181 \n",
       "L 287.545386 195.495181 \n",
       "L 286.424904 195.495181 \n",
       "L 285.304422 195.495181 \n",
       "L 284.18394 195.495181 \n",
       "L 283.063458 195.495181 \n",
       "L 281.942976 195.495181 \n",
       "L 280.822494 195.495181 \n",
       "L 279.702012 195.495181 \n",
       "L 278.58153 195.495181 \n",
       "L 277.461049 195.495181 \n",
       "L 276.340567 195.495181 \n",
       "L 275.220085 195.495181 \n",
       "L 274.099603 195.495181 \n",
       "L 272.979121 195.495181 \n",
       "L 271.858639 195.495181 \n",
       "L 270.738157 195.495181 \n",
       "L 269.617675 195.495181 \n",
       "L 268.497193 195.495181 \n",
       "L 267.376711 195.495181 \n",
       "L 266.256229 195.495181 \n",
       "L 265.135747 195.495181 \n",
       "L 264.015265 195.495181 \n",
       "L 262.894784 195.495181 \n",
       "L 261.774302 195.495181 \n",
       "L 260.65382 195.495181 \n",
       "L 259.533338 195.495181 \n",
       "L 258.412856 195.495181 \n",
       "L 257.292374 195.495181 \n",
       "L 256.171892 195.495181 \n",
       "L 255.05141 195.495181 \n",
       "L 253.930928 195.495181 \n",
       "L 252.810446 195.495181 \n",
       "L 251.689964 195.495181 \n",
       "L 250.569482 195.495181 \n",
       "L 249.449 195.495181 \n",
       "L 248.328518 195.495181 \n",
       "L 247.208037 195.495181 \n",
       "L 246.087555 195.495181 \n",
       "L 244.967073 195.495181 \n",
       "L 243.846591 195.495181 \n",
       "L 242.726109 195.495181 \n",
       "L 241.605627 195.495181 \n",
       "L 240.485145 195.495181 \n",
       "L 239.364663 195.495181 \n",
       "L 238.244181 195.495181 \n",
       "L 237.123699 195.495181 \n",
       "L 236.003217 195.495181 \n",
       "L 234.882735 195.495181 \n",
       "L 233.762253 195.495181 \n",
       "L 232.641771 195.495181 \n",
       "L 231.52129 195.495181 \n",
       "L 230.400808 195.495181 \n",
       "L 229.280326 195.495181 \n",
       "L 228.159844 195.495181 \n",
       "L 227.039362 195.495181 \n",
       "L 225.91888 195.495181 \n",
       "L 224.798398 194.949398 \n",
       "L 224.798398 193.857831 \n",
       "L 224.798398 192.766265 \n",
       "L 224.798398 191.674699 \n",
       "L 224.798398 190.583133 \n",
       "L 224.798398 189.491566 \n",
       "L 224.798398 188.4 \n",
       "L 224.798398 187.308434 \n",
       "L 224.798398 186.216867 \n",
       "L 224.798398 185.125301 \n",
       "L 224.798398 184.033735 \n",
       "L 224.798398 182.942169 \n",
       "L 224.798398 181.850602 \n",
       "L 224.798398 180.759036 \n",
       "L 224.798398 179.66747 \n",
       "L 224.798398 178.575904 \n",
       "L 224.798398 177.484337 \n",
       "L 224.798398 176.392771 \n",
       "L 224.798398 175.301205 \n",
       "L 224.798398 174.209639 \n",
       "L 224.798398 173.118072 \n",
       "L 224.798398 172.026506 \n",
       "L 224.798398 170.93494 \n",
       "L 224.798398 169.843373 \n",
       "L 224.798398 168.751807 \n",
       "L 224.798398 167.660241 \n",
       "L 224.798398 166.568675 \n",
       "L 224.798398 165.477108 \n",
       "L 224.798398 164.385542 \n",
       "L 224.798398 163.293976 \n",
       "L 224.798398 162.20241 \n",
       "L 224.798398 161.110843 \n",
       "L 224.798398 160.019277 \n",
       "L 224.798398 158.927711 \n",
       "L 224.798398 157.836145 \n",
       "L 224.798398 156.744578 \n",
       "L 224.798398 155.653012 \n",
       "L 224.798398 154.561446 \n",
       "L 224.798398 153.46988 \n",
       "L 224.798398 152.378313 \n",
       "L 224.798398 151.286747 \n",
       "L 224.798398 150.195181 \n",
       "L 224.798398 149.103614 \n",
       "L 224.798398 148.012048 \n",
       "L 224.798398 146.920482 \n",
       "L 224.798398 145.828916 \n",
       "L 224.798398 144.737349 \n",
       "L 224.798398 143.645783 \n",
       "L 224.798398 142.554217 \n",
       "L 224.798398 141.462651 \n",
       "L 224.798398 140.371084 \n",
       "L 224.798398 139.279518 \n",
       "L 224.798398 138.187952 \n",
       "L 224.798398 137.096386 \n",
       "L 224.798398 136.004819 \n",
       "L 224.798398 134.913253 \n",
       "L 224.798398 133.821687 \n",
       "L 224.798398 132.73012 \n",
       "L 224.798398 131.638554 \n",
       "L 224.798398 130.546988 \n",
       "L 224.798398 129.455422 \n",
       "L 224.798398 128.363855 \n",
       "L 224.798398 127.272289 \n",
       "L 224.798398 126.180723 \n",
       "L 224.798398 125.089157 \n",
       "L 224.798398 123.99759 \n",
       "L 224.798398 122.906024 \n",
       "L 224.798398 121.814458 \n",
       "L 224.798398 120.722892 \n",
       "L 224.798398 119.631325 \n",
       "L 224.798398 118.539759 \n",
       "L 224.798398 117.448193 \n",
       "L 225.91888 116.356627 \n",
       "L 227.039362 116.356627 \n",
       "L 228.159844 116.356627 \n",
       "L 229.280326 116.356627 \n",
       "L 230.400808 116.356627 \n",
       "L 231.52129 116.356627 \n",
       "L 232.641771 116.356627 \n",
       "L 233.762253 116.356627 \n",
       "L 234.882735 116.356627 \n",
       "L 236.003217 116.356627 \n",
       "L 237.123699 116.356627 \n",
       "L 238.244181 116.356627 \n",
       "L 239.364663 116.356627 \n",
       "L 240.485145 116.356627 \n",
       "L 241.605627 116.356627 \n",
       "L 242.726109 116.356627 \n",
       "L 243.846591 116.356627 \n",
       "L 244.967073 116.356627 \n",
       "L 246.087555 116.356627 \n",
       "L 247.208037 116.356627 \n",
       "L 247.208037 115.26506 \n",
       "L 247.208037 114.173494 \n",
       "L 247.208037 113.081928 \n",
       "L 247.208037 111.990361 \n",
       "L 247.208037 110.898795 \n",
       "L 247.208037 109.807229 \n",
       "L 247.208037 108.715663 \n",
       "L 247.208037 107.624096 \n",
       "L 247.208037 106.53253 \n",
       "L 247.208037 105.440964 \n",
       "L 247.208037 104.349398 \n",
       "L 247.208037 103.257831 \n",
       "L 247.208037 102.166265 \n",
       "L 247.208037 101.074699 \n",
       "L 247.208037 99.983133 \n",
       "L 247.208037 98.891566 \n",
       "L 247.208037 97.8 \n",
       "L 247.208037 96.708434 \n",
       "L 246.087555 96.708434 \n",
       "L 244.967073 96.708434 \n",
       "L 243.846591 96.708434 \n",
       "L 242.726109 96.708434 \n",
       "L 241.605627 96.708434 \n",
       "L 240.485145 96.708434 \n",
       "L 239.364663 96.708434 \n",
       "L 238.244181 96.708434 \n",
       "L 237.123699 96.708434 \n",
       "L 236.003217 96.708434 \n",
       "L 234.882735 96.708434 \n",
       "L 233.762253 96.708434 \n",
       "L 232.641771 96.708434 \n",
       "L 231.52129 96.708434 \n",
       "L 230.400808 96.708434 \n",
       "L 229.280326 96.708434 \n",
       "L 228.159844 96.708434 \n",
       "L 227.039362 96.708434 \n",
       "L 225.91888 96.708434 \n",
       "L 225.91888 97.8 \n",
       "L 225.91888 98.891566 \n",
       "L 225.91888 99.983133 \n",
       "L 225.91888 101.074699 \n",
       "L 225.91888 102.166265 \n",
       "L 225.91888 103.257831 \n",
       "L 225.91888 104.349398 \n",
       "L 225.91888 105.440964 \n",
       "L 224.798398 106.53253 \n",
       "L 223.677916 106.53253 \n",
       "L 222.557434 106.53253 \n",
       "L 221.436952 106.53253 \n",
       "L 220.31647 106.53253 \n",
       "L 219.195988 106.53253 \n",
       "L 218.075506 106.53253 \n",
       "L 216.955024 106.53253 \n",
       "L 215.834543 106.53253 \n",
       "L 214.714061 106.53253 \n",
       "L 213.593579 106.53253 \n",
       "L 212.473097 106.53253 \n",
       "L 211.352615 106.53253 \n",
       "L 210.232133 106.53253 \n",
       "L 209.111651 106.53253 \n",
       "L 207.991169 106.53253 \n",
       "L 206.870687 106.53253 \n",
       "L 205.750205 106.53253 \n",
       "L 204.629723 106.53253 \n",
       "L 203.509241 106.53253 \n",
       "L 202.388759 106.53253 \n",
       "L 201.268277 106.53253 \n",
       "L 200.147796 106.53253 \n",
       "L 199.027314 106.53253 \n",
       "L 197.906832 106.53253 \n",
       "L 196.78635 106.53253 \n",
       "L 195.665868 106.53253 \n",
       "L 194.545386 106.53253 \n",
       "L 193.424904 106.53253 \n",
       "L 192.304422 106.53253 \n",
       "L 191.18394 106.53253 \n",
       "L 190.063458 106.53253 \n",
       "L 188.942976 106.53253 \n",
       "L 187.822494 106.53253 \n",
       "L 186.702012 106.53253 \n",
       "L 185.58153 106.53253 \n",
       "L 184.461049 106.53253 \n",
       "L 183.340567 106.53253 \n",
       "L 182.220085 106.53253 \n",
       "L 181.099603 106.53253 \n",
       "L 179.979121 106.53253 \n",
       "L 178.858639 106.53253 \n",
       "L 177.738157 106.53253 \n",
       "L 176.617675 106.53253 \n",
       "L 175.497193 106.53253 \n",
       "L 174.376711 106.53253 \n",
       "L 173.256229 106.53253 \n",
       "L 172.135747 106.53253 \n",
       "L 171.015265 106.53253 \n",
       "L 169.894784 106.53253 \n",
       "L 168.774302 106.53253 \n",
       "L 167.65382 106.53253 \n",
       "L 166.533338 106.53253 \n",
       "L 165.412856 106.53253 \n",
       "L 164.292374 106.53253 \n",
       "L 163.171892 106.53253 \n",
       "L 162.05141 106.53253 \n",
       "L 160.930928 106.53253 \n",
       "L 159.810446 106.53253 \n",
       "L 158.689964 106.53253 \n",
       "L 157.569482 106.53253 \n",
       "L 156.449 106.53253 \n",
       "L 155.328518 106.53253 \n",
       "L 154.208037 106.53253 \n",
       "L 153.087555 106.53253 \n",
       "L 151.967073 106.53253 \n",
       "L 150.846591 106.53253 \n",
       "L 149.726109 106.53253 \n",
       "L 148.605627 106.53253 \n",
       "L 147.485145 106.53253 \n",
       "L 146.364663 106.53253 \n",
       "L 145.244181 106.53253 \n",
       "L 144.123699 106.53253 \n",
       "L 143.003217 106.53253 \n",
       "L 141.882735 106.53253 \n",
       "L 140.762253 106.53253 \n",
       "L 139.641771 106.53253 \n",
       "L 138.52129 106.53253 \n",
       "L 137.400808 106.53253 \n",
       "L 136.280326 106.53253 \n",
       "L 135.159844 106.53253 \n",
       "L 134.039362 106.53253 \n",
       "L 132.91888 106.53253 \n",
       "L 131.798398 106.53253 \n",
       "L 130.677916 106.53253 \n",
       "L 129.557434 106.53253 \n",
       "L 128.436952 106.53253 \n",
       "L 127.31647 106.53253 \n",
       "L 126.195988 106.53253 \n",
       "L 125.075506 106.53253 \n",
       "L 123.955024 106.53253 \n",
       "L 122.834543 106.53253 \n",
       "L 121.714061 106.53253 \n",
       "L 120.593579 106.53253 \n",
       "L 119.473097 106.53253 \n",
       "L 118.352615 106.53253 \n",
       "L 117.232133 106.53253 \n",
       "L 116.111651 106.53253 \n",
       "L 114.991169 106.53253 \n",
       "L 113.870687 106.53253 \n",
       "L 112.750205 106.53253 \n",
       "L 111.629723 106.53253 \n",
       "L 110.509241 106.53253 \n",
       "L 109.388759 106.53253 \n",
       "L 108.268277 106.53253 \n",
       "L 107.147796 106.53253 \n",
       "L 106.027314 106.53253 \n",
       "L 104.906832 106.53253 \n",
       "L 103.78635 106.53253 \n",
       "L 102.665868 106.53253 \n",
       "L 101.545386 106.53253 \n",
       "L 100.424904 106.53253 \n",
       "L 99.304422 106.53253 \n",
       "L 98.18394 106.53253 \n",
       "L 97.063458 106.53253 \n",
       "L 95.942976 106.53253 \n",
       "L 94.822494 106.53253 \n",
       "L 93.702012 106.53253 \n",
       "L 92.58153 106.53253 \n",
       "L 91.461049 106.53253 \n",
       "L 90.340567 106.53253 \n",
       "L 89.220085 106.53253 \n",
       "L 88.099603 106.53253 \n",
       "L 86.979121 106.53253 \n",
       "L 85.858639 106.53253 \n",
       "L 84.738157 106.53253 \n",
       "L 83.617675 106.53253 \n",
       "L 82.497193 106.53253 \n",
       "L 81.376711 106.53253 \n",
       "L 80.256229 106.53253 \n",
       "L 79.135747 106.53253 \n",
       "L 78.015265 106.53253 \n",
       "L 76.894784 106.53253 \n",
       "L 75.774302 106.53253 \n",
       "L 74.65382 106.53253 \n",
       "L 73.533338 106.53253 \n",
       "L 72.412856 106.53253 \n",
       "L 71.292374 106.53253 \n",
       "L 70.171892 106.53253 \n",
       "L 69.05141 106.53253 \n",
       "L 67.930928 106.53253 \n",
       "L 66.810446 106.53253 \n",
       "L 65.689964 106.53253 \n",
       "L 64.569482 106.53253 \n",
       "L 63.449 106.53253 \n",
       "L 62.328518 106.53253 \n",
       "L 61.208037 106.53253 \n",
       "L 60.087555 106.53253 \n",
       "L 58.967073 106.53253 \n",
       "L 57.846591 106.53253 \n",
       "L 56.726109 106.53253 \n",
       "L 55.605627 106.53253 \n",
       "L 54.485145 106.53253 \n",
       "L 53.364663 106.53253 \n",
       "L 52.244181 106.53253 \n",
       "L 51.123699 106.53253 \n",
       "L 50.003217 106.53253 \n",
       "L 48.882735 106.53253 \n",
       "L 47.762253 106.53253 \n",
       "L 46.641771 106.53253 \n",
       "L 45.52129 106.53253 \n",
       "L 44.400808 106.53253 \n",
       "L 43.280326 106.53253 \n",
       "L 42.159844 106.53253 \n",
       "L 42.159844 107.624096 \n",
       "L 42.159844 108.715663 \n",
       "L 42.159844 109.807229 \n",
       "L 42.159844 110.898795 \n",
       "L 42.159844 111.990361 \n",
       "L 42.159844 113.081928 \n",
       "L 42.159844 114.173494 \n",
       "L 42.159844 115.26506 \n",
       "L 42.159844 116.356627 \n",
       "L 42.159844 117.448193 \n",
       "L 42.159844 118.539759 \n",
       "L 42.159844 119.631325 \n",
       "L 42.159844 120.722892 \n",
       "L 42.159844 121.814458 \n",
       "L 42.159844 122.906024 \n",
       "L 42.159844 123.99759 \n",
       "L 42.159844 125.089157 \n",
       "L 42.159844 126.180723 \n",
       "L 42.159844 127.272289 \n",
       "L 42.159844 128.363855 \n",
       "L 42.159844 129.455422 \n",
       "L 42.159844 130.546988 \n",
       "L 42.159844 131.638554 \n",
       "L 42.159844 132.73012 \n",
       "L 42.159844 133.821687 \n",
       "L 42.159844 134.913253 \n",
       "L 42.159844 136.004819 \n",
       "L 42.159844 137.096386 \n",
       "L 42.159844 138.187952 \n",
       "L 42.159844 139.279518 \n",
       "L 42.159844 140.371084 \n",
       "L 42.159844 141.462651 \n",
       "L 42.159844 142.554217 \n",
       "L 42.159844 143.645783 \n",
       "L 42.159844 144.737349 \n",
       "L 42.159844 145.828916 \n",
       "L 42.159844 146.920482 \n",
       "L 42.159844 148.012048 \n",
       "L 42.159844 149.103614 \n",
       "L 42.159844 150.195181 \n",
       "L 42.159844 151.286747 \n",
       "L 42.159844 152.378313 \n",
       "L 42.159844 153.46988 \n",
       "L 42.159844 154.561446 \n",
       "L 42.159844 155.653012 \n",
       "L 42.159844 156.744578 \n",
       "L 42.159844 157.836145 \n",
       "L 42.159844 158.927711 \n",
       "L 42.159844 160.019277 \n",
       "L 42.159844 161.110843 \n",
       "L 42.159844 162.20241 \n",
       "L 42.159844 163.293976 \n",
       "L 42.159844 164.385542 \n",
       "L 42.159844 165.477108 \n",
       "L 42.159844 166.568675 \n",
       "L 42.159844 167.660241 \n",
       "L 42.159844 168.751807 \n",
       "L 42.159844 169.843373 \n",
       "L 42.159844 170.93494 \n",
       "L 42.159844 172.026506 \n",
       "L 42.159844 173.118072 \n",
       "L 42.159844 174.209639 \n",
       "L 42.159844 175.301205 \n",
       "L 42.159844 176.392771 \n",
       "L 42.159844 177.484337 \n",
       "L 42.159844 178.575904 \n",
       "L 42.159844 179.66747 \n",
       "L 42.159844 180.759036 \n",
       "L 42.159844 181.850602 \n",
       "L 42.159844 182.942169 \n",
       "L 42.159844 184.033735 \n",
       "L 42.159844 185.125301 \n",
       "L 42.159844 186.216867 \n",
       "L 42.159844 187.308434 \n",
       "L 42.159844 188.4 \n",
       "L 42.159844 189.491566 \n",
       "L 42.159844 190.583133 \n",
       "L 42.159844 191.674699 \n",
       "L 42.159844 192.766265 \n",
       "L 42.159844 193.857831 \n",
       "L 42.159844 194.949398 \n",
       "L 42.159844 195.222289 \n",
       "z\n",
       "\" style=\"fill:#9467bd;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_5\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 225.91888 195.495181 \n",
       "L 227.039362 195.495181 \n",
       "L 228.159844 195.495181 \n",
       "L 229.280326 195.495181 \n",
       "L 230.400808 195.495181 \n",
       "L 231.52129 195.495181 \n",
       "L 232.641771 195.495181 \n",
       "L 233.762253 195.495181 \n",
       "L 234.882735 195.495181 \n",
       "L 236.003217 195.495181 \n",
       "L 237.123699 195.495181 \n",
       "L 238.244181 195.495181 \n",
       "L 239.364663 195.495181 \n",
       "L 240.485145 195.495181 \n",
       "L 241.605627 195.495181 \n",
       "L 242.726109 195.495181 \n",
       "L 243.846591 195.495181 \n",
       "L 244.967073 195.495181 \n",
       "L 246.087555 195.495181 \n",
       "L 247.208037 195.495181 \n",
       "L 248.328518 195.495181 \n",
       "L 249.449 195.495181 \n",
       "L 250.569482 195.495181 \n",
       "L 251.689964 195.495181 \n",
       "L 252.810446 195.495181 \n",
       "L 253.930928 195.495181 \n",
       "L 255.05141 195.495181 \n",
       "L 256.171892 195.495181 \n",
       "L 257.292374 195.495181 \n",
       "L 258.412856 195.495181 \n",
       "L 259.533338 195.495181 \n",
       "L 260.65382 195.495181 \n",
       "L 261.774302 195.495181 \n",
       "L 262.894784 195.495181 \n",
       "L 264.015265 195.495181 \n",
       "L 265.135747 195.495181 \n",
       "L 266.256229 195.495181 \n",
       "L 267.376711 195.495181 \n",
       "L 268.497193 195.495181 \n",
       "L 269.617675 195.495181 \n",
       "L 270.738157 195.495181 \n",
       "L 271.858639 195.495181 \n",
       "L 272.979121 195.495181 \n",
       "L 274.099603 195.495181 \n",
       "L 275.220085 195.495181 \n",
       "L 276.340567 195.495181 \n",
       "L 277.461049 195.495181 \n",
       "L 278.58153 195.495181 \n",
       "L 279.702012 195.495181 \n",
       "L 280.822494 195.495181 \n",
       "L 281.942976 195.495181 \n",
       "L 283.063458 195.495181 \n",
       "L 284.18394 195.495181 \n",
       "L 285.304422 195.495181 \n",
       "L 286.424904 195.495181 \n",
       "L 287.545386 195.495181 \n",
       "L 288.665868 195.495181 \n",
       "L 289.78635 195.495181 \n",
       "L 290.906832 195.495181 \n",
       "L 292.027314 195.495181 \n",
       "L 293.147796 195.495181 \n",
       "L 294.268277 195.495181 \n",
       "L 295.388759 195.495181 \n",
       "L 296.509241 195.495181 \n",
       "L 297.629723 195.495181 \n",
       "L 298.750205 195.495181 \n",
       "L 299.870687 195.495181 \n",
       "L 300.991169 195.495181 \n",
       "L 302.111651 195.495181 \n",
       "L 303.232133 195.495181 \n",
       "L 304.352615 195.495181 \n",
       "L 305.473097 195.495181 \n",
       "L 306.593579 195.495181 \n",
       "L 307.714061 195.495181 \n",
       "L 308.834543 195.495181 \n",
       "L 309.955024 195.495181 \n",
       "L 311.075506 195.495181 \n",
       "L 312.195988 195.495181 \n",
       "L 313.31647 195.495181 \n",
       "L 314.436952 195.495181 \n",
       "L 315.557434 195.495181 \n",
       "L 316.677916 195.495181 \n",
       "L 317.798398 195.495181 \n",
       "L 318.91888 195.495181 \n",
       "L 320.039362 195.495181 \n",
       "L 321.159844 195.495181 \n",
       "L 321.159844 195.358735 \n",
       "L 320.039362 195.358735 \n",
       "L 318.91888 195.358735 \n",
       "L 317.798398 195.358735 \n",
       "L 316.677916 195.358735 \n",
       "L 315.557434 195.358735 \n",
       "L 314.436952 195.358735 \n",
       "L 313.31647 195.358735 \n",
       "L 312.195988 195.358735 \n",
       "L 311.075506 195.358735 \n",
       "L 309.955024 195.358735 \n",
       "L 308.834543 195.358735 \n",
       "L 307.714061 195.358735 \n",
       "L 306.593579 195.358735 \n",
       "L 305.473097 195.358735 \n",
       "L 304.352615 195.358735 \n",
       "L 303.232133 195.358735 \n",
       "L 302.111651 195.358735 \n",
       "L 300.991169 195.358735 \n",
       "L 299.870687 195.358735 \n",
       "L 298.750205 195.358735 \n",
       "L 297.629723 195.358735 \n",
       "L 296.509241 195.358735 \n",
       "L 295.388759 195.358735 \n",
       "L 294.268277 195.358735 \n",
       "L 293.147796 195.358735 \n",
       "L 292.027314 195.358735 \n",
       "L 290.906832 195.358735 \n",
       "L 289.78635 195.358735 \n",
       "L 288.665868 195.358735 \n",
       "L 287.545386 195.358735 \n",
       "L 286.424904 195.358735 \n",
       "L 285.304422 195.358735 \n",
       "L 284.18394 195.358735 \n",
       "L 283.063458 195.358735 \n",
       "L 281.942976 195.358735 \n",
       "L 280.822494 195.358735 \n",
       "L 279.702012 195.358735 \n",
       "L 278.58153 195.358735 \n",
       "L 277.461049 195.358735 \n",
       "L 276.340567 195.358735 \n",
       "L 275.220085 195.358735 \n",
       "L 274.099603 195.358735 \n",
       "L 272.979121 195.358735 \n",
       "L 271.858639 195.358735 \n",
       "L 270.738157 195.358735 \n",
       "L 269.617675 195.358735 \n",
       "L 268.497193 195.358735 \n",
       "L 267.376711 195.358735 \n",
       "L 266.256229 195.358735 \n",
       "L 265.135747 195.358735 \n",
       "L 264.015265 195.358735 \n",
       "L 262.894784 195.358735 \n",
       "L 261.774302 195.358735 \n",
       "L 260.65382 195.358735 \n",
       "L 259.533338 195.358735 \n",
       "L 258.412856 195.358735 \n",
       "L 257.292374 195.358735 \n",
       "L 256.171892 195.358735 \n",
       "L 255.05141 195.358735 \n",
       "L 253.930928 195.358735 \n",
       "L 252.810446 195.358735 \n",
       "L 251.689964 195.358735 \n",
       "L 250.569482 195.358735 \n",
       "L 249.449 195.358735 \n",
       "L 248.328518 195.358735 \n",
       "L 247.208037 195.358735 \n",
       "L 246.087555 195.358735 \n",
       "L 244.967073 195.358735 \n",
       "L 243.846591 195.358735 \n",
       "L 242.726109 195.358735 \n",
       "L 241.605627 195.358735 \n",
       "L 240.485145 195.358735 \n",
       "L 239.364663 195.358735 \n",
       "L 238.244181 195.358735 \n",
       "L 237.123699 195.358735 \n",
       "L 236.003217 195.358735 \n",
       "L 234.882735 195.358735 \n",
       "L 233.762253 195.358735 \n",
       "L 232.641771 195.358735 \n",
       "L 231.52129 195.358735 \n",
       "L 230.400808 195.358735 \n",
       "L 229.280326 195.358735 \n",
       "L 228.159844 195.358735 \n",
       "L 227.039362 195.358735 \n",
       "L 225.91888 195.358735 \n",
       "L 225.078518 194.949398 \n",
       "L 225.078518 193.857831 \n",
       "L 225.078518 192.766265 \n",
       "L 225.078518 191.674699 \n",
       "L 225.078518 190.583133 \n",
       "L 225.078518 189.491566 \n",
       "L 225.078518 188.4 \n",
       "L 225.078518 187.308434 \n",
       "L 225.078518 186.216867 \n",
       "L 225.078518 185.125301 \n",
       "L 225.078518 184.033735 \n",
       "L 225.078518 182.942169 \n",
       "L 225.078518 181.850602 \n",
       "L 225.078518 180.759036 \n",
       "L 225.078518 179.66747 \n",
       "L 225.078518 178.575904 \n",
       "L 225.078518 177.484337 \n",
       "L 225.078518 176.392771 \n",
       "L 225.078518 175.301205 \n",
       "L 225.078518 174.209639 \n",
       "L 225.078518 173.118072 \n",
       "L 225.078518 172.026506 \n",
       "L 225.078518 170.93494 \n",
       "L 225.078518 169.843373 \n",
       "L 225.078518 168.751807 \n",
       "L 225.078518 167.660241 \n",
       "L 225.078518 166.568675 \n",
       "L 225.078518 165.477108 \n",
       "L 225.078518 164.385542 \n",
       "L 225.078518 163.293976 \n",
       "L 225.078518 162.20241 \n",
       "L 225.078518 161.110843 \n",
       "L 225.078518 160.019277 \n",
       "L 225.078518 158.927711 \n",
       "L 225.078518 157.836145 \n",
       "L 225.078518 156.744578 \n",
       "L 225.078518 155.653012 \n",
       "L 225.078518 154.561446 \n",
       "L 225.078518 153.46988 \n",
       "L 225.078518 152.378313 \n",
       "L 225.078518 151.286747 \n",
       "L 225.078518 150.195181 \n",
       "L 225.078518 149.103614 \n",
       "L 225.078518 148.012048 \n",
       "L 225.078518 146.920482 \n",
       "L 225.078518 145.828916 \n",
       "L 225.078518 144.737349 \n",
       "L 225.078518 143.645783 \n",
       "L 225.078518 142.554217 \n",
       "L 225.078518 141.462651 \n",
       "L 225.078518 140.371084 \n",
       "L 225.078518 139.279518 \n",
       "L 225.078518 138.187952 \n",
       "L 225.078518 137.096386 \n",
       "L 225.078518 136.004819 \n",
       "L 225.078518 134.913253 \n",
       "L 225.078518 133.821687 \n",
       "L 225.078518 132.73012 \n",
       "L 225.078518 131.638554 \n",
       "L 225.078518 130.546988 \n",
       "L 225.078518 129.455422 \n",
       "L 225.078518 128.363855 \n",
       "L 225.078518 127.272289 \n",
       "L 225.078518 126.180723 \n",
       "L 225.078518 125.089157 \n",
       "L 225.078518 123.99759 \n",
       "L 225.078518 122.906024 \n",
       "L 225.078518 121.814458 \n",
       "L 225.078518 120.722892 \n",
       "L 225.078518 119.631325 \n",
       "L 225.078518 118.539759 \n",
       "L 225.078518 117.448193 \n",
       "L 225.91888 116.629518 \n",
       "L 227.039362 116.629518 \n",
       "L 228.159844 116.629518 \n",
       "L 229.280326 116.629518 \n",
       "L 230.400808 116.629518 \n",
       "L 231.52129 116.629518 \n",
       "L 232.641771 116.629518 \n",
       "L 233.762253 116.629518 \n",
       "L 234.882735 116.629518 \n",
       "L 236.003217 116.629518 \n",
       "L 237.123699 116.629518 \n",
       "L 238.244181 116.629518 \n",
       "L 239.364663 116.629518 \n",
       "L 240.485145 116.629518 \n",
       "L 241.605627 116.629518 \n",
       "L 242.726109 116.629518 \n",
       "L 243.846591 116.629518 \n",
       "L 244.967073 116.629518 \n",
       "L 246.087555 116.629518 \n",
       "L 247.208037 116.629518 \n",
       "L 247.488157 116.356627 \n",
       "L 247.488157 115.26506 \n",
       "L 247.488157 114.173494 \n",
       "L 247.488157 113.081928 \n",
       "L 247.488157 111.990361 \n",
       "L 247.488157 110.898795 \n",
       "L 247.488157 109.807229 \n",
       "L 247.488157 108.715663 \n",
       "L 247.488157 107.624096 \n",
       "L 247.488157 106.53253 \n",
       "L 247.488157 105.440964 \n",
       "L 247.488157 104.349398 \n",
       "L 247.488157 103.257831 \n",
       "L 247.488157 102.166265 \n",
       "L 247.488157 101.074699 \n",
       "L 247.488157 99.983133 \n",
       "L 247.488157 98.891566 \n",
       "L 247.488157 97.8 \n",
       "L 247.488157 96.708434 \n",
       "L 247.208037 96.435542 \n",
       "L 246.087555 96.435542 \n",
       "L 244.967073 96.435542 \n",
       "L 243.846591 96.435542 \n",
       "L 242.726109 96.435542 \n",
       "L 241.605627 96.435542 \n",
       "L 240.485145 96.435542 \n",
       "L 239.364663 96.435542 \n",
       "L 238.244181 96.435542 \n",
       "L 237.123699 96.435542 \n",
       "L 236.003217 96.435542 \n",
       "L 234.882735 96.435542 \n",
       "L 233.762253 96.435542 \n",
       "L 232.641771 96.435542 \n",
       "L 231.52129 96.435542 \n",
       "L 230.400808 96.435542 \n",
       "L 229.280326 96.435542 \n",
       "L 228.159844 96.435542 \n",
       "L 227.039362 96.435542 \n",
       "L 225.91888 96.435542 \n",
       "L 225.638759 96.708434 \n",
       "L 225.638759 97.8 \n",
       "L 225.638759 98.891566 \n",
       "L 225.638759 99.983133 \n",
       "L 225.638759 101.074699 \n",
       "L 225.638759 102.166265 \n",
       "L 225.638759 103.257831 \n",
       "L 225.638759 104.349398 \n",
       "L 225.638759 105.440964 \n",
       "L 224.798398 106.259639 \n",
       "L 223.677916 106.259639 \n",
       "L 222.557434 106.259639 \n",
       "L 221.436952 106.259639 \n",
       "L 220.31647 106.259639 \n",
       "L 219.195988 106.259639 \n",
       "L 218.075506 106.259639 \n",
       "L 216.955024 106.259639 \n",
       "L 215.834543 106.259639 \n",
       "L 214.714061 106.259639 \n",
       "L 213.593579 106.259639 \n",
       "L 212.473097 106.259639 \n",
       "L 211.352615 106.259639 \n",
       "L 210.232133 106.259639 \n",
       "L 209.111651 106.259639 \n",
       "L 207.991169 106.259639 \n",
       "L 206.870687 106.259639 \n",
       "L 205.750205 106.259639 \n",
       "L 204.629723 106.259639 \n",
       "L 203.509241 106.259639 \n",
       "L 202.388759 106.259639 \n",
       "L 201.268277 106.259639 \n",
       "L 200.147796 106.259639 \n",
       "L 199.027314 106.259639 \n",
       "L 197.906832 106.259639 \n",
       "L 196.78635 106.259639 \n",
       "L 195.665868 106.259639 \n",
       "L 194.545386 106.259639 \n",
       "L 193.424904 106.259639 \n",
       "L 192.304422 106.259639 \n",
       "L 191.18394 106.259639 \n",
       "L 190.063458 106.259639 \n",
       "L 188.942976 106.259639 \n",
       "L 187.822494 106.259639 \n",
       "L 186.702012 106.259639 \n",
       "L 185.58153 106.259639 \n",
       "L 184.461049 106.259639 \n",
       "L 183.340567 106.259639 \n",
       "L 182.220085 106.259639 \n",
       "L 181.099603 106.259639 \n",
       "L 179.979121 106.259639 \n",
       "L 178.858639 106.259639 \n",
       "L 177.738157 106.259639 \n",
       "L 176.617675 106.259639 \n",
       "L 175.497193 106.259639 \n",
       "L 174.376711 106.259639 \n",
       "L 173.256229 106.259639 \n",
       "L 172.135747 106.259639 \n",
       "L 171.015265 106.259639 \n",
       "L 169.894784 106.259639 \n",
       "L 168.774302 106.259639 \n",
       "L 167.65382 106.259639 \n",
       "L 166.533338 106.259639 \n",
       "L 165.412856 106.259639 \n",
       "L 164.292374 106.259639 \n",
       "L 163.171892 106.259639 \n",
       "L 162.05141 106.259639 \n",
       "L 160.930928 106.259639 \n",
       "L 159.810446 106.259639 \n",
       "L 158.689964 106.259639 \n",
       "L 157.569482 106.259639 \n",
       "L 156.449 106.259639 \n",
       "L 155.328518 106.259639 \n",
       "L 154.208037 106.259639 \n",
       "L 153.087555 106.259639 \n",
       "L 151.967073 106.259639 \n",
       "L 150.846591 106.259639 \n",
       "L 149.726109 106.259639 \n",
       "L 148.605627 106.259639 \n",
       "L 147.485145 106.259639 \n",
       "L 146.364663 106.259639 \n",
       "L 145.244181 106.259639 \n",
       "L 144.123699 106.259639 \n",
       "L 143.003217 106.259639 \n",
       "L 141.882735 106.259639 \n",
       "L 140.762253 106.259639 \n",
       "L 139.641771 106.259639 \n",
       "L 138.52129 106.259639 \n",
       "L 137.400808 106.259639 \n",
       "L 136.280326 106.259639 \n",
       "L 135.159844 106.259639 \n",
       "L 134.039362 106.259639 \n",
       "L 132.91888 106.259639 \n",
       "L 131.798398 106.259639 \n",
       "L 130.677916 106.259639 \n",
       "L 129.557434 106.259639 \n",
       "L 128.436952 106.259639 \n",
       "L 127.31647 106.259639 \n",
       "L 126.195988 106.259639 \n",
       "L 125.075506 106.259639 \n",
       "L 123.955024 106.259639 \n",
       "L 122.834543 106.259639 \n",
       "L 121.714061 106.259639 \n",
       "L 120.593579 106.259639 \n",
       "L 119.473097 106.259639 \n",
       "L 118.352615 106.259639 \n",
       "L 117.232133 106.259639 \n",
       "L 116.111651 106.259639 \n",
       "L 114.991169 106.259639 \n",
       "L 113.870687 106.259639 \n",
       "L 112.750205 106.259639 \n",
       "L 111.629723 106.259639 \n",
       "L 110.509241 106.259639 \n",
       "L 109.388759 106.259639 \n",
       "L 108.268277 106.259639 \n",
       "L 107.147796 106.259639 \n",
       "L 106.027314 106.259639 \n",
       "L 104.906832 106.259639 \n",
       "L 103.78635 106.259639 \n",
       "L 102.665868 106.259639 \n",
       "L 101.545386 106.259639 \n",
       "L 100.424904 106.259639 \n",
       "L 99.304422 106.259639 \n",
       "L 98.18394 106.259639 \n",
       "L 97.063458 106.259639 \n",
       "L 95.942976 106.259639 \n",
       "L 94.822494 106.259639 \n",
       "L 93.702012 106.259639 \n",
       "L 92.58153 106.259639 \n",
       "L 91.461049 106.259639 \n",
       "L 90.340567 106.259639 \n",
       "L 89.220085 106.259639 \n",
       "L 88.099603 106.259639 \n",
       "L 86.979121 106.259639 \n",
       "L 85.858639 106.259639 \n",
       "L 84.738157 106.259639 \n",
       "L 83.617675 106.259639 \n",
       "L 82.497193 106.259639 \n",
       "L 81.376711 106.259639 \n",
       "L 80.256229 106.259639 \n",
       "L 79.135747 106.259639 \n",
       "L 78.015265 106.259639 \n",
       "L 76.894784 106.259639 \n",
       "L 75.774302 106.259639 \n",
       "L 74.65382 106.259639 \n",
       "L 73.533338 106.259639 \n",
       "L 72.412856 106.259639 \n",
       "L 71.292374 106.259639 \n",
       "L 70.171892 106.259639 \n",
       "L 69.05141 106.259639 \n",
       "L 67.930928 106.259639 \n",
       "L 66.810446 106.259639 \n",
       "L 65.689964 106.259639 \n",
       "L 64.569482 106.259639 \n",
       "L 63.449 106.259639 \n",
       "L 62.328518 106.259639 \n",
       "L 61.208037 106.259639 \n",
       "L 60.087555 106.259639 \n",
       "L 58.967073 106.259639 \n",
       "L 57.846591 106.259639 \n",
       "L 56.726109 106.259639 \n",
       "L 55.605627 106.259639 \n",
       "L 54.485145 106.259639 \n",
       "L 53.364663 106.259639 \n",
       "L 52.244181 106.259639 \n",
       "L 51.123699 106.259639 \n",
       "L 50.003217 106.259639 \n",
       "L 48.882735 106.259639 \n",
       "L 47.762253 106.259639 \n",
       "L 46.641771 106.259639 \n",
       "L 45.52129 106.259639 \n",
       "L 44.400808 106.259639 \n",
       "L 43.280326 106.259639 \n",
       "L 42.159844 106.259639 \n",
       "L 42.159844 106.53253 \n",
       "L 43.280326 106.53253 \n",
       "L 44.400808 106.53253 \n",
       "L 45.52129 106.53253 \n",
       "L 46.641771 106.53253 \n",
       "L 47.762253 106.53253 \n",
       "L 48.882735 106.53253 \n",
       "L 50.003217 106.53253 \n",
       "L 51.123699 106.53253 \n",
       "L 52.244181 106.53253 \n",
       "L 53.364663 106.53253 \n",
       "L 54.485145 106.53253 \n",
       "L 55.605627 106.53253 \n",
       "L 56.726109 106.53253 \n",
       "L 57.846591 106.53253 \n",
       "L 58.967073 106.53253 \n",
       "L 60.087555 106.53253 \n",
       "L 61.208037 106.53253 \n",
       "L 62.328518 106.53253 \n",
       "L 63.449 106.53253 \n",
       "L 64.569482 106.53253 \n",
       "L 65.689964 106.53253 \n",
       "L 66.810446 106.53253 \n",
       "L 67.930928 106.53253 \n",
       "L 69.05141 106.53253 \n",
       "L 70.171892 106.53253 \n",
       "L 71.292374 106.53253 \n",
       "L 72.412856 106.53253 \n",
       "L 73.533338 106.53253 \n",
       "L 74.65382 106.53253 \n",
       "L 75.774302 106.53253 \n",
       "L 76.894784 106.53253 \n",
       "L 78.015265 106.53253 \n",
       "L 79.135747 106.53253 \n",
       "L 80.256229 106.53253 \n",
       "L 81.376711 106.53253 \n",
       "L 82.497193 106.53253 \n",
       "L 83.617675 106.53253 \n",
       "L 84.738157 106.53253 \n",
       "L 85.858639 106.53253 \n",
       "L 86.979121 106.53253 \n",
       "L 88.099603 106.53253 \n",
       "L 89.220085 106.53253 \n",
       "L 90.340567 106.53253 \n",
       "L 91.461049 106.53253 \n",
       "L 92.58153 106.53253 \n",
       "L 93.702012 106.53253 \n",
       "L 94.822494 106.53253 \n",
       "L 95.942976 106.53253 \n",
       "L 97.063458 106.53253 \n",
       "L 98.18394 106.53253 \n",
       "L 99.304422 106.53253 \n",
       "L 100.424904 106.53253 \n",
       "L 101.545386 106.53253 \n",
       "L 102.665868 106.53253 \n",
       "L 103.78635 106.53253 \n",
       "L 104.906832 106.53253 \n",
       "L 106.027314 106.53253 \n",
       "L 107.147796 106.53253 \n",
       "L 108.268277 106.53253 \n",
       "L 109.388759 106.53253 \n",
       "L 110.509241 106.53253 \n",
       "L 111.629723 106.53253 \n",
       "L 112.750205 106.53253 \n",
       "L 113.870687 106.53253 \n",
       "L 114.991169 106.53253 \n",
       "L 116.111651 106.53253 \n",
       "L 117.232133 106.53253 \n",
       "L 118.352615 106.53253 \n",
       "L 119.473097 106.53253 \n",
       "L 120.593579 106.53253 \n",
       "L 121.714061 106.53253 \n",
       "L 122.834543 106.53253 \n",
       "L 123.955024 106.53253 \n",
       "L 125.075506 106.53253 \n",
       "L 126.195988 106.53253 \n",
       "L 127.31647 106.53253 \n",
       "L 128.436952 106.53253 \n",
       "L 129.557434 106.53253 \n",
       "L 130.677916 106.53253 \n",
       "L 131.798398 106.53253 \n",
       "L 132.91888 106.53253 \n",
       "L 134.039362 106.53253 \n",
       "L 135.159844 106.53253 \n",
       "L 136.280326 106.53253 \n",
       "L 137.400808 106.53253 \n",
       "L 138.52129 106.53253 \n",
       "L 139.641771 106.53253 \n",
       "L 140.762253 106.53253 \n",
       "L 141.882735 106.53253 \n",
       "L 143.003217 106.53253 \n",
       "L 144.123699 106.53253 \n",
       "L 145.244181 106.53253 \n",
       "L 146.364663 106.53253 \n",
       "L 147.485145 106.53253 \n",
       "L 148.605627 106.53253 \n",
       "L 149.726109 106.53253 \n",
       "L 150.846591 106.53253 \n",
       "L 151.967073 106.53253 \n",
       "L 153.087555 106.53253 \n",
       "L 154.208037 106.53253 \n",
       "L 155.328518 106.53253 \n",
       "L 156.449 106.53253 \n",
       "L 157.569482 106.53253 \n",
       "L 158.689964 106.53253 \n",
       "L 159.810446 106.53253 \n",
       "L 160.930928 106.53253 \n",
       "L 162.05141 106.53253 \n",
       "L 163.171892 106.53253 \n",
       "L 164.292374 106.53253 \n",
       "L 165.412856 106.53253 \n",
       "L 166.533338 106.53253 \n",
       "L 167.65382 106.53253 \n",
       "L 168.774302 106.53253 \n",
       "L 169.894784 106.53253 \n",
       "L 171.015265 106.53253 \n",
       "L 172.135747 106.53253 \n",
       "L 173.256229 106.53253 \n",
       "L 174.376711 106.53253 \n",
       "L 175.497193 106.53253 \n",
       "L 176.617675 106.53253 \n",
       "L 177.738157 106.53253 \n",
       "L 178.858639 106.53253 \n",
       "L 179.979121 106.53253 \n",
       "L 181.099603 106.53253 \n",
       "L 182.220085 106.53253 \n",
       "L 183.340567 106.53253 \n",
       "L 184.461049 106.53253 \n",
       "L 185.58153 106.53253 \n",
       "L 186.702012 106.53253 \n",
       "L 187.822494 106.53253 \n",
       "L 188.942976 106.53253 \n",
       "L 190.063458 106.53253 \n",
       "L 191.18394 106.53253 \n",
       "L 192.304422 106.53253 \n",
       "L 193.424904 106.53253 \n",
       "L 194.545386 106.53253 \n",
       "L 195.665868 106.53253 \n",
       "L 196.78635 106.53253 \n",
       "L 197.906832 106.53253 \n",
       "L 199.027314 106.53253 \n",
       "L 200.147796 106.53253 \n",
       "L 201.268277 106.53253 \n",
       "L 202.388759 106.53253 \n",
       "L 203.509241 106.53253 \n",
       "L 204.629723 106.53253 \n",
       "L 205.750205 106.53253 \n",
       "L 206.870687 106.53253 \n",
       "L 207.991169 106.53253 \n",
       "L 209.111651 106.53253 \n",
       "L 210.232133 106.53253 \n",
       "L 211.352615 106.53253 \n",
       "L 212.473097 106.53253 \n",
       "L 213.593579 106.53253 \n",
       "L 214.714061 106.53253 \n",
       "L 215.834543 106.53253 \n",
       "L 216.955024 106.53253 \n",
       "L 218.075506 106.53253 \n",
       "L 219.195988 106.53253 \n",
       "L 220.31647 106.53253 \n",
       "L 221.436952 106.53253 \n",
       "L 222.557434 106.53253 \n",
       "L 223.677916 106.53253 \n",
       "L 224.798398 106.53253 \n",
       "L 225.91888 105.440964 \n",
       "L 225.91888 104.349398 \n",
       "L 225.91888 103.257831 \n",
       "L 225.91888 102.166265 \n",
       "L 225.91888 101.074699 \n",
       "L 225.91888 99.983133 \n",
       "L 225.91888 98.891566 \n",
       "L 225.91888 97.8 \n",
       "L 225.91888 96.708434 \n",
       "L 227.039362 96.708434 \n",
       "L 228.159844 96.708434 \n",
       "L 229.280326 96.708434 \n",
       "L 230.400808 96.708434 \n",
       "L 231.52129 96.708434 \n",
       "L 232.641771 96.708434 \n",
       "L 233.762253 96.708434 \n",
       "L 234.882735 96.708434 \n",
       "L 236.003217 96.708434 \n",
       "L 237.123699 96.708434 \n",
       "L 238.244181 96.708434 \n",
       "L 239.364663 96.708434 \n",
       "L 240.485145 96.708434 \n",
       "L 241.605627 96.708434 \n",
       "L 242.726109 96.708434 \n",
       "L 243.846591 96.708434 \n",
       "L 244.967073 96.708434 \n",
       "L 246.087555 96.708434 \n",
       "L 247.208037 96.708434 \n",
       "L 247.208037 97.8 \n",
       "L 247.208037 98.891566 \n",
       "L 247.208037 99.983133 \n",
       "L 247.208037 101.074699 \n",
       "L 247.208037 102.166265 \n",
       "L 247.208037 103.257831 \n",
       "L 247.208037 104.349398 \n",
       "L 247.208037 105.440964 \n",
       "L 247.208037 106.53253 \n",
       "L 247.208037 107.624096 \n",
       "L 247.208037 108.715663 \n",
       "L 247.208037 109.807229 \n",
       "L 247.208037 110.898795 \n",
       "L 247.208037 111.990361 \n",
       "L 247.208037 113.081928 \n",
       "L 247.208037 114.173494 \n",
       "L 247.208037 115.26506 \n",
       "L 247.208037 116.356627 \n",
       "L 246.087555 116.356627 \n",
       "L 244.967073 116.356627 \n",
       "L 243.846591 116.356627 \n",
       "L 242.726109 116.356627 \n",
       "L 241.605627 116.356627 \n",
       "L 240.485145 116.356627 \n",
       "L 239.364663 116.356627 \n",
       "L 238.244181 116.356627 \n",
       "L 237.123699 116.356627 \n",
       "L 236.003217 116.356627 \n",
       "L 234.882735 116.356627 \n",
       "L 233.762253 116.356627 \n",
       "L 232.641771 116.356627 \n",
       "L 231.52129 116.356627 \n",
       "L 230.400808 116.356627 \n",
       "L 229.280326 116.356627 \n",
       "L 228.159844 116.356627 \n",
       "L 227.039362 116.356627 \n",
       "L 225.91888 116.356627 \n",
       "L 224.798398 117.448193 \n",
       "L 224.798398 118.539759 \n",
       "L 224.798398 119.631325 \n",
       "L 224.798398 120.722892 \n",
       "L 224.798398 121.814458 \n",
       "L 224.798398 122.906024 \n",
       "L 224.798398 123.99759 \n",
       "L 224.798398 125.089157 \n",
       "L 224.798398 126.180723 \n",
       "L 224.798398 127.272289 \n",
       "L 224.798398 128.363855 \n",
       "L 224.798398 129.455422 \n",
       "L 224.798398 130.546988 \n",
       "L 224.798398 131.638554 \n",
       "L 224.798398 132.73012 \n",
       "L 224.798398 133.821687 \n",
       "L 224.798398 134.913253 \n",
       "L 224.798398 136.004819 \n",
       "L 224.798398 137.096386 \n",
       "L 224.798398 138.187952 \n",
       "L 224.798398 139.279518 \n",
       "L 224.798398 140.371084 \n",
       "L 224.798398 141.462651 \n",
       "L 224.798398 142.554217 \n",
       "L 224.798398 143.645783 \n",
       "L 224.798398 144.737349 \n",
       "L 224.798398 145.828916 \n",
       "L 224.798398 146.920482 \n",
       "L 224.798398 148.012048 \n",
       "L 224.798398 149.103614 \n",
       "L 224.798398 150.195181 \n",
       "L 224.798398 151.286747 \n",
       "L 224.798398 152.378313 \n",
       "L 224.798398 153.46988 \n",
       "L 224.798398 154.561446 \n",
       "L 224.798398 155.653012 \n",
       "L 224.798398 156.744578 \n",
       "L 224.798398 157.836145 \n",
       "L 224.798398 158.927711 \n",
       "L 224.798398 160.019277 \n",
       "L 224.798398 161.110843 \n",
       "L 224.798398 162.20241 \n",
       "L 224.798398 163.293976 \n",
       "L 224.798398 164.385542 \n",
       "L 224.798398 165.477108 \n",
       "L 224.798398 166.568675 \n",
       "L 224.798398 167.660241 \n",
       "L 224.798398 168.751807 \n",
       "L 224.798398 169.843373 \n",
       "L 224.798398 170.93494 \n",
       "L 224.798398 172.026506 \n",
       "L 224.798398 173.118072 \n",
       "L 224.798398 174.209639 \n",
       "L 224.798398 175.301205 \n",
       "L 224.798398 176.392771 \n",
       "L 224.798398 177.484337 \n",
       "L 224.798398 178.575904 \n",
       "L 224.798398 179.66747 \n",
       "L 224.798398 180.759036 \n",
       "L 224.798398 181.850602 \n",
       "L 224.798398 182.942169 \n",
       "L 224.798398 184.033735 \n",
       "L 224.798398 185.125301 \n",
       "L 224.798398 186.216867 \n",
       "L 224.798398 187.308434 \n",
       "L 224.798398 188.4 \n",
       "L 224.798398 189.491566 \n",
       "L 224.798398 190.583133 \n",
       "L 224.798398 191.674699 \n",
       "L 224.798398 192.766265 \n",
       "L 224.798398 193.857831 \n",
       "L 224.798398 194.949398 \n",
       "z\n",
       "\" style=\"fill:#8c564b;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_6\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 225.91888 195.358735 \n",
       "L 227.039362 195.358735 \n",
       "L 228.159844 195.358735 \n",
       "L 229.280326 195.358735 \n",
       "L 230.400808 195.358735 \n",
       "L 231.52129 195.358735 \n",
       "L 232.641771 195.358735 \n",
       "L 233.762253 195.358735 \n",
       "L 234.882735 195.358735 \n",
       "L 236.003217 195.358735 \n",
       "L 237.123699 195.358735 \n",
       "L 238.244181 195.358735 \n",
       "L 239.364663 195.358735 \n",
       "L 240.485145 195.358735 \n",
       "L 241.605627 195.358735 \n",
       "L 242.726109 195.358735 \n",
       "L 243.846591 195.358735 \n",
       "L 244.967073 195.358735 \n",
       "L 246.087555 195.358735 \n",
       "L 247.208037 195.358735 \n",
       "L 248.328518 195.358735 \n",
       "L 249.449 195.358735 \n",
       "L 250.569482 195.358735 \n",
       "L 251.689964 195.358735 \n",
       "L 252.810446 195.358735 \n",
       "L 253.930928 195.358735 \n",
       "L 255.05141 195.358735 \n",
       "L 256.171892 195.358735 \n",
       "L 257.292374 195.358735 \n",
       "L 258.412856 195.358735 \n",
       "L 259.533338 195.358735 \n",
       "L 260.65382 195.358735 \n",
       "L 261.774302 195.358735 \n",
       "L 262.894784 195.358735 \n",
       "L 264.015265 195.358735 \n",
       "L 265.135747 195.358735 \n",
       "L 266.256229 195.358735 \n",
       "L 267.376711 195.358735 \n",
       "L 268.497193 195.358735 \n",
       "L 269.617675 195.358735 \n",
       "L 270.738157 195.358735 \n",
       "L 271.858639 195.358735 \n",
       "L 272.979121 195.358735 \n",
       "L 274.099603 195.358735 \n",
       "L 275.220085 195.358735 \n",
       "L 276.340567 195.358735 \n",
       "L 277.461049 195.358735 \n",
       "L 278.58153 195.358735 \n",
       "L 279.702012 195.358735 \n",
       "L 280.822494 195.358735 \n",
       "L 281.942976 195.358735 \n",
       "L 283.063458 195.358735 \n",
       "L 284.18394 195.358735 \n",
       "L 285.304422 195.358735 \n",
       "L 286.424904 195.358735 \n",
       "L 287.545386 195.358735 \n",
       "L 288.665868 195.358735 \n",
       "L 289.78635 195.358735 \n",
       "L 290.906832 195.358735 \n",
       "L 292.027314 195.358735 \n",
       "L 293.147796 195.358735 \n",
       "L 294.268277 195.358735 \n",
       "L 295.388759 195.358735 \n",
       "L 296.509241 195.358735 \n",
       "L 297.629723 195.358735 \n",
       "L 298.750205 195.358735 \n",
       "L 299.870687 195.358735 \n",
       "L 300.991169 195.358735 \n",
       "L 302.111651 195.358735 \n",
       "L 303.232133 195.358735 \n",
       "L 304.352615 195.358735 \n",
       "L 305.473097 195.358735 \n",
       "L 306.593579 195.358735 \n",
       "L 307.714061 195.358735 \n",
       "L 308.834543 195.358735 \n",
       "L 309.955024 195.358735 \n",
       "L 311.075506 195.358735 \n",
       "L 312.195988 195.358735 \n",
       "L 313.31647 195.358735 \n",
       "L 314.436952 195.358735 \n",
       "L 315.557434 195.358735 \n",
       "L 316.677916 195.358735 \n",
       "L 317.798398 195.358735 \n",
       "L 318.91888 195.358735 \n",
       "L 320.039362 195.358735 \n",
       "L 321.159844 195.358735 \n",
       "L 321.159844 195.222289 \n",
       "L 320.039362 195.222289 \n",
       "L 318.91888 195.222289 \n",
       "L 317.798398 195.222289 \n",
       "L 316.677916 195.222289 \n",
       "L 315.557434 195.222289 \n",
       "L 314.436952 195.222289 \n",
       "L 313.31647 195.222289 \n",
       "L 312.195988 195.222289 \n",
       "L 311.075506 195.222289 \n",
       "L 309.955024 195.222289 \n",
       "L 308.834543 195.222289 \n",
       "L 307.714061 195.222289 \n",
       "L 306.593579 195.222289 \n",
       "L 305.473097 195.222289 \n",
       "L 304.352615 195.222289 \n",
       "L 303.232133 195.222289 \n",
       "L 302.111651 195.222289 \n",
       "L 300.991169 195.222289 \n",
       "L 299.870687 195.222289 \n",
       "L 298.750205 195.222289 \n",
       "L 297.629723 195.222289 \n",
       "L 296.509241 195.222289 \n",
       "L 295.388759 195.222289 \n",
       "L 294.268277 195.222289 \n",
       "L 293.147796 195.222289 \n",
       "L 292.027314 195.222289 \n",
       "L 290.906832 195.222289 \n",
       "L 289.78635 195.222289 \n",
       "L 288.665868 195.222289 \n",
       "L 287.545386 195.222289 \n",
       "L 286.424904 195.222289 \n",
       "L 285.304422 195.222289 \n",
       "L 284.18394 195.222289 \n",
       "L 283.063458 195.222289 \n",
       "L 281.942976 195.222289 \n",
       "L 280.822494 195.222289 \n",
       "L 279.702012 195.222289 \n",
       "L 278.58153 195.222289 \n",
       "L 277.461049 195.222289 \n",
       "L 276.340567 195.222289 \n",
       "L 275.220085 195.222289 \n",
       "L 274.099603 195.222289 \n",
       "L 272.979121 195.222289 \n",
       "L 271.858639 195.222289 \n",
       "L 270.738157 195.222289 \n",
       "L 269.617675 195.222289 \n",
       "L 268.497193 195.222289 \n",
       "L 267.376711 195.222289 \n",
       "L 266.256229 195.222289 \n",
       "L 265.135747 195.222289 \n",
       "L 264.015265 195.222289 \n",
       "L 262.894784 195.222289 \n",
       "L 261.774302 195.222289 \n",
       "L 260.65382 195.222289 \n",
       "L 259.533338 195.222289 \n",
       "L 258.412856 195.222289 \n",
       "L 257.292374 195.222289 \n",
       "L 256.171892 195.222289 \n",
       "L 255.05141 195.222289 \n",
       "L 253.930928 195.222289 \n",
       "L 252.810446 195.222289 \n",
       "L 251.689964 195.222289 \n",
       "L 250.569482 195.222289 \n",
       "L 249.449 195.222289 \n",
       "L 248.328518 195.222289 \n",
       "L 247.208037 195.222289 \n",
       "L 246.087555 195.222289 \n",
       "L 244.967073 195.222289 \n",
       "L 243.846591 195.222289 \n",
       "L 242.726109 195.222289 \n",
       "L 241.605627 195.222289 \n",
       "L 240.485145 195.222289 \n",
       "L 239.364663 195.222289 \n",
       "L 238.244181 195.222289 \n",
       "L 237.123699 195.222289 \n",
       "L 236.003217 195.222289 \n",
       "L 234.882735 195.222289 \n",
       "L 233.762253 195.222289 \n",
       "L 232.641771 195.222289 \n",
       "L 231.52129 195.222289 \n",
       "L 230.400808 195.222289 \n",
       "L 229.280326 195.222289 \n",
       "L 228.159844 195.222289 \n",
       "L 227.039362 195.222289 \n",
       "L 225.91888 195.222289 \n",
       "L 225.358639 194.949398 \n",
       "L 225.358639 193.857831 \n",
       "L 225.358639 192.766265 \n",
       "L 225.358639 191.674699 \n",
       "L 225.358639 190.583133 \n",
       "L 225.358639 189.491566 \n",
       "L 225.358639 188.4 \n",
       "L 225.358639 187.308434 \n",
       "L 225.358639 186.216867 \n",
       "L 225.358639 185.125301 \n",
       "L 225.358639 184.033735 \n",
       "L 225.358639 182.942169 \n",
       "L 225.358639 181.850602 \n",
       "L 225.358639 180.759036 \n",
       "L 225.358639 179.66747 \n",
       "L 225.358639 178.575904 \n",
       "L 225.358639 177.484337 \n",
       "L 225.358639 176.392771 \n",
       "L 225.358639 175.301205 \n",
       "L 225.358639 174.209639 \n",
       "L 225.358639 173.118072 \n",
       "L 225.358639 172.026506 \n",
       "L 225.358639 170.93494 \n",
       "L 225.358639 169.843373 \n",
       "L 225.358639 168.751807 \n",
       "L 225.358639 167.660241 \n",
       "L 225.358639 166.568675 \n",
       "L 225.358639 165.477108 \n",
       "L 225.358639 164.385542 \n",
       "L 225.358639 163.293976 \n",
       "L 225.358639 162.20241 \n",
       "L 225.358639 161.110843 \n",
       "L 225.358639 160.019277 \n",
       "L 225.358639 158.927711 \n",
       "L 225.358639 157.836145 \n",
       "L 225.358639 156.744578 \n",
       "L 225.358639 155.653012 \n",
       "L 225.358639 154.561446 \n",
       "L 225.358639 153.46988 \n",
       "L 225.358639 152.378313 \n",
       "L 225.358639 151.286747 \n",
       "L 225.358639 150.195181 \n",
       "L 225.358639 149.103614 \n",
       "L 225.358639 148.012048 \n",
       "L 225.358639 146.920482 \n",
       "L 225.358639 145.828916 \n",
       "L 225.358639 144.737349 \n",
       "L 225.358639 143.645783 \n",
       "L 225.358639 142.554217 \n",
       "L 225.358639 141.462651 \n",
       "L 225.358639 140.371084 \n",
       "L 225.358639 139.279518 \n",
       "L 225.358639 138.187952 \n",
       "L 225.358639 137.096386 \n",
       "L 225.358639 136.004819 \n",
       "L 225.358639 134.913253 \n",
       "L 225.358639 133.821687 \n",
       "L 225.358639 132.73012 \n",
       "L 225.358639 131.638554 \n",
       "L 225.358639 130.546988 \n",
       "L 225.358639 129.455422 \n",
       "L 225.358639 128.363855 \n",
       "L 225.358639 127.272289 \n",
       "L 225.358639 126.180723 \n",
       "L 225.358639 125.089157 \n",
       "L 225.358639 123.99759 \n",
       "L 225.358639 122.906024 \n",
       "L 225.358639 121.814458 \n",
       "L 225.358639 120.722892 \n",
       "L 225.358639 119.631325 \n",
       "L 225.358639 118.539759 \n",
       "L 225.358639 117.448193 \n",
       "L 225.91888 116.90241 \n",
       "L 227.039362 116.90241 \n",
       "L 228.159844 116.90241 \n",
       "L 229.280326 116.90241 \n",
       "L 230.400808 116.90241 \n",
       "L 231.52129 116.90241 \n",
       "L 232.641771 116.90241 \n",
       "L 233.762253 116.90241 \n",
       "L 234.882735 116.90241 \n",
       "L 236.003217 116.90241 \n",
       "L 237.123699 116.90241 \n",
       "L 238.244181 116.90241 \n",
       "L 239.364663 116.90241 \n",
       "L 240.485145 116.90241 \n",
       "L 241.605627 116.90241 \n",
       "L 242.726109 116.90241 \n",
       "L 243.846591 116.90241 \n",
       "L 244.967073 116.90241 \n",
       "L 246.087555 116.90241 \n",
       "L 247.208037 116.90241 \n",
       "L 247.768277 116.356627 \n",
       "L 247.768277 115.26506 \n",
       "L 247.768277 114.173494 \n",
       "L 247.768277 113.081928 \n",
       "L 247.768277 111.990361 \n",
       "L 247.768277 110.898795 \n",
       "L 247.768277 109.807229 \n",
       "L 247.768277 108.715663 \n",
       "L 247.768277 107.624096 \n",
       "L 247.768277 106.53253 \n",
       "L 247.768277 105.440964 \n",
       "L 247.768277 104.349398 \n",
       "L 247.768277 103.257831 \n",
       "L 247.768277 102.166265 \n",
       "L 247.768277 101.074699 \n",
       "L 247.768277 99.983133 \n",
       "L 247.768277 98.891566 \n",
       "L 247.768277 97.8 \n",
       "L 247.768277 96.708434 \n",
       "L 247.208037 96.162651 \n",
       "L 246.087555 96.162651 \n",
       "L 244.967073 96.162651 \n",
       "L 243.846591 96.162651 \n",
       "L 242.726109 96.162651 \n",
       "L 241.605627 96.162651 \n",
       "L 240.485145 96.162651 \n",
       "L 239.364663 96.162651 \n",
       "L 238.244181 96.162651 \n",
       "L 237.123699 96.162651 \n",
       "L 236.003217 96.162651 \n",
       "L 234.882735 96.162651 \n",
       "L 233.762253 96.162651 \n",
       "L 232.641771 96.162651 \n",
       "L 231.52129 96.162651 \n",
       "L 230.400808 96.162651 \n",
       "L 229.280326 96.162651 \n",
       "L 228.159844 96.162651 \n",
       "L 227.039362 96.162651 \n",
       "L 225.91888 96.162651 \n",
       "L 225.358639 96.708434 \n",
       "L 225.358639 97.8 \n",
       "L 225.358639 98.891566 \n",
       "L 225.358639 99.983133 \n",
       "L 225.358639 101.074699 \n",
       "L 225.358639 102.166265 \n",
       "L 225.358639 103.257831 \n",
       "L 225.358639 104.349398 \n",
       "L 225.358639 105.440964 \n",
       "L 224.798398 105.986747 \n",
       "L 223.677916 105.986747 \n",
       "L 222.557434 105.986747 \n",
       "L 221.436952 105.986747 \n",
       "L 220.31647 105.986747 \n",
       "L 219.195988 105.986747 \n",
       "L 218.075506 105.986747 \n",
       "L 216.955024 105.986747 \n",
       "L 215.834543 105.986747 \n",
       "L 214.714061 105.986747 \n",
       "L 213.593579 105.986747 \n",
       "L 212.473097 105.986747 \n",
       "L 211.352615 105.986747 \n",
       "L 210.232133 105.986747 \n",
       "L 209.111651 105.986747 \n",
       "L 207.991169 105.986747 \n",
       "L 206.870687 105.986747 \n",
       "L 205.750205 105.986747 \n",
       "L 204.629723 105.986747 \n",
       "L 203.509241 105.986747 \n",
       "L 202.388759 105.986747 \n",
       "L 201.268277 105.986747 \n",
       "L 200.147796 105.986747 \n",
       "L 199.027314 105.986747 \n",
       "L 197.906832 105.986747 \n",
       "L 196.78635 105.986747 \n",
       "L 195.665868 105.986747 \n",
       "L 194.545386 105.986747 \n",
       "L 193.424904 105.986747 \n",
       "L 192.304422 105.986747 \n",
       "L 191.18394 105.986747 \n",
       "L 190.063458 105.986747 \n",
       "L 188.942976 105.986747 \n",
       "L 187.822494 105.986747 \n",
       "L 186.702012 105.986747 \n",
       "L 185.58153 105.986747 \n",
       "L 184.461049 105.986747 \n",
       "L 183.340567 105.986747 \n",
       "L 182.220085 105.986747 \n",
       "L 181.099603 105.986747 \n",
       "L 179.979121 105.986747 \n",
       "L 178.858639 105.986747 \n",
       "L 177.738157 105.986747 \n",
       "L 176.617675 105.986747 \n",
       "L 175.497193 105.986747 \n",
       "L 174.376711 105.986747 \n",
       "L 173.256229 105.986747 \n",
       "L 172.135747 105.986747 \n",
       "L 171.015265 105.986747 \n",
       "L 169.894784 105.986747 \n",
       "L 168.774302 105.986747 \n",
       "L 167.65382 105.986747 \n",
       "L 166.533338 105.986747 \n",
       "L 165.412856 105.986747 \n",
       "L 164.292374 105.986747 \n",
       "L 163.171892 105.986747 \n",
       "L 162.05141 105.986747 \n",
       "L 160.930928 105.986747 \n",
       "L 159.810446 105.986747 \n",
       "L 158.689964 105.986747 \n",
       "L 157.569482 105.986747 \n",
       "L 156.449 105.986747 \n",
       "L 155.328518 105.986747 \n",
       "L 154.208037 105.986747 \n",
       "L 153.087555 105.986747 \n",
       "L 151.967073 105.986747 \n",
       "L 150.846591 105.986747 \n",
       "L 149.726109 105.986747 \n",
       "L 148.605627 105.986747 \n",
       "L 147.485145 105.986747 \n",
       "L 146.364663 105.986747 \n",
       "L 145.244181 105.986747 \n",
       "L 144.123699 105.986747 \n",
       "L 143.003217 105.986747 \n",
       "L 141.882735 105.986747 \n",
       "L 140.762253 105.986747 \n",
       "L 139.641771 105.986747 \n",
       "L 138.52129 105.986747 \n",
       "L 137.400808 105.986747 \n",
       "L 136.280326 105.986747 \n",
       "L 135.159844 105.986747 \n",
       "L 134.039362 105.986747 \n",
       "L 132.91888 105.986747 \n",
       "L 131.798398 105.986747 \n",
       "L 130.677916 105.986747 \n",
       "L 129.557434 105.986747 \n",
       "L 128.436952 105.986747 \n",
       "L 127.31647 105.986747 \n",
       "L 126.195988 105.986747 \n",
       "L 125.075506 105.986747 \n",
       "L 123.955024 105.986747 \n",
       "L 122.834543 105.986747 \n",
       "L 121.714061 105.986747 \n",
       "L 120.593579 105.986747 \n",
       "L 119.473097 105.986747 \n",
       "L 118.352615 105.986747 \n",
       "L 117.232133 105.986747 \n",
       "L 116.111651 105.986747 \n",
       "L 114.991169 105.986747 \n",
       "L 113.870687 105.986747 \n",
       "L 112.750205 105.986747 \n",
       "L 111.629723 105.986747 \n",
       "L 110.509241 105.986747 \n",
       "L 109.388759 105.986747 \n",
       "L 108.268277 105.986747 \n",
       "L 107.147796 105.986747 \n",
       "L 106.027314 105.986747 \n",
       "L 104.906832 105.986747 \n",
       "L 103.78635 105.986747 \n",
       "L 102.665868 105.986747 \n",
       "L 101.545386 105.986747 \n",
       "L 100.424904 105.986747 \n",
       "L 99.304422 105.986747 \n",
       "L 98.18394 105.986747 \n",
       "L 97.063458 105.986747 \n",
       "L 95.942976 105.986747 \n",
       "L 94.822494 105.986747 \n",
       "L 93.702012 105.986747 \n",
       "L 92.58153 105.986747 \n",
       "L 91.461049 105.986747 \n",
       "L 90.340567 105.986747 \n",
       "L 89.220085 105.986747 \n",
       "L 88.099603 105.986747 \n",
       "L 86.979121 105.986747 \n",
       "L 85.858639 105.986747 \n",
       "L 84.738157 105.986747 \n",
       "L 83.617675 105.986747 \n",
       "L 82.497193 105.986747 \n",
       "L 81.376711 105.986747 \n",
       "L 80.256229 105.986747 \n",
       "L 79.135747 105.986747 \n",
       "L 78.015265 105.986747 \n",
       "L 76.894784 105.986747 \n",
       "L 75.774302 105.986747 \n",
       "L 74.65382 105.986747 \n",
       "L 73.533338 105.986747 \n",
       "L 72.412856 105.986747 \n",
       "L 71.292374 105.986747 \n",
       "L 70.171892 105.986747 \n",
       "L 69.05141 105.986747 \n",
       "L 67.930928 105.986747 \n",
       "L 66.810446 105.986747 \n",
       "L 65.689964 105.986747 \n",
       "L 64.569482 105.986747 \n",
       "L 63.449 105.986747 \n",
       "L 62.328518 105.986747 \n",
       "L 61.208037 105.986747 \n",
       "L 60.087555 105.986747 \n",
       "L 58.967073 105.986747 \n",
       "L 57.846591 105.986747 \n",
       "L 56.726109 105.986747 \n",
       "L 55.605627 105.986747 \n",
       "L 54.485145 105.986747 \n",
       "L 53.364663 105.986747 \n",
       "L 52.244181 105.986747 \n",
       "L 51.123699 105.986747 \n",
       "L 50.003217 105.986747 \n",
       "L 48.882735 105.986747 \n",
       "L 47.762253 105.986747 \n",
       "L 46.641771 105.986747 \n",
       "L 45.52129 105.986747 \n",
       "L 44.400808 105.986747 \n",
       "L 43.280326 105.986747 \n",
       "L 42.159844 105.986747 \n",
       "L 42.159844 106.259639 \n",
       "L 43.280326 106.259639 \n",
       "L 44.400808 106.259639 \n",
       "L 45.52129 106.259639 \n",
       "L 46.641771 106.259639 \n",
       "L 47.762253 106.259639 \n",
       "L 48.882735 106.259639 \n",
       "L 50.003217 106.259639 \n",
       "L 51.123699 106.259639 \n",
       "L 52.244181 106.259639 \n",
       "L 53.364663 106.259639 \n",
       "L 54.485145 106.259639 \n",
       "L 55.605627 106.259639 \n",
       "L 56.726109 106.259639 \n",
       "L 57.846591 106.259639 \n",
       "L 58.967073 106.259639 \n",
       "L 60.087555 106.259639 \n",
       "L 61.208037 106.259639 \n",
       "L 62.328518 106.259639 \n",
       "L 63.449 106.259639 \n",
       "L 64.569482 106.259639 \n",
       "L 65.689964 106.259639 \n",
       "L 66.810446 106.259639 \n",
       "L 67.930928 106.259639 \n",
       "L 69.05141 106.259639 \n",
       "L 70.171892 106.259639 \n",
       "L 71.292374 106.259639 \n",
       "L 72.412856 106.259639 \n",
       "L 73.533338 106.259639 \n",
       "L 74.65382 106.259639 \n",
       "L 75.774302 106.259639 \n",
       "L 76.894784 106.259639 \n",
       "L 78.015265 106.259639 \n",
       "L 79.135747 106.259639 \n",
       "L 80.256229 106.259639 \n",
       "L 81.376711 106.259639 \n",
       "L 82.497193 106.259639 \n",
       "L 83.617675 106.259639 \n",
       "L 84.738157 106.259639 \n",
       "L 85.858639 106.259639 \n",
       "L 86.979121 106.259639 \n",
       "L 88.099603 106.259639 \n",
       "L 89.220085 106.259639 \n",
       "L 90.340567 106.259639 \n",
       "L 91.461049 106.259639 \n",
       "L 92.58153 106.259639 \n",
       "L 93.702012 106.259639 \n",
       "L 94.822494 106.259639 \n",
       "L 95.942976 106.259639 \n",
       "L 97.063458 106.259639 \n",
       "L 98.18394 106.259639 \n",
       "L 99.304422 106.259639 \n",
       "L 100.424904 106.259639 \n",
       "L 101.545386 106.259639 \n",
       "L 102.665868 106.259639 \n",
       "L 103.78635 106.259639 \n",
       "L 104.906832 106.259639 \n",
       "L 106.027314 106.259639 \n",
       "L 107.147796 106.259639 \n",
       "L 108.268277 106.259639 \n",
       "L 109.388759 106.259639 \n",
       "L 110.509241 106.259639 \n",
       "L 111.629723 106.259639 \n",
       "L 112.750205 106.259639 \n",
       "L 113.870687 106.259639 \n",
       "L 114.991169 106.259639 \n",
       "L 116.111651 106.259639 \n",
       "L 117.232133 106.259639 \n",
       "L 118.352615 106.259639 \n",
       "L 119.473097 106.259639 \n",
       "L 120.593579 106.259639 \n",
       "L 121.714061 106.259639 \n",
       "L 122.834543 106.259639 \n",
       "L 123.955024 106.259639 \n",
       "L 125.075506 106.259639 \n",
       "L 126.195988 106.259639 \n",
       "L 127.31647 106.259639 \n",
       "L 128.436952 106.259639 \n",
       "L 129.557434 106.259639 \n",
       "L 130.677916 106.259639 \n",
       "L 131.798398 106.259639 \n",
       "L 132.91888 106.259639 \n",
       "L 134.039362 106.259639 \n",
       "L 135.159844 106.259639 \n",
       "L 136.280326 106.259639 \n",
       "L 137.400808 106.259639 \n",
       "L 138.52129 106.259639 \n",
       "L 139.641771 106.259639 \n",
       "L 140.762253 106.259639 \n",
       "L 141.882735 106.259639 \n",
       "L 143.003217 106.259639 \n",
       "L 144.123699 106.259639 \n",
       "L 145.244181 106.259639 \n",
       "L 146.364663 106.259639 \n",
       "L 147.485145 106.259639 \n",
       "L 148.605627 106.259639 \n",
       "L 149.726109 106.259639 \n",
       "L 150.846591 106.259639 \n",
       "L 151.967073 106.259639 \n",
       "L 153.087555 106.259639 \n",
       "L 154.208037 106.259639 \n",
       "L 155.328518 106.259639 \n",
       "L 156.449 106.259639 \n",
       "L 157.569482 106.259639 \n",
       "L 158.689964 106.259639 \n",
       "L 159.810446 106.259639 \n",
       "L 160.930928 106.259639 \n",
       "L 162.05141 106.259639 \n",
       "L 163.171892 106.259639 \n",
       "L 164.292374 106.259639 \n",
       "L 165.412856 106.259639 \n",
       "L 166.533338 106.259639 \n",
       "L 167.65382 106.259639 \n",
       "L 168.774302 106.259639 \n",
       "L 169.894784 106.259639 \n",
       "L 171.015265 106.259639 \n",
       "L 172.135747 106.259639 \n",
       "L 173.256229 106.259639 \n",
       "L 174.376711 106.259639 \n",
       "L 175.497193 106.259639 \n",
       "L 176.617675 106.259639 \n",
       "L 177.738157 106.259639 \n",
       "L 178.858639 106.259639 \n",
       "L 179.979121 106.259639 \n",
       "L 181.099603 106.259639 \n",
       "L 182.220085 106.259639 \n",
       "L 183.340567 106.259639 \n",
       "L 184.461049 106.259639 \n",
       "L 185.58153 106.259639 \n",
       "L 186.702012 106.259639 \n",
       "L 187.822494 106.259639 \n",
       "L 188.942976 106.259639 \n",
       "L 190.063458 106.259639 \n",
       "L 191.18394 106.259639 \n",
       "L 192.304422 106.259639 \n",
       "L 193.424904 106.259639 \n",
       "L 194.545386 106.259639 \n",
       "L 195.665868 106.259639 \n",
       "L 196.78635 106.259639 \n",
       "L 197.906832 106.259639 \n",
       "L 199.027314 106.259639 \n",
       "L 200.147796 106.259639 \n",
       "L 201.268277 106.259639 \n",
       "L 202.388759 106.259639 \n",
       "L 203.509241 106.259639 \n",
       "L 204.629723 106.259639 \n",
       "L 205.750205 106.259639 \n",
       "L 206.870687 106.259639 \n",
       "L 207.991169 106.259639 \n",
       "L 209.111651 106.259639 \n",
       "L 210.232133 106.259639 \n",
       "L 211.352615 106.259639 \n",
       "L 212.473097 106.259639 \n",
       "L 213.593579 106.259639 \n",
       "L 214.714061 106.259639 \n",
       "L 215.834543 106.259639 \n",
       "L 216.955024 106.259639 \n",
       "L 218.075506 106.259639 \n",
       "L 219.195988 106.259639 \n",
       "L 220.31647 106.259639 \n",
       "L 221.436952 106.259639 \n",
       "L 222.557434 106.259639 \n",
       "L 223.677916 106.259639 \n",
       "L 224.798398 106.259639 \n",
       "L 225.638759 105.440964 \n",
       "L 225.638759 104.349398 \n",
       "L 225.638759 103.257831 \n",
       "L 225.638759 102.166265 \n",
       "L 225.638759 101.074699 \n",
       "L 225.638759 99.983133 \n",
       "L 225.638759 98.891566 \n",
       "L 225.638759 97.8 \n",
       "L 225.638759 96.708434 \n",
       "L 225.91888 96.435542 \n",
       "L 227.039362 96.435542 \n",
       "L 228.159844 96.435542 \n",
       "L 229.280326 96.435542 \n",
       "L 230.400808 96.435542 \n",
       "L 231.52129 96.435542 \n",
       "L 232.641771 96.435542 \n",
       "L 233.762253 96.435542 \n",
       "L 234.882735 96.435542 \n",
       "L 236.003217 96.435542 \n",
       "L 237.123699 96.435542 \n",
       "L 238.244181 96.435542 \n",
       "L 239.364663 96.435542 \n",
       "L 240.485145 96.435542 \n",
       "L 241.605627 96.435542 \n",
       "L 242.726109 96.435542 \n",
       "L 243.846591 96.435542 \n",
       "L 244.967073 96.435542 \n",
       "L 246.087555 96.435542 \n",
       "L 247.208037 96.435542 \n",
       "L 247.488157 96.708434 \n",
       "L 247.488157 97.8 \n",
       "L 247.488157 98.891566 \n",
       "L 247.488157 99.983133 \n",
       "L 247.488157 101.074699 \n",
       "L 247.488157 102.166265 \n",
       "L 247.488157 103.257831 \n",
       "L 247.488157 104.349398 \n",
       "L 247.488157 105.440964 \n",
       "L 247.488157 106.53253 \n",
       "L 247.488157 107.624096 \n",
       "L 247.488157 108.715663 \n",
       "L 247.488157 109.807229 \n",
       "L 247.488157 110.898795 \n",
       "L 247.488157 111.990361 \n",
       "L 247.488157 113.081928 \n",
       "L 247.488157 114.173494 \n",
       "L 247.488157 115.26506 \n",
       "L 247.488157 116.356627 \n",
       "L 247.208037 116.629518 \n",
       "L 246.087555 116.629518 \n",
       "L 244.967073 116.629518 \n",
       "L 243.846591 116.629518 \n",
       "L 242.726109 116.629518 \n",
       "L 241.605627 116.629518 \n",
       "L 240.485145 116.629518 \n",
       "L 239.364663 116.629518 \n",
       "L 238.244181 116.629518 \n",
       "L 237.123699 116.629518 \n",
       "L 236.003217 116.629518 \n",
       "L 234.882735 116.629518 \n",
       "L 233.762253 116.629518 \n",
       "L 232.641771 116.629518 \n",
       "L 231.52129 116.629518 \n",
       "L 230.400808 116.629518 \n",
       "L 229.280326 116.629518 \n",
       "L 228.159844 116.629518 \n",
       "L 227.039362 116.629518 \n",
       "L 225.91888 116.629518 \n",
       "L 225.078518 117.448193 \n",
       "L 225.078518 118.539759 \n",
       "L 225.078518 119.631325 \n",
       "L 225.078518 120.722892 \n",
       "L 225.078518 121.814458 \n",
       "L 225.078518 122.906024 \n",
       "L 225.078518 123.99759 \n",
       "L 225.078518 125.089157 \n",
       "L 225.078518 126.180723 \n",
       "L 225.078518 127.272289 \n",
       "L 225.078518 128.363855 \n",
       "L 225.078518 129.455422 \n",
       "L 225.078518 130.546988 \n",
       "L 225.078518 131.638554 \n",
       "L 225.078518 132.73012 \n",
       "L 225.078518 133.821687 \n",
       "L 225.078518 134.913253 \n",
       "L 225.078518 136.004819 \n",
       "L 225.078518 137.096386 \n",
       "L 225.078518 138.187952 \n",
       "L 225.078518 139.279518 \n",
       "L 225.078518 140.371084 \n",
       "L 225.078518 141.462651 \n",
       "L 225.078518 142.554217 \n",
       "L 225.078518 143.645783 \n",
       "L 225.078518 144.737349 \n",
       "L 225.078518 145.828916 \n",
       "L 225.078518 146.920482 \n",
       "L 225.078518 148.012048 \n",
       "L 225.078518 149.103614 \n",
       "L 225.078518 150.195181 \n",
       "L 225.078518 151.286747 \n",
       "L 225.078518 152.378313 \n",
       "L 225.078518 153.46988 \n",
       "L 225.078518 154.561446 \n",
       "L 225.078518 155.653012 \n",
       "L 225.078518 156.744578 \n",
       "L 225.078518 157.836145 \n",
       "L 225.078518 158.927711 \n",
       "L 225.078518 160.019277 \n",
       "L 225.078518 161.110843 \n",
       "L 225.078518 162.20241 \n",
       "L 225.078518 163.293976 \n",
       "L 225.078518 164.385542 \n",
       "L 225.078518 165.477108 \n",
       "L 225.078518 166.568675 \n",
       "L 225.078518 167.660241 \n",
       "L 225.078518 168.751807 \n",
       "L 225.078518 169.843373 \n",
       "L 225.078518 170.93494 \n",
       "L 225.078518 172.026506 \n",
       "L 225.078518 173.118072 \n",
       "L 225.078518 174.209639 \n",
       "L 225.078518 175.301205 \n",
       "L 225.078518 176.392771 \n",
       "L 225.078518 177.484337 \n",
       "L 225.078518 178.575904 \n",
       "L 225.078518 179.66747 \n",
       "L 225.078518 180.759036 \n",
       "L 225.078518 181.850602 \n",
       "L 225.078518 182.942169 \n",
       "L 225.078518 184.033735 \n",
       "L 225.078518 185.125301 \n",
       "L 225.078518 186.216867 \n",
       "L 225.078518 187.308434 \n",
       "L 225.078518 188.4 \n",
       "L 225.078518 189.491566 \n",
       "L 225.078518 190.583133 \n",
       "L 225.078518 191.674699 \n",
       "L 225.078518 192.766265 \n",
       "L 225.078518 193.857831 \n",
       "L 225.078518 194.949398 \n",
       "z\n",
       "\" style=\"fill:#e377c2;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_7\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 225.91888 195.222289 \n",
       "L 227.039362 195.222289 \n",
       "L 228.159844 195.222289 \n",
       "L 229.280326 195.222289 \n",
       "L 230.400808 195.222289 \n",
       "L 231.52129 195.222289 \n",
       "L 232.641771 195.222289 \n",
       "L 233.762253 195.222289 \n",
       "L 234.882735 195.222289 \n",
       "L 236.003217 195.222289 \n",
       "L 237.123699 195.222289 \n",
       "L 238.244181 195.222289 \n",
       "L 239.364663 195.222289 \n",
       "L 240.485145 195.222289 \n",
       "L 241.605627 195.222289 \n",
       "L 242.726109 195.222289 \n",
       "L 243.846591 195.222289 \n",
       "L 244.967073 195.222289 \n",
       "L 246.087555 195.222289 \n",
       "L 247.208037 195.222289 \n",
       "L 248.328518 195.222289 \n",
       "L 249.449 195.222289 \n",
       "L 250.569482 195.222289 \n",
       "L 251.689964 195.222289 \n",
       "L 252.810446 195.222289 \n",
       "L 253.930928 195.222289 \n",
       "L 255.05141 195.222289 \n",
       "L 256.171892 195.222289 \n",
       "L 257.292374 195.222289 \n",
       "L 258.412856 195.222289 \n",
       "L 259.533338 195.222289 \n",
       "L 260.65382 195.222289 \n",
       "L 261.774302 195.222289 \n",
       "L 262.894784 195.222289 \n",
       "L 264.015265 195.222289 \n",
       "L 265.135747 195.222289 \n",
       "L 266.256229 195.222289 \n",
       "L 267.376711 195.222289 \n",
       "L 268.497193 195.222289 \n",
       "L 269.617675 195.222289 \n",
       "L 270.738157 195.222289 \n",
       "L 271.858639 195.222289 \n",
       "L 272.979121 195.222289 \n",
       "L 274.099603 195.222289 \n",
       "L 275.220085 195.222289 \n",
       "L 276.340567 195.222289 \n",
       "L 277.461049 195.222289 \n",
       "L 278.58153 195.222289 \n",
       "L 279.702012 195.222289 \n",
       "L 280.822494 195.222289 \n",
       "L 281.942976 195.222289 \n",
       "L 283.063458 195.222289 \n",
       "L 284.18394 195.222289 \n",
       "L 285.304422 195.222289 \n",
       "L 286.424904 195.222289 \n",
       "L 287.545386 195.222289 \n",
       "L 288.665868 195.222289 \n",
       "L 289.78635 195.222289 \n",
       "L 290.906832 195.222289 \n",
       "L 292.027314 195.222289 \n",
       "L 293.147796 195.222289 \n",
       "L 294.268277 195.222289 \n",
       "L 295.388759 195.222289 \n",
       "L 296.509241 195.222289 \n",
       "L 297.629723 195.222289 \n",
       "L 298.750205 195.222289 \n",
       "L 299.870687 195.222289 \n",
       "L 300.991169 195.222289 \n",
       "L 302.111651 195.222289 \n",
       "L 303.232133 195.222289 \n",
       "L 304.352615 195.222289 \n",
       "L 305.473097 195.222289 \n",
       "L 306.593579 195.222289 \n",
       "L 307.714061 195.222289 \n",
       "L 308.834543 195.222289 \n",
       "L 309.955024 195.222289 \n",
       "L 311.075506 195.222289 \n",
       "L 312.195988 195.222289 \n",
       "L 313.31647 195.222289 \n",
       "L 314.436952 195.222289 \n",
       "L 315.557434 195.222289 \n",
       "L 316.677916 195.222289 \n",
       "L 317.798398 195.222289 \n",
       "L 318.91888 195.222289 \n",
       "L 320.039362 195.222289 \n",
       "L 321.159844 195.222289 \n",
       "L 321.159844 195.085843 \n",
       "L 320.039362 195.085843 \n",
       "L 318.91888 195.085843 \n",
       "L 317.798398 195.085843 \n",
       "L 316.677916 195.085843 \n",
       "L 315.557434 195.085843 \n",
       "L 314.436952 195.085843 \n",
       "L 313.31647 195.085843 \n",
       "L 312.195988 195.085843 \n",
       "L 311.075506 195.085843 \n",
       "L 309.955024 195.085843 \n",
       "L 308.834543 195.085843 \n",
       "L 307.714061 195.085843 \n",
       "L 306.593579 195.085843 \n",
       "L 305.473097 195.085843 \n",
       "L 304.352615 195.085843 \n",
       "L 303.232133 195.085843 \n",
       "L 302.111651 195.085843 \n",
       "L 300.991169 195.085843 \n",
       "L 299.870687 195.085843 \n",
       "L 298.750205 195.085843 \n",
       "L 297.629723 195.085843 \n",
       "L 296.509241 195.085843 \n",
       "L 295.388759 195.085843 \n",
       "L 294.268277 195.085843 \n",
       "L 293.147796 195.085843 \n",
       "L 292.027314 195.085843 \n",
       "L 290.906832 195.085843 \n",
       "L 289.78635 195.085843 \n",
       "L 288.665868 195.085843 \n",
       "L 287.545386 195.085843 \n",
       "L 286.424904 195.085843 \n",
       "L 285.304422 195.085843 \n",
       "L 284.18394 195.085843 \n",
       "L 283.063458 195.085843 \n",
       "L 281.942976 195.085843 \n",
       "L 280.822494 195.085843 \n",
       "L 279.702012 195.085843 \n",
       "L 278.58153 195.085843 \n",
       "L 277.461049 195.085843 \n",
       "L 276.340567 195.085843 \n",
       "L 275.220085 195.085843 \n",
       "L 274.099603 195.085843 \n",
       "L 272.979121 195.085843 \n",
       "L 271.858639 195.085843 \n",
       "L 270.738157 195.085843 \n",
       "L 269.617675 195.085843 \n",
       "L 268.497193 195.085843 \n",
       "L 267.376711 195.085843 \n",
       "L 266.256229 195.085843 \n",
       "L 265.135747 195.085843 \n",
       "L 264.015265 195.085843 \n",
       "L 262.894784 195.085843 \n",
       "L 261.774302 195.085843 \n",
       "L 260.65382 195.085843 \n",
       "L 259.533338 195.085843 \n",
       "L 258.412856 195.085843 \n",
       "L 257.292374 195.085843 \n",
       "L 256.171892 195.085843 \n",
       "L 255.05141 195.085843 \n",
       "L 253.930928 195.085843 \n",
       "L 252.810446 195.085843 \n",
       "L 251.689964 195.085843 \n",
       "L 250.569482 195.085843 \n",
       "L 249.449 195.085843 \n",
       "L 248.328518 195.085843 \n",
       "L 247.208037 195.085843 \n",
       "L 246.087555 195.085843 \n",
       "L 244.967073 195.085843 \n",
       "L 243.846591 195.085843 \n",
       "L 242.726109 195.085843 \n",
       "L 241.605627 195.085843 \n",
       "L 240.485145 195.085843 \n",
       "L 239.364663 195.085843 \n",
       "L 238.244181 195.085843 \n",
       "L 237.123699 195.085843 \n",
       "L 236.003217 195.085843 \n",
       "L 234.882735 195.085843 \n",
       "L 233.762253 195.085843 \n",
       "L 232.641771 195.085843 \n",
       "L 231.52129 195.085843 \n",
       "L 230.400808 195.085843 \n",
       "L 229.280326 195.085843 \n",
       "L 228.159844 195.085843 \n",
       "L 227.039362 195.085843 \n",
       "L 225.91888 195.085843 \n",
       "L 225.638759 194.949398 \n",
       "L 225.638759 193.857831 \n",
       "L 225.638759 192.766265 \n",
       "L 225.638759 191.674699 \n",
       "L 225.638759 190.583133 \n",
       "L 225.638759 189.491566 \n",
       "L 225.638759 188.4 \n",
       "L 225.638759 187.308434 \n",
       "L 225.638759 186.216867 \n",
       "L 225.638759 185.125301 \n",
       "L 225.638759 184.033735 \n",
       "L 225.638759 182.942169 \n",
       "L 225.638759 181.850602 \n",
       "L 225.638759 180.759036 \n",
       "L 225.638759 179.66747 \n",
       "L 225.638759 178.575904 \n",
       "L 225.638759 177.484337 \n",
       "L 225.638759 176.392771 \n",
       "L 225.638759 175.301205 \n",
       "L 225.638759 174.209639 \n",
       "L 225.638759 173.118072 \n",
       "L 225.638759 172.026506 \n",
       "L 225.638759 170.93494 \n",
       "L 225.638759 169.843373 \n",
       "L 225.638759 168.751807 \n",
       "L 225.638759 167.660241 \n",
       "L 225.638759 166.568675 \n",
       "L 225.638759 165.477108 \n",
       "L 225.638759 164.385542 \n",
       "L 225.638759 163.293976 \n",
       "L 225.638759 162.20241 \n",
       "L 225.638759 161.110843 \n",
       "L 225.638759 160.019277 \n",
       "L 225.638759 158.927711 \n",
       "L 225.638759 157.836145 \n",
       "L 225.638759 156.744578 \n",
       "L 225.638759 155.653012 \n",
       "L 225.638759 154.561446 \n",
       "L 225.638759 153.46988 \n",
       "L 225.638759 152.378313 \n",
       "L 225.638759 151.286747 \n",
       "L 225.638759 150.195181 \n",
       "L 225.638759 149.103614 \n",
       "L 225.638759 148.012048 \n",
       "L 225.638759 146.920482 \n",
       "L 225.638759 145.828916 \n",
       "L 225.638759 144.737349 \n",
       "L 225.638759 143.645783 \n",
       "L 225.638759 142.554217 \n",
       "L 225.638759 141.462651 \n",
       "L 225.638759 140.371084 \n",
       "L 225.638759 139.279518 \n",
       "L 225.638759 138.187952 \n",
       "L 225.638759 137.096386 \n",
       "L 225.638759 136.004819 \n",
       "L 225.638759 134.913253 \n",
       "L 225.638759 133.821687 \n",
       "L 225.638759 132.73012 \n",
       "L 225.638759 131.638554 \n",
       "L 225.638759 130.546988 \n",
       "L 225.638759 129.455422 \n",
       "L 225.638759 128.363855 \n",
       "L 225.638759 127.272289 \n",
       "L 225.638759 126.180723 \n",
       "L 225.638759 125.089157 \n",
       "L 225.638759 123.99759 \n",
       "L 225.638759 122.906024 \n",
       "L 225.638759 121.814458 \n",
       "L 225.638759 120.722892 \n",
       "L 225.638759 119.631325 \n",
       "L 225.638759 118.539759 \n",
       "L 225.638759 117.448193 \n",
       "L 225.91888 117.175301 \n",
       "L 227.039362 117.175301 \n",
       "L 228.159844 117.175301 \n",
       "L 229.280326 117.175301 \n",
       "L 230.400808 117.175301 \n",
       "L 231.52129 117.175301 \n",
       "L 232.641771 117.175301 \n",
       "L 233.762253 117.175301 \n",
       "L 234.882735 117.175301 \n",
       "L 236.003217 117.175301 \n",
       "L 237.123699 117.175301 \n",
       "L 238.244181 117.175301 \n",
       "L 239.364663 117.175301 \n",
       "L 240.485145 117.175301 \n",
       "L 241.605627 117.175301 \n",
       "L 242.726109 117.175301 \n",
       "L 243.846591 117.175301 \n",
       "L 244.967073 117.175301 \n",
       "L 246.087555 117.175301 \n",
       "L 247.208037 117.175301 \n",
       "L 248.048398 116.356627 \n",
       "L 248.048398 115.26506 \n",
       "L 248.048398 114.173494 \n",
       "L 248.048398 113.081928 \n",
       "L 248.048398 111.990361 \n",
       "L 248.048398 110.898795 \n",
       "L 248.048398 109.807229 \n",
       "L 248.048398 108.715663 \n",
       "L 248.048398 107.624096 \n",
       "L 248.048398 106.53253 \n",
       "L 248.048398 105.440964 \n",
       "L 248.048398 104.349398 \n",
       "L 248.048398 103.257831 \n",
       "L 248.048398 102.166265 \n",
       "L 248.048398 101.074699 \n",
       "L 248.048398 99.983133 \n",
       "L 248.048398 98.891566 \n",
       "L 248.048398 97.8 \n",
       "L 248.048398 96.708434 \n",
       "L 247.208037 95.889759 \n",
       "L 246.087555 95.889759 \n",
       "L 244.967073 95.889759 \n",
       "L 243.846591 95.889759 \n",
       "L 242.726109 95.889759 \n",
       "L 241.605627 95.889759 \n",
       "L 240.485145 95.889759 \n",
       "L 239.364663 95.889759 \n",
       "L 238.244181 95.889759 \n",
       "L 237.123699 95.889759 \n",
       "L 236.003217 95.889759 \n",
       "L 234.882735 95.889759 \n",
       "L 233.762253 95.889759 \n",
       "L 232.641771 95.889759 \n",
       "L 231.52129 95.889759 \n",
       "L 230.400808 95.889759 \n",
       "L 229.280326 95.889759 \n",
       "L 228.159844 95.889759 \n",
       "L 227.039362 95.889759 \n",
       "L 225.91888 95.889759 \n",
       "L 225.078518 96.708434 \n",
       "L 225.078518 97.8 \n",
       "L 225.078518 98.891566 \n",
       "L 225.078518 99.983133 \n",
       "L 225.078518 101.074699 \n",
       "L 225.078518 102.166265 \n",
       "L 225.078518 103.257831 \n",
       "L 225.078518 104.349398 \n",
       "L 225.078518 105.440964 \n",
       "L 224.798398 105.713855 \n",
       "L 223.677916 105.713855 \n",
       "L 222.557434 105.713855 \n",
       "L 221.436952 105.713855 \n",
       "L 220.31647 105.713855 \n",
       "L 219.195988 105.713855 \n",
       "L 218.075506 105.713855 \n",
       "L 216.955024 105.713855 \n",
       "L 215.834543 105.713855 \n",
       "L 214.714061 105.713855 \n",
       "L 213.593579 105.713855 \n",
       "L 212.473097 105.713855 \n",
       "L 211.352615 105.713855 \n",
       "L 210.232133 105.713855 \n",
       "L 209.111651 105.713855 \n",
       "L 207.991169 105.713855 \n",
       "L 206.870687 105.713855 \n",
       "L 205.750205 105.713855 \n",
       "L 204.629723 105.713855 \n",
       "L 203.509241 105.713855 \n",
       "L 202.388759 105.713855 \n",
       "L 201.268277 105.713855 \n",
       "L 200.147796 105.713855 \n",
       "L 199.027314 105.713855 \n",
       "L 197.906832 105.713855 \n",
       "L 196.78635 105.713855 \n",
       "L 195.665868 105.713855 \n",
       "L 194.545386 105.713855 \n",
       "L 193.424904 105.713855 \n",
       "L 192.304422 105.713855 \n",
       "L 191.18394 105.713855 \n",
       "L 190.063458 105.713855 \n",
       "L 188.942976 105.713855 \n",
       "L 187.822494 105.713855 \n",
       "L 186.702012 105.713855 \n",
       "L 185.58153 105.713855 \n",
       "L 184.461049 105.713855 \n",
       "L 183.340567 105.713855 \n",
       "L 182.220085 105.713855 \n",
       "L 181.099603 105.713855 \n",
       "L 179.979121 105.713855 \n",
       "L 178.858639 105.713855 \n",
       "L 177.738157 105.713855 \n",
       "L 176.617675 105.713855 \n",
       "L 175.497193 105.713855 \n",
       "L 174.376711 105.713855 \n",
       "L 173.256229 105.713855 \n",
       "L 172.135747 105.713855 \n",
       "L 171.015265 105.713855 \n",
       "L 169.894784 105.713855 \n",
       "L 168.774302 105.713855 \n",
       "L 167.65382 105.713855 \n",
       "L 166.533338 105.713855 \n",
       "L 165.412856 105.713855 \n",
       "L 164.292374 105.713855 \n",
       "L 163.171892 105.713855 \n",
       "L 162.05141 105.713855 \n",
       "L 160.930928 105.713855 \n",
       "L 159.810446 105.713855 \n",
       "L 158.689964 105.713855 \n",
       "L 157.569482 105.713855 \n",
       "L 156.449 105.713855 \n",
       "L 155.328518 105.713855 \n",
       "L 154.208037 105.713855 \n",
       "L 153.087555 105.713855 \n",
       "L 151.967073 105.713855 \n",
       "L 150.846591 105.713855 \n",
       "L 149.726109 105.713855 \n",
       "L 148.605627 105.713855 \n",
       "L 147.485145 105.713855 \n",
       "L 146.364663 105.713855 \n",
       "L 145.244181 105.713855 \n",
       "L 144.123699 105.713855 \n",
       "L 143.003217 105.713855 \n",
       "L 141.882735 105.713855 \n",
       "L 140.762253 105.713855 \n",
       "L 139.641771 105.713855 \n",
       "L 138.52129 105.713855 \n",
       "L 137.400808 105.713855 \n",
       "L 136.280326 105.713855 \n",
       "L 135.159844 105.713855 \n",
       "L 134.039362 105.713855 \n",
       "L 132.91888 105.713855 \n",
       "L 131.798398 105.713855 \n",
       "L 130.677916 105.713855 \n",
       "L 129.557434 105.713855 \n",
       "L 128.436952 105.713855 \n",
       "L 127.31647 105.713855 \n",
       "L 126.195988 105.713855 \n",
       "L 125.075506 105.713855 \n",
       "L 123.955024 105.713855 \n",
       "L 122.834543 105.713855 \n",
       "L 121.714061 105.713855 \n",
       "L 120.593579 105.713855 \n",
       "L 119.473097 105.713855 \n",
       "L 118.352615 105.713855 \n",
       "L 117.232133 105.713855 \n",
       "L 116.111651 105.713855 \n",
       "L 114.991169 105.713855 \n",
       "L 113.870687 105.713855 \n",
       "L 112.750205 105.713855 \n",
       "L 111.629723 105.713855 \n",
       "L 110.509241 105.713855 \n",
       "L 109.388759 105.713855 \n",
       "L 108.268277 105.713855 \n",
       "L 107.147796 105.713855 \n",
       "L 106.027314 105.713855 \n",
       "L 104.906832 105.713855 \n",
       "L 103.78635 105.713855 \n",
       "L 102.665868 105.713855 \n",
       "L 101.545386 105.713855 \n",
       "L 100.424904 105.713855 \n",
       "L 99.304422 105.713855 \n",
       "L 98.18394 105.713855 \n",
       "L 97.063458 105.713855 \n",
       "L 95.942976 105.713855 \n",
       "L 94.822494 105.713855 \n",
       "L 93.702012 105.713855 \n",
       "L 92.58153 105.713855 \n",
       "L 91.461049 105.713855 \n",
       "L 90.340567 105.713855 \n",
       "L 89.220085 105.713855 \n",
       "L 88.099603 105.713855 \n",
       "L 86.979121 105.713855 \n",
       "L 85.858639 105.713855 \n",
       "L 84.738157 105.713855 \n",
       "L 83.617675 105.713855 \n",
       "L 82.497193 105.713855 \n",
       "L 81.376711 105.713855 \n",
       "L 80.256229 105.713855 \n",
       "L 79.135747 105.713855 \n",
       "L 78.015265 105.713855 \n",
       "L 76.894784 105.713855 \n",
       "L 75.774302 105.713855 \n",
       "L 74.65382 105.713855 \n",
       "L 73.533338 105.713855 \n",
       "L 72.412856 105.713855 \n",
       "L 71.292374 105.713855 \n",
       "L 70.171892 105.713855 \n",
       "L 69.05141 105.713855 \n",
       "L 67.930928 105.713855 \n",
       "L 66.810446 105.713855 \n",
       "L 65.689964 105.713855 \n",
       "L 64.569482 105.713855 \n",
       "L 63.449 105.713855 \n",
       "L 62.328518 105.713855 \n",
       "L 61.208037 105.713855 \n",
       "L 60.087555 105.713855 \n",
       "L 58.967073 105.713855 \n",
       "L 57.846591 105.713855 \n",
       "L 56.726109 105.713855 \n",
       "L 55.605627 105.713855 \n",
       "L 54.485145 105.713855 \n",
       "L 53.364663 105.713855 \n",
       "L 52.244181 105.713855 \n",
       "L 51.123699 105.713855 \n",
       "L 50.003217 105.713855 \n",
       "L 48.882735 105.713855 \n",
       "L 47.762253 105.713855 \n",
       "L 46.641771 105.713855 \n",
       "L 45.52129 105.713855 \n",
       "L 44.400808 105.713855 \n",
       "L 43.280326 105.713855 \n",
       "L 42.159844 105.713855 \n",
       "L 42.159844 105.986747 \n",
       "L 43.280326 105.986747 \n",
       "L 44.400808 105.986747 \n",
       "L 45.52129 105.986747 \n",
       "L 46.641771 105.986747 \n",
       "L 47.762253 105.986747 \n",
       "L 48.882735 105.986747 \n",
       "L 50.003217 105.986747 \n",
       "L 51.123699 105.986747 \n",
       "L 52.244181 105.986747 \n",
       "L 53.364663 105.986747 \n",
       "L 54.485145 105.986747 \n",
       "L 55.605627 105.986747 \n",
       "L 56.726109 105.986747 \n",
       "L 57.846591 105.986747 \n",
       "L 58.967073 105.986747 \n",
       "L 60.087555 105.986747 \n",
       "L 61.208037 105.986747 \n",
       "L 62.328518 105.986747 \n",
       "L 63.449 105.986747 \n",
       "L 64.569482 105.986747 \n",
       "L 65.689964 105.986747 \n",
       "L 66.810446 105.986747 \n",
       "L 67.930928 105.986747 \n",
       "L 69.05141 105.986747 \n",
       "L 70.171892 105.986747 \n",
       "L 71.292374 105.986747 \n",
       "L 72.412856 105.986747 \n",
       "L 73.533338 105.986747 \n",
       "L 74.65382 105.986747 \n",
       "L 75.774302 105.986747 \n",
       "L 76.894784 105.986747 \n",
       "L 78.015265 105.986747 \n",
       "L 79.135747 105.986747 \n",
       "L 80.256229 105.986747 \n",
       "L 81.376711 105.986747 \n",
       "L 82.497193 105.986747 \n",
       "L 83.617675 105.986747 \n",
       "L 84.738157 105.986747 \n",
       "L 85.858639 105.986747 \n",
       "L 86.979121 105.986747 \n",
       "L 88.099603 105.986747 \n",
       "L 89.220085 105.986747 \n",
       "L 90.340567 105.986747 \n",
       "L 91.461049 105.986747 \n",
       "L 92.58153 105.986747 \n",
       "L 93.702012 105.986747 \n",
       "L 94.822494 105.986747 \n",
       "L 95.942976 105.986747 \n",
       "L 97.063458 105.986747 \n",
       "L 98.18394 105.986747 \n",
       "L 99.304422 105.986747 \n",
       "L 100.424904 105.986747 \n",
       "L 101.545386 105.986747 \n",
       "L 102.665868 105.986747 \n",
       "L 103.78635 105.986747 \n",
       "L 104.906832 105.986747 \n",
       "L 106.027314 105.986747 \n",
       "L 107.147796 105.986747 \n",
       "L 108.268277 105.986747 \n",
       "L 109.388759 105.986747 \n",
       "L 110.509241 105.986747 \n",
       "L 111.629723 105.986747 \n",
       "L 112.750205 105.986747 \n",
       "L 113.870687 105.986747 \n",
       "L 114.991169 105.986747 \n",
       "L 116.111651 105.986747 \n",
       "L 117.232133 105.986747 \n",
       "L 118.352615 105.986747 \n",
       "L 119.473097 105.986747 \n",
       "L 120.593579 105.986747 \n",
       "L 121.714061 105.986747 \n",
       "L 122.834543 105.986747 \n",
       "L 123.955024 105.986747 \n",
       "L 125.075506 105.986747 \n",
       "L 126.195988 105.986747 \n",
       "L 127.31647 105.986747 \n",
       "L 128.436952 105.986747 \n",
       "L 129.557434 105.986747 \n",
       "L 130.677916 105.986747 \n",
       "L 131.798398 105.986747 \n",
       "L 132.91888 105.986747 \n",
       "L 134.039362 105.986747 \n",
       "L 135.159844 105.986747 \n",
       "L 136.280326 105.986747 \n",
       "L 137.400808 105.986747 \n",
       "L 138.52129 105.986747 \n",
       "L 139.641771 105.986747 \n",
       "L 140.762253 105.986747 \n",
       "L 141.882735 105.986747 \n",
       "L 143.003217 105.986747 \n",
       "L 144.123699 105.986747 \n",
       "L 145.244181 105.986747 \n",
       "L 146.364663 105.986747 \n",
       "L 147.485145 105.986747 \n",
       "L 148.605627 105.986747 \n",
       "L 149.726109 105.986747 \n",
       "L 150.846591 105.986747 \n",
       "L 151.967073 105.986747 \n",
       "L 153.087555 105.986747 \n",
       "L 154.208037 105.986747 \n",
       "L 155.328518 105.986747 \n",
       "L 156.449 105.986747 \n",
       "L 157.569482 105.986747 \n",
       "L 158.689964 105.986747 \n",
       "L 159.810446 105.986747 \n",
       "L 160.930928 105.986747 \n",
       "L 162.05141 105.986747 \n",
       "L 163.171892 105.986747 \n",
       "L 164.292374 105.986747 \n",
       "L 165.412856 105.986747 \n",
       "L 166.533338 105.986747 \n",
       "L 167.65382 105.986747 \n",
       "L 168.774302 105.986747 \n",
       "L 169.894784 105.986747 \n",
       "L 171.015265 105.986747 \n",
       "L 172.135747 105.986747 \n",
       "L 173.256229 105.986747 \n",
       "L 174.376711 105.986747 \n",
       "L 175.497193 105.986747 \n",
       "L 176.617675 105.986747 \n",
       "L 177.738157 105.986747 \n",
       "L 178.858639 105.986747 \n",
       "L 179.979121 105.986747 \n",
       "L 181.099603 105.986747 \n",
       "L 182.220085 105.986747 \n",
       "L 183.340567 105.986747 \n",
       "L 184.461049 105.986747 \n",
       "L 185.58153 105.986747 \n",
       "L 186.702012 105.986747 \n",
       "L 187.822494 105.986747 \n",
       "L 188.942976 105.986747 \n",
       "L 190.063458 105.986747 \n",
       "L 191.18394 105.986747 \n",
       "L 192.304422 105.986747 \n",
       "L 193.424904 105.986747 \n",
       "L 194.545386 105.986747 \n",
       "L 195.665868 105.986747 \n",
       "L 196.78635 105.986747 \n",
       "L 197.906832 105.986747 \n",
       "L 199.027314 105.986747 \n",
       "L 200.147796 105.986747 \n",
       "L 201.268277 105.986747 \n",
       "L 202.388759 105.986747 \n",
       "L 203.509241 105.986747 \n",
       "L 204.629723 105.986747 \n",
       "L 205.750205 105.986747 \n",
       "L 206.870687 105.986747 \n",
       "L 207.991169 105.986747 \n",
       "L 209.111651 105.986747 \n",
       "L 210.232133 105.986747 \n",
       "L 211.352615 105.986747 \n",
       "L 212.473097 105.986747 \n",
       "L 213.593579 105.986747 \n",
       "L 214.714061 105.986747 \n",
       "L 215.834543 105.986747 \n",
       "L 216.955024 105.986747 \n",
       "L 218.075506 105.986747 \n",
       "L 219.195988 105.986747 \n",
       "L 220.31647 105.986747 \n",
       "L 221.436952 105.986747 \n",
       "L 222.557434 105.986747 \n",
       "L 223.677916 105.986747 \n",
       "L 224.798398 105.986747 \n",
       "L 225.358639 105.440964 \n",
       "L 225.358639 104.349398 \n",
       "L 225.358639 103.257831 \n",
       "L 225.358639 102.166265 \n",
       "L 225.358639 101.074699 \n",
       "L 225.358639 99.983133 \n",
       "L 225.358639 98.891566 \n",
       "L 225.358639 97.8 \n",
       "L 225.358639 96.708434 \n",
       "L 225.91888 96.162651 \n",
       "L 227.039362 96.162651 \n",
       "L 228.159844 96.162651 \n",
       "L 229.280326 96.162651 \n",
       "L 230.400808 96.162651 \n",
       "L 231.52129 96.162651 \n",
       "L 232.641771 96.162651 \n",
       "L 233.762253 96.162651 \n",
       "L 234.882735 96.162651 \n",
       "L 236.003217 96.162651 \n",
       "L 237.123699 96.162651 \n",
       "L 238.244181 96.162651 \n",
       "L 239.364663 96.162651 \n",
       "L 240.485145 96.162651 \n",
       "L 241.605627 96.162651 \n",
       "L 242.726109 96.162651 \n",
       "L 243.846591 96.162651 \n",
       "L 244.967073 96.162651 \n",
       "L 246.087555 96.162651 \n",
       "L 247.208037 96.162651 \n",
       "L 247.768277 96.708434 \n",
       "L 247.768277 97.8 \n",
       "L 247.768277 98.891566 \n",
       "L 247.768277 99.983133 \n",
       "L 247.768277 101.074699 \n",
       "L 247.768277 102.166265 \n",
       "L 247.768277 103.257831 \n",
       "L 247.768277 104.349398 \n",
       "L 247.768277 105.440964 \n",
       "L 247.768277 106.53253 \n",
       "L 247.768277 107.624096 \n",
       "L 247.768277 108.715663 \n",
       "L 247.768277 109.807229 \n",
       "L 247.768277 110.898795 \n",
       "L 247.768277 111.990361 \n",
       "L 247.768277 113.081928 \n",
       "L 247.768277 114.173494 \n",
       "L 247.768277 115.26506 \n",
       "L 247.768277 116.356627 \n",
       "L 247.208037 116.90241 \n",
       "L 246.087555 116.90241 \n",
       "L 244.967073 116.90241 \n",
       "L 243.846591 116.90241 \n",
       "L 242.726109 116.90241 \n",
       "L 241.605627 116.90241 \n",
       "L 240.485145 116.90241 \n",
       "L 239.364663 116.90241 \n",
       "L 238.244181 116.90241 \n",
       "L 237.123699 116.90241 \n",
       "L 236.003217 116.90241 \n",
       "L 234.882735 116.90241 \n",
       "L 233.762253 116.90241 \n",
       "L 232.641771 116.90241 \n",
       "L 231.52129 116.90241 \n",
       "L 230.400808 116.90241 \n",
       "L 229.280326 116.90241 \n",
       "L 228.159844 116.90241 \n",
       "L 227.039362 116.90241 \n",
       "L 225.91888 116.90241 \n",
       "L 225.358639 117.448193 \n",
       "L 225.358639 118.539759 \n",
       "L 225.358639 119.631325 \n",
       "L 225.358639 120.722892 \n",
       "L 225.358639 121.814458 \n",
       "L 225.358639 122.906024 \n",
       "L 225.358639 123.99759 \n",
       "L 225.358639 125.089157 \n",
       "L 225.358639 126.180723 \n",
       "L 225.358639 127.272289 \n",
       "L 225.358639 128.363855 \n",
       "L 225.358639 129.455422 \n",
       "L 225.358639 130.546988 \n",
       "L 225.358639 131.638554 \n",
       "L 225.358639 132.73012 \n",
       "L 225.358639 133.821687 \n",
       "L 225.358639 134.913253 \n",
       "L 225.358639 136.004819 \n",
       "L 225.358639 137.096386 \n",
       "L 225.358639 138.187952 \n",
       "L 225.358639 139.279518 \n",
       "L 225.358639 140.371084 \n",
       "L 225.358639 141.462651 \n",
       "L 225.358639 142.554217 \n",
       "L 225.358639 143.645783 \n",
       "L 225.358639 144.737349 \n",
       "L 225.358639 145.828916 \n",
       "L 225.358639 146.920482 \n",
       "L 225.358639 148.012048 \n",
       "L 225.358639 149.103614 \n",
       "L 225.358639 150.195181 \n",
       "L 225.358639 151.286747 \n",
       "L 225.358639 152.378313 \n",
       "L 225.358639 153.46988 \n",
       "L 225.358639 154.561446 \n",
       "L 225.358639 155.653012 \n",
       "L 225.358639 156.744578 \n",
       "L 225.358639 157.836145 \n",
       "L 225.358639 158.927711 \n",
       "L 225.358639 160.019277 \n",
       "L 225.358639 161.110843 \n",
       "L 225.358639 162.20241 \n",
       "L 225.358639 163.293976 \n",
       "L 225.358639 164.385542 \n",
       "L 225.358639 165.477108 \n",
       "L 225.358639 166.568675 \n",
       "L 225.358639 167.660241 \n",
       "L 225.358639 168.751807 \n",
       "L 225.358639 169.843373 \n",
       "L 225.358639 170.93494 \n",
       "L 225.358639 172.026506 \n",
       "L 225.358639 173.118072 \n",
       "L 225.358639 174.209639 \n",
       "L 225.358639 175.301205 \n",
       "L 225.358639 176.392771 \n",
       "L 225.358639 177.484337 \n",
       "L 225.358639 178.575904 \n",
       "L 225.358639 179.66747 \n",
       "L 225.358639 180.759036 \n",
       "L 225.358639 181.850602 \n",
       "L 225.358639 182.942169 \n",
       "L 225.358639 184.033735 \n",
       "L 225.358639 185.125301 \n",
       "L 225.358639 186.216867 \n",
       "L 225.358639 187.308434 \n",
       "L 225.358639 188.4 \n",
       "L 225.358639 189.491566 \n",
       "L 225.358639 190.583133 \n",
       "L 225.358639 191.674699 \n",
       "L 225.358639 192.766265 \n",
       "L 225.358639 193.857831 \n",
       "L 225.358639 194.949398 \n",
       "z\n",
       "\" style=\"fill:#bcbd22;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_8\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 225.91888 195.085843 \n",
       "L 227.039362 195.085843 \n",
       "L 228.159844 195.085843 \n",
       "L 229.280326 195.085843 \n",
       "L 230.400808 195.085843 \n",
       "L 231.52129 195.085843 \n",
       "L 232.641771 195.085843 \n",
       "L 233.762253 195.085843 \n",
       "L 234.882735 195.085843 \n",
       "L 236.003217 195.085843 \n",
       "L 237.123699 195.085843 \n",
       "L 238.244181 195.085843 \n",
       "L 239.364663 195.085843 \n",
       "L 240.485145 195.085843 \n",
       "L 241.605627 195.085843 \n",
       "L 242.726109 195.085843 \n",
       "L 243.846591 195.085843 \n",
       "L 244.967073 195.085843 \n",
       "L 246.087555 195.085843 \n",
       "L 247.208037 195.085843 \n",
       "L 248.328518 195.085843 \n",
       "L 249.449 195.085843 \n",
       "L 250.569482 195.085843 \n",
       "L 251.689964 195.085843 \n",
       "L 252.810446 195.085843 \n",
       "L 253.930928 195.085843 \n",
       "L 255.05141 195.085843 \n",
       "L 256.171892 195.085843 \n",
       "L 257.292374 195.085843 \n",
       "L 258.412856 195.085843 \n",
       "L 259.533338 195.085843 \n",
       "L 260.65382 195.085843 \n",
       "L 261.774302 195.085843 \n",
       "L 262.894784 195.085843 \n",
       "L 264.015265 195.085843 \n",
       "L 265.135747 195.085843 \n",
       "L 266.256229 195.085843 \n",
       "L 267.376711 195.085843 \n",
       "L 268.497193 195.085843 \n",
       "L 269.617675 195.085843 \n",
       "L 270.738157 195.085843 \n",
       "L 271.858639 195.085843 \n",
       "L 272.979121 195.085843 \n",
       "L 274.099603 195.085843 \n",
       "L 275.220085 195.085843 \n",
       "L 276.340567 195.085843 \n",
       "L 277.461049 195.085843 \n",
       "L 278.58153 195.085843 \n",
       "L 279.702012 195.085843 \n",
       "L 280.822494 195.085843 \n",
       "L 281.942976 195.085843 \n",
       "L 283.063458 195.085843 \n",
       "L 284.18394 195.085843 \n",
       "L 285.304422 195.085843 \n",
       "L 286.424904 195.085843 \n",
       "L 287.545386 195.085843 \n",
       "L 288.665868 195.085843 \n",
       "L 289.78635 195.085843 \n",
       "L 290.906832 195.085843 \n",
       "L 292.027314 195.085843 \n",
       "L 293.147796 195.085843 \n",
       "L 294.268277 195.085843 \n",
       "L 295.388759 195.085843 \n",
       "L 296.509241 195.085843 \n",
       "L 297.629723 195.085843 \n",
       "L 298.750205 195.085843 \n",
       "L 299.870687 195.085843 \n",
       "L 300.991169 195.085843 \n",
       "L 302.111651 195.085843 \n",
       "L 303.232133 195.085843 \n",
       "L 304.352615 195.085843 \n",
       "L 305.473097 195.085843 \n",
       "L 306.593579 195.085843 \n",
       "L 307.714061 195.085843 \n",
       "L 308.834543 195.085843 \n",
       "L 309.955024 195.085843 \n",
       "L 311.075506 195.085843 \n",
       "L 312.195988 195.085843 \n",
       "L 313.31647 195.085843 \n",
       "L 314.436952 195.085843 \n",
       "L 315.557434 195.085843 \n",
       "L 316.677916 195.085843 \n",
       "L 317.798398 195.085843 \n",
       "L 318.91888 195.085843 \n",
       "L 320.039362 195.085843 \n",
       "L 321.159844 195.085843 \n",
       "L 321.159844 194.949398 \n",
       "L 321.159844 193.857831 \n",
       "L 321.159844 192.766265 \n",
       "L 321.159844 191.674699 \n",
       "L 321.159844 190.583133 \n",
       "L 321.159844 189.491566 \n",
       "L 321.159844 188.4 \n",
       "L 321.159844 187.308434 \n",
       "L 321.159844 186.216867 \n",
       "L 321.159844 185.125301 \n",
       "L 321.159844 184.033735 \n",
       "L 321.159844 182.942169 \n",
       "L 321.159844 181.850602 \n",
       "L 321.159844 180.759036 \n",
       "L 321.159844 179.66747 \n",
       "L 321.159844 178.575904 \n",
       "L 321.159844 177.484337 \n",
       "L 321.159844 176.392771 \n",
       "L 321.159844 175.301205 \n",
       "L 321.159844 174.209639 \n",
       "L 321.159844 173.118072 \n",
       "L 321.159844 172.026506 \n",
       "L 321.159844 170.93494 \n",
       "L 321.159844 169.843373 \n",
       "L 321.159844 168.751807 \n",
       "L 321.159844 167.660241 \n",
       "L 321.159844 166.568675 \n",
       "L 321.159844 165.477108 \n",
       "L 321.159844 164.385542 \n",
       "L 321.159844 163.293976 \n",
       "L 321.159844 162.20241 \n",
       "L 321.159844 161.110843 \n",
       "L 321.159844 160.019277 \n",
       "L 321.159844 158.927711 \n",
       "L 321.159844 157.836145 \n",
       "L 321.159844 156.744578 \n",
       "L 321.159844 155.653012 \n",
       "L 321.159844 154.561446 \n",
       "L 321.159844 153.46988 \n",
       "L 321.159844 152.378313 \n",
       "L 321.159844 151.286747 \n",
       "L 321.159844 150.195181 \n",
       "L 321.159844 149.103614 \n",
       "L 321.159844 148.012048 \n",
       "L 321.159844 146.920482 \n",
       "L 321.159844 145.828916 \n",
       "L 321.159844 144.737349 \n",
       "L 321.159844 143.645783 \n",
       "L 321.159844 142.554217 \n",
       "L 321.159844 141.462651 \n",
       "L 321.159844 140.371084 \n",
       "L 321.159844 139.279518 \n",
       "L 321.159844 138.187952 \n",
       "L 321.159844 137.096386 \n",
       "L 321.159844 136.004819 \n",
       "L 321.159844 134.913253 \n",
       "L 321.159844 133.821687 \n",
       "L 321.159844 132.73012 \n",
       "L 321.159844 131.638554 \n",
       "L 321.159844 130.546988 \n",
       "L 321.159844 129.455422 \n",
       "L 321.159844 128.363855 \n",
       "L 321.159844 127.272289 \n",
       "L 321.159844 126.180723 \n",
       "L 321.159844 125.089157 \n",
       "L 321.159844 123.99759 \n",
       "L 321.159844 122.906024 \n",
       "L 321.159844 121.814458 \n",
       "L 321.159844 120.722892 \n",
       "L 321.159844 119.631325 \n",
       "L 321.159844 118.539759 \n",
       "L 321.159844 117.448193 \n",
       "L 321.159844 116.356627 \n",
       "L 321.159844 115.26506 \n",
       "L 321.159844 114.173494 \n",
       "L 321.159844 113.081928 \n",
       "L 321.159844 111.990361 \n",
       "L 321.159844 110.898795 \n",
       "L 321.159844 109.807229 \n",
       "L 321.159844 108.715663 \n",
       "L 321.159844 107.624096 \n",
       "L 321.159844 106.53253 \n",
       "L 321.159844 105.440964 \n",
       "L 321.159844 104.349398 \n",
       "L 321.159844 103.257831 \n",
       "L 321.159844 102.166265 \n",
       "L 321.159844 101.074699 \n",
       "L 321.159844 99.983133 \n",
       "L 321.159844 98.891566 \n",
       "L 321.159844 97.8 \n",
       "L 321.159844 96.708434 \n",
       "L 321.159844 95.616867 \n",
       "L 321.159844 94.525301 \n",
       "L 321.159844 93.433735 \n",
       "L 321.159844 92.342169 \n",
       "L 321.159844 91.250602 \n",
       "L 321.159844 90.159036 \n",
       "L 321.159844 89.06747 \n",
       "L 321.159844 87.975904 \n",
       "L 321.159844 86.884337 \n",
       "L 321.159844 85.792771 \n",
       "L 321.159844 84.701205 \n",
       "L 321.159844 83.609639 \n",
       "L 321.159844 82.518072 \n",
       "L 321.159844 81.426506 \n",
       "L 321.159844 80.33494 \n",
       "L 321.159844 79.243373 \n",
       "L 321.159844 78.151807 \n",
       "L 321.159844 77.060241 \n",
       "L 321.159844 75.968675 \n",
       "L 321.159844 74.877108 \n",
       "L 321.159844 73.785542 \n",
       "L 321.159844 72.693976 \n",
       "L 321.159844 71.60241 \n",
       "L 321.159844 70.510843 \n",
       "L 321.159844 69.419277 \n",
       "L 321.159844 68.327711 \n",
       "L 321.159844 67.236145 \n",
       "L 321.159844 66.144578 \n",
       "L 321.159844 65.053012 \n",
       "L 321.159844 63.961446 \n",
       "L 321.159844 62.86988 \n",
       "L 321.159844 61.778313 \n",
       "L 321.159844 60.686747 \n",
       "L 321.159844 59.595181 \n",
       "L 321.159844 58.503614 \n",
       "L 321.159844 57.412048 \n",
       "L 321.159844 56.320482 \n",
       "L 321.159844 55.228916 \n",
       "L 321.159844 54.137349 \n",
       "L 321.159844 53.045783 \n",
       "L 321.159844 51.954217 \n",
       "L 321.159844 50.862651 \n",
       "L 321.159844 49.771084 \n",
       "L 321.159844 48.679518 \n",
       "L 321.159844 47.587952 \n",
       "L 321.159844 46.496386 \n",
       "L 321.159844 45.404819 \n",
       "L 321.159844 44.313253 \n",
       "L 321.159844 43.221687 \n",
       "L 321.159844 42.13012 \n",
       "L 321.159844 41.038554 \n",
       "L 321.159844 39.946988 \n",
       "L 321.159844 38.855422 \n",
       "L 321.159844 37.763855 \n",
       "L 321.159844 36.672289 \n",
       "L 321.159844 35.580723 \n",
       "L 321.159844 34.489157 \n",
       "L 321.159844 33.39759 \n",
       "L 321.159844 32.306024 \n",
       "L 321.159844 31.214458 \n",
       "L 321.159844 30.122892 \n",
       "L 321.159844 29.031325 \n",
       "L 321.159844 27.939759 \n",
       "L 321.159844 26.848193 \n",
       "L 321.159844 25.756627 \n",
       "L 321.159844 24.66506 \n",
       "L 321.159844 23.573494 \n",
       "L 321.159844 22.481928 \n",
       "L 321.159844 21.390361 \n",
       "L 321.159844 20.298795 \n",
       "L 321.159844 19.207229 \n",
       "L 321.159844 18.115663 \n",
       "L 321.159844 17.024096 \n",
       "L 321.159844 15.93253 \n",
       "L 321.159844 14.840964 \n",
       "L 321.159844 13.749398 \n",
       "L 321.159844 12.657831 \n",
       "L 321.159844 11.566265 \n",
       "L 321.159844 10.474699 \n",
       "L 321.159844 9.383133 \n",
       "L 321.159844 8.291566 \n",
       "L 321.159844 7.2 \n",
       "L 320.039362 7.2 \n",
       "L 318.91888 7.2 \n",
       "L 317.798398 7.2 \n",
       "L 316.677916 7.2 \n",
       "L 315.557434 7.2 \n",
       "L 314.436952 7.2 \n",
       "L 313.31647 7.2 \n",
       "L 312.195988 7.2 \n",
       "L 311.075506 7.2 \n",
       "L 309.955024 7.2 \n",
       "L 308.834543 7.2 \n",
       "L 307.714061 7.2 \n",
       "L 306.593579 7.2 \n",
       "L 305.473097 7.2 \n",
       "L 304.352615 7.2 \n",
       "L 303.232133 7.2 \n",
       "L 302.111651 7.2 \n",
       "L 300.991169 7.2 \n",
       "L 299.870687 7.2 \n",
       "L 298.750205 7.2 \n",
       "L 297.629723 7.2 \n",
       "L 296.509241 7.2 \n",
       "L 295.388759 7.2 \n",
       "L 294.268277 7.2 \n",
       "L 293.147796 7.2 \n",
       "L 292.027314 7.2 \n",
       "L 290.906832 7.2 \n",
       "L 289.78635 7.2 \n",
       "L 288.665868 7.2 \n",
       "L 287.545386 7.2 \n",
       "L 286.424904 7.2 \n",
       "L 285.304422 7.2 \n",
       "L 284.18394 7.2 \n",
       "L 283.063458 7.2 \n",
       "L 281.942976 7.2 \n",
       "L 280.822494 7.2 \n",
       "L 279.702012 7.2 \n",
       "L 278.58153 7.2 \n",
       "L 277.461049 7.2 \n",
       "L 276.340567 7.2 \n",
       "L 275.220085 7.2 \n",
       "L 274.099603 7.2 \n",
       "L 272.979121 7.2 \n",
       "L 271.858639 7.2 \n",
       "L 270.738157 7.2 \n",
       "L 269.617675 7.2 \n",
       "L 268.497193 7.2 \n",
       "L 267.376711 7.2 \n",
       "L 266.256229 7.2 \n",
       "L 265.135747 7.2 \n",
       "L 264.015265 7.2 \n",
       "L 262.894784 7.2 \n",
       "L 261.774302 7.2 \n",
       "L 260.65382 7.2 \n",
       "L 259.533338 7.2 \n",
       "L 258.412856 7.2 \n",
       "L 257.292374 7.2 \n",
       "L 256.171892 7.2 \n",
       "L 255.05141 7.2 \n",
       "L 253.930928 7.2 \n",
       "L 252.810446 7.2 \n",
       "L 251.689964 7.2 \n",
       "L 250.569482 7.2 \n",
       "L 249.449 7.2 \n",
       "L 248.328518 7.2 \n",
       "L 247.208037 7.2 \n",
       "L 246.087555 7.2 \n",
       "L 244.967073 7.2 \n",
       "L 243.846591 7.2 \n",
       "L 242.726109 7.2 \n",
       "L 241.605627 7.2 \n",
       "L 240.485145 7.2 \n",
       "L 239.364663 7.2 \n",
       "L 238.244181 7.2 \n",
       "L 237.123699 7.2 \n",
       "L 236.003217 7.2 \n",
       "L 234.882735 7.2 \n",
       "L 233.762253 7.2 \n",
       "L 232.641771 7.2 \n",
       "L 231.52129 7.2 \n",
       "L 230.400808 7.2 \n",
       "L 229.280326 7.2 \n",
       "L 228.159844 7.2 \n",
       "L 227.039362 7.2 \n",
       "L 225.91888 7.2 \n",
       "L 224.798398 7.2 \n",
       "L 223.677916 7.2 \n",
       "L 222.557434 7.2 \n",
       "L 221.436952 7.2 \n",
       "L 220.31647 7.2 \n",
       "L 219.195988 7.2 \n",
       "L 218.075506 7.2 \n",
       "L 216.955024 7.2 \n",
       "L 215.834543 7.2 \n",
       "L 214.714061 7.2 \n",
       "L 213.593579 7.2 \n",
       "L 212.473097 7.2 \n",
       "L 211.352615 7.2 \n",
       "L 210.232133 7.2 \n",
       "L 209.111651 7.2 \n",
       "L 207.991169 7.2 \n",
       "L 206.870687 7.2 \n",
       "L 205.750205 7.2 \n",
       "L 204.629723 7.2 \n",
       "L 203.509241 7.2 \n",
       "L 202.388759 7.2 \n",
       "L 201.268277 7.2 \n",
       "L 200.147796 7.2 \n",
       "L 199.027314 7.2 \n",
       "L 197.906832 7.2 \n",
       "L 196.78635 7.2 \n",
       "L 195.665868 7.2 \n",
       "L 194.545386 7.2 \n",
       "L 193.424904 7.2 \n",
       "L 192.304422 7.2 \n",
       "L 191.18394 7.2 \n",
       "L 190.063458 7.2 \n",
       "L 188.942976 7.2 \n",
       "L 187.822494 7.2 \n",
       "L 186.702012 7.2 \n",
       "L 185.58153 7.2 \n",
       "L 184.461049 7.2 \n",
       "L 183.340567 7.2 \n",
       "L 182.220085 7.2 \n",
       "L 181.099603 7.2 \n",
       "L 179.979121 7.2 \n",
       "L 178.858639 7.2 \n",
       "L 177.738157 7.2 \n",
       "L 176.617675 7.2 \n",
       "L 175.497193 7.2 \n",
       "L 174.376711 7.2 \n",
       "L 173.256229 7.2 \n",
       "L 172.135747 7.2 \n",
       "L 171.015265 7.2 \n",
       "L 169.894784 7.2 \n",
       "L 168.774302 7.2 \n",
       "L 167.65382 7.2 \n",
       "L 166.533338 7.2 \n",
       "L 165.412856 7.2 \n",
       "L 164.292374 7.2 \n",
       "L 163.171892 7.2 \n",
       "L 162.05141 7.2 \n",
       "L 160.930928 7.2 \n",
       "L 159.810446 7.2 \n",
       "L 158.689964 7.2 \n",
       "L 157.569482 7.2 \n",
       "L 156.449 7.2 \n",
       "L 155.328518 7.2 \n",
       "L 154.208037 7.2 \n",
       "L 153.087555 7.2 \n",
       "L 151.967073 7.2 \n",
       "L 150.846591 7.2 \n",
       "L 149.726109 7.2 \n",
       "L 148.605627 7.2 \n",
       "L 147.485145 7.2 \n",
       "L 146.364663 7.2 \n",
       "L 145.244181 7.2 \n",
       "L 144.123699 7.2 \n",
       "L 143.003217 7.2 \n",
       "L 141.882735 7.2 \n",
       "L 140.762253 7.2 \n",
       "L 139.641771 7.2 \n",
       "L 138.52129 7.2 \n",
       "L 137.400808 7.2 \n",
       "L 136.280326 7.2 \n",
       "L 135.159844 7.2 \n",
       "L 134.039362 7.2 \n",
       "L 132.91888 7.2 \n",
       "L 131.798398 7.2 \n",
       "L 130.677916 7.2 \n",
       "L 129.557434 7.2 \n",
       "L 128.436952 7.2 \n",
       "L 127.31647 7.2 \n",
       "L 126.195988 7.2 \n",
       "L 125.075506 7.2 \n",
       "L 123.955024 7.2 \n",
       "L 122.834543 7.2 \n",
       "L 121.714061 7.2 \n",
       "L 120.593579 7.2 \n",
       "L 119.473097 7.2 \n",
       "L 118.352615 7.2 \n",
       "L 117.232133 7.2 \n",
       "L 116.111651 7.2 \n",
       "L 114.991169 7.2 \n",
       "L 113.870687 7.2 \n",
       "L 112.750205 7.2 \n",
       "L 111.629723 7.2 \n",
       "L 110.509241 7.2 \n",
       "L 109.388759 7.2 \n",
       "L 108.268277 7.2 \n",
       "L 107.147796 7.2 \n",
       "L 106.027314 7.2 \n",
       "L 104.906832 7.2 \n",
       "L 103.78635 7.2 \n",
       "L 102.665868 7.2 \n",
       "L 101.545386 7.2 \n",
       "L 100.424904 7.2 \n",
       "L 99.304422 7.2 \n",
       "L 98.18394 7.2 \n",
       "L 97.063458 7.2 \n",
       "L 95.942976 7.2 \n",
       "L 94.822494 7.2 \n",
       "L 93.702012 7.2 \n",
       "L 92.58153 7.2 \n",
       "L 91.461049 7.2 \n",
       "L 90.340567 7.2 \n",
       "L 89.220085 7.2 \n",
       "L 88.099603 7.2 \n",
       "L 86.979121 7.2 \n",
       "L 85.858639 7.2 \n",
       "L 84.738157 7.2 \n",
       "L 83.617675 7.2 \n",
       "L 82.497193 7.2 \n",
       "L 81.376711 7.2 \n",
       "L 80.256229 7.2 \n",
       "L 79.135747 7.2 \n",
       "L 78.015265 7.2 \n",
       "L 76.894784 7.2 \n",
       "L 75.774302 7.2 \n",
       "L 74.65382 7.2 \n",
       "L 73.533338 7.2 \n",
       "L 72.412856 7.2 \n",
       "L 71.292374 7.2 \n",
       "L 70.171892 7.2 \n",
       "L 69.05141 7.2 \n",
       "L 67.930928 7.2 \n",
       "L 66.810446 7.2 \n",
       "L 65.689964 7.2 \n",
       "L 64.569482 7.2 \n",
       "L 63.449 7.2 \n",
       "L 62.328518 7.2 \n",
       "L 61.208037 7.2 \n",
       "L 60.087555 7.2 \n",
       "L 58.967073 7.2 \n",
       "L 57.846591 7.2 \n",
       "L 56.726109 7.2 \n",
       "L 55.605627 7.2 \n",
       "L 54.485145 7.2 \n",
       "L 53.364663 7.2 \n",
       "L 52.244181 7.2 \n",
       "L 51.123699 7.2 \n",
       "L 50.003217 7.2 \n",
       "L 48.882735 7.2 \n",
       "L 47.762253 7.2 \n",
       "L 46.641771 7.2 \n",
       "L 45.52129 7.2 \n",
       "L 44.400808 7.2 \n",
       "L 43.280326 7.2 \n",
       "L 42.159844 7.2 \n",
       "L 42.159844 8.291566 \n",
       "L 42.159844 9.383133 \n",
       "L 42.159844 10.474699 \n",
       "L 42.159844 11.566265 \n",
       "L 42.159844 12.657831 \n",
       "L 42.159844 13.749398 \n",
       "L 42.159844 14.840964 \n",
       "L 42.159844 15.93253 \n",
       "L 42.159844 17.024096 \n",
       "L 42.159844 18.115663 \n",
       "L 42.159844 19.207229 \n",
       "L 42.159844 20.298795 \n",
       "L 42.159844 21.390361 \n",
       "L 42.159844 22.481928 \n",
       "L 42.159844 23.573494 \n",
       "L 42.159844 24.66506 \n",
       "L 42.159844 25.756627 \n",
       "L 42.159844 26.848193 \n",
       "L 42.159844 27.939759 \n",
       "L 42.159844 29.031325 \n",
       "L 42.159844 30.122892 \n",
       "L 42.159844 31.214458 \n",
       "L 42.159844 32.306024 \n",
       "L 42.159844 33.39759 \n",
       "L 42.159844 34.489157 \n",
       "L 42.159844 35.580723 \n",
       "L 42.159844 36.672289 \n",
       "L 42.159844 37.763855 \n",
       "L 42.159844 38.855422 \n",
       "L 42.159844 39.946988 \n",
       "L 42.159844 41.038554 \n",
       "L 42.159844 42.13012 \n",
       "L 42.159844 43.221687 \n",
       "L 42.159844 44.313253 \n",
       "L 42.159844 45.404819 \n",
       "L 42.159844 46.496386 \n",
       "L 42.159844 47.587952 \n",
       "L 42.159844 48.679518 \n",
       "L 42.159844 49.771084 \n",
       "L 42.159844 50.862651 \n",
       "L 42.159844 51.954217 \n",
       "L 42.159844 53.045783 \n",
       "L 42.159844 54.137349 \n",
       "L 42.159844 55.228916 \n",
       "L 42.159844 56.320482 \n",
       "L 42.159844 57.412048 \n",
       "L 42.159844 58.503614 \n",
       "L 42.159844 59.595181 \n",
       "L 42.159844 60.686747 \n",
       "L 42.159844 61.778313 \n",
       "L 42.159844 62.86988 \n",
       "L 42.159844 63.961446 \n",
       "L 42.159844 65.053012 \n",
       "L 42.159844 66.144578 \n",
       "L 42.159844 67.236145 \n",
       "L 42.159844 68.327711 \n",
       "L 42.159844 69.419277 \n",
       "L 42.159844 70.510843 \n",
       "L 42.159844 71.60241 \n",
       "L 42.159844 72.693976 \n",
       "L 42.159844 73.785542 \n",
       "L 42.159844 74.877108 \n",
       "L 42.159844 75.968675 \n",
       "L 42.159844 77.060241 \n",
       "L 42.159844 78.151807 \n",
       "L 42.159844 79.243373 \n",
       "L 42.159844 80.33494 \n",
       "L 42.159844 81.426506 \n",
       "L 42.159844 82.518072 \n",
       "L 42.159844 83.609639 \n",
       "L 42.159844 84.701205 \n",
       "L 42.159844 85.792771 \n",
       "L 42.159844 86.884337 \n",
       "L 42.159844 87.975904 \n",
       "L 42.159844 89.06747 \n",
       "L 42.159844 90.159036 \n",
       "L 42.159844 91.250602 \n",
       "L 42.159844 92.342169 \n",
       "L 42.159844 93.433735 \n",
       "L 42.159844 94.525301 \n",
       "L 42.159844 95.616867 \n",
       "L 42.159844 96.708434 \n",
       "L 42.159844 97.8 \n",
       "L 42.159844 98.891566 \n",
       "L 42.159844 99.983133 \n",
       "L 42.159844 101.074699 \n",
       "L 42.159844 102.166265 \n",
       "L 42.159844 103.257831 \n",
       "L 42.159844 104.349398 \n",
       "L 42.159844 105.440964 \n",
       "L 42.159844 105.713855 \n",
       "L 43.280326 105.713855 \n",
       "L 44.400808 105.713855 \n",
       "L 45.52129 105.713855 \n",
       "L 46.641771 105.713855 \n",
       "L 47.762253 105.713855 \n",
       "L 48.882735 105.713855 \n",
       "L 50.003217 105.713855 \n",
       "L 51.123699 105.713855 \n",
       "L 52.244181 105.713855 \n",
       "L 53.364663 105.713855 \n",
       "L 54.485145 105.713855 \n",
       "L 55.605627 105.713855 \n",
       "L 56.726109 105.713855 \n",
       "L 57.846591 105.713855 \n",
       "L 58.967073 105.713855 \n",
       "L 60.087555 105.713855 \n",
       "L 61.208037 105.713855 \n",
       "L 62.328518 105.713855 \n",
       "L 63.449 105.713855 \n",
       "L 64.569482 105.713855 \n",
       "L 65.689964 105.713855 \n",
       "L 66.810446 105.713855 \n",
       "L 67.930928 105.713855 \n",
       "L 69.05141 105.713855 \n",
       "L 70.171892 105.713855 \n",
       "L 71.292374 105.713855 \n",
       "L 72.412856 105.713855 \n",
       "L 73.533338 105.713855 \n",
       "L 74.65382 105.713855 \n",
       "L 75.774302 105.713855 \n",
       "L 76.894784 105.713855 \n",
       "L 78.015265 105.713855 \n",
       "L 79.135747 105.713855 \n",
       "L 80.256229 105.713855 \n",
       "L 81.376711 105.713855 \n",
       "L 82.497193 105.713855 \n",
       "L 83.617675 105.713855 \n",
       "L 84.738157 105.713855 \n",
       "L 85.858639 105.713855 \n",
       "L 86.979121 105.713855 \n",
       "L 88.099603 105.713855 \n",
       "L 89.220085 105.713855 \n",
       "L 90.340567 105.713855 \n",
       "L 91.461049 105.713855 \n",
       "L 92.58153 105.713855 \n",
       "L 93.702012 105.713855 \n",
       "L 94.822494 105.713855 \n",
       "L 95.942976 105.713855 \n",
       "L 97.063458 105.713855 \n",
       "L 98.18394 105.713855 \n",
       "L 99.304422 105.713855 \n",
       "L 100.424904 105.713855 \n",
       "L 101.545386 105.713855 \n",
       "L 102.665868 105.713855 \n",
       "L 103.78635 105.713855 \n",
       "L 104.906832 105.713855 \n",
       "L 106.027314 105.713855 \n",
       "L 107.147796 105.713855 \n",
       "L 108.268277 105.713855 \n",
       "L 109.388759 105.713855 \n",
       "L 110.509241 105.713855 \n",
       "L 111.629723 105.713855 \n",
       "L 112.750205 105.713855 \n",
       "L 113.870687 105.713855 \n",
       "L 114.991169 105.713855 \n",
       "L 116.111651 105.713855 \n",
       "L 117.232133 105.713855 \n",
       "L 118.352615 105.713855 \n",
       "L 119.473097 105.713855 \n",
       "L 120.593579 105.713855 \n",
       "L 121.714061 105.713855 \n",
       "L 122.834543 105.713855 \n",
       "L 123.955024 105.713855 \n",
       "L 125.075506 105.713855 \n",
       "L 126.195988 105.713855 \n",
       "L 127.31647 105.713855 \n",
       "L 128.436952 105.713855 \n",
       "L 129.557434 105.713855 \n",
       "L 130.677916 105.713855 \n",
       "L 131.798398 105.713855 \n",
       "L 132.91888 105.713855 \n",
       "L 134.039362 105.713855 \n",
       "L 135.159844 105.713855 \n",
       "L 136.280326 105.713855 \n",
       "L 137.400808 105.713855 \n",
       "L 138.52129 105.713855 \n",
       "L 139.641771 105.713855 \n",
       "L 140.762253 105.713855 \n",
       "L 141.882735 105.713855 \n",
       "L 143.003217 105.713855 \n",
       "L 144.123699 105.713855 \n",
       "L 145.244181 105.713855 \n",
       "L 146.364663 105.713855 \n",
       "L 147.485145 105.713855 \n",
       "L 148.605627 105.713855 \n",
       "L 149.726109 105.713855 \n",
       "L 150.846591 105.713855 \n",
       "L 151.967073 105.713855 \n",
       "L 153.087555 105.713855 \n",
       "L 154.208037 105.713855 \n",
       "L 155.328518 105.713855 \n",
       "L 156.449 105.713855 \n",
       "L 157.569482 105.713855 \n",
       "L 158.689964 105.713855 \n",
       "L 159.810446 105.713855 \n",
       "L 160.930928 105.713855 \n",
       "L 162.05141 105.713855 \n",
       "L 163.171892 105.713855 \n",
       "L 164.292374 105.713855 \n",
       "L 165.412856 105.713855 \n",
       "L 166.533338 105.713855 \n",
       "L 167.65382 105.713855 \n",
       "L 168.774302 105.713855 \n",
       "L 169.894784 105.713855 \n",
       "L 171.015265 105.713855 \n",
       "L 172.135747 105.713855 \n",
       "L 173.256229 105.713855 \n",
       "L 174.376711 105.713855 \n",
       "L 175.497193 105.713855 \n",
       "L 176.617675 105.713855 \n",
       "L 177.738157 105.713855 \n",
       "L 178.858639 105.713855 \n",
       "L 179.979121 105.713855 \n",
       "L 181.099603 105.713855 \n",
       "L 182.220085 105.713855 \n",
       "L 183.340567 105.713855 \n",
       "L 184.461049 105.713855 \n",
       "L 185.58153 105.713855 \n",
       "L 186.702012 105.713855 \n",
       "L 187.822494 105.713855 \n",
       "L 188.942976 105.713855 \n",
       "L 190.063458 105.713855 \n",
       "L 191.18394 105.713855 \n",
       "L 192.304422 105.713855 \n",
       "L 193.424904 105.713855 \n",
       "L 194.545386 105.713855 \n",
       "L 195.665868 105.713855 \n",
       "L 196.78635 105.713855 \n",
       "L 197.906832 105.713855 \n",
       "L 199.027314 105.713855 \n",
       "L 200.147796 105.713855 \n",
       "L 201.268277 105.713855 \n",
       "L 202.388759 105.713855 \n",
       "L 203.509241 105.713855 \n",
       "L 204.629723 105.713855 \n",
       "L 205.750205 105.713855 \n",
       "L 206.870687 105.713855 \n",
       "L 207.991169 105.713855 \n",
       "L 209.111651 105.713855 \n",
       "L 210.232133 105.713855 \n",
       "L 211.352615 105.713855 \n",
       "L 212.473097 105.713855 \n",
       "L 213.593579 105.713855 \n",
       "L 214.714061 105.713855 \n",
       "L 215.834543 105.713855 \n",
       "L 216.955024 105.713855 \n",
       "L 218.075506 105.713855 \n",
       "L 219.195988 105.713855 \n",
       "L 220.31647 105.713855 \n",
       "L 221.436952 105.713855 \n",
       "L 222.557434 105.713855 \n",
       "L 223.677916 105.713855 \n",
       "L 224.798398 105.713855 \n",
       "L 225.078518 105.440964 \n",
       "L 225.078518 104.349398 \n",
       "L 225.078518 103.257831 \n",
       "L 225.078518 102.166265 \n",
       "L 225.078518 101.074699 \n",
       "L 225.078518 99.983133 \n",
       "L 225.078518 98.891566 \n",
       "L 225.078518 97.8 \n",
       "L 225.078518 96.708434 \n",
       "L 225.91888 95.889759 \n",
       "L 227.039362 95.889759 \n",
       "L 228.159844 95.889759 \n",
       "L 229.280326 95.889759 \n",
       "L 230.400808 95.889759 \n",
       "L 231.52129 95.889759 \n",
       "L 232.641771 95.889759 \n",
       "L 233.762253 95.889759 \n",
       "L 234.882735 95.889759 \n",
       "L 236.003217 95.889759 \n",
       "L 237.123699 95.889759 \n",
       "L 238.244181 95.889759 \n",
       "L 239.364663 95.889759 \n",
       "L 240.485145 95.889759 \n",
       "L 241.605627 95.889759 \n",
       "L 242.726109 95.889759 \n",
       "L 243.846591 95.889759 \n",
       "L 244.967073 95.889759 \n",
       "L 246.087555 95.889759 \n",
       "L 247.208037 95.889759 \n",
       "L 248.048398 96.708434 \n",
       "L 248.048398 97.8 \n",
       "L 248.048398 98.891566 \n",
       "L 248.048398 99.983133 \n",
       "L 248.048398 101.074699 \n",
       "L 248.048398 102.166265 \n",
       "L 248.048398 103.257831 \n",
       "L 248.048398 104.349398 \n",
       "L 248.048398 105.440964 \n",
       "L 248.048398 106.53253 \n",
       "L 248.048398 107.624096 \n",
       "L 248.048398 108.715663 \n",
       "L 248.048398 109.807229 \n",
       "L 248.048398 110.898795 \n",
       "L 248.048398 111.990361 \n",
       "L 248.048398 113.081928 \n",
       "L 248.048398 114.173494 \n",
       "L 248.048398 115.26506 \n",
       "L 248.048398 116.356627 \n",
       "L 247.208037 117.175301 \n",
       "L 246.087555 117.175301 \n",
       "L 244.967073 117.175301 \n",
       "L 243.846591 117.175301 \n",
       "L 242.726109 117.175301 \n",
       "L 241.605627 117.175301 \n",
       "L 240.485145 117.175301 \n",
       "L 239.364663 117.175301 \n",
       "L 238.244181 117.175301 \n",
       "L 237.123699 117.175301 \n",
       "L 236.003217 117.175301 \n",
       "L 234.882735 117.175301 \n",
       "L 233.762253 117.175301 \n",
       "L 232.641771 117.175301 \n",
       "L 231.52129 117.175301 \n",
       "L 230.400808 117.175301 \n",
       "L 229.280326 117.175301 \n",
       "L 228.159844 117.175301 \n",
       "L 227.039362 117.175301 \n",
       "L 225.91888 117.175301 \n",
       "L 225.638759 117.448193 \n",
       "L 225.638759 118.539759 \n",
       "L 225.638759 119.631325 \n",
       "L 225.638759 120.722892 \n",
       "L 225.638759 121.814458 \n",
       "L 225.638759 122.906024 \n",
       "L 225.638759 123.99759 \n",
       "L 225.638759 125.089157 \n",
       "L 225.638759 126.180723 \n",
       "L 225.638759 127.272289 \n",
       "L 225.638759 128.363855 \n",
       "L 225.638759 129.455422 \n",
       "L 225.638759 130.546988 \n",
       "L 225.638759 131.638554 \n",
       "L 225.638759 132.73012 \n",
       "L 225.638759 133.821687 \n",
       "L 225.638759 134.913253 \n",
       "L 225.638759 136.004819 \n",
       "L 225.638759 137.096386 \n",
       "L 225.638759 138.187952 \n",
       "L 225.638759 139.279518 \n",
       "L 225.638759 140.371084 \n",
       "L 225.638759 141.462651 \n",
       "L 225.638759 142.554217 \n",
       "L 225.638759 143.645783 \n",
       "L 225.638759 144.737349 \n",
       "L 225.638759 145.828916 \n",
       "L 225.638759 146.920482 \n",
       "L 225.638759 148.012048 \n",
       "L 225.638759 149.103614 \n",
       "L 225.638759 150.195181 \n",
       "L 225.638759 151.286747 \n",
       "L 225.638759 152.378313 \n",
       "L 225.638759 153.46988 \n",
       "L 225.638759 154.561446 \n",
       "L 225.638759 155.653012 \n",
       "L 225.638759 156.744578 \n",
       "L 225.638759 157.836145 \n",
       "L 225.638759 158.927711 \n",
       "L 225.638759 160.019277 \n",
       "L 225.638759 161.110843 \n",
       "L 225.638759 162.20241 \n",
       "L 225.638759 163.293976 \n",
       "L 225.638759 164.385542 \n",
       "L 225.638759 165.477108 \n",
       "L 225.638759 166.568675 \n",
       "L 225.638759 167.660241 \n",
       "L 225.638759 168.751807 \n",
       "L 225.638759 169.843373 \n",
       "L 225.638759 170.93494 \n",
       "L 225.638759 172.026506 \n",
       "L 225.638759 173.118072 \n",
       "L 225.638759 174.209639 \n",
       "L 225.638759 175.301205 \n",
       "L 225.638759 176.392771 \n",
       "L 225.638759 177.484337 \n",
       "L 225.638759 178.575904 \n",
       "L 225.638759 179.66747 \n",
       "L 225.638759 180.759036 \n",
       "L 225.638759 181.850602 \n",
       "L 225.638759 182.942169 \n",
       "L 225.638759 184.033735 \n",
       "L 225.638759 185.125301 \n",
       "L 225.638759 186.216867 \n",
       "L 225.638759 187.308434 \n",
       "L 225.638759 188.4 \n",
       "L 225.638759 189.491566 \n",
       "L 225.638759 190.583133 \n",
       "L 225.638759 191.674699 \n",
       "L 225.638759 192.766265 \n",
       "L 225.638759 193.857831 \n",
       "L 225.638759 194.949398 \n",
       "z\n",
       "\" style=\"fill:#17becf;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_9\">\n",
       "    <defs>\n",
       "     <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"C0_0_0cb6c71108\"/>\n",
       "    </defs>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_0cb6c71108\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_0cb6c71108\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"284.555324\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"271.419731\" xlink:href=\"#C0_0_0cb6c71108\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_0cb6c71108\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_0cb6c71108\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_0cb6c71108\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_0cb6c71108\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_0cb6c71108\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_0cb6c71108\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_0cb6c71108\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_0cb6c71108\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"91.899957\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"52.493177\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"91.899957\" xlink:href=\"#C0_0_0cb6c71108\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"196.984703\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"153.199392\" xlink:href=\"#C0_0_0cb6c71108\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_0cb6c71108\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_0cb6c71108\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_0cb6c71108\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_0cb6c71108\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_0cb6c71108\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_0cb6c71108\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_0cb6c71108\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_0cb6c71108\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_0cb6c71108\" y=\"216.502778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"161.956454\" xlink:href=\"#C0_0_0cb6c71108\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_0cb6c71108\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_0cb6c71108\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_0cb6c71108\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_0cb6c71108\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"175.092047\" xlink:href=\"#C0_0_0cb6c71108\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_0cb6c71108\" y=\"101.155556\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_0cb6c71108\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_0cb6c71108\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"236.391482\" xlink:href=\"#C0_0_0cb6c71108\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_0cb6c71108\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_0cb6c71108\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"166.334985\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_0cb6c71108\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"271.419731\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"56.871708\" xlink:href=\"#C0_0_0cb6c71108\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"240.770013\" xlink:href=\"#C0_0_0cb6c71108\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_0cb6c71108\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"140.063799\" xlink:href=\"#C0_0_0cb6c71108\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_0cb6c71108\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"267.0412\" xlink:href=\"#C0_0_0cb6c71108\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_0cb6c71108\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_0cb6c71108\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_0cb6c71108\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_0cb6c71108\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_0cb6c71108\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_0cb6c71108\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"245.148544\" xlink:href=\"#C0_0_0cb6c71108\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_0cb6c71108\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"245.148544\" xlink:href=\"#C0_0_0cb6c71108\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_0cb6c71108\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_0cb6c71108\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"267.0412\" xlink:href=\"#C0_0_0cb6c71108\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"297.690917\" xlink:href=\"#C0_0_0cb6c71108\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_0cb6c71108\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.250239\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_0cb6c71108\" y=\"226.988889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"196.984703\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"310.82651\" xlink:href=\"#C0_0_0cb6c71108\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"153.199392\" xlink:href=\"#C0_0_0cb6c71108\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"288.933855\" xlink:href=\"#C0_0_0cb6c71108\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_0cb6c71108\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.250239\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"302.069448\" xlink:href=\"#C0_0_0cb6c71108\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_0cb6c71108\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_0cb6c71108\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_0cb6c71108\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"161.956454\" xlink:href=\"#C0_0_0cb6c71108\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"240.770013\" xlink:href=\"#C0_0_0cb6c71108\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"236.391482\" xlink:href=\"#C0_0_0cb6c71108\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_0cb6c71108\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_0cb6c71108\" y=\"101.155556\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"170.713516\" xlink:href=\"#C0_0_0cb6c71108\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_0cb6c71108\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_0cb6c71108\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_0cb6c71108\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_0cb6c71108\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_0cb6c71108\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_0cb6c71108\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"302.069448\" xlink:href=\"#C0_0_0cb6c71108\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#pe304051125)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_0cb6c71108\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_1\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 196.040964 \n",
       "L 321.159844 196.040964 \n",
       "L 321.159844 196.040964 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_2\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 195.768072 \n",
       "L 224.798398 195.768072 \n",
       "L 225.91888 195.904518 \n",
       "L 321.159844 195.904518 \n",
       "L 321.159844 195.904518 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_3\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 195.495181 \n",
       "L 224.798398 195.495181 \n",
       "L 225.91888 195.768072 \n",
       "L 321.159844 195.768072 \n",
       "L 321.159844 195.768072 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_4\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 195.222289 \n",
       "L 224.798398 195.222289 \n",
       "L 225.91888 195.631627 \n",
       "L 321.159844 195.631627 \n",
       "L 321.159844 195.631627 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_5\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 106.53253 \n",
       "L 224.798398 106.53253 \n",
       "L 225.91888 105.440964 \n",
       "L 225.91888 96.708434 \n",
       "L 247.208037 96.708434 \n",
       "L 247.208037 116.356627 \n",
       "L 225.91888 116.356627 \n",
       "L 224.798398 117.448193 \n",
       "L 224.798398 194.949398 \n",
       "L 225.91888 195.495181 \n",
       "L 321.159844 195.495181 \n",
       "L 321.159844 195.495181 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_6\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 106.259639 \n",
       "L 224.798398 106.259639 \n",
       "L 225.638759 105.440964 \n",
       "L 225.638759 96.708434 \n",
       "L 225.91888 96.435542 \n",
       "L 247.208037 96.435542 \n",
       "L 247.488157 96.708434 \n",
       "L 247.488157 116.356627 \n",
       "L 247.208037 116.629518 \n",
       "L 225.91888 116.629518 \n",
       "L 225.078518 117.448193 \n",
       "L 225.078518 194.949398 \n",
       "L 225.91888 195.358735 \n",
       "L 321.159844 195.358735 \n",
       "L 321.159844 195.358735 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_7\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 105.986747 \n",
       "L 224.798398 105.986747 \n",
       "L 225.358639 105.440964 \n",
       "L 225.358639 96.708434 \n",
       "L 225.91888 96.162651 \n",
       "L 247.208037 96.162651 \n",
       "L 247.768277 96.708434 \n",
       "L 247.768277 116.356627 \n",
       "L 247.208037 116.90241 \n",
       "L 225.91888 116.90241 \n",
       "L 225.358639 117.448193 \n",
       "L 225.358639 194.949398 \n",
       "L 225.91888 195.222289 \n",
       "L 321.159844 195.222289 \n",
       "L 321.159844 195.222289 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_8\">\n",
       "    <path clip-path=\"url(#pe304051125)\" d=\"M 42.159844 105.713855 \n",
       "L 224.798398 105.713855 \n",
       "L 225.078518 105.440964 \n",
       "L 225.078518 96.708434 \n",
       "L 225.91888 95.889759 \n",
       "L 247.208037 95.889759 \n",
       "L 248.048398 96.708434 \n",
       "L 248.048398 116.356627 \n",
       "L 247.208037 117.175301 \n",
       "L 225.91888 117.175301 \n",
       "L 225.638759 117.448193 \n",
       "L 225.638759 194.949398 \n",
       "L 225.91888 195.085843 \n",
       "L 321.159844 195.085843 \n",
       "L 321.159844 195.085843 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_9\"/>\n",
       "   <g id=\"patch_3\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 42.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_4\">\n",
       "    <path d=\"M 321.159844 279 \n",
       "L 321.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_5\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 321.159844 279 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_6\">\n",
       "    <path d=\"M 42.159844 7.2 \n",
       "L 321.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"legend_1\">\n",
       "    <g id=\"line2d_25\"/>\n",
       "    <g id=\"line2d_26\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"mb01994f406\" style=\"stroke:#1f77b4;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#1f77b4;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#mb01994f406\" y=\"19.857812\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_15\">\n",
       "     <!-- setosa -->\n",
       "     <defs>\n",
       "      <path d=\"M 3.078125 15.484375 \n",
       "L 11.765625 16.84375 \n",
       "Q 12.5 11.625 15.84375 8.84375 \n",
       "Q 19.1875 6.0625 25.203125 6.0625 \n",
       "Q 31.25 6.0625 34.171875 8.515625 \n",
       "Q 37.109375 10.984375 37.109375 14.3125 \n",
       "Q 37.109375 17.28125 34.515625 19 \n",
       "Q 32.71875 20.171875 25.53125 21.96875 \n",
       "Q 15.875 24.421875 12.140625 26.203125 \n",
       "Q 8.40625 27.984375 6.46875 31.125 \n",
       "Q 4.546875 34.28125 4.546875 38.09375 \n",
       "Q 4.546875 41.546875 6.125 44.5 \n",
       "Q 7.71875 47.46875 10.453125 49.421875 \n",
       "Q 12.5 50.921875 16.03125 51.96875 \n",
       "Q 19.578125 53.03125 23.640625 53.03125 \n",
       "Q 29.734375 53.03125 34.34375 51.265625 \n",
       "Q 38.96875 49.515625 41.15625 46.5 \n",
       "Q 43.359375 43.5 44.1875 38.484375 \n",
       "L 35.59375 37.3125 \n",
       "Q 35.015625 41.3125 32.203125 43.546875 \n",
       "Q 29.390625 45.796875 24.265625 45.796875 \n",
       "Q 18.21875 45.796875 15.625 43.796875 \n",
       "Q 13.03125 41.796875 13.03125 39.109375 \n",
       "Q 13.03125 37.40625 14.109375 36.03125 \n",
       "Q 15.1875 34.625 17.484375 33.6875 \n",
       "Q 18.796875 33.203125 25.25 31.453125 \n",
       "Q 34.578125 28.953125 38.25 27.359375 \n",
       "Q 41.9375 25.78125 44.03125 22.75 \n",
       "Q 46.140625 19.734375 46.140625 15.234375 \n",
       "Q 46.140625 10.84375 43.578125 6.953125 \n",
       "Q 41.015625 3.078125 36.171875 0.953125 \n",
       "Q 31.34375 -1.171875 25.25 -1.171875 \n",
       "Q 15.140625 -1.171875 9.84375 3.03125 \n",
       "Q 4.546875 7.234375 3.078125 15.484375 \n",
       "z\n",
       "\" id=\"ArialMT-115\"/>\n",
       "      <path d=\"M 3.328125 25.921875 \n",
       "Q 3.328125 40.328125 11.328125 47.265625 \n",
       "Q 18.015625 53.03125 27.640625 53.03125 \n",
       "Q 38.328125 53.03125 45.109375 46.015625 \n",
       "Q 51.90625 39.015625 51.90625 26.65625 \n",
       "Q 51.90625 16.65625 48.90625 10.90625 \n",
       "Q 45.90625 5.171875 40.15625 2 \n",
       "Q 34.421875 -1.171875 27.640625 -1.171875 \n",
       "Q 16.75 -1.171875 10.03125 5.8125 \n",
       "Q 3.328125 12.796875 3.328125 25.921875 \n",
       "z\n",
       "M 12.359375 25.921875 \n",
       "Q 12.359375 15.96875 16.703125 11.015625 \n",
       "Q 21.046875 6.0625 27.640625 6.0625 \n",
       "Q 34.1875 6.0625 38.53125 11.03125 \n",
       "Q 42.875 16.015625 42.875 26.21875 \n",
       "Q 42.875 35.84375 38.5 40.796875 \n",
       "Q 34.125 45.75 27.640625 45.75 \n",
       "Q 21.046875 45.75 16.703125 40.8125 \n",
       "Q 12.359375 35.890625 12.359375 25.921875 \n",
       "z\n",
       "\" id=\"ArialMT-111\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 23.357812)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"105.615234\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"133.398438\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"189.013672\" xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"239.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"line2d_27\"/>\n",
       "    <g id=\"line2d_28\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"m00da8bc2ae\" style=\"stroke:#8c564b;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#8c564b;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#m00da8bc2ae\" y=\"34.003125\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_16\">\n",
       "     <!-- versicolor -->\n",
       "     <defs>\n",
       "      <path d=\"M 21 0 \n",
       "L 1.265625 51.859375 \n",
       "L 10.546875 51.859375 \n",
       "L 21.6875 20.796875 \n",
       "Q 23.484375 15.765625 25 10.359375 \n",
       "Q 26.171875 14.453125 28.265625 20.21875 \n",
       "L 39.796875 51.859375 \n",
       "L 48.828125 51.859375 \n",
       "L 29.203125 0 \n",
       "z\n",
       "\" id=\"ArialMT-118\"/>\n",
       "      <path d=\"M 6.5 0 \n",
       "L 6.5 51.859375 \n",
       "L 14.40625 51.859375 \n",
       "L 14.40625 44 \n",
       "Q 17.4375 49.515625 20 51.265625 \n",
       "Q 22.5625 53.03125 25.640625 53.03125 \n",
       "Q 30.078125 53.03125 34.671875 50.203125 \n",
       "L 31.640625 42.046875 \n",
       "Q 28.421875 43.953125 25.203125 43.953125 \n",
       "Q 22.3125 43.953125 20.015625 42.21875 \n",
       "Q 17.71875 40.484375 16.75 37.40625 \n",
       "Q 15.28125 32.71875 15.28125 27.15625 \n",
       "L 15.28125 0 \n",
       "z\n",
       "\" id=\"ArialMT-114\"/>\n",
       "      <path d=\"M 40.4375 19 \n",
       "L 49.078125 17.875 \n",
       "Q 47.65625 8.9375 41.8125 3.875 \n",
       "Q 35.984375 -1.171875 27.484375 -1.171875 \n",
       "Q 16.84375 -1.171875 10.375 5.78125 \n",
       "Q 3.90625 12.75 3.90625 25.734375 \n",
       "Q 3.90625 34.125 6.6875 40.421875 \n",
       "Q 9.46875 46.734375 15.15625 49.875 \n",
       "Q 20.84375 53.03125 27.546875 53.03125 \n",
       "Q 35.984375 53.03125 41.359375 48.75 \n",
       "Q 46.734375 44.484375 48.25 36.625 \n",
       "L 39.703125 35.296875 \n",
       "Q 38.484375 40.53125 35.375 43.15625 \n",
       "Q 32.28125 45.796875 27.875 45.796875 \n",
       "Q 21.234375 45.796875 17.078125 41.03125 \n",
       "Q 12.9375 36.28125 12.9375 25.984375 \n",
       "Q 12.9375 15.53125 16.9375 10.796875 \n",
       "Q 20.953125 6.0625 27.390625 6.0625 \n",
       "Q 32.5625 6.0625 36.03125 9.234375 \n",
       "Q 39.5 12.40625 40.4375 19 \n",
       "z\n",
       "\" id=\"ArialMT-99\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 37.503125)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-118\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"105.615234\" xlink:href=\"#ArialMT-114\"/>\n",
       "      <use x=\"138.916016\" xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"188.916016\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"211.132812\" xlink:href=\"#ArialMT-99\"/>\n",
       "      <use x=\"261.132812\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"316.748047\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"338.964844\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"394.580078\" xlink:href=\"#ArialMT-114\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"line2d_29\"/>\n",
       "    <g id=\"line2d_30\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"ma542fc995d\" style=\"stroke:#17becf;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#17becf;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#ma542fc995d\" y=\"48.148437\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_17\">\n",
       "     <!-- virginica -->\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 51.648437)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-118\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"72.216797\" xlink:href=\"#ArialMT-114\"/>\n",
       "      <use x=\"105.517578\" xlink:href=\"#ArialMT-103\"/>\n",
       "      <use x=\"161.132812\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"183.349609\" xlink:href=\"#ArialMT-110\"/>\n",
       "      <use x=\"238.964844\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"261.181641\" xlink:href=\"#ArialMT-99\"/>\n",
       "      <use x=\"311.181641\" xlink:href=\"#ArialMT-97\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "  </g>\n",
       " </g>\n",
       " <defs>\n",
       "  <clipPath id=\"pe304051125\">\n",
       "   <rect height=\"271.8\" width=\"279\" x=\"42.159844\" y=\"7.2\"/>\n",
       "  </clipPath>\n",
       " </defs>\n",
       "</svg>\n"
      ],
      "text/plain": [
       "<Figure size 360x360 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "cls = DecisionTreeClassifier()\n",
    "cls.fit(X2, Y)\n",
    "plot_boundary(cls, X2, Y)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Decision tree implementation\n",
    "\n",
    "In the following section, we'll implement a learning algorithm for decision tree classifiers.\n",
    "\n",
    "To represent a decision tree, we define the two components of decision trees: leaves and branches. A *leaf* corresponds to the base case of a recursion. It is a \"dummy\" tree that always returns a constant value.\n",
    "\n",
    "The code that is used to draw the trees can be ignored if you want."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "metadata": {},
   "outputs": [],
   "source": [
    "class DecisionTreeLeaf:\n",
    "\n",
    "    def __init__(self, value):\n",
    "        self.value = value\n",
    "\n",
    "    # This method computes the prediction for this leaf node. This will just return a constant value.\n",
    "    def predict(self, x):\n",
    "        return self.value\n",
    "\n",
    "    # Utility function to draw a tree visually using graphviz.\n",
    "    def draw_tree(self, graph, node_counter, names):\n",
    "        node_id = str(node_counter)\n",
    "        val_str = f'{self.value:.4g}' if isinstance(self.value, float) else str(self.value)\n",
    "        graph.node(node_id, val_str, style='filled')\n",
    "        return node_counter+1, node_id\n",
    "        \n",
    "    def __eq__(self, other):\n",
    "        if isinstance(other, DecisionTreeLeaf):\n",
    "            return self.value == other.value\n",
    "        else:\n",
    "            return False"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "A *branch* will look at one feature, and will select a subtree depending on the value of the feature. That subtree will then be called recursively to compute the prediction.\n",
    "\n",
    "This implementation assumes that the feature is numerical. Depending on whether the feature is or isn't greater than a threshold, the \"high\" or \"low\" subtree will be selected."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "metadata": {},
   "outputs": [],
   "source": [
    "class DecisionTreeBranch:\n",
    "\n",
    "    def __init__(self, feature, threshold, low_subtree, high_subtree):\n",
    "        self.feature = feature\n",
    "        self.threshold = threshold\n",
    "        self.low_subtree = low_subtree\n",
    "        self.high_subtree = high_subtree\n",
    "\n",
    "    # For a branch node, we compute the prediction by first considering the feature, and then \n",
    "    # calling the upper or lower subtree, depending on whether the feature is or isn't greater\n",
    "    # than the threshold.\n",
    "    def predict(self, x):\n",
    "        if x[self.feature] <= self.threshold:\n",
    "            return self.low_subtree.predict(x)\n",
    "        else:\n",
    "            return self.high_subtree.predict(x)\n",
    "\n",
    "    # Utility function to draw a tree visually using graphviz.\n",
    "    def draw_tree(self, graph, node_counter, names):\n",
    "        node_counter, low_id = self.low_subtree.draw_tree(graph, node_counter, names)\n",
    "        node_counter, high_id = self.high_subtree.draw_tree(graph, node_counter, names)\n",
    "        node_id = str(node_counter)\n",
    "        fname = f'F{self.feature}' if names is None else names[self.feature]\n",
    "        lbl = f'{fname} > {self.threshold:.4g}?'\n",
    "        graph.node(node_id, lbl, shape='box', fillcolor='yellow', style='filled, rounded')\n",
    "        graph.edge(node_id, low_id, 'False')\n",
    "        graph.edge(node_id, high_id, 'True')\n",
    "        return node_counter+1, node_id\n",
    "        "
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now, we have the components needed to implement decision tree classifiers and regression models.\n",
    "\n",
    "Following standard practice in scikit-learn, we inherit from the class [`BaseEstimator`](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html), which is the base class of all classifiers and regression models.\n",
    "\n",
    "We write the `DecisionTree` class as an abstract class that contains the functionality common to all types of decision trees. Classification and regression models will be implemented as subclasses. The classification subclass is given below, while the regression subclass will be implemented as a part of the first assignment. Following scikit-learn naming conventions, we'll call the training and prediction methods `fit` and `predict`, respectively.\n",
    "\n",
    "The functions that the subclasses will need to implement, and which are done differently for classification and regression, are the following:\n",
    "\n",
    "* `get_default_value`: what output value to use if we decide to return a leaf node. For classification, this will be the most common output value, while for regression it will be the mean.\n",
    "* `is_homogeneous`: tests whether a set of output values is homogeneous. For classification, this means that all outputs are identical; for regression, we'll probably test whether the variance is smaller than some threshold.\n",
    "* `best_split`: finds the best splitting point for that feature. For classification, this will be based on one of the classification criteria (information gain, gini impurity, majority sum); for regression, it will be based on variances in the subsets."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "metadata": {},
   "outputs": [],
   "source": [
    "from graphviz import Digraph\n",
    "from sklearn.base import BaseEstimator, ClassifierMixin\n",
    "from abc import ABC, abstractmethod\n",
    "\n",
    "class DecisionTree(ABC, BaseEstimator):\n",
    "\n",
    "    def __init__(self, max_depth):\n",
    "        super().__init__()\n",
    "        self.max_depth = max_depth\n",
    "        \n",
    "    # As usual in scikit-learn, the training method is called *fit*. We first process the dataset so that\n",
    "    # we're sure that it's represented as a NumPy matrix. Then we call the recursive tree-building method\n",
    "    # called make_tree (see below).\n",
    "    def fit(self, X, Y):\n",
    "        if isinstance(X, pd.DataFrame):\n",
    "            self.names = X.columns\n",
    "            X = X.to_numpy()\n",
    "        elif isinstance(X, list):\n",
    "            self.names = None\n",
    "            X = np.array(X)\n",
    "        else:\n",
    "            self.names = None\n",
    "        Y = np.array(Y)        \n",
    "        self.root = self.make_tree(X, Y, self.max_depth)\n",
    "        \n",
    "    def draw_tree(self):\n",
    "        graph = Digraph()\n",
    "        self.root.draw_tree(graph, 0, self.names)\n",
    "        return graph\n",
    "    \n",
    "    # By scikit-learn convention, the method *predict* computes the classification or regression output\n",
    "    # for a set of instances.\n",
    "    # To implement it, we call a separate method that carries out the prediction for one instance.\n",
    "    def predict(self, X):\n",
    "        if isinstance(X, pd.DataFrame):\n",
    "            X = X.to_numpy()\n",
    "        return [self.predict_one(x) for x in X]\n",
    "\n",
    "    # Predicting the output for one instance.\n",
    "    def predict_one(self, x):\n",
    "        return self.root.predict(x)        \n",
    "\n",
    "    # This is the recursive training \n",
    "    def make_tree(self, X, Y, max_depth):\n",
    "\n",
    "        # We start by computing the default value that will be used if we'll return a leaf node.\n",
    "        # For classifiers, this will be the most common value in Y.\n",
    "        default_value = self.get_default_value(Y)\n",
    "\n",
    "        # First the two base cases in the recursion: is the training set completely\n",
    "        # homogeneous, or have we reached the maximum depth? Then we need to return a leaf.\n",
    "\n",
    "        # If we have reached the maximum depth, return a leaf with the majority value.\n",
    "        if max_depth == 0:\n",
    "            return DecisionTreeLeaf(default_value)\n",
    "\n",
    "        # If all the instances in the remaining training set have the same output value,\n",
    "        # return a leaf with this value.\n",
    "        if self.is_homogeneous(Y):\n",
    "            return DecisionTreeLeaf(default_value)\n",
    "\n",
    "        # Select the \"most useful\" feature and split threshold. To rank the \"usefulness\" of features,\n",
    "        # we use one of the classification or regression criteria.\n",
    "        # For each feature, we call best_split (defined in a subclass). We then maximize over the features.\n",
    "        n_features = X.shape[1]\n",
    "        _, best_feature, best_threshold = max(self.best_split(X, Y, feature) for feature in range(n_features))\n",
    "        \n",
    "        if best_feature is None:\n",
    "            return DecisionTreeLeaf(default_value)\n",
    "\n",
    "        # Split the training set into subgroups, based on whether the selected feature is greater than\n",
    "        # the threshold or not\n",
    "        X_low, X_high, Y_low, Y_high = self.split_by_feature(X, Y, best_feature, best_threshold)\n",
    "\n",
    "        # Build the subtrees using a recursive call. Each subtree is associated\n",
    "        # with a value of the feature.\n",
    "        low_subtree = self.make_tree(X_low, Y_low, max_depth-1)\n",
    "        high_subtree = self.make_tree(X_high, Y_high, max_depth-1)\n",
    "\n",
    "        if low_subtree == high_subtree:\n",
    "            return low_subtree\n",
    "\n",
    "        # Return a decision tree branch containing the result.\n",
    "        return DecisionTreeBranch(best_feature, best_threshold, low_subtree, high_subtree)\n",
    "    \n",
    "    # Utility method that splits the data into the \"upper\" and \"lower\" part, based on a feature\n",
    "    # and a threshold.\n",
    "    def split_by_feature(self, X, Y, feature, threshold):\n",
    "        low = X[:,feature] <= threshold\n",
    "        high = ~low\n",
    "        return X[low], X[high], Y[low], Y[high]\n",
    "    \n",
    "    # The following three methods need to be implemented by the classification and regression subclasses.\n",
    "    \n",
    "    @abstractmethod\n",
    "    def get_default_value(self, Y):\n",
    "        pass\n",
    "\n",
    "    @abstractmethod\n",
    "    def is_homogeneous(self, Y):\n",
    "        pass\n",
    "\n",
    "    @abstractmethod\n",
    "    def best_split(self, X, Y, feature):\n",
    "        pass"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "This is the subclass that implements decision tree classification. This implementation makes heavy use of the [`Counter`](https://docs.python.org/3/library/collections.html#collections.Counter) class, which is a standard Python data structure for frequency counting."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "metadata": {},
   "outputs": [],
   "source": [
    "from collections import Counter\n",
    "\n",
    "class TreeClassifier(DecisionTree, ClassifierMixin):\n",
    "\n",
    "    def __init__(self, max_depth=10, criterion='maj_sum'):\n",
    "        super().__init__(max_depth)\n",
    "        self.criterion = criterion\n",
    "        \n",
    "    def fit(self, X, Y):\n",
    "        # For decision tree classifiers, there are some different ways to measure\n",
    "        # the homogeneity of subsets.\n",
    "        if self.criterion == 'maj_sum':\n",
    "            self.criterion_function = majority_sum_scorer\n",
    "        elif self.criterion == 'info_gain':\n",
    "            self.criterion_function = info_gain_scorer\n",
    "        elif self.criterion == 'gini':\n",
    "            self.criterion_function = gini_scorer\n",
    "        else:\n",
    "            raise Exception(f'Unknown criterion: {self.criterion}')\n",
    "        super().fit(X, Y)\n",
    "        self.classes_ = sorted(set(Y))\n",
    "\n",
    "    # Select a default value that is going to be used if we decide to make a leaf.\n",
    "    # We will select the most common value.\n",
    "    def get_default_value(self, Y):\n",
    "        self.class_distribution = Counter(Y)\n",
    "        return self.class_distribution.most_common(1)[0][0]\n",
    "    \n",
    "    # Checks whether a set of output values is homogeneous. In the classification case, \n",
    "    # this means that all output values are identical.\n",
    "    # We assume that we called get_default_value just before, so that we can access\n",
    "    # the class_distribution attribute. If the class distribution contains just one item,\n",
    "    # this means that the set is homogeneous.\n",
    "    def is_homogeneous(self, Y):\n",
    "        return len(self.class_distribution) == 1\n",
    "        \n",
    "    # Finds the best splitting point for a given feature. We'll keep frequency tables (Counters)\n",
    "    # for the upper and lower parts, and then compute the impurity criterion using these tables.\n",
    "    # In the end, we return a triple consisting of\n",
    "    # - the best score we found, according to the criterion we're using\n",
    "    # - the id of the feature\n",
    "    # - the threshold for the best split\n",
    "    def best_split(self, X, Y, feature):\n",
    "\n",
    "        # Create a list of input-output pairs, where we have sorted\n",
    "        # in ascending order by the input feature we're considering.\n",
    "        sorted_indices = np.argsort(X[:, feature])        \n",
    "        X_sorted = list(X[sorted_indices, feature])\n",
    "        Y_sorted = list(Y[sorted_indices])\n",
    "\n",
    "        n = len(Y)\n",
    "\n",
    "        # The frequency tables corresponding to the parts *before and including*\n",
    "        # and *after* the current element.\n",
    "        low_distr = Counter()\n",
    "        high_distr = Counter(Y)\n",
    "\n",
    "        # Keep track of the best result we've seen so far.\n",
    "        max_score = -np.inf\n",
    "        max_i = None\n",
    "\n",
    "        # Go through all the positions (excluding the last position).\n",
    "        for i in range(0, n-1):\n",
    "\n",
    "            # Input and output at the current position.\n",
    "            x_i = X_sorted[i]\n",
    "            y_i = Y_sorted[i]\n",
    "            \n",
    "            # Update the frequency tables.\n",
    "            low_distr[y_i] += 1\n",
    "            high_distr[y_i] -= 1\n",
    "\n",
    "            # If the input is equal to the input at the next position, we will\n",
    "            # not consider a split here.\n",
    "            #x_next = XY[i+1][0]\n",
    "            x_next = X_sorted[i+1]\n",
    "            if x_i == x_next:\n",
    "                continue\n",
    "\n",
    "            # Compute the homogeneity criterion for a split at this position.\n",
    "            score = self.criterion_function(i+1, low_distr, n-i-1, high_distr)\n",
    "\n",
    "            # If this is the best split, remember it.\n",
    "            if score > max_score:\n",
    "                max_score = score\n",
    "                max_i = i\n",
    "\n",
    "        # If we didn't find any split (meaning that all inputs are identical), return\n",
    "        # a dummy value.\n",
    "        if max_i is None:\n",
    "            return -np.inf, None, None\n",
    "\n",
    "        # Otherwise, return the best split we found and its score.\n",
    "        split_point = 0.5*(X_sorted[max_i] + X_sorted[max_i+1])\n",
    "        return max_score, feature, split_point\n"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Here, we define the various criteria we can use to find the \"quality\" of a split in terms of how homogeneous the subsets are. See the reading material for the mathematical definitions of these criteria."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "metadata": {},
   "outputs": [],
   "source": [
    "def majority_sum_scorer(n_low, low_distr, n_high, high_distr):\n",
    "    maj_sum_low = low_distr.most_common(1)[0][1]\n",
    "    maj_sum_high = high_distr.most_common(1)[0][1]\n",
    "    return maj_sum_low + maj_sum_high\n",
    "    \n",
    "def entropy(distr):\n",
    "    n = sum(distr.values())\n",
    "    ps = [n_i/n for n_i in distr.values()]\n",
    "    return -sum(p*np.log2(p) if p > 0 else 0 for p in ps)\n",
    "\n",
    "def info_gain_scorer(n_low, low_distr, n_high, high_distr):\n",
    "    return -(n_low*entropy(low_distr)+n_high*entropy(high_distr))/(n_low+n_high)\n",
    "\n",
    "def gini_impurity(distr):\n",
    "    n = sum(distr.values())\n",
    "    ps = [n_i/n for n_i in distr.values()]\n",
    "    return 1-sum(p**2 for p in ps)\n",
    "    \n",
    "def gini_scorer(n_low, low_distr, n_high, high_distr):\n",
    "    return -(n_low*gini_impurity(low_distr)+n_high*gini_impurity(high_distr))/(n_low+n_high)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Iris example, continued"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "Now, let's train decision trees for the iris dataset, using our own implementation (and not the one that's included in scikit-learn, which we used previously).\n",
    "\n",
    "We can investigate how the `max_depth` hyperparameter affects the decision boundary. Generally, if we allow deeper trees, we can express more complex boundaries. This allows us to handle more \"difficult\" datasets but also introduces the risk of overfitting.\n",
    "\n",
    "(If you didn't install graphviz, you can just remove the line that includes `draw_tree`.)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 27,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/svg+xml": [
       "<?xml version=\"1.0\" encoding=\"UTF-8\" standalone=\"no\"?>\n",
       "<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n",
       " \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n",
       "<!-- Generated by graphviz version 2.40.1 (20161225.0304)\n",
       " -->\n",
       "<!-- Title: %3 Pages: 1 -->\n",
       "<svg width=\"249pt\" height=\"218pt\"\n",
       " viewBox=\"0.00 0.00 248.74 218.00\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n",
       "<g id=\"graph0\" class=\"graph\" transform=\"scale(1 1) rotate(0) translate(4 214)\">\n",
       "<title>%3</title>\n",
       "<polygon fill=\"#ffffff\" stroke=\"transparent\" points=\"-4,4 -4,-214 244.7434,-214 244.7434,4 -4,4\"/>\n",
       "<!-- 0 -->\n",
       "<g id=\"node1\" class=\"node\">\n",
       "<title>0</title>\n",
       "<ellipse fill=\"#d3d3d3\" stroke=\"#000000\" cx=\"32.4971\" cy=\"-105\" rx=\"32.4942\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"32.4971\" y=\"-101.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">setosa</text>\n",
       "</g>\n",
       "<!-- 1 -->\n",
       "<g id=\"node2\" class=\"node\">\n",
       "<title>1</title>\n",
       "<ellipse fill=\"#d3d3d3\" stroke=\"#000000\" cx=\"92.4971\" cy=\"-18\" rx=\"46.2923\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"92.4971\" y=\"-14.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">versicolor</text>\n",
       "</g>\n",
       "<!-- 2 -->\n",
       "<g id=\"node3\" class=\"node\">\n",
       "<title>2</title>\n",
       "<ellipse fill=\"#d3d3d3\" stroke=\"#000000\" cx=\"198.4971\" cy=\"-18\" rx=\"42.4939\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"198.4971\" y=\"-14.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">virginica</text>\n",
       "</g>\n",
       "<!-- 3 -->\n",
       "<g id=\"node4\" class=\"node\">\n",
       "<title>3</title>\n",
       "<path fill=\"#ffff00\" stroke=\"#000000\" d=\"M196.4971,-123C196.4971,-123 94.4971,-123 94.4971,-123 88.4971,-123 82.4971,-117 82.4971,-111 82.4971,-111 82.4971,-99 82.4971,-99 82.4971,-93 88.4971,-87 94.4971,-87 94.4971,-87 196.4971,-87 196.4971,-87 202.4971,-87 208.4971,-93 208.4971,-99 208.4971,-99 208.4971,-111 208.4971,-111 208.4971,-117 202.4971,-123 196.4971,-123\"/>\n",
       "<text text-anchor=\"middle\" x=\"145.4971\" y=\"-101.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">petal_width &gt; 1.65?</text>\n",
       "</g>\n",
       "<!-- 3&#45;&gt;1 -->\n",
       "<g id=\"edge1\" class=\"edge\">\n",
       "<title>3&#45;&gt;1</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M134.5155,-86.9735C127.0041,-74.6435 116.9244,-58.0975 108.5202,-44.3021\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"111.475,-42.4249 103.2833,-35.7057 105.4969,-46.0667 111.475,-42.4249\"/>\n",
       "<text text-anchor=\"middle\" x=\"135.9971\" y=\"-57.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">False</text>\n",
       "</g>\n",
       "<!-- 3&#45;&gt;2 -->\n",
       "<g id=\"edge2\" class=\"edge\">\n",
       "<title>3&#45;&gt;2</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M156.4788,-86.9735C163.9901,-74.6435 174.0699,-58.0975 182.474,-44.3021\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"185.4973,-46.0667 187.7109,-35.7057 179.5192,-42.4249 185.4973,-46.0667\"/>\n",
       "<text text-anchor=\"middle\" x=\"187.4971\" y=\"-57.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">True</text>\n",
       "</g>\n",
       "<!-- 4 -->\n",
       "<g id=\"node5\" class=\"node\">\n",
       "<title>4</title>\n",
       "<path fill=\"#ffff00\" stroke=\"#000000\" d=\"M120.9971,-210C120.9971,-210 25.9971,-210 25.9971,-210 19.9971,-210 13.9971,-204 13.9971,-198 13.9971,-198 13.9971,-186 13.9971,-186 13.9971,-180 19.9971,-174 25.9971,-174 25.9971,-174 120.9971,-174 120.9971,-174 126.9971,-174 132.9971,-180 132.9971,-186 132.9971,-186 132.9971,-198 132.9971,-198 132.9971,-204 126.9971,-210 120.9971,-210\"/>\n",
       "<text text-anchor=\"middle\" x=\"73.4971\" y=\"-188.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">petal_width &gt; 0.8?</text>\n",
       "</g>\n",
       "<!-- 4&#45;&gt;0 -->\n",
       "<g id=\"edge3\" class=\"edge\">\n",
       "<title>4&#45;&gt;0</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M65.0019,-173.9735C59.3019,-161.8784 51.69,-145.7263 45.2653,-132.0934\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"48.2703,-130.2595 40.8412,-122.7057 41.9382,-133.2436 48.2703,-130.2595\"/>\n",
       "<text text-anchor=\"middle\" x=\"69.9971\" y=\"-144.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">False</text>\n",
       "</g>\n",
       "<!-- 4&#45;&gt;3 -->\n",
       "<g id=\"edge4\" class=\"edge\">\n",
       "<title>4&#45;&gt;3</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M88.4156,-173.9735C98.7452,-161.4919 112.6503,-144.6899 124.1501,-130.7944\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"126.9185,-132.9388 130.5978,-123.0034 121.5257,-128.4758 126.9185,-132.9388\"/>\n",
       "<text text-anchor=\"middle\" x=\"126.4971\" y=\"-144.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">True</text>\n",
       "</g>\n",
       "</g>\n",
       "</svg>\n"
      ],
      "text/plain": [
       "<graphviz.dot.Digraph at 0x7fb7d0754510>"
      ]
     },
     "execution_count": 27,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "cls = TreeClassifier(max_depth=2)\n",
    "cls.fit(X2, Y)\n",
    "cls.draw_tree()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 28,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/svg+xml": [
       "<?xml version=\"1.0\" encoding=\"utf-8\" standalone=\"no\"?>\n",
       "<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n",
       "  \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n",
       "<!-- Created with matplotlib (https://matplotlib.org/) -->\n",
       "<svg height=\"316.534063pt\" version=\"1.1\" viewBox=\"0 0 328.359844 316.534063\" width=\"328.359844pt\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n",
       " <defs>\n",
       "  <style type=\"text/css\">\n",
       "*{stroke-linecap:butt;stroke-linejoin:round;}\n",
       "  </style>\n",
       " </defs>\n",
       " <g id=\"figure_1\">\n",
       "  <g id=\"patch_1\">\n",
       "   <path d=\"M 0 316.534063 \n",
       "L 328.359844 316.534063 \n",
       "L 328.359844 0 \n",
       "L 0 0 \n",
       "z\n",
       "\" style=\"fill:none;\"/>\n",
       "  </g>\n",
       "  <g id=\"axes_1\">\n",
       "   <g id=\"patch_2\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 321.159844 279 \n",
       "L 321.159844 7.2 \n",
       "L 42.159844 7.2 \n",
       "z\n",
       "\" style=\"fill:#eaeaf2;\"/>\n",
       "   </g>\n",
       "   <g id=\"matplotlib.axis_1\">\n",
       "    <g id=\"xtick_1\">\n",
       "     <g id=\"line2d_1\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 52.493177 279 \n",
       "L 52.493177 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_2\"/>\n",
       "     <g id=\"text_1\">\n",
       "      <!-- 1 -->\n",
       "      <defs>\n",
       "       <path d=\"M 37.25 0 \n",
       "L 28.46875 0 \n",
       "L 28.46875 56 \n",
       "Q 25.296875 52.984375 20.140625 49.953125 \n",
       "Q 14.984375 46.921875 10.890625 45.40625 \n",
       "L 10.890625 53.90625 \n",
       "Q 18.265625 57.375 23.78125 62.296875 \n",
       "Q 29.296875 67.234375 31.59375 71.875 \n",
       "L 37.25 71.875 \n",
       "z\n",
       "\" id=\"ArialMT-49\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(49.712708 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_2\">\n",
       "     <g id=\"line2d_3\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 96.278488 279 \n",
       "L 96.278488 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_4\"/>\n",
       "     <g id=\"text_2\">\n",
       "      <!-- 2 -->\n",
       "      <defs>\n",
       "       <path d=\"M 50.34375 8.453125 \n",
       "L 50.34375 0 \n",
       "L 3.03125 0 \n",
       "Q 2.9375 3.171875 4.046875 6.109375 \n",
       "Q 5.859375 10.9375 9.828125 15.625 \n",
       "Q 13.8125 20.3125 21.34375 26.46875 \n",
       "Q 33.015625 36.03125 37.109375 41.625 \n",
       "Q 41.21875 47.21875 41.21875 52.203125 \n",
       "Q 41.21875 57.421875 37.46875 61 \n",
       "Q 33.734375 64.59375 27.734375 64.59375 \n",
       "Q 21.390625 64.59375 17.578125 60.78125 \n",
       "Q 13.765625 56.984375 13.71875 50.25 \n",
       "L 4.6875 51.171875 \n",
       "Q 5.609375 61.28125 11.65625 66.578125 \n",
       "Q 17.71875 71.875 27.9375 71.875 \n",
       "Q 38.234375 71.875 44.234375 66.15625 \n",
       "Q 50.25 60.453125 50.25 52 \n",
       "Q 50.25 47.703125 48.484375 43.546875 \n",
       "Q 46.734375 39.40625 42.65625 34.8125 \n",
       "Q 38.578125 30.21875 29.109375 22.21875 \n",
       "Q 21.1875 15.578125 18.9375 13.203125 \n",
       "Q 16.703125 10.84375 15.234375 8.453125 \n",
       "z\n",
       "\" id=\"ArialMT-50\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(93.498019 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_3\">\n",
       "     <g id=\"line2d_5\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 140.063799 279 \n",
       "L 140.063799 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_6\"/>\n",
       "     <g id=\"text_3\">\n",
       "      <!-- 3 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.203125 18.890625 \n",
       "L 12.984375 20.0625 \n",
       "Q 14.5 12.59375 18.140625 9.296875 \n",
       "Q 21.78125 6 27 6 \n",
       "Q 33.203125 6 37.46875 10.296875 \n",
       "Q 41.75 14.59375 41.75 20.953125 \n",
       "Q 41.75 27 37.796875 30.921875 \n",
       "Q 33.84375 34.859375 27.734375 34.859375 \n",
       "Q 25.25 34.859375 21.53125 33.890625 \n",
       "L 22.515625 41.609375 \n",
       "Q 23.390625 41.5 23.921875 41.5 \n",
       "Q 29.546875 41.5 34.03125 44.421875 \n",
       "Q 38.53125 47.359375 38.53125 53.46875 \n",
       "Q 38.53125 58.296875 35.25 61.46875 \n",
       "Q 31.984375 64.65625 26.8125 64.65625 \n",
       "Q 21.6875 64.65625 18.265625 61.421875 \n",
       "Q 14.84375 58.203125 13.875 51.765625 \n",
       "L 5.078125 53.328125 \n",
       "Q 6.6875 62.15625 12.390625 67.015625 \n",
       "Q 18.109375 71.875 26.609375 71.875 \n",
       "Q 32.46875 71.875 37.390625 69.359375 \n",
       "Q 42.328125 66.84375 44.9375 62.5 \n",
       "Q 47.5625 58.15625 47.5625 53.265625 \n",
       "Q 47.5625 48.640625 45.0625 44.828125 \n",
       "Q 42.578125 41.015625 37.703125 38.765625 \n",
       "Q 44.046875 37.3125 47.5625 32.6875 \n",
       "Q 51.078125 28.078125 51.078125 21.140625 \n",
       "Q 51.078125 11.765625 44.234375 5.25 \n",
       "Q 37.40625 -1.265625 26.953125 -1.265625 \n",
       "Q 17.53125 -1.265625 11.296875 4.34375 \n",
       "Q 5.078125 9.96875 4.203125 18.890625 \n",
       "z\n",
       "\" id=\"ArialMT-51\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(137.28333 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-51\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_4\">\n",
       "     <g id=\"line2d_7\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 183.849109 279 \n",
       "L 183.849109 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_8\"/>\n",
       "     <g id=\"text_4\">\n",
       "      <!-- 4 -->\n",
       "      <defs>\n",
       "       <path d=\"M 32.328125 0 \n",
       "L 32.328125 17.140625 \n",
       "L 1.265625 17.140625 \n",
       "L 1.265625 25.203125 \n",
       "L 33.9375 71.578125 \n",
       "L 41.109375 71.578125 \n",
       "L 41.109375 25.203125 \n",
       "L 50.78125 25.203125 \n",
       "L 50.78125 17.140625 \n",
       "L 41.109375 17.140625 \n",
       "L 41.109375 0 \n",
       "z\n",
       "M 32.328125 25.203125 \n",
       "L 32.328125 57.46875 \n",
       "L 9.90625 25.203125 \n",
       "z\n",
       "\" id=\"ArialMT-52\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(181.068641 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-52\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_5\">\n",
       "     <g id=\"line2d_9\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 227.63442 279 \n",
       "L 227.63442 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_10\"/>\n",
       "     <g id=\"text_5\">\n",
       "      <!-- 5 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.15625 18.75 \n",
       "L 13.375 19.53125 \n",
       "Q 14.40625 12.796875 18.140625 9.390625 \n",
       "Q 21.875 6 27.15625 6 \n",
       "Q 33.5 6 37.890625 10.78125 \n",
       "Q 42.28125 15.578125 42.28125 23.484375 \n",
       "Q 42.28125 31 38.0625 35.34375 \n",
       "Q 33.84375 39.703125 27 39.703125 \n",
       "Q 22.75 39.703125 19.328125 37.765625 \n",
       "Q 15.921875 35.84375 13.96875 32.765625 \n",
       "L 5.71875 33.84375 \n",
       "L 12.640625 70.609375 \n",
       "L 48.25 70.609375 \n",
       "L 48.25 62.203125 \n",
       "L 19.671875 62.203125 \n",
       "L 15.828125 42.96875 \n",
       "Q 22.265625 47.46875 29.34375 47.46875 \n",
       "Q 38.71875 47.46875 45.15625 40.96875 \n",
       "Q 51.609375 34.46875 51.609375 24.265625 \n",
       "Q 51.609375 14.546875 45.953125 7.46875 \n",
       "Q 39.0625 -1.21875 27.15625 -1.21875 \n",
       "Q 17.390625 -1.21875 11.203125 4.25 \n",
       "Q 5.03125 9.71875 4.15625 18.75 \n",
       "z\n",
       "\" id=\"ArialMT-53\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(224.853951 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_6\">\n",
       "     <g id=\"line2d_11\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 271.419731 279 \n",
       "L 271.419731 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_12\"/>\n",
       "     <g id=\"text_6\">\n",
       "      <!-- 6 -->\n",
       "      <defs>\n",
       "       <path d=\"M 49.75 54.046875 \n",
       "L 41.015625 53.375 \n",
       "Q 39.84375 58.546875 37.703125 60.890625 \n",
       "Q 34.125 64.65625 28.90625 64.65625 \n",
       "Q 24.703125 64.65625 21.53125 62.3125 \n",
       "Q 17.390625 59.28125 14.984375 53.46875 \n",
       "Q 12.59375 47.65625 12.5 36.921875 \n",
       "Q 15.671875 41.75 20.265625 44.09375 \n",
       "Q 24.859375 46.4375 29.890625 46.4375 \n",
       "Q 38.671875 46.4375 44.84375 39.96875 \n",
       "Q 51.03125 33.5 51.03125 23.25 \n",
       "Q 51.03125 16.5 48.125 10.71875 \n",
       "Q 45.21875 4.9375 40.140625 1.859375 \n",
       "Q 35.0625 -1.21875 28.609375 -1.21875 \n",
       "Q 17.625 -1.21875 10.6875 6.859375 \n",
       "Q 3.765625 14.9375 3.765625 33.5 \n",
       "Q 3.765625 54.25 11.421875 63.671875 \n",
       "Q 18.109375 71.875 29.4375 71.875 \n",
       "Q 37.890625 71.875 43.28125 67.140625 \n",
       "Q 48.6875 62.40625 49.75 54.046875 \n",
       "z\n",
       "M 13.875 23.1875 \n",
       "Q 13.875 18.65625 15.796875 14.5 \n",
       "Q 17.71875 10.359375 21.1875 8.171875 \n",
       "Q 24.65625 6 28.46875 6 \n",
       "Q 34.03125 6 38.03125 10.484375 \n",
       "Q 42.046875 14.984375 42.046875 22.703125 \n",
       "Q 42.046875 30.125 38.078125 34.390625 \n",
       "Q 34.125 38.671875 28.125 38.671875 \n",
       "Q 22.171875 38.671875 18.015625 34.390625 \n",
       "Q 13.875 30.125 13.875 23.1875 \n",
       "z\n",
       "\" id=\"ArialMT-54\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(268.639262 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-54\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_7\">\n",
       "     <g id=\"line2d_13\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 315.205041 279 \n",
       "L 315.205041 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_14\"/>\n",
       "     <g id=\"text_7\">\n",
       "      <!-- 7 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.734375 62.203125 \n",
       "L 4.734375 70.65625 \n",
       "L 51.078125 70.65625 \n",
       "L 51.078125 63.8125 \n",
       "Q 44.234375 56.546875 37.515625 44.484375 \n",
       "Q 30.8125 32.421875 27.15625 19.671875 \n",
       "Q 24.515625 10.6875 23.78125 0 \n",
       "L 14.75 0 \n",
       "Q 14.890625 8.453125 18.0625 20.40625 \n",
       "Q 21.234375 32.375 27.171875 43.484375 \n",
       "Q 33.109375 54.59375 39.796875 62.203125 \n",
       "z\n",
       "\" id=\"ArialMT-55\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(312.424573 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-55\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_8\">\n",
       "     <!-- petal_length -->\n",
       "     <defs>\n",
       "      <path d=\"M 6.59375 -19.875 \n",
       "L 6.59375 51.859375 \n",
       "L 14.59375 51.859375 \n",
       "L 14.59375 45.125 \n",
       "Q 17.4375 49.078125 21 51.046875 \n",
       "Q 24.5625 53.03125 29.640625 53.03125 \n",
       "Q 36.28125 53.03125 41.359375 49.609375 \n",
       "Q 46.4375 46.1875 49.015625 39.953125 \n",
       "Q 51.609375 33.734375 51.609375 26.3125 \n",
       "Q 51.609375 18.359375 48.75 11.984375 \n",
       "Q 45.90625 5.609375 40.453125 2.21875 \n",
       "Q 35.015625 -1.171875 29 -1.171875 \n",
       "Q 24.609375 -1.171875 21.109375 0.6875 \n",
       "Q 17.625 2.546875 15.375 5.375 \n",
       "L 15.375 -19.875 \n",
       "z\n",
       "M 14.546875 25.640625 \n",
       "Q 14.546875 15.625 18.59375 10.84375 \n",
       "Q 22.65625 6.0625 28.421875 6.0625 \n",
       "Q 34.28125 6.0625 38.453125 11.015625 \n",
       "Q 42.625 15.96875 42.625 26.375 \n",
       "Q 42.625 36.28125 38.546875 41.203125 \n",
       "Q 34.46875 46.140625 28.8125 46.140625 \n",
       "Q 23.1875 46.140625 18.859375 40.890625 \n",
       "Q 14.546875 35.640625 14.546875 25.640625 \n",
       "z\n",
       "\" id=\"ArialMT-112\"/>\n",
       "      <path d=\"M 42.09375 16.703125 \n",
       "L 51.171875 15.578125 \n",
       "Q 49.03125 7.625 43.21875 3.21875 \n",
       "Q 37.40625 -1.171875 28.375 -1.171875 \n",
       "Q 17 -1.171875 10.328125 5.828125 \n",
       "Q 3.65625 12.84375 3.65625 25.484375 \n",
       "Q 3.65625 38.578125 10.390625 45.796875 \n",
       "Q 17.140625 53.03125 27.875 53.03125 \n",
       "Q 38.28125 53.03125 44.875 45.953125 \n",
       "Q 51.46875 38.875 51.46875 26.03125 \n",
       "Q 51.46875 25.25 51.421875 23.6875 \n",
       "L 12.75 23.6875 \n",
       "Q 13.234375 15.140625 17.578125 10.59375 \n",
       "Q 21.921875 6.0625 28.421875 6.0625 \n",
       "Q 33.25 6.0625 36.671875 8.59375 \n",
       "Q 40.09375 11.140625 42.09375 16.703125 \n",
       "z\n",
       "M 13.234375 30.90625 \n",
       "L 42.1875 30.90625 \n",
       "Q 41.609375 37.453125 38.875 40.71875 \n",
       "Q 34.671875 45.796875 27.984375 45.796875 \n",
       "Q 21.921875 45.796875 17.796875 41.75 \n",
       "Q 13.671875 37.703125 13.234375 30.90625 \n",
       "z\n",
       "\" id=\"ArialMT-101\"/>\n",
       "      <path d=\"M 25.78125 7.859375 \n",
       "L 27.046875 0.09375 \n",
       "Q 23.34375 -0.6875 20.40625 -0.6875 \n",
       "Q 15.625 -0.6875 12.984375 0.828125 \n",
       "Q 10.359375 2.34375 9.28125 4.8125 \n",
       "Q 8.203125 7.28125 8.203125 15.1875 \n",
       "L 8.203125 45.015625 \n",
       "L 1.765625 45.015625 \n",
       "L 1.765625 51.859375 \n",
       "L 8.203125 51.859375 \n",
       "L 8.203125 64.703125 \n",
       "L 16.9375 69.96875 \n",
       "L 16.9375 51.859375 \n",
       "L 25.78125 51.859375 \n",
       "L 25.78125 45.015625 \n",
       "L 16.9375 45.015625 \n",
       "L 16.9375 14.703125 \n",
       "Q 16.9375 10.9375 17.40625 9.859375 \n",
       "Q 17.875 8.796875 18.921875 8.15625 \n",
       "Q 19.96875 7.515625 21.921875 7.515625 \n",
       "Q 23.390625 7.515625 25.78125 7.859375 \n",
       "z\n",
       "\" id=\"ArialMT-116\"/>\n",
       "      <path d=\"M 40.4375 6.390625 \n",
       "Q 35.546875 2.25 31.03125 0.53125 \n",
       "Q 26.515625 -1.171875 21.34375 -1.171875 \n",
       "Q 12.796875 -1.171875 8.203125 3 \n",
       "Q 3.609375 7.171875 3.609375 13.671875 \n",
       "Q 3.609375 17.484375 5.34375 20.625 \n",
       "Q 7.078125 23.78125 9.890625 25.6875 \n",
       "Q 12.703125 27.59375 16.21875 28.5625 \n",
       "Q 18.796875 29.25 24.03125 29.890625 \n",
       "Q 34.671875 31.15625 39.703125 32.90625 \n",
       "Q 39.75 34.71875 39.75 35.203125 \n",
       "Q 39.75 40.578125 37.25 42.78125 \n",
       "Q 33.890625 45.75 27.25 45.75 \n",
       "Q 21.046875 45.75 18.09375 43.578125 \n",
       "Q 15.140625 41.40625 13.71875 35.890625 \n",
       "L 5.125 37.0625 \n",
       "Q 6.296875 42.578125 8.984375 45.96875 \n",
       "Q 11.671875 49.359375 16.75 51.1875 \n",
       "Q 21.828125 53.03125 28.515625 53.03125 \n",
       "Q 35.15625 53.03125 39.296875 51.46875 \n",
       "Q 43.453125 49.90625 45.40625 47.53125 \n",
       "Q 47.359375 45.171875 48.140625 41.546875 \n",
       "Q 48.578125 39.3125 48.578125 33.453125 \n",
       "L 48.578125 21.734375 \n",
       "Q 48.578125 9.46875 49.140625 6.21875 \n",
       "Q 49.703125 2.984375 51.375 0 \n",
       "L 42.1875 0 \n",
       "Q 40.828125 2.734375 40.4375 6.390625 \n",
       "z\n",
       "M 39.703125 26.03125 \n",
       "Q 34.90625 24.078125 25.34375 22.703125 \n",
       "Q 19.921875 21.921875 17.671875 20.9375 \n",
       "Q 15.4375 19.96875 14.203125 18.09375 \n",
       "Q 12.984375 16.21875 12.984375 13.921875 \n",
       "Q 12.984375 10.40625 15.640625 8.0625 \n",
       "Q 18.3125 5.71875 23.4375 5.71875 \n",
       "Q 28.515625 5.71875 32.46875 7.9375 \n",
       "Q 36.421875 10.15625 38.28125 14.015625 \n",
       "Q 39.703125 17 39.703125 22.796875 \n",
       "z\n",
       "\" id=\"ArialMT-97\"/>\n",
       "      <path d=\"M 6.390625 0 \n",
       "L 6.390625 71.578125 \n",
       "L 15.1875 71.578125 \n",
       "L 15.1875 0 \n",
       "z\n",
       "\" id=\"ArialMT-108\"/>\n",
       "      <path d=\"M -1.515625 -19.875 \n",
       "L -1.515625 -13.53125 \n",
       "L 56.734375 -13.53125 \n",
       "L 56.734375 -19.875 \n",
       "z\n",
       "\" id=\"ArialMT-95\"/>\n",
       "      <path d=\"M 6.59375 0 \n",
       "L 6.59375 51.859375 \n",
       "L 14.5 51.859375 \n",
       "L 14.5 44.484375 \n",
       "Q 20.21875 53.03125 31 53.03125 \n",
       "Q 35.6875 53.03125 39.625 51.34375 \n",
       "Q 43.5625 49.65625 45.515625 46.921875 \n",
       "Q 47.46875 44.1875 48.25 40.4375 \n",
       "Q 48.734375 37.984375 48.734375 31.890625 \n",
       "L 48.734375 0 \n",
       "L 39.9375 0 \n",
       "L 39.9375 31.546875 \n",
       "Q 39.9375 36.921875 38.90625 39.578125 \n",
       "Q 37.890625 42.234375 35.28125 43.8125 \n",
       "Q 32.671875 45.40625 29.15625 45.40625 \n",
       "Q 23.53125 45.40625 19.453125 41.84375 \n",
       "Q 15.375 38.28125 15.375 28.328125 \n",
       "L 15.375 0 \n",
       "z\n",
       "\" id=\"ArialMT-110\"/>\n",
       "      <path d=\"M 4.984375 -4.296875 \n",
       "L 13.53125 -5.5625 \n",
       "Q 14.0625 -9.515625 16.5 -11.328125 \n",
       "Q 19.78125 -13.765625 25.4375 -13.765625 \n",
       "Q 31.546875 -13.765625 34.859375 -11.328125 \n",
       "Q 38.1875 -8.890625 39.359375 -4.5 \n",
       "Q 40.046875 -1.8125 39.984375 6.78125 \n",
       "Q 34.234375 0 25.640625 0 \n",
       "Q 14.9375 0 9.078125 7.71875 \n",
       "Q 3.21875 15.4375 3.21875 26.21875 \n",
       "Q 3.21875 33.640625 5.90625 39.90625 \n",
       "Q 8.59375 46.1875 13.6875 49.609375 \n",
       "Q 18.796875 53.03125 25.6875 53.03125 \n",
       "Q 34.859375 53.03125 40.828125 45.609375 \n",
       "L 40.828125 51.859375 \n",
       "L 48.921875 51.859375 \n",
       "L 48.921875 7.03125 \n",
       "Q 48.921875 -5.078125 46.453125 -10.125 \n",
       "Q 44 -15.1875 38.640625 -18.109375 \n",
       "Q 33.296875 -21.046875 25.484375 -21.046875 \n",
       "Q 16.21875 -21.046875 10.5 -16.875 \n",
       "Q 4.78125 -12.703125 4.984375 -4.296875 \n",
       "z\n",
       "M 12.25 26.859375 \n",
       "Q 12.25 16.65625 16.296875 11.96875 \n",
       "Q 20.359375 7.28125 26.46875 7.28125 \n",
       "Q 32.515625 7.28125 36.609375 11.9375 \n",
       "Q 40.71875 16.609375 40.71875 26.5625 \n",
       "Q 40.71875 36.078125 36.5 40.90625 \n",
       "Q 32.28125 45.75 26.3125 45.75 \n",
       "Q 20.453125 45.75 16.34375 40.984375 \n",
       "Q 12.25 36.234375 12.25 26.859375 \n",
       "z\n",
       "\" id=\"ArialMT-103\"/>\n",
       "      <path d=\"M 6.59375 0 \n",
       "L 6.59375 71.578125 \n",
       "L 15.375 71.578125 \n",
       "L 15.375 45.90625 \n",
       "Q 21.53125 53.03125 30.90625 53.03125 \n",
       "Q 36.671875 53.03125 40.921875 50.75 \n",
       "Q 45.171875 48.484375 47 44.484375 \n",
       "Q 48.828125 40.484375 48.828125 32.859375 \n",
       "L 48.828125 0 \n",
       "L 40.046875 0 \n",
       "L 40.046875 32.859375 \n",
       "Q 40.046875 39.453125 37.1875 42.453125 \n",
       "Q 34.328125 45.453125 29.109375 45.453125 \n",
       "Q 25.203125 45.453125 21.75 43.421875 \n",
       "Q 18.3125 41.40625 16.84375 37.9375 \n",
       "Q 15.375 34.46875 15.375 28.375 \n",
       "L 15.375 0 \n",
       "z\n",
       "\" id=\"ArialMT-104\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(151.691719 307.018906)scale(0.11 -0.11)\">\n",
       "      <use xlink:href=\"#ArialMT-112\"/>\n",
       "      <use x=\"55.615234\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"111.230469\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"139.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "      <use x=\"194.628906\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"216.845703\" xlink:href=\"#ArialMT-95\"/>\n",
       "      <use x=\"272.460938\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"294.677734\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"350.292969\" xlink:href=\"#ArialMT-110\"/>\n",
       "      <use x=\"405.908203\" xlink:href=\"#ArialMT-103\"/>\n",
       "      <use x=\"461.523438\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"489.306641\" xlink:href=\"#ArialMT-104\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"matplotlib.axis_2\">\n",
       "    <g id=\"ytick_1\">\n",
       "     <g id=\"line2d_15\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 226.988889 \n",
       "L 321.159844 226.988889 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_16\"/>\n",
       "     <g id=\"text_9\">\n",
       "      <!-- 0.5 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.15625 35.296875 \n",
       "Q 4.15625 48 6.765625 55.734375 \n",
       "Q 9.375 63.484375 14.515625 67.671875 \n",
       "Q 19.671875 71.875 27.484375 71.875 \n",
       "Q 33.25 71.875 37.59375 69.546875 \n",
       "Q 41.9375 67.234375 44.765625 62.859375 \n",
       "Q 47.609375 58.5 49.21875 52.21875 \n",
       "Q 50.828125 45.953125 50.828125 35.296875 \n",
       "Q 50.828125 22.703125 48.234375 14.96875 \n",
       "Q 45.65625 7.234375 40.5 3 \n",
       "Q 35.359375 -1.21875 27.484375 -1.21875 \n",
       "Q 17.140625 -1.21875 11.234375 6.203125 \n",
       "Q 4.15625 15.140625 4.15625 35.296875 \n",
       "z\n",
       "M 13.1875 35.296875 \n",
       "Q 13.1875 17.671875 17.3125 11.828125 \n",
       "Q 21.4375 6 27.484375 6 \n",
       "Q 33.546875 6 37.671875 11.859375 \n",
       "Q 41.796875 17.71875 41.796875 35.296875 \n",
       "Q 41.796875 52.984375 37.671875 58.78125 \n",
       "Q 33.546875 64.59375 27.390625 64.59375 \n",
       "Q 21.34375 64.59375 17.71875 59.46875 \n",
       "Q 13.1875 52.9375 13.1875 35.296875 \n",
       "z\n",
       "\" id=\"ArialMT-48\"/>\n",
       "       <path d=\"M 9.078125 0 \n",
       "L 9.078125 10.015625 \n",
       "L 19.09375 10.015625 \n",
       "L 19.09375 0 \n",
       "z\n",
       "\" id=\"ArialMT-46\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 230.567795)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-48\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_2\">\n",
       "     <g id=\"line2d_17\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 174.558333 \n",
       "L 321.159844 174.558333 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_18\"/>\n",
       "     <g id=\"text_10\">\n",
       "      <!-- 1.0 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 178.13724)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_3\">\n",
       "     <g id=\"line2d_19\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 122.127778 \n",
       "L 321.159844 122.127778 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_20\"/>\n",
       "     <g id=\"text_11\">\n",
       "      <!-- 1.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 125.706684)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_4\">\n",
       "     <g id=\"line2d_21\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 69.697222 \n",
       "L 321.159844 69.697222 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_22\"/>\n",
       "     <g id=\"text_12\">\n",
       "      <!-- 2.0 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 73.276128)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_5\">\n",
       "     <g id=\"line2d_23\">\n",
       "      <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 17.266667 \n",
       "L 321.159844 17.266667 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_24\"/>\n",
       "     <g id=\"text_13\">\n",
       "      <!-- 2.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 20.845573)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_14\">\n",
       "     <!-- petal_width -->\n",
       "     <defs>\n",
       "      <path d=\"M 16.15625 0 \n",
       "L 0.296875 51.859375 \n",
       "L 9.375 51.859375 \n",
       "L 17.625 21.921875 \n",
       "L 20.703125 10.796875 \n",
       "Q 20.90625 11.625 23.390625 21.484375 \n",
       "L 31.640625 51.859375 \n",
       "L 40.671875 51.859375 \n",
       "L 48.4375 21.78125 \n",
       "L 51.03125 11.859375 \n",
       "L 54 21.875 \n",
       "L 62.890625 51.859375 \n",
       "L 71.4375 51.859375 \n",
       "L 55.21875 0 \n",
       "L 46.09375 0 \n",
       "L 37.84375 31.0625 \n",
       "L 35.84375 39.890625 \n",
       "L 25.34375 0 \n",
       "z\n",
       "\" id=\"ArialMT-119\"/>\n",
       "      <path d=\"M 6.640625 61.46875 \n",
       "L 6.640625 71.578125 \n",
       "L 15.4375 71.578125 \n",
       "L 15.4375 61.46875 \n",
       "z\n",
       "M 6.640625 0 \n",
       "L 6.640625 51.859375 \n",
       "L 15.4375 51.859375 \n",
       "L 15.4375 0 \n",
       "z\n",
       "\" id=\"ArialMT-105\"/>\n",
       "      <path d=\"M 40.234375 0 \n",
       "L 40.234375 6.546875 \n",
       "Q 35.296875 -1.171875 25.734375 -1.171875 \n",
       "Q 19.53125 -1.171875 14.328125 2.25 \n",
       "Q 9.125 5.671875 6.265625 11.796875 \n",
       "Q 3.421875 17.921875 3.421875 25.875 \n",
       "Q 3.421875 33.640625 6 39.96875 \n",
       "Q 8.59375 46.296875 13.765625 49.65625 \n",
       "Q 18.953125 53.03125 25.34375 53.03125 \n",
       "Q 30.03125 53.03125 33.6875 51.046875 \n",
       "Q 37.359375 49.078125 39.65625 45.90625 \n",
       "L 39.65625 71.578125 \n",
       "L 48.390625 71.578125 \n",
       "L 48.390625 0 \n",
       "z\n",
       "M 12.453125 25.875 \n",
       "Q 12.453125 15.921875 16.640625 10.984375 \n",
       "Q 20.84375 6.0625 26.5625 6.0625 \n",
       "Q 32.328125 6.0625 36.34375 10.765625 \n",
       "Q 40.375 15.484375 40.375 25.140625 \n",
       "Q 40.375 35.796875 36.265625 40.765625 \n",
       "Q 32.171875 45.75 26.171875 45.75 \n",
       "Q 20.3125 45.75 16.375 40.96875 \n",
       "Q 12.453125 36.1875 12.453125 25.875 \n",
       "z\n",
       "\" id=\"ArialMT-100\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(15.073594 170.923125)rotate(-90)scale(0.11 -0.11)\">\n",
       "      <use xlink:href=\"#ArialMT-112\"/>\n",
       "      <use x=\"55.615234\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"111.230469\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"139.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "      <use x=\"194.628906\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"216.845703\" xlink:href=\"#ArialMT-95\"/>\n",
       "      <use x=\"272.460938\" xlink:href=\"#ArialMT-119\"/>\n",
       "      <use x=\"344.677734\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"366.894531\" xlink:href=\"#ArialMT-100\"/>\n",
       "      <use x=\"422.509766\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"450.292969\" xlink:href=\"#ArialMT-104\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_1\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 43.280326 279 \n",
       "L 44.400808 279 \n",
       "L 45.52129 279 \n",
       "L 46.641771 279 \n",
       "L 47.762253 279 \n",
       "L 48.882735 279 \n",
       "L 50.003217 279 \n",
       "L 51.123699 279 \n",
       "L 52.244181 279 \n",
       "L 53.364663 279 \n",
       "L 54.485145 279 \n",
       "L 55.605627 279 \n",
       "L 56.726109 279 \n",
       "L 57.846591 279 \n",
       "L 58.967073 279 \n",
       "L 60.087555 279 \n",
       "L 61.208037 279 \n",
       "L 62.328518 279 \n",
       "L 63.449 279 \n",
       "L 64.569482 279 \n",
       "L 65.689964 279 \n",
       "L 66.810446 279 \n",
       "L 67.930928 279 \n",
       "L 69.05141 279 \n",
       "L 70.171892 279 \n",
       "L 71.292374 279 \n",
       "L 72.412856 279 \n",
       "L 73.533338 279 \n",
       "L 74.65382 279 \n",
       "L 75.774302 279 \n",
       "L 76.894784 279 \n",
       "L 78.015265 279 \n",
       "L 79.135747 279 \n",
       "L 80.256229 279 \n",
       "L 81.376711 279 \n",
       "L 82.497193 279 \n",
       "L 83.617675 279 \n",
       "L 84.738157 279 \n",
       "L 85.858639 279 \n",
       "L 86.979121 279 \n",
       "L 88.099603 279 \n",
       "L 89.220085 279 \n",
       "L 90.340567 279 \n",
       "L 91.461049 279 \n",
       "L 92.58153 279 \n",
       "L 93.702012 279 \n",
       "L 94.822494 279 \n",
       "L 95.942976 279 \n",
       "L 97.063458 279 \n",
       "L 98.18394 279 \n",
       "L 99.304422 279 \n",
       "L 100.424904 279 \n",
       "L 101.545386 279 \n",
       "L 102.665868 279 \n",
       "L 103.78635 279 \n",
       "L 104.906832 279 \n",
       "L 106.027314 279 \n",
       "L 107.147796 279 \n",
       "L 108.268277 279 \n",
       "L 109.388759 279 \n",
       "L 110.509241 279 \n",
       "L 111.629723 279 \n",
       "L 112.750205 279 \n",
       "L 113.870687 279 \n",
       "L 114.991169 279 \n",
       "L 116.111651 279 \n",
       "L 117.232133 279 \n",
       "L 118.352615 279 \n",
       "L 119.473097 279 \n",
       "L 120.593579 279 \n",
       "L 121.714061 279 \n",
       "L 122.834543 279 \n",
       "L 123.955024 279 \n",
       "L 125.075506 279 \n",
       "L 126.195988 279 \n",
       "L 127.31647 279 \n",
       "L 128.436952 279 \n",
       "L 129.557434 279 \n",
       "L 130.677916 279 \n",
       "L 131.798398 279 \n",
       "L 132.91888 279 \n",
       "L 134.039362 279 \n",
       "L 135.159844 279 \n",
       "L 136.280326 279 \n",
       "L 137.400808 279 \n",
       "L 138.52129 279 \n",
       "L 139.641771 279 \n",
       "L 140.762253 279 \n",
       "L 141.882735 279 \n",
       "L 143.003217 279 \n",
       "L 144.123699 279 \n",
       "L 145.244181 279 \n",
       "L 146.364663 279 \n",
       "L 147.485145 279 \n",
       "L 148.605627 279 \n",
       "L 149.726109 279 \n",
       "L 150.846591 279 \n",
       "L 151.967073 279 \n",
       "L 153.087555 279 \n",
       "L 154.208037 279 \n",
       "L 155.328518 279 \n",
       "L 156.449 279 \n",
       "L 157.569482 279 \n",
       "L 158.689964 279 \n",
       "L 159.810446 279 \n",
       "L 160.930928 279 \n",
       "L 162.05141 279 \n",
       "L 163.171892 279 \n",
       "L 164.292374 279 \n",
       "L 165.412856 279 \n",
       "L 166.533338 279 \n",
       "L 167.65382 279 \n",
       "L 168.774302 279 \n",
       "L 169.894784 279 \n",
       "L 171.015265 279 \n",
       "L 172.135747 279 \n",
       "L 173.256229 279 \n",
       "L 174.376711 279 \n",
       "L 175.497193 279 \n",
       "L 176.617675 279 \n",
       "L 177.738157 279 \n",
       "L 178.858639 279 \n",
       "L 179.979121 279 \n",
       "L 181.099603 279 \n",
       "L 182.220085 279 \n",
       "L 183.340567 279 \n",
       "L 184.461049 279 \n",
       "L 185.58153 279 \n",
       "L 186.702012 279 \n",
       "L 187.822494 279 \n",
       "L 188.942976 279 \n",
       "L 190.063458 279 \n",
       "L 191.18394 279 \n",
       "L 192.304422 279 \n",
       "L 193.424904 279 \n",
       "L 194.545386 279 \n",
       "L 195.665868 279 \n",
       "L 196.78635 279 \n",
       "L 197.906832 279 \n",
       "L 199.027314 279 \n",
       "L 200.147796 279 \n",
       "L 201.268277 279 \n",
       "L 202.388759 279 \n",
       "L 203.509241 279 \n",
       "L 204.629723 279 \n",
       "L 205.750205 279 \n",
       "L 206.870687 279 \n",
       "L 207.991169 279 \n",
       "L 209.111651 279 \n",
       "L 210.232133 279 \n",
       "L 211.352615 279 \n",
       "L 212.473097 279 \n",
       "L 213.593579 279 \n",
       "L 214.714061 279 \n",
       "L 215.834543 279 \n",
       "L 216.955024 279 \n",
       "L 218.075506 279 \n",
       "L 219.195988 279 \n",
       "L 220.31647 279 \n",
       "L 221.436952 279 \n",
       "L 222.557434 279 \n",
       "L 223.677916 279 \n",
       "L 224.798398 279 \n",
       "L 225.91888 279 \n",
       "L 227.039362 279 \n",
       "L 228.159844 279 \n",
       "L 229.280326 279 \n",
       "L 230.400808 279 \n",
       "L 231.52129 279 \n",
       "L 232.641771 279 \n",
       "L 233.762253 279 \n",
       "L 234.882735 279 \n",
       "L 236.003217 279 \n",
       "L 237.123699 279 \n",
       "L 238.244181 279 \n",
       "L 239.364663 279 \n",
       "L 240.485145 279 \n",
       "L 241.605627 279 \n",
       "L 242.726109 279 \n",
       "L 243.846591 279 \n",
       "L 244.967073 279 \n",
       "L 246.087555 279 \n",
       "L 247.208037 279 \n",
       "L 248.328518 279 \n",
       "L 249.449 279 \n",
       "L 250.569482 279 \n",
       "L 251.689964 279 \n",
       "L 252.810446 279 \n",
       "L 253.930928 279 \n",
       "L 255.05141 279 \n",
       "L 256.171892 279 \n",
       "L 257.292374 279 \n",
       "L 258.412856 279 \n",
       "L 259.533338 279 \n",
       "L 260.65382 279 \n",
       "L 261.774302 279 \n",
       "L 262.894784 279 \n",
       "L 264.015265 279 \n",
       "L 265.135747 279 \n",
       "L 266.256229 279 \n",
       "L 267.376711 279 \n",
       "L 268.497193 279 \n",
       "L 269.617675 279 \n",
       "L 270.738157 279 \n",
       "L 271.858639 279 \n",
       "L 272.979121 279 \n",
       "L 274.099603 279 \n",
       "L 275.220085 279 \n",
       "L 276.340567 279 \n",
       "L 277.461049 279 \n",
       "L 278.58153 279 \n",
       "L 279.702012 279 \n",
       "L 280.822494 279 \n",
       "L 281.942976 279 \n",
       "L 283.063458 279 \n",
       "L 284.18394 279 \n",
       "L 285.304422 279 \n",
       "L 286.424904 279 \n",
       "L 287.545386 279 \n",
       "L 288.665868 279 \n",
       "L 289.78635 279 \n",
       "L 290.906832 279 \n",
       "L 292.027314 279 \n",
       "L 293.147796 279 \n",
       "L 294.268277 279 \n",
       "L 295.388759 279 \n",
       "L 296.509241 279 \n",
       "L 297.629723 279 \n",
       "L 298.750205 279 \n",
       "L 299.870687 279 \n",
       "L 300.991169 279 \n",
       "L 302.111651 279 \n",
       "L 303.232133 279 \n",
       "L 304.352615 279 \n",
       "L 305.473097 279 \n",
       "L 306.593579 279 \n",
       "L 307.714061 279 \n",
       "L 308.834543 279 \n",
       "L 309.955024 279 \n",
       "L 311.075506 279 \n",
       "L 312.195988 279 \n",
       "L 313.31647 279 \n",
       "L 314.436952 279 \n",
       "L 315.557434 279 \n",
       "L 316.677916 279 \n",
       "L 317.798398 279 \n",
       "L 318.91888 279 \n",
       "L 320.039362 279 \n",
       "L 321.159844 279 \n",
       "L 321.159844 277.908434 \n",
       "L 321.159844 276.816867 \n",
       "L 321.159844 275.725301 \n",
       "L 321.159844 274.633735 \n",
       "L 321.159844 273.542169 \n",
       "L 321.159844 272.450602 \n",
       "L 321.159844 271.359036 \n",
       "L 321.159844 270.26747 \n",
       "L 321.159844 269.175904 \n",
       "L 321.159844 268.084337 \n",
       "L 321.159844 266.992771 \n",
       "L 321.159844 265.901205 \n",
       "L 321.159844 264.809639 \n",
       "L 321.159844 263.718072 \n",
       "L 321.159844 262.626506 \n",
       "L 321.159844 261.53494 \n",
       "L 321.159844 260.443373 \n",
       "L 321.159844 259.351807 \n",
       "L 321.159844 258.260241 \n",
       "L 321.159844 257.168675 \n",
       "L 321.159844 256.077108 \n",
       "L 321.159844 254.985542 \n",
       "L 321.159844 253.893976 \n",
       "L 321.159844 252.80241 \n",
       "L 321.159844 251.710843 \n",
       "L 321.159844 250.619277 \n",
       "L 321.159844 249.527711 \n",
       "L 321.159844 248.436145 \n",
       "L 321.159844 247.344578 \n",
       "L 321.159844 246.253012 \n",
       "L 321.159844 245.161446 \n",
       "L 321.159844 244.06988 \n",
       "L 321.159844 242.978313 \n",
       "L 321.159844 241.886747 \n",
       "L 321.159844 240.795181 \n",
       "L 321.159844 239.703614 \n",
       "L 321.159844 238.612048 \n",
       "L 321.159844 237.520482 \n",
       "L 321.159844 236.428916 \n",
       "L 321.159844 235.337349 \n",
       "L 321.159844 234.245783 \n",
       "L 321.159844 233.154217 \n",
       "L 321.159844 232.062651 \n",
       "L 321.159844 230.971084 \n",
       "L 321.159844 229.879518 \n",
       "L 321.159844 228.787952 \n",
       "L 321.159844 227.696386 \n",
       "L 321.159844 226.604819 \n",
       "L 321.159844 225.513253 \n",
       "L 321.159844 224.421687 \n",
       "L 321.159844 223.33012 \n",
       "L 321.159844 222.238554 \n",
       "L 321.159844 221.146988 \n",
       "L 321.159844 220.055422 \n",
       "L 321.159844 218.963855 \n",
       "L 321.159844 217.872289 \n",
       "L 321.159844 216.780723 \n",
       "L 321.159844 215.689157 \n",
       "L 321.159844 214.59759 \n",
       "L 321.159844 213.506024 \n",
       "L 321.159844 212.414458 \n",
       "L 321.159844 211.322892 \n",
       "L 321.159844 210.231325 \n",
       "L 321.159844 209.139759 \n",
       "L 321.159844 208.048193 \n",
       "L 321.159844 206.956627 \n",
       "L 321.159844 205.86506 \n",
       "L 321.159844 204.773494 \n",
       "L 321.159844 203.681928 \n",
       "L 321.159844 202.590361 \n",
       "L 321.159844 201.498795 \n",
       "L 321.159844 200.407229 \n",
       "L 321.159844 199.315663 \n",
       "L 321.159844 198.224096 \n",
       "L 321.159844 197.13253 \n",
       "L 321.159844 196.040964 \n",
       "L 321.159844 195.768072 \n",
       "L 320.039362 195.768072 \n",
       "L 318.91888 195.768072 \n",
       "L 317.798398 195.768072 \n",
       "L 316.677916 195.768072 \n",
       "L 315.557434 195.768072 \n",
       "L 314.436952 195.768072 \n",
       "L 313.31647 195.768072 \n",
       "L 312.195988 195.768072 \n",
       "L 311.075506 195.768072 \n",
       "L 309.955024 195.768072 \n",
       "L 308.834543 195.768072 \n",
       "L 307.714061 195.768072 \n",
       "L 306.593579 195.768072 \n",
       "L 305.473097 195.768072 \n",
       "L 304.352615 195.768072 \n",
       "L 303.232133 195.768072 \n",
       "L 302.111651 195.768072 \n",
       "L 300.991169 195.768072 \n",
       "L 299.870687 195.768072 \n",
       "L 298.750205 195.768072 \n",
       "L 297.629723 195.768072 \n",
       "L 296.509241 195.768072 \n",
       "L 295.388759 195.768072 \n",
       "L 294.268277 195.768072 \n",
       "L 293.147796 195.768072 \n",
       "L 292.027314 195.768072 \n",
       "L 290.906832 195.768072 \n",
       "L 289.78635 195.768072 \n",
       "L 288.665868 195.768072 \n",
       "L 287.545386 195.768072 \n",
       "L 286.424904 195.768072 \n",
       "L 285.304422 195.768072 \n",
       "L 284.18394 195.768072 \n",
       "L 283.063458 195.768072 \n",
       "L 281.942976 195.768072 \n",
       "L 280.822494 195.768072 \n",
       "L 279.702012 195.768072 \n",
       "L 278.58153 195.768072 \n",
       "L 277.461049 195.768072 \n",
       "L 276.340567 195.768072 \n",
       "L 275.220085 195.768072 \n",
       "L 274.099603 195.768072 \n",
       "L 272.979121 195.768072 \n",
       "L 271.858639 195.768072 \n",
       "L 270.738157 195.768072 \n",
       "L 269.617675 195.768072 \n",
       "L 268.497193 195.768072 \n",
       "L 267.376711 195.768072 \n",
       "L 266.256229 195.768072 \n",
       "L 265.135747 195.768072 \n",
       "L 264.015265 195.768072 \n",
       "L 262.894784 195.768072 \n",
       "L 261.774302 195.768072 \n",
       "L 260.65382 195.768072 \n",
       "L 259.533338 195.768072 \n",
       "L 258.412856 195.768072 \n",
       "L 257.292374 195.768072 \n",
       "L 256.171892 195.768072 \n",
       "L 255.05141 195.768072 \n",
       "L 253.930928 195.768072 \n",
       "L 252.810446 195.768072 \n",
       "L 251.689964 195.768072 \n",
       "L 250.569482 195.768072 \n",
       "L 249.449 195.768072 \n",
       "L 248.328518 195.768072 \n",
       "L 247.208037 195.768072 \n",
       "L 246.087555 195.768072 \n",
       "L 244.967073 195.768072 \n",
       "L 243.846591 195.768072 \n",
       "L 242.726109 195.768072 \n",
       "L 241.605627 195.768072 \n",
       "L 240.485145 195.768072 \n",
       "L 239.364663 195.768072 \n",
       "L 238.244181 195.768072 \n",
       "L 237.123699 195.768072 \n",
       "L 236.003217 195.768072 \n",
       "L 234.882735 195.768072 \n",
       "L 233.762253 195.768072 \n",
       "L 232.641771 195.768072 \n",
       "L 231.52129 195.768072 \n",
       "L 230.400808 195.768072 \n",
       "L 229.280326 195.768072 \n",
       "L 228.159844 195.768072 \n",
       "L 227.039362 195.768072 \n",
       "L 225.91888 195.768072 \n",
       "L 224.798398 195.768072 \n",
       "L 223.677916 195.768072 \n",
       "L 222.557434 195.768072 \n",
       "L 221.436952 195.768072 \n",
       "L 220.31647 195.768072 \n",
       "L 219.195988 195.768072 \n",
       "L 218.075506 195.768072 \n",
       "L 216.955024 195.768072 \n",
       "L 215.834543 195.768072 \n",
       "L 214.714061 195.768072 \n",
       "L 213.593579 195.768072 \n",
       "L 212.473097 195.768072 \n",
       "L 211.352615 195.768072 \n",
       "L 210.232133 195.768072 \n",
       "L 209.111651 195.768072 \n",
       "L 207.991169 195.768072 \n",
       "L 206.870687 195.768072 \n",
       "L 205.750205 195.768072 \n",
       "L 204.629723 195.768072 \n",
       "L 203.509241 195.768072 \n",
       "L 202.388759 195.768072 \n",
       "L 201.268277 195.768072 \n",
       "L 200.147796 195.768072 \n",
       "L 199.027314 195.768072 \n",
       "L 197.906832 195.768072 \n",
       "L 196.78635 195.768072 \n",
       "L 195.665868 195.768072 \n",
       "L 194.545386 195.768072 \n",
       "L 193.424904 195.768072 \n",
       "L 192.304422 195.768072 \n",
       "L 191.18394 195.768072 \n",
       "L 190.063458 195.768072 \n",
       "L 188.942976 195.768072 \n",
       "L 187.822494 195.768072 \n",
       "L 186.702012 195.768072 \n",
       "L 185.58153 195.768072 \n",
       "L 184.461049 195.768072 \n",
       "L 183.340567 195.768072 \n",
       "L 182.220085 195.768072 \n",
       "L 181.099603 195.768072 \n",
       "L 179.979121 195.768072 \n",
       "L 178.858639 195.768072 \n",
       "L 177.738157 195.768072 \n",
       "L 176.617675 195.768072 \n",
       "L 175.497193 195.768072 \n",
       "L 174.376711 195.768072 \n",
       "L 173.256229 195.768072 \n",
       "L 172.135747 195.768072 \n",
       "L 171.015265 195.768072 \n",
       "L 169.894784 195.768072 \n",
       "L 168.774302 195.768072 \n",
       "L 167.65382 195.768072 \n",
       "L 166.533338 195.768072 \n",
       "L 165.412856 195.768072 \n",
       "L 164.292374 195.768072 \n",
       "L 163.171892 195.768072 \n",
       "L 162.05141 195.768072 \n",
       "L 160.930928 195.768072 \n",
       "L 159.810446 195.768072 \n",
       "L 158.689964 195.768072 \n",
       "L 157.569482 195.768072 \n",
       "L 156.449 195.768072 \n",
       "L 155.328518 195.768072 \n",
       "L 154.208037 195.768072 \n",
       "L 153.087555 195.768072 \n",
       "L 151.967073 195.768072 \n",
       "L 150.846591 195.768072 \n",
       "L 149.726109 195.768072 \n",
       "L 148.605627 195.768072 \n",
       "L 147.485145 195.768072 \n",
       "L 146.364663 195.768072 \n",
       "L 145.244181 195.768072 \n",
       "L 144.123699 195.768072 \n",
       "L 143.003217 195.768072 \n",
       "L 141.882735 195.768072 \n",
       "L 140.762253 195.768072 \n",
       "L 139.641771 195.768072 \n",
       "L 138.52129 195.768072 \n",
       "L 137.400808 195.768072 \n",
       "L 136.280326 195.768072 \n",
       "L 135.159844 195.768072 \n",
       "L 134.039362 195.768072 \n",
       "L 132.91888 195.768072 \n",
       "L 131.798398 195.768072 \n",
       "L 130.677916 195.768072 \n",
       "L 129.557434 195.768072 \n",
       "L 128.436952 195.768072 \n",
       "L 127.31647 195.768072 \n",
       "L 126.195988 195.768072 \n",
       "L 125.075506 195.768072 \n",
       "L 123.955024 195.768072 \n",
       "L 122.834543 195.768072 \n",
       "L 121.714061 195.768072 \n",
       "L 120.593579 195.768072 \n",
       "L 119.473097 195.768072 \n",
       "L 118.352615 195.768072 \n",
       "L 117.232133 195.768072 \n",
       "L 116.111651 195.768072 \n",
       "L 114.991169 195.768072 \n",
       "L 113.870687 195.768072 \n",
       "L 112.750205 195.768072 \n",
       "L 111.629723 195.768072 \n",
       "L 110.509241 195.768072 \n",
       "L 109.388759 195.768072 \n",
       "L 108.268277 195.768072 \n",
       "L 107.147796 195.768072 \n",
       "L 106.027314 195.768072 \n",
       "L 104.906832 195.768072 \n",
       "L 103.78635 195.768072 \n",
       "L 102.665868 195.768072 \n",
       "L 101.545386 195.768072 \n",
       "L 100.424904 195.768072 \n",
       "L 99.304422 195.768072 \n",
       "L 98.18394 195.768072 \n",
       "L 97.063458 195.768072 \n",
       "L 95.942976 195.768072 \n",
       "L 94.822494 195.768072 \n",
       "L 93.702012 195.768072 \n",
       "L 92.58153 195.768072 \n",
       "L 91.461049 195.768072 \n",
       "L 90.340567 195.768072 \n",
       "L 89.220085 195.768072 \n",
       "L 88.099603 195.768072 \n",
       "L 86.979121 195.768072 \n",
       "L 85.858639 195.768072 \n",
       "L 84.738157 195.768072 \n",
       "L 83.617675 195.768072 \n",
       "L 82.497193 195.768072 \n",
       "L 81.376711 195.768072 \n",
       "L 80.256229 195.768072 \n",
       "L 79.135747 195.768072 \n",
       "L 78.015265 195.768072 \n",
       "L 76.894784 195.768072 \n",
       "L 75.774302 195.768072 \n",
       "L 74.65382 195.768072 \n",
       "L 73.533338 195.768072 \n",
       "L 72.412856 195.768072 \n",
       "L 71.292374 195.768072 \n",
       "L 70.171892 195.768072 \n",
       "L 69.05141 195.768072 \n",
       "L 67.930928 195.768072 \n",
       "L 66.810446 195.768072 \n",
       "L 65.689964 195.768072 \n",
       "L 64.569482 195.768072 \n",
       "L 63.449 195.768072 \n",
       "L 62.328518 195.768072 \n",
       "L 61.208037 195.768072 \n",
       "L 60.087555 195.768072 \n",
       "L 58.967073 195.768072 \n",
       "L 57.846591 195.768072 \n",
       "L 56.726109 195.768072 \n",
       "L 55.605627 195.768072 \n",
       "L 54.485145 195.768072 \n",
       "L 53.364663 195.768072 \n",
       "L 52.244181 195.768072 \n",
       "L 51.123699 195.768072 \n",
       "L 50.003217 195.768072 \n",
       "L 48.882735 195.768072 \n",
       "L 47.762253 195.768072 \n",
       "L 46.641771 195.768072 \n",
       "L 45.52129 195.768072 \n",
       "L 44.400808 195.768072 \n",
       "L 43.280326 195.768072 \n",
       "L 42.159844 195.768072 \n",
       "L 42.159844 196.040964 \n",
       "L 42.159844 197.13253 \n",
       "L 42.159844 198.224096 \n",
       "L 42.159844 199.315663 \n",
       "L 42.159844 200.407229 \n",
       "L 42.159844 201.498795 \n",
       "L 42.159844 202.590361 \n",
       "L 42.159844 203.681928 \n",
       "L 42.159844 204.773494 \n",
       "L 42.159844 205.86506 \n",
       "L 42.159844 206.956627 \n",
       "L 42.159844 208.048193 \n",
       "L 42.159844 209.139759 \n",
       "L 42.159844 210.231325 \n",
       "L 42.159844 211.322892 \n",
       "L 42.159844 212.414458 \n",
       "L 42.159844 213.506024 \n",
       "L 42.159844 214.59759 \n",
       "L 42.159844 215.689157 \n",
       "L 42.159844 216.780723 \n",
       "L 42.159844 217.872289 \n",
       "L 42.159844 218.963855 \n",
       "L 42.159844 220.055422 \n",
       "L 42.159844 221.146988 \n",
       "L 42.159844 222.238554 \n",
       "L 42.159844 223.33012 \n",
       "L 42.159844 224.421687 \n",
       "L 42.159844 225.513253 \n",
       "L 42.159844 226.604819 \n",
       "L 42.159844 227.696386 \n",
       "L 42.159844 228.787952 \n",
       "L 42.159844 229.879518 \n",
       "L 42.159844 230.971084 \n",
       "L 42.159844 232.062651 \n",
       "L 42.159844 233.154217 \n",
       "L 42.159844 234.245783 \n",
       "L 42.159844 235.337349 \n",
       "L 42.159844 236.428916 \n",
       "L 42.159844 237.520482 \n",
       "L 42.159844 238.612048 \n",
       "L 42.159844 239.703614 \n",
       "L 42.159844 240.795181 \n",
       "L 42.159844 241.886747 \n",
       "L 42.159844 242.978313 \n",
       "L 42.159844 244.06988 \n",
       "L 42.159844 245.161446 \n",
       "L 42.159844 246.253012 \n",
       "L 42.159844 247.344578 \n",
       "L 42.159844 248.436145 \n",
       "L 42.159844 249.527711 \n",
       "L 42.159844 250.619277 \n",
       "L 42.159844 251.710843 \n",
       "L 42.159844 252.80241 \n",
       "L 42.159844 253.893976 \n",
       "L 42.159844 254.985542 \n",
       "L 42.159844 256.077108 \n",
       "L 42.159844 257.168675 \n",
       "L 42.159844 258.260241 \n",
       "L 42.159844 259.351807 \n",
       "L 42.159844 260.443373 \n",
       "L 42.159844 261.53494 \n",
       "L 42.159844 262.626506 \n",
       "L 42.159844 263.718072 \n",
       "L 42.159844 264.809639 \n",
       "L 42.159844 265.901205 \n",
       "L 42.159844 266.992771 \n",
       "L 42.159844 268.084337 \n",
       "L 42.159844 269.175904 \n",
       "L 42.159844 270.26747 \n",
       "L 42.159844 271.359036 \n",
       "L 42.159844 272.450602 \n",
       "L 42.159844 273.542169 \n",
       "L 42.159844 274.633735 \n",
       "L 42.159844 275.725301 \n",
       "L 42.159844 276.816867 \n",
       "L 42.159844 277.908434 \n",
       "L 42.159844 279 \n",
       "z\n",
       "\" style=\"fill:#1f77b4;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_2\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 43.280326 195.768072 \n",
       "L 44.400808 195.768072 \n",
       "L 45.52129 195.768072 \n",
       "L 46.641771 195.768072 \n",
       "L 47.762253 195.768072 \n",
       "L 48.882735 195.768072 \n",
       "L 50.003217 195.768072 \n",
       "L 51.123699 195.768072 \n",
       "L 52.244181 195.768072 \n",
       "L 53.364663 195.768072 \n",
       "L 54.485145 195.768072 \n",
       "L 55.605627 195.768072 \n",
       "L 56.726109 195.768072 \n",
       "L 57.846591 195.768072 \n",
       "L 58.967073 195.768072 \n",
       "L 60.087555 195.768072 \n",
       "L 61.208037 195.768072 \n",
       "L 62.328518 195.768072 \n",
       "L 63.449 195.768072 \n",
       "L 64.569482 195.768072 \n",
       "L 65.689964 195.768072 \n",
       "L 66.810446 195.768072 \n",
       "L 67.930928 195.768072 \n",
       "L 69.05141 195.768072 \n",
       "L 70.171892 195.768072 \n",
       "L 71.292374 195.768072 \n",
       "L 72.412856 195.768072 \n",
       "L 73.533338 195.768072 \n",
       "L 74.65382 195.768072 \n",
       "L 75.774302 195.768072 \n",
       "L 76.894784 195.768072 \n",
       "L 78.015265 195.768072 \n",
       "L 79.135747 195.768072 \n",
       "L 80.256229 195.768072 \n",
       "L 81.376711 195.768072 \n",
       "L 82.497193 195.768072 \n",
       "L 83.617675 195.768072 \n",
       "L 84.738157 195.768072 \n",
       "L 85.858639 195.768072 \n",
       "L 86.979121 195.768072 \n",
       "L 88.099603 195.768072 \n",
       "L 89.220085 195.768072 \n",
       "L 90.340567 195.768072 \n",
       "L 91.461049 195.768072 \n",
       "L 92.58153 195.768072 \n",
       "L 93.702012 195.768072 \n",
       "L 94.822494 195.768072 \n",
       "L 95.942976 195.768072 \n",
       "L 97.063458 195.768072 \n",
       "L 98.18394 195.768072 \n",
       "L 99.304422 195.768072 \n",
       "L 100.424904 195.768072 \n",
       "L 101.545386 195.768072 \n",
       "L 102.665868 195.768072 \n",
       "L 103.78635 195.768072 \n",
       "L 104.906832 195.768072 \n",
       "L 106.027314 195.768072 \n",
       "L 107.147796 195.768072 \n",
       "L 108.268277 195.768072 \n",
       "L 109.388759 195.768072 \n",
       "L 110.509241 195.768072 \n",
       "L 111.629723 195.768072 \n",
       "L 112.750205 195.768072 \n",
       "L 113.870687 195.768072 \n",
       "L 114.991169 195.768072 \n",
       "L 116.111651 195.768072 \n",
       "L 117.232133 195.768072 \n",
       "L 118.352615 195.768072 \n",
       "L 119.473097 195.768072 \n",
       "L 120.593579 195.768072 \n",
       "L 121.714061 195.768072 \n",
       "L 122.834543 195.768072 \n",
       "L 123.955024 195.768072 \n",
       "L 125.075506 195.768072 \n",
       "L 126.195988 195.768072 \n",
       "L 127.31647 195.768072 \n",
       "L 128.436952 195.768072 \n",
       "L 129.557434 195.768072 \n",
       "L 130.677916 195.768072 \n",
       "L 131.798398 195.768072 \n",
       "L 132.91888 195.768072 \n",
       "L 134.039362 195.768072 \n",
       "L 135.159844 195.768072 \n",
       "L 136.280326 195.768072 \n",
       "L 137.400808 195.768072 \n",
       "L 138.52129 195.768072 \n",
       "L 139.641771 195.768072 \n",
       "L 140.762253 195.768072 \n",
       "L 141.882735 195.768072 \n",
       "L 143.003217 195.768072 \n",
       "L 144.123699 195.768072 \n",
       "L 145.244181 195.768072 \n",
       "L 146.364663 195.768072 \n",
       "L 147.485145 195.768072 \n",
       "L 148.605627 195.768072 \n",
       "L 149.726109 195.768072 \n",
       "L 150.846591 195.768072 \n",
       "L 151.967073 195.768072 \n",
       "L 153.087555 195.768072 \n",
       "L 154.208037 195.768072 \n",
       "L 155.328518 195.768072 \n",
       "L 156.449 195.768072 \n",
       "L 157.569482 195.768072 \n",
       "L 158.689964 195.768072 \n",
       "L 159.810446 195.768072 \n",
       "L 160.930928 195.768072 \n",
       "L 162.05141 195.768072 \n",
       "L 163.171892 195.768072 \n",
       "L 164.292374 195.768072 \n",
       "L 165.412856 195.768072 \n",
       "L 166.533338 195.768072 \n",
       "L 167.65382 195.768072 \n",
       "L 168.774302 195.768072 \n",
       "L 169.894784 195.768072 \n",
       "L 171.015265 195.768072 \n",
       "L 172.135747 195.768072 \n",
       "L 173.256229 195.768072 \n",
       "L 174.376711 195.768072 \n",
       "L 175.497193 195.768072 \n",
       "L 176.617675 195.768072 \n",
       "L 177.738157 195.768072 \n",
       "L 178.858639 195.768072 \n",
       "L 179.979121 195.768072 \n",
       "L 181.099603 195.768072 \n",
       "L 182.220085 195.768072 \n",
       "L 183.340567 195.768072 \n",
       "L 184.461049 195.768072 \n",
       "L 185.58153 195.768072 \n",
       "L 186.702012 195.768072 \n",
       "L 187.822494 195.768072 \n",
       "L 188.942976 195.768072 \n",
       "L 190.063458 195.768072 \n",
       "L 191.18394 195.768072 \n",
       "L 192.304422 195.768072 \n",
       "L 193.424904 195.768072 \n",
       "L 194.545386 195.768072 \n",
       "L 195.665868 195.768072 \n",
       "L 196.78635 195.768072 \n",
       "L 197.906832 195.768072 \n",
       "L 199.027314 195.768072 \n",
       "L 200.147796 195.768072 \n",
       "L 201.268277 195.768072 \n",
       "L 202.388759 195.768072 \n",
       "L 203.509241 195.768072 \n",
       "L 204.629723 195.768072 \n",
       "L 205.750205 195.768072 \n",
       "L 206.870687 195.768072 \n",
       "L 207.991169 195.768072 \n",
       "L 209.111651 195.768072 \n",
       "L 210.232133 195.768072 \n",
       "L 211.352615 195.768072 \n",
       "L 212.473097 195.768072 \n",
       "L 213.593579 195.768072 \n",
       "L 214.714061 195.768072 \n",
       "L 215.834543 195.768072 \n",
       "L 216.955024 195.768072 \n",
       "L 218.075506 195.768072 \n",
       "L 219.195988 195.768072 \n",
       "L 220.31647 195.768072 \n",
       "L 221.436952 195.768072 \n",
       "L 222.557434 195.768072 \n",
       "L 223.677916 195.768072 \n",
       "L 224.798398 195.768072 \n",
       "L 225.91888 195.768072 \n",
       "L 227.039362 195.768072 \n",
       "L 228.159844 195.768072 \n",
       "L 229.280326 195.768072 \n",
       "L 230.400808 195.768072 \n",
       "L 231.52129 195.768072 \n",
       "L 232.641771 195.768072 \n",
       "L 233.762253 195.768072 \n",
       "L 234.882735 195.768072 \n",
       "L 236.003217 195.768072 \n",
       "L 237.123699 195.768072 \n",
       "L 238.244181 195.768072 \n",
       "L 239.364663 195.768072 \n",
       "L 240.485145 195.768072 \n",
       "L 241.605627 195.768072 \n",
       "L 242.726109 195.768072 \n",
       "L 243.846591 195.768072 \n",
       "L 244.967073 195.768072 \n",
       "L 246.087555 195.768072 \n",
       "L 247.208037 195.768072 \n",
       "L 248.328518 195.768072 \n",
       "L 249.449 195.768072 \n",
       "L 250.569482 195.768072 \n",
       "L 251.689964 195.768072 \n",
       "L 252.810446 195.768072 \n",
       "L 253.930928 195.768072 \n",
       "L 255.05141 195.768072 \n",
       "L 256.171892 195.768072 \n",
       "L 257.292374 195.768072 \n",
       "L 258.412856 195.768072 \n",
       "L 259.533338 195.768072 \n",
       "L 260.65382 195.768072 \n",
       "L 261.774302 195.768072 \n",
       "L 262.894784 195.768072 \n",
       "L 264.015265 195.768072 \n",
       "L 265.135747 195.768072 \n",
       "L 266.256229 195.768072 \n",
       "L 267.376711 195.768072 \n",
       "L 268.497193 195.768072 \n",
       "L 269.617675 195.768072 \n",
       "L 270.738157 195.768072 \n",
       "L 271.858639 195.768072 \n",
       "L 272.979121 195.768072 \n",
       "L 274.099603 195.768072 \n",
       "L 275.220085 195.768072 \n",
       "L 276.340567 195.768072 \n",
       "L 277.461049 195.768072 \n",
       "L 278.58153 195.768072 \n",
       "L 279.702012 195.768072 \n",
       "L 280.822494 195.768072 \n",
       "L 281.942976 195.768072 \n",
       "L 283.063458 195.768072 \n",
       "L 284.18394 195.768072 \n",
       "L 285.304422 195.768072 \n",
       "L 286.424904 195.768072 \n",
       "L 287.545386 195.768072 \n",
       "L 288.665868 195.768072 \n",
       "L 289.78635 195.768072 \n",
       "L 290.906832 195.768072 \n",
       "L 292.027314 195.768072 \n",
       "L 293.147796 195.768072 \n",
       "L 294.268277 195.768072 \n",
       "L 295.388759 195.768072 \n",
       "L 296.509241 195.768072 \n",
       "L 297.629723 195.768072 \n",
       "L 298.750205 195.768072 \n",
       "L 299.870687 195.768072 \n",
       "L 300.991169 195.768072 \n",
       "L 302.111651 195.768072 \n",
       "L 303.232133 195.768072 \n",
       "L 304.352615 195.768072 \n",
       "L 305.473097 195.768072 \n",
       "L 306.593579 195.768072 \n",
       "L 307.714061 195.768072 \n",
       "L 308.834543 195.768072 \n",
       "L 309.955024 195.768072 \n",
       "L 311.075506 195.768072 \n",
       "L 312.195988 195.768072 \n",
       "L 313.31647 195.768072 \n",
       "L 314.436952 195.768072 \n",
       "L 315.557434 195.768072 \n",
       "L 316.677916 195.768072 \n",
       "L 317.798398 195.768072 \n",
       "L 318.91888 195.768072 \n",
       "L 320.039362 195.768072 \n",
       "L 321.159844 195.768072 \n",
       "L 321.159844 195.495181 \n",
       "L 320.039362 195.495181 \n",
       "L 318.91888 195.495181 \n",
       "L 317.798398 195.495181 \n",
       "L 316.677916 195.495181 \n",
       "L 315.557434 195.495181 \n",
       "L 314.436952 195.495181 \n",
       "L 313.31647 195.495181 \n",
       "L 312.195988 195.495181 \n",
       "L 311.075506 195.495181 \n",
       "L 309.955024 195.495181 \n",
       "L 308.834543 195.495181 \n",
       "L 307.714061 195.495181 \n",
       "L 306.593579 195.495181 \n",
       "L 305.473097 195.495181 \n",
       "L 304.352615 195.495181 \n",
       "L 303.232133 195.495181 \n",
       "L 302.111651 195.495181 \n",
       "L 300.991169 195.495181 \n",
       "L 299.870687 195.495181 \n",
       "L 298.750205 195.495181 \n",
       "L 297.629723 195.495181 \n",
       "L 296.509241 195.495181 \n",
       "L 295.388759 195.495181 \n",
       "L 294.268277 195.495181 \n",
       "L 293.147796 195.495181 \n",
       "L 292.027314 195.495181 \n",
       "L 290.906832 195.495181 \n",
       "L 289.78635 195.495181 \n",
       "L 288.665868 195.495181 \n",
       "L 287.545386 195.495181 \n",
       "L 286.424904 195.495181 \n",
       "L 285.304422 195.495181 \n",
       "L 284.18394 195.495181 \n",
       "L 283.063458 195.495181 \n",
       "L 281.942976 195.495181 \n",
       "L 280.822494 195.495181 \n",
       "L 279.702012 195.495181 \n",
       "L 278.58153 195.495181 \n",
       "L 277.461049 195.495181 \n",
       "L 276.340567 195.495181 \n",
       "L 275.220085 195.495181 \n",
       "L 274.099603 195.495181 \n",
       "L 272.979121 195.495181 \n",
       "L 271.858639 195.495181 \n",
       "L 270.738157 195.495181 \n",
       "L 269.617675 195.495181 \n",
       "L 268.497193 195.495181 \n",
       "L 267.376711 195.495181 \n",
       "L 266.256229 195.495181 \n",
       "L 265.135747 195.495181 \n",
       "L 264.015265 195.495181 \n",
       "L 262.894784 195.495181 \n",
       "L 261.774302 195.495181 \n",
       "L 260.65382 195.495181 \n",
       "L 259.533338 195.495181 \n",
       "L 258.412856 195.495181 \n",
       "L 257.292374 195.495181 \n",
       "L 256.171892 195.495181 \n",
       "L 255.05141 195.495181 \n",
       "L 253.930928 195.495181 \n",
       "L 252.810446 195.495181 \n",
       "L 251.689964 195.495181 \n",
       "L 250.569482 195.495181 \n",
       "L 249.449 195.495181 \n",
       "L 248.328518 195.495181 \n",
       "L 247.208037 195.495181 \n",
       "L 246.087555 195.495181 \n",
       "L 244.967073 195.495181 \n",
       "L 243.846591 195.495181 \n",
       "L 242.726109 195.495181 \n",
       "L 241.605627 195.495181 \n",
       "L 240.485145 195.495181 \n",
       "L 239.364663 195.495181 \n",
       "L 238.244181 195.495181 \n",
       "L 237.123699 195.495181 \n",
       "L 236.003217 195.495181 \n",
       "L 234.882735 195.495181 \n",
       "L 233.762253 195.495181 \n",
       "L 232.641771 195.495181 \n",
       "L 231.52129 195.495181 \n",
       "L 230.400808 195.495181 \n",
       "L 229.280326 195.495181 \n",
       "L 228.159844 195.495181 \n",
       "L 227.039362 195.495181 \n",
       "L 225.91888 195.495181 \n",
       "L 224.798398 195.495181 \n",
       "L 223.677916 195.495181 \n",
       "L 222.557434 195.495181 \n",
       "L 221.436952 195.495181 \n",
       "L 220.31647 195.495181 \n",
       "L 219.195988 195.495181 \n",
       "L 218.075506 195.495181 \n",
       "L 216.955024 195.495181 \n",
       "L 215.834543 195.495181 \n",
       "L 214.714061 195.495181 \n",
       "L 213.593579 195.495181 \n",
       "L 212.473097 195.495181 \n",
       "L 211.352615 195.495181 \n",
       "L 210.232133 195.495181 \n",
       "L 209.111651 195.495181 \n",
       "L 207.991169 195.495181 \n",
       "L 206.870687 195.495181 \n",
       "L 205.750205 195.495181 \n",
       "L 204.629723 195.495181 \n",
       "L 203.509241 195.495181 \n",
       "L 202.388759 195.495181 \n",
       "L 201.268277 195.495181 \n",
       "L 200.147796 195.495181 \n",
       "L 199.027314 195.495181 \n",
       "L 197.906832 195.495181 \n",
       "L 196.78635 195.495181 \n",
       "L 195.665868 195.495181 \n",
       "L 194.545386 195.495181 \n",
       "L 193.424904 195.495181 \n",
       "L 192.304422 195.495181 \n",
       "L 191.18394 195.495181 \n",
       "L 190.063458 195.495181 \n",
       "L 188.942976 195.495181 \n",
       "L 187.822494 195.495181 \n",
       "L 186.702012 195.495181 \n",
       "L 185.58153 195.495181 \n",
       "L 184.461049 195.495181 \n",
       "L 183.340567 195.495181 \n",
       "L 182.220085 195.495181 \n",
       "L 181.099603 195.495181 \n",
       "L 179.979121 195.495181 \n",
       "L 178.858639 195.495181 \n",
       "L 177.738157 195.495181 \n",
       "L 176.617675 195.495181 \n",
       "L 175.497193 195.495181 \n",
       "L 174.376711 195.495181 \n",
       "L 173.256229 195.495181 \n",
       "L 172.135747 195.495181 \n",
       "L 171.015265 195.495181 \n",
       "L 169.894784 195.495181 \n",
       "L 168.774302 195.495181 \n",
       "L 167.65382 195.495181 \n",
       "L 166.533338 195.495181 \n",
       "L 165.412856 195.495181 \n",
       "L 164.292374 195.495181 \n",
       "L 163.171892 195.495181 \n",
       "L 162.05141 195.495181 \n",
       "L 160.930928 195.495181 \n",
       "L 159.810446 195.495181 \n",
       "L 158.689964 195.495181 \n",
       "L 157.569482 195.495181 \n",
       "L 156.449 195.495181 \n",
       "L 155.328518 195.495181 \n",
       "L 154.208037 195.495181 \n",
       "L 153.087555 195.495181 \n",
       "L 151.967073 195.495181 \n",
       "L 150.846591 195.495181 \n",
       "L 149.726109 195.495181 \n",
       "L 148.605627 195.495181 \n",
       "L 147.485145 195.495181 \n",
       "L 146.364663 195.495181 \n",
       "L 145.244181 195.495181 \n",
       "L 144.123699 195.495181 \n",
       "L 143.003217 195.495181 \n",
       "L 141.882735 195.495181 \n",
       "L 140.762253 195.495181 \n",
       "L 139.641771 195.495181 \n",
       "L 138.52129 195.495181 \n",
       "L 137.400808 195.495181 \n",
       "L 136.280326 195.495181 \n",
       "L 135.159844 195.495181 \n",
       "L 134.039362 195.495181 \n",
       "L 132.91888 195.495181 \n",
       "L 131.798398 195.495181 \n",
       "L 130.677916 195.495181 \n",
       "L 129.557434 195.495181 \n",
       "L 128.436952 195.495181 \n",
       "L 127.31647 195.495181 \n",
       "L 126.195988 195.495181 \n",
       "L 125.075506 195.495181 \n",
       "L 123.955024 195.495181 \n",
       "L 122.834543 195.495181 \n",
       "L 121.714061 195.495181 \n",
       "L 120.593579 195.495181 \n",
       "L 119.473097 195.495181 \n",
       "L 118.352615 195.495181 \n",
       "L 117.232133 195.495181 \n",
       "L 116.111651 195.495181 \n",
       "L 114.991169 195.495181 \n",
       "L 113.870687 195.495181 \n",
       "L 112.750205 195.495181 \n",
       "L 111.629723 195.495181 \n",
       "L 110.509241 195.495181 \n",
       "L 109.388759 195.495181 \n",
       "L 108.268277 195.495181 \n",
       "L 107.147796 195.495181 \n",
       "L 106.027314 195.495181 \n",
       "L 104.906832 195.495181 \n",
       "L 103.78635 195.495181 \n",
       "L 102.665868 195.495181 \n",
       "L 101.545386 195.495181 \n",
       "L 100.424904 195.495181 \n",
       "L 99.304422 195.495181 \n",
       "L 98.18394 195.495181 \n",
       "L 97.063458 195.495181 \n",
       "L 95.942976 195.495181 \n",
       "L 94.822494 195.495181 \n",
       "L 93.702012 195.495181 \n",
       "L 92.58153 195.495181 \n",
       "L 91.461049 195.495181 \n",
       "L 90.340567 195.495181 \n",
       "L 89.220085 195.495181 \n",
       "L 88.099603 195.495181 \n",
       "L 86.979121 195.495181 \n",
       "L 85.858639 195.495181 \n",
       "L 84.738157 195.495181 \n",
       "L 83.617675 195.495181 \n",
       "L 82.497193 195.495181 \n",
       "L 81.376711 195.495181 \n",
       "L 80.256229 195.495181 \n",
       "L 79.135747 195.495181 \n",
       "L 78.015265 195.495181 \n",
       "L 76.894784 195.495181 \n",
       "L 75.774302 195.495181 \n",
       "L 74.65382 195.495181 \n",
       "L 73.533338 195.495181 \n",
       "L 72.412856 195.495181 \n",
       "L 71.292374 195.495181 \n",
       "L 70.171892 195.495181 \n",
       "L 69.05141 195.495181 \n",
       "L 67.930928 195.495181 \n",
       "L 66.810446 195.495181 \n",
       "L 65.689964 195.495181 \n",
       "L 64.569482 195.495181 \n",
       "L 63.449 195.495181 \n",
       "L 62.328518 195.495181 \n",
       "L 61.208037 195.495181 \n",
       "L 60.087555 195.495181 \n",
       "L 58.967073 195.495181 \n",
       "L 57.846591 195.495181 \n",
       "L 56.726109 195.495181 \n",
       "L 55.605627 195.495181 \n",
       "L 54.485145 195.495181 \n",
       "L 53.364663 195.495181 \n",
       "L 52.244181 195.495181 \n",
       "L 51.123699 195.495181 \n",
       "L 50.003217 195.495181 \n",
       "L 48.882735 195.495181 \n",
       "L 47.762253 195.495181 \n",
       "L 46.641771 195.495181 \n",
       "L 45.52129 195.495181 \n",
       "L 44.400808 195.495181 \n",
       "L 43.280326 195.495181 \n",
       "L 42.159844 195.495181 \n",
       "L 42.159844 195.768072 \n",
       "z\n",
       "\" style=\"fill:#ff7f0e;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_3\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 43.280326 195.495181 \n",
       "L 44.400808 195.495181 \n",
       "L 45.52129 195.495181 \n",
       "L 46.641771 195.495181 \n",
       "L 47.762253 195.495181 \n",
       "L 48.882735 195.495181 \n",
       "L 50.003217 195.495181 \n",
       "L 51.123699 195.495181 \n",
       "L 52.244181 195.495181 \n",
       "L 53.364663 195.495181 \n",
       "L 54.485145 195.495181 \n",
       "L 55.605627 195.495181 \n",
       "L 56.726109 195.495181 \n",
       "L 57.846591 195.495181 \n",
       "L 58.967073 195.495181 \n",
       "L 60.087555 195.495181 \n",
       "L 61.208037 195.495181 \n",
       "L 62.328518 195.495181 \n",
       "L 63.449 195.495181 \n",
       "L 64.569482 195.495181 \n",
       "L 65.689964 195.495181 \n",
       "L 66.810446 195.495181 \n",
       "L 67.930928 195.495181 \n",
       "L 69.05141 195.495181 \n",
       "L 70.171892 195.495181 \n",
       "L 71.292374 195.495181 \n",
       "L 72.412856 195.495181 \n",
       "L 73.533338 195.495181 \n",
       "L 74.65382 195.495181 \n",
       "L 75.774302 195.495181 \n",
       "L 76.894784 195.495181 \n",
       "L 78.015265 195.495181 \n",
       "L 79.135747 195.495181 \n",
       "L 80.256229 195.495181 \n",
       "L 81.376711 195.495181 \n",
       "L 82.497193 195.495181 \n",
       "L 83.617675 195.495181 \n",
       "L 84.738157 195.495181 \n",
       "L 85.858639 195.495181 \n",
       "L 86.979121 195.495181 \n",
       "L 88.099603 195.495181 \n",
       "L 89.220085 195.495181 \n",
       "L 90.340567 195.495181 \n",
       "L 91.461049 195.495181 \n",
       "L 92.58153 195.495181 \n",
       "L 93.702012 195.495181 \n",
       "L 94.822494 195.495181 \n",
       "L 95.942976 195.495181 \n",
       "L 97.063458 195.495181 \n",
       "L 98.18394 195.495181 \n",
       "L 99.304422 195.495181 \n",
       "L 100.424904 195.495181 \n",
       "L 101.545386 195.495181 \n",
       "L 102.665868 195.495181 \n",
       "L 103.78635 195.495181 \n",
       "L 104.906832 195.495181 \n",
       "L 106.027314 195.495181 \n",
       "L 107.147796 195.495181 \n",
       "L 108.268277 195.495181 \n",
       "L 109.388759 195.495181 \n",
       "L 110.509241 195.495181 \n",
       "L 111.629723 195.495181 \n",
       "L 112.750205 195.495181 \n",
       "L 113.870687 195.495181 \n",
       "L 114.991169 195.495181 \n",
       "L 116.111651 195.495181 \n",
       "L 117.232133 195.495181 \n",
       "L 118.352615 195.495181 \n",
       "L 119.473097 195.495181 \n",
       "L 120.593579 195.495181 \n",
       "L 121.714061 195.495181 \n",
       "L 122.834543 195.495181 \n",
       "L 123.955024 195.495181 \n",
       "L 125.075506 195.495181 \n",
       "L 126.195988 195.495181 \n",
       "L 127.31647 195.495181 \n",
       "L 128.436952 195.495181 \n",
       "L 129.557434 195.495181 \n",
       "L 130.677916 195.495181 \n",
       "L 131.798398 195.495181 \n",
       "L 132.91888 195.495181 \n",
       "L 134.039362 195.495181 \n",
       "L 135.159844 195.495181 \n",
       "L 136.280326 195.495181 \n",
       "L 137.400808 195.495181 \n",
       "L 138.52129 195.495181 \n",
       "L 139.641771 195.495181 \n",
       "L 140.762253 195.495181 \n",
       "L 141.882735 195.495181 \n",
       "L 143.003217 195.495181 \n",
       "L 144.123699 195.495181 \n",
       "L 145.244181 195.495181 \n",
       "L 146.364663 195.495181 \n",
       "L 147.485145 195.495181 \n",
       "L 148.605627 195.495181 \n",
       "L 149.726109 195.495181 \n",
       "L 150.846591 195.495181 \n",
       "L 151.967073 195.495181 \n",
       "L 153.087555 195.495181 \n",
       "L 154.208037 195.495181 \n",
       "L 155.328518 195.495181 \n",
       "L 156.449 195.495181 \n",
       "L 157.569482 195.495181 \n",
       "L 158.689964 195.495181 \n",
       "L 159.810446 195.495181 \n",
       "L 160.930928 195.495181 \n",
       "L 162.05141 195.495181 \n",
       "L 163.171892 195.495181 \n",
       "L 164.292374 195.495181 \n",
       "L 165.412856 195.495181 \n",
       "L 166.533338 195.495181 \n",
       "L 167.65382 195.495181 \n",
       "L 168.774302 195.495181 \n",
       "L 169.894784 195.495181 \n",
       "L 171.015265 195.495181 \n",
       "L 172.135747 195.495181 \n",
       "L 173.256229 195.495181 \n",
       "L 174.376711 195.495181 \n",
       "L 175.497193 195.495181 \n",
       "L 176.617675 195.495181 \n",
       "L 177.738157 195.495181 \n",
       "L 178.858639 195.495181 \n",
       "L 179.979121 195.495181 \n",
       "L 181.099603 195.495181 \n",
       "L 182.220085 195.495181 \n",
       "L 183.340567 195.495181 \n",
       "L 184.461049 195.495181 \n",
       "L 185.58153 195.495181 \n",
       "L 186.702012 195.495181 \n",
       "L 187.822494 195.495181 \n",
       "L 188.942976 195.495181 \n",
       "L 190.063458 195.495181 \n",
       "L 191.18394 195.495181 \n",
       "L 192.304422 195.495181 \n",
       "L 193.424904 195.495181 \n",
       "L 194.545386 195.495181 \n",
       "L 195.665868 195.495181 \n",
       "L 196.78635 195.495181 \n",
       "L 197.906832 195.495181 \n",
       "L 199.027314 195.495181 \n",
       "L 200.147796 195.495181 \n",
       "L 201.268277 195.495181 \n",
       "L 202.388759 195.495181 \n",
       "L 203.509241 195.495181 \n",
       "L 204.629723 195.495181 \n",
       "L 205.750205 195.495181 \n",
       "L 206.870687 195.495181 \n",
       "L 207.991169 195.495181 \n",
       "L 209.111651 195.495181 \n",
       "L 210.232133 195.495181 \n",
       "L 211.352615 195.495181 \n",
       "L 212.473097 195.495181 \n",
       "L 213.593579 195.495181 \n",
       "L 214.714061 195.495181 \n",
       "L 215.834543 195.495181 \n",
       "L 216.955024 195.495181 \n",
       "L 218.075506 195.495181 \n",
       "L 219.195988 195.495181 \n",
       "L 220.31647 195.495181 \n",
       "L 221.436952 195.495181 \n",
       "L 222.557434 195.495181 \n",
       "L 223.677916 195.495181 \n",
       "L 224.798398 195.495181 \n",
       "L 225.91888 195.495181 \n",
       "L 227.039362 195.495181 \n",
       "L 228.159844 195.495181 \n",
       "L 229.280326 195.495181 \n",
       "L 230.400808 195.495181 \n",
       "L 231.52129 195.495181 \n",
       "L 232.641771 195.495181 \n",
       "L 233.762253 195.495181 \n",
       "L 234.882735 195.495181 \n",
       "L 236.003217 195.495181 \n",
       "L 237.123699 195.495181 \n",
       "L 238.244181 195.495181 \n",
       "L 239.364663 195.495181 \n",
       "L 240.485145 195.495181 \n",
       "L 241.605627 195.495181 \n",
       "L 242.726109 195.495181 \n",
       "L 243.846591 195.495181 \n",
       "L 244.967073 195.495181 \n",
       "L 246.087555 195.495181 \n",
       "L 247.208037 195.495181 \n",
       "L 248.328518 195.495181 \n",
       "L 249.449 195.495181 \n",
       "L 250.569482 195.495181 \n",
       "L 251.689964 195.495181 \n",
       "L 252.810446 195.495181 \n",
       "L 253.930928 195.495181 \n",
       "L 255.05141 195.495181 \n",
       "L 256.171892 195.495181 \n",
       "L 257.292374 195.495181 \n",
       "L 258.412856 195.495181 \n",
       "L 259.533338 195.495181 \n",
       "L 260.65382 195.495181 \n",
       "L 261.774302 195.495181 \n",
       "L 262.894784 195.495181 \n",
       "L 264.015265 195.495181 \n",
       "L 265.135747 195.495181 \n",
       "L 266.256229 195.495181 \n",
       "L 267.376711 195.495181 \n",
       "L 268.497193 195.495181 \n",
       "L 269.617675 195.495181 \n",
       "L 270.738157 195.495181 \n",
       "L 271.858639 195.495181 \n",
       "L 272.979121 195.495181 \n",
       "L 274.099603 195.495181 \n",
       "L 275.220085 195.495181 \n",
       "L 276.340567 195.495181 \n",
       "L 277.461049 195.495181 \n",
       "L 278.58153 195.495181 \n",
       "L 279.702012 195.495181 \n",
       "L 280.822494 195.495181 \n",
       "L 281.942976 195.495181 \n",
       "L 283.063458 195.495181 \n",
       "L 284.18394 195.495181 \n",
       "L 285.304422 195.495181 \n",
       "L 286.424904 195.495181 \n",
       "L 287.545386 195.495181 \n",
       "L 288.665868 195.495181 \n",
       "L 289.78635 195.495181 \n",
       "L 290.906832 195.495181 \n",
       "L 292.027314 195.495181 \n",
       "L 293.147796 195.495181 \n",
       "L 294.268277 195.495181 \n",
       "L 295.388759 195.495181 \n",
       "L 296.509241 195.495181 \n",
       "L 297.629723 195.495181 \n",
       "L 298.750205 195.495181 \n",
       "L 299.870687 195.495181 \n",
       "L 300.991169 195.495181 \n",
       "L 302.111651 195.495181 \n",
       "L 303.232133 195.495181 \n",
       "L 304.352615 195.495181 \n",
       "L 305.473097 195.495181 \n",
       "L 306.593579 195.495181 \n",
       "L 307.714061 195.495181 \n",
       "L 308.834543 195.495181 \n",
       "L 309.955024 195.495181 \n",
       "L 311.075506 195.495181 \n",
       "L 312.195988 195.495181 \n",
       "L 313.31647 195.495181 \n",
       "L 314.436952 195.495181 \n",
       "L 315.557434 195.495181 \n",
       "L 316.677916 195.495181 \n",
       "L 317.798398 195.495181 \n",
       "L 318.91888 195.495181 \n",
       "L 320.039362 195.495181 \n",
       "L 321.159844 195.495181 \n",
       "L 321.159844 195.222289 \n",
       "L 320.039362 195.222289 \n",
       "L 318.91888 195.222289 \n",
       "L 317.798398 195.222289 \n",
       "L 316.677916 195.222289 \n",
       "L 315.557434 195.222289 \n",
       "L 314.436952 195.222289 \n",
       "L 313.31647 195.222289 \n",
       "L 312.195988 195.222289 \n",
       "L 311.075506 195.222289 \n",
       "L 309.955024 195.222289 \n",
       "L 308.834543 195.222289 \n",
       "L 307.714061 195.222289 \n",
       "L 306.593579 195.222289 \n",
       "L 305.473097 195.222289 \n",
       "L 304.352615 195.222289 \n",
       "L 303.232133 195.222289 \n",
       "L 302.111651 195.222289 \n",
       "L 300.991169 195.222289 \n",
       "L 299.870687 195.222289 \n",
       "L 298.750205 195.222289 \n",
       "L 297.629723 195.222289 \n",
       "L 296.509241 195.222289 \n",
       "L 295.388759 195.222289 \n",
       "L 294.268277 195.222289 \n",
       "L 293.147796 195.222289 \n",
       "L 292.027314 195.222289 \n",
       "L 290.906832 195.222289 \n",
       "L 289.78635 195.222289 \n",
       "L 288.665868 195.222289 \n",
       "L 287.545386 195.222289 \n",
       "L 286.424904 195.222289 \n",
       "L 285.304422 195.222289 \n",
       "L 284.18394 195.222289 \n",
       "L 283.063458 195.222289 \n",
       "L 281.942976 195.222289 \n",
       "L 280.822494 195.222289 \n",
       "L 279.702012 195.222289 \n",
       "L 278.58153 195.222289 \n",
       "L 277.461049 195.222289 \n",
       "L 276.340567 195.222289 \n",
       "L 275.220085 195.222289 \n",
       "L 274.099603 195.222289 \n",
       "L 272.979121 195.222289 \n",
       "L 271.858639 195.222289 \n",
       "L 270.738157 195.222289 \n",
       "L 269.617675 195.222289 \n",
       "L 268.497193 195.222289 \n",
       "L 267.376711 195.222289 \n",
       "L 266.256229 195.222289 \n",
       "L 265.135747 195.222289 \n",
       "L 264.015265 195.222289 \n",
       "L 262.894784 195.222289 \n",
       "L 261.774302 195.222289 \n",
       "L 260.65382 195.222289 \n",
       "L 259.533338 195.222289 \n",
       "L 258.412856 195.222289 \n",
       "L 257.292374 195.222289 \n",
       "L 256.171892 195.222289 \n",
       "L 255.05141 195.222289 \n",
       "L 253.930928 195.222289 \n",
       "L 252.810446 195.222289 \n",
       "L 251.689964 195.222289 \n",
       "L 250.569482 195.222289 \n",
       "L 249.449 195.222289 \n",
       "L 248.328518 195.222289 \n",
       "L 247.208037 195.222289 \n",
       "L 246.087555 195.222289 \n",
       "L 244.967073 195.222289 \n",
       "L 243.846591 195.222289 \n",
       "L 242.726109 195.222289 \n",
       "L 241.605627 195.222289 \n",
       "L 240.485145 195.222289 \n",
       "L 239.364663 195.222289 \n",
       "L 238.244181 195.222289 \n",
       "L 237.123699 195.222289 \n",
       "L 236.003217 195.222289 \n",
       "L 234.882735 195.222289 \n",
       "L 233.762253 195.222289 \n",
       "L 232.641771 195.222289 \n",
       "L 231.52129 195.222289 \n",
       "L 230.400808 195.222289 \n",
       "L 229.280326 195.222289 \n",
       "L 228.159844 195.222289 \n",
       "L 227.039362 195.222289 \n",
       "L 225.91888 195.222289 \n",
       "L 224.798398 195.222289 \n",
       "L 223.677916 195.222289 \n",
       "L 222.557434 195.222289 \n",
       "L 221.436952 195.222289 \n",
       "L 220.31647 195.222289 \n",
       "L 219.195988 195.222289 \n",
       "L 218.075506 195.222289 \n",
       "L 216.955024 195.222289 \n",
       "L 215.834543 195.222289 \n",
       "L 214.714061 195.222289 \n",
       "L 213.593579 195.222289 \n",
       "L 212.473097 195.222289 \n",
       "L 211.352615 195.222289 \n",
       "L 210.232133 195.222289 \n",
       "L 209.111651 195.222289 \n",
       "L 207.991169 195.222289 \n",
       "L 206.870687 195.222289 \n",
       "L 205.750205 195.222289 \n",
       "L 204.629723 195.222289 \n",
       "L 203.509241 195.222289 \n",
       "L 202.388759 195.222289 \n",
       "L 201.268277 195.222289 \n",
       "L 200.147796 195.222289 \n",
       "L 199.027314 195.222289 \n",
       "L 197.906832 195.222289 \n",
       "L 196.78635 195.222289 \n",
       "L 195.665868 195.222289 \n",
       "L 194.545386 195.222289 \n",
       "L 193.424904 195.222289 \n",
       "L 192.304422 195.222289 \n",
       "L 191.18394 195.222289 \n",
       "L 190.063458 195.222289 \n",
       "L 188.942976 195.222289 \n",
       "L 187.822494 195.222289 \n",
       "L 186.702012 195.222289 \n",
       "L 185.58153 195.222289 \n",
       "L 184.461049 195.222289 \n",
       "L 183.340567 195.222289 \n",
       "L 182.220085 195.222289 \n",
       "L 181.099603 195.222289 \n",
       "L 179.979121 195.222289 \n",
       "L 178.858639 195.222289 \n",
       "L 177.738157 195.222289 \n",
       "L 176.617675 195.222289 \n",
       "L 175.497193 195.222289 \n",
       "L 174.376711 195.222289 \n",
       "L 173.256229 195.222289 \n",
       "L 172.135747 195.222289 \n",
       "L 171.015265 195.222289 \n",
       "L 169.894784 195.222289 \n",
       "L 168.774302 195.222289 \n",
       "L 167.65382 195.222289 \n",
       "L 166.533338 195.222289 \n",
       "L 165.412856 195.222289 \n",
       "L 164.292374 195.222289 \n",
       "L 163.171892 195.222289 \n",
       "L 162.05141 195.222289 \n",
       "L 160.930928 195.222289 \n",
       "L 159.810446 195.222289 \n",
       "L 158.689964 195.222289 \n",
       "L 157.569482 195.222289 \n",
       "L 156.449 195.222289 \n",
       "L 155.328518 195.222289 \n",
       "L 154.208037 195.222289 \n",
       "L 153.087555 195.222289 \n",
       "L 151.967073 195.222289 \n",
       "L 150.846591 195.222289 \n",
       "L 149.726109 195.222289 \n",
       "L 148.605627 195.222289 \n",
       "L 147.485145 195.222289 \n",
       "L 146.364663 195.222289 \n",
       "L 145.244181 195.222289 \n",
       "L 144.123699 195.222289 \n",
       "L 143.003217 195.222289 \n",
       "L 141.882735 195.222289 \n",
       "L 140.762253 195.222289 \n",
       "L 139.641771 195.222289 \n",
       "L 138.52129 195.222289 \n",
       "L 137.400808 195.222289 \n",
       "L 136.280326 195.222289 \n",
       "L 135.159844 195.222289 \n",
       "L 134.039362 195.222289 \n",
       "L 132.91888 195.222289 \n",
       "L 131.798398 195.222289 \n",
       "L 130.677916 195.222289 \n",
       "L 129.557434 195.222289 \n",
       "L 128.436952 195.222289 \n",
       "L 127.31647 195.222289 \n",
       "L 126.195988 195.222289 \n",
       "L 125.075506 195.222289 \n",
       "L 123.955024 195.222289 \n",
       "L 122.834543 195.222289 \n",
       "L 121.714061 195.222289 \n",
       "L 120.593579 195.222289 \n",
       "L 119.473097 195.222289 \n",
       "L 118.352615 195.222289 \n",
       "L 117.232133 195.222289 \n",
       "L 116.111651 195.222289 \n",
       "L 114.991169 195.222289 \n",
       "L 113.870687 195.222289 \n",
       "L 112.750205 195.222289 \n",
       "L 111.629723 195.222289 \n",
       "L 110.509241 195.222289 \n",
       "L 109.388759 195.222289 \n",
       "L 108.268277 195.222289 \n",
       "L 107.147796 195.222289 \n",
       "L 106.027314 195.222289 \n",
       "L 104.906832 195.222289 \n",
       "L 103.78635 195.222289 \n",
       "L 102.665868 195.222289 \n",
       "L 101.545386 195.222289 \n",
       "L 100.424904 195.222289 \n",
       "L 99.304422 195.222289 \n",
       "L 98.18394 195.222289 \n",
       "L 97.063458 195.222289 \n",
       "L 95.942976 195.222289 \n",
       "L 94.822494 195.222289 \n",
       "L 93.702012 195.222289 \n",
       "L 92.58153 195.222289 \n",
       "L 91.461049 195.222289 \n",
       "L 90.340567 195.222289 \n",
       "L 89.220085 195.222289 \n",
       "L 88.099603 195.222289 \n",
       "L 86.979121 195.222289 \n",
       "L 85.858639 195.222289 \n",
       "L 84.738157 195.222289 \n",
       "L 83.617675 195.222289 \n",
       "L 82.497193 195.222289 \n",
       "L 81.376711 195.222289 \n",
       "L 80.256229 195.222289 \n",
       "L 79.135747 195.222289 \n",
       "L 78.015265 195.222289 \n",
       "L 76.894784 195.222289 \n",
       "L 75.774302 195.222289 \n",
       "L 74.65382 195.222289 \n",
       "L 73.533338 195.222289 \n",
       "L 72.412856 195.222289 \n",
       "L 71.292374 195.222289 \n",
       "L 70.171892 195.222289 \n",
       "L 69.05141 195.222289 \n",
       "L 67.930928 195.222289 \n",
       "L 66.810446 195.222289 \n",
       "L 65.689964 195.222289 \n",
       "L 64.569482 195.222289 \n",
       "L 63.449 195.222289 \n",
       "L 62.328518 195.222289 \n",
       "L 61.208037 195.222289 \n",
       "L 60.087555 195.222289 \n",
       "L 58.967073 195.222289 \n",
       "L 57.846591 195.222289 \n",
       "L 56.726109 195.222289 \n",
       "L 55.605627 195.222289 \n",
       "L 54.485145 195.222289 \n",
       "L 53.364663 195.222289 \n",
       "L 52.244181 195.222289 \n",
       "L 51.123699 195.222289 \n",
       "L 50.003217 195.222289 \n",
       "L 48.882735 195.222289 \n",
       "L 47.762253 195.222289 \n",
       "L 46.641771 195.222289 \n",
       "L 45.52129 195.222289 \n",
       "L 44.400808 195.222289 \n",
       "L 43.280326 195.222289 \n",
       "L 42.159844 195.222289 \n",
       "L 42.159844 195.495181 \n",
       "z\n",
       "\" style=\"fill:#d62728;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_4\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 43.280326 195.222289 \n",
       "L 44.400808 195.222289 \n",
       "L 45.52129 195.222289 \n",
       "L 46.641771 195.222289 \n",
       "L 47.762253 195.222289 \n",
       "L 48.882735 195.222289 \n",
       "L 50.003217 195.222289 \n",
       "L 51.123699 195.222289 \n",
       "L 52.244181 195.222289 \n",
       "L 53.364663 195.222289 \n",
       "L 54.485145 195.222289 \n",
       "L 55.605627 195.222289 \n",
       "L 56.726109 195.222289 \n",
       "L 57.846591 195.222289 \n",
       "L 58.967073 195.222289 \n",
       "L 60.087555 195.222289 \n",
       "L 61.208037 195.222289 \n",
       "L 62.328518 195.222289 \n",
       "L 63.449 195.222289 \n",
       "L 64.569482 195.222289 \n",
       "L 65.689964 195.222289 \n",
       "L 66.810446 195.222289 \n",
       "L 67.930928 195.222289 \n",
       "L 69.05141 195.222289 \n",
       "L 70.171892 195.222289 \n",
       "L 71.292374 195.222289 \n",
       "L 72.412856 195.222289 \n",
       "L 73.533338 195.222289 \n",
       "L 74.65382 195.222289 \n",
       "L 75.774302 195.222289 \n",
       "L 76.894784 195.222289 \n",
       "L 78.015265 195.222289 \n",
       "L 79.135747 195.222289 \n",
       "L 80.256229 195.222289 \n",
       "L 81.376711 195.222289 \n",
       "L 82.497193 195.222289 \n",
       "L 83.617675 195.222289 \n",
       "L 84.738157 195.222289 \n",
       "L 85.858639 195.222289 \n",
       "L 86.979121 195.222289 \n",
       "L 88.099603 195.222289 \n",
       "L 89.220085 195.222289 \n",
       "L 90.340567 195.222289 \n",
       "L 91.461049 195.222289 \n",
       "L 92.58153 195.222289 \n",
       "L 93.702012 195.222289 \n",
       "L 94.822494 195.222289 \n",
       "L 95.942976 195.222289 \n",
       "L 97.063458 195.222289 \n",
       "L 98.18394 195.222289 \n",
       "L 99.304422 195.222289 \n",
       "L 100.424904 195.222289 \n",
       "L 101.545386 195.222289 \n",
       "L 102.665868 195.222289 \n",
       "L 103.78635 195.222289 \n",
       "L 104.906832 195.222289 \n",
       "L 106.027314 195.222289 \n",
       "L 107.147796 195.222289 \n",
       "L 108.268277 195.222289 \n",
       "L 109.388759 195.222289 \n",
       "L 110.509241 195.222289 \n",
       "L 111.629723 195.222289 \n",
       "L 112.750205 195.222289 \n",
       "L 113.870687 195.222289 \n",
       "L 114.991169 195.222289 \n",
       "L 116.111651 195.222289 \n",
       "L 117.232133 195.222289 \n",
       "L 118.352615 195.222289 \n",
       "L 119.473097 195.222289 \n",
       "L 120.593579 195.222289 \n",
       "L 121.714061 195.222289 \n",
       "L 122.834543 195.222289 \n",
       "L 123.955024 195.222289 \n",
       "L 125.075506 195.222289 \n",
       "L 126.195988 195.222289 \n",
       "L 127.31647 195.222289 \n",
       "L 128.436952 195.222289 \n",
       "L 129.557434 195.222289 \n",
       "L 130.677916 195.222289 \n",
       "L 131.798398 195.222289 \n",
       "L 132.91888 195.222289 \n",
       "L 134.039362 195.222289 \n",
       "L 135.159844 195.222289 \n",
       "L 136.280326 195.222289 \n",
       "L 137.400808 195.222289 \n",
       "L 138.52129 195.222289 \n",
       "L 139.641771 195.222289 \n",
       "L 140.762253 195.222289 \n",
       "L 141.882735 195.222289 \n",
       "L 143.003217 195.222289 \n",
       "L 144.123699 195.222289 \n",
       "L 145.244181 195.222289 \n",
       "L 146.364663 195.222289 \n",
       "L 147.485145 195.222289 \n",
       "L 148.605627 195.222289 \n",
       "L 149.726109 195.222289 \n",
       "L 150.846591 195.222289 \n",
       "L 151.967073 195.222289 \n",
       "L 153.087555 195.222289 \n",
       "L 154.208037 195.222289 \n",
       "L 155.328518 195.222289 \n",
       "L 156.449 195.222289 \n",
       "L 157.569482 195.222289 \n",
       "L 158.689964 195.222289 \n",
       "L 159.810446 195.222289 \n",
       "L 160.930928 195.222289 \n",
       "L 162.05141 195.222289 \n",
       "L 163.171892 195.222289 \n",
       "L 164.292374 195.222289 \n",
       "L 165.412856 195.222289 \n",
       "L 166.533338 195.222289 \n",
       "L 167.65382 195.222289 \n",
       "L 168.774302 195.222289 \n",
       "L 169.894784 195.222289 \n",
       "L 171.015265 195.222289 \n",
       "L 172.135747 195.222289 \n",
       "L 173.256229 195.222289 \n",
       "L 174.376711 195.222289 \n",
       "L 175.497193 195.222289 \n",
       "L 176.617675 195.222289 \n",
       "L 177.738157 195.222289 \n",
       "L 178.858639 195.222289 \n",
       "L 179.979121 195.222289 \n",
       "L 181.099603 195.222289 \n",
       "L 182.220085 195.222289 \n",
       "L 183.340567 195.222289 \n",
       "L 184.461049 195.222289 \n",
       "L 185.58153 195.222289 \n",
       "L 186.702012 195.222289 \n",
       "L 187.822494 195.222289 \n",
       "L 188.942976 195.222289 \n",
       "L 190.063458 195.222289 \n",
       "L 191.18394 195.222289 \n",
       "L 192.304422 195.222289 \n",
       "L 193.424904 195.222289 \n",
       "L 194.545386 195.222289 \n",
       "L 195.665868 195.222289 \n",
       "L 196.78635 195.222289 \n",
       "L 197.906832 195.222289 \n",
       "L 199.027314 195.222289 \n",
       "L 200.147796 195.222289 \n",
       "L 201.268277 195.222289 \n",
       "L 202.388759 195.222289 \n",
       "L 203.509241 195.222289 \n",
       "L 204.629723 195.222289 \n",
       "L 205.750205 195.222289 \n",
       "L 206.870687 195.222289 \n",
       "L 207.991169 195.222289 \n",
       "L 209.111651 195.222289 \n",
       "L 210.232133 195.222289 \n",
       "L 211.352615 195.222289 \n",
       "L 212.473097 195.222289 \n",
       "L 213.593579 195.222289 \n",
       "L 214.714061 195.222289 \n",
       "L 215.834543 195.222289 \n",
       "L 216.955024 195.222289 \n",
       "L 218.075506 195.222289 \n",
       "L 219.195988 195.222289 \n",
       "L 220.31647 195.222289 \n",
       "L 221.436952 195.222289 \n",
       "L 222.557434 195.222289 \n",
       "L 223.677916 195.222289 \n",
       "L 224.798398 195.222289 \n",
       "L 225.91888 195.222289 \n",
       "L 227.039362 195.222289 \n",
       "L 228.159844 195.222289 \n",
       "L 229.280326 195.222289 \n",
       "L 230.400808 195.222289 \n",
       "L 231.52129 195.222289 \n",
       "L 232.641771 195.222289 \n",
       "L 233.762253 195.222289 \n",
       "L 234.882735 195.222289 \n",
       "L 236.003217 195.222289 \n",
       "L 237.123699 195.222289 \n",
       "L 238.244181 195.222289 \n",
       "L 239.364663 195.222289 \n",
       "L 240.485145 195.222289 \n",
       "L 241.605627 195.222289 \n",
       "L 242.726109 195.222289 \n",
       "L 243.846591 195.222289 \n",
       "L 244.967073 195.222289 \n",
       "L 246.087555 195.222289 \n",
       "L 247.208037 195.222289 \n",
       "L 248.328518 195.222289 \n",
       "L 249.449 195.222289 \n",
       "L 250.569482 195.222289 \n",
       "L 251.689964 195.222289 \n",
       "L 252.810446 195.222289 \n",
       "L 253.930928 195.222289 \n",
       "L 255.05141 195.222289 \n",
       "L 256.171892 195.222289 \n",
       "L 257.292374 195.222289 \n",
       "L 258.412856 195.222289 \n",
       "L 259.533338 195.222289 \n",
       "L 260.65382 195.222289 \n",
       "L 261.774302 195.222289 \n",
       "L 262.894784 195.222289 \n",
       "L 264.015265 195.222289 \n",
       "L 265.135747 195.222289 \n",
       "L 266.256229 195.222289 \n",
       "L 267.376711 195.222289 \n",
       "L 268.497193 195.222289 \n",
       "L 269.617675 195.222289 \n",
       "L 270.738157 195.222289 \n",
       "L 271.858639 195.222289 \n",
       "L 272.979121 195.222289 \n",
       "L 274.099603 195.222289 \n",
       "L 275.220085 195.222289 \n",
       "L 276.340567 195.222289 \n",
       "L 277.461049 195.222289 \n",
       "L 278.58153 195.222289 \n",
       "L 279.702012 195.222289 \n",
       "L 280.822494 195.222289 \n",
       "L 281.942976 195.222289 \n",
       "L 283.063458 195.222289 \n",
       "L 284.18394 195.222289 \n",
       "L 285.304422 195.222289 \n",
       "L 286.424904 195.222289 \n",
       "L 287.545386 195.222289 \n",
       "L 288.665868 195.222289 \n",
       "L 289.78635 195.222289 \n",
       "L 290.906832 195.222289 \n",
       "L 292.027314 195.222289 \n",
       "L 293.147796 195.222289 \n",
       "L 294.268277 195.222289 \n",
       "L 295.388759 195.222289 \n",
       "L 296.509241 195.222289 \n",
       "L 297.629723 195.222289 \n",
       "L 298.750205 195.222289 \n",
       "L 299.870687 195.222289 \n",
       "L 300.991169 195.222289 \n",
       "L 302.111651 195.222289 \n",
       "L 303.232133 195.222289 \n",
       "L 304.352615 195.222289 \n",
       "L 305.473097 195.222289 \n",
       "L 306.593579 195.222289 \n",
       "L 307.714061 195.222289 \n",
       "L 308.834543 195.222289 \n",
       "L 309.955024 195.222289 \n",
       "L 311.075506 195.222289 \n",
       "L 312.195988 195.222289 \n",
       "L 313.31647 195.222289 \n",
       "L 314.436952 195.222289 \n",
       "L 315.557434 195.222289 \n",
       "L 316.677916 195.222289 \n",
       "L 317.798398 195.222289 \n",
       "L 318.91888 195.222289 \n",
       "L 320.039362 195.222289 \n",
       "L 321.159844 195.222289 \n",
       "L 321.159844 194.949398 \n",
       "L 321.159844 193.857831 \n",
       "L 321.159844 192.766265 \n",
       "L 321.159844 191.674699 \n",
       "L 321.159844 190.583133 \n",
       "L 321.159844 189.491566 \n",
       "L 321.159844 188.4 \n",
       "L 321.159844 187.308434 \n",
       "L 321.159844 186.216867 \n",
       "L 321.159844 185.125301 \n",
       "L 321.159844 184.033735 \n",
       "L 321.159844 182.942169 \n",
       "L 321.159844 181.850602 \n",
       "L 321.159844 180.759036 \n",
       "L 321.159844 179.66747 \n",
       "L 321.159844 178.575904 \n",
       "L 321.159844 177.484337 \n",
       "L 321.159844 176.392771 \n",
       "L 321.159844 175.301205 \n",
       "L 321.159844 174.209639 \n",
       "L 321.159844 173.118072 \n",
       "L 321.159844 172.026506 \n",
       "L 321.159844 170.93494 \n",
       "L 321.159844 169.843373 \n",
       "L 321.159844 168.751807 \n",
       "L 321.159844 167.660241 \n",
       "L 321.159844 166.568675 \n",
       "L 321.159844 165.477108 \n",
       "L 321.159844 164.385542 \n",
       "L 321.159844 163.293976 \n",
       "L 321.159844 162.20241 \n",
       "L 321.159844 161.110843 \n",
       "L 321.159844 160.019277 \n",
       "L 321.159844 158.927711 \n",
       "L 321.159844 157.836145 \n",
       "L 321.159844 156.744578 \n",
       "L 321.159844 155.653012 \n",
       "L 321.159844 154.561446 \n",
       "L 321.159844 153.46988 \n",
       "L 321.159844 152.378313 \n",
       "L 321.159844 151.286747 \n",
       "L 321.159844 150.195181 \n",
       "L 321.159844 149.103614 \n",
       "L 321.159844 148.012048 \n",
       "L 321.159844 146.920482 \n",
       "L 321.159844 145.828916 \n",
       "L 321.159844 144.737349 \n",
       "L 321.159844 143.645783 \n",
       "L 321.159844 142.554217 \n",
       "L 321.159844 141.462651 \n",
       "L 321.159844 140.371084 \n",
       "L 321.159844 139.279518 \n",
       "L 321.159844 138.187952 \n",
       "L 321.159844 137.096386 \n",
       "L 321.159844 136.004819 \n",
       "L 321.159844 134.913253 \n",
       "L 321.159844 133.821687 \n",
       "L 321.159844 132.73012 \n",
       "L 321.159844 131.638554 \n",
       "L 321.159844 130.546988 \n",
       "L 321.159844 129.455422 \n",
       "L 321.159844 128.363855 \n",
       "L 321.159844 127.272289 \n",
       "L 321.159844 126.180723 \n",
       "L 321.159844 125.089157 \n",
       "L 321.159844 123.99759 \n",
       "L 321.159844 122.906024 \n",
       "L 321.159844 121.814458 \n",
       "L 321.159844 120.722892 \n",
       "L 321.159844 119.631325 \n",
       "L 321.159844 118.539759 \n",
       "L 321.159844 117.448193 \n",
       "L 321.159844 116.356627 \n",
       "L 321.159844 115.26506 \n",
       "L 321.159844 114.173494 \n",
       "L 321.159844 113.081928 \n",
       "L 321.159844 111.990361 \n",
       "L 321.159844 110.898795 \n",
       "L 321.159844 109.807229 \n",
       "L 321.159844 108.715663 \n",
       "L 321.159844 107.624096 \n",
       "L 321.159844 106.53253 \n",
       "L 320.039362 106.53253 \n",
       "L 318.91888 106.53253 \n",
       "L 317.798398 106.53253 \n",
       "L 316.677916 106.53253 \n",
       "L 315.557434 106.53253 \n",
       "L 314.436952 106.53253 \n",
       "L 313.31647 106.53253 \n",
       "L 312.195988 106.53253 \n",
       "L 311.075506 106.53253 \n",
       "L 309.955024 106.53253 \n",
       "L 308.834543 106.53253 \n",
       "L 307.714061 106.53253 \n",
       "L 306.593579 106.53253 \n",
       "L 305.473097 106.53253 \n",
       "L 304.352615 106.53253 \n",
       "L 303.232133 106.53253 \n",
       "L 302.111651 106.53253 \n",
       "L 300.991169 106.53253 \n",
       "L 299.870687 106.53253 \n",
       "L 298.750205 106.53253 \n",
       "L 297.629723 106.53253 \n",
       "L 296.509241 106.53253 \n",
       "L 295.388759 106.53253 \n",
       "L 294.268277 106.53253 \n",
       "L 293.147796 106.53253 \n",
       "L 292.027314 106.53253 \n",
       "L 290.906832 106.53253 \n",
       "L 289.78635 106.53253 \n",
       "L 288.665868 106.53253 \n",
       "L 287.545386 106.53253 \n",
       "L 286.424904 106.53253 \n",
       "L 285.304422 106.53253 \n",
       "L 284.18394 106.53253 \n",
       "L 283.063458 106.53253 \n",
       "L 281.942976 106.53253 \n",
       "L 280.822494 106.53253 \n",
       "L 279.702012 106.53253 \n",
       "L 278.58153 106.53253 \n",
       "L 277.461049 106.53253 \n",
       "L 276.340567 106.53253 \n",
       "L 275.220085 106.53253 \n",
       "L 274.099603 106.53253 \n",
       "L 272.979121 106.53253 \n",
       "L 271.858639 106.53253 \n",
       "L 270.738157 106.53253 \n",
       "L 269.617675 106.53253 \n",
       "L 268.497193 106.53253 \n",
       "L 267.376711 106.53253 \n",
       "L 266.256229 106.53253 \n",
       "L 265.135747 106.53253 \n",
       "L 264.015265 106.53253 \n",
       "L 262.894784 106.53253 \n",
       "L 261.774302 106.53253 \n",
       "L 260.65382 106.53253 \n",
       "L 259.533338 106.53253 \n",
       "L 258.412856 106.53253 \n",
       "L 257.292374 106.53253 \n",
       "L 256.171892 106.53253 \n",
       "L 255.05141 106.53253 \n",
       "L 253.930928 106.53253 \n",
       "L 252.810446 106.53253 \n",
       "L 251.689964 106.53253 \n",
       "L 250.569482 106.53253 \n",
       "L 249.449 106.53253 \n",
       "L 248.328518 106.53253 \n",
       "L 247.208037 106.53253 \n",
       "L 246.087555 106.53253 \n",
       "L 244.967073 106.53253 \n",
       "L 243.846591 106.53253 \n",
       "L 242.726109 106.53253 \n",
       "L 241.605627 106.53253 \n",
       "L 240.485145 106.53253 \n",
       "L 239.364663 106.53253 \n",
       "L 238.244181 106.53253 \n",
       "L 237.123699 106.53253 \n",
       "L 236.003217 106.53253 \n",
       "L 234.882735 106.53253 \n",
       "L 233.762253 106.53253 \n",
       "L 232.641771 106.53253 \n",
       "L 231.52129 106.53253 \n",
       "L 230.400808 106.53253 \n",
       "L 229.280326 106.53253 \n",
       "L 228.159844 106.53253 \n",
       "L 227.039362 106.53253 \n",
       "L 225.91888 106.53253 \n",
       "L 224.798398 106.53253 \n",
       "L 223.677916 106.53253 \n",
       "L 222.557434 106.53253 \n",
       "L 221.436952 106.53253 \n",
       "L 220.31647 106.53253 \n",
       "L 219.195988 106.53253 \n",
       "L 218.075506 106.53253 \n",
       "L 216.955024 106.53253 \n",
       "L 215.834543 106.53253 \n",
       "L 214.714061 106.53253 \n",
       "L 213.593579 106.53253 \n",
       "L 212.473097 106.53253 \n",
       "L 211.352615 106.53253 \n",
       "L 210.232133 106.53253 \n",
       "L 209.111651 106.53253 \n",
       "L 207.991169 106.53253 \n",
       "L 206.870687 106.53253 \n",
       "L 205.750205 106.53253 \n",
       "L 204.629723 106.53253 \n",
       "L 203.509241 106.53253 \n",
       "L 202.388759 106.53253 \n",
       "L 201.268277 106.53253 \n",
       "L 200.147796 106.53253 \n",
       "L 199.027314 106.53253 \n",
       "L 197.906832 106.53253 \n",
       "L 196.78635 106.53253 \n",
       "L 195.665868 106.53253 \n",
       "L 194.545386 106.53253 \n",
       "L 193.424904 106.53253 \n",
       "L 192.304422 106.53253 \n",
       "L 191.18394 106.53253 \n",
       "L 190.063458 106.53253 \n",
       "L 188.942976 106.53253 \n",
       "L 187.822494 106.53253 \n",
       "L 186.702012 106.53253 \n",
       "L 185.58153 106.53253 \n",
       "L 184.461049 106.53253 \n",
       "L 183.340567 106.53253 \n",
       "L 182.220085 106.53253 \n",
       "L 181.099603 106.53253 \n",
       "L 179.979121 106.53253 \n",
       "L 178.858639 106.53253 \n",
       "L 177.738157 106.53253 \n",
       "L 176.617675 106.53253 \n",
       "L 175.497193 106.53253 \n",
       "L 174.376711 106.53253 \n",
       "L 173.256229 106.53253 \n",
       "L 172.135747 106.53253 \n",
       "L 171.015265 106.53253 \n",
       "L 169.894784 106.53253 \n",
       "L 168.774302 106.53253 \n",
       "L 167.65382 106.53253 \n",
       "L 166.533338 106.53253 \n",
       "L 165.412856 106.53253 \n",
       "L 164.292374 106.53253 \n",
       "L 163.171892 106.53253 \n",
       "L 162.05141 106.53253 \n",
       "L 160.930928 106.53253 \n",
       "L 159.810446 106.53253 \n",
       "L 158.689964 106.53253 \n",
       "L 157.569482 106.53253 \n",
       "L 156.449 106.53253 \n",
       "L 155.328518 106.53253 \n",
       "L 154.208037 106.53253 \n",
       "L 153.087555 106.53253 \n",
       "L 151.967073 106.53253 \n",
       "L 150.846591 106.53253 \n",
       "L 149.726109 106.53253 \n",
       "L 148.605627 106.53253 \n",
       "L 147.485145 106.53253 \n",
       "L 146.364663 106.53253 \n",
       "L 145.244181 106.53253 \n",
       "L 144.123699 106.53253 \n",
       "L 143.003217 106.53253 \n",
       "L 141.882735 106.53253 \n",
       "L 140.762253 106.53253 \n",
       "L 139.641771 106.53253 \n",
       "L 138.52129 106.53253 \n",
       "L 137.400808 106.53253 \n",
       "L 136.280326 106.53253 \n",
       "L 135.159844 106.53253 \n",
       "L 134.039362 106.53253 \n",
       "L 132.91888 106.53253 \n",
       "L 131.798398 106.53253 \n",
       "L 130.677916 106.53253 \n",
       "L 129.557434 106.53253 \n",
       "L 128.436952 106.53253 \n",
       "L 127.31647 106.53253 \n",
       "L 126.195988 106.53253 \n",
       "L 125.075506 106.53253 \n",
       "L 123.955024 106.53253 \n",
       "L 122.834543 106.53253 \n",
       "L 121.714061 106.53253 \n",
       "L 120.593579 106.53253 \n",
       "L 119.473097 106.53253 \n",
       "L 118.352615 106.53253 \n",
       "L 117.232133 106.53253 \n",
       "L 116.111651 106.53253 \n",
       "L 114.991169 106.53253 \n",
       "L 113.870687 106.53253 \n",
       "L 112.750205 106.53253 \n",
       "L 111.629723 106.53253 \n",
       "L 110.509241 106.53253 \n",
       "L 109.388759 106.53253 \n",
       "L 108.268277 106.53253 \n",
       "L 107.147796 106.53253 \n",
       "L 106.027314 106.53253 \n",
       "L 104.906832 106.53253 \n",
       "L 103.78635 106.53253 \n",
       "L 102.665868 106.53253 \n",
       "L 101.545386 106.53253 \n",
       "L 100.424904 106.53253 \n",
       "L 99.304422 106.53253 \n",
       "L 98.18394 106.53253 \n",
       "L 97.063458 106.53253 \n",
       "L 95.942976 106.53253 \n",
       "L 94.822494 106.53253 \n",
       "L 93.702012 106.53253 \n",
       "L 92.58153 106.53253 \n",
       "L 91.461049 106.53253 \n",
       "L 90.340567 106.53253 \n",
       "L 89.220085 106.53253 \n",
       "L 88.099603 106.53253 \n",
       "L 86.979121 106.53253 \n",
       "L 85.858639 106.53253 \n",
       "L 84.738157 106.53253 \n",
       "L 83.617675 106.53253 \n",
       "L 82.497193 106.53253 \n",
       "L 81.376711 106.53253 \n",
       "L 80.256229 106.53253 \n",
       "L 79.135747 106.53253 \n",
       "L 78.015265 106.53253 \n",
       "L 76.894784 106.53253 \n",
       "L 75.774302 106.53253 \n",
       "L 74.65382 106.53253 \n",
       "L 73.533338 106.53253 \n",
       "L 72.412856 106.53253 \n",
       "L 71.292374 106.53253 \n",
       "L 70.171892 106.53253 \n",
       "L 69.05141 106.53253 \n",
       "L 67.930928 106.53253 \n",
       "L 66.810446 106.53253 \n",
       "L 65.689964 106.53253 \n",
       "L 64.569482 106.53253 \n",
       "L 63.449 106.53253 \n",
       "L 62.328518 106.53253 \n",
       "L 61.208037 106.53253 \n",
       "L 60.087555 106.53253 \n",
       "L 58.967073 106.53253 \n",
       "L 57.846591 106.53253 \n",
       "L 56.726109 106.53253 \n",
       "L 55.605627 106.53253 \n",
       "L 54.485145 106.53253 \n",
       "L 53.364663 106.53253 \n",
       "L 52.244181 106.53253 \n",
       "L 51.123699 106.53253 \n",
       "L 50.003217 106.53253 \n",
       "L 48.882735 106.53253 \n",
       "L 47.762253 106.53253 \n",
       "L 46.641771 106.53253 \n",
       "L 45.52129 106.53253 \n",
       "L 44.400808 106.53253 \n",
       "L 43.280326 106.53253 \n",
       "L 42.159844 106.53253 \n",
       "L 42.159844 107.624096 \n",
       "L 42.159844 108.715663 \n",
       "L 42.159844 109.807229 \n",
       "L 42.159844 110.898795 \n",
       "L 42.159844 111.990361 \n",
       "L 42.159844 113.081928 \n",
       "L 42.159844 114.173494 \n",
       "L 42.159844 115.26506 \n",
       "L 42.159844 116.356627 \n",
       "L 42.159844 117.448193 \n",
       "L 42.159844 118.539759 \n",
       "L 42.159844 119.631325 \n",
       "L 42.159844 120.722892 \n",
       "L 42.159844 121.814458 \n",
       "L 42.159844 122.906024 \n",
       "L 42.159844 123.99759 \n",
       "L 42.159844 125.089157 \n",
       "L 42.159844 126.180723 \n",
       "L 42.159844 127.272289 \n",
       "L 42.159844 128.363855 \n",
       "L 42.159844 129.455422 \n",
       "L 42.159844 130.546988 \n",
       "L 42.159844 131.638554 \n",
       "L 42.159844 132.73012 \n",
       "L 42.159844 133.821687 \n",
       "L 42.159844 134.913253 \n",
       "L 42.159844 136.004819 \n",
       "L 42.159844 137.096386 \n",
       "L 42.159844 138.187952 \n",
       "L 42.159844 139.279518 \n",
       "L 42.159844 140.371084 \n",
       "L 42.159844 141.462651 \n",
       "L 42.159844 142.554217 \n",
       "L 42.159844 143.645783 \n",
       "L 42.159844 144.737349 \n",
       "L 42.159844 145.828916 \n",
       "L 42.159844 146.920482 \n",
       "L 42.159844 148.012048 \n",
       "L 42.159844 149.103614 \n",
       "L 42.159844 150.195181 \n",
       "L 42.159844 151.286747 \n",
       "L 42.159844 152.378313 \n",
       "L 42.159844 153.46988 \n",
       "L 42.159844 154.561446 \n",
       "L 42.159844 155.653012 \n",
       "L 42.159844 156.744578 \n",
       "L 42.159844 157.836145 \n",
       "L 42.159844 158.927711 \n",
       "L 42.159844 160.019277 \n",
       "L 42.159844 161.110843 \n",
       "L 42.159844 162.20241 \n",
       "L 42.159844 163.293976 \n",
       "L 42.159844 164.385542 \n",
       "L 42.159844 165.477108 \n",
       "L 42.159844 166.568675 \n",
       "L 42.159844 167.660241 \n",
       "L 42.159844 168.751807 \n",
       "L 42.159844 169.843373 \n",
       "L 42.159844 170.93494 \n",
       "L 42.159844 172.026506 \n",
       "L 42.159844 173.118072 \n",
       "L 42.159844 174.209639 \n",
       "L 42.159844 175.301205 \n",
       "L 42.159844 176.392771 \n",
       "L 42.159844 177.484337 \n",
       "L 42.159844 178.575904 \n",
       "L 42.159844 179.66747 \n",
       "L 42.159844 180.759036 \n",
       "L 42.159844 181.850602 \n",
       "L 42.159844 182.942169 \n",
       "L 42.159844 184.033735 \n",
       "L 42.159844 185.125301 \n",
       "L 42.159844 186.216867 \n",
       "L 42.159844 187.308434 \n",
       "L 42.159844 188.4 \n",
       "L 42.159844 189.491566 \n",
       "L 42.159844 190.583133 \n",
       "L 42.159844 191.674699 \n",
       "L 42.159844 192.766265 \n",
       "L 42.159844 193.857831 \n",
       "L 42.159844 194.949398 \n",
       "L 42.159844 195.222289 \n",
       "z\n",
       "\" style=\"fill:#9467bd;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_5\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 43.280326 106.53253 \n",
       "L 44.400808 106.53253 \n",
       "L 45.52129 106.53253 \n",
       "L 46.641771 106.53253 \n",
       "L 47.762253 106.53253 \n",
       "L 48.882735 106.53253 \n",
       "L 50.003217 106.53253 \n",
       "L 51.123699 106.53253 \n",
       "L 52.244181 106.53253 \n",
       "L 53.364663 106.53253 \n",
       "L 54.485145 106.53253 \n",
       "L 55.605627 106.53253 \n",
       "L 56.726109 106.53253 \n",
       "L 57.846591 106.53253 \n",
       "L 58.967073 106.53253 \n",
       "L 60.087555 106.53253 \n",
       "L 61.208037 106.53253 \n",
       "L 62.328518 106.53253 \n",
       "L 63.449 106.53253 \n",
       "L 64.569482 106.53253 \n",
       "L 65.689964 106.53253 \n",
       "L 66.810446 106.53253 \n",
       "L 67.930928 106.53253 \n",
       "L 69.05141 106.53253 \n",
       "L 70.171892 106.53253 \n",
       "L 71.292374 106.53253 \n",
       "L 72.412856 106.53253 \n",
       "L 73.533338 106.53253 \n",
       "L 74.65382 106.53253 \n",
       "L 75.774302 106.53253 \n",
       "L 76.894784 106.53253 \n",
       "L 78.015265 106.53253 \n",
       "L 79.135747 106.53253 \n",
       "L 80.256229 106.53253 \n",
       "L 81.376711 106.53253 \n",
       "L 82.497193 106.53253 \n",
       "L 83.617675 106.53253 \n",
       "L 84.738157 106.53253 \n",
       "L 85.858639 106.53253 \n",
       "L 86.979121 106.53253 \n",
       "L 88.099603 106.53253 \n",
       "L 89.220085 106.53253 \n",
       "L 90.340567 106.53253 \n",
       "L 91.461049 106.53253 \n",
       "L 92.58153 106.53253 \n",
       "L 93.702012 106.53253 \n",
       "L 94.822494 106.53253 \n",
       "L 95.942976 106.53253 \n",
       "L 97.063458 106.53253 \n",
       "L 98.18394 106.53253 \n",
       "L 99.304422 106.53253 \n",
       "L 100.424904 106.53253 \n",
       "L 101.545386 106.53253 \n",
       "L 102.665868 106.53253 \n",
       "L 103.78635 106.53253 \n",
       "L 104.906832 106.53253 \n",
       "L 106.027314 106.53253 \n",
       "L 107.147796 106.53253 \n",
       "L 108.268277 106.53253 \n",
       "L 109.388759 106.53253 \n",
       "L 110.509241 106.53253 \n",
       "L 111.629723 106.53253 \n",
       "L 112.750205 106.53253 \n",
       "L 113.870687 106.53253 \n",
       "L 114.991169 106.53253 \n",
       "L 116.111651 106.53253 \n",
       "L 117.232133 106.53253 \n",
       "L 118.352615 106.53253 \n",
       "L 119.473097 106.53253 \n",
       "L 120.593579 106.53253 \n",
       "L 121.714061 106.53253 \n",
       "L 122.834543 106.53253 \n",
       "L 123.955024 106.53253 \n",
       "L 125.075506 106.53253 \n",
       "L 126.195988 106.53253 \n",
       "L 127.31647 106.53253 \n",
       "L 128.436952 106.53253 \n",
       "L 129.557434 106.53253 \n",
       "L 130.677916 106.53253 \n",
       "L 131.798398 106.53253 \n",
       "L 132.91888 106.53253 \n",
       "L 134.039362 106.53253 \n",
       "L 135.159844 106.53253 \n",
       "L 136.280326 106.53253 \n",
       "L 137.400808 106.53253 \n",
       "L 138.52129 106.53253 \n",
       "L 139.641771 106.53253 \n",
       "L 140.762253 106.53253 \n",
       "L 141.882735 106.53253 \n",
       "L 143.003217 106.53253 \n",
       "L 144.123699 106.53253 \n",
       "L 145.244181 106.53253 \n",
       "L 146.364663 106.53253 \n",
       "L 147.485145 106.53253 \n",
       "L 148.605627 106.53253 \n",
       "L 149.726109 106.53253 \n",
       "L 150.846591 106.53253 \n",
       "L 151.967073 106.53253 \n",
       "L 153.087555 106.53253 \n",
       "L 154.208037 106.53253 \n",
       "L 155.328518 106.53253 \n",
       "L 156.449 106.53253 \n",
       "L 157.569482 106.53253 \n",
       "L 158.689964 106.53253 \n",
       "L 159.810446 106.53253 \n",
       "L 160.930928 106.53253 \n",
       "L 162.05141 106.53253 \n",
       "L 163.171892 106.53253 \n",
       "L 164.292374 106.53253 \n",
       "L 165.412856 106.53253 \n",
       "L 166.533338 106.53253 \n",
       "L 167.65382 106.53253 \n",
       "L 168.774302 106.53253 \n",
       "L 169.894784 106.53253 \n",
       "L 171.015265 106.53253 \n",
       "L 172.135747 106.53253 \n",
       "L 173.256229 106.53253 \n",
       "L 174.376711 106.53253 \n",
       "L 175.497193 106.53253 \n",
       "L 176.617675 106.53253 \n",
       "L 177.738157 106.53253 \n",
       "L 178.858639 106.53253 \n",
       "L 179.979121 106.53253 \n",
       "L 181.099603 106.53253 \n",
       "L 182.220085 106.53253 \n",
       "L 183.340567 106.53253 \n",
       "L 184.461049 106.53253 \n",
       "L 185.58153 106.53253 \n",
       "L 186.702012 106.53253 \n",
       "L 187.822494 106.53253 \n",
       "L 188.942976 106.53253 \n",
       "L 190.063458 106.53253 \n",
       "L 191.18394 106.53253 \n",
       "L 192.304422 106.53253 \n",
       "L 193.424904 106.53253 \n",
       "L 194.545386 106.53253 \n",
       "L 195.665868 106.53253 \n",
       "L 196.78635 106.53253 \n",
       "L 197.906832 106.53253 \n",
       "L 199.027314 106.53253 \n",
       "L 200.147796 106.53253 \n",
       "L 201.268277 106.53253 \n",
       "L 202.388759 106.53253 \n",
       "L 203.509241 106.53253 \n",
       "L 204.629723 106.53253 \n",
       "L 205.750205 106.53253 \n",
       "L 206.870687 106.53253 \n",
       "L 207.991169 106.53253 \n",
       "L 209.111651 106.53253 \n",
       "L 210.232133 106.53253 \n",
       "L 211.352615 106.53253 \n",
       "L 212.473097 106.53253 \n",
       "L 213.593579 106.53253 \n",
       "L 214.714061 106.53253 \n",
       "L 215.834543 106.53253 \n",
       "L 216.955024 106.53253 \n",
       "L 218.075506 106.53253 \n",
       "L 219.195988 106.53253 \n",
       "L 220.31647 106.53253 \n",
       "L 221.436952 106.53253 \n",
       "L 222.557434 106.53253 \n",
       "L 223.677916 106.53253 \n",
       "L 224.798398 106.53253 \n",
       "L 225.91888 106.53253 \n",
       "L 227.039362 106.53253 \n",
       "L 228.159844 106.53253 \n",
       "L 229.280326 106.53253 \n",
       "L 230.400808 106.53253 \n",
       "L 231.52129 106.53253 \n",
       "L 232.641771 106.53253 \n",
       "L 233.762253 106.53253 \n",
       "L 234.882735 106.53253 \n",
       "L 236.003217 106.53253 \n",
       "L 237.123699 106.53253 \n",
       "L 238.244181 106.53253 \n",
       "L 239.364663 106.53253 \n",
       "L 240.485145 106.53253 \n",
       "L 241.605627 106.53253 \n",
       "L 242.726109 106.53253 \n",
       "L 243.846591 106.53253 \n",
       "L 244.967073 106.53253 \n",
       "L 246.087555 106.53253 \n",
       "L 247.208037 106.53253 \n",
       "L 248.328518 106.53253 \n",
       "L 249.449 106.53253 \n",
       "L 250.569482 106.53253 \n",
       "L 251.689964 106.53253 \n",
       "L 252.810446 106.53253 \n",
       "L 253.930928 106.53253 \n",
       "L 255.05141 106.53253 \n",
       "L 256.171892 106.53253 \n",
       "L 257.292374 106.53253 \n",
       "L 258.412856 106.53253 \n",
       "L 259.533338 106.53253 \n",
       "L 260.65382 106.53253 \n",
       "L 261.774302 106.53253 \n",
       "L 262.894784 106.53253 \n",
       "L 264.015265 106.53253 \n",
       "L 265.135747 106.53253 \n",
       "L 266.256229 106.53253 \n",
       "L 267.376711 106.53253 \n",
       "L 268.497193 106.53253 \n",
       "L 269.617675 106.53253 \n",
       "L 270.738157 106.53253 \n",
       "L 271.858639 106.53253 \n",
       "L 272.979121 106.53253 \n",
       "L 274.099603 106.53253 \n",
       "L 275.220085 106.53253 \n",
       "L 276.340567 106.53253 \n",
       "L 277.461049 106.53253 \n",
       "L 278.58153 106.53253 \n",
       "L 279.702012 106.53253 \n",
       "L 280.822494 106.53253 \n",
       "L 281.942976 106.53253 \n",
       "L 283.063458 106.53253 \n",
       "L 284.18394 106.53253 \n",
       "L 285.304422 106.53253 \n",
       "L 286.424904 106.53253 \n",
       "L 287.545386 106.53253 \n",
       "L 288.665868 106.53253 \n",
       "L 289.78635 106.53253 \n",
       "L 290.906832 106.53253 \n",
       "L 292.027314 106.53253 \n",
       "L 293.147796 106.53253 \n",
       "L 294.268277 106.53253 \n",
       "L 295.388759 106.53253 \n",
       "L 296.509241 106.53253 \n",
       "L 297.629723 106.53253 \n",
       "L 298.750205 106.53253 \n",
       "L 299.870687 106.53253 \n",
       "L 300.991169 106.53253 \n",
       "L 302.111651 106.53253 \n",
       "L 303.232133 106.53253 \n",
       "L 304.352615 106.53253 \n",
       "L 305.473097 106.53253 \n",
       "L 306.593579 106.53253 \n",
       "L 307.714061 106.53253 \n",
       "L 308.834543 106.53253 \n",
       "L 309.955024 106.53253 \n",
       "L 311.075506 106.53253 \n",
       "L 312.195988 106.53253 \n",
       "L 313.31647 106.53253 \n",
       "L 314.436952 106.53253 \n",
       "L 315.557434 106.53253 \n",
       "L 316.677916 106.53253 \n",
       "L 317.798398 106.53253 \n",
       "L 318.91888 106.53253 \n",
       "L 320.039362 106.53253 \n",
       "L 321.159844 106.53253 \n",
       "L 321.159844 106.259639 \n",
       "L 320.039362 106.259639 \n",
       "L 318.91888 106.259639 \n",
       "L 317.798398 106.259639 \n",
       "L 316.677916 106.259639 \n",
       "L 315.557434 106.259639 \n",
       "L 314.436952 106.259639 \n",
       "L 313.31647 106.259639 \n",
       "L 312.195988 106.259639 \n",
       "L 311.075506 106.259639 \n",
       "L 309.955024 106.259639 \n",
       "L 308.834543 106.259639 \n",
       "L 307.714061 106.259639 \n",
       "L 306.593579 106.259639 \n",
       "L 305.473097 106.259639 \n",
       "L 304.352615 106.259639 \n",
       "L 303.232133 106.259639 \n",
       "L 302.111651 106.259639 \n",
       "L 300.991169 106.259639 \n",
       "L 299.870687 106.259639 \n",
       "L 298.750205 106.259639 \n",
       "L 297.629723 106.259639 \n",
       "L 296.509241 106.259639 \n",
       "L 295.388759 106.259639 \n",
       "L 294.268277 106.259639 \n",
       "L 293.147796 106.259639 \n",
       "L 292.027314 106.259639 \n",
       "L 290.906832 106.259639 \n",
       "L 289.78635 106.259639 \n",
       "L 288.665868 106.259639 \n",
       "L 287.545386 106.259639 \n",
       "L 286.424904 106.259639 \n",
       "L 285.304422 106.259639 \n",
       "L 284.18394 106.259639 \n",
       "L 283.063458 106.259639 \n",
       "L 281.942976 106.259639 \n",
       "L 280.822494 106.259639 \n",
       "L 279.702012 106.259639 \n",
       "L 278.58153 106.259639 \n",
       "L 277.461049 106.259639 \n",
       "L 276.340567 106.259639 \n",
       "L 275.220085 106.259639 \n",
       "L 274.099603 106.259639 \n",
       "L 272.979121 106.259639 \n",
       "L 271.858639 106.259639 \n",
       "L 270.738157 106.259639 \n",
       "L 269.617675 106.259639 \n",
       "L 268.497193 106.259639 \n",
       "L 267.376711 106.259639 \n",
       "L 266.256229 106.259639 \n",
       "L 265.135747 106.259639 \n",
       "L 264.015265 106.259639 \n",
       "L 262.894784 106.259639 \n",
       "L 261.774302 106.259639 \n",
       "L 260.65382 106.259639 \n",
       "L 259.533338 106.259639 \n",
       "L 258.412856 106.259639 \n",
       "L 257.292374 106.259639 \n",
       "L 256.171892 106.259639 \n",
       "L 255.05141 106.259639 \n",
       "L 253.930928 106.259639 \n",
       "L 252.810446 106.259639 \n",
       "L 251.689964 106.259639 \n",
       "L 250.569482 106.259639 \n",
       "L 249.449 106.259639 \n",
       "L 248.328518 106.259639 \n",
       "L 247.208037 106.259639 \n",
       "L 246.087555 106.259639 \n",
       "L 244.967073 106.259639 \n",
       "L 243.846591 106.259639 \n",
       "L 242.726109 106.259639 \n",
       "L 241.605627 106.259639 \n",
       "L 240.485145 106.259639 \n",
       "L 239.364663 106.259639 \n",
       "L 238.244181 106.259639 \n",
       "L 237.123699 106.259639 \n",
       "L 236.003217 106.259639 \n",
       "L 234.882735 106.259639 \n",
       "L 233.762253 106.259639 \n",
       "L 232.641771 106.259639 \n",
       "L 231.52129 106.259639 \n",
       "L 230.400808 106.259639 \n",
       "L 229.280326 106.259639 \n",
       "L 228.159844 106.259639 \n",
       "L 227.039362 106.259639 \n",
       "L 225.91888 106.259639 \n",
       "L 224.798398 106.259639 \n",
       "L 223.677916 106.259639 \n",
       "L 222.557434 106.259639 \n",
       "L 221.436952 106.259639 \n",
       "L 220.31647 106.259639 \n",
       "L 219.195988 106.259639 \n",
       "L 218.075506 106.259639 \n",
       "L 216.955024 106.259639 \n",
       "L 215.834543 106.259639 \n",
       "L 214.714061 106.259639 \n",
       "L 213.593579 106.259639 \n",
       "L 212.473097 106.259639 \n",
       "L 211.352615 106.259639 \n",
       "L 210.232133 106.259639 \n",
       "L 209.111651 106.259639 \n",
       "L 207.991169 106.259639 \n",
       "L 206.870687 106.259639 \n",
       "L 205.750205 106.259639 \n",
       "L 204.629723 106.259639 \n",
       "L 203.509241 106.259639 \n",
       "L 202.388759 106.259639 \n",
       "L 201.268277 106.259639 \n",
       "L 200.147796 106.259639 \n",
       "L 199.027314 106.259639 \n",
       "L 197.906832 106.259639 \n",
       "L 196.78635 106.259639 \n",
       "L 195.665868 106.259639 \n",
       "L 194.545386 106.259639 \n",
       "L 193.424904 106.259639 \n",
       "L 192.304422 106.259639 \n",
       "L 191.18394 106.259639 \n",
       "L 190.063458 106.259639 \n",
       "L 188.942976 106.259639 \n",
       "L 187.822494 106.259639 \n",
       "L 186.702012 106.259639 \n",
       "L 185.58153 106.259639 \n",
       "L 184.461049 106.259639 \n",
       "L 183.340567 106.259639 \n",
       "L 182.220085 106.259639 \n",
       "L 181.099603 106.259639 \n",
       "L 179.979121 106.259639 \n",
       "L 178.858639 106.259639 \n",
       "L 177.738157 106.259639 \n",
       "L 176.617675 106.259639 \n",
       "L 175.497193 106.259639 \n",
       "L 174.376711 106.259639 \n",
       "L 173.256229 106.259639 \n",
       "L 172.135747 106.259639 \n",
       "L 171.015265 106.259639 \n",
       "L 169.894784 106.259639 \n",
       "L 168.774302 106.259639 \n",
       "L 167.65382 106.259639 \n",
       "L 166.533338 106.259639 \n",
       "L 165.412856 106.259639 \n",
       "L 164.292374 106.259639 \n",
       "L 163.171892 106.259639 \n",
       "L 162.05141 106.259639 \n",
       "L 160.930928 106.259639 \n",
       "L 159.810446 106.259639 \n",
       "L 158.689964 106.259639 \n",
       "L 157.569482 106.259639 \n",
       "L 156.449 106.259639 \n",
       "L 155.328518 106.259639 \n",
       "L 154.208037 106.259639 \n",
       "L 153.087555 106.259639 \n",
       "L 151.967073 106.259639 \n",
       "L 150.846591 106.259639 \n",
       "L 149.726109 106.259639 \n",
       "L 148.605627 106.259639 \n",
       "L 147.485145 106.259639 \n",
       "L 146.364663 106.259639 \n",
       "L 145.244181 106.259639 \n",
       "L 144.123699 106.259639 \n",
       "L 143.003217 106.259639 \n",
       "L 141.882735 106.259639 \n",
       "L 140.762253 106.259639 \n",
       "L 139.641771 106.259639 \n",
       "L 138.52129 106.259639 \n",
       "L 137.400808 106.259639 \n",
       "L 136.280326 106.259639 \n",
       "L 135.159844 106.259639 \n",
       "L 134.039362 106.259639 \n",
       "L 132.91888 106.259639 \n",
       "L 131.798398 106.259639 \n",
       "L 130.677916 106.259639 \n",
       "L 129.557434 106.259639 \n",
       "L 128.436952 106.259639 \n",
       "L 127.31647 106.259639 \n",
       "L 126.195988 106.259639 \n",
       "L 125.075506 106.259639 \n",
       "L 123.955024 106.259639 \n",
       "L 122.834543 106.259639 \n",
       "L 121.714061 106.259639 \n",
       "L 120.593579 106.259639 \n",
       "L 119.473097 106.259639 \n",
       "L 118.352615 106.259639 \n",
       "L 117.232133 106.259639 \n",
       "L 116.111651 106.259639 \n",
       "L 114.991169 106.259639 \n",
       "L 113.870687 106.259639 \n",
       "L 112.750205 106.259639 \n",
       "L 111.629723 106.259639 \n",
       "L 110.509241 106.259639 \n",
       "L 109.388759 106.259639 \n",
       "L 108.268277 106.259639 \n",
       "L 107.147796 106.259639 \n",
       "L 106.027314 106.259639 \n",
       "L 104.906832 106.259639 \n",
       "L 103.78635 106.259639 \n",
       "L 102.665868 106.259639 \n",
       "L 101.545386 106.259639 \n",
       "L 100.424904 106.259639 \n",
       "L 99.304422 106.259639 \n",
       "L 98.18394 106.259639 \n",
       "L 97.063458 106.259639 \n",
       "L 95.942976 106.259639 \n",
       "L 94.822494 106.259639 \n",
       "L 93.702012 106.259639 \n",
       "L 92.58153 106.259639 \n",
       "L 91.461049 106.259639 \n",
       "L 90.340567 106.259639 \n",
       "L 89.220085 106.259639 \n",
       "L 88.099603 106.259639 \n",
       "L 86.979121 106.259639 \n",
       "L 85.858639 106.259639 \n",
       "L 84.738157 106.259639 \n",
       "L 83.617675 106.259639 \n",
       "L 82.497193 106.259639 \n",
       "L 81.376711 106.259639 \n",
       "L 80.256229 106.259639 \n",
       "L 79.135747 106.259639 \n",
       "L 78.015265 106.259639 \n",
       "L 76.894784 106.259639 \n",
       "L 75.774302 106.259639 \n",
       "L 74.65382 106.259639 \n",
       "L 73.533338 106.259639 \n",
       "L 72.412856 106.259639 \n",
       "L 71.292374 106.259639 \n",
       "L 70.171892 106.259639 \n",
       "L 69.05141 106.259639 \n",
       "L 67.930928 106.259639 \n",
       "L 66.810446 106.259639 \n",
       "L 65.689964 106.259639 \n",
       "L 64.569482 106.259639 \n",
       "L 63.449 106.259639 \n",
       "L 62.328518 106.259639 \n",
       "L 61.208037 106.259639 \n",
       "L 60.087555 106.259639 \n",
       "L 58.967073 106.259639 \n",
       "L 57.846591 106.259639 \n",
       "L 56.726109 106.259639 \n",
       "L 55.605627 106.259639 \n",
       "L 54.485145 106.259639 \n",
       "L 53.364663 106.259639 \n",
       "L 52.244181 106.259639 \n",
       "L 51.123699 106.259639 \n",
       "L 50.003217 106.259639 \n",
       "L 48.882735 106.259639 \n",
       "L 47.762253 106.259639 \n",
       "L 46.641771 106.259639 \n",
       "L 45.52129 106.259639 \n",
       "L 44.400808 106.259639 \n",
       "L 43.280326 106.259639 \n",
       "L 42.159844 106.259639 \n",
       "L 42.159844 106.53253 \n",
       "z\n",
       "\" style=\"fill:#8c564b;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_6\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 43.280326 106.259639 \n",
       "L 44.400808 106.259639 \n",
       "L 45.52129 106.259639 \n",
       "L 46.641771 106.259639 \n",
       "L 47.762253 106.259639 \n",
       "L 48.882735 106.259639 \n",
       "L 50.003217 106.259639 \n",
       "L 51.123699 106.259639 \n",
       "L 52.244181 106.259639 \n",
       "L 53.364663 106.259639 \n",
       "L 54.485145 106.259639 \n",
       "L 55.605627 106.259639 \n",
       "L 56.726109 106.259639 \n",
       "L 57.846591 106.259639 \n",
       "L 58.967073 106.259639 \n",
       "L 60.087555 106.259639 \n",
       "L 61.208037 106.259639 \n",
       "L 62.328518 106.259639 \n",
       "L 63.449 106.259639 \n",
       "L 64.569482 106.259639 \n",
       "L 65.689964 106.259639 \n",
       "L 66.810446 106.259639 \n",
       "L 67.930928 106.259639 \n",
       "L 69.05141 106.259639 \n",
       "L 70.171892 106.259639 \n",
       "L 71.292374 106.259639 \n",
       "L 72.412856 106.259639 \n",
       "L 73.533338 106.259639 \n",
       "L 74.65382 106.259639 \n",
       "L 75.774302 106.259639 \n",
       "L 76.894784 106.259639 \n",
       "L 78.015265 106.259639 \n",
       "L 79.135747 106.259639 \n",
       "L 80.256229 106.259639 \n",
       "L 81.376711 106.259639 \n",
       "L 82.497193 106.259639 \n",
       "L 83.617675 106.259639 \n",
       "L 84.738157 106.259639 \n",
       "L 85.858639 106.259639 \n",
       "L 86.979121 106.259639 \n",
       "L 88.099603 106.259639 \n",
       "L 89.220085 106.259639 \n",
       "L 90.340567 106.259639 \n",
       "L 91.461049 106.259639 \n",
       "L 92.58153 106.259639 \n",
       "L 93.702012 106.259639 \n",
       "L 94.822494 106.259639 \n",
       "L 95.942976 106.259639 \n",
       "L 97.063458 106.259639 \n",
       "L 98.18394 106.259639 \n",
       "L 99.304422 106.259639 \n",
       "L 100.424904 106.259639 \n",
       "L 101.545386 106.259639 \n",
       "L 102.665868 106.259639 \n",
       "L 103.78635 106.259639 \n",
       "L 104.906832 106.259639 \n",
       "L 106.027314 106.259639 \n",
       "L 107.147796 106.259639 \n",
       "L 108.268277 106.259639 \n",
       "L 109.388759 106.259639 \n",
       "L 110.509241 106.259639 \n",
       "L 111.629723 106.259639 \n",
       "L 112.750205 106.259639 \n",
       "L 113.870687 106.259639 \n",
       "L 114.991169 106.259639 \n",
       "L 116.111651 106.259639 \n",
       "L 117.232133 106.259639 \n",
       "L 118.352615 106.259639 \n",
       "L 119.473097 106.259639 \n",
       "L 120.593579 106.259639 \n",
       "L 121.714061 106.259639 \n",
       "L 122.834543 106.259639 \n",
       "L 123.955024 106.259639 \n",
       "L 125.075506 106.259639 \n",
       "L 126.195988 106.259639 \n",
       "L 127.31647 106.259639 \n",
       "L 128.436952 106.259639 \n",
       "L 129.557434 106.259639 \n",
       "L 130.677916 106.259639 \n",
       "L 131.798398 106.259639 \n",
       "L 132.91888 106.259639 \n",
       "L 134.039362 106.259639 \n",
       "L 135.159844 106.259639 \n",
       "L 136.280326 106.259639 \n",
       "L 137.400808 106.259639 \n",
       "L 138.52129 106.259639 \n",
       "L 139.641771 106.259639 \n",
       "L 140.762253 106.259639 \n",
       "L 141.882735 106.259639 \n",
       "L 143.003217 106.259639 \n",
       "L 144.123699 106.259639 \n",
       "L 145.244181 106.259639 \n",
       "L 146.364663 106.259639 \n",
       "L 147.485145 106.259639 \n",
       "L 148.605627 106.259639 \n",
       "L 149.726109 106.259639 \n",
       "L 150.846591 106.259639 \n",
       "L 151.967073 106.259639 \n",
       "L 153.087555 106.259639 \n",
       "L 154.208037 106.259639 \n",
       "L 155.328518 106.259639 \n",
       "L 156.449 106.259639 \n",
       "L 157.569482 106.259639 \n",
       "L 158.689964 106.259639 \n",
       "L 159.810446 106.259639 \n",
       "L 160.930928 106.259639 \n",
       "L 162.05141 106.259639 \n",
       "L 163.171892 106.259639 \n",
       "L 164.292374 106.259639 \n",
       "L 165.412856 106.259639 \n",
       "L 166.533338 106.259639 \n",
       "L 167.65382 106.259639 \n",
       "L 168.774302 106.259639 \n",
       "L 169.894784 106.259639 \n",
       "L 171.015265 106.259639 \n",
       "L 172.135747 106.259639 \n",
       "L 173.256229 106.259639 \n",
       "L 174.376711 106.259639 \n",
       "L 175.497193 106.259639 \n",
       "L 176.617675 106.259639 \n",
       "L 177.738157 106.259639 \n",
       "L 178.858639 106.259639 \n",
       "L 179.979121 106.259639 \n",
       "L 181.099603 106.259639 \n",
       "L 182.220085 106.259639 \n",
       "L 183.340567 106.259639 \n",
       "L 184.461049 106.259639 \n",
       "L 185.58153 106.259639 \n",
       "L 186.702012 106.259639 \n",
       "L 187.822494 106.259639 \n",
       "L 188.942976 106.259639 \n",
       "L 190.063458 106.259639 \n",
       "L 191.18394 106.259639 \n",
       "L 192.304422 106.259639 \n",
       "L 193.424904 106.259639 \n",
       "L 194.545386 106.259639 \n",
       "L 195.665868 106.259639 \n",
       "L 196.78635 106.259639 \n",
       "L 197.906832 106.259639 \n",
       "L 199.027314 106.259639 \n",
       "L 200.147796 106.259639 \n",
       "L 201.268277 106.259639 \n",
       "L 202.388759 106.259639 \n",
       "L 203.509241 106.259639 \n",
       "L 204.629723 106.259639 \n",
       "L 205.750205 106.259639 \n",
       "L 206.870687 106.259639 \n",
       "L 207.991169 106.259639 \n",
       "L 209.111651 106.259639 \n",
       "L 210.232133 106.259639 \n",
       "L 211.352615 106.259639 \n",
       "L 212.473097 106.259639 \n",
       "L 213.593579 106.259639 \n",
       "L 214.714061 106.259639 \n",
       "L 215.834543 106.259639 \n",
       "L 216.955024 106.259639 \n",
       "L 218.075506 106.259639 \n",
       "L 219.195988 106.259639 \n",
       "L 220.31647 106.259639 \n",
       "L 221.436952 106.259639 \n",
       "L 222.557434 106.259639 \n",
       "L 223.677916 106.259639 \n",
       "L 224.798398 106.259639 \n",
       "L 225.91888 106.259639 \n",
       "L 227.039362 106.259639 \n",
       "L 228.159844 106.259639 \n",
       "L 229.280326 106.259639 \n",
       "L 230.400808 106.259639 \n",
       "L 231.52129 106.259639 \n",
       "L 232.641771 106.259639 \n",
       "L 233.762253 106.259639 \n",
       "L 234.882735 106.259639 \n",
       "L 236.003217 106.259639 \n",
       "L 237.123699 106.259639 \n",
       "L 238.244181 106.259639 \n",
       "L 239.364663 106.259639 \n",
       "L 240.485145 106.259639 \n",
       "L 241.605627 106.259639 \n",
       "L 242.726109 106.259639 \n",
       "L 243.846591 106.259639 \n",
       "L 244.967073 106.259639 \n",
       "L 246.087555 106.259639 \n",
       "L 247.208037 106.259639 \n",
       "L 248.328518 106.259639 \n",
       "L 249.449 106.259639 \n",
       "L 250.569482 106.259639 \n",
       "L 251.689964 106.259639 \n",
       "L 252.810446 106.259639 \n",
       "L 253.930928 106.259639 \n",
       "L 255.05141 106.259639 \n",
       "L 256.171892 106.259639 \n",
       "L 257.292374 106.259639 \n",
       "L 258.412856 106.259639 \n",
       "L 259.533338 106.259639 \n",
       "L 260.65382 106.259639 \n",
       "L 261.774302 106.259639 \n",
       "L 262.894784 106.259639 \n",
       "L 264.015265 106.259639 \n",
       "L 265.135747 106.259639 \n",
       "L 266.256229 106.259639 \n",
       "L 267.376711 106.259639 \n",
       "L 268.497193 106.259639 \n",
       "L 269.617675 106.259639 \n",
       "L 270.738157 106.259639 \n",
       "L 271.858639 106.259639 \n",
       "L 272.979121 106.259639 \n",
       "L 274.099603 106.259639 \n",
       "L 275.220085 106.259639 \n",
       "L 276.340567 106.259639 \n",
       "L 277.461049 106.259639 \n",
       "L 278.58153 106.259639 \n",
       "L 279.702012 106.259639 \n",
       "L 280.822494 106.259639 \n",
       "L 281.942976 106.259639 \n",
       "L 283.063458 106.259639 \n",
       "L 284.18394 106.259639 \n",
       "L 285.304422 106.259639 \n",
       "L 286.424904 106.259639 \n",
       "L 287.545386 106.259639 \n",
       "L 288.665868 106.259639 \n",
       "L 289.78635 106.259639 \n",
       "L 290.906832 106.259639 \n",
       "L 292.027314 106.259639 \n",
       "L 293.147796 106.259639 \n",
       "L 294.268277 106.259639 \n",
       "L 295.388759 106.259639 \n",
       "L 296.509241 106.259639 \n",
       "L 297.629723 106.259639 \n",
       "L 298.750205 106.259639 \n",
       "L 299.870687 106.259639 \n",
       "L 300.991169 106.259639 \n",
       "L 302.111651 106.259639 \n",
       "L 303.232133 106.259639 \n",
       "L 304.352615 106.259639 \n",
       "L 305.473097 106.259639 \n",
       "L 306.593579 106.259639 \n",
       "L 307.714061 106.259639 \n",
       "L 308.834543 106.259639 \n",
       "L 309.955024 106.259639 \n",
       "L 311.075506 106.259639 \n",
       "L 312.195988 106.259639 \n",
       "L 313.31647 106.259639 \n",
       "L 314.436952 106.259639 \n",
       "L 315.557434 106.259639 \n",
       "L 316.677916 106.259639 \n",
       "L 317.798398 106.259639 \n",
       "L 318.91888 106.259639 \n",
       "L 320.039362 106.259639 \n",
       "L 321.159844 106.259639 \n",
       "L 321.159844 105.986747 \n",
       "L 320.039362 105.986747 \n",
       "L 318.91888 105.986747 \n",
       "L 317.798398 105.986747 \n",
       "L 316.677916 105.986747 \n",
       "L 315.557434 105.986747 \n",
       "L 314.436952 105.986747 \n",
       "L 313.31647 105.986747 \n",
       "L 312.195988 105.986747 \n",
       "L 311.075506 105.986747 \n",
       "L 309.955024 105.986747 \n",
       "L 308.834543 105.986747 \n",
       "L 307.714061 105.986747 \n",
       "L 306.593579 105.986747 \n",
       "L 305.473097 105.986747 \n",
       "L 304.352615 105.986747 \n",
       "L 303.232133 105.986747 \n",
       "L 302.111651 105.986747 \n",
       "L 300.991169 105.986747 \n",
       "L 299.870687 105.986747 \n",
       "L 298.750205 105.986747 \n",
       "L 297.629723 105.986747 \n",
       "L 296.509241 105.986747 \n",
       "L 295.388759 105.986747 \n",
       "L 294.268277 105.986747 \n",
       "L 293.147796 105.986747 \n",
       "L 292.027314 105.986747 \n",
       "L 290.906832 105.986747 \n",
       "L 289.78635 105.986747 \n",
       "L 288.665868 105.986747 \n",
       "L 287.545386 105.986747 \n",
       "L 286.424904 105.986747 \n",
       "L 285.304422 105.986747 \n",
       "L 284.18394 105.986747 \n",
       "L 283.063458 105.986747 \n",
       "L 281.942976 105.986747 \n",
       "L 280.822494 105.986747 \n",
       "L 279.702012 105.986747 \n",
       "L 278.58153 105.986747 \n",
       "L 277.461049 105.986747 \n",
       "L 276.340567 105.986747 \n",
       "L 275.220085 105.986747 \n",
       "L 274.099603 105.986747 \n",
       "L 272.979121 105.986747 \n",
       "L 271.858639 105.986747 \n",
       "L 270.738157 105.986747 \n",
       "L 269.617675 105.986747 \n",
       "L 268.497193 105.986747 \n",
       "L 267.376711 105.986747 \n",
       "L 266.256229 105.986747 \n",
       "L 265.135747 105.986747 \n",
       "L 264.015265 105.986747 \n",
       "L 262.894784 105.986747 \n",
       "L 261.774302 105.986747 \n",
       "L 260.65382 105.986747 \n",
       "L 259.533338 105.986747 \n",
       "L 258.412856 105.986747 \n",
       "L 257.292374 105.986747 \n",
       "L 256.171892 105.986747 \n",
       "L 255.05141 105.986747 \n",
       "L 253.930928 105.986747 \n",
       "L 252.810446 105.986747 \n",
       "L 251.689964 105.986747 \n",
       "L 250.569482 105.986747 \n",
       "L 249.449 105.986747 \n",
       "L 248.328518 105.986747 \n",
       "L 247.208037 105.986747 \n",
       "L 246.087555 105.986747 \n",
       "L 244.967073 105.986747 \n",
       "L 243.846591 105.986747 \n",
       "L 242.726109 105.986747 \n",
       "L 241.605627 105.986747 \n",
       "L 240.485145 105.986747 \n",
       "L 239.364663 105.986747 \n",
       "L 238.244181 105.986747 \n",
       "L 237.123699 105.986747 \n",
       "L 236.003217 105.986747 \n",
       "L 234.882735 105.986747 \n",
       "L 233.762253 105.986747 \n",
       "L 232.641771 105.986747 \n",
       "L 231.52129 105.986747 \n",
       "L 230.400808 105.986747 \n",
       "L 229.280326 105.986747 \n",
       "L 228.159844 105.986747 \n",
       "L 227.039362 105.986747 \n",
       "L 225.91888 105.986747 \n",
       "L 224.798398 105.986747 \n",
       "L 223.677916 105.986747 \n",
       "L 222.557434 105.986747 \n",
       "L 221.436952 105.986747 \n",
       "L 220.31647 105.986747 \n",
       "L 219.195988 105.986747 \n",
       "L 218.075506 105.986747 \n",
       "L 216.955024 105.986747 \n",
       "L 215.834543 105.986747 \n",
       "L 214.714061 105.986747 \n",
       "L 213.593579 105.986747 \n",
       "L 212.473097 105.986747 \n",
       "L 211.352615 105.986747 \n",
       "L 210.232133 105.986747 \n",
       "L 209.111651 105.986747 \n",
       "L 207.991169 105.986747 \n",
       "L 206.870687 105.986747 \n",
       "L 205.750205 105.986747 \n",
       "L 204.629723 105.986747 \n",
       "L 203.509241 105.986747 \n",
       "L 202.388759 105.986747 \n",
       "L 201.268277 105.986747 \n",
       "L 200.147796 105.986747 \n",
       "L 199.027314 105.986747 \n",
       "L 197.906832 105.986747 \n",
       "L 196.78635 105.986747 \n",
       "L 195.665868 105.986747 \n",
       "L 194.545386 105.986747 \n",
       "L 193.424904 105.986747 \n",
       "L 192.304422 105.986747 \n",
       "L 191.18394 105.986747 \n",
       "L 190.063458 105.986747 \n",
       "L 188.942976 105.986747 \n",
       "L 187.822494 105.986747 \n",
       "L 186.702012 105.986747 \n",
       "L 185.58153 105.986747 \n",
       "L 184.461049 105.986747 \n",
       "L 183.340567 105.986747 \n",
       "L 182.220085 105.986747 \n",
       "L 181.099603 105.986747 \n",
       "L 179.979121 105.986747 \n",
       "L 178.858639 105.986747 \n",
       "L 177.738157 105.986747 \n",
       "L 176.617675 105.986747 \n",
       "L 175.497193 105.986747 \n",
       "L 174.376711 105.986747 \n",
       "L 173.256229 105.986747 \n",
       "L 172.135747 105.986747 \n",
       "L 171.015265 105.986747 \n",
       "L 169.894784 105.986747 \n",
       "L 168.774302 105.986747 \n",
       "L 167.65382 105.986747 \n",
       "L 166.533338 105.986747 \n",
       "L 165.412856 105.986747 \n",
       "L 164.292374 105.986747 \n",
       "L 163.171892 105.986747 \n",
       "L 162.05141 105.986747 \n",
       "L 160.930928 105.986747 \n",
       "L 159.810446 105.986747 \n",
       "L 158.689964 105.986747 \n",
       "L 157.569482 105.986747 \n",
       "L 156.449 105.986747 \n",
       "L 155.328518 105.986747 \n",
       "L 154.208037 105.986747 \n",
       "L 153.087555 105.986747 \n",
       "L 151.967073 105.986747 \n",
       "L 150.846591 105.986747 \n",
       "L 149.726109 105.986747 \n",
       "L 148.605627 105.986747 \n",
       "L 147.485145 105.986747 \n",
       "L 146.364663 105.986747 \n",
       "L 145.244181 105.986747 \n",
       "L 144.123699 105.986747 \n",
       "L 143.003217 105.986747 \n",
       "L 141.882735 105.986747 \n",
       "L 140.762253 105.986747 \n",
       "L 139.641771 105.986747 \n",
       "L 138.52129 105.986747 \n",
       "L 137.400808 105.986747 \n",
       "L 136.280326 105.986747 \n",
       "L 135.159844 105.986747 \n",
       "L 134.039362 105.986747 \n",
       "L 132.91888 105.986747 \n",
       "L 131.798398 105.986747 \n",
       "L 130.677916 105.986747 \n",
       "L 129.557434 105.986747 \n",
       "L 128.436952 105.986747 \n",
       "L 127.31647 105.986747 \n",
       "L 126.195988 105.986747 \n",
       "L 125.075506 105.986747 \n",
       "L 123.955024 105.986747 \n",
       "L 122.834543 105.986747 \n",
       "L 121.714061 105.986747 \n",
       "L 120.593579 105.986747 \n",
       "L 119.473097 105.986747 \n",
       "L 118.352615 105.986747 \n",
       "L 117.232133 105.986747 \n",
       "L 116.111651 105.986747 \n",
       "L 114.991169 105.986747 \n",
       "L 113.870687 105.986747 \n",
       "L 112.750205 105.986747 \n",
       "L 111.629723 105.986747 \n",
       "L 110.509241 105.986747 \n",
       "L 109.388759 105.986747 \n",
       "L 108.268277 105.986747 \n",
       "L 107.147796 105.986747 \n",
       "L 106.027314 105.986747 \n",
       "L 104.906832 105.986747 \n",
       "L 103.78635 105.986747 \n",
       "L 102.665868 105.986747 \n",
       "L 101.545386 105.986747 \n",
       "L 100.424904 105.986747 \n",
       "L 99.304422 105.986747 \n",
       "L 98.18394 105.986747 \n",
       "L 97.063458 105.986747 \n",
       "L 95.942976 105.986747 \n",
       "L 94.822494 105.986747 \n",
       "L 93.702012 105.986747 \n",
       "L 92.58153 105.986747 \n",
       "L 91.461049 105.986747 \n",
       "L 90.340567 105.986747 \n",
       "L 89.220085 105.986747 \n",
       "L 88.099603 105.986747 \n",
       "L 86.979121 105.986747 \n",
       "L 85.858639 105.986747 \n",
       "L 84.738157 105.986747 \n",
       "L 83.617675 105.986747 \n",
       "L 82.497193 105.986747 \n",
       "L 81.376711 105.986747 \n",
       "L 80.256229 105.986747 \n",
       "L 79.135747 105.986747 \n",
       "L 78.015265 105.986747 \n",
       "L 76.894784 105.986747 \n",
       "L 75.774302 105.986747 \n",
       "L 74.65382 105.986747 \n",
       "L 73.533338 105.986747 \n",
       "L 72.412856 105.986747 \n",
       "L 71.292374 105.986747 \n",
       "L 70.171892 105.986747 \n",
       "L 69.05141 105.986747 \n",
       "L 67.930928 105.986747 \n",
       "L 66.810446 105.986747 \n",
       "L 65.689964 105.986747 \n",
       "L 64.569482 105.986747 \n",
       "L 63.449 105.986747 \n",
       "L 62.328518 105.986747 \n",
       "L 61.208037 105.986747 \n",
       "L 60.087555 105.986747 \n",
       "L 58.967073 105.986747 \n",
       "L 57.846591 105.986747 \n",
       "L 56.726109 105.986747 \n",
       "L 55.605627 105.986747 \n",
       "L 54.485145 105.986747 \n",
       "L 53.364663 105.986747 \n",
       "L 52.244181 105.986747 \n",
       "L 51.123699 105.986747 \n",
       "L 50.003217 105.986747 \n",
       "L 48.882735 105.986747 \n",
       "L 47.762253 105.986747 \n",
       "L 46.641771 105.986747 \n",
       "L 45.52129 105.986747 \n",
       "L 44.400808 105.986747 \n",
       "L 43.280326 105.986747 \n",
       "L 42.159844 105.986747 \n",
       "L 42.159844 106.259639 \n",
       "z\n",
       "\" style=\"fill:#e377c2;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_7\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 43.280326 105.986747 \n",
       "L 44.400808 105.986747 \n",
       "L 45.52129 105.986747 \n",
       "L 46.641771 105.986747 \n",
       "L 47.762253 105.986747 \n",
       "L 48.882735 105.986747 \n",
       "L 50.003217 105.986747 \n",
       "L 51.123699 105.986747 \n",
       "L 52.244181 105.986747 \n",
       "L 53.364663 105.986747 \n",
       "L 54.485145 105.986747 \n",
       "L 55.605627 105.986747 \n",
       "L 56.726109 105.986747 \n",
       "L 57.846591 105.986747 \n",
       "L 58.967073 105.986747 \n",
       "L 60.087555 105.986747 \n",
       "L 61.208037 105.986747 \n",
       "L 62.328518 105.986747 \n",
       "L 63.449 105.986747 \n",
       "L 64.569482 105.986747 \n",
       "L 65.689964 105.986747 \n",
       "L 66.810446 105.986747 \n",
       "L 67.930928 105.986747 \n",
       "L 69.05141 105.986747 \n",
       "L 70.171892 105.986747 \n",
       "L 71.292374 105.986747 \n",
       "L 72.412856 105.986747 \n",
       "L 73.533338 105.986747 \n",
       "L 74.65382 105.986747 \n",
       "L 75.774302 105.986747 \n",
       "L 76.894784 105.986747 \n",
       "L 78.015265 105.986747 \n",
       "L 79.135747 105.986747 \n",
       "L 80.256229 105.986747 \n",
       "L 81.376711 105.986747 \n",
       "L 82.497193 105.986747 \n",
       "L 83.617675 105.986747 \n",
       "L 84.738157 105.986747 \n",
       "L 85.858639 105.986747 \n",
       "L 86.979121 105.986747 \n",
       "L 88.099603 105.986747 \n",
       "L 89.220085 105.986747 \n",
       "L 90.340567 105.986747 \n",
       "L 91.461049 105.986747 \n",
       "L 92.58153 105.986747 \n",
       "L 93.702012 105.986747 \n",
       "L 94.822494 105.986747 \n",
       "L 95.942976 105.986747 \n",
       "L 97.063458 105.986747 \n",
       "L 98.18394 105.986747 \n",
       "L 99.304422 105.986747 \n",
       "L 100.424904 105.986747 \n",
       "L 101.545386 105.986747 \n",
       "L 102.665868 105.986747 \n",
       "L 103.78635 105.986747 \n",
       "L 104.906832 105.986747 \n",
       "L 106.027314 105.986747 \n",
       "L 107.147796 105.986747 \n",
       "L 108.268277 105.986747 \n",
       "L 109.388759 105.986747 \n",
       "L 110.509241 105.986747 \n",
       "L 111.629723 105.986747 \n",
       "L 112.750205 105.986747 \n",
       "L 113.870687 105.986747 \n",
       "L 114.991169 105.986747 \n",
       "L 116.111651 105.986747 \n",
       "L 117.232133 105.986747 \n",
       "L 118.352615 105.986747 \n",
       "L 119.473097 105.986747 \n",
       "L 120.593579 105.986747 \n",
       "L 121.714061 105.986747 \n",
       "L 122.834543 105.986747 \n",
       "L 123.955024 105.986747 \n",
       "L 125.075506 105.986747 \n",
       "L 126.195988 105.986747 \n",
       "L 127.31647 105.986747 \n",
       "L 128.436952 105.986747 \n",
       "L 129.557434 105.986747 \n",
       "L 130.677916 105.986747 \n",
       "L 131.798398 105.986747 \n",
       "L 132.91888 105.986747 \n",
       "L 134.039362 105.986747 \n",
       "L 135.159844 105.986747 \n",
       "L 136.280326 105.986747 \n",
       "L 137.400808 105.986747 \n",
       "L 138.52129 105.986747 \n",
       "L 139.641771 105.986747 \n",
       "L 140.762253 105.986747 \n",
       "L 141.882735 105.986747 \n",
       "L 143.003217 105.986747 \n",
       "L 144.123699 105.986747 \n",
       "L 145.244181 105.986747 \n",
       "L 146.364663 105.986747 \n",
       "L 147.485145 105.986747 \n",
       "L 148.605627 105.986747 \n",
       "L 149.726109 105.986747 \n",
       "L 150.846591 105.986747 \n",
       "L 151.967073 105.986747 \n",
       "L 153.087555 105.986747 \n",
       "L 154.208037 105.986747 \n",
       "L 155.328518 105.986747 \n",
       "L 156.449 105.986747 \n",
       "L 157.569482 105.986747 \n",
       "L 158.689964 105.986747 \n",
       "L 159.810446 105.986747 \n",
       "L 160.930928 105.986747 \n",
       "L 162.05141 105.986747 \n",
       "L 163.171892 105.986747 \n",
       "L 164.292374 105.986747 \n",
       "L 165.412856 105.986747 \n",
       "L 166.533338 105.986747 \n",
       "L 167.65382 105.986747 \n",
       "L 168.774302 105.986747 \n",
       "L 169.894784 105.986747 \n",
       "L 171.015265 105.986747 \n",
       "L 172.135747 105.986747 \n",
       "L 173.256229 105.986747 \n",
       "L 174.376711 105.986747 \n",
       "L 175.497193 105.986747 \n",
       "L 176.617675 105.986747 \n",
       "L 177.738157 105.986747 \n",
       "L 178.858639 105.986747 \n",
       "L 179.979121 105.986747 \n",
       "L 181.099603 105.986747 \n",
       "L 182.220085 105.986747 \n",
       "L 183.340567 105.986747 \n",
       "L 184.461049 105.986747 \n",
       "L 185.58153 105.986747 \n",
       "L 186.702012 105.986747 \n",
       "L 187.822494 105.986747 \n",
       "L 188.942976 105.986747 \n",
       "L 190.063458 105.986747 \n",
       "L 191.18394 105.986747 \n",
       "L 192.304422 105.986747 \n",
       "L 193.424904 105.986747 \n",
       "L 194.545386 105.986747 \n",
       "L 195.665868 105.986747 \n",
       "L 196.78635 105.986747 \n",
       "L 197.906832 105.986747 \n",
       "L 199.027314 105.986747 \n",
       "L 200.147796 105.986747 \n",
       "L 201.268277 105.986747 \n",
       "L 202.388759 105.986747 \n",
       "L 203.509241 105.986747 \n",
       "L 204.629723 105.986747 \n",
       "L 205.750205 105.986747 \n",
       "L 206.870687 105.986747 \n",
       "L 207.991169 105.986747 \n",
       "L 209.111651 105.986747 \n",
       "L 210.232133 105.986747 \n",
       "L 211.352615 105.986747 \n",
       "L 212.473097 105.986747 \n",
       "L 213.593579 105.986747 \n",
       "L 214.714061 105.986747 \n",
       "L 215.834543 105.986747 \n",
       "L 216.955024 105.986747 \n",
       "L 218.075506 105.986747 \n",
       "L 219.195988 105.986747 \n",
       "L 220.31647 105.986747 \n",
       "L 221.436952 105.986747 \n",
       "L 222.557434 105.986747 \n",
       "L 223.677916 105.986747 \n",
       "L 224.798398 105.986747 \n",
       "L 225.91888 105.986747 \n",
       "L 227.039362 105.986747 \n",
       "L 228.159844 105.986747 \n",
       "L 229.280326 105.986747 \n",
       "L 230.400808 105.986747 \n",
       "L 231.52129 105.986747 \n",
       "L 232.641771 105.986747 \n",
       "L 233.762253 105.986747 \n",
       "L 234.882735 105.986747 \n",
       "L 236.003217 105.986747 \n",
       "L 237.123699 105.986747 \n",
       "L 238.244181 105.986747 \n",
       "L 239.364663 105.986747 \n",
       "L 240.485145 105.986747 \n",
       "L 241.605627 105.986747 \n",
       "L 242.726109 105.986747 \n",
       "L 243.846591 105.986747 \n",
       "L 244.967073 105.986747 \n",
       "L 246.087555 105.986747 \n",
       "L 247.208037 105.986747 \n",
       "L 248.328518 105.986747 \n",
       "L 249.449 105.986747 \n",
       "L 250.569482 105.986747 \n",
       "L 251.689964 105.986747 \n",
       "L 252.810446 105.986747 \n",
       "L 253.930928 105.986747 \n",
       "L 255.05141 105.986747 \n",
       "L 256.171892 105.986747 \n",
       "L 257.292374 105.986747 \n",
       "L 258.412856 105.986747 \n",
       "L 259.533338 105.986747 \n",
       "L 260.65382 105.986747 \n",
       "L 261.774302 105.986747 \n",
       "L 262.894784 105.986747 \n",
       "L 264.015265 105.986747 \n",
       "L 265.135747 105.986747 \n",
       "L 266.256229 105.986747 \n",
       "L 267.376711 105.986747 \n",
       "L 268.497193 105.986747 \n",
       "L 269.617675 105.986747 \n",
       "L 270.738157 105.986747 \n",
       "L 271.858639 105.986747 \n",
       "L 272.979121 105.986747 \n",
       "L 274.099603 105.986747 \n",
       "L 275.220085 105.986747 \n",
       "L 276.340567 105.986747 \n",
       "L 277.461049 105.986747 \n",
       "L 278.58153 105.986747 \n",
       "L 279.702012 105.986747 \n",
       "L 280.822494 105.986747 \n",
       "L 281.942976 105.986747 \n",
       "L 283.063458 105.986747 \n",
       "L 284.18394 105.986747 \n",
       "L 285.304422 105.986747 \n",
       "L 286.424904 105.986747 \n",
       "L 287.545386 105.986747 \n",
       "L 288.665868 105.986747 \n",
       "L 289.78635 105.986747 \n",
       "L 290.906832 105.986747 \n",
       "L 292.027314 105.986747 \n",
       "L 293.147796 105.986747 \n",
       "L 294.268277 105.986747 \n",
       "L 295.388759 105.986747 \n",
       "L 296.509241 105.986747 \n",
       "L 297.629723 105.986747 \n",
       "L 298.750205 105.986747 \n",
       "L 299.870687 105.986747 \n",
       "L 300.991169 105.986747 \n",
       "L 302.111651 105.986747 \n",
       "L 303.232133 105.986747 \n",
       "L 304.352615 105.986747 \n",
       "L 305.473097 105.986747 \n",
       "L 306.593579 105.986747 \n",
       "L 307.714061 105.986747 \n",
       "L 308.834543 105.986747 \n",
       "L 309.955024 105.986747 \n",
       "L 311.075506 105.986747 \n",
       "L 312.195988 105.986747 \n",
       "L 313.31647 105.986747 \n",
       "L 314.436952 105.986747 \n",
       "L 315.557434 105.986747 \n",
       "L 316.677916 105.986747 \n",
       "L 317.798398 105.986747 \n",
       "L 318.91888 105.986747 \n",
       "L 320.039362 105.986747 \n",
       "L 321.159844 105.986747 \n",
       "L 321.159844 105.713855 \n",
       "L 320.039362 105.713855 \n",
       "L 318.91888 105.713855 \n",
       "L 317.798398 105.713855 \n",
       "L 316.677916 105.713855 \n",
       "L 315.557434 105.713855 \n",
       "L 314.436952 105.713855 \n",
       "L 313.31647 105.713855 \n",
       "L 312.195988 105.713855 \n",
       "L 311.075506 105.713855 \n",
       "L 309.955024 105.713855 \n",
       "L 308.834543 105.713855 \n",
       "L 307.714061 105.713855 \n",
       "L 306.593579 105.713855 \n",
       "L 305.473097 105.713855 \n",
       "L 304.352615 105.713855 \n",
       "L 303.232133 105.713855 \n",
       "L 302.111651 105.713855 \n",
       "L 300.991169 105.713855 \n",
       "L 299.870687 105.713855 \n",
       "L 298.750205 105.713855 \n",
       "L 297.629723 105.713855 \n",
       "L 296.509241 105.713855 \n",
       "L 295.388759 105.713855 \n",
       "L 294.268277 105.713855 \n",
       "L 293.147796 105.713855 \n",
       "L 292.027314 105.713855 \n",
       "L 290.906832 105.713855 \n",
       "L 289.78635 105.713855 \n",
       "L 288.665868 105.713855 \n",
       "L 287.545386 105.713855 \n",
       "L 286.424904 105.713855 \n",
       "L 285.304422 105.713855 \n",
       "L 284.18394 105.713855 \n",
       "L 283.063458 105.713855 \n",
       "L 281.942976 105.713855 \n",
       "L 280.822494 105.713855 \n",
       "L 279.702012 105.713855 \n",
       "L 278.58153 105.713855 \n",
       "L 277.461049 105.713855 \n",
       "L 276.340567 105.713855 \n",
       "L 275.220085 105.713855 \n",
       "L 274.099603 105.713855 \n",
       "L 272.979121 105.713855 \n",
       "L 271.858639 105.713855 \n",
       "L 270.738157 105.713855 \n",
       "L 269.617675 105.713855 \n",
       "L 268.497193 105.713855 \n",
       "L 267.376711 105.713855 \n",
       "L 266.256229 105.713855 \n",
       "L 265.135747 105.713855 \n",
       "L 264.015265 105.713855 \n",
       "L 262.894784 105.713855 \n",
       "L 261.774302 105.713855 \n",
       "L 260.65382 105.713855 \n",
       "L 259.533338 105.713855 \n",
       "L 258.412856 105.713855 \n",
       "L 257.292374 105.713855 \n",
       "L 256.171892 105.713855 \n",
       "L 255.05141 105.713855 \n",
       "L 253.930928 105.713855 \n",
       "L 252.810446 105.713855 \n",
       "L 251.689964 105.713855 \n",
       "L 250.569482 105.713855 \n",
       "L 249.449 105.713855 \n",
       "L 248.328518 105.713855 \n",
       "L 247.208037 105.713855 \n",
       "L 246.087555 105.713855 \n",
       "L 244.967073 105.713855 \n",
       "L 243.846591 105.713855 \n",
       "L 242.726109 105.713855 \n",
       "L 241.605627 105.713855 \n",
       "L 240.485145 105.713855 \n",
       "L 239.364663 105.713855 \n",
       "L 238.244181 105.713855 \n",
       "L 237.123699 105.713855 \n",
       "L 236.003217 105.713855 \n",
       "L 234.882735 105.713855 \n",
       "L 233.762253 105.713855 \n",
       "L 232.641771 105.713855 \n",
       "L 231.52129 105.713855 \n",
       "L 230.400808 105.713855 \n",
       "L 229.280326 105.713855 \n",
       "L 228.159844 105.713855 \n",
       "L 227.039362 105.713855 \n",
       "L 225.91888 105.713855 \n",
       "L 224.798398 105.713855 \n",
       "L 223.677916 105.713855 \n",
       "L 222.557434 105.713855 \n",
       "L 221.436952 105.713855 \n",
       "L 220.31647 105.713855 \n",
       "L 219.195988 105.713855 \n",
       "L 218.075506 105.713855 \n",
       "L 216.955024 105.713855 \n",
       "L 215.834543 105.713855 \n",
       "L 214.714061 105.713855 \n",
       "L 213.593579 105.713855 \n",
       "L 212.473097 105.713855 \n",
       "L 211.352615 105.713855 \n",
       "L 210.232133 105.713855 \n",
       "L 209.111651 105.713855 \n",
       "L 207.991169 105.713855 \n",
       "L 206.870687 105.713855 \n",
       "L 205.750205 105.713855 \n",
       "L 204.629723 105.713855 \n",
       "L 203.509241 105.713855 \n",
       "L 202.388759 105.713855 \n",
       "L 201.268277 105.713855 \n",
       "L 200.147796 105.713855 \n",
       "L 199.027314 105.713855 \n",
       "L 197.906832 105.713855 \n",
       "L 196.78635 105.713855 \n",
       "L 195.665868 105.713855 \n",
       "L 194.545386 105.713855 \n",
       "L 193.424904 105.713855 \n",
       "L 192.304422 105.713855 \n",
       "L 191.18394 105.713855 \n",
       "L 190.063458 105.713855 \n",
       "L 188.942976 105.713855 \n",
       "L 187.822494 105.713855 \n",
       "L 186.702012 105.713855 \n",
       "L 185.58153 105.713855 \n",
       "L 184.461049 105.713855 \n",
       "L 183.340567 105.713855 \n",
       "L 182.220085 105.713855 \n",
       "L 181.099603 105.713855 \n",
       "L 179.979121 105.713855 \n",
       "L 178.858639 105.713855 \n",
       "L 177.738157 105.713855 \n",
       "L 176.617675 105.713855 \n",
       "L 175.497193 105.713855 \n",
       "L 174.376711 105.713855 \n",
       "L 173.256229 105.713855 \n",
       "L 172.135747 105.713855 \n",
       "L 171.015265 105.713855 \n",
       "L 169.894784 105.713855 \n",
       "L 168.774302 105.713855 \n",
       "L 167.65382 105.713855 \n",
       "L 166.533338 105.713855 \n",
       "L 165.412856 105.713855 \n",
       "L 164.292374 105.713855 \n",
       "L 163.171892 105.713855 \n",
       "L 162.05141 105.713855 \n",
       "L 160.930928 105.713855 \n",
       "L 159.810446 105.713855 \n",
       "L 158.689964 105.713855 \n",
       "L 157.569482 105.713855 \n",
       "L 156.449 105.713855 \n",
       "L 155.328518 105.713855 \n",
       "L 154.208037 105.713855 \n",
       "L 153.087555 105.713855 \n",
       "L 151.967073 105.713855 \n",
       "L 150.846591 105.713855 \n",
       "L 149.726109 105.713855 \n",
       "L 148.605627 105.713855 \n",
       "L 147.485145 105.713855 \n",
       "L 146.364663 105.713855 \n",
       "L 145.244181 105.713855 \n",
       "L 144.123699 105.713855 \n",
       "L 143.003217 105.713855 \n",
       "L 141.882735 105.713855 \n",
       "L 140.762253 105.713855 \n",
       "L 139.641771 105.713855 \n",
       "L 138.52129 105.713855 \n",
       "L 137.400808 105.713855 \n",
       "L 136.280326 105.713855 \n",
       "L 135.159844 105.713855 \n",
       "L 134.039362 105.713855 \n",
       "L 132.91888 105.713855 \n",
       "L 131.798398 105.713855 \n",
       "L 130.677916 105.713855 \n",
       "L 129.557434 105.713855 \n",
       "L 128.436952 105.713855 \n",
       "L 127.31647 105.713855 \n",
       "L 126.195988 105.713855 \n",
       "L 125.075506 105.713855 \n",
       "L 123.955024 105.713855 \n",
       "L 122.834543 105.713855 \n",
       "L 121.714061 105.713855 \n",
       "L 120.593579 105.713855 \n",
       "L 119.473097 105.713855 \n",
       "L 118.352615 105.713855 \n",
       "L 117.232133 105.713855 \n",
       "L 116.111651 105.713855 \n",
       "L 114.991169 105.713855 \n",
       "L 113.870687 105.713855 \n",
       "L 112.750205 105.713855 \n",
       "L 111.629723 105.713855 \n",
       "L 110.509241 105.713855 \n",
       "L 109.388759 105.713855 \n",
       "L 108.268277 105.713855 \n",
       "L 107.147796 105.713855 \n",
       "L 106.027314 105.713855 \n",
       "L 104.906832 105.713855 \n",
       "L 103.78635 105.713855 \n",
       "L 102.665868 105.713855 \n",
       "L 101.545386 105.713855 \n",
       "L 100.424904 105.713855 \n",
       "L 99.304422 105.713855 \n",
       "L 98.18394 105.713855 \n",
       "L 97.063458 105.713855 \n",
       "L 95.942976 105.713855 \n",
       "L 94.822494 105.713855 \n",
       "L 93.702012 105.713855 \n",
       "L 92.58153 105.713855 \n",
       "L 91.461049 105.713855 \n",
       "L 90.340567 105.713855 \n",
       "L 89.220085 105.713855 \n",
       "L 88.099603 105.713855 \n",
       "L 86.979121 105.713855 \n",
       "L 85.858639 105.713855 \n",
       "L 84.738157 105.713855 \n",
       "L 83.617675 105.713855 \n",
       "L 82.497193 105.713855 \n",
       "L 81.376711 105.713855 \n",
       "L 80.256229 105.713855 \n",
       "L 79.135747 105.713855 \n",
       "L 78.015265 105.713855 \n",
       "L 76.894784 105.713855 \n",
       "L 75.774302 105.713855 \n",
       "L 74.65382 105.713855 \n",
       "L 73.533338 105.713855 \n",
       "L 72.412856 105.713855 \n",
       "L 71.292374 105.713855 \n",
       "L 70.171892 105.713855 \n",
       "L 69.05141 105.713855 \n",
       "L 67.930928 105.713855 \n",
       "L 66.810446 105.713855 \n",
       "L 65.689964 105.713855 \n",
       "L 64.569482 105.713855 \n",
       "L 63.449 105.713855 \n",
       "L 62.328518 105.713855 \n",
       "L 61.208037 105.713855 \n",
       "L 60.087555 105.713855 \n",
       "L 58.967073 105.713855 \n",
       "L 57.846591 105.713855 \n",
       "L 56.726109 105.713855 \n",
       "L 55.605627 105.713855 \n",
       "L 54.485145 105.713855 \n",
       "L 53.364663 105.713855 \n",
       "L 52.244181 105.713855 \n",
       "L 51.123699 105.713855 \n",
       "L 50.003217 105.713855 \n",
       "L 48.882735 105.713855 \n",
       "L 47.762253 105.713855 \n",
       "L 46.641771 105.713855 \n",
       "L 45.52129 105.713855 \n",
       "L 44.400808 105.713855 \n",
       "L 43.280326 105.713855 \n",
       "L 42.159844 105.713855 \n",
       "L 42.159844 105.986747 \n",
       "z\n",
       "\" style=\"fill:#bcbd22;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_8\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 43.280326 105.713855 \n",
       "L 44.400808 105.713855 \n",
       "L 45.52129 105.713855 \n",
       "L 46.641771 105.713855 \n",
       "L 47.762253 105.713855 \n",
       "L 48.882735 105.713855 \n",
       "L 50.003217 105.713855 \n",
       "L 51.123699 105.713855 \n",
       "L 52.244181 105.713855 \n",
       "L 53.364663 105.713855 \n",
       "L 54.485145 105.713855 \n",
       "L 55.605627 105.713855 \n",
       "L 56.726109 105.713855 \n",
       "L 57.846591 105.713855 \n",
       "L 58.967073 105.713855 \n",
       "L 60.087555 105.713855 \n",
       "L 61.208037 105.713855 \n",
       "L 62.328518 105.713855 \n",
       "L 63.449 105.713855 \n",
       "L 64.569482 105.713855 \n",
       "L 65.689964 105.713855 \n",
       "L 66.810446 105.713855 \n",
       "L 67.930928 105.713855 \n",
       "L 69.05141 105.713855 \n",
       "L 70.171892 105.713855 \n",
       "L 71.292374 105.713855 \n",
       "L 72.412856 105.713855 \n",
       "L 73.533338 105.713855 \n",
       "L 74.65382 105.713855 \n",
       "L 75.774302 105.713855 \n",
       "L 76.894784 105.713855 \n",
       "L 78.015265 105.713855 \n",
       "L 79.135747 105.713855 \n",
       "L 80.256229 105.713855 \n",
       "L 81.376711 105.713855 \n",
       "L 82.497193 105.713855 \n",
       "L 83.617675 105.713855 \n",
       "L 84.738157 105.713855 \n",
       "L 85.858639 105.713855 \n",
       "L 86.979121 105.713855 \n",
       "L 88.099603 105.713855 \n",
       "L 89.220085 105.713855 \n",
       "L 90.340567 105.713855 \n",
       "L 91.461049 105.713855 \n",
       "L 92.58153 105.713855 \n",
       "L 93.702012 105.713855 \n",
       "L 94.822494 105.713855 \n",
       "L 95.942976 105.713855 \n",
       "L 97.063458 105.713855 \n",
       "L 98.18394 105.713855 \n",
       "L 99.304422 105.713855 \n",
       "L 100.424904 105.713855 \n",
       "L 101.545386 105.713855 \n",
       "L 102.665868 105.713855 \n",
       "L 103.78635 105.713855 \n",
       "L 104.906832 105.713855 \n",
       "L 106.027314 105.713855 \n",
       "L 107.147796 105.713855 \n",
       "L 108.268277 105.713855 \n",
       "L 109.388759 105.713855 \n",
       "L 110.509241 105.713855 \n",
       "L 111.629723 105.713855 \n",
       "L 112.750205 105.713855 \n",
       "L 113.870687 105.713855 \n",
       "L 114.991169 105.713855 \n",
       "L 116.111651 105.713855 \n",
       "L 117.232133 105.713855 \n",
       "L 118.352615 105.713855 \n",
       "L 119.473097 105.713855 \n",
       "L 120.593579 105.713855 \n",
       "L 121.714061 105.713855 \n",
       "L 122.834543 105.713855 \n",
       "L 123.955024 105.713855 \n",
       "L 125.075506 105.713855 \n",
       "L 126.195988 105.713855 \n",
       "L 127.31647 105.713855 \n",
       "L 128.436952 105.713855 \n",
       "L 129.557434 105.713855 \n",
       "L 130.677916 105.713855 \n",
       "L 131.798398 105.713855 \n",
       "L 132.91888 105.713855 \n",
       "L 134.039362 105.713855 \n",
       "L 135.159844 105.713855 \n",
       "L 136.280326 105.713855 \n",
       "L 137.400808 105.713855 \n",
       "L 138.52129 105.713855 \n",
       "L 139.641771 105.713855 \n",
       "L 140.762253 105.713855 \n",
       "L 141.882735 105.713855 \n",
       "L 143.003217 105.713855 \n",
       "L 144.123699 105.713855 \n",
       "L 145.244181 105.713855 \n",
       "L 146.364663 105.713855 \n",
       "L 147.485145 105.713855 \n",
       "L 148.605627 105.713855 \n",
       "L 149.726109 105.713855 \n",
       "L 150.846591 105.713855 \n",
       "L 151.967073 105.713855 \n",
       "L 153.087555 105.713855 \n",
       "L 154.208037 105.713855 \n",
       "L 155.328518 105.713855 \n",
       "L 156.449 105.713855 \n",
       "L 157.569482 105.713855 \n",
       "L 158.689964 105.713855 \n",
       "L 159.810446 105.713855 \n",
       "L 160.930928 105.713855 \n",
       "L 162.05141 105.713855 \n",
       "L 163.171892 105.713855 \n",
       "L 164.292374 105.713855 \n",
       "L 165.412856 105.713855 \n",
       "L 166.533338 105.713855 \n",
       "L 167.65382 105.713855 \n",
       "L 168.774302 105.713855 \n",
       "L 169.894784 105.713855 \n",
       "L 171.015265 105.713855 \n",
       "L 172.135747 105.713855 \n",
       "L 173.256229 105.713855 \n",
       "L 174.376711 105.713855 \n",
       "L 175.497193 105.713855 \n",
       "L 176.617675 105.713855 \n",
       "L 177.738157 105.713855 \n",
       "L 178.858639 105.713855 \n",
       "L 179.979121 105.713855 \n",
       "L 181.099603 105.713855 \n",
       "L 182.220085 105.713855 \n",
       "L 183.340567 105.713855 \n",
       "L 184.461049 105.713855 \n",
       "L 185.58153 105.713855 \n",
       "L 186.702012 105.713855 \n",
       "L 187.822494 105.713855 \n",
       "L 188.942976 105.713855 \n",
       "L 190.063458 105.713855 \n",
       "L 191.18394 105.713855 \n",
       "L 192.304422 105.713855 \n",
       "L 193.424904 105.713855 \n",
       "L 194.545386 105.713855 \n",
       "L 195.665868 105.713855 \n",
       "L 196.78635 105.713855 \n",
       "L 197.906832 105.713855 \n",
       "L 199.027314 105.713855 \n",
       "L 200.147796 105.713855 \n",
       "L 201.268277 105.713855 \n",
       "L 202.388759 105.713855 \n",
       "L 203.509241 105.713855 \n",
       "L 204.629723 105.713855 \n",
       "L 205.750205 105.713855 \n",
       "L 206.870687 105.713855 \n",
       "L 207.991169 105.713855 \n",
       "L 209.111651 105.713855 \n",
       "L 210.232133 105.713855 \n",
       "L 211.352615 105.713855 \n",
       "L 212.473097 105.713855 \n",
       "L 213.593579 105.713855 \n",
       "L 214.714061 105.713855 \n",
       "L 215.834543 105.713855 \n",
       "L 216.955024 105.713855 \n",
       "L 218.075506 105.713855 \n",
       "L 219.195988 105.713855 \n",
       "L 220.31647 105.713855 \n",
       "L 221.436952 105.713855 \n",
       "L 222.557434 105.713855 \n",
       "L 223.677916 105.713855 \n",
       "L 224.798398 105.713855 \n",
       "L 225.91888 105.713855 \n",
       "L 227.039362 105.713855 \n",
       "L 228.159844 105.713855 \n",
       "L 229.280326 105.713855 \n",
       "L 230.400808 105.713855 \n",
       "L 231.52129 105.713855 \n",
       "L 232.641771 105.713855 \n",
       "L 233.762253 105.713855 \n",
       "L 234.882735 105.713855 \n",
       "L 236.003217 105.713855 \n",
       "L 237.123699 105.713855 \n",
       "L 238.244181 105.713855 \n",
       "L 239.364663 105.713855 \n",
       "L 240.485145 105.713855 \n",
       "L 241.605627 105.713855 \n",
       "L 242.726109 105.713855 \n",
       "L 243.846591 105.713855 \n",
       "L 244.967073 105.713855 \n",
       "L 246.087555 105.713855 \n",
       "L 247.208037 105.713855 \n",
       "L 248.328518 105.713855 \n",
       "L 249.449 105.713855 \n",
       "L 250.569482 105.713855 \n",
       "L 251.689964 105.713855 \n",
       "L 252.810446 105.713855 \n",
       "L 253.930928 105.713855 \n",
       "L 255.05141 105.713855 \n",
       "L 256.171892 105.713855 \n",
       "L 257.292374 105.713855 \n",
       "L 258.412856 105.713855 \n",
       "L 259.533338 105.713855 \n",
       "L 260.65382 105.713855 \n",
       "L 261.774302 105.713855 \n",
       "L 262.894784 105.713855 \n",
       "L 264.015265 105.713855 \n",
       "L 265.135747 105.713855 \n",
       "L 266.256229 105.713855 \n",
       "L 267.376711 105.713855 \n",
       "L 268.497193 105.713855 \n",
       "L 269.617675 105.713855 \n",
       "L 270.738157 105.713855 \n",
       "L 271.858639 105.713855 \n",
       "L 272.979121 105.713855 \n",
       "L 274.099603 105.713855 \n",
       "L 275.220085 105.713855 \n",
       "L 276.340567 105.713855 \n",
       "L 277.461049 105.713855 \n",
       "L 278.58153 105.713855 \n",
       "L 279.702012 105.713855 \n",
       "L 280.822494 105.713855 \n",
       "L 281.942976 105.713855 \n",
       "L 283.063458 105.713855 \n",
       "L 284.18394 105.713855 \n",
       "L 285.304422 105.713855 \n",
       "L 286.424904 105.713855 \n",
       "L 287.545386 105.713855 \n",
       "L 288.665868 105.713855 \n",
       "L 289.78635 105.713855 \n",
       "L 290.906832 105.713855 \n",
       "L 292.027314 105.713855 \n",
       "L 293.147796 105.713855 \n",
       "L 294.268277 105.713855 \n",
       "L 295.388759 105.713855 \n",
       "L 296.509241 105.713855 \n",
       "L 297.629723 105.713855 \n",
       "L 298.750205 105.713855 \n",
       "L 299.870687 105.713855 \n",
       "L 300.991169 105.713855 \n",
       "L 302.111651 105.713855 \n",
       "L 303.232133 105.713855 \n",
       "L 304.352615 105.713855 \n",
       "L 305.473097 105.713855 \n",
       "L 306.593579 105.713855 \n",
       "L 307.714061 105.713855 \n",
       "L 308.834543 105.713855 \n",
       "L 309.955024 105.713855 \n",
       "L 311.075506 105.713855 \n",
       "L 312.195988 105.713855 \n",
       "L 313.31647 105.713855 \n",
       "L 314.436952 105.713855 \n",
       "L 315.557434 105.713855 \n",
       "L 316.677916 105.713855 \n",
       "L 317.798398 105.713855 \n",
       "L 318.91888 105.713855 \n",
       "L 320.039362 105.713855 \n",
       "L 321.159844 105.713855 \n",
       "L 321.159844 105.440964 \n",
       "L 321.159844 104.349398 \n",
       "L 321.159844 103.257831 \n",
       "L 321.159844 102.166265 \n",
       "L 321.159844 101.074699 \n",
       "L 321.159844 99.983133 \n",
       "L 321.159844 98.891566 \n",
       "L 321.159844 97.8 \n",
       "L 321.159844 96.708434 \n",
       "L 321.159844 95.616867 \n",
       "L 321.159844 94.525301 \n",
       "L 321.159844 93.433735 \n",
       "L 321.159844 92.342169 \n",
       "L 321.159844 91.250602 \n",
       "L 321.159844 90.159036 \n",
       "L 321.159844 89.06747 \n",
       "L 321.159844 87.975904 \n",
       "L 321.159844 86.884337 \n",
       "L 321.159844 85.792771 \n",
       "L 321.159844 84.701205 \n",
       "L 321.159844 83.609639 \n",
       "L 321.159844 82.518072 \n",
       "L 321.159844 81.426506 \n",
       "L 321.159844 80.33494 \n",
       "L 321.159844 79.243373 \n",
       "L 321.159844 78.151807 \n",
       "L 321.159844 77.060241 \n",
       "L 321.159844 75.968675 \n",
       "L 321.159844 74.877108 \n",
       "L 321.159844 73.785542 \n",
       "L 321.159844 72.693976 \n",
       "L 321.159844 71.60241 \n",
       "L 321.159844 70.510843 \n",
       "L 321.159844 69.419277 \n",
       "L 321.159844 68.327711 \n",
       "L 321.159844 67.236145 \n",
       "L 321.159844 66.144578 \n",
       "L 321.159844 65.053012 \n",
       "L 321.159844 63.961446 \n",
       "L 321.159844 62.86988 \n",
       "L 321.159844 61.778313 \n",
       "L 321.159844 60.686747 \n",
       "L 321.159844 59.595181 \n",
       "L 321.159844 58.503614 \n",
       "L 321.159844 57.412048 \n",
       "L 321.159844 56.320482 \n",
       "L 321.159844 55.228916 \n",
       "L 321.159844 54.137349 \n",
       "L 321.159844 53.045783 \n",
       "L 321.159844 51.954217 \n",
       "L 321.159844 50.862651 \n",
       "L 321.159844 49.771084 \n",
       "L 321.159844 48.679518 \n",
       "L 321.159844 47.587952 \n",
       "L 321.159844 46.496386 \n",
       "L 321.159844 45.404819 \n",
       "L 321.159844 44.313253 \n",
       "L 321.159844 43.221687 \n",
       "L 321.159844 42.13012 \n",
       "L 321.159844 41.038554 \n",
       "L 321.159844 39.946988 \n",
       "L 321.159844 38.855422 \n",
       "L 321.159844 37.763855 \n",
       "L 321.159844 36.672289 \n",
       "L 321.159844 35.580723 \n",
       "L 321.159844 34.489157 \n",
       "L 321.159844 33.39759 \n",
       "L 321.159844 32.306024 \n",
       "L 321.159844 31.214458 \n",
       "L 321.159844 30.122892 \n",
       "L 321.159844 29.031325 \n",
       "L 321.159844 27.939759 \n",
       "L 321.159844 26.848193 \n",
       "L 321.159844 25.756627 \n",
       "L 321.159844 24.66506 \n",
       "L 321.159844 23.573494 \n",
       "L 321.159844 22.481928 \n",
       "L 321.159844 21.390361 \n",
       "L 321.159844 20.298795 \n",
       "L 321.159844 19.207229 \n",
       "L 321.159844 18.115663 \n",
       "L 321.159844 17.024096 \n",
       "L 321.159844 15.93253 \n",
       "L 321.159844 14.840964 \n",
       "L 321.159844 13.749398 \n",
       "L 321.159844 12.657831 \n",
       "L 321.159844 11.566265 \n",
       "L 321.159844 10.474699 \n",
       "L 321.159844 9.383133 \n",
       "L 321.159844 8.291566 \n",
       "L 321.159844 7.2 \n",
       "L 320.039362 7.2 \n",
       "L 318.91888 7.2 \n",
       "L 317.798398 7.2 \n",
       "L 316.677916 7.2 \n",
       "L 315.557434 7.2 \n",
       "L 314.436952 7.2 \n",
       "L 313.31647 7.2 \n",
       "L 312.195988 7.2 \n",
       "L 311.075506 7.2 \n",
       "L 309.955024 7.2 \n",
       "L 308.834543 7.2 \n",
       "L 307.714061 7.2 \n",
       "L 306.593579 7.2 \n",
       "L 305.473097 7.2 \n",
       "L 304.352615 7.2 \n",
       "L 303.232133 7.2 \n",
       "L 302.111651 7.2 \n",
       "L 300.991169 7.2 \n",
       "L 299.870687 7.2 \n",
       "L 298.750205 7.2 \n",
       "L 297.629723 7.2 \n",
       "L 296.509241 7.2 \n",
       "L 295.388759 7.2 \n",
       "L 294.268277 7.2 \n",
       "L 293.147796 7.2 \n",
       "L 292.027314 7.2 \n",
       "L 290.906832 7.2 \n",
       "L 289.78635 7.2 \n",
       "L 288.665868 7.2 \n",
       "L 287.545386 7.2 \n",
       "L 286.424904 7.2 \n",
       "L 285.304422 7.2 \n",
       "L 284.18394 7.2 \n",
       "L 283.063458 7.2 \n",
       "L 281.942976 7.2 \n",
       "L 280.822494 7.2 \n",
       "L 279.702012 7.2 \n",
       "L 278.58153 7.2 \n",
       "L 277.461049 7.2 \n",
       "L 276.340567 7.2 \n",
       "L 275.220085 7.2 \n",
       "L 274.099603 7.2 \n",
       "L 272.979121 7.2 \n",
       "L 271.858639 7.2 \n",
       "L 270.738157 7.2 \n",
       "L 269.617675 7.2 \n",
       "L 268.497193 7.2 \n",
       "L 267.376711 7.2 \n",
       "L 266.256229 7.2 \n",
       "L 265.135747 7.2 \n",
       "L 264.015265 7.2 \n",
       "L 262.894784 7.2 \n",
       "L 261.774302 7.2 \n",
       "L 260.65382 7.2 \n",
       "L 259.533338 7.2 \n",
       "L 258.412856 7.2 \n",
       "L 257.292374 7.2 \n",
       "L 256.171892 7.2 \n",
       "L 255.05141 7.2 \n",
       "L 253.930928 7.2 \n",
       "L 252.810446 7.2 \n",
       "L 251.689964 7.2 \n",
       "L 250.569482 7.2 \n",
       "L 249.449 7.2 \n",
       "L 248.328518 7.2 \n",
       "L 247.208037 7.2 \n",
       "L 246.087555 7.2 \n",
       "L 244.967073 7.2 \n",
       "L 243.846591 7.2 \n",
       "L 242.726109 7.2 \n",
       "L 241.605627 7.2 \n",
       "L 240.485145 7.2 \n",
       "L 239.364663 7.2 \n",
       "L 238.244181 7.2 \n",
       "L 237.123699 7.2 \n",
       "L 236.003217 7.2 \n",
       "L 234.882735 7.2 \n",
       "L 233.762253 7.2 \n",
       "L 232.641771 7.2 \n",
       "L 231.52129 7.2 \n",
       "L 230.400808 7.2 \n",
       "L 229.280326 7.2 \n",
       "L 228.159844 7.2 \n",
       "L 227.039362 7.2 \n",
       "L 225.91888 7.2 \n",
       "L 224.798398 7.2 \n",
       "L 223.677916 7.2 \n",
       "L 222.557434 7.2 \n",
       "L 221.436952 7.2 \n",
       "L 220.31647 7.2 \n",
       "L 219.195988 7.2 \n",
       "L 218.075506 7.2 \n",
       "L 216.955024 7.2 \n",
       "L 215.834543 7.2 \n",
       "L 214.714061 7.2 \n",
       "L 213.593579 7.2 \n",
       "L 212.473097 7.2 \n",
       "L 211.352615 7.2 \n",
       "L 210.232133 7.2 \n",
       "L 209.111651 7.2 \n",
       "L 207.991169 7.2 \n",
       "L 206.870687 7.2 \n",
       "L 205.750205 7.2 \n",
       "L 204.629723 7.2 \n",
       "L 203.509241 7.2 \n",
       "L 202.388759 7.2 \n",
       "L 201.268277 7.2 \n",
       "L 200.147796 7.2 \n",
       "L 199.027314 7.2 \n",
       "L 197.906832 7.2 \n",
       "L 196.78635 7.2 \n",
       "L 195.665868 7.2 \n",
       "L 194.545386 7.2 \n",
       "L 193.424904 7.2 \n",
       "L 192.304422 7.2 \n",
       "L 191.18394 7.2 \n",
       "L 190.063458 7.2 \n",
       "L 188.942976 7.2 \n",
       "L 187.822494 7.2 \n",
       "L 186.702012 7.2 \n",
       "L 185.58153 7.2 \n",
       "L 184.461049 7.2 \n",
       "L 183.340567 7.2 \n",
       "L 182.220085 7.2 \n",
       "L 181.099603 7.2 \n",
       "L 179.979121 7.2 \n",
       "L 178.858639 7.2 \n",
       "L 177.738157 7.2 \n",
       "L 176.617675 7.2 \n",
       "L 175.497193 7.2 \n",
       "L 174.376711 7.2 \n",
       "L 173.256229 7.2 \n",
       "L 172.135747 7.2 \n",
       "L 171.015265 7.2 \n",
       "L 169.894784 7.2 \n",
       "L 168.774302 7.2 \n",
       "L 167.65382 7.2 \n",
       "L 166.533338 7.2 \n",
       "L 165.412856 7.2 \n",
       "L 164.292374 7.2 \n",
       "L 163.171892 7.2 \n",
       "L 162.05141 7.2 \n",
       "L 160.930928 7.2 \n",
       "L 159.810446 7.2 \n",
       "L 158.689964 7.2 \n",
       "L 157.569482 7.2 \n",
       "L 156.449 7.2 \n",
       "L 155.328518 7.2 \n",
       "L 154.208037 7.2 \n",
       "L 153.087555 7.2 \n",
       "L 151.967073 7.2 \n",
       "L 150.846591 7.2 \n",
       "L 149.726109 7.2 \n",
       "L 148.605627 7.2 \n",
       "L 147.485145 7.2 \n",
       "L 146.364663 7.2 \n",
       "L 145.244181 7.2 \n",
       "L 144.123699 7.2 \n",
       "L 143.003217 7.2 \n",
       "L 141.882735 7.2 \n",
       "L 140.762253 7.2 \n",
       "L 139.641771 7.2 \n",
       "L 138.52129 7.2 \n",
       "L 137.400808 7.2 \n",
       "L 136.280326 7.2 \n",
       "L 135.159844 7.2 \n",
       "L 134.039362 7.2 \n",
       "L 132.91888 7.2 \n",
       "L 131.798398 7.2 \n",
       "L 130.677916 7.2 \n",
       "L 129.557434 7.2 \n",
       "L 128.436952 7.2 \n",
       "L 127.31647 7.2 \n",
       "L 126.195988 7.2 \n",
       "L 125.075506 7.2 \n",
       "L 123.955024 7.2 \n",
       "L 122.834543 7.2 \n",
       "L 121.714061 7.2 \n",
       "L 120.593579 7.2 \n",
       "L 119.473097 7.2 \n",
       "L 118.352615 7.2 \n",
       "L 117.232133 7.2 \n",
       "L 116.111651 7.2 \n",
       "L 114.991169 7.2 \n",
       "L 113.870687 7.2 \n",
       "L 112.750205 7.2 \n",
       "L 111.629723 7.2 \n",
       "L 110.509241 7.2 \n",
       "L 109.388759 7.2 \n",
       "L 108.268277 7.2 \n",
       "L 107.147796 7.2 \n",
       "L 106.027314 7.2 \n",
       "L 104.906832 7.2 \n",
       "L 103.78635 7.2 \n",
       "L 102.665868 7.2 \n",
       "L 101.545386 7.2 \n",
       "L 100.424904 7.2 \n",
       "L 99.304422 7.2 \n",
       "L 98.18394 7.2 \n",
       "L 97.063458 7.2 \n",
       "L 95.942976 7.2 \n",
       "L 94.822494 7.2 \n",
       "L 93.702012 7.2 \n",
       "L 92.58153 7.2 \n",
       "L 91.461049 7.2 \n",
       "L 90.340567 7.2 \n",
       "L 89.220085 7.2 \n",
       "L 88.099603 7.2 \n",
       "L 86.979121 7.2 \n",
       "L 85.858639 7.2 \n",
       "L 84.738157 7.2 \n",
       "L 83.617675 7.2 \n",
       "L 82.497193 7.2 \n",
       "L 81.376711 7.2 \n",
       "L 80.256229 7.2 \n",
       "L 79.135747 7.2 \n",
       "L 78.015265 7.2 \n",
       "L 76.894784 7.2 \n",
       "L 75.774302 7.2 \n",
       "L 74.65382 7.2 \n",
       "L 73.533338 7.2 \n",
       "L 72.412856 7.2 \n",
       "L 71.292374 7.2 \n",
       "L 70.171892 7.2 \n",
       "L 69.05141 7.2 \n",
       "L 67.930928 7.2 \n",
       "L 66.810446 7.2 \n",
       "L 65.689964 7.2 \n",
       "L 64.569482 7.2 \n",
       "L 63.449 7.2 \n",
       "L 62.328518 7.2 \n",
       "L 61.208037 7.2 \n",
       "L 60.087555 7.2 \n",
       "L 58.967073 7.2 \n",
       "L 57.846591 7.2 \n",
       "L 56.726109 7.2 \n",
       "L 55.605627 7.2 \n",
       "L 54.485145 7.2 \n",
       "L 53.364663 7.2 \n",
       "L 52.244181 7.2 \n",
       "L 51.123699 7.2 \n",
       "L 50.003217 7.2 \n",
       "L 48.882735 7.2 \n",
       "L 47.762253 7.2 \n",
       "L 46.641771 7.2 \n",
       "L 45.52129 7.2 \n",
       "L 44.400808 7.2 \n",
       "L 43.280326 7.2 \n",
       "L 42.159844 7.2 \n",
       "L 42.159844 8.291566 \n",
       "L 42.159844 9.383133 \n",
       "L 42.159844 10.474699 \n",
       "L 42.159844 11.566265 \n",
       "L 42.159844 12.657831 \n",
       "L 42.159844 13.749398 \n",
       "L 42.159844 14.840964 \n",
       "L 42.159844 15.93253 \n",
       "L 42.159844 17.024096 \n",
       "L 42.159844 18.115663 \n",
       "L 42.159844 19.207229 \n",
       "L 42.159844 20.298795 \n",
       "L 42.159844 21.390361 \n",
       "L 42.159844 22.481928 \n",
       "L 42.159844 23.573494 \n",
       "L 42.159844 24.66506 \n",
       "L 42.159844 25.756627 \n",
       "L 42.159844 26.848193 \n",
       "L 42.159844 27.939759 \n",
       "L 42.159844 29.031325 \n",
       "L 42.159844 30.122892 \n",
       "L 42.159844 31.214458 \n",
       "L 42.159844 32.306024 \n",
       "L 42.159844 33.39759 \n",
       "L 42.159844 34.489157 \n",
       "L 42.159844 35.580723 \n",
       "L 42.159844 36.672289 \n",
       "L 42.159844 37.763855 \n",
       "L 42.159844 38.855422 \n",
       "L 42.159844 39.946988 \n",
       "L 42.159844 41.038554 \n",
       "L 42.159844 42.13012 \n",
       "L 42.159844 43.221687 \n",
       "L 42.159844 44.313253 \n",
       "L 42.159844 45.404819 \n",
       "L 42.159844 46.496386 \n",
       "L 42.159844 47.587952 \n",
       "L 42.159844 48.679518 \n",
       "L 42.159844 49.771084 \n",
       "L 42.159844 50.862651 \n",
       "L 42.159844 51.954217 \n",
       "L 42.159844 53.045783 \n",
       "L 42.159844 54.137349 \n",
       "L 42.159844 55.228916 \n",
       "L 42.159844 56.320482 \n",
       "L 42.159844 57.412048 \n",
       "L 42.159844 58.503614 \n",
       "L 42.159844 59.595181 \n",
       "L 42.159844 60.686747 \n",
       "L 42.159844 61.778313 \n",
       "L 42.159844 62.86988 \n",
       "L 42.159844 63.961446 \n",
       "L 42.159844 65.053012 \n",
       "L 42.159844 66.144578 \n",
       "L 42.159844 67.236145 \n",
       "L 42.159844 68.327711 \n",
       "L 42.159844 69.419277 \n",
       "L 42.159844 70.510843 \n",
       "L 42.159844 71.60241 \n",
       "L 42.159844 72.693976 \n",
       "L 42.159844 73.785542 \n",
       "L 42.159844 74.877108 \n",
       "L 42.159844 75.968675 \n",
       "L 42.159844 77.060241 \n",
       "L 42.159844 78.151807 \n",
       "L 42.159844 79.243373 \n",
       "L 42.159844 80.33494 \n",
       "L 42.159844 81.426506 \n",
       "L 42.159844 82.518072 \n",
       "L 42.159844 83.609639 \n",
       "L 42.159844 84.701205 \n",
       "L 42.159844 85.792771 \n",
       "L 42.159844 86.884337 \n",
       "L 42.159844 87.975904 \n",
       "L 42.159844 89.06747 \n",
       "L 42.159844 90.159036 \n",
       "L 42.159844 91.250602 \n",
       "L 42.159844 92.342169 \n",
       "L 42.159844 93.433735 \n",
       "L 42.159844 94.525301 \n",
       "L 42.159844 95.616867 \n",
       "L 42.159844 96.708434 \n",
       "L 42.159844 97.8 \n",
       "L 42.159844 98.891566 \n",
       "L 42.159844 99.983133 \n",
       "L 42.159844 101.074699 \n",
       "L 42.159844 102.166265 \n",
       "L 42.159844 103.257831 \n",
       "L 42.159844 104.349398 \n",
       "L 42.159844 105.440964 \n",
       "L 42.159844 105.713855 \n",
       "z\n",
       "\" style=\"fill:#17becf;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_9\">\n",
       "    <defs>\n",
       "     <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"C0_0_d38420308e\"/>\n",
       "    </defs>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_d38420308e\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_d38420308e\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"284.555324\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"271.419731\" xlink:href=\"#C0_0_d38420308e\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_d38420308e\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_d38420308e\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_d38420308e\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_d38420308e\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_d38420308e\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_d38420308e\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_d38420308e\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_d38420308e\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"91.899957\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"52.493177\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"91.899957\" xlink:href=\"#C0_0_d38420308e\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"196.984703\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"153.199392\" xlink:href=\"#C0_0_d38420308e\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_d38420308e\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_d38420308e\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_d38420308e\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_d38420308e\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_d38420308e\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_d38420308e\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_d38420308e\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_d38420308e\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_d38420308e\" y=\"216.502778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"161.956454\" xlink:href=\"#C0_0_d38420308e\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_d38420308e\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_d38420308e\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_d38420308e\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_d38420308e\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"175.092047\" xlink:href=\"#C0_0_d38420308e\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_d38420308e\" y=\"101.155556\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_d38420308e\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_d38420308e\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"236.391482\" xlink:href=\"#C0_0_d38420308e\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_d38420308e\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_d38420308e\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"166.334985\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_d38420308e\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"271.419731\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"56.871708\" xlink:href=\"#C0_0_d38420308e\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"240.770013\" xlink:href=\"#C0_0_d38420308e\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_d38420308e\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"140.063799\" xlink:href=\"#C0_0_d38420308e\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_d38420308e\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"267.0412\" xlink:href=\"#C0_0_d38420308e\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_d38420308e\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_d38420308e\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_d38420308e\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_d38420308e\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_d38420308e\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_d38420308e\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"245.148544\" xlink:href=\"#C0_0_d38420308e\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_d38420308e\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"245.148544\" xlink:href=\"#C0_0_d38420308e\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_d38420308e\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_d38420308e\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"267.0412\" xlink:href=\"#C0_0_d38420308e\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"297.690917\" xlink:href=\"#C0_0_d38420308e\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_d38420308e\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.250239\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_d38420308e\" y=\"226.988889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"196.984703\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"310.82651\" xlink:href=\"#C0_0_d38420308e\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"153.199392\" xlink:href=\"#C0_0_d38420308e\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"288.933855\" xlink:href=\"#C0_0_d38420308e\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_d38420308e\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.250239\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"302.069448\" xlink:href=\"#C0_0_d38420308e\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_d38420308e\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_d38420308e\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_d38420308e\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"161.956454\" xlink:href=\"#C0_0_d38420308e\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"240.770013\" xlink:href=\"#C0_0_d38420308e\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"236.391482\" xlink:href=\"#C0_0_d38420308e\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_d38420308e\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_d38420308e\" y=\"101.155556\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"170.713516\" xlink:href=\"#C0_0_d38420308e\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_d38420308e\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_d38420308e\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_d38420308e\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_d38420308e\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_d38420308e\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_d38420308e\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"302.069448\" xlink:href=\"#C0_0_d38420308e\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p1c21df506c)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_d38420308e\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_1\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 196.040964 \n",
       "L 321.159844 196.040964 \n",
       "L 321.159844 196.040964 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_2\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 195.768072 \n",
       "L 321.159844 195.768072 \n",
       "L 321.159844 195.768072 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_3\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 195.495181 \n",
       "L 321.159844 195.495181 \n",
       "L 321.159844 195.495181 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_4\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 195.222289 \n",
       "L 321.159844 195.222289 \n",
       "L 321.159844 195.222289 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_5\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 106.53253 \n",
       "L 321.159844 106.53253 \n",
       "L 321.159844 106.53253 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_6\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 106.259639 \n",
       "L 321.159844 106.259639 \n",
       "L 321.159844 106.259639 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_7\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 105.986747 \n",
       "L 321.159844 105.986747 \n",
       "L 321.159844 105.986747 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_8\">\n",
       "    <path clip-path=\"url(#p1c21df506c)\" d=\"M 42.159844 105.713855 \n",
       "L 321.159844 105.713855 \n",
       "L 321.159844 105.713855 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_9\"/>\n",
       "   <g id=\"patch_3\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 42.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_4\">\n",
       "    <path d=\"M 321.159844 279 \n",
       "L 321.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_5\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 321.159844 279 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_6\">\n",
       "    <path d=\"M 42.159844 7.2 \n",
       "L 321.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"legend_1\">\n",
       "    <g id=\"line2d_25\"/>\n",
       "    <g id=\"line2d_26\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"m1c672fca24\" style=\"stroke:#1f77b4;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#1f77b4;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#m1c672fca24\" y=\"19.857812\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_15\">\n",
       "     <!-- setosa -->\n",
       "     <defs>\n",
       "      <path d=\"M 3.078125 15.484375 \n",
       "L 11.765625 16.84375 \n",
       "Q 12.5 11.625 15.84375 8.84375 \n",
       "Q 19.1875 6.0625 25.203125 6.0625 \n",
       "Q 31.25 6.0625 34.171875 8.515625 \n",
       "Q 37.109375 10.984375 37.109375 14.3125 \n",
       "Q 37.109375 17.28125 34.515625 19 \n",
       "Q 32.71875 20.171875 25.53125 21.96875 \n",
       "Q 15.875 24.421875 12.140625 26.203125 \n",
       "Q 8.40625 27.984375 6.46875 31.125 \n",
       "Q 4.546875 34.28125 4.546875 38.09375 \n",
       "Q 4.546875 41.546875 6.125 44.5 \n",
       "Q 7.71875 47.46875 10.453125 49.421875 \n",
       "Q 12.5 50.921875 16.03125 51.96875 \n",
       "Q 19.578125 53.03125 23.640625 53.03125 \n",
       "Q 29.734375 53.03125 34.34375 51.265625 \n",
       "Q 38.96875 49.515625 41.15625 46.5 \n",
       "Q 43.359375 43.5 44.1875 38.484375 \n",
       "L 35.59375 37.3125 \n",
       "Q 35.015625 41.3125 32.203125 43.546875 \n",
       "Q 29.390625 45.796875 24.265625 45.796875 \n",
       "Q 18.21875 45.796875 15.625 43.796875 \n",
       "Q 13.03125 41.796875 13.03125 39.109375 \n",
       "Q 13.03125 37.40625 14.109375 36.03125 \n",
       "Q 15.1875 34.625 17.484375 33.6875 \n",
       "Q 18.796875 33.203125 25.25 31.453125 \n",
       "Q 34.578125 28.953125 38.25 27.359375 \n",
       "Q 41.9375 25.78125 44.03125 22.75 \n",
       "Q 46.140625 19.734375 46.140625 15.234375 \n",
       "Q 46.140625 10.84375 43.578125 6.953125 \n",
       "Q 41.015625 3.078125 36.171875 0.953125 \n",
       "Q 31.34375 -1.171875 25.25 -1.171875 \n",
       "Q 15.140625 -1.171875 9.84375 3.03125 \n",
       "Q 4.546875 7.234375 3.078125 15.484375 \n",
       "z\n",
       "\" id=\"ArialMT-115\"/>\n",
       "      <path d=\"M 3.328125 25.921875 \n",
       "Q 3.328125 40.328125 11.328125 47.265625 \n",
       "Q 18.015625 53.03125 27.640625 53.03125 \n",
       "Q 38.328125 53.03125 45.109375 46.015625 \n",
       "Q 51.90625 39.015625 51.90625 26.65625 \n",
       "Q 51.90625 16.65625 48.90625 10.90625 \n",
       "Q 45.90625 5.171875 40.15625 2 \n",
       "Q 34.421875 -1.171875 27.640625 -1.171875 \n",
       "Q 16.75 -1.171875 10.03125 5.8125 \n",
       "Q 3.328125 12.796875 3.328125 25.921875 \n",
       "z\n",
       "M 12.359375 25.921875 \n",
       "Q 12.359375 15.96875 16.703125 11.015625 \n",
       "Q 21.046875 6.0625 27.640625 6.0625 \n",
       "Q 34.1875 6.0625 38.53125 11.03125 \n",
       "Q 42.875 16.015625 42.875 26.21875 \n",
       "Q 42.875 35.84375 38.5 40.796875 \n",
       "Q 34.125 45.75 27.640625 45.75 \n",
       "Q 21.046875 45.75 16.703125 40.8125 \n",
       "Q 12.359375 35.890625 12.359375 25.921875 \n",
       "z\n",
       "\" id=\"ArialMT-111\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 23.357812)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"105.615234\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"133.398438\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"189.013672\" xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"239.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"line2d_27\"/>\n",
       "    <g id=\"line2d_28\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"m7841ef2a7c\" style=\"stroke:#8c564b;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#8c564b;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#m7841ef2a7c\" y=\"34.003125\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_16\">\n",
       "     <!-- versicolor -->\n",
       "     <defs>\n",
       "      <path d=\"M 21 0 \n",
       "L 1.265625 51.859375 \n",
       "L 10.546875 51.859375 \n",
       "L 21.6875 20.796875 \n",
       "Q 23.484375 15.765625 25 10.359375 \n",
       "Q 26.171875 14.453125 28.265625 20.21875 \n",
       "L 39.796875 51.859375 \n",
       "L 48.828125 51.859375 \n",
       "L 29.203125 0 \n",
       "z\n",
       "\" id=\"ArialMT-118\"/>\n",
       "      <path d=\"M 6.5 0 \n",
       "L 6.5 51.859375 \n",
       "L 14.40625 51.859375 \n",
       "L 14.40625 44 \n",
       "Q 17.4375 49.515625 20 51.265625 \n",
       "Q 22.5625 53.03125 25.640625 53.03125 \n",
       "Q 30.078125 53.03125 34.671875 50.203125 \n",
       "L 31.640625 42.046875 \n",
       "Q 28.421875 43.953125 25.203125 43.953125 \n",
       "Q 22.3125 43.953125 20.015625 42.21875 \n",
       "Q 17.71875 40.484375 16.75 37.40625 \n",
       "Q 15.28125 32.71875 15.28125 27.15625 \n",
       "L 15.28125 0 \n",
       "z\n",
       "\" id=\"ArialMT-114\"/>\n",
       "      <path d=\"M 40.4375 19 \n",
       "L 49.078125 17.875 \n",
       "Q 47.65625 8.9375 41.8125 3.875 \n",
       "Q 35.984375 -1.171875 27.484375 -1.171875 \n",
       "Q 16.84375 -1.171875 10.375 5.78125 \n",
       "Q 3.90625 12.75 3.90625 25.734375 \n",
       "Q 3.90625 34.125 6.6875 40.421875 \n",
       "Q 9.46875 46.734375 15.15625 49.875 \n",
       "Q 20.84375 53.03125 27.546875 53.03125 \n",
       "Q 35.984375 53.03125 41.359375 48.75 \n",
       "Q 46.734375 44.484375 48.25 36.625 \n",
       "L 39.703125 35.296875 \n",
       "Q 38.484375 40.53125 35.375 43.15625 \n",
       "Q 32.28125 45.796875 27.875 45.796875 \n",
       "Q 21.234375 45.796875 17.078125 41.03125 \n",
       "Q 12.9375 36.28125 12.9375 25.984375 \n",
       "Q 12.9375 15.53125 16.9375 10.796875 \n",
       "Q 20.953125 6.0625 27.390625 6.0625 \n",
       "Q 32.5625 6.0625 36.03125 9.234375 \n",
       "Q 39.5 12.40625 40.4375 19 \n",
       "z\n",
       "\" id=\"ArialMT-99\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 37.503125)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-118\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"105.615234\" xlink:href=\"#ArialMT-114\"/>\n",
       "      <use x=\"138.916016\" xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"188.916016\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"211.132812\" xlink:href=\"#ArialMT-99\"/>\n",
       "      <use x=\"261.132812\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"316.748047\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"338.964844\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"394.580078\" xlink:href=\"#ArialMT-114\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"line2d_29\"/>\n",
       "    <g id=\"line2d_30\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"m2320300544\" style=\"stroke:#17becf;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#17becf;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#m2320300544\" y=\"48.148437\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_17\">\n",
       "     <!-- virginica -->\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 51.648437)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-118\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"72.216797\" xlink:href=\"#ArialMT-114\"/>\n",
       "      <use x=\"105.517578\" xlink:href=\"#ArialMT-103\"/>\n",
       "      <use x=\"161.132812\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"183.349609\" xlink:href=\"#ArialMT-110\"/>\n",
       "      <use x=\"238.964844\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"261.181641\" xlink:href=\"#ArialMT-99\"/>\n",
       "      <use x=\"311.181641\" xlink:href=\"#ArialMT-97\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "  </g>\n",
       " </g>\n",
       " <defs>\n",
       "  <clipPath id=\"p1c21df506c\">\n",
       "   <rect height=\"271.8\" width=\"279\" x=\"42.159844\" y=\"7.2\"/>\n",
       "  </clipPath>\n",
       " </defs>\n",
       "</svg>\n"
      ],
      "text/plain": [
       "<Figure size 360x360 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "plot_boundary(cls, X2, Y)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/svg+xml": [
       "<?xml version=\"1.0\" encoding=\"UTF-8\" standalone=\"no\"?>\n",
       "<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n",
       " \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n",
       "<!-- Generated by graphviz version 2.40.1 (20161225.0304)\n",
       " -->\n",
       "<!-- Title: %3 Pages: 1 -->\n",
       "<svg width=\"424pt\" height=\"392pt\"\n",
       " viewBox=\"0.00 0.00 424.29 392.00\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n",
       "<g id=\"graph0\" class=\"graph\" transform=\"scale(1 1) rotate(0) translate(4 388)\">\n",
       "<title>%3</title>\n",
       "<polygon fill=\"#ffffff\" stroke=\"transparent\" points=\"-4,4 -4,-388 420.2918,-388 420.2918,4 -4,4\"/>\n",
       "<!-- 0 -->\n",
       "<g id=\"node1\" class=\"node\">\n",
       "<title>0</title>\n",
       "<ellipse fill=\"#d3d3d3\" stroke=\"#000000\" cx=\"181.1459\" cy=\"-279\" rx=\"32.4942\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"181.1459\" y=\"-275.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">setosa</text>\n",
       "</g>\n",
       "<!-- 1 -->\n",
       "<g id=\"node2\" class=\"node\">\n",
       "<title>1</title>\n",
       "<ellipse fill=\"#d3d3d3\" stroke=\"#000000\" cx=\"46.1459\" cy=\"-18\" rx=\"46.2923\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"46.1459\" y=\"-14.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">versicolor</text>\n",
       "</g>\n",
       "<!-- 2 -->\n",
       "<g id=\"node3\" class=\"node\">\n",
       "<title>2</title>\n",
       "<ellipse fill=\"#d3d3d3\" stroke=\"#000000\" cx=\"152.1459\" cy=\"-18\" rx=\"42.4939\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"152.1459\" y=\"-14.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">virginica</text>\n",
       "</g>\n",
       "<!-- 3 -->\n",
       "<g id=\"node4\" class=\"node\">\n",
       "<title>3</title>\n",
       "<path fill=\"#ffff00\" stroke=\"#000000\" d=\"M192.1459,-123C192.1459,-123 90.1459,-123 90.1459,-123 84.1459,-123 78.1459,-117 78.1459,-111 78.1459,-111 78.1459,-99 78.1459,-99 78.1459,-93 84.1459,-87 90.1459,-87 90.1459,-87 192.1459,-87 192.1459,-87 198.1459,-87 204.1459,-93 204.1459,-99 204.1459,-99 204.1459,-111 204.1459,-111 204.1459,-117 198.1459,-123 192.1459,-123\"/>\n",
       "<text text-anchor=\"middle\" x=\"141.1459\" y=\"-101.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">petal_width &gt; 1.65?</text>\n",
       "</g>\n",
       "<!-- 3&#45;&gt;1 -->\n",
       "<g id=\"edge1\" class=\"edge\">\n",
       "<title>3&#45;&gt;1</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M121.4618,-86.9735C107.2074,-73.9195 87.7929,-56.1399 72.2456,-41.9018\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"74.2542,-38.9954 64.5156,-34.8228 69.5266,-44.1577 74.2542,-38.9954\"/>\n",
       "<text text-anchor=\"middle\" x=\"114.6459\" y=\"-57.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">False</text>\n",
       "</g>\n",
       "<!-- 3&#45;&gt;2 -->\n",
       "<g id=\"edge2\" class=\"edge\">\n",
       "<title>3&#45;&gt;2</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M143.4251,-86.9735C144.9148,-75.1918 146.8911,-59.5607 148.5857,-46.1581\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"152.0875,-46.3634 149.8696,-36.0034 145.1428,-45.4853 152.0875,-46.3634\"/>\n",
       "<text text-anchor=\"middle\" x=\"161.1459\" y=\"-57.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">True</text>\n",
       "</g>\n",
       "<!-- 4 -->\n",
       "<g id=\"node5\" class=\"node\">\n",
       "<title>4</title>\n",
       "<ellipse fill=\"#d3d3d3\" stroke=\"#000000\" cx=\"264.1459\" cy=\"-18\" rx=\"42.4939\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"264.1459\" y=\"-14.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">virginica</text>\n",
       "</g>\n",
       "<!-- 5 -->\n",
       "<g id=\"node6\" class=\"node\">\n",
       "<title>5</title>\n",
       "<ellipse fill=\"#d3d3d3\" stroke=\"#000000\" cx=\"370.1459\" cy=\"-18\" rx=\"46.2923\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"370.1459\" y=\"-14.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">versicolor</text>\n",
       "</g>\n",
       "<!-- 6 -->\n",
       "<g id=\"node7\" class=\"node\">\n",
       "<title>6</title>\n",
       "<path fill=\"#ffff00\" stroke=\"#000000\" d=\"M336.1459,-123C336.1459,-123 234.1459,-123 234.1459,-123 228.1459,-123 222.1459,-117 222.1459,-111 222.1459,-111 222.1459,-99 222.1459,-99 222.1459,-93 228.1459,-87 234.1459,-87 234.1459,-87 336.1459,-87 336.1459,-87 342.1459,-87 348.1459,-93 348.1459,-99 348.1459,-99 348.1459,-111 348.1459,-111 348.1459,-117 342.1459,-123 336.1459,-123\"/>\n",
       "<text text-anchor=\"middle\" x=\"285.1459\" y=\"-101.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">petal_width &gt; 1.55?</text>\n",
       "</g>\n",
       "<!-- 6&#45;&gt;4 -->\n",
       "<g id=\"edge3\" class=\"edge\">\n",
       "<title>6&#45;&gt;4</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M280.7947,-86.9735C277.9227,-75.0751 274.103,-59.2508 270.8467,-45.7606\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"274.2403,-44.9029 268.4915,-36.0034 267.4357,-46.5455 274.2403,-44.9029\"/>\n",
       "<text text-anchor=\"middle\" x=\"290.6459\" y=\"-57.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">False</text>\n",
       "</g>\n",
       "<!-- 6&#45;&gt;5 -->\n",
       "<g id=\"edge4\" class=\"edge\">\n",
       "<title>6&#45;&gt;5</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M302.758,-86.9735C315.3949,-74.0393 332.5645,-56.4657 346.4099,-42.2945\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"349.2249,-44.4216 353.7098,-34.8228 344.2179,-39.5297 349.2249,-44.4216\"/>\n",
       "<text text-anchor=\"middle\" x=\"346.1459\" y=\"-57.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">True</text>\n",
       "</g>\n",
       "<!-- 7 -->\n",
       "<g id=\"node8\" class=\"node\">\n",
       "<title>7</title>\n",
       "<path fill=\"#ffff00\" stroke=\"#000000\" d=\"M265.6459,-210C265.6459,-210 160.6459,-210 160.6459,-210 154.6459,-210 148.6459,-204 148.6459,-198 148.6459,-198 148.6459,-186 148.6459,-186 148.6459,-180 154.6459,-174 160.6459,-174 160.6459,-174 265.6459,-174 265.6459,-174 271.6459,-174 277.6459,-180 277.6459,-186 277.6459,-186 277.6459,-198 277.6459,-198 277.6459,-204 271.6459,-210 265.6459,-210\"/>\n",
       "<text text-anchor=\"middle\" x=\"213.1459\" y=\"-188.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">petal_length &gt; 4.95?</text>\n",
       "</g>\n",
       "<!-- 7&#45;&gt;3 -->\n",
       "<g id=\"edge5\" class=\"edge\">\n",
       "<title>7&#45;&gt;3</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M198.2275,-173.9735C187.8978,-161.4919 173.9927,-144.6899 162.493,-130.7944\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"165.1173,-128.4758 156.0452,-123.0034 159.7246,-132.9388 165.1173,-128.4758\"/>\n",
       "<text text-anchor=\"middle\" x=\"196.6459\" y=\"-144.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">False</text>\n",
       "</g>\n",
       "<!-- 7&#45;&gt;6 -->\n",
       "<g id=\"edge6\" class=\"edge\">\n",
       "<title>7&#45;&gt;6</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M228.0644,-173.9735C238.394,-161.4919 252.2991,-144.6899 263.7988,-130.7944\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"266.5672,-132.9388 270.2466,-123.0034 261.1745,-128.4758 266.5672,-132.9388\"/>\n",
       "<text text-anchor=\"middle\" x=\"267.1459\" y=\"-144.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">True</text>\n",
       "</g>\n",
       "<!-- 8 -->\n",
       "<g id=\"node9\" class=\"node\">\n",
       "<title>8</title>\n",
       "<ellipse fill=\"#d3d3d3\" stroke=\"#000000\" cx=\"338.1459\" cy=\"-192\" rx=\"42.4939\" ry=\"18\"/>\n",
       "<text text-anchor=\"middle\" x=\"338.1459\" y=\"-188.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">virginica</text>\n",
       "</g>\n",
       "<!-- 9 -->\n",
       "<g id=\"node10\" class=\"node\">\n",
       "<title>9</title>\n",
       "<path fill=\"#ffff00\" stroke=\"#000000\" d=\"M345.1459,-297C345.1459,-297 243.1459,-297 243.1459,-297 237.1459,-297 231.1459,-291 231.1459,-285 231.1459,-285 231.1459,-273 231.1459,-273 231.1459,-267 237.1459,-261 243.1459,-261 243.1459,-261 345.1459,-261 345.1459,-261 351.1459,-261 357.1459,-267 357.1459,-273 357.1459,-273 357.1459,-285 357.1459,-285 357.1459,-291 351.1459,-297 345.1459,-297\"/>\n",
       "<text text-anchor=\"middle\" x=\"294.1459\" y=\"-275.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">petal_width &gt; 1.75?</text>\n",
       "</g>\n",
       "<!-- 9&#45;&gt;7 -->\n",
       "<g id=\"edge7\" class=\"edge\">\n",
       "<title>9&#45;&gt;7</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M277.3627,-260.9735C265.6332,-248.3752 249.8058,-231.3755 236.7993,-217.4055\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"239.2835,-214.9374 229.9077,-210.0034 234.1602,-219.7073 239.2835,-214.9374\"/>\n",
       "<text text-anchor=\"middle\" x=\"273.6459\" y=\"-231.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">False</text>\n",
       "</g>\n",
       "<!-- 9&#45;&gt;8 -->\n",
       "<g id=\"edge8\" class=\"edge\">\n",
       "<title>9&#45;&gt;8</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M303.2627,-260.9735C309.4392,-248.7609 317.7075,-232.4123 324.644,-218.697\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"327.8014,-220.209 329.1913,-209.7057 321.5548,-217.0498 327.8014,-220.209\"/>\n",
       "<text text-anchor=\"middle\" x=\"332.1459\" y=\"-231.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">True</text>\n",
       "</g>\n",
       "<!-- 10 -->\n",
       "<g id=\"node11\" class=\"node\">\n",
       "<title>10</title>\n",
       "<path fill=\"#ffff00\" stroke=\"#000000\" d=\"M254.6459,-384C254.6459,-384 159.6459,-384 159.6459,-384 153.6459,-384 147.6459,-378 147.6459,-372 147.6459,-372 147.6459,-360 147.6459,-360 147.6459,-354 153.6459,-348 159.6459,-348 159.6459,-348 254.6459,-348 254.6459,-348 260.6459,-348 266.6459,-354 266.6459,-360 266.6459,-360 266.6459,-372 266.6459,-372 266.6459,-378 260.6459,-384 254.6459,-384\"/>\n",
       "<text text-anchor=\"middle\" x=\"207.1459\" y=\"-362.3\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">petal_width &gt; 0.8?</text>\n",
       "</g>\n",
       "<!-- 10&#45;&gt;0 -->\n",
       "<g id=\"edge9\" class=\"edge\">\n",
       "<title>10&#45;&gt;0</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M201.7587,-347.9735C198.2028,-336.0751 193.4737,-320.2508 189.4422,-306.7606\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"192.7431,-305.5825 186.5262,-297.0034 186.0362,-307.5869 192.7431,-305.5825\"/>\n",
       "<text text-anchor=\"middle\" x=\"210.6459\" y=\"-318.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">False</text>\n",
       "</g>\n",
       "<!-- 10&#45;&gt;9 -->\n",
       "<g id=\"edge10\" class=\"edge\">\n",
       "<title>10&#45;&gt;9</title>\n",
       "<path fill=\"none\" stroke=\"#000000\" d=\"M225.1724,-347.9735C237.7707,-335.3752 254.7704,-318.3755 268.7404,-304.4055\"/>\n",
       "<polygon fill=\"#000000\" stroke=\"#000000\" points=\"271.5463,-306.5493 276.1425,-297.0034 266.5966,-301.5996 271.5463,-306.5493\"/>\n",
       "<text text-anchor=\"middle\" x=\"269.1459\" y=\"-318.8\" font-family=\"Times,serif\" font-size=\"14.00\" fill=\"#000000\">True</text>\n",
       "</g>\n",
       "</g>\n",
       "</svg>\n"
      ],
      "text/plain": [
       "<graphviz.dot.Digraph at 0x7fb7d0754e90>"
      ]
     },
     "execution_count": 29,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "cls = TreeClassifier(max_depth=4, criterion='gini')\n",
    "cls.fit(X2, Y)\n",
    "cls.draw_tree()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 30,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/svg+xml": [
       "<?xml version=\"1.0\" encoding=\"utf-8\" standalone=\"no\"?>\n",
       "<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\"\n",
       "  \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n",
       "<!-- Created with matplotlib (https://matplotlib.org/) -->\n",
       "<svg height=\"316.534063pt\" version=\"1.1\" viewBox=\"0 0 328.359844 316.534063\" width=\"328.359844pt\" xmlns=\"http://www.w3.org/2000/svg\" xmlns:xlink=\"http://www.w3.org/1999/xlink\">\n",
       " <defs>\n",
       "  <style type=\"text/css\">\n",
       "*{stroke-linecap:butt;stroke-linejoin:round;}\n",
       "  </style>\n",
       " </defs>\n",
       " <g id=\"figure_1\">\n",
       "  <g id=\"patch_1\">\n",
       "   <path d=\"M 0 316.534063 \n",
       "L 328.359844 316.534063 \n",
       "L 328.359844 0 \n",
       "L 0 0 \n",
       "z\n",
       "\" style=\"fill:none;\"/>\n",
       "  </g>\n",
       "  <g id=\"axes_1\">\n",
       "   <g id=\"patch_2\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 321.159844 279 \n",
       "L 321.159844 7.2 \n",
       "L 42.159844 7.2 \n",
       "z\n",
       "\" style=\"fill:#eaeaf2;\"/>\n",
       "   </g>\n",
       "   <g id=\"matplotlib.axis_1\">\n",
       "    <g id=\"xtick_1\">\n",
       "     <g id=\"line2d_1\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 52.493177 279 \n",
       "L 52.493177 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_2\"/>\n",
       "     <g id=\"text_1\">\n",
       "      <!-- 1 -->\n",
       "      <defs>\n",
       "       <path d=\"M 37.25 0 \n",
       "L 28.46875 0 \n",
       "L 28.46875 56 \n",
       "Q 25.296875 52.984375 20.140625 49.953125 \n",
       "Q 14.984375 46.921875 10.890625 45.40625 \n",
       "L 10.890625 53.90625 \n",
       "Q 18.265625 57.375 23.78125 62.296875 \n",
       "Q 29.296875 67.234375 31.59375 71.875 \n",
       "L 37.25 71.875 \n",
       "z\n",
       "\" id=\"ArialMT-49\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(49.712708 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_2\">\n",
       "     <g id=\"line2d_3\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 96.278488 279 \n",
       "L 96.278488 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_4\"/>\n",
       "     <g id=\"text_2\">\n",
       "      <!-- 2 -->\n",
       "      <defs>\n",
       "       <path d=\"M 50.34375 8.453125 \n",
       "L 50.34375 0 \n",
       "L 3.03125 0 \n",
       "Q 2.9375 3.171875 4.046875 6.109375 \n",
       "Q 5.859375 10.9375 9.828125 15.625 \n",
       "Q 13.8125 20.3125 21.34375 26.46875 \n",
       "Q 33.015625 36.03125 37.109375 41.625 \n",
       "Q 41.21875 47.21875 41.21875 52.203125 \n",
       "Q 41.21875 57.421875 37.46875 61 \n",
       "Q 33.734375 64.59375 27.734375 64.59375 \n",
       "Q 21.390625 64.59375 17.578125 60.78125 \n",
       "Q 13.765625 56.984375 13.71875 50.25 \n",
       "L 4.6875 51.171875 \n",
       "Q 5.609375 61.28125 11.65625 66.578125 \n",
       "Q 17.71875 71.875 27.9375 71.875 \n",
       "Q 38.234375 71.875 44.234375 66.15625 \n",
       "Q 50.25 60.453125 50.25 52 \n",
       "Q 50.25 47.703125 48.484375 43.546875 \n",
       "Q 46.734375 39.40625 42.65625 34.8125 \n",
       "Q 38.578125 30.21875 29.109375 22.21875 \n",
       "Q 21.1875 15.578125 18.9375 13.203125 \n",
       "Q 16.703125 10.84375 15.234375 8.453125 \n",
       "z\n",
       "\" id=\"ArialMT-50\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(93.498019 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_3\">\n",
       "     <g id=\"line2d_5\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 140.063799 279 \n",
       "L 140.063799 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_6\"/>\n",
       "     <g id=\"text_3\">\n",
       "      <!-- 3 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.203125 18.890625 \n",
       "L 12.984375 20.0625 \n",
       "Q 14.5 12.59375 18.140625 9.296875 \n",
       "Q 21.78125 6 27 6 \n",
       "Q 33.203125 6 37.46875 10.296875 \n",
       "Q 41.75 14.59375 41.75 20.953125 \n",
       "Q 41.75 27 37.796875 30.921875 \n",
       "Q 33.84375 34.859375 27.734375 34.859375 \n",
       "Q 25.25 34.859375 21.53125 33.890625 \n",
       "L 22.515625 41.609375 \n",
       "Q 23.390625 41.5 23.921875 41.5 \n",
       "Q 29.546875 41.5 34.03125 44.421875 \n",
       "Q 38.53125 47.359375 38.53125 53.46875 \n",
       "Q 38.53125 58.296875 35.25 61.46875 \n",
       "Q 31.984375 64.65625 26.8125 64.65625 \n",
       "Q 21.6875 64.65625 18.265625 61.421875 \n",
       "Q 14.84375 58.203125 13.875 51.765625 \n",
       "L 5.078125 53.328125 \n",
       "Q 6.6875 62.15625 12.390625 67.015625 \n",
       "Q 18.109375 71.875 26.609375 71.875 \n",
       "Q 32.46875 71.875 37.390625 69.359375 \n",
       "Q 42.328125 66.84375 44.9375 62.5 \n",
       "Q 47.5625 58.15625 47.5625 53.265625 \n",
       "Q 47.5625 48.640625 45.0625 44.828125 \n",
       "Q 42.578125 41.015625 37.703125 38.765625 \n",
       "Q 44.046875 37.3125 47.5625 32.6875 \n",
       "Q 51.078125 28.078125 51.078125 21.140625 \n",
       "Q 51.078125 11.765625 44.234375 5.25 \n",
       "Q 37.40625 -1.265625 26.953125 -1.265625 \n",
       "Q 17.53125 -1.265625 11.296875 4.34375 \n",
       "Q 5.078125 9.96875 4.203125 18.890625 \n",
       "z\n",
       "\" id=\"ArialMT-51\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(137.28333 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-51\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_4\">\n",
       "     <g id=\"line2d_7\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 183.849109 279 \n",
       "L 183.849109 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_8\"/>\n",
       "     <g id=\"text_4\">\n",
       "      <!-- 4 -->\n",
       "      <defs>\n",
       "       <path d=\"M 32.328125 0 \n",
       "L 32.328125 17.140625 \n",
       "L 1.265625 17.140625 \n",
       "L 1.265625 25.203125 \n",
       "L 33.9375 71.578125 \n",
       "L 41.109375 71.578125 \n",
       "L 41.109375 25.203125 \n",
       "L 50.78125 25.203125 \n",
       "L 50.78125 17.140625 \n",
       "L 41.109375 17.140625 \n",
       "L 41.109375 0 \n",
       "z\n",
       "M 32.328125 25.203125 \n",
       "L 32.328125 57.46875 \n",
       "L 9.90625 25.203125 \n",
       "z\n",
       "\" id=\"ArialMT-52\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(181.068641 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-52\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_5\">\n",
       "     <g id=\"line2d_9\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 227.63442 279 \n",
       "L 227.63442 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_10\"/>\n",
       "     <g id=\"text_5\">\n",
       "      <!-- 5 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.15625 18.75 \n",
       "L 13.375 19.53125 \n",
       "Q 14.40625 12.796875 18.140625 9.390625 \n",
       "Q 21.875 6 27.15625 6 \n",
       "Q 33.5 6 37.890625 10.78125 \n",
       "Q 42.28125 15.578125 42.28125 23.484375 \n",
       "Q 42.28125 31 38.0625 35.34375 \n",
       "Q 33.84375 39.703125 27 39.703125 \n",
       "Q 22.75 39.703125 19.328125 37.765625 \n",
       "Q 15.921875 35.84375 13.96875 32.765625 \n",
       "L 5.71875 33.84375 \n",
       "L 12.640625 70.609375 \n",
       "L 48.25 70.609375 \n",
       "L 48.25 62.203125 \n",
       "L 19.671875 62.203125 \n",
       "L 15.828125 42.96875 \n",
       "Q 22.265625 47.46875 29.34375 47.46875 \n",
       "Q 38.71875 47.46875 45.15625 40.96875 \n",
       "Q 51.609375 34.46875 51.609375 24.265625 \n",
       "Q 51.609375 14.546875 45.953125 7.46875 \n",
       "Q 39.0625 -1.21875 27.15625 -1.21875 \n",
       "Q 17.390625 -1.21875 11.203125 4.25 \n",
       "Q 5.03125 9.71875 4.15625 18.75 \n",
       "z\n",
       "\" id=\"ArialMT-53\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(224.853951 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_6\">\n",
       "     <g id=\"line2d_11\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 271.419731 279 \n",
       "L 271.419731 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_12\"/>\n",
       "     <g id=\"text_6\">\n",
       "      <!-- 6 -->\n",
       "      <defs>\n",
       "       <path d=\"M 49.75 54.046875 \n",
       "L 41.015625 53.375 \n",
       "Q 39.84375 58.546875 37.703125 60.890625 \n",
       "Q 34.125 64.65625 28.90625 64.65625 \n",
       "Q 24.703125 64.65625 21.53125 62.3125 \n",
       "Q 17.390625 59.28125 14.984375 53.46875 \n",
       "Q 12.59375 47.65625 12.5 36.921875 \n",
       "Q 15.671875 41.75 20.265625 44.09375 \n",
       "Q 24.859375 46.4375 29.890625 46.4375 \n",
       "Q 38.671875 46.4375 44.84375 39.96875 \n",
       "Q 51.03125 33.5 51.03125 23.25 \n",
       "Q 51.03125 16.5 48.125 10.71875 \n",
       "Q 45.21875 4.9375 40.140625 1.859375 \n",
       "Q 35.0625 -1.21875 28.609375 -1.21875 \n",
       "Q 17.625 -1.21875 10.6875 6.859375 \n",
       "Q 3.765625 14.9375 3.765625 33.5 \n",
       "Q 3.765625 54.25 11.421875 63.671875 \n",
       "Q 18.109375 71.875 29.4375 71.875 \n",
       "Q 37.890625 71.875 43.28125 67.140625 \n",
       "Q 48.6875 62.40625 49.75 54.046875 \n",
       "z\n",
       "M 13.875 23.1875 \n",
       "Q 13.875 18.65625 15.796875 14.5 \n",
       "Q 17.71875 10.359375 21.1875 8.171875 \n",
       "Q 24.65625 6 28.46875 6 \n",
       "Q 34.03125 6 38.03125 10.484375 \n",
       "Q 42.046875 14.984375 42.046875 22.703125 \n",
       "Q 42.046875 30.125 38.078125 34.390625 \n",
       "Q 34.125 38.671875 28.125 38.671875 \n",
       "Q 22.171875 38.671875 18.015625 34.390625 \n",
       "Q 13.875 30.125 13.875 23.1875 \n",
       "z\n",
       "\" id=\"ArialMT-54\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(268.639262 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-54\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"xtick_7\">\n",
       "     <g id=\"line2d_13\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 315.205041 279 \n",
       "L 315.205041 7.2 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_14\"/>\n",
       "     <g id=\"text_7\">\n",
       "      <!-- 7 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.734375 62.203125 \n",
       "L 4.734375 70.65625 \n",
       "L 51.078125 70.65625 \n",
       "L 51.078125 63.8125 \n",
       "Q 44.234375 56.546875 37.515625 44.484375 \n",
       "Q 30.8125 32.421875 27.15625 19.671875 \n",
       "Q 24.515625 10.6875 23.78125 0 \n",
       "L 14.75 0 \n",
       "Q 14.890625 8.453125 18.0625 20.40625 \n",
       "Q 21.234375 32.375 27.171875 43.484375 \n",
       "Q 33.109375 54.59375 39.796875 62.203125 \n",
       "z\n",
       "\" id=\"ArialMT-55\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(312.424573 293.157812)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-55\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_8\">\n",
       "     <!-- petal_length -->\n",
       "     <defs>\n",
       "      <path d=\"M 6.59375 -19.875 \n",
       "L 6.59375 51.859375 \n",
       "L 14.59375 51.859375 \n",
       "L 14.59375 45.125 \n",
       "Q 17.4375 49.078125 21 51.046875 \n",
       "Q 24.5625 53.03125 29.640625 53.03125 \n",
       "Q 36.28125 53.03125 41.359375 49.609375 \n",
       "Q 46.4375 46.1875 49.015625 39.953125 \n",
       "Q 51.609375 33.734375 51.609375 26.3125 \n",
       "Q 51.609375 18.359375 48.75 11.984375 \n",
       "Q 45.90625 5.609375 40.453125 2.21875 \n",
       "Q 35.015625 -1.171875 29 -1.171875 \n",
       "Q 24.609375 -1.171875 21.109375 0.6875 \n",
       "Q 17.625 2.546875 15.375 5.375 \n",
       "L 15.375 -19.875 \n",
       "z\n",
       "M 14.546875 25.640625 \n",
       "Q 14.546875 15.625 18.59375 10.84375 \n",
       "Q 22.65625 6.0625 28.421875 6.0625 \n",
       "Q 34.28125 6.0625 38.453125 11.015625 \n",
       "Q 42.625 15.96875 42.625 26.375 \n",
       "Q 42.625 36.28125 38.546875 41.203125 \n",
       "Q 34.46875 46.140625 28.8125 46.140625 \n",
       "Q 23.1875 46.140625 18.859375 40.890625 \n",
       "Q 14.546875 35.640625 14.546875 25.640625 \n",
       "z\n",
       "\" id=\"ArialMT-112\"/>\n",
       "      <path d=\"M 42.09375 16.703125 \n",
       "L 51.171875 15.578125 \n",
       "Q 49.03125 7.625 43.21875 3.21875 \n",
       "Q 37.40625 -1.171875 28.375 -1.171875 \n",
       "Q 17 -1.171875 10.328125 5.828125 \n",
       "Q 3.65625 12.84375 3.65625 25.484375 \n",
       "Q 3.65625 38.578125 10.390625 45.796875 \n",
       "Q 17.140625 53.03125 27.875 53.03125 \n",
       "Q 38.28125 53.03125 44.875 45.953125 \n",
       "Q 51.46875 38.875 51.46875 26.03125 \n",
       "Q 51.46875 25.25 51.421875 23.6875 \n",
       "L 12.75 23.6875 \n",
       "Q 13.234375 15.140625 17.578125 10.59375 \n",
       "Q 21.921875 6.0625 28.421875 6.0625 \n",
       "Q 33.25 6.0625 36.671875 8.59375 \n",
       "Q 40.09375 11.140625 42.09375 16.703125 \n",
       "z\n",
       "M 13.234375 30.90625 \n",
       "L 42.1875 30.90625 \n",
       "Q 41.609375 37.453125 38.875 40.71875 \n",
       "Q 34.671875 45.796875 27.984375 45.796875 \n",
       "Q 21.921875 45.796875 17.796875 41.75 \n",
       "Q 13.671875 37.703125 13.234375 30.90625 \n",
       "z\n",
       "\" id=\"ArialMT-101\"/>\n",
       "      <path d=\"M 25.78125 7.859375 \n",
       "L 27.046875 0.09375 \n",
       "Q 23.34375 -0.6875 20.40625 -0.6875 \n",
       "Q 15.625 -0.6875 12.984375 0.828125 \n",
       "Q 10.359375 2.34375 9.28125 4.8125 \n",
       "Q 8.203125 7.28125 8.203125 15.1875 \n",
       "L 8.203125 45.015625 \n",
       "L 1.765625 45.015625 \n",
       "L 1.765625 51.859375 \n",
       "L 8.203125 51.859375 \n",
       "L 8.203125 64.703125 \n",
       "L 16.9375 69.96875 \n",
       "L 16.9375 51.859375 \n",
       "L 25.78125 51.859375 \n",
       "L 25.78125 45.015625 \n",
       "L 16.9375 45.015625 \n",
       "L 16.9375 14.703125 \n",
       "Q 16.9375 10.9375 17.40625 9.859375 \n",
       "Q 17.875 8.796875 18.921875 8.15625 \n",
       "Q 19.96875 7.515625 21.921875 7.515625 \n",
       "Q 23.390625 7.515625 25.78125 7.859375 \n",
       "z\n",
       "\" id=\"ArialMT-116\"/>\n",
       "      <path d=\"M 40.4375 6.390625 \n",
       "Q 35.546875 2.25 31.03125 0.53125 \n",
       "Q 26.515625 -1.171875 21.34375 -1.171875 \n",
       "Q 12.796875 -1.171875 8.203125 3 \n",
       "Q 3.609375 7.171875 3.609375 13.671875 \n",
       "Q 3.609375 17.484375 5.34375 20.625 \n",
       "Q 7.078125 23.78125 9.890625 25.6875 \n",
       "Q 12.703125 27.59375 16.21875 28.5625 \n",
       "Q 18.796875 29.25 24.03125 29.890625 \n",
       "Q 34.671875 31.15625 39.703125 32.90625 \n",
       "Q 39.75 34.71875 39.75 35.203125 \n",
       "Q 39.75 40.578125 37.25 42.78125 \n",
       "Q 33.890625 45.75 27.25 45.75 \n",
       "Q 21.046875 45.75 18.09375 43.578125 \n",
       "Q 15.140625 41.40625 13.71875 35.890625 \n",
       "L 5.125 37.0625 \n",
       "Q 6.296875 42.578125 8.984375 45.96875 \n",
       "Q 11.671875 49.359375 16.75 51.1875 \n",
       "Q 21.828125 53.03125 28.515625 53.03125 \n",
       "Q 35.15625 53.03125 39.296875 51.46875 \n",
       "Q 43.453125 49.90625 45.40625 47.53125 \n",
       "Q 47.359375 45.171875 48.140625 41.546875 \n",
       "Q 48.578125 39.3125 48.578125 33.453125 \n",
       "L 48.578125 21.734375 \n",
       "Q 48.578125 9.46875 49.140625 6.21875 \n",
       "Q 49.703125 2.984375 51.375 0 \n",
       "L 42.1875 0 \n",
       "Q 40.828125 2.734375 40.4375 6.390625 \n",
       "z\n",
       "M 39.703125 26.03125 \n",
       "Q 34.90625 24.078125 25.34375 22.703125 \n",
       "Q 19.921875 21.921875 17.671875 20.9375 \n",
       "Q 15.4375 19.96875 14.203125 18.09375 \n",
       "Q 12.984375 16.21875 12.984375 13.921875 \n",
       "Q 12.984375 10.40625 15.640625 8.0625 \n",
       "Q 18.3125 5.71875 23.4375 5.71875 \n",
       "Q 28.515625 5.71875 32.46875 7.9375 \n",
       "Q 36.421875 10.15625 38.28125 14.015625 \n",
       "Q 39.703125 17 39.703125 22.796875 \n",
       "z\n",
       "\" id=\"ArialMT-97\"/>\n",
       "      <path d=\"M 6.390625 0 \n",
       "L 6.390625 71.578125 \n",
       "L 15.1875 71.578125 \n",
       "L 15.1875 0 \n",
       "z\n",
       "\" id=\"ArialMT-108\"/>\n",
       "      <path d=\"M -1.515625 -19.875 \n",
       "L -1.515625 -13.53125 \n",
       "L 56.734375 -13.53125 \n",
       "L 56.734375 -19.875 \n",
       "z\n",
       "\" id=\"ArialMT-95\"/>\n",
       "      <path d=\"M 6.59375 0 \n",
       "L 6.59375 51.859375 \n",
       "L 14.5 51.859375 \n",
       "L 14.5 44.484375 \n",
       "Q 20.21875 53.03125 31 53.03125 \n",
       "Q 35.6875 53.03125 39.625 51.34375 \n",
       "Q 43.5625 49.65625 45.515625 46.921875 \n",
       "Q 47.46875 44.1875 48.25 40.4375 \n",
       "Q 48.734375 37.984375 48.734375 31.890625 \n",
       "L 48.734375 0 \n",
       "L 39.9375 0 \n",
       "L 39.9375 31.546875 \n",
       "Q 39.9375 36.921875 38.90625 39.578125 \n",
       "Q 37.890625 42.234375 35.28125 43.8125 \n",
       "Q 32.671875 45.40625 29.15625 45.40625 \n",
       "Q 23.53125 45.40625 19.453125 41.84375 \n",
       "Q 15.375 38.28125 15.375 28.328125 \n",
       "L 15.375 0 \n",
       "z\n",
       "\" id=\"ArialMT-110\"/>\n",
       "      <path d=\"M 4.984375 -4.296875 \n",
       "L 13.53125 -5.5625 \n",
       "Q 14.0625 -9.515625 16.5 -11.328125 \n",
       "Q 19.78125 -13.765625 25.4375 -13.765625 \n",
       "Q 31.546875 -13.765625 34.859375 -11.328125 \n",
       "Q 38.1875 -8.890625 39.359375 -4.5 \n",
       "Q 40.046875 -1.8125 39.984375 6.78125 \n",
       "Q 34.234375 0 25.640625 0 \n",
       "Q 14.9375 0 9.078125 7.71875 \n",
       "Q 3.21875 15.4375 3.21875 26.21875 \n",
       "Q 3.21875 33.640625 5.90625 39.90625 \n",
       "Q 8.59375 46.1875 13.6875 49.609375 \n",
       "Q 18.796875 53.03125 25.6875 53.03125 \n",
       "Q 34.859375 53.03125 40.828125 45.609375 \n",
       "L 40.828125 51.859375 \n",
       "L 48.921875 51.859375 \n",
       "L 48.921875 7.03125 \n",
       "Q 48.921875 -5.078125 46.453125 -10.125 \n",
       "Q 44 -15.1875 38.640625 -18.109375 \n",
       "Q 33.296875 -21.046875 25.484375 -21.046875 \n",
       "Q 16.21875 -21.046875 10.5 -16.875 \n",
       "Q 4.78125 -12.703125 4.984375 -4.296875 \n",
       "z\n",
       "M 12.25 26.859375 \n",
       "Q 12.25 16.65625 16.296875 11.96875 \n",
       "Q 20.359375 7.28125 26.46875 7.28125 \n",
       "Q 32.515625 7.28125 36.609375 11.9375 \n",
       "Q 40.71875 16.609375 40.71875 26.5625 \n",
       "Q 40.71875 36.078125 36.5 40.90625 \n",
       "Q 32.28125 45.75 26.3125 45.75 \n",
       "Q 20.453125 45.75 16.34375 40.984375 \n",
       "Q 12.25 36.234375 12.25 26.859375 \n",
       "z\n",
       "\" id=\"ArialMT-103\"/>\n",
       "      <path d=\"M 6.59375 0 \n",
       "L 6.59375 71.578125 \n",
       "L 15.375 71.578125 \n",
       "L 15.375 45.90625 \n",
       "Q 21.53125 53.03125 30.90625 53.03125 \n",
       "Q 36.671875 53.03125 40.921875 50.75 \n",
       "Q 45.171875 48.484375 47 44.484375 \n",
       "Q 48.828125 40.484375 48.828125 32.859375 \n",
       "L 48.828125 0 \n",
       "L 40.046875 0 \n",
       "L 40.046875 32.859375 \n",
       "Q 40.046875 39.453125 37.1875 42.453125 \n",
       "Q 34.328125 45.453125 29.109375 45.453125 \n",
       "Q 25.203125 45.453125 21.75 43.421875 \n",
       "Q 18.3125 41.40625 16.84375 37.9375 \n",
       "Q 15.375 34.46875 15.375 28.375 \n",
       "L 15.375 0 \n",
       "z\n",
       "\" id=\"ArialMT-104\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(151.691719 307.018906)scale(0.11 -0.11)\">\n",
       "      <use xlink:href=\"#ArialMT-112\"/>\n",
       "      <use x=\"55.615234\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"111.230469\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"139.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "      <use x=\"194.628906\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"216.845703\" xlink:href=\"#ArialMT-95\"/>\n",
       "      <use x=\"272.460938\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"294.677734\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"350.292969\" xlink:href=\"#ArialMT-110\"/>\n",
       "      <use x=\"405.908203\" xlink:href=\"#ArialMT-103\"/>\n",
       "      <use x=\"461.523438\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"489.306641\" xlink:href=\"#ArialMT-104\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"matplotlib.axis_2\">\n",
       "    <g id=\"ytick_1\">\n",
       "     <g id=\"line2d_15\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 226.988889 \n",
       "L 321.159844 226.988889 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_16\"/>\n",
       "     <g id=\"text_9\">\n",
       "      <!-- 0.5 -->\n",
       "      <defs>\n",
       "       <path d=\"M 4.15625 35.296875 \n",
       "Q 4.15625 48 6.765625 55.734375 \n",
       "Q 9.375 63.484375 14.515625 67.671875 \n",
       "Q 19.671875 71.875 27.484375 71.875 \n",
       "Q 33.25 71.875 37.59375 69.546875 \n",
       "Q 41.9375 67.234375 44.765625 62.859375 \n",
       "Q 47.609375 58.5 49.21875 52.21875 \n",
       "Q 50.828125 45.953125 50.828125 35.296875 \n",
       "Q 50.828125 22.703125 48.234375 14.96875 \n",
       "Q 45.65625 7.234375 40.5 3 \n",
       "Q 35.359375 -1.21875 27.484375 -1.21875 \n",
       "Q 17.140625 -1.21875 11.234375 6.203125 \n",
       "Q 4.15625 15.140625 4.15625 35.296875 \n",
       "z\n",
       "M 13.1875 35.296875 \n",
       "Q 13.1875 17.671875 17.3125 11.828125 \n",
       "Q 21.4375 6 27.484375 6 \n",
       "Q 33.546875 6 37.671875 11.859375 \n",
       "Q 41.796875 17.71875 41.796875 35.296875 \n",
       "Q 41.796875 52.984375 37.671875 58.78125 \n",
       "Q 33.546875 64.59375 27.390625 64.59375 \n",
       "Q 21.34375 64.59375 17.71875 59.46875 \n",
       "Q 13.1875 52.9375 13.1875 35.296875 \n",
       "z\n",
       "\" id=\"ArialMT-48\"/>\n",
       "       <path d=\"M 9.078125 0 \n",
       "L 9.078125 10.015625 \n",
       "L 19.09375 10.015625 \n",
       "L 19.09375 0 \n",
       "z\n",
       "\" id=\"ArialMT-46\"/>\n",
       "      </defs>\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 230.567795)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-48\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_2\">\n",
       "     <g id=\"line2d_17\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 174.558333 \n",
       "L 321.159844 174.558333 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_18\"/>\n",
       "     <g id=\"text_10\">\n",
       "      <!-- 1.0 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 178.13724)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_3\">\n",
       "     <g id=\"line2d_19\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 122.127778 \n",
       "L 321.159844 122.127778 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_20\"/>\n",
       "     <g id=\"text_11\">\n",
       "      <!-- 1.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 125.706684)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-49\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_4\">\n",
       "     <g id=\"line2d_21\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 69.697222 \n",
       "L 321.159844 69.697222 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_22\"/>\n",
       "     <g id=\"text_12\">\n",
       "      <!-- 2.0 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 73.276128)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-48\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"ytick_5\">\n",
       "     <g id=\"line2d_23\">\n",
       "      <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 17.266667 \n",
       "L 321.159844 17.266667 \n",
       "\" style=\"fill:none;stroke:#ffffff;stroke-linecap:round;\"/>\n",
       "     </g>\n",
       "     <g id=\"line2d_24\"/>\n",
       "     <g id=\"text_13\">\n",
       "      <!-- 2.5 -->\n",
       "      <g style=\"fill:#262626;\" transform=\"translate(21.259844 20.845573)scale(0.1 -0.1)\">\n",
       "       <use xlink:href=\"#ArialMT-50\"/>\n",
       "       <use x=\"55.615234\" xlink:href=\"#ArialMT-46\"/>\n",
       "       <use x=\"83.398438\" xlink:href=\"#ArialMT-53\"/>\n",
       "      </g>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_14\">\n",
       "     <!-- petal_width -->\n",
       "     <defs>\n",
       "      <path d=\"M 16.15625 0 \n",
       "L 0.296875 51.859375 \n",
       "L 9.375 51.859375 \n",
       "L 17.625 21.921875 \n",
       "L 20.703125 10.796875 \n",
       "Q 20.90625 11.625 23.390625 21.484375 \n",
       "L 31.640625 51.859375 \n",
       "L 40.671875 51.859375 \n",
       "L 48.4375 21.78125 \n",
       "L 51.03125 11.859375 \n",
       "L 54 21.875 \n",
       "L 62.890625 51.859375 \n",
       "L 71.4375 51.859375 \n",
       "L 55.21875 0 \n",
       "L 46.09375 0 \n",
       "L 37.84375 31.0625 \n",
       "L 35.84375 39.890625 \n",
       "L 25.34375 0 \n",
       "z\n",
       "\" id=\"ArialMT-119\"/>\n",
       "      <path d=\"M 6.640625 61.46875 \n",
       "L 6.640625 71.578125 \n",
       "L 15.4375 71.578125 \n",
       "L 15.4375 61.46875 \n",
       "z\n",
       "M 6.640625 0 \n",
       "L 6.640625 51.859375 \n",
       "L 15.4375 51.859375 \n",
       "L 15.4375 0 \n",
       "z\n",
       "\" id=\"ArialMT-105\"/>\n",
       "      <path d=\"M 40.234375 0 \n",
       "L 40.234375 6.546875 \n",
       "Q 35.296875 -1.171875 25.734375 -1.171875 \n",
       "Q 19.53125 -1.171875 14.328125 2.25 \n",
       "Q 9.125 5.671875 6.265625 11.796875 \n",
       "Q 3.421875 17.921875 3.421875 25.875 \n",
       "Q 3.421875 33.640625 6 39.96875 \n",
       "Q 8.59375 46.296875 13.765625 49.65625 \n",
       "Q 18.953125 53.03125 25.34375 53.03125 \n",
       "Q 30.03125 53.03125 33.6875 51.046875 \n",
       "Q 37.359375 49.078125 39.65625 45.90625 \n",
       "L 39.65625 71.578125 \n",
       "L 48.390625 71.578125 \n",
       "L 48.390625 0 \n",
       "z\n",
       "M 12.453125 25.875 \n",
       "Q 12.453125 15.921875 16.640625 10.984375 \n",
       "Q 20.84375 6.0625 26.5625 6.0625 \n",
       "Q 32.328125 6.0625 36.34375 10.765625 \n",
       "Q 40.375 15.484375 40.375 25.140625 \n",
       "Q 40.375 35.796875 36.265625 40.765625 \n",
       "Q 32.171875 45.75 26.171875 45.75 \n",
       "Q 20.3125 45.75 16.375 40.96875 \n",
       "Q 12.453125 36.1875 12.453125 25.875 \n",
       "z\n",
       "\" id=\"ArialMT-100\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(15.073594 170.923125)rotate(-90)scale(0.11 -0.11)\">\n",
       "      <use xlink:href=\"#ArialMT-112\"/>\n",
       "      <use x=\"55.615234\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"111.230469\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"139.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "      <use x=\"194.628906\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"216.845703\" xlink:href=\"#ArialMT-95\"/>\n",
       "      <use x=\"272.460938\" xlink:href=\"#ArialMT-119\"/>\n",
       "      <use x=\"344.677734\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"366.894531\" xlink:href=\"#ArialMT-100\"/>\n",
       "      <use x=\"422.509766\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"450.292969\" xlink:href=\"#ArialMT-104\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_1\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 43.280326 279 \n",
       "L 44.400808 279 \n",
       "L 45.52129 279 \n",
       "L 46.641771 279 \n",
       "L 47.762253 279 \n",
       "L 48.882735 279 \n",
       "L 50.003217 279 \n",
       "L 51.123699 279 \n",
       "L 52.244181 279 \n",
       "L 53.364663 279 \n",
       "L 54.485145 279 \n",
       "L 55.605627 279 \n",
       "L 56.726109 279 \n",
       "L 57.846591 279 \n",
       "L 58.967073 279 \n",
       "L 60.087555 279 \n",
       "L 61.208037 279 \n",
       "L 62.328518 279 \n",
       "L 63.449 279 \n",
       "L 64.569482 279 \n",
       "L 65.689964 279 \n",
       "L 66.810446 279 \n",
       "L 67.930928 279 \n",
       "L 69.05141 279 \n",
       "L 70.171892 279 \n",
       "L 71.292374 279 \n",
       "L 72.412856 279 \n",
       "L 73.533338 279 \n",
       "L 74.65382 279 \n",
       "L 75.774302 279 \n",
       "L 76.894784 279 \n",
       "L 78.015265 279 \n",
       "L 79.135747 279 \n",
       "L 80.256229 279 \n",
       "L 81.376711 279 \n",
       "L 82.497193 279 \n",
       "L 83.617675 279 \n",
       "L 84.738157 279 \n",
       "L 85.858639 279 \n",
       "L 86.979121 279 \n",
       "L 88.099603 279 \n",
       "L 89.220085 279 \n",
       "L 90.340567 279 \n",
       "L 91.461049 279 \n",
       "L 92.58153 279 \n",
       "L 93.702012 279 \n",
       "L 94.822494 279 \n",
       "L 95.942976 279 \n",
       "L 97.063458 279 \n",
       "L 98.18394 279 \n",
       "L 99.304422 279 \n",
       "L 100.424904 279 \n",
       "L 101.545386 279 \n",
       "L 102.665868 279 \n",
       "L 103.78635 279 \n",
       "L 104.906832 279 \n",
       "L 106.027314 279 \n",
       "L 107.147796 279 \n",
       "L 108.268277 279 \n",
       "L 109.388759 279 \n",
       "L 110.509241 279 \n",
       "L 111.629723 279 \n",
       "L 112.750205 279 \n",
       "L 113.870687 279 \n",
       "L 114.991169 279 \n",
       "L 116.111651 279 \n",
       "L 117.232133 279 \n",
       "L 118.352615 279 \n",
       "L 119.473097 279 \n",
       "L 120.593579 279 \n",
       "L 121.714061 279 \n",
       "L 122.834543 279 \n",
       "L 123.955024 279 \n",
       "L 125.075506 279 \n",
       "L 126.195988 279 \n",
       "L 127.31647 279 \n",
       "L 128.436952 279 \n",
       "L 129.557434 279 \n",
       "L 130.677916 279 \n",
       "L 131.798398 279 \n",
       "L 132.91888 279 \n",
       "L 134.039362 279 \n",
       "L 135.159844 279 \n",
       "L 136.280326 279 \n",
       "L 137.400808 279 \n",
       "L 138.52129 279 \n",
       "L 139.641771 279 \n",
       "L 140.762253 279 \n",
       "L 141.882735 279 \n",
       "L 143.003217 279 \n",
       "L 144.123699 279 \n",
       "L 145.244181 279 \n",
       "L 146.364663 279 \n",
       "L 147.485145 279 \n",
       "L 148.605627 279 \n",
       "L 149.726109 279 \n",
       "L 150.846591 279 \n",
       "L 151.967073 279 \n",
       "L 153.087555 279 \n",
       "L 154.208037 279 \n",
       "L 155.328518 279 \n",
       "L 156.449 279 \n",
       "L 157.569482 279 \n",
       "L 158.689964 279 \n",
       "L 159.810446 279 \n",
       "L 160.930928 279 \n",
       "L 162.05141 279 \n",
       "L 163.171892 279 \n",
       "L 164.292374 279 \n",
       "L 165.412856 279 \n",
       "L 166.533338 279 \n",
       "L 167.65382 279 \n",
       "L 168.774302 279 \n",
       "L 169.894784 279 \n",
       "L 171.015265 279 \n",
       "L 172.135747 279 \n",
       "L 173.256229 279 \n",
       "L 174.376711 279 \n",
       "L 175.497193 279 \n",
       "L 176.617675 279 \n",
       "L 177.738157 279 \n",
       "L 178.858639 279 \n",
       "L 179.979121 279 \n",
       "L 181.099603 279 \n",
       "L 182.220085 279 \n",
       "L 183.340567 279 \n",
       "L 184.461049 279 \n",
       "L 185.58153 279 \n",
       "L 186.702012 279 \n",
       "L 187.822494 279 \n",
       "L 188.942976 279 \n",
       "L 190.063458 279 \n",
       "L 191.18394 279 \n",
       "L 192.304422 279 \n",
       "L 193.424904 279 \n",
       "L 194.545386 279 \n",
       "L 195.665868 279 \n",
       "L 196.78635 279 \n",
       "L 197.906832 279 \n",
       "L 199.027314 279 \n",
       "L 200.147796 279 \n",
       "L 201.268277 279 \n",
       "L 202.388759 279 \n",
       "L 203.509241 279 \n",
       "L 204.629723 279 \n",
       "L 205.750205 279 \n",
       "L 206.870687 279 \n",
       "L 207.991169 279 \n",
       "L 209.111651 279 \n",
       "L 210.232133 279 \n",
       "L 211.352615 279 \n",
       "L 212.473097 279 \n",
       "L 213.593579 279 \n",
       "L 214.714061 279 \n",
       "L 215.834543 279 \n",
       "L 216.955024 279 \n",
       "L 218.075506 279 \n",
       "L 219.195988 279 \n",
       "L 220.31647 279 \n",
       "L 221.436952 279 \n",
       "L 222.557434 279 \n",
       "L 223.677916 279 \n",
       "L 224.798398 279 \n",
       "L 225.91888 279 \n",
       "L 227.039362 279 \n",
       "L 228.159844 279 \n",
       "L 229.280326 279 \n",
       "L 230.400808 279 \n",
       "L 231.52129 279 \n",
       "L 232.641771 279 \n",
       "L 233.762253 279 \n",
       "L 234.882735 279 \n",
       "L 236.003217 279 \n",
       "L 237.123699 279 \n",
       "L 238.244181 279 \n",
       "L 239.364663 279 \n",
       "L 240.485145 279 \n",
       "L 241.605627 279 \n",
       "L 242.726109 279 \n",
       "L 243.846591 279 \n",
       "L 244.967073 279 \n",
       "L 246.087555 279 \n",
       "L 247.208037 279 \n",
       "L 248.328518 279 \n",
       "L 249.449 279 \n",
       "L 250.569482 279 \n",
       "L 251.689964 279 \n",
       "L 252.810446 279 \n",
       "L 253.930928 279 \n",
       "L 255.05141 279 \n",
       "L 256.171892 279 \n",
       "L 257.292374 279 \n",
       "L 258.412856 279 \n",
       "L 259.533338 279 \n",
       "L 260.65382 279 \n",
       "L 261.774302 279 \n",
       "L 262.894784 279 \n",
       "L 264.015265 279 \n",
       "L 265.135747 279 \n",
       "L 266.256229 279 \n",
       "L 267.376711 279 \n",
       "L 268.497193 279 \n",
       "L 269.617675 279 \n",
       "L 270.738157 279 \n",
       "L 271.858639 279 \n",
       "L 272.979121 279 \n",
       "L 274.099603 279 \n",
       "L 275.220085 279 \n",
       "L 276.340567 279 \n",
       "L 277.461049 279 \n",
       "L 278.58153 279 \n",
       "L 279.702012 279 \n",
       "L 280.822494 279 \n",
       "L 281.942976 279 \n",
       "L 283.063458 279 \n",
       "L 284.18394 279 \n",
       "L 285.304422 279 \n",
       "L 286.424904 279 \n",
       "L 287.545386 279 \n",
       "L 288.665868 279 \n",
       "L 289.78635 279 \n",
       "L 290.906832 279 \n",
       "L 292.027314 279 \n",
       "L 293.147796 279 \n",
       "L 294.268277 279 \n",
       "L 295.388759 279 \n",
       "L 296.509241 279 \n",
       "L 297.629723 279 \n",
       "L 298.750205 279 \n",
       "L 299.870687 279 \n",
       "L 300.991169 279 \n",
       "L 302.111651 279 \n",
       "L 303.232133 279 \n",
       "L 304.352615 279 \n",
       "L 305.473097 279 \n",
       "L 306.593579 279 \n",
       "L 307.714061 279 \n",
       "L 308.834543 279 \n",
       "L 309.955024 279 \n",
       "L 311.075506 279 \n",
       "L 312.195988 279 \n",
       "L 313.31647 279 \n",
       "L 314.436952 279 \n",
       "L 315.557434 279 \n",
       "L 316.677916 279 \n",
       "L 317.798398 279 \n",
       "L 318.91888 279 \n",
       "L 320.039362 279 \n",
       "L 321.159844 279 \n",
       "L 321.159844 277.908434 \n",
       "L 321.159844 276.816867 \n",
       "L 321.159844 275.725301 \n",
       "L 321.159844 274.633735 \n",
       "L 321.159844 273.542169 \n",
       "L 321.159844 272.450602 \n",
       "L 321.159844 271.359036 \n",
       "L 321.159844 270.26747 \n",
       "L 321.159844 269.175904 \n",
       "L 321.159844 268.084337 \n",
       "L 321.159844 266.992771 \n",
       "L 321.159844 265.901205 \n",
       "L 321.159844 264.809639 \n",
       "L 321.159844 263.718072 \n",
       "L 321.159844 262.626506 \n",
       "L 321.159844 261.53494 \n",
       "L 321.159844 260.443373 \n",
       "L 321.159844 259.351807 \n",
       "L 321.159844 258.260241 \n",
       "L 321.159844 257.168675 \n",
       "L 321.159844 256.077108 \n",
       "L 321.159844 254.985542 \n",
       "L 321.159844 253.893976 \n",
       "L 321.159844 252.80241 \n",
       "L 321.159844 251.710843 \n",
       "L 321.159844 250.619277 \n",
       "L 321.159844 249.527711 \n",
       "L 321.159844 248.436145 \n",
       "L 321.159844 247.344578 \n",
       "L 321.159844 246.253012 \n",
       "L 321.159844 245.161446 \n",
       "L 321.159844 244.06988 \n",
       "L 321.159844 242.978313 \n",
       "L 321.159844 241.886747 \n",
       "L 321.159844 240.795181 \n",
       "L 321.159844 239.703614 \n",
       "L 321.159844 238.612048 \n",
       "L 321.159844 237.520482 \n",
       "L 321.159844 236.428916 \n",
       "L 321.159844 235.337349 \n",
       "L 321.159844 234.245783 \n",
       "L 321.159844 233.154217 \n",
       "L 321.159844 232.062651 \n",
       "L 321.159844 230.971084 \n",
       "L 321.159844 229.879518 \n",
       "L 321.159844 228.787952 \n",
       "L 321.159844 227.696386 \n",
       "L 321.159844 226.604819 \n",
       "L 321.159844 225.513253 \n",
       "L 321.159844 224.421687 \n",
       "L 321.159844 223.33012 \n",
       "L 321.159844 222.238554 \n",
       "L 321.159844 221.146988 \n",
       "L 321.159844 220.055422 \n",
       "L 321.159844 218.963855 \n",
       "L 321.159844 217.872289 \n",
       "L 321.159844 216.780723 \n",
       "L 321.159844 215.689157 \n",
       "L 321.159844 214.59759 \n",
       "L 321.159844 213.506024 \n",
       "L 321.159844 212.414458 \n",
       "L 321.159844 211.322892 \n",
       "L 321.159844 210.231325 \n",
       "L 321.159844 209.139759 \n",
       "L 321.159844 208.048193 \n",
       "L 321.159844 206.956627 \n",
       "L 321.159844 205.86506 \n",
       "L 321.159844 204.773494 \n",
       "L 321.159844 203.681928 \n",
       "L 321.159844 202.590361 \n",
       "L 321.159844 201.498795 \n",
       "L 321.159844 200.407229 \n",
       "L 321.159844 199.315663 \n",
       "L 321.159844 198.224096 \n",
       "L 321.159844 197.13253 \n",
       "L 321.159844 196.040964 \n",
       "L 321.159844 195.904518 \n",
       "L 320.039362 195.904518 \n",
       "L 318.91888 195.904518 \n",
       "L 317.798398 195.904518 \n",
       "L 316.677916 195.904518 \n",
       "L 315.557434 195.904518 \n",
       "L 314.436952 195.904518 \n",
       "L 313.31647 195.904518 \n",
       "L 312.195988 195.904518 \n",
       "L 311.075506 195.904518 \n",
       "L 309.955024 195.904518 \n",
       "L 308.834543 195.904518 \n",
       "L 307.714061 195.904518 \n",
       "L 306.593579 195.904518 \n",
       "L 305.473097 195.904518 \n",
       "L 304.352615 195.904518 \n",
       "L 303.232133 195.904518 \n",
       "L 302.111651 195.904518 \n",
       "L 300.991169 195.904518 \n",
       "L 299.870687 195.904518 \n",
       "L 298.750205 195.904518 \n",
       "L 297.629723 195.904518 \n",
       "L 296.509241 195.904518 \n",
       "L 295.388759 195.904518 \n",
       "L 294.268277 195.904518 \n",
       "L 293.147796 195.904518 \n",
       "L 292.027314 195.904518 \n",
       "L 290.906832 195.904518 \n",
       "L 289.78635 195.904518 \n",
       "L 288.665868 195.904518 \n",
       "L 287.545386 195.904518 \n",
       "L 286.424904 195.904518 \n",
       "L 285.304422 195.904518 \n",
       "L 284.18394 195.904518 \n",
       "L 283.063458 195.904518 \n",
       "L 281.942976 195.904518 \n",
       "L 280.822494 195.904518 \n",
       "L 279.702012 195.904518 \n",
       "L 278.58153 195.904518 \n",
       "L 277.461049 195.904518 \n",
       "L 276.340567 195.904518 \n",
       "L 275.220085 195.904518 \n",
       "L 274.099603 195.904518 \n",
       "L 272.979121 195.904518 \n",
       "L 271.858639 195.904518 \n",
       "L 270.738157 195.904518 \n",
       "L 269.617675 195.904518 \n",
       "L 268.497193 195.904518 \n",
       "L 267.376711 195.904518 \n",
       "L 266.256229 195.904518 \n",
       "L 265.135747 195.904518 \n",
       "L 264.015265 195.904518 \n",
       "L 262.894784 195.904518 \n",
       "L 261.774302 195.904518 \n",
       "L 260.65382 195.904518 \n",
       "L 259.533338 195.904518 \n",
       "L 258.412856 195.904518 \n",
       "L 257.292374 195.904518 \n",
       "L 256.171892 195.904518 \n",
       "L 255.05141 195.904518 \n",
       "L 253.930928 195.904518 \n",
       "L 252.810446 195.904518 \n",
       "L 251.689964 195.904518 \n",
       "L 250.569482 195.904518 \n",
       "L 249.449 195.904518 \n",
       "L 248.328518 195.904518 \n",
       "L 247.208037 195.904518 \n",
       "L 246.087555 195.904518 \n",
       "L 244.967073 195.904518 \n",
       "L 243.846591 195.904518 \n",
       "L 242.726109 195.904518 \n",
       "L 241.605627 195.904518 \n",
       "L 240.485145 195.904518 \n",
       "L 239.364663 195.904518 \n",
       "L 238.244181 195.904518 \n",
       "L 237.123699 195.904518 \n",
       "L 236.003217 195.904518 \n",
       "L 234.882735 195.904518 \n",
       "L 233.762253 195.904518 \n",
       "L 232.641771 195.904518 \n",
       "L 231.52129 195.904518 \n",
       "L 230.400808 195.904518 \n",
       "L 229.280326 195.904518 \n",
       "L 228.159844 195.904518 \n",
       "L 227.039362 195.904518 \n",
       "L 225.91888 195.904518 \n",
       "L 224.798398 195.768072 \n",
       "L 223.677916 195.768072 \n",
       "L 222.557434 195.768072 \n",
       "L 221.436952 195.768072 \n",
       "L 220.31647 195.768072 \n",
       "L 219.195988 195.768072 \n",
       "L 218.075506 195.768072 \n",
       "L 216.955024 195.768072 \n",
       "L 215.834543 195.768072 \n",
       "L 214.714061 195.768072 \n",
       "L 213.593579 195.768072 \n",
       "L 212.473097 195.768072 \n",
       "L 211.352615 195.768072 \n",
       "L 210.232133 195.768072 \n",
       "L 209.111651 195.768072 \n",
       "L 207.991169 195.768072 \n",
       "L 206.870687 195.768072 \n",
       "L 205.750205 195.768072 \n",
       "L 204.629723 195.768072 \n",
       "L 203.509241 195.768072 \n",
       "L 202.388759 195.768072 \n",
       "L 201.268277 195.768072 \n",
       "L 200.147796 195.768072 \n",
       "L 199.027314 195.768072 \n",
       "L 197.906832 195.768072 \n",
       "L 196.78635 195.768072 \n",
       "L 195.665868 195.768072 \n",
       "L 194.545386 195.768072 \n",
       "L 193.424904 195.768072 \n",
       "L 192.304422 195.768072 \n",
       "L 191.18394 195.768072 \n",
       "L 190.063458 195.768072 \n",
       "L 188.942976 195.768072 \n",
       "L 187.822494 195.768072 \n",
       "L 186.702012 195.768072 \n",
       "L 185.58153 195.768072 \n",
       "L 184.461049 195.768072 \n",
       "L 183.340567 195.768072 \n",
       "L 182.220085 195.768072 \n",
       "L 181.099603 195.768072 \n",
       "L 179.979121 195.768072 \n",
       "L 178.858639 195.768072 \n",
       "L 177.738157 195.768072 \n",
       "L 176.617675 195.768072 \n",
       "L 175.497193 195.768072 \n",
       "L 174.376711 195.768072 \n",
       "L 173.256229 195.768072 \n",
       "L 172.135747 195.768072 \n",
       "L 171.015265 195.768072 \n",
       "L 169.894784 195.768072 \n",
       "L 168.774302 195.768072 \n",
       "L 167.65382 195.768072 \n",
       "L 166.533338 195.768072 \n",
       "L 165.412856 195.768072 \n",
       "L 164.292374 195.768072 \n",
       "L 163.171892 195.768072 \n",
       "L 162.05141 195.768072 \n",
       "L 160.930928 195.768072 \n",
       "L 159.810446 195.768072 \n",
       "L 158.689964 195.768072 \n",
       "L 157.569482 195.768072 \n",
       "L 156.449 195.768072 \n",
       "L 155.328518 195.768072 \n",
       "L 154.208037 195.768072 \n",
       "L 153.087555 195.768072 \n",
       "L 151.967073 195.768072 \n",
       "L 150.846591 195.768072 \n",
       "L 149.726109 195.768072 \n",
       "L 148.605627 195.768072 \n",
       "L 147.485145 195.768072 \n",
       "L 146.364663 195.768072 \n",
       "L 145.244181 195.768072 \n",
       "L 144.123699 195.768072 \n",
       "L 143.003217 195.768072 \n",
       "L 141.882735 195.768072 \n",
       "L 140.762253 195.768072 \n",
       "L 139.641771 195.768072 \n",
       "L 138.52129 195.768072 \n",
       "L 137.400808 195.768072 \n",
       "L 136.280326 195.768072 \n",
       "L 135.159844 195.768072 \n",
       "L 134.039362 195.768072 \n",
       "L 132.91888 195.768072 \n",
       "L 131.798398 195.768072 \n",
       "L 130.677916 195.768072 \n",
       "L 129.557434 195.768072 \n",
       "L 128.436952 195.768072 \n",
       "L 127.31647 195.768072 \n",
       "L 126.195988 195.768072 \n",
       "L 125.075506 195.768072 \n",
       "L 123.955024 195.768072 \n",
       "L 122.834543 195.768072 \n",
       "L 121.714061 195.768072 \n",
       "L 120.593579 195.768072 \n",
       "L 119.473097 195.768072 \n",
       "L 118.352615 195.768072 \n",
       "L 117.232133 195.768072 \n",
       "L 116.111651 195.768072 \n",
       "L 114.991169 195.768072 \n",
       "L 113.870687 195.768072 \n",
       "L 112.750205 195.768072 \n",
       "L 111.629723 195.768072 \n",
       "L 110.509241 195.768072 \n",
       "L 109.388759 195.768072 \n",
       "L 108.268277 195.768072 \n",
       "L 107.147796 195.768072 \n",
       "L 106.027314 195.768072 \n",
       "L 104.906832 195.768072 \n",
       "L 103.78635 195.768072 \n",
       "L 102.665868 195.768072 \n",
       "L 101.545386 195.768072 \n",
       "L 100.424904 195.768072 \n",
       "L 99.304422 195.768072 \n",
       "L 98.18394 195.768072 \n",
       "L 97.063458 195.768072 \n",
       "L 95.942976 195.768072 \n",
       "L 94.822494 195.768072 \n",
       "L 93.702012 195.768072 \n",
       "L 92.58153 195.768072 \n",
       "L 91.461049 195.768072 \n",
       "L 90.340567 195.768072 \n",
       "L 89.220085 195.768072 \n",
       "L 88.099603 195.768072 \n",
       "L 86.979121 195.768072 \n",
       "L 85.858639 195.768072 \n",
       "L 84.738157 195.768072 \n",
       "L 83.617675 195.768072 \n",
       "L 82.497193 195.768072 \n",
       "L 81.376711 195.768072 \n",
       "L 80.256229 195.768072 \n",
       "L 79.135747 195.768072 \n",
       "L 78.015265 195.768072 \n",
       "L 76.894784 195.768072 \n",
       "L 75.774302 195.768072 \n",
       "L 74.65382 195.768072 \n",
       "L 73.533338 195.768072 \n",
       "L 72.412856 195.768072 \n",
       "L 71.292374 195.768072 \n",
       "L 70.171892 195.768072 \n",
       "L 69.05141 195.768072 \n",
       "L 67.930928 195.768072 \n",
       "L 66.810446 195.768072 \n",
       "L 65.689964 195.768072 \n",
       "L 64.569482 195.768072 \n",
       "L 63.449 195.768072 \n",
       "L 62.328518 195.768072 \n",
       "L 61.208037 195.768072 \n",
       "L 60.087555 195.768072 \n",
       "L 58.967073 195.768072 \n",
       "L 57.846591 195.768072 \n",
       "L 56.726109 195.768072 \n",
       "L 55.605627 195.768072 \n",
       "L 54.485145 195.768072 \n",
       "L 53.364663 195.768072 \n",
       "L 52.244181 195.768072 \n",
       "L 51.123699 195.768072 \n",
       "L 50.003217 195.768072 \n",
       "L 48.882735 195.768072 \n",
       "L 47.762253 195.768072 \n",
       "L 46.641771 195.768072 \n",
       "L 45.52129 195.768072 \n",
       "L 44.400808 195.768072 \n",
       "L 43.280326 195.768072 \n",
       "L 42.159844 195.768072 \n",
       "L 42.159844 196.040964 \n",
       "L 42.159844 197.13253 \n",
       "L 42.159844 198.224096 \n",
       "L 42.159844 199.315663 \n",
       "L 42.159844 200.407229 \n",
       "L 42.159844 201.498795 \n",
       "L 42.159844 202.590361 \n",
       "L 42.159844 203.681928 \n",
       "L 42.159844 204.773494 \n",
       "L 42.159844 205.86506 \n",
       "L 42.159844 206.956627 \n",
       "L 42.159844 208.048193 \n",
       "L 42.159844 209.139759 \n",
       "L 42.159844 210.231325 \n",
       "L 42.159844 211.322892 \n",
       "L 42.159844 212.414458 \n",
       "L 42.159844 213.506024 \n",
       "L 42.159844 214.59759 \n",
       "L 42.159844 215.689157 \n",
       "L 42.159844 216.780723 \n",
       "L 42.159844 217.872289 \n",
       "L 42.159844 218.963855 \n",
       "L 42.159844 220.055422 \n",
       "L 42.159844 221.146988 \n",
       "L 42.159844 222.238554 \n",
       "L 42.159844 223.33012 \n",
       "L 42.159844 224.421687 \n",
       "L 42.159844 225.513253 \n",
       "L 42.159844 226.604819 \n",
       "L 42.159844 227.696386 \n",
       "L 42.159844 228.787952 \n",
       "L 42.159844 229.879518 \n",
       "L 42.159844 230.971084 \n",
       "L 42.159844 232.062651 \n",
       "L 42.159844 233.154217 \n",
       "L 42.159844 234.245783 \n",
       "L 42.159844 235.337349 \n",
       "L 42.159844 236.428916 \n",
       "L 42.159844 237.520482 \n",
       "L 42.159844 238.612048 \n",
       "L 42.159844 239.703614 \n",
       "L 42.159844 240.795181 \n",
       "L 42.159844 241.886747 \n",
       "L 42.159844 242.978313 \n",
       "L 42.159844 244.06988 \n",
       "L 42.159844 245.161446 \n",
       "L 42.159844 246.253012 \n",
       "L 42.159844 247.344578 \n",
       "L 42.159844 248.436145 \n",
       "L 42.159844 249.527711 \n",
       "L 42.159844 250.619277 \n",
       "L 42.159844 251.710843 \n",
       "L 42.159844 252.80241 \n",
       "L 42.159844 253.893976 \n",
       "L 42.159844 254.985542 \n",
       "L 42.159844 256.077108 \n",
       "L 42.159844 257.168675 \n",
       "L 42.159844 258.260241 \n",
       "L 42.159844 259.351807 \n",
       "L 42.159844 260.443373 \n",
       "L 42.159844 261.53494 \n",
       "L 42.159844 262.626506 \n",
       "L 42.159844 263.718072 \n",
       "L 42.159844 264.809639 \n",
       "L 42.159844 265.901205 \n",
       "L 42.159844 266.992771 \n",
       "L 42.159844 268.084337 \n",
       "L 42.159844 269.175904 \n",
       "L 42.159844 270.26747 \n",
       "L 42.159844 271.359036 \n",
       "L 42.159844 272.450602 \n",
       "L 42.159844 273.542169 \n",
       "L 42.159844 274.633735 \n",
       "L 42.159844 275.725301 \n",
       "L 42.159844 276.816867 \n",
       "L 42.159844 277.908434 \n",
       "L 42.159844 279 \n",
       "z\n",
       "\" style=\"fill:#1f77b4;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_2\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 43.280326 195.768072 \n",
       "L 44.400808 195.768072 \n",
       "L 45.52129 195.768072 \n",
       "L 46.641771 195.768072 \n",
       "L 47.762253 195.768072 \n",
       "L 48.882735 195.768072 \n",
       "L 50.003217 195.768072 \n",
       "L 51.123699 195.768072 \n",
       "L 52.244181 195.768072 \n",
       "L 53.364663 195.768072 \n",
       "L 54.485145 195.768072 \n",
       "L 55.605627 195.768072 \n",
       "L 56.726109 195.768072 \n",
       "L 57.846591 195.768072 \n",
       "L 58.967073 195.768072 \n",
       "L 60.087555 195.768072 \n",
       "L 61.208037 195.768072 \n",
       "L 62.328518 195.768072 \n",
       "L 63.449 195.768072 \n",
       "L 64.569482 195.768072 \n",
       "L 65.689964 195.768072 \n",
       "L 66.810446 195.768072 \n",
       "L 67.930928 195.768072 \n",
       "L 69.05141 195.768072 \n",
       "L 70.171892 195.768072 \n",
       "L 71.292374 195.768072 \n",
       "L 72.412856 195.768072 \n",
       "L 73.533338 195.768072 \n",
       "L 74.65382 195.768072 \n",
       "L 75.774302 195.768072 \n",
       "L 76.894784 195.768072 \n",
       "L 78.015265 195.768072 \n",
       "L 79.135747 195.768072 \n",
       "L 80.256229 195.768072 \n",
       "L 81.376711 195.768072 \n",
       "L 82.497193 195.768072 \n",
       "L 83.617675 195.768072 \n",
       "L 84.738157 195.768072 \n",
       "L 85.858639 195.768072 \n",
       "L 86.979121 195.768072 \n",
       "L 88.099603 195.768072 \n",
       "L 89.220085 195.768072 \n",
       "L 90.340567 195.768072 \n",
       "L 91.461049 195.768072 \n",
       "L 92.58153 195.768072 \n",
       "L 93.702012 195.768072 \n",
       "L 94.822494 195.768072 \n",
       "L 95.942976 195.768072 \n",
       "L 97.063458 195.768072 \n",
       "L 98.18394 195.768072 \n",
       "L 99.304422 195.768072 \n",
       "L 100.424904 195.768072 \n",
       "L 101.545386 195.768072 \n",
       "L 102.665868 195.768072 \n",
       "L 103.78635 195.768072 \n",
       "L 104.906832 195.768072 \n",
       "L 106.027314 195.768072 \n",
       "L 107.147796 195.768072 \n",
       "L 108.268277 195.768072 \n",
       "L 109.388759 195.768072 \n",
       "L 110.509241 195.768072 \n",
       "L 111.629723 195.768072 \n",
       "L 112.750205 195.768072 \n",
       "L 113.870687 195.768072 \n",
       "L 114.991169 195.768072 \n",
       "L 116.111651 195.768072 \n",
       "L 117.232133 195.768072 \n",
       "L 118.352615 195.768072 \n",
       "L 119.473097 195.768072 \n",
       "L 120.593579 195.768072 \n",
       "L 121.714061 195.768072 \n",
       "L 122.834543 195.768072 \n",
       "L 123.955024 195.768072 \n",
       "L 125.075506 195.768072 \n",
       "L 126.195988 195.768072 \n",
       "L 127.31647 195.768072 \n",
       "L 128.436952 195.768072 \n",
       "L 129.557434 195.768072 \n",
       "L 130.677916 195.768072 \n",
       "L 131.798398 195.768072 \n",
       "L 132.91888 195.768072 \n",
       "L 134.039362 195.768072 \n",
       "L 135.159844 195.768072 \n",
       "L 136.280326 195.768072 \n",
       "L 137.400808 195.768072 \n",
       "L 138.52129 195.768072 \n",
       "L 139.641771 195.768072 \n",
       "L 140.762253 195.768072 \n",
       "L 141.882735 195.768072 \n",
       "L 143.003217 195.768072 \n",
       "L 144.123699 195.768072 \n",
       "L 145.244181 195.768072 \n",
       "L 146.364663 195.768072 \n",
       "L 147.485145 195.768072 \n",
       "L 148.605627 195.768072 \n",
       "L 149.726109 195.768072 \n",
       "L 150.846591 195.768072 \n",
       "L 151.967073 195.768072 \n",
       "L 153.087555 195.768072 \n",
       "L 154.208037 195.768072 \n",
       "L 155.328518 195.768072 \n",
       "L 156.449 195.768072 \n",
       "L 157.569482 195.768072 \n",
       "L 158.689964 195.768072 \n",
       "L 159.810446 195.768072 \n",
       "L 160.930928 195.768072 \n",
       "L 162.05141 195.768072 \n",
       "L 163.171892 195.768072 \n",
       "L 164.292374 195.768072 \n",
       "L 165.412856 195.768072 \n",
       "L 166.533338 195.768072 \n",
       "L 167.65382 195.768072 \n",
       "L 168.774302 195.768072 \n",
       "L 169.894784 195.768072 \n",
       "L 171.015265 195.768072 \n",
       "L 172.135747 195.768072 \n",
       "L 173.256229 195.768072 \n",
       "L 174.376711 195.768072 \n",
       "L 175.497193 195.768072 \n",
       "L 176.617675 195.768072 \n",
       "L 177.738157 195.768072 \n",
       "L 178.858639 195.768072 \n",
       "L 179.979121 195.768072 \n",
       "L 181.099603 195.768072 \n",
       "L 182.220085 195.768072 \n",
       "L 183.340567 195.768072 \n",
       "L 184.461049 195.768072 \n",
       "L 185.58153 195.768072 \n",
       "L 186.702012 195.768072 \n",
       "L 187.822494 195.768072 \n",
       "L 188.942976 195.768072 \n",
       "L 190.063458 195.768072 \n",
       "L 191.18394 195.768072 \n",
       "L 192.304422 195.768072 \n",
       "L 193.424904 195.768072 \n",
       "L 194.545386 195.768072 \n",
       "L 195.665868 195.768072 \n",
       "L 196.78635 195.768072 \n",
       "L 197.906832 195.768072 \n",
       "L 199.027314 195.768072 \n",
       "L 200.147796 195.768072 \n",
       "L 201.268277 195.768072 \n",
       "L 202.388759 195.768072 \n",
       "L 203.509241 195.768072 \n",
       "L 204.629723 195.768072 \n",
       "L 205.750205 195.768072 \n",
       "L 206.870687 195.768072 \n",
       "L 207.991169 195.768072 \n",
       "L 209.111651 195.768072 \n",
       "L 210.232133 195.768072 \n",
       "L 211.352615 195.768072 \n",
       "L 212.473097 195.768072 \n",
       "L 213.593579 195.768072 \n",
       "L 214.714061 195.768072 \n",
       "L 215.834543 195.768072 \n",
       "L 216.955024 195.768072 \n",
       "L 218.075506 195.768072 \n",
       "L 219.195988 195.768072 \n",
       "L 220.31647 195.768072 \n",
       "L 221.436952 195.768072 \n",
       "L 222.557434 195.768072 \n",
       "L 223.677916 195.768072 \n",
       "L 224.798398 195.768072 \n",
       "L 225.91888 195.904518 \n",
       "L 227.039362 195.904518 \n",
       "L 228.159844 195.904518 \n",
       "L 229.280326 195.904518 \n",
       "L 230.400808 195.904518 \n",
       "L 231.52129 195.904518 \n",
       "L 232.641771 195.904518 \n",
       "L 233.762253 195.904518 \n",
       "L 234.882735 195.904518 \n",
       "L 236.003217 195.904518 \n",
       "L 237.123699 195.904518 \n",
       "L 238.244181 195.904518 \n",
       "L 239.364663 195.904518 \n",
       "L 240.485145 195.904518 \n",
       "L 241.605627 195.904518 \n",
       "L 242.726109 195.904518 \n",
       "L 243.846591 195.904518 \n",
       "L 244.967073 195.904518 \n",
       "L 246.087555 195.904518 \n",
       "L 247.208037 195.904518 \n",
       "L 248.328518 195.904518 \n",
       "L 249.449 195.904518 \n",
       "L 250.569482 195.904518 \n",
       "L 251.689964 195.904518 \n",
       "L 252.810446 195.904518 \n",
       "L 253.930928 195.904518 \n",
       "L 255.05141 195.904518 \n",
       "L 256.171892 195.904518 \n",
       "L 257.292374 195.904518 \n",
       "L 258.412856 195.904518 \n",
       "L 259.533338 195.904518 \n",
       "L 260.65382 195.904518 \n",
       "L 261.774302 195.904518 \n",
       "L 262.894784 195.904518 \n",
       "L 264.015265 195.904518 \n",
       "L 265.135747 195.904518 \n",
       "L 266.256229 195.904518 \n",
       "L 267.376711 195.904518 \n",
       "L 268.497193 195.904518 \n",
       "L 269.617675 195.904518 \n",
       "L 270.738157 195.904518 \n",
       "L 271.858639 195.904518 \n",
       "L 272.979121 195.904518 \n",
       "L 274.099603 195.904518 \n",
       "L 275.220085 195.904518 \n",
       "L 276.340567 195.904518 \n",
       "L 277.461049 195.904518 \n",
       "L 278.58153 195.904518 \n",
       "L 279.702012 195.904518 \n",
       "L 280.822494 195.904518 \n",
       "L 281.942976 195.904518 \n",
       "L 283.063458 195.904518 \n",
       "L 284.18394 195.904518 \n",
       "L 285.304422 195.904518 \n",
       "L 286.424904 195.904518 \n",
       "L 287.545386 195.904518 \n",
       "L 288.665868 195.904518 \n",
       "L 289.78635 195.904518 \n",
       "L 290.906832 195.904518 \n",
       "L 292.027314 195.904518 \n",
       "L 293.147796 195.904518 \n",
       "L 294.268277 195.904518 \n",
       "L 295.388759 195.904518 \n",
       "L 296.509241 195.904518 \n",
       "L 297.629723 195.904518 \n",
       "L 298.750205 195.904518 \n",
       "L 299.870687 195.904518 \n",
       "L 300.991169 195.904518 \n",
       "L 302.111651 195.904518 \n",
       "L 303.232133 195.904518 \n",
       "L 304.352615 195.904518 \n",
       "L 305.473097 195.904518 \n",
       "L 306.593579 195.904518 \n",
       "L 307.714061 195.904518 \n",
       "L 308.834543 195.904518 \n",
       "L 309.955024 195.904518 \n",
       "L 311.075506 195.904518 \n",
       "L 312.195988 195.904518 \n",
       "L 313.31647 195.904518 \n",
       "L 314.436952 195.904518 \n",
       "L 315.557434 195.904518 \n",
       "L 316.677916 195.904518 \n",
       "L 317.798398 195.904518 \n",
       "L 318.91888 195.904518 \n",
       "L 320.039362 195.904518 \n",
       "L 321.159844 195.904518 \n",
       "L 321.159844 195.768072 \n",
       "L 320.039362 195.768072 \n",
       "L 318.91888 195.768072 \n",
       "L 317.798398 195.768072 \n",
       "L 316.677916 195.768072 \n",
       "L 315.557434 195.768072 \n",
       "L 314.436952 195.768072 \n",
       "L 313.31647 195.768072 \n",
       "L 312.195988 195.768072 \n",
       "L 311.075506 195.768072 \n",
       "L 309.955024 195.768072 \n",
       "L 308.834543 195.768072 \n",
       "L 307.714061 195.768072 \n",
       "L 306.593579 195.768072 \n",
       "L 305.473097 195.768072 \n",
       "L 304.352615 195.768072 \n",
       "L 303.232133 195.768072 \n",
       "L 302.111651 195.768072 \n",
       "L 300.991169 195.768072 \n",
       "L 299.870687 195.768072 \n",
       "L 298.750205 195.768072 \n",
       "L 297.629723 195.768072 \n",
       "L 296.509241 195.768072 \n",
       "L 295.388759 195.768072 \n",
       "L 294.268277 195.768072 \n",
       "L 293.147796 195.768072 \n",
       "L 292.027314 195.768072 \n",
       "L 290.906832 195.768072 \n",
       "L 289.78635 195.768072 \n",
       "L 288.665868 195.768072 \n",
       "L 287.545386 195.768072 \n",
       "L 286.424904 195.768072 \n",
       "L 285.304422 195.768072 \n",
       "L 284.18394 195.768072 \n",
       "L 283.063458 195.768072 \n",
       "L 281.942976 195.768072 \n",
       "L 280.822494 195.768072 \n",
       "L 279.702012 195.768072 \n",
       "L 278.58153 195.768072 \n",
       "L 277.461049 195.768072 \n",
       "L 276.340567 195.768072 \n",
       "L 275.220085 195.768072 \n",
       "L 274.099603 195.768072 \n",
       "L 272.979121 195.768072 \n",
       "L 271.858639 195.768072 \n",
       "L 270.738157 195.768072 \n",
       "L 269.617675 195.768072 \n",
       "L 268.497193 195.768072 \n",
       "L 267.376711 195.768072 \n",
       "L 266.256229 195.768072 \n",
       "L 265.135747 195.768072 \n",
       "L 264.015265 195.768072 \n",
       "L 262.894784 195.768072 \n",
       "L 261.774302 195.768072 \n",
       "L 260.65382 195.768072 \n",
       "L 259.533338 195.768072 \n",
       "L 258.412856 195.768072 \n",
       "L 257.292374 195.768072 \n",
       "L 256.171892 195.768072 \n",
       "L 255.05141 195.768072 \n",
       "L 253.930928 195.768072 \n",
       "L 252.810446 195.768072 \n",
       "L 251.689964 195.768072 \n",
       "L 250.569482 195.768072 \n",
       "L 249.449 195.768072 \n",
       "L 248.328518 195.768072 \n",
       "L 247.208037 195.768072 \n",
       "L 246.087555 195.768072 \n",
       "L 244.967073 195.768072 \n",
       "L 243.846591 195.768072 \n",
       "L 242.726109 195.768072 \n",
       "L 241.605627 195.768072 \n",
       "L 240.485145 195.768072 \n",
       "L 239.364663 195.768072 \n",
       "L 238.244181 195.768072 \n",
       "L 237.123699 195.768072 \n",
       "L 236.003217 195.768072 \n",
       "L 234.882735 195.768072 \n",
       "L 233.762253 195.768072 \n",
       "L 232.641771 195.768072 \n",
       "L 231.52129 195.768072 \n",
       "L 230.400808 195.768072 \n",
       "L 229.280326 195.768072 \n",
       "L 228.159844 195.768072 \n",
       "L 227.039362 195.768072 \n",
       "L 225.91888 195.768072 \n",
       "L 224.798398 195.495181 \n",
       "L 223.677916 195.495181 \n",
       "L 222.557434 195.495181 \n",
       "L 221.436952 195.495181 \n",
       "L 220.31647 195.495181 \n",
       "L 219.195988 195.495181 \n",
       "L 218.075506 195.495181 \n",
       "L 216.955024 195.495181 \n",
       "L 215.834543 195.495181 \n",
       "L 214.714061 195.495181 \n",
       "L 213.593579 195.495181 \n",
       "L 212.473097 195.495181 \n",
       "L 211.352615 195.495181 \n",
       "L 210.232133 195.495181 \n",
       "L 209.111651 195.495181 \n",
       "L 207.991169 195.495181 \n",
       "L 206.870687 195.495181 \n",
       "L 205.750205 195.495181 \n",
       "L 204.629723 195.495181 \n",
       "L 203.509241 195.495181 \n",
       "L 202.388759 195.495181 \n",
       "L 201.268277 195.495181 \n",
       "L 200.147796 195.495181 \n",
       "L 199.027314 195.495181 \n",
       "L 197.906832 195.495181 \n",
       "L 196.78635 195.495181 \n",
       "L 195.665868 195.495181 \n",
       "L 194.545386 195.495181 \n",
       "L 193.424904 195.495181 \n",
       "L 192.304422 195.495181 \n",
       "L 191.18394 195.495181 \n",
       "L 190.063458 195.495181 \n",
       "L 188.942976 195.495181 \n",
       "L 187.822494 195.495181 \n",
       "L 186.702012 195.495181 \n",
       "L 185.58153 195.495181 \n",
       "L 184.461049 195.495181 \n",
       "L 183.340567 195.495181 \n",
       "L 182.220085 195.495181 \n",
       "L 181.099603 195.495181 \n",
       "L 179.979121 195.495181 \n",
       "L 178.858639 195.495181 \n",
       "L 177.738157 195.495181 \n",
       "L 176.617675 195.495181 \n",
       "L 175.497193 195.495181 \n",
       "L 174.376711 195.495181 \n",
       "L 173.256229 195.495181 \n",
       "L 172.135747 195.495181 \n",
       "L 171.015265 195.495181 \n",
       "L 169.894784 195.495181 \n",
       "L 168.774302 195.495181 \n",
       "L 167.65382 195.495181 \n",
       "L 166.533338 195.495181 \n",
       "L 165.412856 195.495181 \n",
       "L 164.292374 195.495181 \n",
       "L 163.171892 195.495181 \n",
       "L 162.05141 195.495181 \n",
       "L 160.930928 195.495181 \n",
       "L 159.810446 195.495181 \n",
       "L 158.689964 195.495181 \n",
       "L 157.569482 195.495181 \n",
       "L 156.449 195.495181 \n",
       "L 155.328518 195.495181 \n",
       "L 154.208037 195.495181 \n",
       "L 153.087555 195.495181 \n",
       "L 151.967073 195.495181 \n",
       "L 150.846591 195.495181 \n",
       "L 149.726109 195.495181 \n",
       "L 148.605627 195.495181 \n",
       "L 147.485145 195.495181 \n",
       "L 146.364663 195.495181 \n",
       "L 145.244181 195.495181 \n",
       "L 144.123699 195.495181 \n",
       "L 143.003217 195.495181 \n",
       "L 141.882735 195.495181 \n",
       "L 140.762253 195.495181 \n",
       "L 139.641771 195.495181 \n",
       "L 138.52129 195.495181 \n",
       "L 137.400808 195.495181 \n",
       "L 136.280326 195.495181 \n",
       "L 135.159844 195.495181 \n",
       "L 134.039362 195.495181 \n",
       "L 132.91888 195.495181 \n",
       "L 131.798398 195.495181 \n",
       "L 130.677916 195.495181 \n",
       "L 129.557434 195.495181 \n",
       "L 128.436952 195.495181 \n",
       "L 127.31647 195.495181 \n",
       "L 126.195988 195.495181 \n",
       "L 125.075506 195.495181 \n",
       "L 123.955024 195.495181 \n",
       "L 122.834543 195.495181 \n",
       "L 121.714061 195.495181 \n",
       "L 120.593579 195.495181 \n",
       "L 119.473097 195.495181 \n",
       "L 118.352615 195.495181 \n",
       "L 117.232133 195.495181 \n",
       "L 116.111651 195.495181 \n",
       "L 114.991169 195.495181 \n",
       "L 113.870687 195.495181 \n",
       "L 112.750205 195.495181 \n",
       "L 111.629723 195.495181 \n",
       "L 110.509241 195.495181 \n",
       "L 109.388759 195.495181 \n",
       "L 108.268277 195.495181 \n",
       "L 107.147796 195.495181 \n",
       "L 106.027314 195.495181 \n",
       "L 104.906832 195.495181 \n",
       "L 103.78635 195.495181 \n",
       "L 102.665868 195.495181 \n",
       "L 101.545386 195.495181 \n",
       "L 100.424904 195.495181 \n",
       "L 99.304422 195.495181 \n",
       "L 98.18394 195.495181 \n",
       "L 97.063458 195.495181 \n",
       "L 95.942976 195.495181 \n",
       "L 94.822494 195.495181 \n",
       "L 93.702012 195.495181 \n",
       "L 92.58153 195.495181 \n",
       "L 91.461049 195.495181 \n",
       "L 90.340567 195.495181 \n",
       "L 89.220085 195.495181 \n",
       "L 88.099603 195.495181 \n",
       "L 86.979121 195.495181 \n",
       "L 85.858639 195.495181 \n",
       "L 84.738157 195.495181 \n",
       "L 83.617675 195.495181 \n",
       "L 82.497193 195.495181 \n",
       "L 81.376711 195.495181 \n",
       "L 80.256229 195.495181 \n",
       "L 79.135747 195.495181 \n",
       "L 78.015265 195.495181 \n",
       "L 76.894784 195.495181 \n",
       "L 75.774302 195.495181 \n",
       "L 74.65382 195.495181 \n",
       "L 73.533338 195.495181 \n",
       "L 72.412856 195.495181 \n",
       "L 71.292374 195.495181 \n",
       "L 70.171892 195.495181 \n",
       "L 69.05141 195.495181 \n",
       "L 67.930928 195.495181 \n",
       "L 66.810446 195.495181 \n",
       "L 65.689964 195.495181 \n",
       "L 64.569482 195.495181 \n",
       "L 63.449 195.495181 \n",
       "L 62.328518 195.495181 \n",
       "L 61.208037 195.495181 \n",
       "L 60.087555 195.495181 \n",
       "L 58.967073 195.495181 \n",
       "L 57.846591 195.495181 \n",
       "L 56.726109 195.495181 \n",
       "L 55.605627 195.495181 \n",
       "L 54.485145 195.495181 \n",
       "L 53.364663 195.495181 \n",
       "L 52.244181 195.495181 \n",
       "L 51.123699 195.495181 \n",
       "L 50.003217 195.495181 \n",
       "L 48.882735 195.495181 \n",
       "L 47.762253 195.495181 \n",
       "L 46.641771 195.495181 \n",
       "L 45.52129 195.495181 \n",
       "L 44.400808 195.495181 \n",
       "L 43.280326 195.495181 \n",
       "L 42.159844 195.495181 \n",
       "L 42.159844 195.768072 \n",
       "z\n",
       "\" style=\"fill:#ff7f0e;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_3\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 43.280326 195.495181 \n",
       "L 44.400808 195.495181 \n",
       "L 45.52129 195.495181 \n",
       "L 46.641771 195.495181 \n",
       "L 47.762253 195.495181 \n",
       "L 48.882735 195.495181 \n",
       "L 50.003217 195.495181 \n",
       "L 51.123699 195.495181 \n",
       "L 52.244181 195.495181 \n",
       "L 53.364663 195.495181 \n",
       "L 54.485145 195.495181 \n",
       "L 55.605627 195.495181 \n",
       "L 56.726109 195.495181 \n",
       "L 57.846591 195.495181 \n",
       "L 58.967073 195.495181 \n",
       "L 60.087555 195.495181 \n",
       "L 61.208037 195.495181 \n",
       "L 62.328518 195.495181 \n",
       "L 63.449 195.495181 \n",
       "L 64.569482 195.495181 \n",
       "L 65.689964 195.495181 \n",
       "L 66.810446 195.495181 \n",
       "L 67.930928 195.495181 \n",
       "L 69.05141 195.495181 \n",
       "L 70.171892 195.495181 \n",
       "L 71.292374 195.495181 \n",
       "L 72.412856 195.495181 \n",
       "L 73.533338 195.495181 \n",
       "L 74.65382 195.495181 \n",
       "L 75.774302 195.495181 \n",
       "L 76.894784 195.495181 \n",
       "L 78.015265 195.495181 \n",
       "L 79.135747 195.495181 \n",
       "L 80.256229 195.495181 \n",
       "L 81.376711 195.495181 \n",
       "L 82.497193 195.495181 \n",
       "L 83.617675 195.495181 \n",
       "L 84.738157 195.495181 \n",
       "L 85.858639 195.495181 \n",
       "L 86.979121 195.495181 \n",
       "L 88.099603 195.495181 \n",
       "L 89.220085 195.495181 \n",
       "L 90.340567 195.495181 \n",
       "L 91.461049 195.495181 \n",
       "L 92.58153 195.495181 \n",
       "L 93.702012 195.495181 \n",
       "L 94.822494 195.495181 \n",
       "L 95.942976 195.495181 \n",
       "L 97.063458 195.495181 \n",
       "L 98.18394 195.495181 \n",
       "L 99.304422 195.495181 \n",
       "L 100.424904 195.495181 \n",
       "L 101.545386 195.495181 \n",
       "L 102.665868 195.495181 \n",
       "L 103.78635 195.495181 \n",
       "L 104.906832 195.495181 \n",
       "L 106.027314 195.495181 \n",
       "L 107.147796 195.495181 \n",
       "L 108.268277 195.495181 \n",
       "L 109.388759 195.495181 \n",
       "L 110.509241 195.495181 \n",
       "L 111.629723 195.495181 \n",
       "L 112.750205 195.495181 \n",
       "L 113.870687 195.495181 \n",
       "L 114.991169 195.495181 \n",
       "L 116.111651 195.495181 \n",
       "L 117.232133 195.495181 \n",
       "L 118.352615 195.495181 \n",
       "L 119.473097 195.495181 \n",
       "L 120.593579 195.495181 \n",
       "L 121.714061 195.495181 \n",
       "L 122.834543 195.495181 \n",
       "L 123.955024 195.495181 \n",
       "L 125.075506 195.495181 \n",
       "L 126.195988 195.495181 \n",
       "L 127.31647 195.495181 \n",
       "L 128.436952 195.495181 \n",
       "L 129.557434 195.495181 \n",
       "L 130.677916 195.495181 \n",
       "L 131.798398 195.495181 \n",
       "L 132.91888 195.495181 \n",
       "L 134.039362 195.495181 \n",
       "L 135.159844 195.495181 \n",
       "L 136.280326 195.495181 \n",
       "L 137.400808 195.495181 \n",
       "L 138.52129 195.495181 \n",
       "L 139.641771 195.495181 \n",
       "L 140.762253 195.495181 \n",
       "L 141.882735 195.495181 \n",
       "L 143.003217 195.495181 \n",
       "L 144.123699 195.495181 \n",
       "L 145.244181 195.495181 \n",
       "L 146.364663 195.495181 \n",
       "L 147.485145 195.495181 \n",
       "L 148.605627 195.495181 \n",
       "L 149.726109 195.495181 \n",
       "L 150.846591 195.495181 \n",
       "L 151.967073 195.495181 \n",
       "L 153.087555 195.495181 \n",
       "L 154.208037 195.495181 \n",
       "L 155.328518 195.495181 \n",
       "L 156.449 195.495181 \n",
       "L 157.569482 195.495181 \n",
       "L 158.689964 195.495181 \n",
       "L 159.810446 195.495181 \n",
       "L 160.930928 195.495181 \n",
       "L 162.05141 195.495181 \n",
       "L 163.171892 195.495181 \n",
       "L 164.292374 195.495181 \n",
       "L 165.412856 195.495181 \n",
       "L 166.533338 195.495181 \n",
       "L 167.65382 195.495181 \n",
       "L 168.774302 195.495181 \n",
       "L 169.894784 195.495181 \n",
       "L 171.015265 195.495181 \n",
       "L 172.135747 195.495181 \n",
       "L 173.256229 195.495181 \n",
       "L 174.376711 195.495181 \n",
       "L 175.497193 195.495181 \n",
       "L 176.617675 195.495181 \n",
       "L 177.738157 195.495181 \n",
       "L 178.858639 195.495181 \n",
       "L 179.979121 195.495181 \n",
       "L 181.099603 195.495181 \n",
       "L 182.220085 195.495181 \n",
       "L 183.340567 195.495181 \n",
       "L 184.461049 195.495181 \n",
       "L 185.58153 195.495181 \n",
       "L 186.702012 195.495181 \n",
       "L 187.822494 195.495181 \n",
       "L 188.942976 195.495181 \n",
       "L 190.063458 195.495181 \n",
       "L 191.18394 195.495181 \n",
       "L 192.304422 195.495181 \n",
       "L 193.424904 195.495181 \n",
       "L 194.545386 195.495181 \n",
       "L 195.665868 195.495181 \n",
       "L 196.78635 195.495181 \n",
       "L 197.906832 195.495181 \n",
       "L 199.027314 195.495181 \n",
       "L 200.147796 195.495181 \n",
       "L 201.268277 195.495181 \n",
       "L 202.388759 195.495181 \n",
       "L 203.509241 195.495181 \n",
       "L 204.629723 195.495181 \n",
       "L 205.750205 195.495181 \n",
       "L 206.870687 195.495181 \n",
       "L 207.991169 195.495181 \n",
       "L 209.111651 195.495181 \n",
       "L 210.232133 195.495181 \n",
       "L 211.352615 195.495181 \n",
       "L 212.473097 195.495181 \n",
       "L 213.593579 195.495181 \n",
       "L 214.714061 195.495181 \n",
       "L 215.834543 195.495181 \n",
       "L 216.955024 195.495181 \n",
       "L 218.075506 195.495181 \n",
       "L 219.195988 195.495181 \n",
       "L 220.31647 195.495181 \n",
       "L 221.436952 195.495181 \n",
       "L 222.557434 195.495181 \n",
       "L 223.677916 195.495181 \n",
       "L 224.798398 195.495181 \n",
       "L 225.91888 195.768072 \n",
       "L 227.039362 195.768072 \n",
       "L 228.159844 195.768072 \n",
       "L 229.280326 195.768072 \n",
       "L 230.400808 195.768072 \n",
       "L 231.52129 195.768072 \n",
       "L 232.641771 195.768072 \n",
       "L 233.762253 195.768072 \n",
       "L 234.882735 195.768072 \n",
       "L 236.003217 195.768072 \n",
       "L 237.123699 195.768072 \n",
       "L 238.244181 195.768072 \n",
       "L 239.364663 195.768072 \n",
       "L 240.485145 195.768072 \n",
       "L 241.605627 195.768072 \n",
       "L 242.726109 195.768072 \n",
       "L 243.846591 195.768072 \n",
       "L 244.967073 195.768072 \n",
       "L 246.087555 195.768072 \n",
       "L 247.208037 195.768072 \n",
       "L 248.328518 195.768072 \n",
       "L 249.449 195.768072 \n",
       "L 250.569482 195.768072 \n",
       "L 251.689964 195.768072 \n",
       "L 252.810446 195.768072 \n",
       "L 253.930928 195.768072 \n",
       "L 255.05141 195.768072 \n",
       "L 256.171892 195.768072 \n",
       "L 257.292374 195.768072 \n",
       "L 258.412856 195.768072 \n",
       "L 259.533338 195.768072 \n",
       "L 260.65382 195.768072 \n",
       "L 261.774302 195.768072 \n",
       "L 262.894784 195.768072 \n",
       "L 264.015265 195.768072 \n",
       "L 265.135747 195.768072 \n",
       "L 266.256229 195.768072 \n",
       "L 267.376711 195.768072 \n",
       "L 268.497193 195.768072 \n",
       "L 269.617675 195.768072 \n",
       "L 270.738157 195.768072 \n",
       "L 271.858639 195.768072 \n",
       "L 272.979121 195.768072 \n",
       "L 274.099603 195.768072 \n",
       "L 275.220085 195.768072 \n",
       "L 276.340567 195.768072 \n",
       "L 277.461049 195.768072 \n",
       "L 278.58153 195.768072 \n",
       "L 279.702012 195.768072 \n",
       "L 280.822494 195.768072 \n",
       "L 281.942976 195.768072 \n",
       "L 283.063458 195.768072 \n",
       "L 284.18394 195.768072 \n",
       "L 285.304422 195.768072 \n",
       "L 286.424904 195.768072 \n",
       "L 287.545386 195.768072 \n",
       "L 288.665868 195.768072 \n",
       "L 289.78635 195.768072 \n",
       "L 290.906832 195.768072 \n",
       "L 292.027314 195.768072 \n",
       "L 293.147796 195.768072 \n",
       "L 294.268277 195.768072 \n",
       "L 295.388759 195.768072 \n",
       "L 296.509241 195.768072 \n",
       "L 297.629723 195.768072 \n",
       "L 298.750205 195.768072 \n",
       "L 299.870687 195.768072 \n",
       "L 300.991169 195.768072 \n",
       "L 302.111651 195.768072 \n",
       "L 303.232133 195.768072 \n",
       "L 304.352615 195.768072 \n",
       "L 305.473097 195.768072 \n",
       "L 306.593579 195.768072 \n",
       "L 307.714061 195.768072 \n",
       "L 308.834543 195.768072 \n",
       "L 309.955024 195.768072 \n",
       "L 311.075506 195.768072 \n",
       "L 312.195988 195.768072 \n",
       "L 313.31647 195.768072 \n",
       "L 314.436952 195.768072 \n",
       "L 315.557434 195.768072 \n",
       "L 316.677916 195.768072 \n",
       "L 317.798398 195.768072 \n",
       "L 318.91888 195.768072 \n",
       "L 320.039362 195.768072 \n",
       "L 321.159844 195.768072 \n",
       "L 321.159844 195.631627 \n",
       "L 320.039362 195.631627 \n",
       "L 318.91888 195.631627 \n",
       "L 317.798398 195.631627 \n",
       "L 316.677916 195.631627 \n",
       "L 315.557434 195.631627 \n",
       "L 314.436952 195.631627 \n",
       "L 313.31647 195.631627 \n",
       "L 312.195988 195.631627 \n",
       "L 311.075506 195.631627 \n",
       "L 309.955024 195.631627 \n",
       "L 308.834543 195.631627 \n",
       "L 307.714061 195.631627 \n",
       "L 306.593579 195.631627 \n",
       "L 305.473097 195.631627 \n",
       "L 304.352615 195.631627 \n",
       "L 303.232133 195.631627 \n",
       "L 302.111651 195.631627 \n",
       "L 300.991169 195.631627 \n",
       "L 299.870687 195.631627 \n",
       "L 298.750205 195.631627 \n",
       "L 297.629723 195.631627 \n",
       "L 296.509241 195.631627 \n",
       "L 295.388759 195.631627 \n",
       "L 294.268277 195.631627 \n",
       "L 293.147796 195.631627 \n",
       "L 292.027314 195.631627 \n",
       "L 290.906832 195.631627 \n",
       "L 289.78635 195.631627 \n",
       "L 288.665868 195.631627 \n",
       "L 287.545386 195.631627 \n",
       "L 286.424904 195.631627 \n",
       "L 285.304422 195.631627 \n",
       "L 284.18394 195.631627 \n",
       "L 283.063458 195.631627 \n",
       "L 281.942976 195.631627 \n",
       "L 280.822494 195.631627 \n",
       "L 279.702012 195.631627 \n",
       "L 278.58153 195.631627 \n",
       "L 277.461049 195.631627 \n",
       "L 276.340567 195.631627 \n",
       "L 275.220085 195.631627 \n",
       "L 274.099603 195.631627 \n",
       "L 272.979121 195.631627 \n",
       "L 271.858639 195.631627 \n",
       "L 270.738157 195.631627 \n",
       "L 269.617675 195.631627 \n",
       "L 268.497193 195.631627 \n",
       "L 267.376711 195.631627 \n",
       "L 266.256229 195.631627 \n",
       "L 265.135747 195.631627 \n",
       "L 264.015265 195.631627 \n",
       "L 262.894784 195.631627 \n",
       "L 261.774302 195.631627 \n",
       "L 260.65382 195.631627 \n",
       "L 259.533338 195.631627 \n",
       "L 258.412856 195.631627 \n",
       "L 257.292374 195.631627 \n",
       "L 256.171892 195.631627 \n",
       "L 255.05141 195.631627 \n",
       "L 253.930928 195.631627 \n",
       "L 252.810446 195.631627 \n",
       "L 251.689964 195.631627 \n",
       "L 250.569482 195.631627 \n",
       "L 249.449 195.631627 \n",
       "L 248.328518 195.631627 \n",
       "L 247.208037 195.631627 \n",
       "L 246.087555 195.631627 \n",
       "L 244.967073 195.631627 \n",
       "L 243.846591 195.631627 \n",
       "L 242.726109 195.631627 \n",
       "L 241.605627 195.631627 \n",
       "L 240.485145 195.631627 \n",
       "L 239.364663 195.631627 \n",
       "L 238.244181 195.631627 \n",
       "L 237.123699 195.631627 \n",
       "L 236.003217 195.631627 \n",
       "L 234.882735 195.631627 \n",
       "L 233.762253 195.631627 \n",
       "L 232.641771 195.631627 \n",
       "L 231.52129 195.631627 \n",
       "L 230.400808 195.631627 \n",
       "L 229.280326 195.631627 \n",
       "L 228.159844 195.631627 \n",
       "L 227.039362 195.631627 \n",
       "L 225.91888 195.631627 \n",
       "L 224.798398 195.222289 \n",
       "L 223.677916 195.222289 \n",
       "L 222.557434 195.222289 \n",
       "L 221.436952 195.222289 \n",
       "L 220.31647 195.222289 \n",
       "L 219.195988 195.222289 \n",
       "L 218.075506 195.222289 \n",
       "L 216.955024 195.222289 \n",
       "L 215.834543 195.222289 \n",
       "L 214.714061 195.222289 \n",
       "L 213.593579 195.222289 \n",
       "L 212.473097 195.222289 \n",
       "L 211.352615 195.222289 \n",
       "L 210.232133 195.222289 \n",
       "L 209.111651 195.222289 \n",
       "L 207.991169 195.222289 \n",
       "L 206.870687 195.222289 \n",
       "L 205.750205 195.222289 \n",
       "L 204.629723 195.222289 \n",
       "L 203.509241 195.222289 \n",
       "L 202.388759 195.222289 \n",
       "L 201.268277 195.222289 \n",
       "L 200.147796 195.222289 \n",
       "L 199.027314 195.222289 \n",
       "L 197.906832 195.222289 \n",
       "L 196.78635 195.222289 \n",
       "L 195.665868 195.222289 \n",
       "L 194.545386 195.222289 \n",
       "L 193.424904 195.222289 \n",
       "L 192.304422 195.222289 \n",
       "L 191.18394 195.222289 \n",
       "L 190.063458 195.222289 \n",
       "L 188.942976 195.222289 \n",
       "L 187.822494 195.222289 \n",
       "L 186.702012 195.222289 \n",
       "L 185.58153 195.222289 \n",
       "L 184.461049 195.222289 \n",
       "L 183.340567 195.222289 \n",
       "L 182.220085 195.222289 \n",
       "L 181.099603 195.222289 \n",
       "L 179.979121 195.222289 \n",
       "L 178.858639 195.222289 \n",
       "L 177.738157 195.222289 \n",
       "L 176.617675 195.222289 \n",
       "L 175.497193 195.222289 \n",
       "L 174.376711 195.222289 \n",
       "L 173.256229 195.222289 \n",
       "L 172.135747 195.222289 \n",
       "L 171.015265 195.222289 \n",
       "L 169.894784 195.222289 \n",
       "L 168.774302 195.222289 \n",
       "L 167.65382 195.222289 \n",
       "L 166.533338 195.222289 \n",
       "L 165.412856 195.222289 \n",
       "L 164.292374 195.222289 \n",
       "L 163.171892 195.222289 \n",
       "L 162.05141 195.222289 \n",
       "L 160.930928 195.222289 \n",
       "L 159.810446 195.222289 \n",
       "L 158.689964 195.222289 \n",
       "L 157.569482 195.222289 \n",
       "L 156.449 195.222289 \n",
       "L 155.328518 195.222289 \n",
       "L 154.208037 195.222289 \n",
       "L 153.087555 195.222289 \n",
       "L 151.967073 195.222289 \n",
       "L 150.846591 195.222289 \n",
       "L 149.726109 195.222289 \n",
       "L 148.605627 195.222289 \n",
       "L 147.485145 195.222289 \n",
       "L 146.364663 195.222289 \n",
       "L 145.244181 195.222289 \n",
       "L 144.123699 195.222289 \n",
       "L 143.003217 195.222289 \n",
       "L 141.882735 195.222289 \n",
       "L 140.762253 195.222289 \n",
       "L 139.641771 195.222289 \n",
       "L 138.52129 195.222289 \n",
       "L 137.400808 195.222289 \n",
       "L 136.280326 195.222289 \n",
       "L 135.159844 195.222289 \n",
       "L 134.039362 195.222289 \n",
       "L 132.91888 195.222289 \n",
       "L 131.798398 195.222289 \n",
       "L 130.677916 195.222289 \n",
       "L 129.557434 195.222289 \n",
       "L 128.436952 195.222289 \n",
       "L 127.31647 195.222289 \n",
       "L 126.195988 195.222289 \n",
       "L 125.075506 195.222289 \n",
       "L 123.955024 195.222289 \n",
       "L 122.834543 195.222289 \n",
       "L 121.714061 195.222289 \n",
       "L 120.593579 195.222289 \n",
       "L 119.473097 195.222289 \n",
       "L 118.352615 195.222289 \n",
       "L 117.232133 195.222289 \n",
       "L 116.111651 195.222289 \n",
       "L 114.991169 195.222289 \n",
       "L 113.870687 195.222289 \n",
       "L 112.750205 195.222289 \n",
       "L 111.629723 195.222289 \n",
       "L 110.509241 195.222289 \n",
       "L 109.388759 195.222289 \n",
       "L 108.268277 195.222289 \n",
       "L 107.147796 195.222289 \n",
       "L 106.027314 195.222289 \n",
       "L 104.906832 195.222289 \n",
       "L 103.78635 195.222289 \n",
       "L 102.665868 195.222289 \n",
       "L 101.545386 195.222289 \n",
       "L 100.424904 195.222289 \n",
       "L 99.304422 195.222289 \n",
       "L 98.18394 195.222289 \n",
       "L 97.063458 195.222289 \n",
       "L 95.942976 195.222289 \n",
       "L 94.822494 195.222289 \n",
       "L 93.702012 195.222289 \n",
       "L 92.58153 195.222289 \n",
       "L 91.461049 195.222289 \n",
       "L 90.340567 195.222289 \n",
       "L 89.220085 195.222289 \n",
       "L 88.099603 195.222289 \n",
       "L 86.979121 195.222289 \n",
       "L 85.858639 195.222289 \n",
       "L 84.738157 195.222289 \n",
       "L 83.617675 195.222289 \n",
       "L 82.497193 195.222289 \n",
       "L 81.376711 195.222289 \n",
       "L 80.256229 195.222289 \n",
       "L 79.135747 195.222289 \n",
       "L 78.015265 195.222289 \n",
       "L 76.894784 195.222289 \n",
       "L 75.774302 195.222289 \n",
       "L 74.65382 195.222289 \n",
       "L 73.533338 195.222289 \n",
       "L 72.412856 195.222289 \n",
       "L 71.292374 195.222289 \n",
       "L 70.171892 195.222289 \n",
       "L 69.05141 195.222289 \n",
       "L 67.930928 195.222289 \n",
       "L 66.810446 195.222289 \n",
       "L 65.689964 195.222289 \n",
       "L 64.569482 195.222289 \n",
       "L 63.449 195.222289 \n",
       "L 62.328518 195.222289 \n",
       "L 61.208037 195.222289 \n",
       "L 60.087555 195.222289 \n",
       "L 58.967073 195.222289 \n",
       "L 57.846591 195.222289 \n",
       "L 56.726109 195.222289 \n",
       "L 55.605627 195.222289 \n",
       "L 54.485145 195.222289 \n",
       "L 53.364663 195.222289 \n",
       "L 52.244181 195.222289 \n",
       "L 51.123699 195.222289 \n",
       "L 50.003217 195.222289 \n",
       "L 48.882735 195.222289 \n",
       "L 47.762253 195.222289 \n",
       "L 46.641771 195.222289 \n",
       "L 45.52129 195.222289 \n",
       "L 44.400808 195.222289 \n",
       "L 43.280326 195.222289 \n",
       "L 42.159844 195.222289 \n",
       "L 42.159844 195.495181 \n",
       "z\n",
       "\" style=\"fill:#d62728;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_4\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 43.280326 195.222289 \n",
       "L 44.400808 195.222289 \n",
       "L 45.52129 195.222289 \n",
       "L 46.641771 195.222289 \n",
       "L 47.762253 195.222289 \n",
       "L 48.882735 195.222289 \n",
       "L 50.003217 195.222289 \n",
       "L 51.123699 195.222289 \n",
       "L 52.244181 195.222289 \n",
       "L 53.364663 195.222289 \n",
       "L 54.485145 195.222289 \n",
       "L 55.605627 195.222289 \n",
       "L 56.726109 195.222289 \n",
       "L 57.846591 195.222289 \n",
       "L 58.967073 195.222289 \n",
       "L 60.087555 195.222289 \n",
       "L 61.208037 195.222289 \n",
       "L 62.328518 195.222289 \n",
       "L 63.449 195.222289 \n",
       "L 64.569482 195.222289 \n",
       "L 65.689964 195.222289 \n",
       "L 66.810446 195.222289 \n",
       "L 67.930928 195.222289 \n",
       "L 69.05141 195.222289 \n",
       "L 70.171892 195.222289 \n",
       "L 71.292374 195.222289 \n",
       "L 72.412856 195.222289 \n",
       "L 73.533338 195.222289 \n",
       "L 74.65382 195.222289 \n",
       "L 75.774302 195.222289 \n",
       "L 76.894784 195.222289 \n",
       "L 78.015265 195.222289 \n",
       "L 79.135747 195.222289 \n",
       "L 80.256229 195.222289 \n",
       "L 81.376711 195.222289 \n",
       "L 82.497193 195.222289 \n",
       "L 83.617675 195.222289 \n",
       "L 84.738157 195.222289 \n",
       "L 85.858639 195.222289 \n",
       "L 86.979121 195.222289 \n",
       "L 88.099603 195.222289 \n",
       "L 89.220085 195.222289 \n",
       "L 90.340567 195.222289 \n",
       "L 91.461049 195.222289 \n",
       "L 92.58153 195.222289 \n",
       "L 93.702012 195.222289 \n",
       "L 94.822494 195.222289 \n",
       "L 95.942976 195.222289 \n",
       "L 97.063458 195.222289 \n",
       "L 98.18394 195.222289 \n",
       "L 99.304422 195.222289 \n",
       "L 100.424904 195.222289 \n",
       "L 101.545386 195.222289 \n",
       "L 102.665868 195.222289 \n",
       "L 103.78635 195.222289 \n",
       "L 104.906832 195.222289 \n",
       "L 106.027314 195.222289 \n",
       "L 107.147796 195.222289 \n",
       "L 108.268277 195.222289 \n",
       "L 109.388759 195.222289 \n",
       "L 110.509241 195.222289 \n",
       "L 111.629723 195.222289 \n",
       "L 112.750205 195.222289 \n",
       "L 113.870687 195.222289 \n",
       "L 114.991169 195.222289 \n",
       "L 116.111651 195.222289 \n",
       "L 117.232133 195.222289 \n",
       "L 118.352615 195.222289 \n",
       "L 119.473097 195.222289 \n",
       "L 120.593579 195.222289 \n",
       "L 121.714061 195.222289 \n",
       "L 122.834543 195.222289 \n",
       "L 123.955024 195.222289 \n",
       "L 125.075506 195.222289 \n",
       "L 126.195988 195.222289 \n",
       "L 127.31647 195.222289 \n",
       "L 128.436952 195.222289 \n",
       "L 129.557434 195.222289 \n",
       "L 130.677916 195.222289 \n",
       "L 131.798398 195.222289 \n",
       "L 132.91888 195.222289 \n",
       "L 134.039362 195.222289 \n",
       "L 135.159844 195.222289 \n",
       "L 136.280326 195.222289 \n",
       "L 137.400808 195.222289 \n",
       "L 138.52129 195.222289 \n",
       "L 139.641771 195.222289 \n",
       "L 140.762253 195.222289 \n",
       "L 141.882735 195.222289 \n",
       "L 143.003217 195.222289 \n",
       "L 144.123699 195.222289 \n",
       "L 145.244181 195.222289 \n",
       "L 146.364663 195.222289 \n",
       "L 147.485145 195.222289 \n",
       "L 148.605627 195.222289 \n",
       "L 149.726109 195.222289 \n",
       "L 150.846591 195.222289 \n",
       "L 151.967073 195.222289 \n",
       "L 153.087555 195.222289 \n",
       "L 154.208037 195.222289 \n",
       "L 155.328518 195.222289 \n",
       "L 156.449 195.222289 \n",
       "L 157.569482 195.222289 \n",
       "L 158.689964 195.222289 \n",
       "L 159.810446 195.222289 \n",
       "L 160.930928 195.222289 \n",
       "L 162.05141 195.222289 \n",
       "L 163.171892 195.222289 \n",
       "L 164.292374 195.222289 \n",
       "L 165.412856 195.222289 \n",
       "L 166.533338 195.222289 \n",
       "L 167.65382 195.222289 \n",
       "L 168.774302 195.222289 \n",
       "L 169.894784 195.222289 \n",
       "L 171.015265 195.222289 \n",
       "L 172.135747 195.222289 \n",
       "L 173.256229 195.222289 \n",
       "L 174.376711 195.222289 \n",
       "L 175.497193 195.222289 \n",
       "L 176.617675 195.222289 \n",
       "L 177.738157 195.222289 \n",
       "L 178.858639 195.222289 \n",
       "L 179.979121 195.222289 \n",
       "L 181.099603 195.222289 \n",
       "L 182.220085 195.222289 \n",
       "L 183.340567 195.222289 \n",
       "L 184.461049 195.222289 \n",
       "L 185.58153 195.222289 \n",
       "L 186.702012 195.222289 \n",
       "L 187.822494 195.222289 \n",
       "L 188.942976 195.222289 \n",
       "L 190.063458 195.222289 \n",
       "L 191.18394 195.222289 \n",
       "L 192.304422 195.222289 \n",
       "L 193.424904 195.222289 \n",
       "L 194.545386 195.222289 \n",
       "L 195.665868 195.222289 \n",
       "L 196.78635 195.222289 \n",
       "L 197.906832 195.222289 \n",
       "L 199.027314 195.222289 \n",
       "L 200.147796 195.222289 \n",
       "L 201.268277 195.222289 \n",
       "L 202.388759 195.222289 \n",
       "L 203.509241 195.222289 \n",
       "L 204.629723 195.222289 \n",
       "L 205.750205 195.222289 \n",
       "L 206.870687 195.222289 \n",
       "L 207.991169 195.222289 \n",
       "L 209.111651 195.222289 \n",
       "L 210.232133 195.222289 \n",
       "L 211.352615 195.222289 \n",
       "L 212.473097 195.222289 \n",
       "L 213.593579 195.222289 \n",
       "L 214.714061 195.222289 \n",
       "L 215.834543 195.222289 \n",
       "L 216.955024 195.222289 \n",
       "L 218.075506 195.222289 \n",
       "L 219.195988 195.222289 \n",
       "L 220.31647 195.222289 \n",
       "L 221.436952 195.222289 \n",
       "L 222.557434 195.222289 \n",
       "L 223.677916 195.222289 \n",
       "L 224.798398 195.222289 \n",
       "L 225.91888 195.631627 \n",
       "L 227.039362 195.631627 \n",
       "L 228.159844 195.631627 \n",
       "L 229.280326 195.631627 \n",
       "L 230.400808 195.631627 \n",
       "L 231.52129 195.631627 \n",
       "L 232.641771 195.631627 \n",
       "L 233.762253 195.631627 \n",
       "L 234.882735 195.631627 \n",
       "L 236.003217 195.631627 \n",
       "L 237.123699 195.631627 \n",
       "L 238.244181 195.631627 \n",
       "L 239.364663 195.631627 \n",
       "L 240.485145 195.631627 \n",
       "L 241.605627 195.631627 \n",
       "L 242.726109 195.631627 \n",
       "L 243.846591 195.631627 \n",
       "L 244.967073 195.631627 \n",
       "L 246.087555 195.631627 \n",
       "L 247.208037 195.631627 \n",
       "L 248.328518 195.631627 \n",
       "L 249.449 195.631627 \n",
       "L 250.569482 195.631627 \n",
       "L 251.689964 195.631627 \n",
       "L 252.810446 195.631627 \n",
       "L 253.930928 195.631627 \n",
       "L 255.05141 195.631627 \n",
       "L 256.171892 195.631627 \n",
       "L 257.292374 195.631627 \n",
       "L 258.412856 195.631627 \n",
       "L 259.533338 195.631627 \n",
       "L 260.65382 195.631627 \n",
       "L 261.774302 195.631627 \n",
       "L 262.894784 195.631627 \n",
       "L 264.015265 195.631627 \n",
       "L 265.135747 195.631627 \n",
       "L 266.256229 195.631627 \n",
       "L 267.376711 195.631627 \n",
       "L 268.497193 195.631627 \n",
       "L 269.617675 195.631627 \n",
       "L 270.738157 195.631627 \n",
       "L 271.858639 195.631627 \n",
       "L 272.979121 195.631627 \n",
       "L 274.099603 195.631627 \n",
       "L 275.220085 195.631627 \n",
       "L 276.340567 195.631627 \n",
       "L 277.461049 195.631627 \n",
       "L 278.58153 195.631627 \n",
       "L 279.702012 195.631627 \n",
       "L 280.822494 195.631627 \n",
       "L 281.942976 195.631627 \n",
       "L 283.063458 195.631627 \n",
       "L 284.18394 195.631627 \n",
       "L 285.304422 195.631627 \n",
       "L 286.424904 195.631627 \n",
       "L 287.545386 195.631627 \n",
       "L 288.665868 195.631627 \n",
       "L 289.78635 195.631627 \n",
       "L 290.906832 195.631627 \n",
       "L 292.027314 195.631627 \n",
       "L 293.147796 195.631627 \n",
       "L 294.268277 195.631627 \n",
       "L 295.388759 195.631627 \n",
       "L 296.509241 195.631627 \n",
       "L 297.629723 195.631627 \n",
       "L 298.750205 195.631627 \n",
       "L 299.870687 195.631627 \n",
       "L 300.991169 195.631627 \n",
       "L 302.111651 195.631627 \n",
       "L 303.232133 195.631627 \n",
       "L 304.352615 195.631627 \n",
       "L 305.473097 195.631627 \n",
       "L 306.593579 195.631627 \n",
       "L 307.714061 195.631627 \n",
       "L 308.834543 195.631627 \n",
       "L 309.955024 195.631627 \n",
       "L 311.075506 195.631627 \n",
       "L 312.195988 195.631627 \n",
       "L 313.31647 195.631627 \n",
       "L 314.436952 195.631627 \n",
       "L 315.557434 195.631627 \n",
       "L 316.677916 195.631627 \n",
       "L 317.798398 195.631627 \n",
       "L 318.91888 195.631627 \n",
       "L 320.039362 195.631627 \n",
       "L 321.159844 195.631627 \n",
       "L 321.159844 195.495181 \n",
       "L 320.039362 195.495181 \n",
       "L 318.91888 195.495181 \n",
       "L 317.798398 195.495181 \n",
       "L 316.677916 195.495181 \n",
       "L 315.557434 195.495181 \n",
       "L 314.436952 195.495181 \n",
       "L 313.31647 195.495181 \n",
       "L 312.195988 195.495181 \n",
       "L 311.075506 195.495181 \n",
       "L 309.955024 195.495181 \n",
       "L 308.834543 195.495181 \n",
       "L 307.714061 195.495181 \n",
       "L 306.593579 195.495181 \n",
       "L 305.473097 195.495181 \n",
       "L 304.352615 195.495181 \n",
       "L 303.232133 195.495181 \n",
       "L 302.111651 195.495181 \n",
       "L 300.991169 195.495181 \n",
       "L 299.870687 195.495181 \n",
       "L 298.750205 195.495181 \n",
       "L 297.629723 195.495181 \n",
       "L 296.509241 195.495181 \n",
       "L 295.388759 195.495181 \n",
       "L 294.268277 195.495181 \n",
       "L 293.147796 195.495181 \n",
       "L 292.027314 195.495181 \n",
       "L 290.906832 195.495181 \n",
       "L 289.78635 195.495181 \n",
       "L 288.665868 195.495181 \n",
       "L 287.545386 195.495181 \n",
       "L 286.424904 195.495181 \n",
       "L 285.304422 195.495181 \n",
       "L 284.18394 195.495181 \n",
       "L 283.063458 195.495181 \n",
       "L 281.942976 195.495181 \n",
       "L 280.822494 195.495181 \n",
       "L 279.702012 195.495181 \n",
       "L 278.58153 195.495181 \n",
       "L 277.461049 195.495181 \n",
       "L 276.340567 195.495181 \n",
       "L 275.220085 195.495181 \n",
       "L 274.099603 195.495181 \n",
       "L 272.979121 195.495181 \n",
       "L 271.858639 195.495181 \n",
       "L 270.738157 195.495181 \n",
       "L 269.617675 195.495181 \n",
       "L 268.497193 195.495181 \n",
       "L 267.376711 195.495181 \n",
       "L 266.256229 195.495181 \n",
       "L 265.135747 195.495181 \n",
       "L 264.015265 195.495181 \n",
       "L 262.894784 195.495181 \n",
       "L 261.774302 195.495181 \n",
       "L 260.65382 195.495181 \n",
       "L 259.533338 195.495181 \n",
       "L 258.412856 195.495181 \n",
       "L 257.292374 195.495181 \n",
       "L 256.171892 195.495181 \n",
       "L 255.05141 195.495181 \n",
       "L 253.930928 195.495181 \n",
       "L 252.810446 195.495181 \n",
       "L 251.689964 195.495181 \n",
       "L 250.569482 195.495181 \n",
       "L 249.449 195.495181 \n",
       "L 248.328518 195.495181 \n",
       "L 247.208037 195.495181 \n",
       "L 246.087555 195.495181 \n",
       "L 244.967073 195.495181 \n",
       "L 243.846591 195.495181 \n",
       "L 242.726109 195.495181 \n",
       "L 241.605627 195.495181 \n",
       "L 240.485145 195.495181 \n",
       "L 239.364663 195.495181 \n",
       "L 238.244181 195.495181 \n",
       "L 237.123699 195.495181 \n",
       "L 236.003217 195.495181 \n",
       "L 234.882735 195.495181 \n",
       "L 233.762253 195.495181 \n",
       "L 232.641771 195.495181 \n",
       "L 231.52129 195.495181 \n",
       "L 230.400808 195.495181 \n",
       "L 229.280326 195.495181 \n",
       "L 228.159844 195.495181 \n",
       "L 227.039362 195.495181 \n",
       "L 225.91888 195.495181 \n",
       "L 224.798398 194.949398 \n",
       "L 224.798398 193.857831 \n",
       "L 224.798398 192.766265 \n",
       "L 224.798398 191.674699 \n",
       "L 224.798398 190.583133 \n",
       "L 224.798398 189.491566 \n",
       "L 224.798398 188.4 \n",
       "L 224.798398 187.308434 \n",
       "L 224.798398 186.216867 \n",
       "L 224.798398 185.125301 \n",
       "L 224.798398 184.033735 \n",
       "L 224.798398 182.942169 \n",
       "L 224.798398 181.850602 \n",
       "L 224.798398 180.759036 \n",
       "L 224.798398 179.66747 \n",
       "L 224.798398 178.575904 \n",
       "L 224.798398 177.484337 \n",
       "L 224.798398 176.392771 \n",
       "L 224.798398 175.301205 \n",
       "L 224.798398 174.209639 \n",
       "L 224.798398 173.118072 \n",
       "L 224.798398 172.026506 \n",
       "L 224.798398 170.93494 \n",
       "L 224.798398 169.843373 \n",
       "L 224.798398 168.751807 \n",
       "L 224.798398 167.660241 \n",
       "L 224.798398 166.568675 \n",
       "L 224.798398 165.477108 \n",
       "L 224.798398 164.385542 \n",
       "L 224.798398 163.293976 \n",
       "L 224.798398 162.20241 \n",
       "L 224.798398 161.110843 \n",
       "L 224.798398 160.019277 \n",
       "L 224.798398 158.927711 \n",
       "L 224.798398 157.836145 \n",
       "L 224.798398 156.744578 \n",
       "L 224.798398 155.653012 \n",
       "L 224.798398 154.561446 \n",
       "L 224.798398 153.46988 \n",
       "L 224.798398 152.378313 \n",
       "L 224.798398 151.286747 \n",
       "L 224.798398 150.195181 \n",
       "L 224.798398 149.103614 \n",
       "L 224.798398 148.012048 \n",
       "L 224.798398 146.920482 \n",
       "L 224.798398 145.828916 \n",
       "L 224.798398 144.737349 \n",
       "L 224.798398 143.645783 \n",
       "L 224.798398 142.554217 \n",
       "L 224.798398 141.462651 \n",
       "L 224.798398 140.371084 \n",
       "L 224.798398 139.279518 \n",
       "L 224.798398 138.187952 \n",
       "L 224.798398 137.096386 \n",
       "L 224.798398 136.004819 \n",
       "L 224.798398 134.913253 \n",
       "L 224.798398 133.821687 \n",
       "L 224.798398 132.73012 \n",
       "L 224.798398 131.638554 \n",
       "L 224.798398 130.546988 \n",
       "L 224.798398 129.455422 \n",
       "L 224.798398 128.363855 \n",
       "L 224.798398 127.272289 \n",
       "L 224.798398 126.180723 \n",
       "L 224.798398 125.089157 \n",
       "L 224.798398 123.99759 \n",
       "L 224.798398 122.906024 \n",
       "L 224.798398 121.814458 \n",
       "L 224.798398 120.722892 \n",
       "L 224.798398 119.631325 \n",
       "L 224.798398 118.539759 \n",
       "L 224.798398 117.448193 \n",
       "L 225.91888 116.356627 \n",
       "L 227.039362 116.356627 \n",
       "L 228.159844 116.356627 \n",
       "L 229.280326 116.356627 \n",
       "L 230.400808 116.356627 \n",
       "L 231.52129 116.356627 \n",
       "L 232.641771 116.356627 \n",
       "L 233.762253 116.356627 \n",
       "L 234.882735 116.356627 \n",
       "L 236.003217 116.356627 \n",
       "L 237.123699 116.356627 \n",
       "L 238.244181 116.356627 \n",
       "L 239.364663 116.356627 \n",
       "L 240.485145 116.356627 \n",
       "L 241.605627 116.356627 \n",
       "L 242.726109 116.356627 \n",
       "L 243.846591 116.356627 \n",
       "L 244.967073 116.356627 \n",
       "L 246.087555 116.356627 \n",
       "L 247.208037 116.356627 \n",
       "L 248.328518 116.356627 \n",
       "L 249.449 116.356627 \n",
       "L 250.569482 116.356627 \n",
       "L 251.689964 116.356627 \n",
       "L 252.810446 116.356627 \n",
       "L 253.930928 116.356627 \n",
       "L 255.05141 116.356627 \n",
       "L 256.171892 116.356627 \n",
       "L 257.292374 116.356627 \n",
       "L 258.412856 116.356627 \n",
       "L 259.533338 116.356627 \n",
       "L 260.65382 116.356627 \n",
       "L 261.774302 116.356627 \n",
       "L 262.894784 116.356627 \n",
       "L 264.015265 116.356627 \n",
       "L 265.135747 116.356627 \n",
       "L 266.256229 116.356627 \n",
       "L 267.376711 116.356627 \n",
       "L 268.497193 116.356627 \n",
       "L 269.617675 116.356627 \n",
       "L 270.738157 116.356627 \n",
       "L 271.858639 116.356627 \n",
       "L 272.979121 116.356627 \n",
       "L 274.099603 116.356627 \n",
       "L 275.220085 116.356627 \n",
       "L 276.340567 116.356627 \n",
       "L 277.461049 116.356627 \n",
       "L 278.58153 116.356627 \n",
       "L 279.702012 116.356627 \n",
       "L 280.822494 116.356627 \n",
       "L 281.942976 116.356627 \n",
       "L 283.063458 116.356627 \n",
       "L 284.18394 116.356627 \n",
       "L 285.304422 116.356627 \n",
       "L 286.424904 116.356627 \n",
       "L 287.545386 116.356627 \n",
       "L 288.665868 116.356627 \n",
       "L 289.78635 116.356627 \n",
       "L 290.906832 116.356627 \n",
       "L 292.027314 116.356627 \n",
       "L 293.147796 116.356627 \n",
       "L 294.268277 116.356627 \n",
       "L 295.388759 116.356627 \n",
       "L 296.509241 116.356627 \n",
       "L 297.629723 116.356627 \n",
       "L 298.750205 116.356627 \n",
       "L 299.870687 116.356627 \n",
       "L 300.991169 116.356627 \n",
       "L 302.111651 116.356627 \n",
       "L 303.232133 116.356627 \n",
       "L 304.352615 116.356627 \n",
       "L 305.473097 116.356627 \n",
       "L 306.593579 116.356627 \n",
       "L 307.714061 116.356627 \n",
       "L 308.834543 116.356627 \n",
       "L 309.955024 116.356627 \n",
       "L 311.075506 116.356627 \n",
       "L 312.195988 116.356627 \n",
       "L 313.31647 116.356627 \n",
       "L 314.436952 116.356627 \n",
       "L 315.557434 116.356627 \n",
       "L 316.677916 116.356627 \n",
       "L 317.798398 116.356627 \n",
       "L 318.91888 116.356627 \n",
       "L 320.039362 116.356627 \n",
       "L 321.159844 116.356627 \n",
       "L 321.159844 115.26506 \n",
       "L 321.159844 114.173494 \n",
       "L 321.159844 113.081928 \n",
       "L 321.159844 111.990361 \n",
       "L 321.159844 110.898795 \n",
       "L 321.159844 109.807229 \n",
       "L 321.159844 108.715663 \n",
       "L 321.159844 107.624096 \n",
       "L 321.159844 106.53253 \n",
       "L 321.159844 105.440964 \n",
       "L 321.159844 104.349398 \n",
       "L 321.159844 103.257831 \n",
       "L 321.159844 102.166265 \n",
       "L 321.159844 101.074699 \n",
       "L 321.159844 99.983133 \n",
       "L 321.159844 98.891566 \n",
       "L 321.159844 97.8 \n",
       "L 321.159844 96.708434 \n",
       "L 320.039362 96.708434 \n",
       "L 318.91888 96.708434 \n",
       "L 317.798398 96.708434 \n",
       "L 316.677916 96.708434 \n",
       "L 315.557434 96.708434 \n",
       "L 314.436952 96.708434 \n",
       "L 313.31647 96.708434 \n",
       "L 312.195988 96.708434 \n",
       "L 311.075506 96.708434 \n",
       "L 309.955024 96.708434 \n",
       "L 308.834543 96.708434 \n",
       "L 307.714061 96.708434 \n",
       "L 306.593579 96.708434 \n",
       "L 305.473097 96.708434 \n",
       "L 304.352615 96.708434 \n",
       "L 303.232133 96.708434 \n",
       "L 302.111651 96.708434 \n",
       "L 300.991169 96.708434 \n",
       "L 299.870687 96.708434 \n",
       "L 298.750205 96.708434 \n",
       "L 297.629723 96.708434 \n",
       "L 296.509241 96.708434 \n",
       "L 295.388759 96.708434 \n",
       "L 294.268277 96.708434 \n",
       "L 293.147796 96.708434 \n",
       "L 292.027314 96.708434 \n",
       "L 290.906832 96.708434 \n",
       "L 289.78635 96.708434 \n",
       "L 288.665868 96.708434 \n",
       "L 287.545386 96.708434 \n",
       "L 286.424904 96.708434 \n",
       "L 285.304422 96.708434 \n",
       "L 284.18394 96.708434 \n",
       "L 283.063458 96.708434 \n",
       "L 281.942976 96.708434 \n",
       "L 280.822494 96.708434 \n",
       "L 279.702012 96.708434 \n",
       "L 278.58153 96.708434 \n",
       "L 277.461049 96.708434 \n",
       "L 276.340567 96.708434 \n",
       "L 275.220085 96.708434 \n",
       "L 274.099603 96.708434 \n",
       "L 272.979121 96.708434 \n",
       "L 271.858639 96.708434 \n",
       "L 270.738157 96.708434 \n",
       "L 269.617675 96.708434 \n",
       "L 268.497193 96.708434 \n",
       "L 267.376711 96.708434 \n",
       "L 266.256229 96.708434 \n",
       "L 265.135747 96.708434 \n",
       "L 264.015265 96.708434 \n",
       "L 262.894784 96.708434 \n",
       "L 261.774302 96.708434 \n",
       "L 260.65382 96.708434 \n",
       "L 259.533338 96.708434 \n",
       "L 258.412856 96.708434 \n",
       "L 257.292374 96.708434 \n",
       "L 256.171892 96.708434 \n",
       "L 255.05141 96.708434 \n",
       "L 253.930928 96.708434 \n",
       "L 252.810446 96.708434 \n",
       "L 251.689964 96.708434 \n",
       "L 250.569482 96.708434 \n",
       "L 249.449 96.708434 \n",
       "L 248.328518 96.708434 \n",
       "L 247.208037 96.708434 \n",
       "L 246.087555 96.708434 \n",
       "L 244.967073 96.708434 \n",
       "L 243.846591 96.708434 \n",
       "L 242.726109 96.708434 \n",
       "L 241.605627 96.708434 \n",
       "L 240.485145 96.708434 \n",
       "L 239.364663 96.708434 \n",
       "L 238.244181 96.708434 \n",
       "L 237.123699 96.708434 \n",
       "L 236.003217 96.708434 \n",
       "L 234.882735 96.708434 \n",
       "L 233.762253 96.708434 \n",
       "L 232.641771 96.708434 \n",
       "L 231.52129 96.708434 \n",
       "L 230.400808 96.708434 \n",
       "L 229.280326 96.708434 \n",
       "L 228.159844 96.708434 \n",
       "L 227.039362 96.708434 \n",
       "L 225.91888 96.708434 \n",
       "L 225.91888 97.8 \n",
       "L 225.91888 98.891566 \n",
       "L 225.91888 99.983133 \n",
       "L 225.91888 101.074699 \n",
       "L 225.91888 102.166265 \n",
       "L 225.91888 103.257831 \n",
       "L 225.91888 104.349398 \n",
       "L 225.91888 105.440964 \n",
       "L 224.798398 106.53253 \n",
       "L 223.677916 106.53253 \n",
       "L 222.557434 106.53253 \n",
       "L 221.436952 106.53253 \n",
       "L 220.31647 106.53253 \n",
       "L 219.195988 106.53253 \n",
       "L 218.075506 106.53253 \n",
       "L 216.955024 106.53253 \n",
       "L 215.834543 106.53253 \n",
       "L 214.714061 106.53253 \n",
       "L 213.593579 106.53253 \n",
       "L 212.473097 106.53253 \n",
       "L 211.352615 106.53253 \n",
       "L 210.232133 106.53253 \n",
       "L 209.111651 106.53253 \n",
       "L 207.991169 106.53253 \n",
       "L 206.870687 106.53253 \n",
       "L 205.750205 106.53253 \n",
       "L 204.629723 106.53253 \n",
       "L 203.509241 106.53253 \n",
       "L 202.388759 106.53253 \n",
       "L 201.268277 106.53253 \n",
       "L 200.147796 106.53253 \n",
       "L 199.027314 106.53253 \n",
       "L 197.906832 106.53253 \n",
       "L 196.78635 106.53253 \n",
       "L 195.665868 106.53253 \n",
       "L 194.545386 106.53253 \n",
       "L 193.424904 106.53253 \n",
       "L 192.304422 106.53253 \n",
       "L 191.18394 106.53253 \n",
       "L 190.063458 106.53253 \n",
       "L 188.942976 106.53253 \n",
       "L 187.822494 106.53253 \n",
       "L 186.702012 106.53253 \n",
       "L 185.58153 106.53253 \n",
       "L 184.461049 106.53253 \n",
       "L 183.340567 106.53253 \n",
       "L 182.220085 106.53253 \n",
       "L 181.099603 106.53253 \n",
       "L 179.979121 106.53253 \n",
       "L 178.858639 106.53253 \n",
       "L 177.738157 106.53253 \n",
       "L 176.617675 106.53253 \n",
       "L 175.497193 106.53253 \n",
       "L 174.376711 106.53253 \n",
       "L 173.256229 106.53253 \n",
       "L 172.135747 106.53253 \n",
       "L 171.015265 106.53253 \n",
       "L 169.894784 106.53253 \n",
       "L 168.774302 106.53253 \n",
       "L 167.65382 106.53253 \n",
       "L 166.533338 106.53253 \n",
       "L 165.412856 106.53253 \n",
       "L 164.292374 106.53253 \n",
       "L 163.171892 106.53253 \n",
       "L 162.05141 106.53253 \n",
       "L 160.930928 106.53253 \n",
       "L 159.810446 106.53253 \n",
       "L 158.689964 106.53253 \n",
       "L 157.569482 106.53253 \n",
       "L 156.449 106.53253 \n",
       "L 155.328518 106.53253 \n",
       "L 154.208037 106.53253 \n",
       "L 153.087555 106.53253 \n",
       "L 151.967073 106.53253 \n",
       "L 150.846591 106.53253 \n",
       "L 149.726109 106.53253 \n",
       "L 148.605627 106.53253 \n",
       "L 147.485145 106.53253 \n",
       "L 146.364663 106.53253 \n",
       "L 145.244181 106.53253 \n",
       "L 144.123699 106.53253 \n",
       "L 143.003217 106.53253 \n",
       "L 141.882735 106.53253 \n",
       "L 140.762253 106.53253 \n",
       "L 139.641771 106.53253 \n",
       "L 138.52129 106.53253 \n",
       "L 137.400808 106.53253 \n",
       "L 136.280326 106.53253 \n",
       "L 135.159844 106.53253 \n",
       "L 134.039362 106.53253 \n",
       "L 132.91888 106.53253 \n",
       "L 131.798398 106.53253 \n",
       "L 130.677916 106.53253 \n",
       "L 129.557434 106.53253 \n",
       "L 128.436952 106.53253 \n",
       "L 127.31647 106.53253 \n",
       "L 126.195988 106.53253 \n",
       "L 125.075506 106.53253 \n",
       "L 123.955024 106.53253 \n",
       "L 122.834543 106.53253 \n",
       "L 121.714061 106.53253 \n",
       "L 120.593579 106.53253 \n",
       "L 119.473097 106.53253 \n",
       "L 118.352615 106.53253 \n",
       "L 117.232133 106.53253 \n",
       "L 116.111651 106.53253 \n",
       "L 114.991169 106.53253 \n",
       "L 113.870687 106.53253 \n",
       "L 112.750205 106.53253 \n",
       "L 111.629723 106.53253 \n",
       "L 110.509241 106.53253 \n",
       "L 109.388759 106.53253 \n",
       "L 108.268277 106.53253 \n",
       "L 107.147796 106.53253 \n",
       "L 106.027314 106.53253 \n",
       "L 104.906832 106.53253 \n",
       "L 103.78635 106.53253 \n",
       "L 102.665868 106.53253 \n",
       "L 101.545386 106.53253 \n",
       "L 100.424904 106.53253 \n",
       "L 99.304422 106.53253 \n",
       "L 98.18394 106.53253 \n",
       "L 97.063458 106.53253 \n",
       "L 95.942976 106.53253 \n",
       "L 94.822494 106.53253 \n",
       "L 93.702012 106.53253 \n",
       "L 92.58153 106.53253 \n",
       "L 91.461049 106.53253 \n",
       "L 90.340567 106.53253 \n",
       "L 89.220085 106.53253 \n",
       "L 88.099603 106.53253 \n",
       "L 86.979121 106.53253 \n",
       "L 85.858639 106.53253 \n",
       "L 84.738157 106.53253 \n",
       "L 83.617675 106.53253 \n",
       "L 82.497193 106.53253 \n",
       "L 81.376711 106.53253 \n",
       "L 80.256229 106.53253 \n",
       "L 79.135747 106.53253 \n",
       "L 78.015265 106.53253 \n",
       "L 76.894784 106.53253 \n",
       "L 75.774302 106.53253 \n",
       "L 74.65382 106.53253 \n",
       "L 73.533338 106.53253 \n",
       "L 72.412856 106.53253 \n",
       "L 71.292374 106.53253 \n",
       "L 70.171892 106.53253 \n",
       "L 69.05141 106.53253 \n",
       "L 67.930928 106.53253 \n",
       "L 66.810446 106.53253 \n",
       "L 65.689964 106.53253 \n",
       "L 64.569482 106.53253 \n",
       "L 63.449 106.53253 \n",
       "L 62.328518 106.53253 \n",
       "L 61.208037 106.53253 \n",
       "L 60.087555 106.53253 \n",
       "L 58.967073 106.53253 \n",
       "L 57.846591 106.53253 \n",
       "L 56.726109 106.53253 \n",
       "L 55.605627 106.53253 \n",
       "L 54.485145 106.53253 \n",
       "L 53.364663 106.53253 \n",
       "L 52.244181 106.53253 \n",
       "L 51.123699 106.53253 \n",
       "L 50.003217 106.53253 \n",
       "L 48.882735 106.53253 \n",
       "L 47.762253 106.53253 \n",
       "L 46.641771 106.53253 \n",
       "L 45.52129 106.53253 \n",
       "L 44.400808 106.53253 \n",
       "L 43.280326 106.53253 \n",
       "L 42.159844 106.53253 \n",
       "L 42.159844 107.624096 \n",
       "L 42.159844 108.715663 \n",
       "L 42.159844 109.807229 \n",
       "L 42.159844 110.898795 \n",
       "L 42.159844 111.990361 \n",
       "L 42.159844 113.081928 \n",
       "L 42.159844 114.173494 \n",
       "L 42.159844 115.26506 \n",
       "L 42.159844 116.356627 \n",
       "L 42.159844 117.448193 \n",
       "L 42.159844 118.539759 \n",
       "L 42.159844 119.631325 \n",
       "L 42.159844 120.722892 \n",
       "L 42.159844 121.814458 \n",
       "L 42.159844 122.906024 \n",
       "L 42.159844 123.99759 \n",
       "L 42.159844 125.089157 \n",
       "L 42.159844 126.180723 \n",
       "L 42.159844 127.272289 \n",
       "L 42.159844 128.363855 \n",
       "L 42.159844 129.455422 \n",
       "L 42.159844 130.546988 \n",
       "L 42.159844 131.638554 \n",
       "L 42.159844 132.73012 \n",
       "L 42.159844 133.821687 \n",
       "L 42.159844 134.913253 \n",
       "L 42.159844 136.004819 \n",
       "L 42.159844 137.096386 \n",
       "L 42.159844 138.187952 \n",
       "L 42.159844 139.279518 \n",
       "L 42.159844 140.371084 \n",
       "L 42.159844 141.462651 \n",
       "L 42.159844 142.554217 \n",
       "L 42.159844 143.645783 \n",
       "L 42.159844 144.737349 \n",
       "L 42.159844 145.828916 \n",
       "L 42.159844 146.920482 \n",
       "L 42.159844 148.012048 \n",
       "L 42.159844 149.103614 \n",
       "L 42.159844 150.195181 \n",
       "L 42.159844 151.286747 \n",
       "L 42.159844 152.378313 \n",
       "L 42.159844 153.46988 \n",
       "L 42.159844 154.561446 \n",
       "L 42.159844 155.653012 \n",
       "L 42.159844 156.744578 \n",
       "L 42.159844 157.836145 \n",
       "L 42.159844 158.927711 \n",
       "L 42.159844 160.019277 \n",
       "L 42.159844 161.110843 \n",
       "L 42.159844 162.20241 \n",
       "L 42.159844 163.293976 \n",
       "L 42.159844 164.385542 \n",
       "L 42.159844 165.477108 \n",
       "L 42.159844 166.568675 \n",
       "L 42.159844 167.660241 \n",
       "L 42.159844 168.751807 \n",
       "L 42.159844 169.843373 \n",
       "L 42.159844 170.93494 \n",
       "L 42.159844 172.026506 \n",
       "L 42.159844 173.118072 \n",
       "L 42.159844 174.209639 \n",
       "L 42.159844 175.301205 \n",
       "L 42.159844 176.392771 \n",
       "L 42.159844 177.484337 \n",
       "L 42.159844 178.575904 \n",
       "L 42.159844 179.66747 \n",
       "L 42.159844 180.759036 \n",
       "L 42.159844 181.850602 \n",
       "L 42.159844 182.942169 \n",
       "L 42.159844 184.033735 \n",
       "L 42.159844 185.125301 \n",
       "L 42.159844 186.216867 \n",
       "L 42.159844 187.308434 \n",
       "L 42.159844 188.4 \n",
       "L 42.159844 189.491566 \n",
       "L 42.159844 190.583133 \n",
       "L 42.159844 191.674699 \n",
       "L 42.159844 192.766265 \n",
       "L 42.159844 193.857831 \n",
       "L 42.159844 194.949398 \n",
       "L 42.159844 195.222289 \n",
       "z\n",
       "\" style=\"fill:#9467bd;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_5\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 225.91888 195.495181 \n",
       "L 227.039362 195.495181 \n",
       "L 228.159844 195.495181 \n",
       "L 229.280326 195.495181 \n",
       "L 230.400808 195.495181 \n",
       "L 231.52129 195.495181 \n",
       "L 232.641771 195.495181 \n",
       "L 233.762253 195.495181 \n",
       "L 234.882735 195.495181 \n",
       "L 236.003217 195.495181 \n",
       "L 237.123699 195.495181 \n",
       "L 238.244181 195.495181 \n",
       "L 239.364663 195.495181 \n",
       "L 240.485145 195.495181 \n",
       "L 241.605627 195.495181 \n",
       "L 242.726109 195.495181 \n",
       "L 243.846591 195.495181 \n",
       "L 244.967073 195.495181 \n",
       "L 246.087555 195.495181 \n",
       "L 247.208037 195.495181 \n",
       "L 248.328518 195.495181 \n",
       "L 249.449 195.495181 \n",
       "L 250.569482 195.495181 \n",
       "L 251.689964 195.495181 \n",
       "L 252.810446 195.495181 \n",
       "L 253.930928 195.495181 \n",
       "L 255.05141 195.495181 \n",
       "L 256.171892 195.495181 \n",
       "L 257.292374 195.495181 \n",
       "L 258.412856 195.495181 \n",
       "L 259.533338 195.495181 \n",
       "L 260.65382 195.495181 \n",
       "L 261.774302 195.495181 \n",
       "L 262.894784 195.495181 \n",
       "L 264.015265 195.495181 \n",
       "L 265.135747 195.495181 \n",
       "L 266.256229 195.495181 \n",
       "L 267.376711 195.495181 \n",
       "L 268.497193 195.495181 \n",
       "L 269.617675 195.495181 \n",
       "L 270.738157 195.495181 \n",
       "L 271.858639 195.495181 \n",
       "L 272.979121 195.495181 \n",
       "L 274.099603 195.495181 \n",
       "L 275.220085 195.495181 \n",
       "L 276.340567 195.495181 \n",
       "L 277.461049 195.495181 \n",
       "L 278.58153 195.495181 \n",
       "L 279.702012 195.495181 \n",
       "L 280.822494 195.495181 \n",
       "L 281.942976 195.495181 \n",
       "L 283.063458 195.495181 \n",
       "L 284.18394 195.495181 \n",
       "L 285.304422 195.495181 \n",
       "L 286.424904 195.495181 \n",
       "L 287.545386 195.495181 \n",
       "L 288.665868 195.495181 \n",
       "L 289.78635 195.495181 \n",
       "L 290.906832 195.495181 \n",
       "L 292.027314 195.495181 \n",
       "L 293.147796 195.495181 \n",
       "L 294.268277 195.495181 \n",
       "L 295.388759 195.495181 \n",
       "L 296.509241 195.495181 \n",
       "L 297.629723 195.495181 \n",
       "L 298.750205 195.495181 \n",
       "L 299.870687 195.495181 \n",
       "L 300.991169 195.495181 \n",
       "L 302.111651 195.495181 \n",
       "L 303.232133 195.495181 \n",
       "L 304.352615 195.495181 \n",
       "L 305.473097 195.495181 \n",
       "L 306.593579 195.495181 \n",
       "L 307.714061 195.495181 \n",
       "L 308.834543 195.495181 \n",
       "L 309.955024 195.495181 \n",
       "L 311.075506 195.495181 \n",
       "L 312.195988 195.495181 \n",
       "L 313.31647 195.495181 \n",
       "L 314.436952 195.495181 \n",
       "L 315.557434 195.495181 \n",
       "L 316.677916 195.495181 \n",
       "L 317.798398 195.495181 \n",
       "L 318.91888 195.495181 \n",
       "L 320.039362 195.495181 \n",
       "L 321.159844 195.495181 \n",
       "L 321.159844 195.358735 \n",
       "L 320.039362 195.358735 \n",
       "L 318.91888 195.358735 \n",
       "L 317.798398 195.358735 \n",
       "L 316.677916 195.358735 \n",
       "L 315.557434 195.358735 \n",
       "L 314.436952 195.358735 \n",
       "L 313.31647 195.358735 \n",
       "L 312.195988 195.358735 \n",
       "L 311.075506 195.358735 \n",
       "L 309.955024 195.358735 \n",
       "L 308.834543 195.358735 \n",
       "L 307.714061 195.358735 \n",
       "L 306.593579 195.358735 \n",
       "L 305.473097 195.358735 \n",
       "L 304.352615 195.358735 \n",
       "L 303.232133 195.358735 \n",
       "L 302.111651 195.358735 \n",
       "L 300.991169 195.358735 \n",
       "L 299.870687 195.358735 \n",
       "L 298.750205 195.358735 \n",
       "L 297.629723 195.358735 \n",
       "L 296.509241 195.358735 \n",
       "L 295.388759 195.358735 \n",
       "L 294.268277 195.358735 \n",
       "L 293.147796 195.358735 \n",
       "L 292.027314 195.358735 \n",
       "L 290.906832 195.358735 \n",
       "L 289.78635 195.358735 \n",
       "L 288.665868 195.358735 \n",
       "L 287.545386 195.358735 \n",
       "L 286.424904 195.358735 \n",
       "L 285.304422 195.358735 \n",
       "L 284.18394 195.358735 \n",
       "L 283.063458 195.358735 \n",
       "L 281.942976 195.358735 \n",
       "L 280.822494 195.358735 \n",
       "L 279.702012 195.358735 \n",
       "L 278.58153 195.358735 \n",
       "L 277.461049 195.358735 \n",
       "L 276.340567 195.358735 \n",
       "L 275.220085 195.358735 \n",
       "L 274.099603 195.358735 \n",
       "L 272.979121 195.358735 \n",
       "L 271.858639 195.358735 \n",
       "L 270.738157 195.358735 \n",
       "L 269.617675 195.358735 \n",
       "L 268.497193 195.358735 \n",
       "L 267.376711 195.358735 \n",
       "L 266.256229 195.358735 \n",
       "L 265.135747 195.358735 \n",
       "L 264.015265 195.358735 \n",
       "L 262.894784 195.358735 \n",
       "L 261.774302 195.358735 \n",
       "L 260.65382 195.358735 \n",
       "L 259.533338 195.358735 \n",
       "L 258.412856 195.358735 \n",
       "L 257.292374 195.358735 \n",
       "L 256.171892 195.358735 \n",
       "L 255.05141 195.358735 \n",
       "L 253.930928 195.358735 \n",
       "L 252.810446 195.358735 \n",
       "L 251.689964 195.358735 \n",
       "L 250.569482 195.358735 \n",
       "L 249.449 195.358735 \n",
       "L 248.328518 195.358735 \n",
       "L 247.208037 195.358735 \n",
       "L 246.087555 195.358735 \n",
       "L 244.967073 195.358735 \n",
       "L 243.846591 195.358735 \n",
       "L 242.726109 195.358735 \n",
       "L 241.605627 195.358735 \n",
       "L 240.485145 195.358735 \n",
       "L 239.364663 195.358735 \n",
       "L 238.244181 195.358735 \n",
       "L 237.123699 195.358735 \n",
       "L 236.003217 195.358735 \n",
       "L 234.882735 195.358735 \n",
       "L 233.762253 195.358735 \n",
       "L 232.641771 195.358735 \n",
       "L 231.52129 195.358735 \n",
       "L 230.400808 195.358735 \n",
       "L 229.280326 195.358735 \n",
       "L 228.159844 195.358735 \n",
       "L 227.039362 195.358735 \n",
       "L 225.91888 195.358735 \n",
       "L 225.078518 194.949398 \n",
       "L 225.078518 193.857831 \n",
       "L 225.078518 192.766265 \n",
       "L 225.078518 191.674699 \n",
       "L 225.078518 190.583133 \n",
       "L 225.078518 189.491566 \n",
       "L 225.078518 188.4 \n",
       "L 225.078518 187.308434 \n",
       "L 225.078518 186.216867 \n",
       "L 225.078518 185.125301 \n",
       "L 225.078518 184.033735 \n",
       "L 225.078518 182.942169 \n",
       "L 225.078518 181.850602 \n",
       "L 225.078518 180.759036 \n",
       "L 225.078518 179.66747 \n",
       "L 225.078518 178.575904 \n",
       "L 225.078518 177.484337 \n",
       "L 225.078518 176.392771 \n",
       "L 225.078518 175.301205 \n",
       "L 225.078518 174.209639 \n",
       "L 225.078518 173.118072 \n",
       "L 225.078518 172.026506 \n",
       "L 225.078518 170.93494 \n",
       "L 225.078518 169.843373 \n",
       "L 225.078518 168.751807 \n",
       "L 225.078518 167.660241 \n",
       "L 225.078518 166.568675 \n",
       "L 225.078518 165.477108 \n",
       "L 225.078518 164.385542 \n",
       "L 225.078518 163.293976 \n",
       "L 225.078518 162.20241 \n",
       "L 225.078518 161.110843 \n",
       "L 225.078518 160.019277 \n",
       "L 225.078518 158.927711 \n",
       "L 225.078518 157.836145 \n",
       "L 225.078518 156.744578 \n",
       "L 225.078518 155.653012 \n",
       "L 225.078518 154.561446 \n",
       "L 225.078518 153.46988 \n",
       "L 225.078518 152.378313 \n",
       "L 225.078518 151.286747 \n",
       "L 225.078518 150.195181 \n",
       "L 225.078518 149.103614 \n",
       "L 225.078518 148.012048 \n",
       "L 225.078518 146.920482 \n",
       "L 225.078518 145.828916 \n",
       "L 225.078518 144.737349 \n",
       "L 225.078518 143.645783 \n",
       "L 225.078518 142.554217 \n",
       "L 225.078518 141.462651 \n",
       "L 225.078518 140.371084 \n",
       "L 225.078518 139.279518 \n",
       "L 225.078518 138.187952 \n",
       "L 225.078518 137.096386 \n",
       "L 225.078518 136.004819 \n",
       "L 225.078518 134.913253 \n",
       "L 225.078518 133.821687 \n",
       "L 225.078518 132.73012 \n",
       "L 225.078518 131.638554 \n",
       "L 225.078518 130.546988 \n",
       "L 225.078518 129.455422 \n",
       "L 225.078518 128.363855 \n",
       "L 225.078518 127.272289 \n",
       "L 225.078518 126.180723 \n",
       "L 225.078518 125.089157 \n",
       "L 225.078518 123.99759 \n",
       "L 225.078518 122.906024 \n",
       "L 225.078518 121.814458 \n",
       "L 225.078518 120.722892 \n",
       "L 225.078518 119.631325 \n",
       "L 225.078518 118.539759 \n",
       "L 225.078518 117.448193 \n",
       "L 225.91888 116.629518 \n",
       "L 227.039362 116.629518 \n",
       "L 228.159844 116.629518 \n",
       "L 229.280326 116.629518 \n",
       "L 230.400808 116.629518 \n",
       "L 231.52129 116.629518 \n",
       "L 232.641771 116.629518 \n",
       "L 233.762253 116.629518 \n",
       "L 234.882735 116.629518 \n",
       "L 236.003217 116.629518 \n",
       "L 237.123699 116.629518 \n",
       "L 238.244181 116.629518 \n",
       "L 239.364663 116.629518 \n",
       "L 240.485145 116.629518 \n",
       "L 241.605627 116.629518 \n",
       "L 242.726109 116.629518 \n",
       "L 243.846591 116.629518 \n",
       "L 244.967073 116.629518 \n",
       "L 246.087555 116.629518 \n",
       "L 247.208037 116.629518 \n",
       "L 248.328518 116.629518 \n",
       "L 249.449 116.629518 \n",
       "L 250.569482 116.629518 \n",
       "L 251.689964 116.629518 \n",
       "L 252.810446 116.629518 \n",
       "L 253.930928 116.629518 \n",
       "L 255.05141 116.629518 \n",
       "L 256.171892 116.629518 \n",
       "L 257.292374 116.629518 \n",
       "L 258.412856 116.629518 \n",
       "L 259.533338 116.629518 \n",
       "L 260.65382 116.629518 \n",
       "L 261.774302 116.629518 \n",
       "L 262.894784 116.629518 \n",
       "L 264.015265 116.629518 \n",
       "L 265.135747 116.629518 \n",
       "L 266.256229 116.629518 \n",
       "L 267.376711 116.629518 \n",
       "L 268.497193 116.629518 \n",
       "L 269.617675 116.629518 \n",
       "L 270.738157 116.629518 \n",
       "L 271.858639 116.629518 \n",
       "L 272.979121 116.629518 \n",
       "L 274.099603 116.629518 \n",
       "L 275.220085 116.629518 \n",
       "L 276.340567 116.629518 \n",
       "L 277.461049 116.629518 \n",
       "L 278.58153 116.629518 \n",
       "L 279.702012 116.629518 \n",
       "L 280.822494 116.629518 \n",
       "L 281.942976 116.629518 \n",
       "L 283.063458 116.629518 \n",
       "L 284.18394 116.629518 \n",
       "L 285.304422 116.629518 \n",
       "L 286.424904 116.629518 \n",
       "L 287.545386 116.629518 \n",
       "L 288.665868 116.629518 \n",
       "L 289.78635 116.629518 \n",
       "L 290.906832 116.629518 \n",
       "L 292.027314 116.629518 \n",
       "L 293.147796 116.629518 \n",
       "L 294.268277 116.629518 \n",
       "L 295.388759 116.629518 \n",
       "L 296.509241 116.629518 \n",
       "L 297.629723 116.629518 \n",
       "L 298.750205 116.629518 \n",
       "L 299.870687 116.629518 \n",
       "L 300.991169 116.629518 \n",
       "L 302.111651 116.629518 \n",
       "L 303.232133 116.629518 \n",
       "L 304.352615 116.629518 \n",
       "L 305.473097 116.629518 \n",
       "L 306.593579 116.629518 \n",
       "L 307.714061 116.629518 \n",
       "L 308.834543 116.629518 \n",
       "L 309.955024 116.629518 \n",
       "L 311.075506 116.629518 \n",
       "L 312.195988 116.629518 \n",
       "L 313.31647 116.629518 \n",
       "L 314.436952 116.629518 \n",
       "L 315.557434 116.629518 \n",
       "L 316.677916 116.629518 \n",
       "L 317.798398 116.629518 \n",
       "L 318.91888 116.629518 \n",
       "L 320.039362 116.629518 \n",
       "L 321.159844 116.629518 \n",
       "L 321.159844 116.356627 \n",
       "L 320.039362 116.356627 \n",
       "L 318.91888 116.356627 \n",
       "L 317.798398 116.356627 \n",
       "L 316.677916 116.356627 \n",
       "L 315.557434 116.356627 \n",
       "L 314.436952 116.356627 \n",
       "L 313.31647 116.356627 \n",
       "L 312.195988 116.356627 \n",
       "L 311.075506 116.356627 \n",
       "L 309.955024 116.356627 \n",
       "L 308.834543 116.356627 \n",
       "L 307.714061 116.356627 \n",
       "L 306.593579 116.356627 \n",
       "L 305.473097 116.356627 \n",
       "L 304.352615 116.356627 \n",
       "L 303.232133 116.356627 \n",
       "L 302.111651 116.356627 \n",
       "L 300.991169 116.356627 \n",
       "L 299.870687 116.356627 \n",
       "L 298.750205 116.356627 \n",
       "L 297.629723 116.356627 \n",
       "L 296.509241 116.356627 \n",
       "L 295.388759 116.356627 \n",
       "L 294.268277 116.356627 \n",
       "L 293.147796 116.356627 \n",
       "L 292.027314 116.356627 \n",
       "L 290.906832 116.356627 \n",
       "L 289.78635 116.356627 \n",
       "L 288.665868 116.356627 \n",
       "L 287.545386 116.356627 \n",
       "L 286.424904 116.356627 \n",
       "L 285.304422 116.356627 \n",
       "L 284.18394 116.356627 \n",
       "L 283.063458 116.356627 \n",
       "L 281.942976 116.356627 \n",
       "L 280.822494 116.356627 \n",
       "L 279.702012 116.356627 \n",
       "L 278.58153 116.356627 \n",
       "L 277.461049 116.356627 \n",
       "L 276.340567 116.356627 \n",
       "L 275.220085 116.356627 \n",
       "L 274.099603 116.356627 \n",
       "L 272.979121 116.356627 \n",
       "L 271.858639 116.356627 \n",
       "L 270.738157 116.356627 \n",
       "L 269.617675 116.356627 \n",
       "L 268.497193 116.356627 \n",
       "L 267.376711 116.356627 \n",
       "L 266.256229 116.356627 \n",
       "L 265.135747 116.356627 \n",
       "L 264.015265 116.356627 \n",
       "L 262.894784 116.356627 \n",
       "L 261.774302 116.356627 \n",
       "L 260.65382 116.356627 \n",
       "L 259.533338 116.356627 \n",
       "L 258.412856 116.356627 \n",
       "L 257.292374 116.356627 \n",
       "L 256.171892 116.356627 \n",
       "L 255.05141 116.356627 \n",
       "L 253.930928 116.356627 \n",
       "L 252.810446 116.356627 \n",
       "L 251.689964 116.356627 \n",
       "L 250.569482 116.356627 \n",
       "L 249.449 116.356627 \n",
       "L 248.328518 116.356627 \n",
       "L 247.208037 116.356627 \n",
       "L 246.087555 116.356627 \n",
       "L 244.967073 116.356627 \n",
       "L 243.846591 116.356627 \n",
       "L 242.726109 116.356627 \n",
       "L 241.605627 116.356627 \n",
       "L 240.485145 116.356627 \n",
       "L 239.364663 116.356627 \n",
       "L 238.244181 116.356627 \n",
       "L 237.123699 116.356627 \n",
       "L 236.003217 116.356627 \n",
       "L 234.882735 116.356627 \n",
       "L 233.762253 116.356627 \n",
       "L 232.641771 116.356627 \n",
       "L 231.52129 116.356627 \n",
       "L 230.400808 116.356627 \n",
       "L 229.280326 116.356627 \n",
       "L 228.159844 116.356627 \n",
       "L 227.039362 116.356627 \n",
       "L 225.91888 116.356627 \n",
       "L 224.798398 117.448193 \n",
       "L 224.798398 118.539759 \n",
       "L 224.798398 119.631325 \n",
       "L 224.798398 120.722892 \n",
       "L 224.798398 121.814458 \n",
       "L 224.798398 122.906024 \n",
       "L 224.798398 123.99759 \n",
       "L 224.798398 125.089157 \n",
       "L 224.798398 126.180723 \n",
       "L 224.798398 127.272289 \n",
       "L 224.798398 128.363855 \n",
       "L 224.798398 129.455422 \n",
       "L 224.798398 130.546988 \n",
       "L 224.798398 131.638554 \n",
       "L 224.798398 132.73012 \n",
       "L 224.798398 133.821687 \n",
       "L 224.798398 134.913253 \n",
       "L 224.798398 136.004819 \n",
       "L 224.798398 137.096386 \n",
       "L 224.798398 138.187952 \n",
       "L 224.798398 139.279518 \n",
       "L 224.798398 140.371084 \n",
       "L 224.798398 141.462651 \n",
       "L 224.798398 142.554217 \n",
       "L 224.798398 143.645783 \n",
       "L 224.798398 144.737349 \n",
       "L 224.798398 145.828916 \n",
       "L 224.798398 146.920482 \n",
       "L 224.798398 148.012048 \n",
       "L 224.798398 149.103614 \n",
       "L 224.798398 150.195181 \n",
       "L 224.798398 151.286747 \n",
       "L 224.798398 152.378313 \n",
       "L 224.798398 153.46988 \n",
       "L 224.798398 154.561446 \n",
       "L 224.798398 155.653012 \n",
       "L 224.798398 156.744578 \n",
       "L 224.798398 157.836145 \n",
       "L 224.798398 158.927711 \n",
       "L 224.798398 160.019277 \n",
       "L 224.798398 161.110843 \n",
       "L 224.798398 162.20241 \n",
       "L 224.798398 163.293976 \n",
       "L 224.798398 164.385542 \n",
       "L 224.798398 165.477108 \n",
       "L 224.798398 166.568675 \n",
       "L 224.798398 167.660241 \n",
       "L 224.798398 168.751807 \n",
       "L 224.798398 169.843373 \n",
       "L 224.798398 170.93494 \n",
       "L 224.798398 172.026506 \n",
       "L 224.798398 173.118072 \n",
       "L 224.798398 174.209639 \n",
       "L 224.798398 175.301205 \n",
       "L 224.798398 176.392771 \n",
       "L 224.798398 177.484337 \n",
       "L 224.798398 178.575904 \n",
       "L 224.798398 179.66747 \n",
       "L 224.798398 180.759036 \n",
       "L 224.798398 181.850602 \n",
       "L 224.798398 182.942169 \n",
       "L 224.798398 184.033735 \n",
       "L 224.798398 185.125301 \n",
       "L 224.798398 186.216867 \n",
       "L 224.798398 187.308434 \n",
       "L 224.798398 188.4 \n",
       "L 224.798398 189.491566 \n",
       "L 224.798398 190.583133 \n",
       "L 224.798398 191.674699 \n",
       "L 224.798398 192.766265 \n",
       "L 224.798398 193.857831 \n",
       "L 224.798398 194.949398 \n",
       "z\n",
       "\" style=\"fill:#8c564b;fill-opacity:0.15;\"/>\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 43.280326 106.53253 \n",
       "L 44.400808 106.53253 \n",
       "L 45.52129 106.53253 \n",
       "L 46.641771 106.53253 \n",
       "L 47.762253 106.53253 \n",
       "L 48.882735 106.53253 \n",
       "L 50.003217 106.53253 \n",
       "L 51.123699 106.53253 \n",
       "L 52.244181 106.53253 \n",
       "L 53.364663 106.53253 \n",
       "L 54.485145 106.53253 \n",
       "L 55.605627 106.53253 \n",
       "L 56.726109 106.53253 \n",
       "L 57.846591 106.53253 \n",
       "L 58.967073 106.53253 \n",
       "L 60.087555 106.53253 \n",
       "L 61.208037 106.53253 \n",
       "L 62.328518 106.53253 \n",
       "L 63.449 106.53253 \n",
       "L 64.569482 106.53253 \n",
       "L 65.689964 106.53253 \n",
       "L 66.810446 106.53253 \n",
       "L 67.930928 106.53253 \n",
       "L 69.05141 106.53253 \n",
       "L 70.171892 106.53253 \n",
       "L 71.292374 106.53253 \n",
       "L 72.412856 106.53253 \n",
       "L 73.533338 106.53253 \n",
       "L 74.65382 106.53253 \n",
       "L 75.774302 106.53253 \n",
       "L 76.894784 106.53253 \n",
       "L 78.015265 106.53253 \n",
       "L 79.135747 106.53253 \n",
       "L 80.256229 106.53253 \n",
       "L 81.376711 106.53253 \n",
       "L 82.497193 106.53253 \n",
       "L 83.617675 106.53253 \n",
       "L 84.738157 106.53253 \n",
       "L 85.858639 106.53253 \n",
       "L 86.979121 106.53253 \n",
       "L 88.099603 106.53253 \n",
       "L 89.220085 106.53253 \n",
       "L 90.340567 106.53253 \n",
       "L 91.461049 106.53253 \n",
       "L 92.58153 106.53253 \n",
       "L 93.702012 106.53253 \n",
       "L 94.822494 106.53253 \n",
       "L 95.942976 106.53253 \n",
       "L 97.063458 106.53253 \n",
       "L 98.18394 106.53253 \n",
       "L 99.304422 106.53253 \n",
       "L 100.424904 106.53253 \n",
       "L 101.545386 106.53253 \n",
       "L 102.665868 106.53253 \n",
       "L 103.78635 106.53253 \n",
       "L 104.906832 106.53253 \n",
       "L 106.027314 106.53253 \n",
       "L 107.147796 106.53253 \n",
       "L 108.268277 106.53253 \n",
       "L 109.388759 106.53253 \n",
       "L 110.509241 106.53253 \n",
       "L 111.629723 106.53253 \n",
       "L 112.750205 106.53253 \n",
       "L 113.870687 106.53253 \n",
       "L 114.991169 106.53253 \n",
       "L 116.111651 106.53253 \n",
       "L 117.232133 106.53253 \n",
       "L 118.352615 106.53253 \n",
       "L 119.473097 106.53253 \n",
       "L 120.593579 106.53253 \n",
       "L 121.714061 106.53253 \n",
       "L 122.834543 106.53253 \n",
       "L 123.955024 106.53253 \n",
       "L 125.075506 106.53253 \n",
       "L 126.195988 106.53253 \n",
       "L 127.31647 106.53253 \n",
       "L 128.436952 106.53253 \n",
       "L 129.557434 106.53253 \n",
       "L 130.677916 106.53253 \n",
       "L 131.798398 106.53253 \n",
       "L 132.91888 106.53253 \n",
       "L 134.039362 106.53253 \n",
       "L 135.159844 106.53253 \n",
       "L 136.280326 106.53253 \n",
       "L 137.400808 106.53253 \n",
       "L 138.52129 106.53253 \n",
       "L 139.641771 106.53253 \n",
       "L 140.762253 106.53253 \n",
       "L 141.882735 106.53253 \n",
       "L 143.003217 106.53253 \n",
       "L 144.123699 106.53253 \n",
       "L 145.244181 106.53253 \n",
       "L 146.364663 106.53253 \n",
       "L 147.485145 106.53253 \n",
       "L 148.605627 106.53253 \n",
       "L 149.726109 106.53253 \n",
       "L 150.846591 106.53253 \n",
       "L 151.967073 106.53253 \n",
       "L 153.087555 106.53253 \n",
       "L 154.208037 106.53253 \n",
       "L 155.328518 106.53253 \n",
       "L 156.449 106.53253 \n",
       "L 157.569482 106.53253 \n",
       "L 158.689964 106.53253 \n",
       "L 159.810446 106.53253 \n",
       "L 160.930928 106.53253 \n",
       "L 162.05141 106.53253 \n",
       "L 163.171892 106.53253 \n",
       "L 164.292374 106.53253 \n",
       "L 165.412856 106.53253 \n",
       "L 166.533338 106.53253 \n",
       "L 167.65382 106.53253 \n",
       "L 168.774302 106.53253 \n",
       "L 169.894784 106.53253 \n",
       "L 171.015265 106.53253 \n",
       "L 172.135747 106.53253 \n",
       "L 173.256229 106.53253 \n",
       "L 174.376711 106.53253 \n",
       "L 175.497193 106.53253 \n",
       "L 176.617675 106.53253 \n",
       "L 177.738157 106.53253 \n",
       "L 178.858639 106.53253 \n",
       "L 179.979121 106.53253 \n",
       "L 181.099603 106.53253 \n",
       "L 182.220085 106.53253 \n",
       "L 183.340567 106.53253 \n",
       "L 184.461049 106.53253 \n",
       "L 185.58153 106.53253 \n",
       "L 186.702012 106.53253 \n",
       "L 187.822494 106.53253 \n",
       "L 188.942976 106.53253 \n",
       "L 190.063458 106.53253 \n",
       "L 191.18394 106.53253 \n",
       "L 192.304422 106.53253 \n",
       "L 193.424904 106.53253 \n",
       "L 194.545386 106.53253 \n",
       "L 195.665868 106.53253 \n",
       "L 196.78635 106.53253 \n",
       "L 197.906832 106.53253 \n",
       "L 199.027314 106.53253 \n",
       "L 200.147796 106.53253 \n",
       "L 201.268277 106.53253 \n",
       "L 202.388759 106.53253 \n",
       "L 203.509241 106.53253 \n",
       "L 204.629723 106.53253 \n",
       "L 205.750205 106.53253 \n",
       "L 206.870687 106.53253 \n",
       "L 207.991169 106.53253 \n",
       "L 209.111651 106.53253 \n",
       "L 210.232133 106.53253 \n",
       "L 211.352615 106.53253 \n",
       "L 212.473097 106.53253 \n",
       "L 213.593579 106.53253 \n",
       "L 214.714061 106.53253 \n",
       "L 215.834543 106.53253 \n",
       "L 216.955024 106.53253 \n",
       "L 218.075506 106.53253 \n",
       "L 219.195988 106.53253 \n",
       "L 220.31647 106.53253 \n",
       "L 221.436952 106.53253 \n",
       "L 222.557434 106.53253 \n",
       "L 223.677916 106.53253 \n",
       "L 224.798398 106.53253 \n",
       "L 225.91888 105.440964 \n",
       "L 225.91888 104.349398 \n",
       "L 225.91888 103.257831 \n",
       "L 225.91888 102.166265 \n",
       "L 225.91888 101.074699 \n",
       "L 225.91888 99.983133 \n",
       "L 225.91888 98.891566 \n",
       "L 225.91888 97.8 \n",
       "L 225.91888 96.708434 \n",
       "L 227.039362 96.708434 \n",
       "L 228.159844 96.708434 \n",
       "L 229.280326 96.708434 \n",
       "L 230.400808 96.708434 \n",
       "L 231.52129 96.708434 \n",
       "L 232.641771 96.708434 \n",
       "L 233.762253 96.708434 \n",
       "L 234.882735 96.708434 \n",
       "L 236.003217 96.708434 \n",
       "L 237.123699 96.708434 \n",
       "L 238.244181 96.708434 \n",
       "L 239.364663 96.708434 \n",
       "L 240.485145 96.708434 \n",
       "L 241.605627 96.708434 \n",
       "L 242.726109 96.708434 \n",
       "L 243.846591 96.708434 \n",
       "L 244.967073 96.708434 \n",
       "L 246.087555 96.708434 \n",
       "L 247.208037 96.708434 \n",
       "L 248.328518 96.708434 \n",
       "L 249.449 96.708434 \n",
       "L 250.569482 96.708434 \n",
       "L 251.689964 96.708434 \n",
       "L 252.810446 96.708434 \n",
       "L 253.930928 96.708434 \n",
       "L 255.05141 96.708434 \n",
       "L 256.171892 96.708434 \n",
       "L 257.292374 96.708434 \n",
       "L 258.412856 96.708434 \n",
       "L 259.533338 96.708434 \n",
       "L 260.65382 96.708434 \n",
       "L 261.774302 96.708434 \n",
       "L 262.894784 96.708434 \n",
       "L 264.015265 96.708434 \n",
       "L 265.135747 96.708434 \n",
       "L 266.256229 96.708434 \n",
       "L 267.376711 96.708434 \n",
       "L 268.497193 96.708434 \n",
       "L 269.617675 96.708434 \n",
       "L 270.738157 96.708434 \n",
       "L 271.858639 96.708434 \n",
       "L 272.979121 96.708434 \n",
       "L 274.099603 96.708434 \n",
       "L 275.220085 96.708434 \n",
       "L 276.340567 96.708434 \n",
       "L 277.461049 96.708434 \n",
       "L 278.58153 96.708434 \n",
       "L 279.702012 96.708434 \n",
       "L 280.822494 96.708434 \n",
       "L 281.942976 96.708434 \n",
       "L 283.063458 96.708434 \n",
       "L 284.18394 96.708434 \n",
       "L 285.304422 96.708434 \n",
       "L 286.424904 96.708434 \n",
       "L 287.545386 96.708434 \n",
       "L 288.665868 96.708434 \n",
       "L 289.78635 96.708434 \n",
       "L 290.906832 96.708434 \n",
       "L 292.027314 96.708434 \n",
       "L 293.147796 96.708434 \n",
       "L 294.268277 96.708434 \n",
       "L 295.388759 96.708434 \n",
       "L 296.509241 96.708434 \n",
       "L 297.629723 96.708434 \n",
       "L 298.750205 96.708434 \n",
       "L 299.870687 96.708434 \n",
       "L 300.991169 96.708434 \n",
       "L 302.111651 96.708434 \n",
       "L 303.232133 96.708434 \n",
       "L 304.352615 96.708434 \n",
       "L 305.473097 96.708434 \n",
       "L 306.593579 96.708434 \n",
       "L 307.714061 96.708434 \n",
       "L 308.834543 96.708434 \n",
       "L 309.955024 96.708434 \n",
       "L 311.075506 96.708434 \n",
       "L 312.195988 96.708434 \n",
       "L 313.31647 96.708434 \n",
       "L 314.436952 96.708434 \n",
       "L 315.557434 96.708434 \n",
       "L 316.677916 96.708434 \n",
       "L 317.798398 96.708434 \n",
       "L 318.91888 96.708434 \n",
       "L 320.039362 96.708434 \n",
       "L 321.159844 96.708434 \n",
       "L 321.159844 96.435542 \n",
       "L 320.039362 96.435542 \n",
       "L 318.91888 96.435542 \n",
       "L 317.798398 96.435542 \n",
       "L 316.677916 96.435542 \n",
       "L 315.557434 96.435542 \n",
       "L 314.436952 96.435542 \n",
       "L 313.31647 96.435542 \n",
       "L 312.195988 96.435542 \n",
       "L 311.075506 96.435542 \n",
       "L 309.955024 96.435542 \n",
       "L 308.834543 96.435542 \n",
       "L 307.714061 96.435542 \n",
       "L 306.593579 96.435542 \n",
       "L 305.473097 96.435542 \n",
       "L 304.352615 96.435542 \n",
       "L 303.232133 96.435542 \n",
       "L 302.111651 96.435542 \n",
       "L 300.991169 96.435542 \n",
       "L 299.870687 96.435542 \n",
       "L 298.750205 96.435542 \n",
       "L 297.629723 96.435542 \n",
       "L 296.509241 96.435542 \n",
       "L 295.388759 96.435542 \n",
       "L 294.268277 96.435542 \n",
       "L 293.147796 96.435542 \n",
       "L 292.027314 96.435542 \n",
       "L 290.906832 96.435542 \n",
       "L 289.78635 96.435542 \n",
       "L 288.665868 96.435542 \n",
       "L 287.545386 96.435542 \n",
       "L 286.424904 96.435542 \n",
       "L 285.304422 96.435542 \n",
       "L 284.18394 96.435542 \n",
       "L 283.063458 96.435542 \n",
       "L 281.942976 96.435542 \n",
       "L 280.822494 96.435542 \n",
       "L 279.702012 96.435542 \n",
       "L 278.58153 96.435542 \n",
       "L 277.461049 96.435542 \n",
       "L 276.340567 96.435542 \n",
       "L 275.220085 96.435542 \n",
       "L 274.099603 96.435542 \n",
       "L 272.979121 96.435542 \n",
       "L 271.858639 96.435542 \n",
       "L 270.738157 96.435542 \n",
       "L 269.617675 96.435542 \n",
       "L 268.497193 96.435542 \n",
       "L 267.376711 96.435542 \n",
       "L 266.256229 96.435542 \n",
       "L 265.135747 96.435542 \n",
       "L 264.015265 96.435542 \n",
       "L 262.894784 96.435542 \n",
       "L 261.774302 96.435542 \n",
       "L 260.65382 96.435542 \n",
       "L 259.533338 96.435542 \n",
       "L 258.412856 96.435542 \n",
       "L 257.292374 96.435542 \n",
       "L 256.171892 96.435542 \n",
       "L 255.05141 96.435542 \n",
       "L 253.930928 96.435542 \n",
       "L 252.810446 96.435542 \n",
       "L 251.689964 96.435542 \n",
       "L 250.569482 96.435542 \n",
       "L 249.449 96.435542 \n",
       "L 248.328518 96.435542 \n",
       "L 247.208037 96.435542 \n",
       "L 246.087555 96.435542 \n",
       "L 244.967073 96.435542 \n",
       "L 243.846591 96.435542 \n",
       "L 242.726109 96.435542 \n",
       "L 241.605627 96.435542 \n",
       "L 240.485145 96.435542 \n",
       "L 239.364663 96.435542 \n",
       "L 238.244181 96.435542 \n",
       "L 237.123699 96.435542 \n",
       "L 236.003217 96.435542 \n",
       "L 234.882735 96.435542 \n",
       "L 233.762253 96.435542 \n",
       "L 232.641771 96.435542 \n",
       "L 231.52129 96.435542 \n",
       "L 230.400808 96.435542 \n",
       "L 229.280326 96.435542 \n",
       "L 228.159844 96.435542 \n",
       "L 227.039362 96.435542 \n",
       "L 225.91888 96.435542 \n",
       "L 225.638759 96.708434 \n",
       "L 225.638759 97.8 \n",
       "L 225.638759 98.891566 \n",
       "L 225.638759 99.983133 \n",
       "L 225.638759 101.074699 \n",
       "L 225.638759 102.166265 \n",
       "L 225.638759 103.257831 \n",
       "L 225.638759 104.349398 \n",
       "L 225.638759 105.440964 \n",
       "L 224.798398 106.259639 \n",
       "L 223.677916 106.259639 \n",
       "L 222.557434 106.259639 \n",
       "L 221.436952 106.259639 \n",
       "L 220.31647 106.259639 \n",
       "L 219.195988 106.259639 \n",
       "L 218.075506 106.259639 \n",
       "L 216.955024 106.259639 \n",
       "L 215.834543 106.259639 \n",
       "L 214.714061 106.259639 \n",
       "L 213.593579 106.259639 \n",
       "L 212.473097 106.259639 \n",
       "L 211.352615 106.259639 \n",
       "L 210.232133 106.259639 \n",
       "L 209.111651 106.259639 \n",
       "L 207.991169 106.259639 \n",
       "L 206.870687 106.259639 \n",
       "L 205.750205 106.259639 \n",
       "L 204.629723 106.259639 \n",
       "L 203.509241 106.259639 \n",
       "L 202.388759 106.259639 \n",
       "L 201.268277 106.259639 \n",
       "L 200.147796 106.259639 \n",
       "L 199.027314 106.259639 \n",
       "L 197.906832 106.259639 \n",
       "L 196.78635 106.259639 \n",
       "L 195.665868 106.259639 \n",
       "L 194.545386 106.259639 \n",
       "L 193.424904 106.259639 \n",
       "L 192.304422 106.259639 \n",
       "L 191.18394 106.259639 \n",
       "L 190.063458 106.259639 \n",
       "L 188.942976 106.259639 \n",
       "L 187.822494 106.259639 \n",
       "L 186.702012 106.259639 \n",
       "L 185.58153 106.259639 \n",
       "L 184.461049 106.259639 \n",
       "L 183.340567 106.259639 \n",
       "L 182.220085 106.259639 \n",
       "L 181.099603 106.259639 \n",
       "L 179.979121 106.259639 \n",
       "L 178.858639 106.259639 \n",
       "L 177.738157 106.259639 \n",
       "L 176.617675 106.259639 \n",
       "L 175.497193 106.259639 \n",
       "L 174.376711 106.259639 \n",
       "L 173.256229 106.259639 \n",
       "L 172.135747 106.259639 \n",
       "L 171.015265 106.259639 \n",
       "L 169.894784 106.259639 \n",
       "L 168.774302 106.259639 \n",
       "L 167.65382 106.259639 \n",
       "L 166.533338 106.259639 \n",
       "L 165.412856 106.259639 \n",
       "L 164.292374 106.259639 \n",
       "L 163.171892 106.259639 \n",
       "L 162.05141 106.259639 \n",
       "L 160.930928 106.259639 \n",
       "L 159.810446 106.259639 \n",
       "L 158.689964 106.259639 \n",
       "L 157.569482 106.259639 \n",
       "L 156.449 106.259639 \n",
       "L 155.328518 106.259639 \n",
       "L 154.208037 106.259639 \n",
       "L 153.087555 106.259639 \n",
       "L 151.967073 106.259639 \n",
       "L 150.846591 106.259639 \n",
       "L 149.726109 106.259639 \n",
       "L 148.605627 106.259639 \n",
       "L 147.485145 106.259639 \n",
       "L 146.364663 106.259639 \n",
       "L 145.244181 106.259639 \n",
       "L 144.123699 106.259639 \n",
       "L 143.003217 106.259639 \n",
       "L 141.882735 106.259639 \n",
       "L 140.762253 106.259639 \n",
       "L 139.641771 106.259639 \n",
       "L 138.52129 106.259639 \n",
       "L 137.400808 106.259639 \n",
       "L 136.280326 106.259639 \n",
       "L 135.159844 106.259639 \n",
       "L 134.039362 106.259639 \n",
       "L 132.91888 106.259639 \n",
       "L 131.798398 106.259639 \n",
       "L 130.677916 106.259639 \n",
       "L 129.557434 106.259639 \n",
       "L 128.436952 106.259639 \n",
       "L 127.31647 106.259639 \n",
       "L 126.195988 106.259639 \n",
       "L 125.075506 106.259639 \n",
       "L 123.955024 106.259639 \n",
       "L 122.834543 106.259639 \n",
       "L 121.714061 106.259639 \n",
       "L 120.593579 106.259639 \n",
       "L 119.473097 106.259639 \n",
       "L 118.352615 106.259639 \n",
       "L 117.232133 106.259639 \n",
       "L 116.111651 106.259639 \n",
       "L 114.991169 106.259639 \n",
       "L 113.870687 106.259639 \n",
       "L 112.750205 106.259639 \n",
       "L 111.629723 106.259639 \n",
       "L 110.509241 106.259639 \n",
       "L 109.388759 106.259639 \n",
       "L 108.268277 106.259639 \n",
       "L 107.147796 106.259639 \n",
       "L 106.027314 106.259639 \n",
       "L 104.906832 106.259639 \n",
       "L 103.78635 106.259639 \n",
       "L 102.665868 106.259639 \n",
       "L 101.545386 106.259639 \n",
       "L 100.424904 106.259639 \n",
       "L 99.304422 106.259639 \n",
       "L 98.18394 106.259639 \n",
       "L 97.063458 106.259639 \n",
       "L 95.942976 106.259639 \n",
       "L 94.822494 106.259639 \n",
       "L 93.702012 106.259639 \n",
       "L 92.58153 106.259639 \n",
       "L 91.461049 106.259639 \n",
       "L 90.340567 106.259639 \n",
       "L 89.220085 106.259639 \n",
       "L 88.099603 106.259639 \n",
       "L 86.979121 106.259639 \n",
       "L 85.858639 106.259639 \n",
       "L 84.738157 106.259639 \n",
       "L 83.617675 106.259639 \n",
       "L 82.497193 106.259639 \n",
       "L 81.376711 106.259639 \n",
       "L 80.256229 106.259639 \n",
       "L 79.135747 106.259639 \n",
       "L 78.015265 106.259639 \n",
       "L 76.894784 106.259639 \n",
       "L 75.774302 106.259639 \n",
       "L 74.65382 106.259639 \n",
       "L 73.533338 106.259639 \n",
       "L 72.412856 106.259639 \n",
       "L 71.292374 106.259639 \n",
       "L 70.171892 106.259639 \n",
       "L 69.05141 106.259639 \n",
       "L 67.930928 106.259639 \n",
       "L 66.810446 106.259639 \n",
       "L 65.689964 106.259639 \n",
       "L 64.569482 106.259639 \n",
       "L 63.449 106.259639 \n",
       "L 62.328518 106.259639 \n",
       "L 61.208037 106.259639 \n",
       "L 60.087555 106.259639 \n",
       "L 58.967073 106.259639 \n",
       "L 57.846591 106.259639 \n",
       "L 56.726109 106.259639 \n",
       "L 55.605627 106.259639 \n",
       "L 54.485145 106.259639 \n",
       "L 53.364663 106.259639 \n",
       "L 52.244181 106.259639 \n",
       "L 51.123699 106.259639 \n",
       "L 50.003217 106.259639 \n",
       "L 48.882735 106.259639 \n",
       "L 47.762253 106.259639 \n",
       "L 46.641771 106.259639 \n",
       "L 45.52129 106.259639 \n",
       "L 44.400808 106.259639 \n",
       "L 43.280326 106.259639 \n",
       "L 42.159844 106.259639 \n",
       "L 42.159844 106.53253 \n",
       "z\n",
       "\" style=\"fill:#8c564b;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_6\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 225.91888 195.358735 \n",
       "L 227.039362 195.358735 \n",
       "L 228.159844 195.358735 \n",
       "L 229.280326 195.358735 \n",
       "L 230.400808 195.358735 \n",
       "L 231.52129 195.358735 \n",
       "L 232.641771 195.358735 \n",
       "L 233.762253 195.358735 \n",
       "L 234.882735 195.358735 \n",
       "L 236.003217 195.358735 \n",
       "L 237.123699 195.358735 \n",
       "L 238.244181 195.358735 \n",
       "L 239.364663 195.358735 \n",
       "L 240.485145 195.358735 \n",
       "L 241.605627 195.358735 \n",
       "L 242.726109 195.358735 \n",
       "L 243.846591 195.358735 \n",
       "L 244.967073 195.358735 \n",
       "L 246.087555 195.358735 \n",
       "L 247.208037 195.358735 \n",
       "L 248.328518 195.358735 \n",
       "L 249.449 195.358735 \n",
       "L 250.569482 195.358735 \n",
       "L 251.689964 195.358735 \n",
       "L 252.810446 195.358735 \n",
       "L 253.930928 195.358735 \n",
       "L 255.05141 195.358735 \n",
       "L 256.171892 195.358735 \n",
       "L 257.292374 195.358735 \n",
       "L 258.412856 195.358735 \n",
       "L 259.533338 195.358735 \n",
       "L 260.65382 195.358735 \n",
       "L 261.774302 195.358735 \n",
       "L 262.894784 195.358735 \n",
       "L 264.015265 195.358735 \n",
       "L 265.135747 195.358735 \n",
       "L 266.256229 195.358735 \n",
       "L 267.376711 195.358735 \n",
       "L 268.497193 195.358735 \n",
       "L 269.617675 195.358735 \n",
       "L 270.738157 195.358735 \n",
       "L 271.858639 195.358735 \n",
       "L 272.979121 195.358735 \n",
       "L 274.099603 195.358735 \n",
       "L 275.220085 195.358735 \n",
       "L 276.340567 195.358735 \n",
       "L 277.461049 195.358735 \n",
       "L 278.58153 195.358735 \n",
       "L 279.702012 195.358735 \n",
       "L 280.822494 195.358735 \n",
       "L 281.942976 195.358735 \n",
       "L 283.063458 195.358735 \n",
       "L 284.18394 195.358735 \n",
       "L 285.304422 195.358735 \n",
       "L 286.424904 195.358735 \n",
       "L 287.545386 195.358735 \n",
       "L 288.665868 195.358735 \n",
       "L 289.78635 195.358735 \n",
       "L 290.906832 195.358735 \n",
       "L 292.027314 195.358735 \n",
       "L 293.147796 195.358735 \n",
       "L 294.268277 195.358735 \n",
       "L 295.388759 195.358735 \n",
       "L 296.509241 195.358735 \n",
       "L 297.629723 195.358735 \n",
       "L 298.750205 195.358735 \n",
       "L 299.870687 195.358735 \n",
       "L 300.991169 195.358735 \n",
       "L 302.111651 195.358735 \n",
       "L 303.232133 195.358735 \n",
       "L 304.352615 195.358735 \n",
       "L 305.473097 195.358735 \n",
       "L 306.593579 195.358735 \n",
       "L 307.714061 195.358735 \n",
       "L 308.834543 195.358735 \n",
       "L 309.955024 195.358735 \n",
       "L 311.075506 195.358735 \n",
       "L 312.195988 195.358735 \n",
       "L 313.31647 195.358735 \n",
       "L 314.436952 195.358735 \n",
       "L 315.557434 195.358735 \n",
       "L 316.677916 195.358735 \n",
       "L 317.798398 195.358735 \n",
       "L 318.91888 195.358735 \n",
       "L 320.039362 195.358735 \n",
       "L 321.159844 195.358735 \n",
       "L 321.159844 195.222289 \n",
       "L 320.039362 195.222289 \n",
       "L 318.91888 195.222289 \n",
       "L 317.798398 195.222289 \n",
       "L 316.677916 195.222289 \n",
       "L 315.557434 195.222289 \n",
       "L 314.436952 195.222289 \n",
       "L 313.31647 195.222289 \n",
       "L 312.195988 195.222289 \n",
       "L 311.075506 195.222289 \n",
       "L 309.955024 195.222289 \n",
       "L 308.834543 195.222289 \n",
       "L 307.714061 195.222289 \n",
       "L 306.593579 195.222289 \n",
       "L 305.473097 195.222289 \n",
       "L 304.352615 195.222289 \n",
       "L 303.232133 195.222289 \n",
       "L 302.111651 195.222289 \n",
       "L 300.991169 195.222289 \n",
       "L 299.870687 195.222289 \n",
       "L 298.750205 195.222289 \n",
       "L 297.629723 195.222289 \n",
       "L 296.509241 195.222289 \n",
       "L 295.388759 195.222289 \n",
       "L 294.268277 195.222289 \n",
       "L 293.147796 195.222289 \n",
       "L 292.027314 195.222289 \n",
       "L 290.906832 195.222289 \n",
       "L 289.78635 195.222289 \n",
       "L 288.665868 195.222289 \n",
       "L 287.545386 195.222289 \n",
       "L 286.424904 195.222289 \n",
       "L 285.304422 195.222289 \n",
       "L 284.18394 195.222289 \n",
       "L 283.063458 195.222289 \n",
       "L 281.942976 195.222289 \n",
       "L 280.822494 195.222289 \n",
       "L 279.702012 195.222289 \n",
       "L 278.58153 195.222289 \n",
       "L 277.461049 195.222289 \n",
       "L 276.340567 195.222289 \n",
       "L 275.220085 195.222289 \n",
       "L 274.099603 195.222289 \n",
       "L 272.979121 195.222289 \n",
       "L 271.858639 195.222289 \n",
       "L 270.738157 195.222289 \n",
       "L 269.617675 195.222289 \n",
       "L 268.497193 195.222289 \n",
       "L 267.376711 195.222289 \n",
       "L 266.256229 195.222289 \n",
       "L 265.135747 195.222289 \n",
       "L 264.015265 195.222289 \n",
       "L 262.894784 195.222289 \n",
       "L 261.774302 195.222289 \n",
       "L 260.65382 195.222289 \n",
       "L 259.533338 195.222289 \n",
       "L 258.412856 195.222289 \n",
       "L 257.292374 195.222289 \n",
       "L 256.171892 195.222289 \n",
       "L 255.05141 195.222289 \n",
       "L 253.930928 195.222289 \n",
       "L 252.810446 195.222289 \n",
       "L 251.689964 195.222289 \n",
       "L 250.569482 195.222289 \n",
       "L 249.449 195.222289 \n",
       "L 248.328518 195.222289 \n",
       "L 247.208037 195.222289 \n",
       "L 246.087555 195.222289 \n",
       "L 244.967073 195.222289 \n",
       "L 243.846591 195.222289 \n",
       "L 242.726109 195.222289 \n",
       "L 241.605627 195.222289 \n",
       "L 240.485145 195.222289 \n",
       "L 239.364663 195.222289 \n",
       "L 238.244181 195.222289 \n",
       "L 237.123699 195.222289 \n",
       "L 236.003217 195.222289 \n",
       "L 234.882735 195.222289 \n",
       "L 233.762253 195.222289 \n",
       "L 232.641771 195.222289 \n",
       "L 231.52129 195.222289 \n",
       "L 230.400808 195.222289 \n",
       "L 229.280326 195.222289 \n",
       "L 228.159844 195.222289 \n",
       "L 227.039362 195.222289 \n",
       "L 225.91888 195.222289 \n",
       "L 225.358639 194.949398 \n",
       "L 225.358639 193.857831 \n",
       "L 225.358639 192.766265 \n",
       "L 225.358639 191.674699 \n",
       "L 225.358639 190.583133 \n",
       "L 225.358639 189.491566 \n",
       "L 225.358639 188.4 \n",
       "L 225.358639 187.308434 \n",
       "L 225.358639 186.216867 \n",
       "L 225.358639 185.125301 \n",
       "L 225.358639 184.033735 \n",
       "L 225.358639 182.942169 \n",
       "L 225.358639 181.850602 \n",
       "L 225.358639 180.759036 \n",
       "L 225.358639 179.66747 \n",
       "L 225.358639 178.575904 \n",
       "L 225.358639 177.484337 \n",
       "L 225.358639 176.392771 \n",
       "L 225.358639 175.301205 \n",
       "L 225.358639 174.209639 \n",
       "L 225.358639 173.118072 \n",
       "L 225.358639 172.026506 \n",
       "L 225.358639 170.93494 \n",
       "L 225.358639 169.843373 \n",
       "L 225.358639 168.751807 \n",
       "L 225.358639 167.660241 \n",
       "L 225.358639 166.568675 \n",
       "L 225.358639 165.477108 \n",
       "L 225.358639 164.385542 \n",
       "L 225.358639 163.293976 \n",
       "L 225.358639 162.20241 \n",
       "L 225.358639 161.110843 \n",
       "L 225.358639 160.019277 \n",
       "L 225.358639 158.927711 \n",
       "L 225.358639 157.836145 \n",
       "L 225.358639 156.744578 \n",
       "L 225.358639 155.653012 \n",
       "L 225.358639 154.561446 \n",
       "L 225.358639 153.46988 \n",
       "L 225.358639 152.378313 \n",
       "L 225.358639 151.286747 \n",
       "L 225.358639 150.195181 \n",
       "L 225.358639 149.103614 \n",
       "L 225.358639 148.012048 \n",
       "L 225.358639 146.920482 \n",
       "L 225.358639 145.828916 \n",
       "L 225.358639 144.737349 \n",
       "L 225.358639 143.645783 \n",
       "L 225.358639 142.554217 \n",
       "L 225.358639 141.462651 \n",
       "L 225.358639 140.371084 \n",
       "L 225.358639 139.279518 \n",
       "L 225.358639 138.187952 \n",
       "L 225.358639 137.096386 \n",
       "L 225.358639 136.004819 \n",
       "L 225.358639 134.913253 \n",
       "L 225.358639 133.821687 \n",
       "L 225.358639 132.73012 \n",
       "L 225.358639 131.638554 \n",
       "L 225.358639 130.546988 \n",
       "L 225.358639 129.455422 \n",
       "L 225.358639 128.363855 \n",
       "L 225.358639 127.272289 \n",
       "L 225.358639 126.180723 \n",
       "L 225.358639 125.089157 \n",
       "L 225.358639 123.99759 \n",
       "L 225.358639 122.906024 \n",
       "L 225.358639 121.814458 \n",
       "L 225.358639 120.722892 \n",
       "L 225.358639 119.631325 \n",
       "L 225.358639 118.539759 \n",
       "L 225.358639 117.448193 \n",
       "L 225.91888 116.90241 \n",
       "L 227.039362 116.90241 \n",
       "L 228.159844 116.90241 \n",
       "L 229.280326 116.90241 \n",
       "L 230.400808 116.90241 \n",
       "L 231.52129 116.90241 \n",
       "L 232.641771 116.90241 \n",
       "L 233.762253 116.90241 \n",
       "L 234.882735 116.90241 \n",
       "L 236.003217 116.90241 \n",
       "L 237.123699 116.90241 \n",
       "L 238.244181 116.90241 \n",
       "L 239.364663 116.90241 \n",
       "L 240.485145 116.90241 \n",
       "L 241.605627 116.90241 \n",
       "L 242.726109 116.90241 \n",
       "L 243.846591 116.90241 \n",
       "L 244.967073 116.90241 \n",
       "L 246.087555 116.90241 \n",
       "L 247.208037 116.90241 \n",
       "L 248.328518 116.90241 \n",
       "L 249.449 116.90241 \n",
       "L 250.569482 116.90241 \n",
       "L 251.689964 116.90241 \n",
       "L 252.810446 116.90241 \n",
       "L 253.930928 116.90241 \n",
       "L 255.05141 116.90241 \n",
       "L 256.171892 116.90241 \n",
       "L 257.292374 116.90241 \n",
       "L 258.412856 116.90241 \n",
       "L 259.533338 116.90241 \n",
       "L 260.65382 116.90241 \n",
       "L 261.774302 116.90241 \n",
       "L 262.894784 116.90241 \n",
       "L 264.015265 116.90241 \n",
       "L 265.135747 116.90241 \n",
       "L 266.256229 116.90241 \n",
       "L 267.376711 116.90241 \n",
       "L 268.497193 116.90241 \n",
       "L 269.617675 116.90241 \n",
       "L 270.738157 116.90241 \n",
       "L 271.858639 116.90241 \n",
       "L 272.979121 116.90241 \n",
       "L 274.099603 116.90241 \n",
       "L 275.220085 116.90241 \n",
       "L 276.340567 116.90241 \n",
       "L 277.461049 116.90241 \n",
       "L 278.58153 116.90241 \n",
       "L 279.702012 116.90241 \n",
       "L 280.822494 116.90241 \n",
       "L 281.942976 116.90241 \n",
       "L 283.063458 116.90241 \n",
       "L 284.18394 116.90241 \n",
       "L 285.304422 116.90241 \n",
       "L 286.424904 116.90241 \n",
       "L 287.545386 116.90241 \n",
       "L 288.665868 116.90241 \n",
       "L 289.78635 116.90241 \n",
       "L 290.906832 116.90241 \n",
       "L 292.027314 116.90241 \n",
       "L 293.147796 116.90241 \n",
       "L 294.268277 116.90241 \n",
       "L 295.388759 116.90241 \n",
       "L 296.509241 116.90241 \n",
       "L 297.629723 116.90241 \n",
       "L 298.750205 116.90241 \n",
       "L 299.870687 116.90241 \n",
       "L 300.991169 116.90241 \n",
       "L 302.111651 116.90241 \n",
       "L 303.232133 116.90241 \n",
       "L 304.352615 116.90241 \n",
       "L 305.473097 116.90241 \n",
       "L 306.593579 116.90241 \n",
       "L 307.714061 116.90241 \n",
       "L 308.834543 116.90241 \n",
       "L 309.955024 116.90241 \n",
       "L 311.075506 116.90241 \n",
       "L 312.195988 116.90241 \n",
       "L 313.31647 116.90241 \n",
       "L 314.436952 116.90241 \n",
       "L 315.557434 116.90241 \n",
       "L 316.677916 116.90241 \n",
       "L 317.798398 116.90241 \n",
       "L 318.91888 116.90241 \n",
       "L 320.039362 116.90241 \n",
       "L 321.159844 116.90241 \n",
       "L 321.159844 116.629518 \n",
       "L 320.039362 116.629518 \n",
       "L 318.91888 116.629518 \n",
       "L 317.798398 116.629518 \n",
       "L 316.677916 116.629518 \n",
       "L 315.557434 116.629518 \n",
       "L 314.436952 116.629518 \n",
       "L 313.31647 116.629518 \n",
       "L 312.195988 116.629518 \n",
       "L 311.075506 116.629518 \n",
       "L 309.955024 116.629518 \n",
       "L 308.834543 116.629518 \n",
       "L 307.714061 116.629518 \n",
       "L 306.593579 116.629518 \n",
       "L 305.473097 116.629518 \n",
       "L 304.352615 116.629518 \n",
       "L 303.232133 116.629518 \n",
       "L 302.111651 116.629518 \n",
       "L 300.991169 116.629518 \n",
       "L 299.870687 116.629518 \n",
       "L 298.750205 116.629518 \n",
       "L 297.629723 116.629518 \n",
       "L 296.509241 116.629518 \n",
       "L 295.388759 116.629518 \n",
       "L 294.268277 116.629518 \n",
       "L 293.147796 116.629518 \n",
       "L 292.027314 116.629518 \n",
       "L 290.906832 116.629518 \n",
       "L 289.78635 116.629518 \n",
       "L 288.665868 116.629518 \n",
       "L 287.545386 116.629518 \n",
       "L 286.424904 116.629518 \n",
       "L 285.304422 116.629518 \n",
       "L 284.18394 116.629518 \n",
       "L 283.063458 116.629518 \n",
       "L 281.942976 116.629518 \n",
       "L 280.822494 116.629518 \n",
       "L 279.702012 116.629518 \n",
       "L 278.58153 116.629518 \n",
       "L 277.461049 116.629518 \n",
       "L 276.340567 116.629518 \n",
       "L 275.220085 116.629518 \n",
       "L 274.099603 116.629518 \n",
       "L 272.979121 116.629518 \n",
       "L 271.858639 116.629518 \n",
       "L 270.738157 116.629518 \n",
       "L 269.617675 116.629518 \n",
       "L 268.497193 116.629518 \n",
       "L 267.376711 116.629518 \n",
       "L 266.256229 116.629518 \n",
       "L 265.135747 116.629518 \n",
       "L 264.015265 116.629518 \n",
       "L 262.894784 116.629518 \n",
       "L 261.774302 116.629518 \n",
       "L 260.65382 116.629518 \n",
       "L 259.533338 116.629518 \n",
       "L 258.412856 116.629518 \n",
       "L 257.292374 116.629518 \n",
       "L 256.171892 116.629518 \n",
       "L 255.05141 116.629518 \n",
       "L 253.930928 116.629518 \n",
       "L 252.810446 116.629518 \n",
       "L 251.689964 116.629518 \n",
       "L 250.569482 116.629518 \n",
       "L 249.449 116.629518 \n",
       "L 248.328518 116.629518 \n",
       "L 247.208037 116.629518 \n",
       "L 246.087555 116.629518 \n",
       "L 244.967073 116.629518 \n",
       "L 243.846591 116.629518 \n",
       "L 242.726109 116.629518 \n",
       "L 241.605627 116.629518 \n",
       "L 240.485145 116.629518 \n",
       "L 239.364663 116.629518 \n",
       "L 238.244181 116.629518 \n",
       "L 237.123699 116.629518 \n",
       "L 236.003217 116.629518 \n",
       "L 234.882735 116.629518 \n",
       "L 233.762253 116.629518 \n",
       "L 232.641771 116.629518 \n",
       "L 231.52129 116.629518 \n",
       "L 230.400808 116.629518 \n",
       "L 229.280326 116.629518 \n",
       "L 228.159844 116.629518 \n",
       "L 227.039362 116.629518 \n",
       "L 225.91888 116.629518 \n",
       "L 225.078518 117.448193 \n",
       "L 225.078518 118.539759 \n",
       "L 225.078518 119.631325 \n",
       "L 225.078518 120.722892 \n",
       "L 225.078518 121.814458 \n",
       "L 225.078518 122.906024 \n",
       "L 225.078518 123.99759 \n",
       "L 225.078518 125.089157 \n",
       "L 225.078518 126.180723 \n",
       "L 225.078518 127.272289 \n",
       "L 225.078518 128.363855 \n",
       "L 225.078518 129.455422 \n",
       "L 225.078518 130.546988 \n",
       "L 225.078518 131.638554 \n",
       "L 225.078518 132.73012 \n",
       "L 225.078518 133.821687 \n",
       "L 225.078518 134.913253 \n",
       "L 225.078518 136.004819 \n",
       "L 225.078518 137.096386 \n",
       "L 225.078518 138.187952 \n",
       "L 225.078518 139.279518 \n",
       "L 225.078518 140.371084 \n",
       "L 225.078518 141.462651 \n",
       "L 225.078518 142.554217 \n",
       "L 225.078518 143.645783 \n",
       "L 225.078518 144.737349 \n",
       "L 225.078518 145.828916 \n",
       "L 225.078518 146.920482 \n",
       "L 225.078518 148.012048 \n",
       "L 225.078518 149.103614 \n",
       "L 225.078518 150.195181 \n",
       "L 225.078518 151.286747 \n",
       "L 225.078518 152.378313 \n",
       "L 225.078518 153.46988 \n",
       "L 225.078518 154.561446 \n",
       "L 225.078518 155.653012 \n",
       "L 225.078518 156.744578 \n",
       "L 225.078518 157.836145 \n",
       "L 225.078518 158.927711 \n",
       "L 225.078518 160.019277 \n",
       "L 225.078518 161.110843 \n",
       "L 225.078518 162.20241 \n",
       "L 225.078518 163.293976 \n",
       "L 225.078518 164.385542 \n",
       "L 225.078518 165.477108 \n",
       "L 225.078518 166.568675 \n",
       "L 225.078518 167.660241 \n",
       "L 225.078518 168.751807 \n",
       "L 225.078518 169.843373 \n",
       "L 225.078518 170.93494 \n",
       "L 225.078518 172.026506 \n",
       "L 225.078518 173.118072 \n",
       "L 225.078518 174.209639 \n",
       "L 225.078518 175.301205 \n",
       "L 225.078518 176.392771 \n",
       "L 225.078518 177.484337 \n",
       "L 225.078518 178.575904 \n",
       "L 225.078518 179.66747 \n",
       "L 225.078518 180.759036 \n",
       "L 225.078518 181.850602 \n",
       "L 225.078518 182.942169 \n",
       "L 225.078518 184.033735 \n",
       "L 225.078518 185.125301 \n",
       "L 225.078518 186.216867 \n",
       "L 225.078518 187.308434 \n",
       "L 225.078518 188.4 \n",
       "L 225.078518 189.491566 \n",
       "L 225.078518 190.583133 \n",
       "L 225.078518 191.674699 \n",
       "L 225.078518 192.766265 \n",
       "L 225.078518 193.857831 \n",
       "L 225.078518 194.949398 \n",
       "z\n",
       "\" style=\"fill:#e377c2;fill-opacity:0.15;\"/>\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 43.280326 106.259639 \n",
       "L 44.400808 106.259639 \n",
       "L 45.52129 106.259639 \n",
       "L 46.641771 106.259639 \n",
       "L 47.762253 106.259639 \n",
       "L 48.882735 106.259639 \n",
       "L 50.003217 106.259639 \n",
       "L 51.123699 106.259639 \n",
       "L 52.244181 106.259639 \n",
       "L 53.364663 106.259639 \n",
       "L 54.485145 106.259639 \n",
       "L 55.605627 106.259639 \n",
       "L 56.726109 106.259639 \n",
       "L 57.846591 106.259639 \n",
       "L 58.967073 106.259639 \n",
       "L 60.087555 106.259639 \n",
       "L 61.208037 106.259639 \n",
       "L 62.328518 106.259639 \n",
       "L 63.449 106.259639 \n",
       "L 64.569482 106.259639 \n",
       "L 65.689964 106.259639 \n",
       "L 66.810446 106.259639 \n",
       "L 67.930928 106.259639 \n",
       "L 69.05141 106.259639 \n",
       "L 70.171892 106.259639 \n",
       "L 71.292374 106.259639 \n",
       "L 72.412856 106.259639 \n",
       "L 73.533338 106.259639 \n",
       "L 74.65382 106.259639 \n",
       "L 75.774302 106.259639 \n",
       "L 76.894784 106.259639 \n",
       "L 78.015265 106.259639 \n",
       "L 79.135747 106.259639 \n",
       "L 80.256229 106.259639 \n",
       "L 81.376711 106.259639 \n",
       "L 82.497193 106.259639 \n",
       "L 83.617675 106.259639 \n",
       "L 84.738157 106.259639 \n",
       "L 85.858639 106.259639 \n",
       "L 86.979121 106.259639 \n",
       "L 88.099603 106.259639 \n",
       "L 89.220085 106.259639 \n",
       "L 90.340567 106.259639 \n",
       "L 91.461049 106.259639 \n",
       "L 92.58153 106.259639 \n",
       "L 93.702012 106.259639 \n",
       "L 94.822494 106.259639 \n",
       "L 95.942976 106.259639 \n",
       "L 97.063458 106.259639 \n",
       "L 98.18394 106.259639 \n",
       "L 99.304422 106.259639 \n",
       "L 100.424904 106.259639 \n",
       "L 101.545386 106.259639 \n",
       "L 102.665868 106.259639 \n",
       "L 103.78635 106.259639 \n",
       "L 104.906832 106.259639 \n",
       "L 106.027314 106.259639 \n",
       "L 107.147796 106.259639 \n",
       "L 108.268277 106.259639 \n",
       "L 109.388759 106.259639 \n",
       "L 110.509241 106.259639 \n",
       "L 111.629723 106.259639 \n",
       "L 112.750205 106.259639 \n",
       "L 113.870687 106.259639 \n",
       "L 114.991169 106.259639 \n",
       "L 116.111651 106.259639 \n",
       "L 117.232133 106.259639 \n",
       "L 118.352615 106.259639 \n",
       "L 119.473097 106.259639 \n",
       "L 120.593579 106.259639 \n",
       "L 121.714061 106.259639 \n",
       "L 122.834543 106.259639 \n",
       "L 123.955024 106.259639 \n",
       "L 125.075506 106.259639 \n",
       "L 126.195988 106.259639 \n",
       "L 127.31647 106.259639 \n",
       "L 128.436952 106.259639 \n",
       "L 129.557434 106.259639 \n",
       "L 130.677916 106.259639 \n",
       "L 131.798398 106.259639 \n",
       "L 132.91888 106.259639 \n",
       "L 134.039362 106.259639 \n",
       "L 135.159844 106.259639 \n",
       "L 136.280326 106.259639 \n",
       "L 137.400808 106.259639 \n",
       "L 138.52129 106.259639 \n",
       "L 139.641771 106.259639 \n",
       "L 140.762253 106.259639 \n",
       "L 141.882735 106.259639 \n",
       "L 143.003217 106.259639 \n",
       "L 144.123699 106.259639 \n",
       "L 145.244181 106.259639 \n",
       "L 146.364663 106.259639 \n",
       "L 147.485145 106.259639 \n",
       "L 148.605627 106.259639 \n",
       "L 149.726109 106.259639 \n",
       "L 150.846591 106.259639 \n",
       "L 151.967073 106.259639 \n",
       "L 153.087555 106.259639 \n",
       "L 154.208037 106.259639 \n",
       "L 155.328518 106.259639 \n",
       "L 156.449 106.259639 \n",
       "L 157.569482 106.259639 \n",
       "L 158.689964 106.259639 \n",
       "L 159.810446 106.259639 \n",
       "L 160.930928 106.259639 \n",
       "L 162.05141 106.259639 \n",
       "L 163.171892 106.259639 \n",
       "L 164.292374 106.259639 \n",
       "L 165.412856 106.259639 \n",
       "L 166.533338 106.259639 \n",
       "L 167.65382 106.259639 \n",
       "L 168.774302 106.259639 \n",
       "L 169.894784 106.259639 \n",
       "L 171.015265 106.259639 \n",
       "L 172.135747 106.259639 \n",
       "L 173.256229 106.259639 \n",
       "L 174.376711 106.259639 \n",
       "L 175.497193 106.259639 \n",
       "L 176.617675 106.259639 \n",
       "L 177.738157 106.259639 \n",
       "L 178.858639 106.259639 \n",
       "L 179.979121 106.259639 \n",
       "L 181.099603 106.259639 \n",
       "L 182.220085 106.259639 \n",
       "L 183.340567 106.259639 \n",
       "L 184.461049 106.259639 \n",
       "L 185.58153 106.259639 \n",
       "L 186.702012 106.259639 \n",
       "L 187.822494 106.259639 \n",
       "L 188.942976 106.259639 \n",
       "L 190.063458 106.259639 \n",
       "L 191.18394 106.259639 \n",
       "L 192.304422 106.259639 \n",
       "L 193.424904 106.259639 \n",
       "L 194.545386 106.259639 \n",
       "L 195.665868 106.259639 \n",
       "L 196.78635 106.259639 \n",
       "L 197.906832 106.259639 \n",
       "L 199.027314 106.259639 \n",
       "L 200.147796 106.259639 \n",
       "L 201.268277 106.259639 \n",
       "L 202.388759 106.259639 \n",
       "L 203.509241 106.259639 \n",
       "L 204.629723 106.259639 \n",
       "L 205.750205 106.259639 \n",
       "L 206.870687 106.259639 \n",
       "L 207.991169 106.259639 \n",
       "L 209.111651 106.259639 \n",
       "L 210.232133 106.259639 \n",
       "L 211.352615 106.259639 \n",
       "L 212.473097 106.259639 \n",
       "L 213.593579 106.259639 \n",
       "L 214.714061 106.259639 \n",
       "L 215.834543 106.259639 \n",
       "L 216.955024 106.259639 \n",
       "L 218.075506 106.259639 \n",
       "L 219.195988 106.259639 \n",
       "L 220.31647 106.259639 \n",
       "L 221.436952 106.259639 \n",
       "L 222.557434 106.259639 \n",
       "L 223.677916 106.259639 \n",
       "L 224.798398 106.259639 \n",
       "L 225.638759 105.440964 \n",
       "L 225.638759 104.349398 \n",
       "L 225.638759 103.257831 \n",
       "L 225.638759 102.166265 \n",
       "L 225.638759 101.074699 \n",
       "L 225.638759 99.983133 \n",
       "L 225.638759 98.891566 \n",
       "L 225.638759 97.8 \n",
       "L 225.638759 96.708434 \n",
       "L 225.91888 96.435542 \n",
       "L 227.039362 96.435542 \n",
       "L 228.159844 96.435542 \n",
       "L 229.280326 96.435542 \n",
       "L 230.400808 96.435542 \n",
       "L 231.52129 96.435542 \n",
       "L 232.641771 96.435542 \n",
       "L 233.762253 96.435542 \n",
       "L 234.882735 96.435542 \n",
       "L 236.003217 96.435542 \n",
       "L 237.123699 96.435542 \n",
       "L 238.244181 96.435542 \n",
       "L 239.364663 96.435542 \n",
       "L 240.485145 96.435542 \n",
       "L 241.605627 96.435542 \n",
       "L 242.726109 96.435542 \n",
       "L 243.846591 96.435542 \n",
       "L 244.967073 96.435542 \n",
       "L 246.087555 96.435542 \n",
       "L 247.208037 96.435542 \n",
       "L 248.328518 96.435542 \n",
       "L 249.449 96.435542 \n",
       "L 250.569482 96.435542 \n",
       "L 251.689964 96.435542 \n",
       "L 252.810446 96.435542 \n",
       "L 253.930928 96.435542 \n",
       "L 255.05141 96.435542 \n",
       "L 256.171892 96.435542 \n",
       "L 257.292374 96.435542 \n",
       "L 258.412856 96.435542 \n",
       "L 259.533338 96.435542 \n",
       "L 260.65382 96.435542 \n",
       "L 261.774302 96.435542 \n",
       "L 262.894784 96.435542 \n",
       "L 264.015265 96.435542 \n",
       "L 265.135747 96.435542 \n",
       "L 266.256229 96.435542 \n",
       "L 267.376711 96.435542 \n",
       "L 268.497193 96.435542 \n",
       "L 269.617675 96.435542 \n",
       "L 270.738157 96.435542 \n",
       "L 271.858639 96.435542 \n",
       "L 272.979121 96.435542 \n",
       "L 274.099603 96.435542 \n",
       "L 275.220085 96.435542 \n",
       "L 276.340567 96.435542 \n",
       "L 277.461049 96.435542 \n",
       "L 278.58153 96.435542 \n",
       "L 279.702012 96.435542 \n",
       "L 280.822494 96.435542 \n",
       "L 281.942976 96.435542 \n",
       "L 283.063458 96.435542 \n",
       "L 284.18394 96.435542 \n",
       "L 285.304422 96.435542 \n",
       "L 286.424904 96.435542 \n",
       "L 287.545386 96.435542 \n",
       "L 288.665868 96.435542 \n",
       "L 289.78635 96.435542 \n",
       "L 290.906832 96.435542 \n",
       "L 292.027314 96.435542 \n",
       "L 293.147796 96.435542 \n",
       "L 294.268277 96.435542 \n",
       "L 295.388759 96.435542 \n",
       "L 296.509241 96.435542 \n",
       "L 297.629723 96.435542 \n",
       "L 298.750205 96.435542 \n",
       "L 299.870687 96.435542 \n",
       "L 300.991169 96.435542 \n",
       "L 302.111651 96.435542 \n",
       "L 303.232133 96.435542 \n",
       "L 304.352615 96.435542 \n",
       "L 305.473097 96.435542 \n",
       "L 306.593579 96.435542 \n",
       "L 307.714061 96.435542 \n",
       "L 308.834543 96.435542 \n",
       "L 309.955024 96.435542 \n",
       "L 311.075506 96.435542 \n",
       "L 312.195988 96.435542 \n",
       "L 313.31647 96.435542 \n",
       "L 314.436952 96.435542 \n",
       "L 315.557434 96.435542 \n",
       "L 316.677916 96.435542 \n",
       "L 317.798398 96.435542 \n",
       "L 318.91888 96.435542 \n",
       "L 320.039362 96.435542 \n",
       "L 321.159844 96.435542 \n",
       "L 321.159844 96.162651 \n",
       "L 320.039362 96.162651 \n",
       "L 318.91888 96.162651 \n",
       "L 317.798398 96.162651 \n",
       "L 316.677916 96.162651 \n",
       "L 315.557434 96.162651 \n",
       "L 314.436952 96.162651 \n",
       "L 313.31647 96.162651 \n",
       "L 312.195988 96.162651 \n",
       "L 311.075506 96.162651 \n",
       "L 309.955024 96.162651 \n",
       "L 308.834543 96.162651 \n",
       "L 307.714061 96.162651 \n",
       "L 306.593579 96.162651 \n",
       "L 305.473097 96.162651 \n",
       "L 304.352615 96.162651 \n",
       "L 303.232133 96.162651 \n",
       "L 302.111651 96.162651 \n",
       "L 300.991169 96.162651 \n",
       "L 299.870687 96.162651 \n",
       "L 298.750205 96.162651 \n",
       "L 297.629723 96.162651 \n",
       "L 296.509241 96.162651 \n",
       "L 295.388759 96.162651 \n",
       "L 294.268277 96.162651 \n",
       "L 293.147796 96.162651 \n",
       "L 292.027314 96.162651 \n",
       "L 290.906832 96.162651 \n",
       "L 289.78635 96.162651 \n",
       "L 288.665868 96.162651 \n",
       "L 287.545386 96.162651 \n",
       "L 286.424904 96.162651 \n",
       "L 285.304422 96.162651 \n",
       "L 284.18394 96.162651 \n",
       "L 283.063458 96.162651 \n",
       "L 281.942976 96.162651 \n",
       "L 280.822494 96.162651 \n",
       "L 279.702012 96.162651 \n",
       "L 278.58153 96.162651 \n",
       "L 277.461049 96.162651 \n",
       "L 276.340567 96.162651 \n",
       "L 275.220085 96.162651 \n",
       "L 274.099603 96.162651 \n",
       "L 272.979121 96.162651 \n",
       "L 271.858639 96.162651 \n",
       "L 270.738157 96.162651 \n",
       "L 269.617675 96.162651 \n",
       "L 268.497193 96.162651 \n",
       "L 267.376711 96.162651 \n",
       "L 266.256229 96.162651 \n",
       "L 265.135747 96.162651 \n",
       "L 264.015265 96.162651 \n",
       "L 262.894784 96.162651 \n",
       "L 261.774302 96.162651 \n",
       "L 260.65382 96.162651 \n",
       "L 259.533338 96.162651 \n",
       "L 258.412856 96.162651 \n",
       "L 257.292374 96.162651 \n",
       "L 256.171892 96.162651 \n",
       "L 255.05141 96.162651 \n",
       "L 253.930928 96.162651 \n",
       "L 252.810446 96.162651 \n",
       "L 251.689964 96.162651 \n",
       "L 250.569482 96.162651 \n",
       "L 249.449 96.162651 \n",
       "L 248.328518 96.162651 \n",
       "L 247.208037 96.162651 \n",
       "L 246.087555 96.162651 \n",
       "L 244.967073 96.162651 \n",
       "L 243.846591 96.162651 \n",
       "L 242.726109 96.162651 \n",
       "L 241.605627 96.162651 \n",
       "L 240.485145 96.162651 \n",
       "L 239.364663 96.162651 \n",
       "L 238.244181 96.162651 \n",
       "L 237.123699 96.162651 \n",
       "L 236.003217 96.162651 \n",
       "L 234.882735 96.162651 \n",
       "L 233.762253 96.162651 \n",
       "L 232.641771 96.162651 \n",
       "L 231.52129 96.162651 \n",
       "L 230.400808 96.162651 \n",
       "L 229.280326 96.162651 \n",
       "L 228.159844 96.162651 \n",
       "L 227.039362 96.162651 \n",
       "L 225.91888 96.162651 \n",
       "L 225.358639 96.708434 \n",
       "L 225.358639 97.8 \n",
       "L 225.358639 98.891566 \n",
       "L 225.358639 99.983133 \n",
       "L 225.358639 101.074699 \n",
       "L 225.358639 102.166265 \n",
       "L 225.358639 103.257831 \n",
       "L 225.358639 104.349398 \n",
       "L 225.358639 105.440964 \n",
       "L 224.798398 105.986747 \n",
       "L 223.677916 105.986747 \n",
       "L 222.557434 105.986747 \n",
       "L 221.436952 105.986747 \n",
       "L 220.31647 105.986747 \n",
       "L 219.195988 105.986747 \n",
       "L 218.075506 105.986747 \n",
       "L 216.955024 105.986747 \n",
       "L 215.834543 105.986747 \n",
       "L 214.714061 105.986747 \n",
       "L 213.593579 105.986747 \n",
       "L 212.473097 105.986747 \n",
       "L 211.352615 105.986747 \n",
       "L 210.232133 105.986747 \n",
       "L 209.111651 105.986747 \n",
       "L 207.991169 105.986747 \n",
       "L 206.870687 105.986747 \n",
       "L 205.750205 105.986747 \n",
       "L 204.629723 105.986747 \n",
       "L 203.509241 105.986747 \n",
       "L 202.388759 105.986747 \n",
       "L 201.268277 105.986747 \n",
       "L 200.147796 105.986747 \n",
       "L 199.027314 105.986747 \n",
       "L 197.906832 105.986747 \n",
       "L 196.78635 105.986747 \n",
       "L 195.665868 105.986747 \n",
       "L 194.545386 105.986747 \n",
       "L 193.424904 105.986747 \n",
       "L 192.304422 105.986747 \n",
       "L 191.18394 105.986747 \n",
       "L 190.063458 105.986747 \n",
       "L 188.942976 105.986747 \n",
       "L 187.822494 105.986747 \n",
       "L 186.702012 105.986747 \n",
       "L 185.58153 105.986747 \n",
       "L 184.461049 105.986747 \n",
       "L 183.340567 105.986747 \n",
       "L 182.220085 105.986747 \n",
       "L 181.099603 105.986747 \n",
       "L 179.979121 105.986747 \n",
       "L 178.858639 105.986747 \n",
       "L 177.738157 105.986747 \n",
       "L 176.617675 105.986747 \n",
       "L 175.497193 105.986747 \n",
       "L 174.376711 105.986747 \n",
       "L 173.256229 105.986747 \n",
       "L 172.135747 105.986747 \n",
       "L 171.015265 105.986747 \n",
       "L 169.894784 105.986747 \n",
       "L 168.774302 105.986747 \n",
       "L 167.65382 105.986747 \n",
       "L 166.533338 105.986747 \n",
       "L 165.412856 105.986747 \n",
       "L 164.292374 105.986747 \n",
       "L 163.171892 105.986747 \n",
       "L 162.05141 105.986747 \n",
       "L 160.930928 105.986747 \n",
       "L 159.810446 105.986747 \n",
       "L 158.689964 105.986747 \n",
       "L 157.569482 105.986747 \n",
       "L 156.449 105.986747 \n",
       "L 155.328518 105.986747 \n",
       "L 154.208037 105.986747 \n",
       "L 153.087555 105.986747 \n",
       "L 151.967073 105.986747 \n",
       "L 150.846591 105.986747 \n",
       "L 149.726109 105.986747 \n",
       "L 148.605627 105.986747 \n",
       "L 147.485145 105.986747 \n",
       "L 146.364663 105.986747 \n",
       "L 145.244181 105.986747 \n",
       "L 144.123699 105.986747 \n",
       "L 143.003217 105.986747 \n",
       "L 141.882735 105.986747 \n",
       "L 140.762253 105.986747 \n",
       "L 139.641771 105.986747 \n",
       "L 138.52129 105.986747 \n",
       "L 137.400808 105.986747 \n",
       "L 136.280326 105.986747 \n",
       "L 135.159844 105.986747 \n",
       "L 134.039362 105.986747 \n",
       "L 132.91888 105.986747 \n",
       "L 131.798398 105.986747 \n",
       "L 130.677916 105.986747 \n",
       "L 129.557434 105.986747 \n",
       "L 128.436952 105.986747 \n",
       "L 127.31647 105.986747 \n",
       "L 126.195988 105.986747 \n",
       "L 125.075506 105.986747 \n",
       "L 123.955024 105.986747 \n",
       "L 122.834543 105.986747 \n",
       "L 121.714061 105.986747 \n",
       "L 120.593579 105.986747 \n",
       "L 119.473097 105.986747 \n",
       "L 118.352615 105.986747 \n",
       "L 117.232133 105.986747 \n",
       "L 116.111651 105.986747 \n",
       "L 114.991169 105.986747 \n",
       "L 113.870687 105.986747 \n",
       "L 112.750205 105.986747 \n",
       "L 111.629723 105.986747 \n",
       "L 110.509241 105.986747 \n",
       "L 109.388759 105.986747 \n",
       "L 108.268277 105.986747 \n",
       "L 107.147796 105.986747 \n",
       "L 106.027314 105.986747 \n",
       "L 104.906832 105.986747 \n",
       "L 103.78635 105.986747 \n",
       "L 102.665868 105.986747 \n",
       "L 101.545386 105.986747 \n",
       "L 100.424904 105.986747 \n",
       "L 99.304422 105.986747 \n",
       "L 98.18394 105.986747 \n",
       "L 97.063458 105.986747 \n",
       "L 95.942976 105.986747 \n",
       "L 94.822494 105.986747 \n",
       "L 93.702012 105.986747 \n",
       "L 92.58153 105.986747 \n",
       "L 91.461049 105.986747 \n",
       "L 90.340567 105.986747 \n",
       "L 89.220085 105.986747 \n",
       "L 88.099603 105.986747 \n",
       "L 86.979121 105.986747 \n",
       "L 85.858639 105.986747 \n",
       "L 84.738157 105.986747 \n",
       "L 83.617675 105.986747 \n",
       "L 82.497193 105.986747 \n",
       "L 81.376711 105.986747 \n",
       "L 80.256229 105.986747 \n",
       "L 79.135747 105.986747 \n",
       "L 78.015265 105.986747 \n",
       "L 76.894784 105.986747 \n",
       "L 75.774302 105.986747 \n",
       "L 74.65382 105.986747 \n",
       "L 73.533338 105.986747 \n",
       "L 72.412856 105.986747 \n",
       "L 71.292374 105.986747 \n",
       "L 70.171892 105.986747 \n",
       "L 69.05141 105.986747 \n",
       "L 67.930928 105.986747 \n",
       "L 66.810446 105.986747 \n",
       "L 65.689964 105.986747 \n",
       "L 64.569482 105.986747 \n",
       "L 63.449 105.986747 \n",
       "L 62.328518 105.986747 \n",
       "L 61.208037 105.986747 \n",
       "L 60.087555 105.986747 \n",
       "L 58.967073 105.986747 \n",
       "L 57.846591 105.986747 \n",
       "L 56.726109 105.986747 \n",
       "L 55.605627 105.986747 \n",
       "L 54.485145 105.986747 \n",
       "L 53.364663 105.986747 \n",
       "L 52.244181 105.986747 \n",
       "L 51.123699 105.986747 \n",
       "L 50.003217 105.986747 \n",
       "L 48.882735 105.986747 \n",
       "L 47.762253 105.986747 \n",
       "L 46.641771 105.986747 \n",
       "L 45.52129 105.986747 \n",
       "L 44.400808 105.986747 \n",
       "L 43.280326 105.986747 \n",
       "L 42.159844 105.986747 \n",
       "L 42.159844 106.259639 \n",
       "z\n",
       "\" style=\"fill:#e377c2;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_7\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 225.91888 195.222289 \n",
       "L 227.039362 195.222289 \n",
       "L 228.159844 195.222289 \n",
       "L 229.280326 195.222289 \n",
       "L 230.400808 195.222289 \n",
       "L 231.52129 195.222289 \n",
       "L 232.641771 195.222289 \n",
       "L 233.762253 195.222289 \n",
       "L 234.882735 195.222289 \n",
       "L 236.003217 195.222289 \n",
       "L 237.123699 195.222289 \n",
       "L 238.244181 195.222289 \n",
       "L 239.364663 195.222289 \n",
       "L 240.485145 195.222289 \n",
       "L 241.605627 195.222289 \n",
       "L 242.726109 195.222289 \n",
       "L 243.846591 195.222289 \n",
       "L 244.967073 195.222289 \n",
       "L 246.087555 195.222289 \n",
       "L 247.208037 195.222289 \n",
       "L 248.328518 195.222289 \n",
       "L 249.449 195.222289 \n",
       "L 250.569482 195.222289 \n",
       "L 251.689964 195.222289 \n",
       "L 252.810446 195.222289 \n",
       "L 253.930928 195.222289 \n",
       "L 255.05141 195.222289 \n",
       "L 256.171892 195.222289 \n",
       "L 257.292374 195.222289 \n",
       "L 258.412856 195.222289 \n",
       "L 259.533338 195.222289 \n",
       "L 260.65382 195.222289 \n",
       "L 261.774302 195.222289 \n",
       "L 262.894784 195.222289 \n",
       "L 264.015265 195.222289 \n",
       "L 265.135747 195.222289 \n",
       "L 266.256229 195.222289 \n",
       "L 267.376711 195.222289 \n",
       "L 268.497193 195.222289 \n",
       "L 269.617675 195.222289 \n",
       "L 270.738157 195.222289 \n",
       "L 271.858639 195.222289 \n",
       "L 272.979121 195.222289 \n",
       "L 274.099603 195.222289 \n",
       "L 275.220085 195.222289 \n",
       "L 276.340567 195.222289 \n",
       "L 277.461049 195.222289 \n",
       "L 278.58153 195.222289 \n",
       "L 279.702012 195.222289 \n",
       "L 280.822494 195.222289 \n",
       "L 281.942976 195.222289 \n",
       "L 283.063458 195.222289 \n",
       "L 284.18394 195.222289 \n",
       "L 285.304422 195.222289 \n",
       "L 286.424904 195.222289 \n",
       "L 287.545386 195.222289 \n",
       "L 288.665868 195.222289 \n",
       "L 289.78635 195.222289 \n",
       "L 290.906832 195.222289 \n",
       "L 292.027314 195.222289 \n",
       "L 293.147796 195.222289 \n",
       "L 294.268277 195.222289 \n",
       "L 295.388759 195.222289 \n",
       "L 296.509241 195.222289 \n",
       "L 297.629723 195.222289 \n",
       "L 298.750205 195.222289 \n",
       "L 299.870687 195.222289 \n",
       "L 300.991169 195.222289 \n",
       "L 302.111651 195.222289 \n",
       "L 303.232133 195.222289 \n",
       "L 304.352615 195.222289 \n",
       "L 305.473097 195.222289 \n",
       "L 306.593579 195.222289 \n",
       "L 307.714061 195.222289 \n",
       "L 308.834543 195.222289 \n",
       "L 309.955024 195.222289 \n",
       "L 311.075506 195.222289 \n",
       "L 312.195988 195.222289 \n",
       "L 313.31647 195.222289 \n",
       "L 314.436952 195.222289 \n",
       "L 315.557434 195.222289 \n",
       "L 316.677916 195.222289 \n",
       "L 317.798398 195.222289 \n",
       "L 318.91888 195.222289 \n",
       "L 320.039362 195.222289 \n",
       "L 321.159844 195.222289 \n",
       "L 321.159844 195.085843 \n",
       "L 320.039362 195.085843 \n",
       "L 318.91888 195.085843 \n",
       "L 317.798398 195.085843 \n",
       "L 316.677916 195.085843 \n",
       "L 315.557434 195.085843 \n",
       "L 314.436952 195.085843 \n",
       "L 313.31647 195.085843 \n",
       "L 312.195988 195.085843 \n",
       "L 311.075506 195.085843 \n",
       "L 309.955024 195.085843 \n",
       "L 308.834543 195.085843 \n",
       "L 307.714061 195.085843 \n",
       "L 306.593579 195.085843 \n",
       "L 305.473097 195.085843 \n",
       "L 304.352615 195.085843 \n",
       "L 303.232133 195.085843 \n",
       "L 302.111651 195.085843 \n",
       "L 300.991169 195.085843 \n",
       "L 299.870687 195.085843 \n",
       "L 298.750205 195.085843 \n",
       "L 297.629723 195.085843 \n",
       "L 296.509241 195.085843 \n",
       "L 295.388759 195.085843 \n",
       "L 294.268277 195.085843 \n",
       "L 293.147796 195.085843 \n",
       "L 292.027314 195.085843 \n",
       "L 290.906832 195.085843 \n",
       "L 289.78635 195.085843 \n",
       "L 288.665868 195.085843 \n",
       "L 287.545386 195.085843 \n",
       "L 286.424904 195.085843 \n",
       "L 285.304422 195.085843 \n",
       "L 284.18394 195.085843 \n",
       "L 283.063458 195.085843 \n",
       "L 281.942976 195.085843 \n",
       "L 280.822494 195.085843 \n",
       "L 279.702012 195.085843 \n",
       "L 278.58153 195.085843 \n",
       "L 277.461049 195.085843 \n",
       "L 276.340567 195.085843 \n",
       "L 275.220085 195.085843 \n",
       "L 274.099603 195.085843 \n",
       "L 272.979121 195.085843 \n",
       "L 271.858639 195.085843 \n",
       "L 270.738157 195.085843 \n",
       "L 269.617675 195.085843 \n",
       "L 268.497193 195.085843 \n",
       "L 267.376711 195.085843 \n",
       "L 266.256229 195.085843 \n",
       "L 265.135747 195.085843 \n",
       "L 264.015265 195.085843 \n",
       "L 262.894784 195.085843 \n",
       "L 261.774302 195.085843 \n",
       "L 260.65382 195.085843 \n",
       "L 259.533338 195.085843 \n",
       "L 258.412856 195.085843 \n",
       "L 257.292374 195.085843 \n",
       "L 256.171892 195.085843 \n",
       "L 255.05141 195.085843 \n",
       "L 253.930928 195.085843 \n",
       "L 252.810446 195.085843 \n",
       "L 251.689964 195.085843 \n",
       "L 250.569482 195.085843 \n",
       "L 249.449 195.085843 \n",
       "L 248.328518 195.085843 \n",
       "L 247.208037 195.085843 \n",
       "L 246.087555 195.085843 \n",
       "L 244.967073 195.085843 \n",
       "L 243.846591 195.085843 \n",
       "L 242.726109 195.085843 \n",
       "L 241.605627 195.085843 \n",
       "L 240.485145 195.085843 \n",
       "L 239.364663 195.085843 \n",
       "L 238.244181 195.085843 \n",
       "L 237.123699 195.085843 \n",
       "L 236.003217 195.085843 \n",
       "L 234.882735 195.085843 \n",
       "L 233.762253 195.085843 \n",
       "L 232.641771 195.085843 \n",
       "L 231.52129 195.085843 \n",
       "L 230.400808 195.085843 \n",
       "L 229.280326 195.085843 \n",
       "L 228.159844 195.085843 \n",
       "L 227.039362 195.085843 \n",
       "L 225.91888 195.085843 \n",
       "L 225.638759 194.949398 \n",
       "L 225.638759 193.857831 \n",
       "L 225.638759 192.766265 \n",
       "L 225.638759 191.674699 \n",
       "L 225.638759 190.583133 \n",
       "L 225.638759 189.491566 \n",
       "L 225.638759 188.4 \n",
       "L 225.638759 187.308434 \n",
       "L 225.638759 186.216867 \n",
       "L 225.638759 185.125301 \n",
       "L 225.638759 184.033735 \n",
       "L 225.638759 182.942169 \n",
       "L 225.638759 181.850602 \n",
       "L 225.638759 180.759036 \n",
       "L 225.638759 179.66747 \n",
       "L 225.638759 178.575904 \n",
       "L 225.638759 177.484337 \n",
       "L 225.638759 176.392771 \n",
       "L 225.638759 175.301205 \n",
       "L 225.638759 174.209639 \n",
       "L 225.638759 173.118072 \n",
       "L 225.638759 172.026506 \n",
       "L 225.638759 170.93494 \n",
       "L 225.638759 169.843373 \n",
       "L 225.638759 168.751807 \n",
       "L 225.638759 167.660241 \n",
       "L 225.638759 166.568675 \n",
       "L 225.638759 165.477108 \n",
       "L 225.638759 164.385542 \n",
       "L 225.638759 163.293976 \n",
       "L 225.638759 162.20241 \n",
       "L 225.638759 161.110843 \n",
       "L 225.638759 160.019277 \n",
       "L 225.638759 158.927711 \n",
       "L 225.638759 157.836145 \n",
       "L 225.638759 156.744578 \n",
       "L 225.638759 155.653012 \n",
       "L 225.638759 154.561446 \n",
       "L 225.638759 153.46988 \n",
       "L 225.638759 152.378313 \n",
       "L 225.638759 151.286747 \n",
       "L 225.638759 150.195181 \n",
       "L 225.638759 149.103614 \n",
       "L 225.638759 148.012048 \n",
       "L 225.638759 146.920482 \n",
       "L 225.638759 145.828916 \n",
       "L 225.638759 144.737349 \n",
       "L 225.638759 143.645783 \n",
       "L 225.638759 142.554217 \n",
       "L 225.638759 141.462651 \n",
       "L 225.638759 140.371084 \n",
       "L 225.638759 139.279518 \n",
       "L 225.638759 138.187952 \n",
       "L 225.638759 137.096386 \n",
       "L 225.638759 136.004819 \n",
       "L 225.638759 134.913253 \n",
       "L 225.638759 133.821687 \n",
       "L 225.638759 132.73012 \n",
       "L 225.638759 131.638554 \n",
       "L 225.638759 130.546988 \n",
       "L 225.638759 129.455422 \n",
       "L 225.638759 128.363855 \n",
       "L 225.638759 127.272289 \n",
       "L 225.638759 126.180723 \n",
       "L 225.638759 125.089157 \n",
       "L 225.638759 123.99759 \n",
       "L 225.638759 122.906024 \n",
       "L 225.638759 121.814458 \n",
       "L 225.638759 120.722892 \n",
       "L 225.638759 119.631325 \n",
       "L 225.638759 118.539759 \n",
       "L 225.638759 117.448193 \n",
       "L 225.91888 117.175301 \n",
       "L 227.039362 117.175301 \n",
       "L 228.159844 117.175301 \n",
       "L 229.280326 117.175301 \n",
       "L 230.400808 117.175301 \n",
       "L 231.52129 117.175301 \n",
       "L 232.641771 117.175301 \n",
       "L 233.762253 117.175301 \n",
       "L 234.882735 117.175301 \n",
       "L 236.003217 117.175301 \n",
       "L 237.123699 117.175301 \n",
       "L 238.244181 117.175301 \n",
       "L 239.364663 117.175301 \n",
       "L 240.485145 117.175301 \n",
       "L 241.605627 117.175301 \n",
       "L 242.726109 117.175301 \n",
       "L 243.846591 117.175301 \n",
       "L 244.967073 117.175301 \n",
       "L 246.087555 117.175301 \n",
       "L 247.208037 117.175301 \n",
       "L 248.328518 117.175301 \n",
       "L 249.449 117.175301 \n",
       "L 250.569482 117.175301 \n",
       "L 251.689964 117.175301 \n",
       "L 252.810446 117.175301 \n",
       "L 253.930928 117.175301 \n",
       "L 255.05141 117.175301 \n",
       "L 256.171892 117.175301 \n",
       "L 257.292374 117.175301 \n",
       "L 258.412856 117.175301 \n",
       "L 259.533338 117.175301 \n",
       "L 260.65382 117.175301 \n",
       "L 261.774302 117.175301 \n",
       "L 262.894784 117.175301 \n",
       "L 264.015265 117.175301 \n",
       "L 265.135747 117.175301 \n",
       "L 266.256229 117.175301 \n",
       "L 267.376711 117.175301 \n",
       "L 268.497193 117.175301 \n",
       "L 269.617675 117.175301 \n",
       "L 270.738157 117.175301 \n",
       "L 271.858639 117.175301 \n",
       "L 272.979121 117.175301 \n",
       "L 274.099603 117.175301 \n",
       "L 275.220085 117.175301 \n",
       "L 276.340567 117.175301 \n",
       "L 277.461049 117.175301 \n",
       "L 278.58153 117.175301 \n",
       "L 279.702012 117.175301 \n",
       "L 280.822494 117.175301 \n",
       "L 281.942976 117.175301 \n",
       "L 283.063458 117.175301 \n",
       "L 284.18394 117.175301 \n",
       "L 285.304422 117.175301 \n",
       "L 286.424904 117.175301 \n",
       "L 287.545386 117.175301 \n",
       "L 288.665868 117.175301 \n",
       "L 289.78635 117.175301 \n",
       "L 290.906832 117.175301 \n",
       "L 292.027314 117.175301 \n",
       "L 293.147796 117.175301 \n",
       "L 294.268277 117.175301 \n",
       "L 295.388759 117.175301 \n",
       "L 296.509241 117.175301 \n",
       "L 297.629723 117.175301 \n",
       "L 298.750205 117.175301 \n",
       "L 299.870687 117.175301 \n",
       "L 300.991169 117.175301 \n",
       "L 302.111651 117.175301 \n",
       "L 303.232133 117.175301 \n",
       "L 304.352615 117.175301 \n",
       "L 305.473097 117.175301 \n",
       "L 306.593579 117.175301 \n",
       "L 307.714061 117.175301 \n",
       "L 308.834543 117.175301 \n",
       "L 309.955024 117.175301 \n",
       "L 311.075506 117.175301 \n",
       "L 312.195988 117.175301 \n",
       "L 313.31647 117.175301 \n",
       "L 314.436952 117.175301 \n",
       "L 315.557434 117.175301 \n",
       "L 316.677916 117.175301 \n",
       "L 317.798398 117.175301 \n",
       "L 318.91888 117.175301 \n",
       "L 320.039362 117.175301 \n",
       "L 321.159844 117.175301 \n",
       "L 321.159844 116.90241 \n",
       "L 320.039362 116.90241 \n",
       "L 318.91888 116.90241 \n",
       "L 317.798398 116.90241 \n",
       "L 316.677916 116.90241 \n",
       "L 315.557434 116.90241 \n",
       "L 314.436952 116.90241 \n",
       "L 313.31647 116.90241 \n",
       "L 312.195988 116.90241 \n",
       "L 311.075506 116.90241 \n",
       "L 309.955024 116.90241 \n",
       "L 308.834543 116.90241 \n",
       "L 307.714061 116.90241 \n",
       "L 306.593579 116.90241 \n",
       "L 305.473097 116.90241 \n",
       "L 304.352615 116.90241 \n",
       "L 303.232133 116.90241 \n",
       "L 302.111651 116.90241 \n",
       "L 300.991169 116.90241 \n",
       "L 299.870687 116.90241 \n",
       "L 298.750205 116.90241 \n",
       "L 297.629723 116.90241 \n",
       "L 296.509241 116.90241 \n",
       "L 295.388759 116.90241 \n",
       "L 294.268277 116.90241 \n",
       "L 293.147796 116.90241 \n",
       "L 292.027314 116.90241 \n",
       "L 290.906832 116.90241 \n",
       "L 289.78635 116.90241 \n",
       "L 288.665868 116.90241 \n",
       "L 287.545386 116.90241 \n",
       "L 286.424904 116.90241 \n",
       "L 285.304422 116.90241 \n",
       "L 284.18394 116.90241 \n",
       "L 283.063458 116.90241 \n",
       "L 281.942976 116.90241 \n",
       "L 280.822494 116.90241 \n",
       "L 279.702012 116.90241 \n",
       "L 278.58153 116.90241 \n",
       "L 277.461049 116.90241 \n",
       "L 276.340567 116.90241 \n",
       "L 275.220085 116.90241 \n",
       "L 274.099603 116.90241 \n",
       "L 272.979121 116.90241 \n",
       "L 271.858639 116.90241 \n",
       "L 270.738157 116.90241 \n",
       "L 269.617675 116.90241 \n",
       "L 268.497193 116.90241 \n",
       "L 267.376711 116.90241 \n",
       "L 266.256229 116.90241 \n",
       "L 265.135747 116.90241 \n",
       "L 264.015265 116.90241 \n",
       "L 262.894784 116.90241 \n",
       "L 261.774302 116.90241 \n",
       "L 260.65382 116.90241 \n",
       "L 259.533338 116.90241 \n",
       "L 258.412856 116.90241 \n",
       "L 257.292374 116.90241 \n",
       "L 256.171892 116.90241 \n",
       "L 255.05141 116.90241 \n",
       "L 253.930928 116.90241 \n",
       "L 252.810446 116.90241 \n",
       "L 251.689964 116.90241 \n",
       "L 250.569482 116.90241 \n",
       "L 249.449 116.90241 \n",
       "L 248.328518 116.90241 \n",
       "L 247.208037 116.90241 \n",
       "L 246.087555 116.90241 \n",
       "L 244.967073 116.90241 \n",
       "L 243.846591 116.90241 \n",
       "L 242.726109 116.90241 \n",
       "L 241.605627 116.90241 \n",
       "L 240.485145 116.90241 \n",
       "L 239.364663 116.90241 \n",
       "L 238.244181 116.90241 \n",
       "L 237.123699 116.90241 \n",
       "L 236.003217 116.90241 \n",
       "L 234.882735 116.90241 \n",
       "L 233.762253 116.90241 \n",
       "L 232.641771 116.90241 \n",
       "L 231.52129 116.90241 \n",
       "L 230.400808 116.90241 \n",
       "L 229.280326 116.90241 \n",
       "L 228.159844 116.90241 \n",
       "L 227.039362 116.90241 \n",
       "L 225.91888 116.90241 \n",
       "L 225.358639 117.448193 \n",
       "L 225.358639 118.539759 \n",
       "L 225.358639 119.631325 \n",
       "L 225.358639 120.722892 \n",
       "L 225.358639 121.814458 \n",
       "L 225.358639 122.906024 \n",
       "L 225.358639 123.99759 \n",
       "L 225.358639 125.089157 \n",
       "L 225.358639 126.180723 \n",
       "L 225.358639 127.272289 \n",
       "L 225.358639 128.363855 \n",
       "L 225.358639 129.455422 \n",
       "L 225.358639 130.546988 \n",
       "L 225.358639 131.638554 \n",
       "L 225.358639 132.73012 \n",
       "L 225.358639 133.821687 \n",
       "L 225.358639 134.913253 \n",
       "L 225.358639 136.004819 \n",
       "L 225.358639 137.096386 \n",
       "L 225.358639 138.187952 \n",
       "L 225.358639 139.279518 \n",
       "L 225.358639 140.371084 \n",
       "L 225.358639 141.462651 \n",
       "L 225.358639 142.554217 \n",
       "L 225.358639 143.645783 \n",
       "L 225.358639 144.737349 \n",
       "L 225.358639 145.828916 \n",
       "L 225.358639 146.920482 \n",
       "L 225.358639 148.012048 \n",
       "L 225.358639 149.103614 \n",
       "L 225.358639 150.195181 \n",
       "L 225.358639 151.286747 \n",
       "L 225.358639 152.378313 \n",
       "L 225.358639 153.46988 \n",
       "L 225.358639 154.561446 \n",
       "L 225.358639 155.653012 \n",
       "L 225.358639 156.744578 \n",
       "L 225.358639 157.836145 \n",
       "L 225.358639 158.927711 \n",
       "L 225.358639 160.019277 \n",
       "L 225.358639 161.110843 \n",
       "L 225.358639 162.20241 \n",
       "L 225.358639 163.293976 \n",
       "L 225.358639 164.385542 \n",
       "L 225.358639 165.477108 \n",
       "L 225.358639 166.568675 \n",
       "L 225.358639 167.660241 \n",
       "L 225.358639 168.751807 \n",
       "L 225.358639 169.843373 \n",
       "L 225.358639 170.93494 \n",
       "L 225.358639 172.026506 \n",
       "L 225.358639 173.118072 \n",
       "L 225.358639 174.209639 \n",
       "L 225.358639 175.301205 \n",
       "L 225.358639 176.392771 \n",
       "L 225.358639 177.484337 \n",
       "L 225.358639 178.575904 \n",
       "L 225.358639 179.66747 \n",
       "L 225.358639 180.759036 \n",
       "L 225.358639 181.850602 \n",
       "L 225.358639 182.942169 \n",
       "L 225.358639 184.033735 \n",
       "L 225.358639 185.125301 \n",
       "L 225.358639 186.216867 \n",
       "L 225.358639 187.308434 \n",
       "L 225.358639 188.4 \n",
       "L 225.358639 189.491566 \n",
       "L 225.358639 190.583133 \n",
       "L 225.358639 191.674699 \n",
       "L 225.358639 192.766265 \n",
       "L 225.358639 193.857831 \n",
       "L 225.358639 194.949398 \n",
       "z\n",
       "\" style=\"fill:#bcbd22;fill-opacity:0.15;\"/>\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 43.280326 105.986747 \n",
       "L 44.400808 105.986747 \n",
       "L 45.52129 105.986747 \n",
       "L 46.641771 105.986747 \n",
       "L 47.762253 105.986747 \n",
       "L 48.882735 105.986747 \n",
       "L 50.003217 105.986747 \n",
       "L 51.123699 105.986747 \n",
       "L 52.244181 105.986747 \n",
       "L 53.364663 105.986747 \n",
       "L 54.485145 105.986747 \n",
       "L 55.605627 105.986747 \n",
       "L 56.726109 105.986747 \n",
       "L 57.846591 105.986747 \n",
       "L 58.967073 105.986747 \n",
       "L 60.087555 105.986747 \n",
       "L 61.208037 105.986747 \n",
       "L 62.328518 105.986747 \n",
       "L 63.449 105.986747 \n",
       "L 64.569482 105.986747 \n",
       "L 65.689964 105.986747 \n",
       "L 66.810446 105.986747 \n",
       "L 67.930928 105.986747 \n",
       "L 69.05141 105.986747 \n",
       "L 70.171892 105.986747 \n",
       "L 71.292374 105.986747 \n",
       "L 72.412856 105.986747 \n",
       "L 73.533338 105.986747 \n",
       "L 74.65382 105.986747 \n",
       "L 75.774302 105.986747 \n",
       "L 76.894784 105.986747 \n",
       "L 78.015265 105.986747 \n",
       "L 79.135747 105.986747 \n",
       "L 80.256229 105.986747 \n",
       "L 81.376711 105.986747 \n",
       "L 82.497193 105.986747 \n",
       "L 83.617675 105.986747 \n",
       "L 84.738157 105.986747 \n",
       "L 85.858639 105.986747 \n",
       "L 86.979121 105.986747 \n",
       "L 88.099603 105.986747 \n",
       "L 89.220085 105.986747 \n",
       "L 90.340567 105.986747 \n",
       "L 91.461049 105.986747 \n",
       "L 92.58153 105.986747 \n",
       "L 93.702012 105.986747 \n",
       "L 94.822494 105.986747 \n",
       "L 95.942976 105.986747 \n",
       "L 97.063458 105.986747 \n",
       "L 98.18394 105.986747 \n",
       "L 99.304422 105.986747 \n",
       "L 100.424904 105.986747 \n",
       "L 101.545386 105.986747 \n",
       "L 102.665868 105.986747 \n",
       "L 103.78635 105.986747 \n",
       "L 104.906832 105.986747 \n",
       "L 106.027314 105.986747 \n",
       "L 107.147796 105.986747 \n",
       "L 108.268277 105.986747 \n",
       "L 109.388759 105.986747 \n",
       "L 110.509241 105.986747 \n",
       "L 111.629723 105.986747 \n",
       "L 112.750205 105.986747 \n",
       "L 113.870687 105.986747 \n",
       "L 114.991169 105.986747 \n",
       "L 116.111651 105.986747 \n",
       "L 117.232133 105.986747 \n",
       "L 118.352615 105.986747 \n",
       "L 119.473097 105.986747 \n",
       "L 120.593579 105.986747 \n",
       "L 121.714061 105.986747 \n",
       "L 122.834543 105.986747 \n",
       "L 123.955024 105.986747 \n",
       "L 125.075506 105.986747 \n",
       "L 126.195988 105.986747 \n",
       "L 127.31647 105.986747 \n",
       "L 128.436952 105.986747 \n",
       "L 129.557434 105.986747 \n",
       "L 130.677916 105.986747 \n",
       "L 131.798398 105.986747 \n",
       "L 132.91888 105.986747 \n",
       "L 134.039362 105.986747 \n",
       "L 135.159844 105.986747 \n",
       "L 136.280326 105.986747 \n",
       "L 137.400808 105.986747 \n",
       "L 138.52129 105.986747 \n",
       "L 139.641771 105.986747 \n",
       "L 140.762253 105.986747 \n",
       "L 141.882735 105.986747 \n",
       "L 143.003217 105.986747 \n",
       "L 144.123699 105.986747 \n",
       "L 145.244181 105.986747 \n",
       "L 146.364663 105.986747 \n",
       "L 147.485145 105.986747 \n",
       "L 148.605627 105.986747 \n",
       "L 149.726109 105.986747 \n",
       "L 150.846591 105.986747 \n",
       "L 151.967073 105.986747 \n",
       "L 153.087555 105.986747 \n",
       "L 154.208037 105.986747 \n",
       "L 155.328518 105.986747 \n",
       "L 156.449 105.986747 \n",
       "L 157.569482 105.986747 \n",
       "L 158.689964 105.986747 \n",
       "L 159.810446 105.986747 \n",
       "L 160.930928 105.986747 \n",
       "L 162.05141 105.986747 \n",
       "L 163.171892 105.986747 \n",
       "L 164.292374 105.986747 \n",
       "L 165.412856 105.986747 \n",
       "L 166.533338 105.986747 \n",
       "L 167.65382 105.986747 \n",
       "L 168.774302 105.986747 \n",
       "L 169.894784 105.986747 \n",
       "L 171.015265 105.986747 \n",
       "L 172.135747 105.986747 \n",
       "L 173.256229 105.986747 \n",
       "L 174.376711 105.986747 \n",
       "L 175.497193 105.986747 \n",
       "L 176.617675 105.986747 \n",
       "L 177.738157 105.986747 \n",
       "L 178.858639 105.986747 \n",
       "L 179.979121 105.986747 \n",
       "L 181.099603 105.986747 \n",
       "L 182.220085 105.986747 \n",
       "L 183.340567 105.986747 \n",
       "L 184.461049 105.986747 \n",
       "L 185.58153 105.986747 \n",
       "L 186.702012 105.986747 \n",
       "L 187.822494 105.986747 \n",
       "L 188.942976 105.986747 \n",
       "L 190.063458 105.986747 \n",
       "L 191.18394 105.986747 \n",
       "L 192.304422 105.986747 \n",
       "L 193.424904 105.986747 \n",
       "L 194.545386 105.986747 \n",
       "L 195.665868 105.986747 \n",
       "L 196.78635 105.986747 \n",
       "L 197.906832 105.986747 \n",
       "L 199.027314 105.986747 \n",
       "L 200.147796 105.986747 \n",
       "L 201.268277 105.986747 \n",
       "L 202.388759 105.986747 \n",
       "L 203.509241 105.986747 \n",
       "L 204.629723 105.986747 \n",
       "L 205.750205 105.986747 \n",
       "L 206.870687 105.986747 \n",
       "L 207.991169 105.986747 \n",
       "L 209.111651 105.986747 \n",
       "L 210.232133 105.986747 \n",
       "L 211.352615 105.986747 \n",
       "L 212.473097 105.986747 \n",
       "L 213.593579 105.986747 \n",
       "L 214.714061 105.986747 \n",
       "L 215.834543 105.986747 \n",
       "L 216.955024 105.986747 \n",
       "L 218.075506 105.986747 \n",
       "L 219.195988 105.986747 \n",
       "L 220.31647 105.986747 \n",
       "L 221.436952 105.986747 \n",
       "L 222.557434 105.986747 \n",
       "L 223.677916 105.986747 \n",
       "L 224.798398 105.986747 \n",
       "L 225.358639 105.440964 \n",
       "L 225.358639 104.349398 \n",
       "L 225.358639 103.257831 \n",
       "L 225.358639 102.166265 \n",
       "L 225.358639 101.074699 \n",
       "L 225.358639 99.983133 \n",
       "L 225.358639 98.891566 \n",
       "L 225.358639 97.8 \n",
       "L 225.358639 96.708434 \n",
       "L 225.91888 96.162651 \n",
       "L 227.039362 96.162651 \n",
       "L 228.159844 96.162651 \n",
       "L 229.280326 96.162651 \n",
       "L 230.400808 96.162651 \n",
       "L 231.52129 96.162651 \n",
       "L 232.641771 96.162651 \n",
       "L 233.762253 96.162651 \n",
       "L 234.882735 96.162651 \n",
       "L 236.003217 96.162651 \n",
       "L 237.123699 96.162651 \n",
       "L 238.244181 96.162651 \n",
       "L 239.364663 96.162651 \n",
       "L 240.485145 96.162651 \n",
       "L 241.605627 96.162651 \n",
       "L 242.726109 96.162651 \n",
       "L 243.846591 96.162651 \n",
       "L 244.967073 96.162651 \n",
       "L 246.087555 96.162651 \n",
       "L 247.208037 96.162651 \n",
       "L 248.328518 96.162651 \n",
       "L 249.449 96.162651 \n",
       "L 250.569482 96.162651 \n",
       "L 251.689964 96.162651 \n",
       "L 252.810446 96.162651 \n",
       "L 253.930928 96.162651 \n",
       "L 255.05141 96.162651 \n",
       "L 256.171892 96.162651 \n",
       "L 257.292374 96.162651 \n",
       "L 258.412856 96.162651 \n",
       "L 259.533338 96.162651 \n",
       "L 260.65382 96.162651 \n",
       "L 261.774302 96.162651 \n",
       "L 262.894784 96.162651 \n",
       "L 264.015265 96.162651 \n",
       "L 265.135747 96.162651 \n",
       "L 266.256229 96.162651 \n",
       "L 267.376711 96.162651 \n",
       "L 268.497193 96.162651 \n",
       "L 269.617675 96.162651 \n",
       "L 270.738157 96.162651 \n",
       "L 271.858639 96.162651 \n",
       "L 272.979121 96.162651 \n",
       "L 274.099603 96.162651 \n",
       "L 275.220085 96.162651 \n",
       "L 276.340567 96.162651 \n",
       "L 277.461049 96.162651 \n",
       "L 278.58153 96.162651 \n",
       "L 279.702012 96.162651 \n",
       "L 280.822494 96.162651 \n",
       "L 281.942976 96.162651 \n",
       "L 283.063458 96.162651 \n",
       "L 284.18394 96.162651 \n",
       "L 285.304422 96.162651 \n",
       "L 286.424904 96.162651 \n",
       "L 287.545386 96.162651 \n",
       "L 288.665868 96.162651 \n",
       "L 289.78635 96.162651 \n",
       "L 290.906832 96.162651 \n",
       "L 292.027314 96.162651 \n",
       "L 293.147796 96.162651 \n",
       "L 294.268277 96.162651 \n",
       "L 295.388759 96.162651 \n",
       "L 296.509241 96.162651 \n",
       "L 297.629723 96.162651 \n",
       "L 298.750205 96.162651 \n",
       "L 299.870687 96.162651 \n",
       "L 300.991169 96.162651 \n",
       "L 302.111651 96.162651 \n",
       "L 303.232133 96.162651 \n",
       "L 304.352615 96.162651 \n",
       "L 305.473097 96.162651 \n",
       "L 306.593579 96.162651 \n",
       "L 307.714061 96.162651 \n",
       "L 308.834543 96.162651 \n",
       "L 309.955024 96.162651 \n",
       "L 311.075506 96.162651 \n",
       "L 312.195988 96.162651 \n",
       "L 313.31647 96.162651 \n",
       "L 314.436952 96.162651 \n",
       "L 315.557434 96.162651 \n",
       "L 316.677916 96.162651 \n",
       "L 317.798398 96.162651 \n",
       "L 318.91888 96.162651 \n",
       "L 320.039362 96.162651 \n",
       "L 321.159844 96.162651 \n",
       "L 321.159844 95.889759 \n",
       "L 320.039362 95.889759 \n",
       "L 318.91888 95.889759 \n",
       "L 317.798398 95.889759 \n",
       "L 316.677916 95.889759 \n",
       "L 315.557434 95.889759 \n",
       "L 314.436952 95.889759 \n",
       "L 313.31647 95.889759 \n",
       "L 312.195988 95.889759 \n",
       "L 311.075506 95.889759 \n",
       "L 309.955024 95.889759 \n",
       "L 308.834543 95.889759 \n",
       "L 307.714061 95.889759 \n",
       "L 306.593579 95.889759 \n",
       "L 305.473097 95.889759 \n",
       "L 304.352615 95.889759 \n",
       "L 303.232133 95.889759 \n",
       "L 302.111651 95.889759 \n",
       "L 300.991169 95.889759 \n",
       "L 299.870687 95.889759 \n",
       "L 298.750205 95.889759 \n",
       "L 297.629723 95.889759 \n",
       "L 296.509241 95.889759 \n",
       "L 295.388759 95.889759 \n",
       "L 294.268277 95.889759 \n",
       "L 293.147796 95.889759 \n",
       "L 292.027314 95.889759 \n",
       "L 290.906832 95.889759 \n",
       "L 289.78635 95.889759 \n",
       "L 288.665868 95.889759 \n",
       "L 287.545386 95.889759 \n",
       "L 286.424904 95.889759 \n",
       "L 285.304422 95.889759 \n",
       "L 284.18394 95.889759 \n",
       "L 283.063458 95.889759 \n",
       "L 281.942976 95.889759 \n",
       "L 280.822494 95.889759 \n",
       "L 279.702012 95.889759 \n",
       "L 278.58153 95.889759 \n",
       "L 277.461049 95.889759 \n",
       "L 276.340567 95.889759 \n",
       "L 275.220085 95.889759 \n",
       "L 274.099603 95.889759 \n",
       "L 272.979121 95.889759 \n",
       "L 271.858639 95.889759 \n",
       "L 270.738157 95.889759 \n",
       "L 269.617675 95.889759 \n",
       "L 268.497193 95.889759 \n",
       "L 267.376711 95.889759 \n",
       "L 266.256229 95.889759 \n",
       "L 265.135747 95.889759 \n",
       "L 264.015265 95.889759 \n",
       "L 262.894784 95.889759 \n",
       "L 261.774302 95.889759 \n",
       "L 260.65382 95.889759 \n",
       "L 259.533338 95.889759 \n",
       "L 258.412856 95.889759 \n",
       "L 257.292374 95.889759 \n",
       "L 256.171892 95.889759 \n",
       "L 255.05141 95.889759 \n",
       "L 253.930928 95.889759 \n",
       "L 252.810446 95.889759 \n",
       "L 251.689964 95.889759 \n",
       "L 250.569482 95.889759 \n",
       "L 249.449 95.889759 \n",
       "L 248.328518 95.889759 \n",
       "L 247.208037 95.889759 \n",
       "L 246.087555 95.889759 \n",
       "L 244.967073 95.889759 \n",
       "L 243.846591 95.889759 \n",
       "L 242.726109 95.889759 \n",
       "L 241.605627 95.889759 \n",
       "L 240.485145 95.889759 \n",
       "L 239.364663 95.889759 \n",
       "L 238.244181 95.889759 \n",
       "L 237.123699 95.889759 \n",
       "L 236.003217 95.889759 \n",
       "L 234.882735 95.889759 \n",
       "L 233.762253 95.889759 \n",
       "L 232.641771 95.889759 \n",
       "L 231.52129 95.889759 \n",
       "L 230.400808 95.889759 \n",
       "L 229.280326 95.889759 \n",
       "L 228.159844 95.889759 \n",
       "L 227.039362 95.889759 \n",
       "L 225.91888 95.889759 \n",
       "L 225.078518 96.708434 \n",
       "L 225.078518 97.8 \n",
       "L 225.078518 98.891566 \n",
       "L 225.078518 99.983133 \n",
       "L 225.078518 101.074699 \n",
       "L 225.078518 102.166265 \n",
       "L 225.078518 103.257831 \n",
       "L 225.078518 104.349398 \n",
       "L 225.078518 105.440964 \n",
       "L 224.798398 105.713855 \n",
       "L 223.677916 105.713855 \n",
       "L 222.557434 105.713855 \n",
       "L 221.436952 105.713855 \n",
       "L 220.31647 105.713855 \n",
       "L 219.195988 105.713855 \n",
       "L 218.075506 105.713855 \n",
       "L 216.955024 105.713855 \n",
       "L 215.834543 105.713855 \n",
       "L 214.714061 105.713855 \n",
       "L 213.593579 105.713855 \n",
       "L 212.473097 105.713855 \n",
       "L 211.352615 105.713855 \n",
       "L 210.232133 105.713855 \n",
       "L 209.111651 105.713855 \n",
       "L 207.991169 105.713855 \n",
       "L 206.870687 105.713855 \n",
       "L 205.750205 105.713855 \n",
       "L 204.629723 105.713855 \n",
       "L 203.509241 105.713855 \n",
       "L 202.388759 105.713855 \n",
       "L 201.268277 105.713855 \n",
       "L 200.147796 105.713855 \n",
       "L 199.027314 105.713855 \n",
       "L 197.906832 105.713855 \n",
       "L 196.78635 105.713855 \n",
       "L 195.665868 105.713855 \n",
       "L 194.545386 105.713855 \n",
       "L 193.424904 105.713855 \n",
       "L 192.304422 105.713855 \n",
       "L 191.18394 105.713855 \n",
       "L 190.063458 105.713855 \n",
       "L 188.942976 105.713855 \n",
       "L 187.822494 105.713855 \n",
       "L 186.702012 105.713855 \n",
       "L 185.58153 105.713855 \n",
       "L 184.461049 105.713855 \n",
       "L 183.340567 105.713855 \n",
       "L 182.220085 105.713855 \n",
       "L 181.099603 105.713855 \n",
       "L 179.979121 105.713855 \n",
       "L 178.858639 105.713855 \n",
       "L 177.738157 105.713855 \n",
       "L 176.617675 105.713855 \n",
       "L 175.497193 105.713855 \n",
       "L 174.376711 105.713855 \n",
       "L 173.256229 105.713855 \n",
       "L 172.135747 105.713855 \n",
       "L 171.015265 105.713855 \n",
       "L 169.894784 105.713855 \n",
       "L 168.774302 105.713855 \n",
       "L 167.65382 105.713855 \n",
       "L 166.533338 105.713855 \n",
       "L 165.412856 105.713855 \n",
       "L 164.292374 105.713855 \n",
       "L 163.171892 105.713855 \n",
       "L 162.05141 105.713855 \n",
       "L 160.930928 105.713855 \n",
       "L 159.810446 105.713855 \n",
       "L 158.689964 105.713855 \n",
       "L 157.569482 105.713855 \n",
       "L 156.449 105.713855 \n",
       "L 155.328518 105.713855 \n",
       "L 154.208037 105.713855 \n",
       "L 153.087555 105.713855 \n",
       "L 151.967073 105.713855 \n",
       "L 150.846591 105.713855 \n",
       "L 149.726109 105.713855 \n",
       "L 148.605627 105.713855 \n",
       "L 147.485145 105.713855 \n",
       "L 146.364663 105.713855 \n",
       "L 145.244181 105.713855 \n",
       "L 144.123699 105.713855 \n",
       "L 143.003217 105.713855 \n",
       "L 141.882735 105.713855 \n",
       "L 140.762253 105.713855 \n",
       "L 139.641771 105.713855 \n",
       "L 138.52129 105.713855 \n",
       "L 137.400808 105.713855 \n",
       "L 136.280326 105.713855 \n",
       "L 135.159844 105.713855 \n",
       "L 134.039362 105.713855 \n",
       "L 132.91888 105.713855 \n",
       "L 131.798398 105.713855 \n",
       "L 130.677916 105.713855 \n",
       "L 129.557434 105.713855 \n",
       "L 128.436952 105.713855 \n",
       "L 127.31647 105.713855 \n",
       "L 126.195988 105.713855 \n",
       "L 125.075506 105.713855 \n",
       "L 123.955024 105.713855 \n",
       "L 122.834543 105.713855 \n",
       "L 121.714061 105.713855 \n",
       "L 120.593579 105.713855 \n",
       "L 119.473097 105.713855 \n",
       "L 118.352615 105.713855 \n",
       "L 117.232133 105.713855 \n",
       "L 116.111651 105.713855 \n",
       "L 114.991169 105.713855 \n",
       "L 113.870687 105.713855 \n",
       "L 112.750205 105.713855 \n",
       "L 111.629723 105.713855 \n",
       "L 110.509241 105.713855 \n",
       "L 109.388759 105.713855 \n",
       "L 108.268277 105.713855 \n",
       "L 107.147796 105.713855 \n",
       "L 106.027314 105.713855 \n",
       "L 104.906832 105.713855 \n",
       "L 103.78635 105.713855 \n",
       "L 102.665868 105.713855 \n",
       "L 101.545386 105.713855 \n",
       "L 100.424904 105.713855 \n",
       "L 99.304422 105.713855 \n",
       "L 98.18394 105.713855 \n",
       "L 97.063458 105.713855 \n",
       "L 95.942976 105.713855 \n",
       "L 94.822494 105.713855 \n",
       "L 93.702012 105.713855 \n",
       "L 92.58153 105.713855 \n",
       "L 91.461049 105.713855 \n",
       "L 90.340567 105.713855 \n",
       "L 89.220085 105.713855 \n",
       "L 88.099603 105.713855 \n",
       "L 86.979121 105.713855 \n",
       "L 85.858639 105.713855 \n",
       "L 84.738157 105.713855 \n",
       "L 83.617675 105.713855 \n",
       "L 82.497193 105.713855 \n",
       "L 81.376711 105.713855 \n",
       "L 80.256229 105.713855 \n",
       "L 79.135747 105.713855 \n",
       "L 78.015265 105.713855 \n",
       "L 76.894784 105.713855 \n",
       "L 75.774302 105.713855 \n",
       "L 74.65382 105.713855 \n",
       "L 73.533338 105.713855 \n",
       "L 72.412856 105.713855 \n",
       "L 71.292374 105.713855 \n",
       "L 70.171892 105.713855 \n",
       "L 69.05141 105.713855 \n",
       "L 67.930928 105.713855 \n",
       "L 66.810446 105.713855 \n",
       "L 65.689964 105.713855 \n",
       "L 64.569482 105.713855 \n",
       "L 63.449 105.713855 \n",
       "L 62.328518 105.713855 \n",
       "L 61.208037 105.713855 \n",
       "L 60.087555 105.713855 \n",
       "L 58.967073 105.713855 \n",
       "L 57.846591 105.713855 \n",
       "L 56.726109 105.713855 \n",
       "L 55.605627 105.713855 \n",
       "L 54.485145 105.713855 \n",
       "L 53.364663 105.713855 \n",
       "L 52.244181 105.713855 \n",
       "L 51.123699 105.713855 \n",
       "L 50.003217 105.713855 \n",
       "L 48.882735 105.713855 \n",
       "L 47.762253 105.713855 \n",
       "L 46.641771 105.713855 \n",
       "L 45.52129 105.713855 \n",
       "L 44.400808 105.713855 \n",
       "L 43.280326 105.713855 \n",
       "L 42.159844 105.713855 \n",
       "L 42.159844 105.986747 \n",
       "z\n",
       "\" style=\"fill:#bcbd22;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_8\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 225.91888 195.085843 \n",
       "L 227.039362 195.085843 \n",
       "L 228.159844 195.085843 \n",
       "L 229.280326 195.085843 \n",
       "L 230.400808 195.085843 \n",
       "L 231.52129 195.085843 \n",
       "L 232.641771 195.085843 \n",
       "L 233.762253 195.085843 \n",
       "L 234.882735 195.085843 \n",
       "L 236.003217 195.085843 \n",
       "L 237.123699 195.085843 \n",
       "L 238.244181 195.085843 \n",
       "L 239.364663 195.085843 \n",
       "L 240.485145 195.085843 \n",
       "L 241.605627 195.085843 \n",
       "L 242.726109 195.085843 \n",
       "L 243.846591 195.085843 \n",
       "L 244.967073 195.085843 \n",
       "L 246.087555 195.085843 \n",
       "L 247.208037 195.085843 \n",
       "L 248.328518 195.085843 \n",
       "L 249.449 195.085843 \n",
       "L 250.569482 195.085843 \n",
       "L 251.689964 195.085843 \n",
       "L 252.810446 195.085843 \n",
       "L 253.930928 195.085843 \n",
       "L 255.05141 195.085843 \n",
       "L 256.171892 195.085843 \n",
       "L 257.292374 195.085843 \n",
       "L 258.412856 195.085843 \n",
       "L 259.533338 195.085843 \n",
       "L 260.65382 195.085843 \n",
       "L 261.774302 195.085843 \n",
       "L 262.894784 195.085843 \n",
       "L 264.015265 195.085843 \n",
       "L 265.135747 195.085843 \n",
       "L 266.256229 195.085843 \n",
       "L 267.376711 195.085843 \n",
       "L 268.497193 195.085843 \n",
       "L 269.617675 195.085843 \n",
       "L 270.738157 195.085843 \n",
       "L 271.858639 195.085843 \n",
       "L 272.979121 195.085843 \n",
       "L 274.099603 195.085843 \n",
       "L 275.220085 195.085843 \n",
       "L 276.340567 195.085843 \n",
       "L 277.461049 195.085843 \n",
       "L 278.58153 195.085843 \n",
       "L 279.702012 195.085843 \n",
       "L 280.822494 195.085843 \n",
       "L 281.942976 195.085843 \n",
       "L 283.063458 195.085843 \n",
       "L 284.18394 195.085843 \n",
       "L 285.304422 195.085843 \n",
       "L 286.424904 195.085843 \n",
       "L 287.545386 195.085843 \n",
       "L 288.665868 195.085843 \n",
       "L 289.78635 195.085843 \n",
       "L 290.906832 195.085843 \n",
       "L 292.027314 195.085843 \n",
       "L 293.147796 195.085843 \n",
       "L 294.268277 195.085843 \n",
       "L 295.388759 195.085843 \n",
       "L 296.509241 195.085843 \n",
       "L 297.629723 195.085843 \n",
       "L 298.750205 195.085843 \n",
       "L 299.870687 195.085843 \n",
       "L 300.991169 195.085843 \n",
       "L 302.111651 195.085843 \n",
       "L 303.232133 195.085843 \n",
       "L 304.352615 195.085843 \n",
       "L 305.473097 195.085843 \n",
       "L 306.593579 195.085843 \n",
       "L 307.714061 195.085843 \n",
       "L 308.834543 195.085843 \n",
       "L 309.955024 195.085843 \n",
       "L 311.075506 195.085843 \n",
       "L 312.195988 195.085843 \n",
       "L 313.31647 195.085843 \n",
       "L 314.436952 195.085843 \n",
       "L 315.557434 195.085843 \n",
       "L 316.677916 195.085843 \n",
       "L 317.798398 195.085843 \n",
       "L 318.91888 195.085843 \n",
       "L 320.039362 195.085843 \n",
       "L 321.159844 195.085843 \n",
       "L 321.159844 194.949398 \n",
       "L 321.159844 193.857831 \n",
       "L 321.159844 192.766265 \n",
       "L 321.159844 191.674699 \n",
       "L 321.159844 190.583133 \n",
       "L 321.159844 189.491566 \n",
       "L 321.159844 188.4 \n",
       "L 321.159844 187.308434 \n",
       "L 321.159844 186.216867 \n",
       "L 321.159844 185.125301 \n",
       "L 321.159844 184.033735 \n",
       "L 321.159844 182.942169 \n",
       "L 321.159844 181.850602 \n",
       "L 321.159844 180.759036 \n",
       "L 321.159844 179.66747 \n",
       "L 321.159844 178.575904 \n",
       "L 321.159844 177.484337 \n",
       "L 321.159844 176.392771 \n",
       "L 321.159844 175.301205 \n",
       "L 321.159844 174.209639 \n",
       "L 321.159844 173.118072 \n",
       "L 321.159844 172.026506 \n",
       "L 321.159844 170.93494 \n",
       "L 321.159844 169.843373 \n",
       "L 321.159844 168.751807 \n",
       "L 321.159844 167.660241 \n",
       "L 321.159844 166.568675 \n",
       "L 321.159844 165.477108 \n",
       "L 321.159844 164.385542 \n",
       "L 321.159844 163.293976 \n",
       "L 321.159844 162.20241 \n",
       "L 321.159844 161.110843 \n",
       "L 321.159844 160.019277 \n",
       "L 321.159844 158.927711 \n",
       "L 321.159844 157.836145 \n",
       "L 321.159844 156.744578 \n",
       "L 321.159844 155.653012 \n",
       "L 321.159844 154.561446 \n",
       "L 321.159844 153.46988 \n",
       "L 321.159844 152.378313 \n",
       "L 321.159844 151.286747 \n",
       "L 321.159844 150.195181 \n",
       "L 321.159844 149.103614 \n",
       "L 321.159844 148.012048 \n",
       "L 321.159844 146.920482 \n",
       "L 321.159844 145.828916 \n",
       "L 321.159844 144.737349 \n",
       "L 321.159844 143.645783 \n",
       "L 321.159844 142.554217 \n",
       "L 321.159844 141.462651 \n",
       "L 321.159844 140.371084 \n",
       "L 321.159844 139.279518 \n",
       "L 321.159844 138.187952 \n",
       "L 321.159844 137.096386 \n",
       "L 321.159844 136.004819 \n",
       "L 321.159844 134.913253 \n",
       "L 321.159844 133.821687 \n",
       "L 321.159844 132.73012 \n",
       "L 321.159844 131.638554 \n",
       "L 321.159844 130.546988 \n",
       "L 321.159844 129.455422 \n",
       "L 321.159844 128.363855 \n",
       "L 321.159844 127.272289 \n",
       "L 321.159844 126.180723 \n",
       "L 321.159844 125.089157 \n",
       "L 321.159844 123.99759 \n",
       "L 321.159844 122.906024 \n",
       "L 321.159844 121.814458 \n",
       "L 321.159844 120.722892 \n",
       "L 321.159844 119.631325 \n",
       "L 321.159844 118.539759 \n",
       "L 321.159844 117.448193 \n",
       "L 321.159844 117.175301 \n",
       "L 320.039362 117.175301 \n",
       "L 318.91888 117.175301 \n",
       "L 317.798398 117.175301 \n",
       "L 316.677916 117.175301 \n",
       "L 315.557434 117.175301 \n",
       "L 314.436952 117.175301 \n",
       "L 313.31647 117.175301 \n",
       "L 312.195988 117.175301 \n",
       "L 311.075506 117.175301 \n",
       "L 309.955024 117.175301 \n",
       "L 308.834543 117.175301 \n",
       "L 307.714061 117.175301 \n",
       "L 306.593579 117.175301 \n",
       "L 305.473097 117.175301 \n",
       "L 304.352615 117.175301 \n",
       "L 303.232133 117.175301 \n",
       "L 302.111651 117.175301 \n",
       "L 300.991169 117.175301 \n",
       "L 299.870687 117.175301 \n",
       "L 298.750205 117.175301 \n",
       "L 297.629723 117.175301 \n",
       "L 296.509241 117.175301 \n",
       "L 295.388759 117.175301 \n",
       "L 294.268277 117.175301 \n",
       "L 293.147796 117.175301 \n",
       "L 292.027314 117.175301 \n",
       "L 290.906832 117.175301 \n",
       "L 289.78635 117.175301 \n",
       "L 288.665868 117.175301 \n",
       "L 287.545386 117.175301 \n",
       "L 286.424904 117.175301 \n",
       "L 285.304422 117.175301 \n",
       "L 284.18394 117.175301 \n",
       "L 283.063458 117.175301 \n",
       "L 281.942976 117.175301 \n",
       "L 280.822494 117.175301 \n",
       "L 279.702012 117.175301 \n",
       "L 278.58153 117.175301 \n",
       "L 277.461049 117.175301 \n",
       "L 276.340567 117.175301 \n",
       "L 275.220085 117.175301 \n",
       "L 274.099603 117.175301 \n",
       "L 272.979121 117.175301 \n",
       "L 271.858639 117.175301 \n",
       "L 270.738157 117.175301 \n",
       "L 269.617675 117.175301 \n",
       "L 268.497193 117.175301 \n",
       "L 267.376711 117.175301 \n",
       "L 266.256229 117.175301 \n",
       "L 265.135747 117.175301 \n",
       "L 264.015265 117.175301 \n",
       "L 262.894784 117.175301 \n",
       "L 261.774302 117.175301 \n",
       "L 260.65382 117.175301 \n",
       "L 259.533338 117.175301 \n",
       "L 258.412856 117.175301 \n",
       "L 257.292374 117.175301 \n",
       "L 256.171892 117.175301 \n",
       "L 255.05141 117.175301 \n",
       "L 253.930928 117.175301 \n",
       "L 252.810446 117.175301 \n",
       "L 251.689964 117.175301 \n",
       "L 250.569482 117.175301 \n",
       "L 249.449 117.175301 \n",
       "L 248.328518 117.175301 \n",
       "L 247.208037 117.175301 \n",
       "L 246.087555 117.175301 \n",
       "L 244.967073 117.175301 \n",
       "L 243.846591 117.175301 \n",
       "L 242.726109 117.175301 \n",
       "L 241.605627 117.175301 \n",
       "L 240.485145 117.175301 \n",
       "L 239.364663 117.175301 \n",
       "L 238.244181 117.175301 \n",
       "L 237.123699 117.175301 \n",
       "L 236.003217 117.175301 \n",
       "L 234.882735 117.175301 \n",
       "L 233.762253 117.175301 \n",
       "L 232.641771 117.175301 \n",
       "L 231.52129 117.175301 \n",
       "L 230.400808 117.175301 \n",
       "L 229.280326 117.175301 \n",
       "L 228.159844 117.175301 \n",
       "L 227.039362 117.175301 \n",
       "L 225.91888 117.175301 \n",
       "L 225.638759 117.448193 \n",
       "L 225.638759 118.539759 \n",
       "L 225.638759 119.631325 \n",
       "L 225.638759 120.722892 \n",
       "L 225.638759 121.814458 \n",
       "L 225.638759 122.906024 \n",
       "L 225.638759 123.99759 \n",
       "L 225.638759 125.089157 \n",
       "L 225.638759 126.180723 \n",
       "L 225.638759 127.272289 \n",
       "L 225.638759 128.363855 \n",
       "L 225.638759 129.455422 \n",
       "L 225.638759 130.546988 \n",
       "L 225.638759 131.638554 \n",
       "L 225.638759 132.73012 \n",
       "L 225.638759 133.821687 \n",
       "L 225.638759 134.913253 \n",
       "L 225.638759 136.004819 \n",
       "L 225.638759 137.096386 \n",
       "L 225.638759 138.187952 \n",
       "L 225.638759 139.279518 \n",
       "L 225.638759 140.371084 \n",
       "L 225.638759 141.462651 \n",
       "L 225.638759 142.554217 \n",
       "L 225.638759 143.645783 \n",
       "L 225.638759 144.737349 \n",
       "L 225.638759 145.828916 \n",
       "L 225.638759 146.920482 \n",
       "L 225.638759 148.012048 \n",
       "L 225.638759 149.103614 \n",
       "L 225.638759 150.195181 \n",
       "L 225.638759 151.286747 \n",
       "L 225.638759 152.378313 \n",
       "L 225.638759 153.46988 \n",
       "L 225.638759 154.561446 \n",
       "L 225.638759 155.653012 \n",
       "L 225.638759 156.744578 \n",
       "L 225.638759 157.836145 \n",
       "L 225.638759 158.927711 \n",
       "L 225.638759 160.019277 \n",
       "L 225.638759 161.110843 \n",
       "L 225.638759 162.20241 \n",
       "L 225.638759 163.293976 \n",
       "L 225.638759 164.385542 \n",
       "L 225.638759 165.477108 \n",
       "L 225.638759 166.568675 \n",
       "L 225.638759 167.660241 \n",
       "L 225.638759 168.751807 \n",
       "L 225.638759 169.843373 \n",
       "L 225.638759 170.93494 \n",
       "L 225.638759 172.026506 \n",
       "L 225.638759 173.118072 \n",
       "L 225.638759 174.209639 \n",
       "L 225.638759 175.301205 \n",
       "L 225.638759 176.392771 \n",
       "L 225.638759 177.484337 \n",
       "L 225.638759 178.575904 \n",
       "L 225.638759 179.66747 \n",
       "L 225.638759 180.759036 \n",
       "L 225.638759 181.850602 \n",
       "L 225.638759 182.942169 \n",
       "L 225.638759 184.033735 \n",
       "L 225.638759 185.125301 \n",
       "L 225.638759 186.216867 \n",
       "L 225.638759 187.308434 \n",
       "L 225.638759 188.4 \n",
       "L 225.638759 189.491566 \n",
       "L 225.638759 190.583133 \n",
       "L 225.638759 191.674699 \n",
       "L 225.638759 192.766265 \n",
       "L 225.638759 193.857831 \n",
       "L 225.638759 194.949398 \n",
       "z\n",
       "\" style=\"fill:#17becf;fill-opacity:0.15;\"/>\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 43.280326 105.713855 \n",
       "L 44.400808 105.713855 \n",
       "L 45.52129 105.713855 \n",
       "L 46.641771 105.713855 \n",
       "L 47.762253 105.713855 \n",
       "L 48.882735 105.713855 \n",
       "L 50.003217 105.713855 \n",
       "L 51.123699 105.713855 \n",
       "L 52.244181 105.713855 \n",
       "L 53.364663 105.713855 \n",
       "L 54.485145 105.713855 \n",
       "L 55.605627 105.713855 \n",
       "L 56.726109 105.713855 \n",
       "L 57.846591 105.713855 \n",
       "L 58.967073 105.713855 \n",
       "L 60.087555 105.713855 \n",
       "L 61.208037 105.713855 \n",
       "L 62.328518 105.713855 \n",
       "L 63.449 105.713855 \n",
       "L 64.569482 105.713855 \n",
       "L 65.689964 105.713855 \n",
       "L 66.810446 105.713855 \n",
       "L 67.930928 105.713855 \n",
       "L 69.05141 105.713855 \n",
       "L 70.171892 105.713855 \n",
       "L 71.292374 105.713855 \n",
       "L 72.412856 105.713855 \n",
       "L 73.533338 105.713855 \n",
       "L 74.65382 105.713855 \n",
       "L 75.774302 105.713855 \n",
       "L 76.894784 105.713855 \n",
       "L 78.015265 105.713855 \n",
       "L 79.135747 105.713855 \n",
       "L 80.256229 105.713855 \n",
       "L 81.376711 105.713855 \n",
       "L 82.497193 105.713855 \n",
       "L 83.617675 105.713855 \n",
       "L 84.738157 105.713855 \n",
       "L 85.858639 105.713855 \n",
       "L 86.979121 105.713855 \n",
       "L 88.099603 105.713855 \n",
       "L 89.220085 105.713855 \n",
       "L 90.340567 105.713855 \n",
       "L 91.461049 105.713855 \n",
       "L 92.58153 105.713855 \n",
       "L 93.702012 105.713855 \n",
       "L 94.822494 105.713855 \n",
       "L 95.942976 105.713855 \n",
       "L 97.063458 105.713855 \n",
       "L 98.18394 105.713855 \n",
       "L 99.304422 105.713855 \n",
       "L 100.424904 105.713855 \n",
       "L 101.545386 105.713855 \n",
       "L 102.665868 105.713855 \n",
       "L 103.78635 105.713855 \n",
       "L 104.906832 105.713855 \n",
       "L 106.027314 105.713855 \n",
       "L 107.147796 105.713855 \n",
       "L 108.268277 105.713855 \n",
       "L 109.388759 105.713855 \n",
       "L 110.509241 105.713855 \n",
       "L 111.629723 105.713855 \n",
       "L 112.750205 105.713855 \n",
       "L 113.870687 105.713855 \n",
       "L 114.991169 105.713855 \n",
       "L 116.111651 105.713855 \n",
       "L 117.232133 105.713855 \n",
       "L 118.352615 105.713855 \n",
       "L 119.473097 105.713855 \n",
       "L 120.593579 105.713855 \n",
       "L 121.714061 105.713855 \n",
       "L 122.834543 105.713855 \n",
       "L 123.955024 105.713855 \n",
       "L 125.075506 105.713855 \n",
       "L 126.195988 105.713855 \n",
       "L 127.31647 105.713855 \n",
       "L 128.436952 105.713855 \n",
       "L 129.557434 105.713855 \n",
       "L 130.677916 105.713855 \n",
       "L 131.798398 105.713855 \n",
       "L 132.91888 105.713855 \n",
       "L 134.039362 105.713855 \n",
       "L 135.159844 105.713855 \n",
       "L 136.280326 105.713855 \n",
       "L 137.400808 105.713855 \n",
       "L 138.52129 105.713855 \n",
       "L 139.641771 105.713855 \n",
       "L 140.762253 105.713855 \n",
       "L 141.882735 105.713855 \n",
       "L 143.003217 105.713855 \n",
       "L 144.123699 105.713855 \n",
       "L 145.244181 105.713855 \n",
       "L 146.364663 105.713855 \n",
       "L 147.485145 105.713855 \n",
       "L 148.605627 105.713855 \n",
       "L 149.726109 105.713855 \n",
       "L 150.846591 105.713855 \n",
       "L 151.967073 105.713855 \n",
       "L 153.087555 105.713855 \n",
       "L 154.208037 105.713855 \n",
       "L 155.328518 105.713855 \n",
       "L 156.449 105.713855 \n",
       "L 157.569482 105.713855 \n",
       "L 158.689964 105.713855 \n",
       "L 159.810446 105.713855 \n",
       "L 160.930928 105.713855 \n",
       "L 162.05141 105.713855 \n",
       "L 163.171892 105.713855 \n",
       "L 164.292374 105.713855 \n",
       "L 165.412856 105.713855 \n",
       "L 166.533338 105.713855 \n",
       "L 167.65382 105.713855 \n",
       "L 168.774302 105.713855 \n",
       "L 169.894784 105.713855 \n",
       "L 171.015265 105.713855 \n",
       "L 172.135747 105.713855 \n",
       "L 173.256229 105.713855 \n",
       "L 174.376711 105.713855 \n",
       "L 175.497193 105.713855 \n",
       "L 176.617675 105.713855 \n",
       "L 177.738157 105.713855 \n",
       "L 178.858639 105.713855 \n",
       "L 179.979121 105.713855 \n",
       "L 181.099603 105.713855 \n",
       "L 182.220085 105.713855 \n",
       "L 183.340567 105.713855 \n",
       "L 184.461049 105.713855 \n",
       "L 185.58153 105.713855 \n",
       "L 186.702012 105.713855 \n",
       "L 187.822494 105.713855 \n",
       "L 188.942976 105.713855 \n",
       "L 190.063458 105.713855 \n",
       "L 191.18394 105.713855 \n",
       "L 192.304422 105.713855 \n",
       "L 193.424904 105.713855 \n",
       "L 194.545386 105.713855 \n",
       "L 195.665868 105.713855 \n",
       "L 196.78635 105.713855 \n",
       "L 197.906832 105.713855 \n",
       "L 199.027314 105.713855 \n",
       "L 200.147796 105.713855 \n",
       "L 201.268277 105.713855 \n",
       "L 202.388759 105.713855 \n",
       "L 203.509241 105.713855 \n",
       "L 204.629723 105.713855 \n",
       "L 205.750205 105.713855 \n",
       "L 206.870687 105.713855 \n",
       "L 207.991169 105.713855 \n",
       "L 209.111651 105.713855 \n",
       "L 210.232133 105.713855 \n",
       "L 211.352615 105.713855 \n",
       "L 212.473097 105.713855 \n",
       "L 213.593579 105.713855 \n",
       "L 214.714061 105.713855 \n",
       "L 215.834543 105.713855 \n",
       "L 216.955024 105.713855 \n",
       "L 218.075506 105.713855 \n",
       "L 219.195988 105.713855 \n",
       "L 220.31647 105.713855 \n",
       "L 221.436952 105.713855 \n",
       "L 222.557434 105.713855 \n",
       "L 223.677916 105.713855 \n",
       "L 224.798398 105.713855 \n",
       "L 225.078518 105.440964 \n",
       "L 225.078518 104.349398 \n",
       "L 225.078518 103.257831 \n",
       "L 225.078518 102.166265 \n",
       "L 225.078518 101.074699 \n",
       "L 225.078518 99.983133 \n",
       "L 225.078518 98.891566 \n",
       "L 225.078518 97.8 \n",
       "L 225.078518 96.708434 \n",
       "L 225.91888 95.889759 \n",
       "L 227.039362 95.889759 \n",
       "L 228.159844 95.889759 \n",
       "L 229.280326 95.889759 \n",
       "L 230.400808 95.889759 \n",
       "L 231.52129 95.889759 \n",
       "L 232.641771 95.889759 \n",
       "L 233.762253 95.889759 \n",
       "L 234.882735 95.889759 \n",
       "L 236.003217 95.889759 \n",
       "L 237.123699 95.889759 \n",
       "L 238.244181 95.889759 \n",
       "L 239.364663 95.889759 \n",
       "L 240.485145 95.889759 \n",
       "L 241.605627 95.889759 \n",
       "L 242.726109 95.889759 \n",
       "L 243.846591 95.889759 \n",
       "L 244.967073 95.889759 \n",
       "L 246.087555 95.889759 \n",
       "L 247.208037 95.889759 \n",
       "L 248.328518 95.889759 \n",
       "L 249.449 95.889759 \n",
       "L 250.569482 95.889759 \n",
       "L 251.689964 95.889759 \n",
       "L 252.810446 95.889759 \n",
       "L 253.930928 95.889759 \n",
       "L 255.05141 95.889759 \n",
       "L 256.171892 95.889759 \n",
       "L 257.292374 95.889759 \n",
       "L 258.412856 95.889759 \n",
       "L 259.533338 95.889759 \n",
       "L 260.65382 95.889759 \n",
       "L 261.774302 95.889759 \n",
       "L 262.894784 95.889759 \n",
       "L 264.015265 95.889759 \n",
       "L 265.135747 95.889759 \n",
       "L 266.256229 95.889759 \n",
       "L 267.376711 95.889759 \n",
       "L 268.497193 95.889759 \n",
       "L 269.617675 95.889759 \n",
       "L 270.738157 95.889759 \n",
       "L 271.858639 95.889759 \n",
       "L 272.979121 95.889759 \n",
       "L 274.099603 95.889759 \n",
       "L 275.220085 95.889759 \n",
       "L 276.340567 95.889759 \n",
       "L 277.461049 95.889759 \n",
       "L 278.58153 95.889759 \n",
       "L 279.702012 95.889759 \n",
       "L 280.822494 95.889759 \n",
       "L 281.942976 95.889759 \n",
       "L 283.063458 95.889759 \n",
       "L 284.18394 95.889759 \n",
       "L 285.304422 95.889759 \n",
       "L 286.424904 95.889759 \n",
       "L 287.545386 95.889759 \n",
       "L 288.665868 95.889759 \n",
       "L 289.78635 95.889759 \n",
       "L 290.906832 95.889759 \n",
       "L 292.027314 95.889759 \n",
       "L 293.147796 95.889759 \n",
       "L 294.268277 95.889759 \n",
       "L 295.388759 95.889759 \n",
       "L 296.509241 95.889759 \n",
       "L 297.629723 95.889759 \n",
       "L 298.750205 95.889759 \n",
       "L 299.870687 95.889759 \n",
       "L 300.991169 95.889759 \n",
       "L 302.111651 95.889759 \n",
       "L 303.232133 95.889759 \n",
       "L 304.352615 95.889759 \n",
       "L 305.473097 95.889759 \n",
       "L 306.593579 95.889759 \n",
       "L 307.714061 95.889759 \n",
       "L 308.834543 95.889759 \n",
       "L 309.955024 95.889759 \n",
       "L 311.075506 95.889759 \n",
       "L 312.195988 95.889759 \n",
       "L 313.31647 95.889759 \n",
       "L 314.436952 95.889759 \n",
       "L 315.557434 95.889759 \n",
       "L 316.677916 95.889759 \n",
       "L 317.798398 95.889759 \n",
       "L 318.91888 95.889759 \n",
       "L 320.039362 95.889759 \n",
       "L 321.159844 95.889759 \n",
       "L 321.159844 95.616867 \n",
       "L 321.159844 94.525301 \n",
       "L 321.159844 93.433735 \n",
       "L 321.159844 92.342169 \n",
       "L 321.159844 91.250602 \n",
       "L 321.159844 90.159036 \n",
       "L 321.159844 89.06747 \n",
       "L 321.159844 87.975904 \n",
       "L 321.159844 86.884337 \n",
       "L 321.159844 85.792771 \n",
       "L 321.159844 84.701205 \n",
       "L 321.159844 83.609639 \n",
       "L 321.159844 82.518072 \n",
       "L 321.159844 81.426506 \n",
       "L 321.159844 80.33494 \n",
       "L 321.159844 79.243373 \n",
       "L 321.159844 78.151807 \n",
       "L 321.159844 77.060241 \n",
       "L 321.159844 75.968675 \n",
       "L 321.159844 74.877108 \n",
       "L 321.159844 73.785542 \n",
       "L 321.159844 72.693976 \n",
       "L 321.159844 71.60241 \n",
       "L 321.159844 70.510843 \n",
       "L 321.159844 69.419277 \n",
       "L 321.159844 68.327711 \n",
       "L 321.159844 67.236145 \n",
       "L 321.159844 66.144578 \n",
       "L 321.159844 65.053012 \n",
       "L 321.159844 63.961446 \n",
       "L 321.159844 62.86988 \n",
       "L 321.159844 61.778313 \n",
       "L 321.159844 60.686747 \n",
       "L 321.159844 59.595181 \n",
       "L 321.159844 58.503614 \n",
       "L 321.159844 57.412048 \n",
       "L 321.159844 56.320482 \n",
       "L 321.159844 55.228916 \n",
       "L 321.159844 54.137349 \n",
       "L 321.159844 53.045783 \n",
       "L 321.159844 51.954217 \n",
       "L 321.159844 50.862651 \n",
       "L 321.159844 49.771084 \n",
       "L 321.159844 48.679518 \n",
       "L 321.159844 47.587952 \n",
       "L 321.159844 46.496386 \n",
       "L 321.159844 45.404819 \n",
       "L 321.159844 44.313253 \n",
       "L 321.159844 43.221687 \n",
       "L 321.159844 42.13012 \n",
       "L 321.159844 41.038554 \n",
       "L 321.159844 39.946988 \n",
       "L 321.159844 38.855422 \n",
       "L 321.159844 37.763855 \n",
       "L 321.159844 36.672289 \n",
       "L 321.159844 35.580723 \n",
       "L 321.159844 34.489157 \n",
       "L 321.159844 33.39759 \n",
       "L 321.159844 32.306024 \n",
       "L 321.159844 31.214458 \n",
       "L 321.159844 30.122892 \n",
       "L 321.159844 29.031325 \n",
       "L 321.159844 27.939759 \n",
       "L 321.159844 26.848193 \n",
       "L 321.159844 25.756627 \n",
       "L 321.159844 24.66506 \n",
       "L 321.159844 23.573494 \n",
       "L 321.159844 22.481928 \n",
       "L 321.159844 21.390361 \n",
       "L 321.159844 20.298795 \n",
       "L 321.159844 19.207229 \n",
       "L 321.159844 18.115663 \n",
       "L 321.159844 17.024096 \n",
       "L 321.159844 15.93253 \n",
       "L 321.159844 14.840964 \n",
       "L 321.159844 13.749398 \n",
       "L 321.159844 12.657831 \n",
       "L 321.159844 11.566265 \n",
       "L 321.159844 10.474699 \n",
       "L 321.159844 9.383133 \n",
       "L 321.159844 8.291566 \n",
       "L 321.159844 7.2 \n",
       "L 320.039362 7.2 \n",
       "L 318.91888 7.2 \n",
       "L 317.798398 7.2 \n",
       "L 316.677916 7.2 \n",
       "L 315.557434 7.2 \n",
       "L 314.436952 7.2 \n",
       "L 313.31647 7.2 \n",
       "L 312.195988 7.2 \n",
       "L 311.075506 7.2 \n",
       "L 309.955024 7.2 \n",
       "L 308.834543 7.2 \n",
       "L 307.714061 7.2 \n",
       "L 306.593579 7.2 \n",
       "L 305.473097 7.2 \n",
       "L 304.352615 7.2 \n",
       "L 303.232133 7.2 \n",
       "L 302.111651 7.2 \n",
       "L 300.991169 7.2 \n",
       "L 299.870687 7.2 \n",
       "L 298.750205 7.2 \n",
       "L 297.629723 7.2 \n",
       "L 296.509241 7.2 \n",
       "L 295.388759 7.2 \n",
       "L 294.268277 7.2 \n",
       "L 293.147796 7.2 \n",
       "L 292.027314 7.2 \n",
       "L 290.906832 7.2 \n",
       "L 289.78635 7.2 \n",
       "L 288.665868 7.2 \n",
       "L 287.545386 7.2 \n",
       "L 286.424904 7.2 \n",
       "L 285.304422 7.2 \n",
       "L 284.18394 7.2 \n",
       "L 283.063458 7.2 \n",
       "L 281.942976 7.2 \n",
       "L 280.822494 7.2 \n",
       "L 279.702012 7.2 \n",
       "L 278.58153 7.2 \n",
       "L 277.461049 7.2 \n",
       "L 276.340567 7.2 \n",
       "L 275.220085 7.2 \n",
       "L 274.099603 7.2 \n",
       "L 272.979121 7.2 \n",
       "L 271.858639 7.2 \n",
       "L 270.738157 7.2 \n",
       "L 269.617675 7.2 \n",
       "L 268.497193 7.2 \n",
       "L 267.376711 7.2 \n",
       "L 266.256229 7.2 \n",
       "L 265.135747 7.2 \n",
       "L 264.015265 7.2 \n",
       "L 262.894784 7.2 \n",
       "L 261.774302 7.2 \n",
       "L 260.65382 7.2 \n",
       "L 259.533338 7.2 \n",
       "L 258.412856 7.2 \n",
       "L 257.292374 7.2 \n",
       "L 256.171892 7.2 \n",
       "L 255.05141 7.2 \n",
       "L 253.930928 7.2 \n",
       "L 252.810446 7.2 \n",
       "L 251.689964 7.2 \n",
       "L 250.569482 7.2 \n",
       "L 249.449 7.2 \n",
       "L 248.328518 7.2 \n",
       "L 247.208037 7.2 \n",
       "L 246.087555 7.2 \n",
       "L 244.967073 7.2 \n",
       "L 243.846591 7.2 \n",
       "L 242.726109 7.2 \n",
       "L 241.605627 7.2 \n",
       "L 240.485145 7.2 \n",
       "L 239.364663 7.2 \n",
       "L 238.244181 7.2 \n",
       "L 237.123699 7.2 \n",
       "L 236.003217 7.2 \n",
       "L 234.882735 7.2 \n",
       "L 233.762253 7.2 \n",
       "L 232.641771 7.2 \n",
       "L 231.52129 7.2 \n",
       "L 230.400808 7.2 \n",
       "L 229.280326 7.2 \n",
       "L 228.159844 7.2 \n",
       "L 227.039362 7.2 \n",
       "L 225.91888 7.2 \n",
       "L 224.798398 7.2 \n",
       "L 223.677916 7.2 \n",
       "L 222.557434 7.2 \n",
       "L 221.436952 7.2 \n",
       "L 220.31647 7.2 \n",
       "L 219.195988 7.2 \n",
       "L 218.075506 7.2 \n",
       "L 216.955024 7.2 \n",
       "L 215.834543 7.2 \n",
       "L 214.714061 7.2 \n",
       "L 213.593579 7.2 \n",
       "L 212.473097 7.2 \n",
       "L 211.352615 7.2 \n",
       "L 210.232133 7.2 \n",
       "L 209.111651 7.2 \n",
       "L 207.991169 7.2 \n",
       "L 206.870687 7.2 \n",
       "L 205.750205 7.2 \n",
       "L 204.629723 7.2 \n",
       "L 203.509241 7.2 \n",
       "L 202.388759 7.2 \n",
       "L 201.268277 7.2 \n",
       "L 200.147796 7.2 \n",
       "L 199.027314 7.2 \n",
       "L 197.906832 7.2 \n",
       "L 196.78635 7.2 \n",
       "L 195.665868 7.2 \n",
       "L 194.545386 7.2 \n",
       "L 193.424904 7.2 \n",
       "L 192.304422 7.2 \n",
       "L 191.18394 7.2 \n",
       "L 190.063458 7.2 \n",
       "L 188.942976 7.2 \n",
       "L 187.822494 7.2 \n",
       "L 186.702012 7.2 \n",
       "L 185.58153 7.2 \n",
       "L 184.461049 7.2 \n",
       "L 183.340567 7.2 \n",
       "L 182.220085 7.2 \n",
       "L 181.099603 7.2 \n",
       "L 179.979121 7.2 \n",
       "L 178.858639 7.2 \n",
       "L 177.738157 7.2 \n",
       "L 176.617675 7.2 \n",
       "L 175.497193 7.2 \n",
       "L 174.376711 7.2 \n",
       "L 173.256229 7.2 \n",
       "L 172.135747 7.2 \n",
       "L 171.015265 7.2 \n",
       "L 169.894784 7.2 \n",
       "L 168.774302 7.2 \n",
       "L 167.65382 7.2 \n",
       "L 166.533338 7.2 \n",
       "L 165.412856 7.2 \n",
       "L 164.292374 7.2 \n",
       "L 163.171892 7.2 \n",
       "L 162.05141 7.2 \n",
       "L 160.930928 7.2 \n",
       "L 159.810446 7.2 \n",
       "L 158.689964 7.2 \n",
       "L 157.569482 7.2 \n",
       "L 156.449 7.2 \n",
       "L 155.328518 7.2 \n",
       "L 154.208037 7.2 \n",
       "L 153.087555 7.2 \n",
       "L 151.967073 7.2 \n",
       "L 150.846591 7.2 \n",
       "L 149.726109 7.2 \n",
       "L 148.605627 7.2 \n",
       "L 147.485145 7.2 \n",
       "L 146.364663 7.2 \n",
       "L 145.244181 7.2 \n",
       "L 144.123699 7.2 \n",
       "L 143.003217 7.2 \n",
       "L 141.882735 7.2 \n",
       "L 140.762253 7.2 \n",
       "L 139.641771 7.2 \n",
       "L 138.52129 7.2 \n",
       "L 137.400808 7.2 \n",
       "L 136.280326 7.2 \n",
       "L 135.159844 7.2 \n",
       "L 134.039362 7.2 \n",
       "L 132.91888 7.2 \n",
       "L 131.798398 7.2 \n",
       "L 130.677916 7.2 \n",
       "L 129.557434 7.2 \n",
       "L 128.436952 7.2 \n",
       "L 127.31647 7.2 \n",
       "L 126.195988 7.2 \n",
       "L 125.075506 7.2 \n",
       "L 123.955024 7.2 \n",
       "L 122.834543 7.2 \n",
       "L 121.714061 7.2 \n",
       "L 120.593579 7.2 \n",
       "L 119.473097 7.2 \n",
       "L 118.352615 7.2 \n",
       "L 117.232133 7.2 \n",
       "L 116.111651 7.2 \n",
       "L 114.991169 7.2 \n",
       "L 113.870687 7.2 \n",
       "L 112.750205 7.2 \n",
       "L 111.629723 7.2 \n",
       "L 110.509241 7.2 \n",
       "L 109.388759 7.2 \n",
       "L 108.268277 7.2 \n",
       "L 107.147796 7.2 \n",
       "L 106.027314 7.2 \n",
       "L 104.906832 7.2 \n",
       "L 103.78635 7.2 \n",
       "L 102.665868 7.2 \n",
       "L 101.545386 7.2 \n",
       "L 100.424904 7.2 \n",
       "L 99.304422 7.2 \n",
       "L 98.18394 7.2 \n",
       "L 97.063458 7.2 \n",
       "L 95.942976 7.2 \n",
       "L 94.822494 7.2 \n",
       "L 93.702012 7.2 \n",
       "L 92.58153 7.2 \n",
       "L 91.461049 7.2 \n",
       "L 90.340567 7.2 \n",
       "L 89.220085 7.2 \n",
       "L 88.099603 7.2 \n",
       "L 86.979121 7.2 \n",
       "L 85.858639 7.2 \n",
       "L 84.738157 7.2 \n",
       "L 83.617675 7.2 \n",
       "L 82.497193 7.2 \n",
       "L 81.376711 7.2 \n",
       "L 80.256229 7.2 \n",
       "L 79.135747 7.2 \n",
       "L 78.015265 7.2 \n",
       "L 76.894784 7.2 \n",
       "L 75.774302 7.2 \n",
       "L 74.65382 7.2 \n",
       "L 73.533338 7.2 \n",
       "L 72.412856 7.2 \n",
       "L 71.292374 7.2 \n",
       "L 70.171892 7.2 \n",
       "L 69.05141 7.2 \n",
       "L 67.930928 7.2 \n",
       "L 66.810446 7.2 \n",
       "L 65.689964 7.2 \n",
       "L 64.569482 7.2 \n",
       "L 63.449 7.2 \n",
       "L 62.328518 7.2 \n",
       "L 61.208037 7.2 \n",
       "L 60.087555 7.2 \n",
       "L 58.967073 7.2 \n",
       "L 57.846591 7.2 \n",
       "L 56.726109 7.2 \n",
       "L 55.605627 7.2 \n",
       "L 54.485145 7.2 \n",
       "L 53.364663 7.2 \n",
       "L 52.244181 7.2 \n",
       "L 51.123699 7.2 \n",
       "L 50.003217 7.2 \n",
       "L 48.882735 7.2 \n",
       "L 47.762253 7.2 \n",
       "L 46.641771 7.2 \n",
       "L 45.52129 7.2 \n",
       "L 44.400808 7.2 \n",
       "L 43.280326 7.2 \n",
       "L 42.159844 7.2 \n",
       "L 42.159844 8.291566 \n",
       "L 42.159844 9.383133 \n",
       "L 42.159844 10.474699 \n",
       "L 42.159844 11.566265 \n",
       "L 42.159844 12.657831 \n",
       "L 42.159844 13.749398 \n",
       "L 42.159844 14.840964 \n",
       "L 42.159844 15.93253 \n",
       "L 42.159844 17.024096 \n",
       "L 42.159844 18.115663 \n",
       "L 42.159844 19.207229 \n",
       "L 42.159844 20.298795 \n",
       "L 42.159844 21.390361 \n",
       "L 42.159844 22.481928 \n",
       "L 42.159844 23.573494 \n",
       "L 42.159844 24.66506 \n",
       "L 42.159844 25.756627 \n",
       "L 42.159844 26.848193 \n",
       "L 42.159844 27.939759 \n",
       "L 42.159844 29.031325 \n",
       "L 42.159844 30.122892 \n",
       "L 42.159844 31.214458 \n",
       "L 42.159844 32.306024 \n",
       "L 42.159844 33.39759 \n",
       "L 42.159844 34.489157 \n",
       "L 42.159844 35.580723 \n",
       "L 42.159844 36.672289 \n",
       "L 42.159844 37.763855 \n",
       "L 42.159844 38.855422 \n",
       "L 42.159844 39.946988 \n",
       "L 42.159844 41.038554 \n",
       "L 42.159844 42.13012 \n",
       "L 42.159844 43.221687 \n",
       "L 42.159844 44.313253 \n",
       "L 42.159844 45.404819 \n",
       "L 42.159844 46.496386 \n",
       "L 42.159844 47.587952 \n",
       "L 42.159844 48.679518 \n",
       "L 42.159844 49.771084 \n",
       "L 42.159844 50.862651 \n",
       "L 42.159844 51.954217 \n",
       "L 42.159844 53.045783 \n",
       "L 42.159844 54.137349 \n",
       "L 42.159844 55.228916 \n",
       "L 42.159844 56.320482 \n",
       "L 42.159844 57.412048 \n",
       "L 42.159844 58.503614 \n",
       "L 42.159844 59.595181 \n",
       "L 42.159844 60.686747 \n",
       "L 42.159844 61.778313 \n",
       "L 42.159844 62.86988 \n",
       "L 42.159844 63.961446 \n",
       "L 42.159844 65.053012 \n",
       "L 42.159844 66.144578 \n",
       "L 42.159844 67.236145 \n",
       "L 42.159844 68.327711 \n",
       "L 42.159844 69.419277 \n",
       "L 42.159844 70.510843 \n",
       "L 42.159844 71.60241 \n",
       "L 42.159844 72.693976 \n",
       "L 42.159844 73.785542 \n",
       "L 42.159844 74.877108 \n",
       "L 42.159844 75.968675 \n",
       "L 42.159844 77.060241 \n",
       "L 42.159844 78.151807 \n",
       "L 42.159844 79.243373 \n",
       "L 42.159844 80.33494 \n",
       "L 42.159844 81.426506 \n",
       "L 42.159844 82.518072 \n",
       "L 42.159844 83.609639 \n",
       "L 42.159844 84.701205 \n",
       "L 42.159844 85.792771 \n",
       "L 42.159844 86.884337 \n",
       "L 42.159844 87.975904 \n",
       "L 42.159844 89.06747 \n",
       "L 42.159844 90.159036 \n",
       "L 42.159844 91.250602 \n",
       "L 42.159844 92.342169 \n",
       "L 42.159844 93.433735 \n",
       "L 42.159844 94.525301 \n",
       "L 42.159844 95.616867 \n",
       "L 42.159844 96.708434 \n",
       "L 42.159844 97.8 \n",
       "L 42.159844 98.891566 \n",
       "L 42.159844 99.983133 \n",
       "L 42.159844 101.074699 \n",
       "L 42.159844 102.166265 \n",
       "L 42.159844 103.257831 \n",
       "L 42.159844 104.349398 \n",
       "L 42.159844 105.440964 \n",
       "L 42.159844 105.713855 \n",
       "z\n",
       "\" style=\"fill:#17becf;fill-opacity:0.15;\"/>\n",
       "   </g>\n",
       "   <g id=\"PathCollection_9\">\n",
       "    <defs>\n",
       "     <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"C0_0_fc33e90c84\"/>\n",
       "    </defs>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_fc33e90c84\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_fc33e90c84\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"284.555324\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"271.419731\" xlink:href=\"#C0_0_fc33e90c84\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_fc33e90c84\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_fc33e90c84\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_fc33e90c84\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_fc33e90c84\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_fc33e90c84\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_fc33e90c84\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_fc33e90c84\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_fc33e90c84\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"91.899957\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"52.493177\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"91.899957\" xlink:href=\"#C0_0_fc33e90c84\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"196.984703\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"153.199392\" xlink:href=\"#C0_0_fc33e90c84\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_fc33e90c84\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_fc33e90c84\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_fc33e90c84\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_fc33e90c84\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_fc33e90c84\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_fc33e90c84\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_fc33e90c84\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_fc33e90c84\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_fc33e90c84\" y=\"216.502778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"161.956454\" xlink:href=\"#C0_0_fc33e90c84\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"249.527075\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_fc33e90c84\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_fc33e90c84\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_fc33e90c84\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_fc33e90c84\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"175.092047\" xlink:href=\"#C0_0_fc33e90c84\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_fc33e90c84\" y=\"101.155556\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_fc33e90c84\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"214.498827\" xlink:href=\"#C0_0_fc33e90c84\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"236.391482\" xlink:href=\"#C0_0_fc33e90c84\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_fc33e90c84\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_fc33e90c84\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"166.334985\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_fc33e90c84\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"271.419731\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"56.871708\" xlink:href=\"#C0_0_fc33e90c84\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"240.770013\" xlink:href=\"#C0_0_fc33e90c84\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_fc33e90c84\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"140.063799\" xlink:href=\"#C0_0_fc33e90c84\" y=\"164.072222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_fc33e90c84\" y=\"17.266667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"192.606171\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"267.0412\" xlink:href=\"#C0_0_fc33e90c84\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_fc33e90c84\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_fc33e90c84\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_fc33e90c84\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_fc33e90c84\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"258.284138\" xlink:href=\"#C0_0_fc33e90c84\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_fc33e90c84\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"245.148544\" xlink:href=\"#C0_0_fc33e90c84\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_fc33e90c84\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"245.148544\" xlink:href=\"#C0_0_fc33e90c84\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"262.662669\" xlink:href=\"#C0_0_fc33e90c84\" y=\"111.641667\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_fc33e90c84\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"183.849109\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"267.0412\" xlink:href=\"#C0_0_fc33e90c84\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"297.690917\" xlink:href=\"#C0_0_fc33e90c84\" y=\"59.211111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"205.741765\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_fc33e90c84\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.250239\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"83.142895\" xlink:href=\"#C0_0_fc33e90c84\" y=\"226.988889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"196.984703\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"310.82651\" xlink:href=\"#C0_0_fc33e90c84\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"153.199392\" xlink:href=\"#C0_0_fc33e90c84\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"288.933855\" xlink:href=\"#C0_0_fc33e90c84\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_fc33e90c84\" y=\"132.613889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.250239\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"302.069448\" xlink:href=\"#C0_0_fc33e90c84\" y=\"69.697222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"247.961111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"275.798262\" xlink:href=\"#C0_0_fc33e90c84\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_fc33e90c84\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"179.470578\" xlink:href=\"#C0_0_fc33e90c84\" y=\"153.586111\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"161.956454\" xlink:href=\"#C0_0_fc33e90c84\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"240.770013\" xlink:href=\"#C0_0_fc33e90c84\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"236.391482\" xlink:href=\"#C0_0_fc33e90c84\" y=\"38.238889\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"223.255889\" xlink:href=\"#C0_0_fc33e90c84\" y=\"122.127778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"227.63442\" xlink:href=\"#C0_0_fc33e90c84\" y=\"101.155556\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"78.764364\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"170.713516\" xlink:href=\"#C0_0_fc33e90c84\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_fc33e90c84\" y=\"27.752778\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"232.012951\" xlink:href=\"#C0_0_fc33e90c84\" y=\"80.183333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"210.120296\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"218.877358\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"201.363234\" xlink:href=\"#C0_0_fc33e90c84\" y=\"143.1\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"65.62877\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"237.475\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"74.385832\" xlink:href=\"#C0_0_fc33e90c84\" y=\"268.933333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"253.905606\" xlink:href=\"#C0_0_fc33e90c84\" y=\"90.669444\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"188.22764\" xlink:href=\"#C0_0_fc33e90c84\" y=\"174.558333\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"302.069448\" xlink:href=\"#C0_0_fc33e90c84\" y=\"48.725\"/>\n",
       "    </g>\n",
       "    <g clip-path=\"url(#p6e54cf6fdd)\">\n",
       "     <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#000000;stroke-opacity:0.5;stroke-width:0.5;\" x=\"70.007301\" xlink:href=\"#C0_0_fc33e90c84\" y=\"258.447222\"/>\n",
       "    </g>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_1\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 196.040964 \n",
       "L 321.159844 196.040964 \n",
       "L 321.159844 196.040964 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_2\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 195.768072 \n",
       "L 224.798398 195.768072 \n",
       "L 225.91888 195.904518 \n",
       "L 321.159844 195.904518 \n",
       "L 321.159844 195.904518 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_3\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 195.495181 \n",
       "L 224.798398 195.495181 \n",
       "L 225.91888 195.768072 \n",
       "L 321.159844 195.768072 \n",
       "L 321.159844 195.768072 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_4\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 195.222289 \n",
       "L 224.798398 195.222289 \n",
       "L 225.91888 195.631627 \n",
       "L 321.159844 195.631627 \n",
       "L 321.159844 195.631627 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_5\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 321.159844 116.356627 \n",
       "L 225.91888 116.356627 \n",
       "L 224.798398 117.448193 \n",
       "L 224.798398 194.949398 \n",
       "L 225.91888 195.495181 \n",
       "L 321.159844 195.495181 \n",
       "L 321.159844 195.495181 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 106.53253 \n",
       "L 224.798398 106.53253 \n",
       "L 225.91888 105.440964 \n",
       "L 225.91888 96.708434 \n",
       "L 321.159844 96.708434 \n",
       "L 321.159844 96.708434 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_6\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 321.159844 116.629518 \n",
       "L 225.91888 116.629518 \n",
       "L 225.078518 117.448193 \n",
       "L 225.078518 194.949398 \n",
       "L 225.91888 195.358735 \n",
       "L 321.159844 195.358735 \n",
       "L 321.159844 195.358735 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 106.259639 \n",
       "L 224.798398 106.259639 \n",
       "L 225.638759 105.440964 \n",
       "L 225.638759 96.708434 \n",
       "L 225.91888 96.435542 \n",
       "L 321.159844 96.435542 \n",
       "L 321.159844 96.435542 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_7\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 321.159844 116.90241 \n",
       "L 225.91888 116.90241 \n",
       "L 225.358639 117.448193 \n",
       "L 225.358639 194.949398 \n",
       "L 225.91888 195.222289 \n",
       "L 321.159844 195.222289 \n",
       "L 321.159844 195.222289 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 105.986747 \n",
       "L 224.798398 105.986747 \n",
       "L 225.358639 105.440964 \n",
       "L 225.358639 96.708434 \n",
       "L 225.91888 96.162651 \n",
       "L 321.159844 96.162651 \n",
       "L 321.159844 96.162651 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_8\">\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 321.159844 117.175301 \n",
       "L 225.91888 117.175301 \n",
       "L 225.638759 117.448193 \n",
       "L 225.638759 194.949398 \n",
       "L 225.91888 195.085843 \n",
       "L 321.159844 195.085843 \n",
       "L 321.159844 195.085843 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "    <path clip-path=\"url(#p6e54cf6fdd)\" d=\"M 42.159844 105.713855 \n",
       "L 224.798398 105.713855 \n",
       "L 225.078518 105.440964 \n",
       "L 225.078518 96.708434 \n",
       "L 225.91888 95.889759 \n",
       "L 321.159844 95.889759 \n",
       "L 321.159844 95.889759 \n",
       "\" style=\"fill:none;stroke:#000000;stroke-width:0.2;\"/>\n",
       "   </g>\n",
       "   <g id=\"LineCollection_9\"/>\n",
       "   <g id=\"patch_3\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 42.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_4\">\n",
       "    <path d=\"M 321.159844 279 \n",
       "L 321.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_5\">\n",
       "    <path d=\"M 42.159844 279 \n",
       "L 321.159844 279 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"patch_6\">\n",
       "    <path d=\"M 42.159844 7.2 \n",
       "L 321.159844 7.2 \n",
       "\" style=\"fill:none;\"/>\n",
       "   </g>\n",
       "   <g id=\"legend_1\">\n",
       "    <g id=\"line2d_25\"/>\n",
       "    <g id=\"line2d_26\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"mbc630d7dc2\" style=\"stroke:#1f77b4;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#1f77b4;fill-opacity:0.5;stroke:#1f77b4;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#mbc630d7dc2\" y=\"19.857812\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_15\">\n",
       "     <!-- setosa -->\n",
       "     <defs>\n",
       "      <path d=\"M 3.078125 15.484375 \n",
       "L 11.765625 16.84375 \n",
       "Q 12.5 11.625 15.84375 8.84375 \n",
       "Q 19.1875 6.0625 25.203125 6.0625 \n",
       "Q 31.25 6.0625 34.171875 8.515625 \n",
       "Q 37.109375 10.984375 37.109375 14.3125 \n",
       "Q 37.109375 17.28125 34.515625 19 \n",
       "Q 32.71875 20.171875 25.53125 21.96875 \n",
       "Q 15.875 24.421875 12.140625 26.203125 \n",
       "Q 8.40625 27.984375 6.46875 31.125 \n",
       "Q 4.546875 34.28125 4.546875 38.09375 \n",
       "Q 4.546875 41.546875 6.125 44.5 \n",
       "Q 7.71875 47.46875 10.453125 49.421875 \n",
       "Q 12.5 50.921875 16.03125 51.96875 \n",
       "Q 19.578125 53.03125 23.640625 53.03125 \n",
       "Q 29.734375 53.03125 34.34375 51.265625 \n",
       "Q 38.96875 49.515625 41.15625 46.5 \n",
       "Q 43.359375 43.5 44.1875 38.484375 \n",
       "L 35.59375 37.3125 \n",
       "Q 35.015625 41.3125 32.203125 43.546875 \n",
       "Q 29.390625 45.796875 24.265625 45.796875 \n",
       "Q 18.21875 45.796875 15.625 43.796875 \n",
       "Q 13.03125 41.796875 13.03125 39.109375 \n",
       "Q 13.03125 37.40625 14.109375 36.03125 \n",
       "Q 15.1875 34.625 17.484375 33.6875 \n",
       "Q 18.796875 33.203125 25.25 31.453125 \n",
       "Q 34.578125 28.953125 38.25 27.359375 \n",
       "Q 41.9375 25.78125 44.03125 22.75 \n",
       "Q 46.140625 19.734375 46.140625 15.234375 \n",
       "Q 46.140625 10.84375 43.578125 6.953125 \n",
       "Q 41.015625 3.078125 36.171875 0.953125 \n",
       "Q 31.34375 -1.171875 25.25 -1.171875 \n",
       "Q 15.140625 -1.171875 9.84375 3.03125 \n",
       "Q 4.546875 7.234375 3.078125 15.484375 \n",
       "z\n",
       "\" id=\"ArialMT-115\"/>\n",
       "      <path d=\"M 3.328125 25.921875 \n",
       "Q 3.328125 40.328125 11.328125 47.265625 \n",
       "Q 18.015625 53.03125 27.640625 53.03125 \n",
       "Q 38.328125 53.03125 45.109375 46.015625 \n",
       "Q 51.90625 39.015625 51.90625 26.65625 \n",
       "Q 51.90625 16.65625 48.90625 10.90625 \n",
       "Q 45.90625 5.171875 40.15625 2 \n",
       "Q 34.421875 -1.171875 27.640625 -1.171875 \n",
       "Q 16.75 -1.171875 10.03125 5.8125 \n",
       "Q 3.328125 12.796875 3.328125 25.921875 \n",
       "z\n",
       "M 12.359375 25.921875 \n",
       "Q 12.359375 15.96875 16.703125 11.015625 \n",
       "Q 21.046875 6.0625 27.640625 6.0625 \n",
       "Q 34.1875 6.0625 38.53125 11.03125 \n",
       "Q 42.875 16.015625 42.875 26.21875 \n",
       "Q 42.875 35.84375 38.5 40.796875 \n",
       "Q 34.125 45.75 27.640625 45.75 \n",
       "Q 21.046875 45.75 16.703125 40.8125 \n",
       "Q 12.359375 35.890625 12.359375 25.921875 \n",
       "z\n",
       "\" id=\"ArialMT-111\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 23.357812)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"105.615234\" xlink:href=\"#ArialMT-116\"/>\n",
       "      <use x=\"133.398438\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"189.013672\" xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"239.013672\" xlink:href=\"#ArialMT-97\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"line2d_27\"/>\n",
       "    <g id=\"line2d_28\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"m272672e2a3\" style=\"stroke:#8c564b;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#8c564b;fill-opacity:0.5;stroke:#8c564b;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#m272672e2a3\" y=\"34.003125\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_16\">\n",
       "     <!-- versicolor -->\n",
       "     <defs>\n",
       "      <path d=\"M 21 0 \n",
       "L 1.265625 51.859375 \n",
       "L 10.546875 51.859375 \n",
       "L 21.6875 20.796875 \n",
       "Q 23.484375 15.765625 25 10.359375 \n",
       "Q 26.171875 14.453125 28.265625 20.21875 \n",
       "L 39.796875 51.859375 \n",
       "L 48.828125 51.859375 \n",
       "L 29.203125 0 \n",
       "z\n",
       "\" id=\"ArialMT-118\"/>\n",
       "      <path d=\"M 6.5 0 \n",
       "L 6.5 51.859375 \n",
       "L 14.40625 51.859375 \n",
       "L 14.40625 44 \n",
       "Q 17.4375 49.515625 20 51.265625 \n",
       "Q 22.5625 53.03125 25.640625 53.03125 \n",
       "Q 30.078125 53.03125 34.671875 50.203125 \n",
       "L 31.640625 42.046875 \n",
       "Q 28.421875 43.953125 25.203125 43.953125 \n",
       "Q 22.3125 43.953125 20.015625 42.21875 \n",
       "Q 17.71875 40.484375 16.75 37.40625 \n",
       "Q 15.28125 32.71875 15.28125 27.15625 \n",
       "L 15.28125 0 \n",
       "z\n",
       "\" id=\"ArialMT-114\"/>\n",
       "      <path d=\"M 40.4375 19 \n",
       "L 49.078125 17.875 \n",
       "Q 47.65625 8.9375 41.8125 3.875 \n",
       "Q 35.984375 -1.171875 27.484375 -1.171875 \n",
       "Q 16.84375 -1.171875 10.375 5.78125 \n",
       "Q 3.90625 12.75 3.90625 25.734375 \n",
       "Q 3.90625 34.125 6.6875 40.421875 \n",
       "Q 9.46875 46.734375 15.15625 49.875 \n",
       "Q 20.84375 53.03125 27.546875 53.03125 \n",
       "Q 35.984375 53.03125 41.359375 48.75 \n",
       "Q 46.734375 44.484375 48.25 36.625 \n",
       "L 39.703125 35.296875 \n",
       "Q 38.484375 40.53125 35.375 43.15625 \n",
       "Q 32.28125 45.796875 27.875 45.796875 \n",
       "Q 21.234375 45.796875 17.078125 41.03125 \n",
       "Q 12.9375 36.28125 12.9375 25.984375 \n",
       "Q 12.9375 15.53125 16.9375 10.796875 \n",
       "Q 20.953125 6.0625 27.390625 6.0625 \n",
       "Q 32.5625 6.0625 36.03125 9.234375 \n",
       "Q 39.5 12.40625 40.4375 19 \n",
       "z\n",
       "\" id=\"ArialMT-99\"/>\n",
       "     </defs>\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 37.503125)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-118\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-101\"/>\n",
       "      <use x=\"105.615234\" xlink:href=\"#ArialMT-114\"/>\n",
       "      <use x=\"138.916016\" xlink:href=\"#ArialMT-115\"/>\n",
       "      <use x=\"188.916016\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"211.132812\" xlink:href=\"#ArialMT-99\"/>\n",
       "      <use x=\"261.132812\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"316.748047\" xlink:href=\"#ArialMT-108\"/>\n",
       "      <use x=\"338.964844\" xlink:href=\"#ArialMT-111\"/>\n",
       "      <use x=\"394.580078\" xlink:href=\"#ArialMT-114\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"line2d_29\"/>\n",
       "    <g id=\"line2d_30\">\n",
       "     <defs>\n",
       "      <path d=\"M 0 3.5 \n",
       "C 0.928211 3.5 1.81853 3.131218 2.474874 2.474874 \n",
       "C 3.131218 1.81853 3.5 0.928211 3.5 0 \n",
       "C 3.5 -0.928211 3.131218 -1.81853 2.474874 -2.474874 \n",
       "C 1.81853 -3.131218 0.928211 -3.5 0 -3.5 \n",
       "C -0.928211 -3.5 -1.81853 -3.131218 -2.474874 -2.474874 \n",
       "C -3.131218 -1.81853 -3.5 -0.928211 -3.5 0 \n",
       "C -3.5 0.928211 -3.131218 1.81853 -2.474874 2.474874 \n",
       "C -1.81853 3.131218 -0.928211 3.5 0 3.5 \n",
       "z\n",
       "\" id=\"m015216fe77\" style=\"stroke:#17becf;stroke-opacity:0.5;stroke-width:0.5;\"/>\n",
       "     </defs>\n",
       "     <g>\n",
       "      <use style=\"fill:#17becf;fill-opacity:0.5;stroke:#17becf;stroke-opacity:0.5;stroke-width:0.5;\" x=\"61.159844\" xlink:href=\"#m015216fe77\" y=\"48.148437\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "    <g id=\"text_17\">\n",
       "     <!-- virginica -->\n",
       "     <g style=\"fill:#262626;\" transform=\"translate(79.159844 51.648437)scale(0.1 -0.1)\">\n",
       "      <use xlink:href=\"#ArialMT-118\"/>\n",
       "      <use x=\"50\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"72.216797\" xlink:href=\"#ArialMT-114\"/>\n",
       "      <use x=\"105.517578\" xlink:href=\"#ArialMT-103\"/>\n",
       "      <use x=\"161.132812\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"183.349609\" xlink:href=\"#ArialMT-110\"/>\n",
       "      <use x=\"238.964844\" xlink:href=\"#ArialMT-105\"/>\n",
       "      <use x=\"261.181641\" xlink:href=\"#ArialMT-99\"/>\n",
       "      <use x=\"311.181641\" xlink:href=\"#ArialMT-97\"/>\n",
       "     </g>\n",
       "    </g>\n",
       "   </g>\n",
       "  </g>\n",
       " </g>\n",
       " <defs>\n",
       "  <clipPath id=\"p6e54cf6fdd\">\n",
       "   <rect height=\"271.8\" width=\"279\" x=\"42.159844\" y=\"7.2\"/>\n",
       "  </clipPath>\n",
       " </defs>\n",
       "</svg>\n"
      ],
      "text/plain": [
       "<Figure size 360x360 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "plot_boundary(cls, X2, Y)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.6"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}
